## Specify MeanShift, Disc loss and U-Net

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
print(tf.__version__)
import tensorflow.keras as keras
%reload_ext tensorboard
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
clrs2 = [[0, 0, 0], [121, 186, 179], [173, 206, 184], [124, 183, 103], [58, 231, 42], [163, 200, 250], [50, 91, 30], [148, 149, 79], [19, 124, 78], [76, 70, 104], [46, 93, 122], [16, 198, 197], [210, 27, 187], [117, 113, 168], [200, 173, 237], [198, 145, 234], [243, 206, 201], [32, 74, 205], [33, 82, 156], [235, 19, 28], [228, 179, 135], [157, 39, 30], [122, 158, 156], [10, 57, 210], [36, 82, 49], [97, 105, 2], [154, 25, 251], [155, 53, 107], [17, 17, 170], [219, 87, 227], [247, 71, 221], [154, 194, 102], [216, 37, 83], [121, 66, 139], [9, 139, 190], [47, 99, 151], [238, 75, 214], [243, 160, 109], [94, 162, 5], [134, 0, 255], [137, 66, 227], [126, 238, 3], [185, 208, 44], [177, 95, 82], [18, 98, 153], [18, 192, 246], [79, 40, 179], [249, 84, 249], [21, 191, 252], [130, 214, 26], [133, 133, 29], [184, 253, 175], [237, 212, 101], [39, 37, 98], [150, 127, 237], [246, 235, 213], [38, 52, 213], [179, 113, 116], [79, 22, 30], [98, 239, 167], [135, 186, 225], [60, 196, 210], [34, 32, 3], [42, 128, 248], [202, 20, 87], [119, 15, 79], [230, 106, 30], [105, 136, 99], [178, 153, 66], [190, 83, 99], [116, 238, 155], [114, 110, 134], [252, 238, 8], [20, 86, 71], [159, 105, 35], [91, 210, 117], [63, 221, 46], [192, 172, 150], [235, 30, 2], [249, 254, 10], [24, 239, 127], [143, 126, 58], [161, 132, 207], [104, 43, 183], [167, 47, 219], [82, 242, 18], [154, 143, 45], [71, 73, 89], [173, 217, 251], [232, 170, 253], [101, 179, 169], [196, 180, 234], [199, 217, 102], [123, 136, 194], [170, 239, 58], [102, 52, 219], [74, 218, 201], [67, 41, 244], [69, 224, 20], [17, 9, 89], [150, 2, 226], [250, 217, 10], [176, 129, 247], [178, 95, 193], [172, 37, 115], [81, 71, 0], [42, 76, 140], [250, 112, 34], [192, 246, 139], [193, 127, 232], [218, 133, 30], [150, 115, 25], [220, 181, 154], [8, 219, 65], [224, 129, 28], [134, 68, 73], [221, 118, 68], [45, 69, 176], [40, 168, 139], [178, 132, 5], [232, 157, 48], [28, 198, 218], [103, 93, 15], [60, 76, 16], [213, 255, 60], [38, 146, 58], [12, 18, 148], [0, 116, 81], [185, 205, 51], [188, 32, 127], [150, 203, 214], [254, 180, 237], [97, 114, 120], [77, 19, 101], [47, 106, 161], [62, 41, 86], [54, 96, 144], [37, 79, 28], [139, 209, 172], [106, 47, 128], [101, 123, 64], [115, 146, 95], [45, 25, 223], [0, 127, 30], [184, 235, 118], [80, 48, 71], [255, 106, 233], [14, 18, 87], [118, 131, 71], [96, 109, 101], [164, 60, 82], [192, 41, 18], [255, 20, 213], [204, 67, 221], [104, 59, 150], [218, 45, 117], [12, 85, 188], [244, 11, 15], [28, 185, 183], [185, 131, 39], [127, 18, 145], [125, 195, 241], [70, 181, 186], [211, 246, 55], [161, 4, 68], [77, 226, 213], [90, 122, 219], [223, 146, 225], [171, 177, 59], [213, 201, 229], [126, 26, 204], [93, 151, 83], [247, 58, 48], [192, 129, 132], [103, 97, 126], [115, 20, 41], [191, 234, 171], [23, 63, 41], [98, 151, 251], [137, 160, 162], [6, 236, 115], [156, 160, 181], [3, 84, 106], [117, 232, 21], [93, 223, 58], [55, 143, 55], [0, 79, 42], [134, 78, 143], [194, 232, 100], [51, 234, 175], [79, 26, 108], [240, 205, 230], [233, 56, 201], [134, 15, 152], [143, 181, 174], [48, 204, 118], [81, 62, 237], [187, 127, 28], [170, 199, 85], [21, 217, 20], [247, 4, 63], [214, 120, 138], [240, 58, 128], [42, 174, 83], [161, 167, 151], [141, 241, 181], [205, 148, 248], [139, 23, 215], [210, 213, 84], [88, 220, 93], [101, 14, 161], [130, 231, 30], [35, 210, 45], [193, 130, 210], [74, 183, 86], [123, 149, 172], [70, 176, 221], [108, 116, 73], [245, 242, 172], [113, 165, 253], [158, 179, 29], [190, 74, 58], [111, 215, 126], [64, 125, 226], [161, 18, 72], [194, 174, 136], [211, 66, 100], [227, 126, 95], [111, 136, 121], [56, 184, 153], [187, 24, 50], [158, 64, 62], [96, 150, 187], [221, 134, 159], [91, 108, 104], [77, 181, 198], [123, 56, 10], [241, 73, 144], [219, 108, 99], [99, 47, 167], [153, 3, 195], [71, 9, 131], [28, 13, 173], [222, 114, 159], [183, 74, 62], [184, 217, 120], [179, 69, 108], [173, 4, 107], [246, 203, 42], [153, 191, 246], [220, 102, 102], [246, 140, 54], [223, 49, 0], [61, 10, 107], [221, 226, 255], [147, 106, 187], [180, 172, 137], [86, 16, 169], [61, 137, 127], [169, 138, 98], [70, 239, 213], [162, 156, 151], [11, 245, 98], [80, 175, 56], [151, 201, 220], [42, 203, 39], [246, 34, 161], [209, 43, 47], [185, 232, 214], [8, 161, 204], [47, 85, 141], [110, 110, 181], [90, 114, 215], [245, 165, 206], [250, 139, 245], [83, 92, 139], [1, 146, 248], [233, 234, 171], [114, 98, 114], [28, 50, 164], [197, 33, 245], [57, 107, 244], [19, 217, 213], [207, 112, 62], [151, 13, 168], [155, 194, 37], [23, 238, 7], [148, 121, 251], [240, 22, 89], [34, 66, 151], [137, 219, 252], [92, 107, 99], [68, 54, 129], [226, 225, 41], [102, 162, 238], [55, 242, 41], [220, 29, 89], [79, 124, 113], [33, 121, 0], [54, 87, 91], [7, 29, 147], [242, 217, 59], [163, 133, 234], [238, 6, 5], [226, 72, 138], [251, 225, 170], [216, 90, 131], [82, 111, 231], [220, 103, 101], [74, 236, 18], [228, 254, 87], [155, 200, 200], [177, 105, 143], [131, 74, 88], [128, 23, 210], [87, 102, 127], [210, 191, 186], [236, 236, 170], [182, 227, 231], [225, 255, 226], [254, 17, 177], [2, 12, 73], [159, 128, 4], [149, 169, 101], [155, 199, 101], [16, 26, 179], [50, 182, 187], [59, 122, 143], [40, 172, 181], [112, 87, 239], [33, 89, 160], [104, 221, 115], [167, 45, 13], [36, 122, 131], [83, 77, 44], [253, 253, 89], [18, 151, 127], [13, 123, 227], [221, 191, 154], [135, 120, 171], [16, 166, 226], [216, 199, 121], [161, 52, 51], [138, 242, 238], [71, 240, 60], [42, 78, 109], [139, 189, 53], [88, 64, 119], [89, 48, 231], [100, 230, 176], [206, 60, 17], [20, 34, 127], [86, 144, 157], [110, 33, 63], [3, 189, 228], [236, 239, 74], [193, 11, 168], [55, 111, 66], [250, 44, 55], [105, 133, 80], [78, 238, 78], [16, 168, 218], [242, 129, 228], [90, 161, 165], [245, 97, 251], [177, 63, 90], [79, 225, 218], [173, 174, 175], [86, 23, 250], [191, 97, 17], [234, 175, 141], [218, 147, 237], [15, 101, 92], [181, 71, 4], [124, 153, 111], [254, 183, 105], [182, 74, 71], [97, 42, 173], [177, 35, 95], [147, 215, 89], [131, 180, 111], [9, 177, 164], [159, 43, 112], [96, 40, 93], [65, 190, 119], [54, 188, 254], [84, 93, 223], [59, 50, 56], [130, 39, 115], [40, 215, 185], [194, 28, 206], [18, 224, 97], [184, 244, 41], [26, 69, 207], [115, 252, 61], [111, 177, 225], [96, 39, 25], [140, 210, 15], [47, 9, 193], [129, 187, 232], [89, 40, 218], [52, 165, 80], [142, 246, 232], [0, 219, 30], [122, 67, 60], [164, 249, 229], [222, 107, 252], [210, 46, 173], [189, 32, 252], [56, 185, 44], [141, 87, 213], [114, 185, 115], [109, 67, 103], [138, 32, 147], [82, 132, 172], [65, 204, 228], [21, 124, 163], [169, 248, 12], [253, 132, 157], [223, 207, 92], [122, 103, 114], [33, 103, 85], [238, 34, 75], [12, 7, 114], [78, 114, 183], [106, 48, 189], [25, 177, 144], [68, 104, 41], [114, 204, 201], [28, 31, 33], [63, 247, 111], [253, 18, 17], [61, 230, 13], [153, 233, 73], [191, 206, 216], [133, 129, 116], [53, 247, 205], [250, 139, 21], [170, 167, 110], [143, 62, 223], [237, 26, 254], [205, 110, 62], [230, 140, 205], [75, 101, 47], [149, 89, 184], [249, 10, 146], [221, 223, 220], [69, 48, 230], [125, 20, 69], [77, 226, 4], [240, 111, 5], [0, 16, 215], [244, 39, 232], [212, 103, 197], [87, 37, 251], [100, 165, 203], [160, 7, 14], [240, 6, 183], [160, 120, 221], [242, 155, 141], [100, 107, 126], [166, 180, 145], [15, 23, 121], [240, 207, 47], [1, 188, 235], [54, 54, 29], [217, 87, 182], [130, 209, 5], [83, 51, 234], [179, 22, 197], [99, 139, 254], [29, 168, 179], [182, 212, 28], [218, 192, 237], [182, 101, 50], [170, 132, 125], [74, 240, 87], [90, 86, 174], [18, 43, 63], [189, 51, 244], [229, 215, 223], [149, 40, 194], [185, 72, 105], [45, 124, 10], [19, 170, 20], [15, 41, 201], [87, 144, 7], [106, 205, 108], [40, 118, 86], [114, 52, 215], [102, 186, 182], [115, 87, 29], [117, 111, 48], [77, 70, 214], [83, 182, 194], [73, 96, 27], [174, 231, 36], [95, 63, 201], [62, 39, 106], [177, 188, 140], [17, 19, 208], [245, 16, 86], [171, 52, 37], [97, 128, 242], [113, 185, 93], [83, 205, 252], [159, 23, 108], [9, 48, 129], [59, 115, 99], [241, 150, 69], [125, 75, 241], [233, 66, 165], [139, 241, 73], [228, 0, 121], [157, 195, 160], [196, 151, 152], [182, 119, 35], [38, 207, 21], [60, 226, 19], [147, 99, 169], [213, 78, 194], [85, 56, 150], [132, 206, 49], [197, 138, 163], [141, 213, 225], [224, 191, 124], [136, 24, 130], [7, 111, 107], [145, 28, 52], [122, 188, 8], [93, 44, 199], [20, 227, 148], [158, 245, 193], [254, 78, 140], [8, 233, 124], [57, 250, 57], [224, 28, 95], [136, 190, 217], [240, 184, 131], [33, 250, 24], [29, 203, 145], [127, 161, 134], [119, 110, 152], [48, 197, 182], [137, 73, 1], [51, 168, 62], [68, 91, 22], [154, 11, 229], [116, 255, 195], [213, 29, 144], [26, 180, 76], [42, 13, 36], [109, 176, 121], [9, 114, 8], [77, 52, 97], [112, 129, 98], [221, 191, 190], [114, 41, 179], [138, 234, 178], [24, 15, 18], [180, 31, 76], [139, 153, 86], [16, 149, 176], [45, 113, 243], [237, 198, 64], [111, 206, 185], [28, 237, 25], [80, 147, 3], [182, 148, 1], [64, 50, 235], [163, 160, 1], [48, 209, 95], [87, 104, 121], [18, 119, 95], [133, 117, 91], [136, 168, 206], [168, 11, 99], [13, 46, 167], [63, 11, 51], [93, 207, 113], [169, 69, 240], [4, 66, 172], [6, 153, 133], [70, 132, 50], [82, 48, 21], [237, 241, 49], [123, 9, 74], [239, 110, 145], [187, 163, 11], [179, 50, 88], [188, 133, 12], [177, 106, 250], [81, 119, 196], [140, 108, 176], [238, 139, 71], [65, 245, 22], [254, 83, 180], [109, 142, 152], [127, 197, 26], [250, 207, 10], [149, 165, 171], [245, 230, 237], [20, 164, 37], [182, 50, 167], [138, 219, 151], [109, 63, 127], [90, 146, 129], [230, 120, 124], [226, 191, 110], [157, 105, 19], [62, 16, 42], [189, 54, 25], [114, 176, 243], [38, 74, 187], [105, 206, 116], [71, 37, 106], [119, 225, 41], [26, 38, 131], [213, 93, 251], [131, 164, 184], [68, 242, 133], [213, 105, 117], [202, 68, 240], [80, 42, 249], [116, 207, 188], [246, 162, 98], [207, 75, 100], [81, 183, 235], [143, 254, 149], [237, 145, 23], [57, 134, 145], [210, 82, 47], [22, 230, 216], [125, 127, 224], [32, 210, 254], [110, 140, 138], [87, 27, 186], [31, 148, 238], [31, 142, 142], [254, 252, 59], [121, 154, 105], [237, 52, 129], [239, 5, 143], [102, 51, 90], [71, 125, 219], [124, 91, 41], [8, 154, 56], [218, 56, 97], [189, 178, 160], [35, 178, 224], [252, 160, 215], [206, 31, 172], [87, 247, 25], [121, 80, 24], [170, 137, 94], [207, 30, 241], [82, 73, 151], [93, 42, 10], [76, 143, 245], [204, 7, 121], [61, 212, 6], [110, 105, 155], [176, 221, 8], [245, 57, 18], [71, 203, 142], [157, 138, 221], [97, 237, 135], [124, 116, 175], [189, 37, 238], [47, 50, 251], [195, 129, 204], [165, 250, 224], [234, 173, 98], [190, 233, 182], [65, 71, 212], [160, 177, 54], [169, 13, 160], [24, 41, 194], [9, 140, 157], [196, 106, 22], [111, 239, 83], [88, 162, 55], [233, 172, 255], [55, 166, 60], [233, 110, 66], [24, 152, 145], [243, 194, 55], [14, 214, 174], [53, 31, 167], [14, 16, 233], [32, 233, 166], [15, 47, 5], [244, 88, 192], [120, 187, 3], [193, 1, 20], [2, 88, 52], [127, 240, 254], [223, 214, 94], [48, 144, 121], [169, 214, 149], [94, 49, 0], [183, 147, 146], [56, 74, 73], [119, 144, 102], [24, 53, 28], [185, 73, 117], [194, 212, 226], [147, 67, 72], [109, 71, 224], [81, 52, 98], [194, 111, 32], [164, 39, 48], [24, 90, 214], [163, 185, 66], [3, 62, 57], [134, 144, 217], [10, 96, 173], [113, 241, 185], [191, 172, 148], [202, 51, 82], [107, 150, 3], [218, 192, 184], [224, 203, 52], [45, 178, 6], [202, 104, 7], [180, 122, 199], [106, 39, 230], [60, 224, 244], [189, 111, 45], [110, 225, 141], [202, 146, 62], [252, 61, 169], [51, 237, 205], [42, 198, 91], [165, 128, 190], [49, 95, 44], [123, 238, 209], [249, 168, 253], [96, 240, 22], [241, 77, 152], [158, 33, 214], [150, 77, 248], [109, 142, 235], [254, 239, 52], [167, 184, 137], [197, 63, 19], [168, 42, 248], [156, 14, 10], [90, 149, 247], [23, 5, 213], [152, 249, 90], [157, 105, 178], [56, 50, 208], [133, 220, 40], [238, 233, 206], [20, 231, 233], [198, 180, 236], [210, 94, 205], [73, 115, 37], [221, 255, 123], [5, 85, 152], [27, 210, 212], [13, 180, 4], [237, 53, 39], [109, 17, 223], [170, 117, 164], [193, 147, 161], [228, 54, 8], [159, 48, 182], [128, 215, 3], [161, 254, 188], [253, 49, 234], [144, 72, 113], [127, 160, 101], [248, 224, 6], [174, 242, 21], [180, 58, 236], [120, 80, 43], [48, 212, 44], [225, 26, 217], [57, 167, 232], [196, 15, 7], [87, 84, 78], [69, 154, 61], [30, 40, 125], [184, 176, 204], [189, 113, 66], [209, 39, 147], [20, 161, 125], [153, 8, 78], [30, 39, 238], [22, 100, 17], [176, 180, 127], [218, 47, 29], [136, 247, 232], [207, 125, 42], [64, 8, 253], [251, 210, 30], [4, 39, 94], [157, 47, 144], [43, 243, 112], [213, 80, 248], [192, 170, 104], [23, 31, 1], [169, 190, 241], [225, 169, 179], [34, 157, 163], [30, 145, 44], [97, 76, 201], [211, 253, 66], [123, 65, 214], [86, 129, 81], [63, 11, 92], [93, 169, 124], [173, 73, 54], [67, 206, 155], [64, 28, 108], [3, 227, 149], [128, 156, 243], [25, 240, 50], [237, 81, 64], [178, 53, 67], [12, 139, 116], [103, 109, 90], [51, 41, 143], [197, 184, 78], [157, 103, 157], [142, 148, 161], [255, 103, 228], [71, 112, 51], [135, 129, 35], [138, 105, 158], [47, 156, 110], [203, 177, 55], [207, 60, 19], [165, 254, 230], [176, 94, 140], [174, 7, 238], [5, 185, 236], [39, 56, 113], [247, 153, 173], [195, 60, 247], [98, 213, 227], [172, 41, 86], [240, 255, 126], [247, 254, 138], [159, 166, 152], [149, 199, 1], [26, 5, 45], [106, 181, 126], [76, 172, 167], [230, 60, 202], [74, 100, 184], [121, 148, 107], [10, 40, 181], [113, 159, 251], [169, 102, 103], [20, 105, 155], [181, 59, 125], [240, 190, 174], [136, 30, 142], [69, 168, 174], [117, 65, 166], [242, 12, 27], [14, 214, 203], [241, 116, 134], [214, 151, 230], [30, 188, 97], [211, 217, 30], [126, 254, 247], [241, 100, 248], [210, 63, 80], [214, 177, 118], [241, 232, 61], [62, 127, 71], [177, 162, 153], [1, 29, 243], [218, 118, 251], [32, 73, 5], [120, 223, 223], [87, 48, 156], [195, 31, 95], [195, 6, 153], [156, 155, 52], [60, 95, 194], [70, 3, 74], [187, 231, 109], [198, 166, 209], [87, 238, 202], [251, 129, 14], [188, 60, 211], [49, 49, 222], [74, 67, 100], [83, 128, 216], [213, 211, 215], [228, 250, 9], [72, 211, 49], [250, 173, 3], [179, 165, 145], [119, 104, 33], [35, 59, 50], [220, 84, 238], [29, 64, 230], [92, 121, 144], [204, 108, 220], [151, 102, 177], [113, 163, 237], [186, 106, 106], [179, 201, 248], [89, 20, 143], [130, 161, 31], [197, 109, 117], [96, 115, 68], [246, 51, 6], [85, 3, 200], [120, 146, 136], [41, 192, 70], [177, 36, 7], [138, 100, 6], [3, 13, 209], [97, 79, 250], [223, 231, 143], [182, 83, 215], [66, 244, 91], [133, 144, 121], [205, 210, 226], [221, 2, 17], [37, 189, 79], [45, 251, 152], [154, 255, 214], [37, 54, 79], [149, 110, 172], [11, 118, 36], [27, 129, 66], [46, 226, 172], [204, 31, 49], [233, 81, 105], [14, 229, 3], [248, 187, 220], [53, 86, 109], [237, 174, 138], [31, 12, 216], [4, 133, 241], [46, 179, 213], [118, 129, 186], [160, 196, 204], [12, 248, 254], [216, 201, 35], [69, 223, 115], [62, 118, 175], [129, 83, 233], [222, 34, 191], [54, 65, 175], [178, 15, 136], [209, 149, 248], [250, 145, 218], [122, 138, 101], [80, 231, 215], [121, 14, 34], [21, 39, 45], [81, 133, 178], [147, 145, 29], [143, 152, 40], [179, 27, 154], [201, 66, 12], [141, 196, 201], [62, 21, 35], [13, 103, 138], [188, 35, 35], [219, 176, 142], [186, 67, 252], [169, 47, 157], [203, 76, 55], [195, 170, 175], [190, 198, 188], [253, 33, 136], [75, 229, 137], [31, 61, 25], [110, 28, 233], [31, 163, 12], [220, 220, 53], [223, 216, 198], [211, 230, 68], [151, 109, 243], [96, 80, 111], [248, 105, 28], [158, 200, 47], [141, 166, 183], [130, 117, 214], [135, 169, 12], [227, 87, 135], [2, 213, 33], [150, 67, 13], [190, 90, 71], [192, 143, 92], [220, 76, 168], [114, 95, 237], [88, 23, 136], [201, 4, 171], [166, 58, 115], [196, 205, 71], [176, 186, 40], [241, 75, 175], [0, 202, 59], [37, 101, 20], [34, 74, 10], [252, 89, 240], [136, 150, 65], [146, 28, 55], [138, 224, 144], [179, 174, 60], [31, 39, 63], [68, 43, 236], [88, 22, 14], [76, 189, 6], [110, 2, 148], [238, 26, 32], [239, 237, 249], [158, 96, 171], [213, 223, 77], [9, 93, 150], [28, 215, 23], [111, 132, 38], [25, 39, 25], [235, 193, 97], [109, 113, 129], [126, 2, 79], [39, 35, 252], [28, 64, 43], [48, 214, 210], [183, 192, 40], [7, 47, 93], [103, 247, 198], [112, 16, 255], [210, 192, 249], [73, 87, 106], [244, 75, 208], [250, 157, 41], [149, 45, 64], [243, 7, 185], [81, 132, 66], [195, 203, 107], [79, 253, 16], [66, 141, 139], [203, 143, 174], [216, 168, 118], [76, 93, 145], [215, 93, 7], [211, 218, 15], [13, 120, 96], [234, 212, 14], [220, 129, 240], [224, 52, 82], [246, 136, 96], [147, 57, 226], [144, 247, 45], [13, 52, 156], [131, 175, 20], [254, 93, 239], [63, 142, 145], [0, 214, 69], [133, 222, 0], [172, 184, 245], [113, 103, 95], [34, 15, 185], [56, 189, 19], [194, 164, 63], [63, 233, 36], [29, 192, 6], [96, 172, 2], [145, 113, 59], [53, 66, 5], [153, 253, 103], [219, 145, 169], [23, 68, 78], [52, 244, 71], [240, 35, 105], [210, 50, 115], [10, 84, 154], [149, 0, 220], [150, 164, 217], [23, 255, 27], [107, 57, 184], [60, 242, 89], [39, 176, 113], [251, 181, 116], [105, 42, 215], [76, 89, 14], [17, 247, 189], [14, 32, 236], [186, 16, 58], [186, 7, 51], [245, 12, 90], [178, 128, 198], [23, 121, 253], [200, 50, 165], [35, 148, 92], [126, 253, 203], [126, 160, 91], [174, 113, 250], [45, 129, 216], [90, 194, 232], [207, 5, 123], [30, 107, 211], [162, 7, 200], [44, 125, 26], [110, 20, 196], [18, 216, 97], [126, 196, 224], [245, 220, 255], [128, 50, 209], [213, 129, 33], [172, 41, 47], [172, 248, 91], [91, 8, 225], [10, 104, 139], [128, 7, 213], [55, 192, 11], [92, 162, 240], [10, 113, 185], [176, 7, 81], [126, 145, 52], [155, 58, 21], [28, 170, 76], [112, 6, 65], [142, 27, 154], [80, 107, 178], [125, 243, 70], [120, 77, 111], [203, 99, 227], [131, 93, 192], [196, 168, 224], [176, 61, 51], [85, 216, 253], [56, 239, 195], [191, 42, 24], [219, 207, 3], [218, 142, 128], [11, 165, 251], [224, 8, 28], [228, 12, 5], [145, 207, 134], [255, 174, 125], [239, 225, 110], [120, 65, 177], [123, 83, 236], [140, 46, 107], [42, 250, 86], [149, 65, 161], [147, 70, 144], [72, 233, 238], [17, 115, 163], [216, 141, 46], [72, 180, 35], [216, 180, 116], [220, 4, 88], [145, 44, 214], [73, 162, 130], [228, 77, 105], [230, 167, 27], [142, 169, 169], [226, 167, 160], [90, 78, 129], [11, 116, 133], [112, 48, 249], [58, 49, 233], [156, 82, 77], [188, 6, 223], [133, 88, 166], [99, 97, 247], [245, 199, 45], [133, 25, 157], [240, 92, 33], [67, 2, 141], [157, 66, 73], [138, 108, 234], [59, 141, 191], [204, 206, 166], [63, 228, 107], [83, 252, 168], [105, 46, 167], [66, 255, 175], [178, 96, 188], [223, 165, 242], [12, 68, 34], [183, 10, 68], [107, 32, 0], [9, 14, 75], [245, 129, 158], [182, 209, 2], [150, 192, 175], [34, 147, 16], [84, 22, 45], [9, 182, 51], [210, 134, 219], [103, 240, 10], [50, 1, 247], [24, 66, 18], [254, 15, 189], [71, 2, 112], [216, 161, 58], [30, 234, 249], [142, 82, 252], [177, 111, 2], [92, 209, 143], [52, 11, 227], [147, 144, 45], [239, 99, 251], [45, 22, 221], [186, 202, 199], [215, 155, 248], [18, 137, 53], [120, 170, 254], [144, 11, 12], [92, 190, 37], [252, 3, 15], [245, 211, 210], [236, 4, 49], [78, 135, 173], [238, 238, 188], [15, 160, 215], [173, 187, 28], [231, 10, 17], [24, 208, 150], [28, 88, 152], [90, 195, 209], [22, 23, 176], [220, 91, 108], [75, 196, 151], [7, 161, 178], [84, 153, 233], [39, 42, 122], [199, 177, 24], [184, 230, 217], [61, 196, 7], [95, 245, 77], [232, 153, 107], [91, 122, 165], [4, 7, 212], [103, 46, 212], [65, 219, 169], [97, 221, 161], [105, 254, 244], [229, 129, 210], [2, 201, 141], [113, 8, 191], [88, 222, 157], [81, 72, 252], [225, 171, 70], [103, 153, 7], [28, 126, 222], [195, 62, 232], [113, 74, 174], [206, 108, 218], [137, 211, 164], [3, 241, 15], [181, 175, 32], [81, 17, 31], [31, 211, 171], [138, 181, 64], [42, 0, 29], [85, 173, 245], [14, 196, 216], [204, 38, 244], [166, 92, 57], [47, 134, 40], [185, 182, 216], [144, 18, 114], [192, 154, 238], [2, 245, 57], [27, 161, 212], [59, 7, 168], [209, 39, 246], [161, 169, 64], [26, 244, 122], [249, 29, 157], [45, 35, 246], [168, 147, 145], [41, 45, 115], [63, 189, 86], [90, 20, 225], [164, 77, 83], [26, 201, 172], [163, 196, 133], [214, 56, 4], [108, 68, 104], [149, 138, 115], [30, 54, 91], [53, 168, 82], [231, 13, 184], [147, 9, 110], [235, 174, 123], [0, 241, 21], [3, 88, 36], [161, 53, 111], [141, 242, 125], [150, 211, 140], [8, 64, 29], [133, 235, 189], [104, 38, 89], [45, 221, 150], [254, 160, 39], [93, 193, 176], [6, 34, 38], [233, 178, 147], [198, 66, 171], [106, 146, 162], [140, 235, 177], [105, 125, 106], [234, 78, 214], [231, 46, 213], [114, 234, 170], [20, 82, 243], [50, 249, 163], [40, 149, 134], [174, 68, 249], [209, 166, 114], [24, 22, 98], [53, 243, 168], [24, 104, 229], [6, 33, 219], [126, 251, 135], [4, 111, 233], [244, 164, 246], [93, 98, 73], [66, 184, 130], [110, 161, 196], [89, 48, 85], [20, 69, 254], [88, 40, 141], [0, 35, 104], [64, 18, 57], [249, 183, 152], [69, 228, 152], [171, 67, 176], [72, 112, 157], [176, 92, 52], [115, 147, 72], [186, 150, 129], [23, 106, 69], [148, 11, 31], [29, 83, 211], [111, 211, 53], [191, 239, 2], [54, 24, 40], [218, 129, 178], [90, 27, 143], [190, 198, 81], [227, 210, 211], [93, 1, 93], [122, 34, 193], [242, 14, 189], [138, 131, 49], [245, 194, 77], [236, 104, 117], [103, 32, 104], [28, 78, 18], [213, 235, 89], [29, 32, 247], [240, 65, 99], [35, 200, 110], [255, 222, 29], [192, 52, 146], [200, 159, 133], [128, 198, 209], [8, 158, 139], [80, 130, 153], [35, 11, 174], [148, 39, 206], [57, 160, 34], [168, 212, 63], [124, 224, 183], [160, 136, 161], [169, 234, 98], [172, 170, 185], [177, 116, 126], [70, 122, 237], [123, 110, 139], [150, 190, 204], [127, 185, 255], [87, 66, 156], [34, 213, 48], [254, 119, 33], [198, 248, 144], [151, 208, 248], [183, 48, 156], [17, 6, 71], [5, 192, 246], [185, 241, 143], [185, 73, 179], [43, 143, 61], [126, 231, 212], [5, 72, 205], [76, 81, 112], [109, 191, 82], [153, 235, 252], [27, 33, 85], [98, 236, 58], [183, 165, 171], [78, 201, 136], [180, 144, 245], [25, 215, 171], [172, 119, 87], [7, 79, 38], [161, 54, 134], [185, 139, 45], [28, 119, 212], [150, 88, 220], [69, 142, 76], [9, 125, 112], [222, 20, 62], [115, 105, 196], [130, 174, 216], [163, 110, 52], [229, 38, 190], [148, 16, 153], [48, 225, 64], [239, 72, 59], [51, 129, 42], [203, 92, 29], [65, 90, 236], [231, 25, 180], [52, 34, 114], [52, 178, 191], [20, 208, 96], [56, 66, 225], [189, 64, 61], [236, 128, 68], [34, 141, 55], [212, 241, 229], [0, 179, 244], [118, 173, 46], [244, 13, 81], [189, 180, 63], [97, 76, 24], [55, 164, 86], [139, 252, 181], [139, 213, 152], [248, 59, 104], [33, 93, 15], [14, 210, 42], [59, 190, 81], [125, 171, 172], [0, 13, 13], [79, 86, 243], [149, 169, 110], [39, 93, 4], [118, 137, 161], [51, 117, 7], [93, 73, 125], [120, 152, 62], [148, 144, 33], [78, 30, 86], [174, 138, 235], [132, 230, 144], [224, 250, 249], [145, 87, 238], [56, 19, 220], [120, 166, 121], [117, 35, 146], [1, 133, 77], [99, 242, 128], [182, 28, 33], [174, 97, 164], [41, 204, 218], [222, 59, 167], [69, 212, 197], [137, 185, 193], [0, 241, 191], [61, 72, 146], [248, 146, 41], [49, 232, 103], [46, 93, 81], [28, 133, 103], [92, 138, 149], [189, 108, 125], [124, 167, 34], [49, 27, 123], [206, 199, 214], [121, 136, 68], [226, 82, 175], [149, 192, 203], [143, 5, 168], [20, 49, 86], [179, 16, 179], [45, 177, 236], [200, 95, 121], [197, 56, 154], [247, 145, 186], [206, 93, 196], [184, 63, 15], [106, 70, 137], [68, 21, 215], [228, 205, 125], [86, 1, 14], [157, 198, 202], [75, 11, 199], [60, 103, 33], [90, 143, 225], [201, 42, 63], [241, 61, 187], [248, 99, 89], [221, 94, 131], [204, 234, 184], [139, 44, 219], [140, 6, 3], [43, 146, 234], [63, 217, 38], [171, 103, 114], [70, 108, 179], [11, 90, 72], [210, 158, 16], [84, 166, 151], [185, 115, 190], [233, 211, 84], [132, 244, 195], [40, 173, 85], [103, 22, 43], [9, 52, 82], [140, 201, 250], [158, 35, 30], [254, 173, 14], [35, 102, 77], [42, 184, 211], [85, 40, 50], [125, 170, 6], [254, 104, 241], [196, 157, 3], [51, 88, 53], [217, 9, 41], [203, 91, 250], [63, 207, 201], [60, 242, 80], [176, 22, 10], [161, 60, 164], [172, 63, 79], [135, 35, 223], [159, 153, 255], [137, 94, 58], [238, 171, 243], [90, 22, 116], [27, 94, 119], [98, 165, 135], [233, 49, 233], [230, 162, 212], [4, 108, 92], [91, 0, 165], [70, 61, 103], [66, 138, 48], [222, 68, 201], [161, 113, 85], [38, 46, 95], [56, 59, 220], [66, 99, 61], [56, 69, 202], [6, 105, 204], [204, 246, 112], [22, 208, 123], [192, 156, 177], [245, 121, 44], [100, 60, 70], [23, 10, 80], [201, 143, 105], [33, 24, 238], [169, 164, 203], [28, 215, 52], [145, 204, 2], [59, 3, 115], [53, 44, 125], [115, 68, 19], [53, 30, 116], [117, 103, 209], [193, 84, 6], [161, 133, 136], [136, 127, 97], [159, 244, 246], [69, 119, 85], [94, 67, 194], [206, 8, 185], [93, 171, 244], [84, 230, 109], [101, 212, 202], [157, 201, 33], [74, 202, 6], [216, 23, 138], [192, 61, 181], [97, 209, 240], [136, 74, 238], [21, 131, 76], [74, 33, 176], [20, 140, 44], [63, 180, 3], [125, 86, 35], [107, 92, 163], [54, 244, 142], [158, 55, 37], [67, 225, 53], [87, 51, 29], [47, 108, 113], [194, 67, 139], [194, 155, 92], [144, 90, 229], [31, 12, 241], [254, 22, 255], [199, 131, 253], [214, 117, 228], [178, 53, 122], [199, 161, 210], [241, 117, 111], [102, 157, 87], [65, 251, 217], [211, 21, 121], [55, 224, 17], [99, 52, 14], [71, 244, 210], [12, 125, 223], [55, 187, 249], [60, 215, 175], [167, 8, 92], [65, 220, 252], [66, 245, 167], [135, 33, 92], [97, 133, 40], [167, 112, 42], [141, 250, 180], [131, 213, 149], [65, 60, 110], [176, 219, 120], [125, 88, 45], [123, 1, 136], [230, 53, 64], [69, 64, 216], [207, 197, 10], [17, 223, 229], [43, 118, 20], [56, 83, 41], [112, 93, 77], [2, 109, 26], [121, 141, 162], [100, 114, 54], [206, 89, 241], [200, 80, 22], [206, 76, 242], [235, 63, 25], [9, 140, 206], [99, 82, 113], [99, 9, 195], [107, 171, 134], [69, 242, 10], [23, 179, 80], [38, 92, 143], [66, 23, 201], [245, 234, 100], [82, 159, 63], [121, 121, 249], [96, 61, 237], [175, 53, 69], [124, 77, 180], [154, 46, 197], [74, 165, 189], [39, 102, 211], [248, 126, 22], [83, 62, 93], [220, 0, 125], [250, 247, 252], [157, 165, 247], [93, 136, 97], [85, 209, 198], [16, 0, 64], [49, 112, 41], [57, 216, 188], [133, 184, 9], [48, 227, 9], [43, 228, 234], [149, 89, 181], [167, 45, 144], [107, 14, 153], [190, 189, 22], [193, 189, 248], [168, 224, 63], [6, 77, 209], [55, 169, 47], [252, 109, 233], [253, 224, 53], [39, 198, 164], [69, 24, 84], [191, 1, 138], [214, 8, 112], [199, 113, 46], [118, 222, 186], [125, 111, 139], [181, 113, 30], [232, 39, 67], [13, 54, 223], [113, 70, 159], [2, 46, 132], [85, 56, 189], [101, 197, 8], [32, 111, 104], [234, 254, 186], [126, 114, 213], [68, 240, 25], [27, 208, 61], [56, 241, 131], [13, 135, 192], [101, 124, 87], [137, 81, 23], [164, 217, 35], [247, 95, 224], [127, 141, 182], [87, 89, 25], [162, 100, 199], [126, 187, 154], [64, 27, 240], [175, 144, 132], [3, 237, 106], [17, 76, 109], [202, 155, 168], [129, 35, 24], [79, 120, 24], [106, 109, 43], [53, 247, 22], [255, 134, 201], [96, 230, 183], [188, 19, 77], [182, 235, 176], [163, 79, 41], [69, 246, 86], [57, 75, 40], [199, 46, 211], [173, 69, 8], [168, 200, 66], [215, 84, 172], [126, 31, 221], [187, 53, 112], [26, 92, 127], [186, 68, 79], [2, 227, 48], [24, 213, 150], [47, 64, 135], [196, 8, 140], [8, 6, 203], [216, 119, 221], [211, 5, 240], [237, 249, 12], [194, 231, 197], [205, 116, 150], [121, 137, 6], [54, 121, 219], [84, 111, 254], [116, 107, 164], [151, 227, 191], [69, 96, 50], [36, 235, 104], [200, 221, 159], [168, 213, 213], [110, 33, 21], [77, 106, 197], [79, 23, 217], [205, 153, 190], [231, 34, 42], [15, 177, 82], [52, 2, 11], [201, 116, 58], [48, 81, 93], [78, 46, 181], [84, 53, 232], [218, 149, 6], [247, 121, 58], [94, 235, 26], [239, 158, 52], [10, 203, 233], [26, 242, 196], [16, 0, 29], [116, 47, 198], [81, 84, 165], [57, 16, 193], [26, 187, 96], [128, 247, 193], [211, 166, 46], [4, 253, 222], [40, 18, 170], [254, 20, 10], [104, 228, 247], [91, 101, 218], [229, 29, 87], [180, 104, 23], [249, 221, 151], [41, 234, 203], [15, 6, 151], [58, 23, 26], [38, 160, 90], [115, 165, 106], [194, 245, 24], [28, 57, 118], [59, 217, 176], [124, 207, 27], [5, 70, 96], [4, 224, 155], [49, 133, 255], [109, 169, 155], [77, 178, 96], [14, 137, 91], [80, 141, 56], [31, 149, 155], [147, 219, 191], [17, 254, 73], [104, 26, 66], [224, 107, 50], [3, 224, 48], [25, 48, 220], [141, 147, 200], [61, 69, 173], [221, 35, 216], [108, 103, 156], [38, 172, 189], [99, 1, 163], [7, 139, 123], [50, 117, 239], [66, 12, 168], [98, 195, 50], [150, 117, 39], [246, 88, 177], [89, 100, 229], [197, 17, 39], [237, 78, 238], [13, 46, 110], [49, 152, 76], [203, 213, 145], [220, 87, 90], [97, 146, 128], [232, 52, 20], [35, 29, 141], [247, 75, 205], [168, 204, 177], [253, 76, 53], [197, 71, 240], [182, 253, 1], [191, 205, 227], [204, 137, 124], [122, 182, 182], [252, 155, 1], [15, 27, 34], [25, 179, 185], [193, 161, 117], [241, 108, 148], [104, 45, 174], [92, 159, 206], [225, 54, 7], [183, 172, 123], [228, 200, 9], [203, 161, 163], [162, 221, 37], [71, 23, 206], [81, 192, 99], [149, 195, 28], [236, 15, 40], [75, 227, 97], [15, 159, 166], [49, 109, 29], [138, 149, 241], [180, 178, 167], [72, 249, 68], [255, 189, 159], [161, 244, 206], [93, 45, 114], [88, 138, 123], [80, 136, 94], [69, 123, 3], [78, 253, 208], [84, 13, 220], [0, 144, 139], [150, 31, 219], [254, 148, 109], [150, 146, 221], [53, 208, 21], [162, 198, 106], [210, 254, 221], [41, 202, 171], [199, 226, 210], [211, 10, 146], [230, 54, 111], [133, 23, 154], [208, 122, 26], [54, 85, 14], [203, 31, 200], [172, 10, 55], [118, 243, 245], [25, 164, 146], [160, 180, 160], [88, 9, 199], [21, 125, 176], [204, 10, 16], [188, 105, 75], [238, 101, 226], [228, 198, 3], [221, 161, 89], [235, 76, 203], [97, 245, 66], [159, 126, 173], [244, 40, 147], [103, 231, 149], [177, 109, 209], [2, 95, 248], [130, 194, 156], [151, 78, 152], [191, 177, 32], [90, 248, 155], [232, 70, 30], [156, 230, 16], [174, 81, 151], [78, 0, 121], [56, 82, 140], [250, 46, 152], [131, 149, 178], [164, 92, 230], [149, 148, 103], [61, 112, 8], [150, 148, 115], [86, 30, 82], [215, 208, 96], [85, 248, 190], [48, 66, 92], [110, 21, 217], [213, 3, 223], [145, 134, 206], [110, 225, 101], [150, 74, 61], [136, 190, 62], [72, 45, 32], [172, 154, 145], [110, 208, 79], [95, 107, 122], [38, 217, 249], [20, 90, 84], [9, 242, 134], [79, 14, 237], [98, 147, 93], [83, 128, 184], [62, 111, 100], [240, 81, 96], [79, 250, 18], [140, 100, 68], [173, 11, 161], [29, 201, 168], [164, 158, 249], [102, 34, 62], [233, 145, 60], [177, 48, 87], [229, 145, 87], [84, 246, 56], [110, 184, 204], [8, 118, 134], [165, 114, 198], [42, 250, 251], [179, 142, 110], [122, 158, 17], [39, 65, 172], [241, 154, 2], [83, 205, 21], [118, 133, 68], [22, 91, 221], [101, 129, 69], [70, 211, 122], [37, 99, 122], [110, 145, 213], [81, 108, 130], [146, 232, 213], [68, 208, 42], [158, 115, 228], [194, 140, 151], [114, 81, 113], [9, 118, 146], [14, 48, 26], [159, 27, 108], [51, 214, 130], [146, 219, 13], [30, 214, 229], [203, 176, 132], [150, 182, 237], [92, 45, 255], [190, 243, 59], [147, 248, 116], [138, 171, 223], [145, 84, 84], [239, 228, 210], [47, 75, 49], [29, 149, 110], [2, 86, 221], [197, 169, 60], [203, 149, 73], [149, 16, 74], [227, 130, 139], [144, 41, 73], [39, 91, 127], [159, 139, 115], [247, 169, 90], [247, 14, 25], [215, 60, 104], [239, 208, 37], [18, 146, 13], [57, 235, 30], [35, 59, 72], [107, 242, 164], [144, 206, 156], [135, 248, 174], [50, 127, 111], [244, 112, 65], [36, 84, 79], [64, 74, 153], [0, 119, 138], [12, 147, 62], [116, 158, 251], [190, 181, 37], [107, 181, 133], [219, 59, 5], [183, 57, 194], [135, 194, 68], [83, 26, 199], [136, 45, 148], [60, 224, 64], [67, 17, 59], [3, 29, 10], [143, 2, 71], [189, 141, 116], [66, 193, 246], [7, 190, 9], [255, 129, 65], [16, 4, 91], [253, 81, 66], [182, 117, 137], [18, 71, 237], [35, 184, 136], [170, 189, 117], [214, 159, 39], [220, 186, 206], [223, 193, 7], [228, 46, 12], [228, 243, 167], [196, 39, 167], [132, 234, 85], [253, 184, 168], [31, 199, 146], [173, 145, 50], [219, 31, 17], [183, 191, 194], [8, 140, 3], [142, 193, 187], [113, 15, 99], [226, 49, 236], [98, 19, 70], [26, 104, 54], [145, 214, 71], [43, 158, 237], [237, 252, 1], [185, 30, 17], [167, 228, 179], [193, 101, 221], [163, 233, 25], [217, 52, 185], [217, 20, 122], [96, 95, 16], [74, 47, 250], [26, 240, 116], [233, 109, 18], [67, 64, 82], [38, 31, 166], [80, 153, 21], [252, 206, 110], [172, 179, 218], [23, 239, 37], [183, 8, 219], [2, 98, 177], [205, 182, 124], [119, 89, 5], [148, 39, 186], [45, 79, 224], [190, 134, 176], [40, 85, 17], [66, 3, 12], [34, 49, 172], [76, 117, 189], [10, 46, 230], [11, 88, 11], [15, 173, 245], [189, 42, 94], [251, 150, 34], [101, 23, 17], [27, 10, 58], [142, 48, 116], [206, 154, 171], [199, 248, 84], [8, 206, 88], [190, 212, 151], [249, 231, 41], [89, 233, 159], [23, 179, 141], [38, 141, 126], [105, 87, 131], [85, 191, 201], [0, 96, 92], [254, 58, 103], [206, 7, 70], [197, 69, 152], [5, 221, 57], [145, 41, 203], [175, 44, 236], [193, 247, 73], [98, 125, 211], [89, 199, 246], [249, 59, 93], [221, 4, 25], [183, 197, 122], [14, 103, 123], [62, 115, 223], [44, 242, 89], [182, 183, 225], [188, 198, 180], [155, 149, 69], [121, 118, 180], [93, 237, 218], [215, 220, 87], [70, 196, 131], [163, 64, 103], [41, 163, 108], [48, 61, 192], [0, 160, 117], [50, 131, 213], [189, 69, 139], [132, 209, 133], [68, 7, 108], [120, 34, 42], [15, 92, 248], [56, 168, 137], [248, 146, 63], [58, 159, 70], [88, 170, 153], [185, 233, 230], [0, 251, 75], [136, 224, 111], [149, 155, 226], [248, 2, 77], [49, 166, 68], [130, 226, 199], [143, 73, 131], [77, 185, 122], [216, 252, 50], [191, 32, 38], [91, 9, 27], [70, 183, 88], [135, 188, 244], [59, 3, 126], [72, 62, 180], [130, 248, 201], [110, 17, 43], [251, 178, 222], [35, 116, 163], [159, 178, 21], [207, 132, 199], [111, 33, 127], [126, 115, 82], [194, 211, 212], [142, 197, 217], [60, 115, 102], [175, 227, 166], [54, 145, 244], [107, 139, 72], [220, 148, 191], [245, 14, 126], [235, 27, 197], [254, 160, 90], [96, 101, 29], [153, 154, 217], [28, 188, 27], [69, 134, 245], [155, 57, 230], [213, 194, 127], [197, 207, 185], [222, 31, 149], [137, 244, 39], [12, 73, 114], [148, 38, 105], [208, 181, 99], [83, 218, 129], [220, 187, 127], [138, 60, 119], [202, 195, 41], [61, 49, 7], [172, 100, 11], [16, 127, 67], [94, 254, 35], [50, 226, 98], [186, 69, 197], [56, 39, 201], [244, 91, 116], [207, 109, 78], [83, 253, 103], [100, 155, 48], [69, 40, 58], [19, 103, 28], [173, 153, 195], [230, 146, 188], [38, 45, 77], [6, 2, 177], [201, 95, 211], [94, 252, 6], [28, 10, 167], [172, 213, 250], [205, 220, 132], [138, 160, 242], [46, 17, 29], [17, 26, 18], [158, 126, 251], [85, 96, 194], [182, 38, 219], [58, 131, 17], [200, 89, 202], [101, 224, 148], [131, 212, 237], [183, 125, 82], [185, 123, 141], [146, 21, 110], [96, 150, 81], [44, 52, 40], [170, 59, 177], [128, 69, 110], [56, 150, 250], [24, 212, 152], [144, 253, 242], [81, 102, 74], [71, 13, 92], [156, 174, 205], [59, 31, 43], [53, 245, 67], [51, 25, 37], [60, 11, 62], [35, 155, 127], [15, 187, 34], [244, 228, 135], [215, 206, 170], [196, 195, 27], [211, 255, 251], [153, 9, 2], [108, 137, 145], [222, 198, 43], [159, 194, 24], [42, 15, 234], [240, 234, 27], [112, 156, 241], [227, 48, 59], [166, 158, 98], [223, 232, 25], [180, 97, 246], [167, 75, 250], [140, 168, 154], [72, 182, 56], [160, 37, 165], [45, 135, 196], [160, 116, 28], [100, 147, 215], [159, 168, 15], [12, 74, 127], [76, 193, 235], [199, 211, 248], [83, 26, 47], [124, 125, 240], [211, 27, 179], [182, 116, 71], [50, 84, 145], [200, 7, 91], [188, 71, 7], [10, 204, 69], [213, 11, 109], [153, 50, 209], [11, 31, 192], [13, 94, 18], [73, 11, 110], [89, 62, 203], [25, 137, 183], [31, 74, 29], [33, 193, 206], [227, 196, 53], [187, 248, 63], [171, 245, 138], [183, 245, 3], [231, 143, 16], [45, 118, 50], [119, 189, 232], [167, 110, 74], [12, 53, 206], [84, 104, 255], [186, 89, 212], [86, 7, 171], [40, 21, 150], [155, 106, 252], [9, 6, 147], [251, 237, 118], [216, 211, 97], [34, 157, 239], [14, 10, 129], [47, 85, 125], [172, 115, 248], [133, 10, 2], [74, 63, 222], [211, 13, 239], [50, 99, 165], [118, 137, 67], [222, 125, 117], [102, 167, 188], [222, 47, 121], [35, 81, 239], [8, 106, 26], [105, 72, 97], [203, 59, 26], [203, 222, 109], [223, 219, 84], [38, 211, 155], [192, 169, 5], [168, 227, 198], [97, 134, 195], [153, 80, 251], [41, 106, 122], [239, 6, 122], [81, 76, 116], [162, 101, 42], [221, 124, 62], [103, 120, 7], [223, 253, 193], [45, 117, 105], [73, 21, 141], [101, 87, 61], [229, 108, 238], [122, 131, 103], [151, 196, 12], [19, 251, 163], [188, 134, 43], [3, 108, 224], [90, 138, 139], [50, 179, 174], [193, 228, 44], [131, 98, 5], [9, 122, 39], [133, 166, 26], [67, 123, 21], [21, 93, 14], [218, 31, 20], [86, 22, 4], [192, 202, 102], [125, 238, 114], [160, 112, 144], [136, 115, 179], [149, 220, 196], [40, 206, 188], [200, 37, 64], [222, 98, 215], [33, 169, 127], [32, 226, 234], [183, 38, 90], [74, 252, 209], [232, 122, 14], [231, 120, 187], [92, 1, 44], [103, 222, 47], [238, 155, 56], [221, 54, 128], [183, 141, 38], [148, 238, 35], [62, 140, 171], [43, 246, 7], [189, 188, 44], [50, 193, 225], [143, 109, 136], [75, 160, 95], [123, 186, 174], [126, 158, 188], [20, 88, 76], [49, 139, 91], [184, 217, 104], [147, 247, 183], [203, 30, 131], [205, 67, 58], [83, 221, 170], [116, 102, 144], [93, 19, 6], [138, 205, 163], [1, 70, 134], [113, 192, 79], [250, 181, 204], [132, 188, 157], [42, 138, 47], [60, 228, 248], [209, 74, 59], [127, 215, 187], [23, 112, 114], [128, 33, 123], [119, 73, 255], [185, 126, 245], [59, 109, 178], [68, 184, 182], [242, 249, 34], [217, 38, 158], [81, 133, 151], [100, 122, 99], [211, 183, 221], [71, 165, 7], [247, 86, 144], [99, 167, 158], [193, 117, 152], [156, 70, 13], [253, 192, 175], [67, 31, 124], [170, 124, 12], [255, 159, 111], [210, 85, 114], [187, 32, 118], [26, 196, 77], [239, 62, 84], [173, 217, 50], [254, 127, 103], [235, 148, 17], [27, 175, 111], [16, 73, 111], [178, 69, 27], [229, 25, 192], [141, 188, 155], [37, 151, 140], [152, 171, 126], [225, 29, 254], [125, 216, 222], [29, 95, 85], [86, 207, 212], [135, 29, 86], [55, 29, 183], [80, 108, 138], [17, 44, 241], [129, 245, 52], [172, 74, 25], [214, 244, 243], [240, 202, 153], [155, 240, 40], [22, 113, 99], [251, 205, 142], [188, 94, 230], [91, 178, 105], [221, 102, 197], [86, 40, 69], [13, 97, 175], [211, 159, 126], [119, 152, 39], [175, 193, 238], [175, 252, 154], [244, 203, 160], [92, 254, 221], [165, 196, 89], [69, 11, 89], [8, 192, 13], [175, 205, 234], [22, 59, 150], [233, 102, 72], [138, 176, 0], [161, 59, 34], [15, 231, 47], [13, 247, 20], [236, 98, 78], [120, 137, 203], [60, 21, 19], [204, 94, 56], [198, 93, 138], [175, 233, 100], [193, 32, 176], [229, 80, 109], [102, 132, 30], [122, 208, 46], [237, 207, 151], [131, 96, 55], [66, 213, 29], [252, 225, 229], [124, 170, 192], [65, 58, 80], [196, 198, 185], [241, 92, 4], [118, 147, 185], [187, 136, 218], [186, 218, 241], [105, 207, 194], [61, 235, 248], [83, 22, 99], [45, 215, 89], [79, 140, 91], [24, 37, 187], [54, 147, 138], [247, 231, 32], [171, 116, 245], [101, 50, 59], [76, 0, 230], [137, 173, 191], [255, 235, 202], [163, 197, 7], [157, 190, 134], [189, 227, 49], [76, 206, 205], [83, 95, 246], [148, 67, 190], [233, 52, 31], [54, 79, 197], [113, 173, 110], [8, 146, 204], [196, 187, 7], [66, 193, 178], [224, 141, 196], [143, 131, 199], [111, 3, 244], [60, 61, 47], [186, 80, 113], [50, 180, 205], [139, 254, 240], [21, 19, 122], [26, 55, 182], [120, 128, 125], [69, 132, 227], [26, 123, 219], [41, 76, 44], [152, 244, 149], [110, 233, 169], [90, 229, 115], [153, 206, 240], [158, 18, 154], [108, 28, 209], [15, 64, 151], [251, 226, 179], [94, 191, 161], [195, 204, 68], [93, 186, 20], [109, 66, 86], [162, 216, 47], [11, 140, 84], [242, 70, 152], [238, 82, 247], [135, 247, 255], [224, 194, 242], [194, 54, 60], [49, 189, 138], [181, 77, 9], [35, 50, 227], [67, 205, 13], [105, 185, 107], [76, 111, 32], [30, 242, 37], [60, 37, 118], [188, 3, 41], [0, 188, 233], [92, 94, 121], [3, 81, 224], [63, 205, 222], [96, 84, 106], [215, 48, 184], [152, 205, 182], [64, 12, 164], [81, 161, 107], [235, 86, 151], [131, 8, 69], [98, 210, 197], [1, 89, 81], [227, 152, 218], [80, 172, 96], [61, 220, 46], [219, 115, 195], [37, 223, 151], [100, 55, 210], [214, 129, 17], [132, 7, 41], [126, 246, 61], [118, 25, 11], [52, 229, 251], [132, 120, 199], [136, 104, 206], [108, 125, 104], [86, 187, 244], [147, 137, 121], [85, 240, 201], [155, 238, 78], [218, 62, 55], [68, 4, 142], [212, 231, 148], [9, 110, 134], [38, 237, 35], [56, 212, 65], [16, 109, 248], [86, 95, 208], [209, 41, 221], [22, 191, 86], [146, 94, 46], [207, 139, 53], [140, 110, 157], [227, 214, 73], [142, 0, 136], [21, 131, 58], [110, 76, 2], [163, 223, 255], [198, 49, 37], [52, 31, 148], [54, 16, 159], [134, 79, 69], [62, 5, 39], [92, 57, 93], [78, 94, 46], [187, 101, 105], [207, 228, 183], [161, 177, 193], [221, 13, 51], [228, 83, 185], [7, 18, 249], [71, 116, 73], [203, 154, 157], [140, 176, 51], [226, 238, 84], [81, 223, 73], [22, 234, 236], [2, 29, 61], [211, 12, 166], [47, 182, 40], [108, 212, 79], [164, 77, 44], [161, 45, 128], [235, 207, 30], [142, 229, 244], [55, 24, 90], [116, 106, 190], [64, 173, 250], [83, 175, 120], [78, 40, 211], [194, 118, 41], [29, 128, 33], [251, 217, 246], [168, 187, 200], [11, 121, 92], [165, 56, 128], [87, 27, 24], [236, 131, 64], [174, 148, 48], [190, 239, 77], [40, 3, 75], [153, 129, 249], [233, 237, 151], [132, 99, 148], [238, 67, 160], [63, 157, 85], [47, 6, 176], [108, 95, 232], [130, 93, 183], [83, 68, 93], [115, 48, 115], [20, 196, 60], [185, 226, 217], [134, 101, 185], [58, 252, 20], [130, 20, 134], [240, 194, 178], [226, 168, 243], [245, 150, 225], [242, 53, 30], [71, 143, 248], [113, 4, 67], [2, 42, 145], [82, 4, 102], [214, 104, 112], [146, 226, 123], [12, 231, 98], [159, 219, 214], [69, 28, 136], [136, 104, 23], [173, 52, 245], [115, 153, 130], [143, 142, 59], [128, 132, 221], [218, 62, 98], [209, 100, 104], [63, 111, 18], [135, 198, 182], [175, 172, 128], [14, 183, 115], [103, 207, 45], [196, 63, 249], [130, 92, 206], [218, 229, 179], [16, 95, 170], [4, 33, 147], [31, 2, 224], [169, 42, 168], [173, 58, 241], [53, 250, 94], [58, 169, 245], [118, 59, 20], [173, 6, 215], [93, 29, 65], [152, 116, 32], [116, 159, 84], [248, 130, 1], [216, 69, 224], [207, 75, 28], [93, 195, 19], [227, 127, 177], [211, 161, 96], [237, 169, 2], [15, 30, 149], [53, 50, 221], [237, 105, 113], [136, 166, 40], [108, 206, 253], [83, 99, 155], [107, 237, 163], [255, 124, 41], [237, 108, 231], [241, 153, 190], [130, 7, 25], [213, 101, 168], [162, 80, 90], [28, 96, 61], [17, 88, 39], [128, 145, 5], [182, 212, 156], [158, 107, 151], [169, 44, 223], [224, 118, 162], [218, 181, 89], [183, 170, 168], [252, 85, 98], [128, 62, 206], [191, 209, 7], [29, 177, 177], [225, 115, 28], [149, 204, 239], [121, 245, 204], [241, 127, 182], [22, 165, 215], [5, 64, 186], [205, 170, 15], [247, 213, 73], [149, 32, 105], [181, 34, 155], [45, 185, 179], [39, 158, 165], [20, 27, 154], [59, 129, 62], [51, 198, 0], [42, 134, 165], [109, 72, 197], [109, 6, 120], [216, 209, 158], [121, 3, 33], [196, 94, 204], [184, 160, 108], [127, 105, 108], [84, 231, 242], [194, 185, 8], [71, 251, 71], [144, 205, 192], [98, 197, 84], [133, 71, 75], [116, 97, 139], [161, 224, 236], [164, 13, 228], [94, 177, 61], [188, 251, 121], [166, 93, 219], [189, 36, 165], [151, 88, 22], [160, 163, 159], [23, 246, 194], [187, 45, 229], [183, 140, 79], [61, 24, 254], [134, 47, 106], [52, 74, 182], [247, 54, 208], [219, 184, 215], [94, 230, 128], [105, 40, 86], [27, 124, 122], [36, 182, 75], [5, 191, 215], [0, 3, 104], [26, 184, 106], [131, 198, 153], [82, 179, 46], [164, 182, 60], [228, 47, 230], [219, 140, 187], [131, 200, 249], [72, 166, 203], [115, 59, 239], [86, 124, 16], [252, 34, 24], [202, 209, 236], [153, 253, 133], [118, 45, 42], [235, 25, 35], [43, 6, 150], [98, 214, 214], [7, 233, 248], [125, 222, 232], [136, 32, 58], [104, 151, 237], [217, 183, 119], [213, 159, 80], [102, 67, 162], [204, 115, 31], [235, 111, 88], [226, 45, 65], [224, 200, 70], [240, 243, 62], [101, 118, 49], [186, 78, 242], [137, 85, 158], [114, 127, 31], [206, 246, 225], [220, 216, 140], [237, 142, 56], [162, 170, 46], [153, 101, 250], [67, 49, 204], [215, 70, 170], [199, 253, 142], [115, 198, 34], [125, 101, 39], [76, 220, 40], [223, 43, 154], [117, 4, 51], [174, 253, 110], [77, 185, 132], [248, 81, 115], [64, 206, 95], [176, 112, 239], [63, 4, 124], [130, 70, 65], [143, 134, 169], [113, 208, 155], [162, 148, 59], [19, 96, 37], [201, 222, 222], [153, 196, 21], [225, 138, 94], [90, 149, 163], [127, 27, 222], [242, 195, 103], [175, 28, 68], [224, 102, 87], [41, 38, 225], [156, 7, 116], [166, 9, 107], [85, 115, 240], [6, 240, 168], [129, 141, 105], [3, 156, 26], [114, 47, 167], [231, 103, 58], [123, 95, 56], [30, 56, 11], [159, 144, 28], [24, 196, 73], [43, 23, 90], [126, 77, 216], [2, 93, 116], [249, 254, 211], [30, 175, 148], [38, 111, 38], [144, 199, 243], [62, 206, 171], [137, 65, 126], [246, 69, 43], [206, 49, 97], [156, 237, 110], [197, 16, 200], [179, 153, 36], [158, 222, 107], [162, 60, 44], [85, 225, 147], [153, 48, 77], [184, 29, 248], [230, 134, 156], [154, 155, 128], [162, 249, 252], [240, 172, 108], [239, 118, 187], [146, 135, 132], [224, 140, 235], [152, 111, 120], [207, 18, 14], [117, 89, 94], [131, 147, 118], [192, 82, 63], [129, 206, 34], [112, 69, 11], [17, 197, 69], [15, 121, 216], [163, 87, 169], [255, 40, 92], [198, 180, 251], [16, 179, 59], [14, 99, 137], [145, 243, 243], [84, 59, 193], [155, 86, 137], [157, 251, 208], [245, 189, 239], [126, 172, 212], [69, 164, 200], [188, 224, 151], [106, 72, 188], [212, 171, 29], [215, 15, 202], [155, 44, 232], [32, 158, 58], [25, 85, 38], [77, 5, 98], [82, 228, 176], [20, 105, 167], [156, 214, 150], [124, 201, 247], [14, 37, 23], [152, 118, 15], [216, 50, 41], [81, 140, 107], [174, 8, 143], [201, 173, 74], [193, 192, 196], [206, 198, 211], [116, 126, 240], [125, 195, 86], [166, 169, 250], [174, 5, 221], [60, 43, 227], [118, 43, 122], [138, 208, 165], [124, 202, 48], [18, 59, 5], [201, 28, 170], [121, 202, 189], [119, 60, 47], [94, 246, 176], [76, 214, 204], [156, 193, 153], [75, 184, 61], [143, 250, 254], [210, 206, 183], [231, 238, 142], [161, 52, 85], [118, 113, 227], [97, 83, 227], [254, 248, 240], [95, 133, 153], [141, 89, 80], [82, 187, 147], [55, 161, 75], [42, 64, 87], [148, 13, 80], [141, 130, 114], [15, 201, 22], [190, 164, 65], [219, 48, 118], [206, 65, 16], [201, 28, 56], [17, 216, 81], [57, 71, 211], [42, 68, 12], [174, 100, 209], [122, 177, 187], [242, 50, 76], [97, 35, 15], [35, 12, 240], [72, 37, 44], [35, 19, 54], [75, 138, 48], [52, 247, 59], [22, 118, 75], [244, 112, 169], [246, 117, 54], [157, 105, 129], [47, 154, 119], [165, 108, 59], [49, 8, 55], [106, 240, 233], [128, 140, 53], [8, 47, 143], [39, 198, 121], [137, 75, 252], [255, 19, 48], [169, 219, 89], [192, 32, 242], [173, 194, 67], [97, 107, 21], [242, 93, 55], [1, 227, 82], [72, 16, 218], [54, 198, 174], [196, 36, 207], [118, 120, 157], [206, 41, 228], [121, 44, 2], [71, 90, 165], [2, 194, 37], [164, 13, 172], [33, 164, 61], [194, 181, 123], [78, 4, 1], [135, 0, 82], [70, 39, 196], [61, 44, 140], [115, 165, 221], [149, 21, 237], [230, 43, 219], [197, 147, 215], [222, 32, 40], [23, 34, 44], [88, 40, 113], [56, 49, 63], [106, 245, 171], [105, 40, 212], [24, 203, 124], [27, 221, 168], [29, 2, 212], [72, 177, 42], [59, 112, 102], [120, 62, 133], [102, 80, 8], [89, 132, 243], [168, 121, 178], [229, 153, 44], [216, 14, 36], [209, 44, 33], [20, 76, 170], [182, 227, 38], [78, 183, 214], [48, 171, 168], [197, 46, 155], [110, 193, 6], [248, 130, 161], [88, 175, 53], [68, 105, 1], [240, 104, 87], [23, 229, 136], [144, 87, 185], [209, 20, 221], [14, 56, 90], [80, 217, 252], [36, 14, 250], [41, 70, 197], [104, 158, 250], [119, 45, 57], [73, 160, 57], [51, 223, 186], [217, 205, 188], [80, 238, 49], [209, 220, 155], [61, 48, 94], [191, 255, 236], [36, 7, 161], [109, 233, 161], [64, 179, 93], [108, 140, 88], [185, 116, 200], [188, 63, 252], [122, 183, 86], [106, 156, 97], [188, 195, 78], [204, 42, 101], [176, 239, 55], [237, 239, 184], [30, 16, 32], [220, 177, 222], [92, 27, 65], [108, 247, 67], [88, 68, 150], [125, 129, 249], [141, 124, 170], [1, 250, 187], [38, 94, 91], [209, 7, 138], [78, 131, 113], [161, 38, 183], [157, 19, 30], [239, 19, 120], [30, 237, 102], [56, 248, 22], [147, 32, 3], [85, 17, 19], [222, 1, 248], [13, 128, 87], [89, 90, 116], [83, 89, 6], [201, 42, 61], [71, 62, 90], [107, 51, 82], [156, 61, 28], [220, 231, 77], [100, 19, 249], [40, 121, 42], [31, 88, 34], [79, 191, 76], [188, 43, 233], [219, 226, 200], [220, 87, 217], [30, 78, 9], [121, 53, 244], [246, 96, 233], [41, 107, 203], [107, 252, 28], [25, 54, 45], [211, 109, 185], [179, 133, 206], [66, 81, 87], [220, 184, 162], [129, 230, 224], [131, 6, 5], [169, 205, 106], [36, 172, 148], [195, 237, 113], [215, 91, 99], [29, 212, 93], [61, 63, 35], [255, 40, 119], [22, 152, 153], [127, 154, 246], [82, 138, 245], [251, 246, 24], [23, 172, 230], [83, 31, 106], [202, 145, 125], [92, 195, 22], [16, 80, 250], [110, 93, 93], [104, 201, 195], [212, 102, 221], [120, 201, 47], [227, 45, 202], [49, 17, 26], [133, 75, 33], [229, 208, 64], [241, 18, 55], [224, 108, 30], [81, 146, 244], [254, 176, 45], [157, 154, 65], [113, 144, 197], [197, 54, 204], [92, 0, 247], [0, 127, 244], [143, 182, 82], [113, 86, 160], [29, 224, 63], [233, 242, 75], [232, 129, 223], [188, 157, 150], [151, 62, 132], [166, 184, 63], [236, 191, 23], [136, 46, 95], [28, 148, 117], [190, 41, 251], [81, 45, 241], [17, 34, 136], [10, 57, 60], [191, 42, 10], [127, 58, 202], [48, 146, 132], [78, 183, 213], [148, 243, 12], [51, 168, 212], [47, 107, 12], [252, 107, 105], [225, 77, 135], [173, 1, 146], [69, 219, 169], [119, 46, 112], [169, 98, 191], [213, 241, 220], [56, 185, 242], [75, 233, 14], [131, 175, 131], [226, 34, 19], [184, 219, 27], [96, 40, 1], [185, 189, 111], [158, 217, 149], [214, 195, 67], [142, 184, 99], [203, 227, 242], [87, 125, 11], [48, 105, 133], [49, 146, 249], [255, 192, 140], [7, 140, 108], [133, 31, 201], [41, 133, 216], [185, 28, 252], [30, 17, 158], [110, 91, 153], [42, 152, 229], [55, 179, 146], [119, 100, 102], [207, 90, 254], [95, 50, 237], [214, 249, 47], [230, 79, 18], [129, 180, 243], [239, 191, 213], [150, 103, 120], [18, 31, 164], [217, 23, 78], [25, 61, 234], [23, 52, 146], [178, 254, 23], [146, 190, 90], [159, 242, 139], [139, 49, 19], [44, 226, 105], [192, 178, 72], [104, 169, 169], [140, 32, 245], [188, 169, 148], [64, 80, 208], [103, 27, 58], [128, 49, 130], [187, 112, 160], [177, 176, 204], [11, 203, 134], [89, 104, 250], [233, 241, 92], [18, 235, 174], [114, 82, 223], [27, 84, 99], [37, 16, 226], [171, 155, 213], [158, 68, 207], [35, 62, 158], [248, 156, 72], [23, 233, 82], [117, 232, 243], [43, 204, 106], [63, 52, 185], [131, 112, 246], [60, 94, 11], [150, 176, 27], [30, 205, 122], [220, 83, 244], [85, 69, 190], [16, 220, 249], [21, 27, 238], [95, 184, 114], [165, 231, 167], [67, 215, 149], [9, 92, 181], [131, 24, 225], [234, 249, 3], [236, 18, 82], [98, 157, 190], [150, 112, 90], [11, 66, 219], [159, 73, 137], [253, 194, 157], [228, 45, 192], [214, 12, 228], [247, 126, 157], [51, 1, 234], [52, 28, 221], [255, 11, 55], [57, 81, 96], [18, 172, 0], [134, 159, 179], [84, 247, 3], [72, 212, 140], [24, 37, 126], [31, 174, 157], [95, 92, 100], [206, 207, 197], [112, 123, 209], [176, 249, 123], [12, 185, 202], [215, 10, 232], [214, 96, 167], [21, 247, 106], [70, 198, 14], [143, 130, 84], [105, 143, 101], [61, 241, 57], [222, 234, 115], [65, 242, 132], [247, 142, 42], [12, 125, 49], [223, 66, 176], [93, 89, 231], [132, 204, 23], [17, 199, 51], [226, 41, 160], [33, 235, 71], [194, 236, 103], [218, 248, 170], [173, 6, 250], [168, 89, 60], [250, 205, 201], [33, 176, 66], [35, 159, 134], [178, 251, 144], [33, 114, 170], [195, 48, 49], [150, 196, 55], [9, 45, 98], [242, 98, 190], [84, 166, 207], [183, 197, 242], [65, 141, 149], [158, 111, 216], [110, 230, 28], [148, 230, 170], [235, 39, 58], [53, 72, 25], [126, 126, 208], [128, 239, 56], [41, 103, 85], [112, 141, 252], [179, 147, 198], [23, 210, 187], [196, 233, 137], [14, 90, 88], [234, 230, 119], [61, 186, 173], [222, 216, 147], [35, 192, 58], [193, 150, 109], [70, 190, 33], [172, 239, 12], [73, 72, 210], [107, 148, 96], [62, 242, 19], [42, 138, 108], [33, 57, 203], [36, 244, 105], [213, 118, 32], [54, 131, 201], [12, 71, 235], [132, 152, 107], [196, 216, 114], [56, 52, 84], [13, 187, 75], [201, 156, 175], [77, 147, 190], [163, 209, 133], [73, 65, 79], [61, 44, 249], [133, 132, 76], [222, 103, 179], [24, 79, 144], [94, 39, 79], [151, 105, 224], [169, 253, 217], [193, 162, 252], [225, 18, 163], [235, 253, 250], [2, 8, 173], [189, 222, 208], [163, 211, 114], [52, 55, 236], [131, 27, 130], [58, 62, 107], [118, 255, 227], [125, 40, 220], [66, 43, 150], [183, 20, 115], [106, 172, 207], [163, 182, 1], [39, 175, 248], [162, 249, 89], [18, 2, 184], [195, 84, 166], [195, 32, 217], [143, 253, 42], [44, 210, 2], [151, 102, 62], [124, 13, 145], [57, 170, 116], [156, 245, 84], [221, 47, 226], [206, 122, 94], [181, 78, 105], [137, 169, 194], [52, 21, 2], [50, 202, 206], [140, 92, 200], [86, 122, 67], [14, 168, 207], [165, 241, 122], [118, 51, 197], [95, 195, 64], [223, 147, 236], [154, 210, 73], [15, 71, 243], [107, 240, 153], [26, 70, 241], [71, 102, 140], [206, 80, 98], [87, 150, 232], [145, 12, 148], [250, 14, 221], [73, 70, 151], [66, 40, 38], [71, 148, 108], [117, 250, 148], [179, 192, 230], [183, 216, 1], [62, 62, 86], [178, 229, 42], [159, 207, 250], [38, 126, 74], [71, 27, 196], [244, 55, 158], [107, 150, 187], [237, 10, 52], [6, 246, 103], [132, 88, 201], [189, 13, 248], [130, 60, 55], [138, 98, 249], [158, 147, 15], [85, 222, 192], [138, 191, 105], [178, 113, 141], [235, 225, 224], [66, 164, 47], [181, 83, 25], [205, 236, 240], [160, 30, 40], [72, 255, 156], [151, 184, 51], [225, 123, 171], [126, 206, 254], [102, 160, 44], [190, 146, 204], [105, 157, 172], [88, 157, 91], [237, 2, 169], [181, 143, 56], [4, 129, 209], [220, 76, 21], [222, 191, 22], [170, 0, 187], [136, 162, 190], [251, 130, 121], [225, 235, 37], [16, 21, 67], [234, 254, 33], [6, 134, 35], [51, 183, 101], [31, 217, 124], [119, 19, 68], [125, 120, 213], [194, 231, 220], [53, 173, 89], [33, 27, 114], [222, 26, 238], [171, 131, 39], [210, 251, 228], [79, 17, 60], [71, 102, 6], [54, 61, 122], [79, 214, 226], [237, 223, 45], [63, 35, 205], [161, 210, 129], [19, 38, 247], [191, 105, 193], [143, 192, 97], [67, 138, 70], [157, 212, 152], [27, 172, 4], [50, 72, 116], [115, 130, 129], [21, 22, 46], [23, 250, 141], [65, 180, 93], [192, 134, 136], [3, 88, 244], [140, 180, 46], [49, 228, 61], [58, 87, 97], [55, 172, 235], [123, 20, 103], [207, 209, 221], [181, 225, 7], [76, 129, 2], [164, 146, 209], [125, 105, 198], [168, 230, 152], [46, 72, 204], [12, 203, 78], [105, 97, 56], [157, 236, 237], [20, 5, 16], [156, 127, 250], [91, 231, 149], [255, 204, 113], [129, 184, 121], [180, 62, 165], [251, 71, 52], [180, 236, 207], [229, 170, 46], [53, 20, 117], [66, 160, 245], [131, 38, 99], [12, 162, 11], [251, 101, 187], [95, 213, 101], [243, 65, 242], [83, 177, 251], [45, 120, 209], [83, 55, 239], [173, 66, 153], [61, 61, 64], [101, 69, 157], [25, 95, 132], [165, 247, 8], [90, 197, 11], [34, 4, 225], [124, 132, 54], [77, 176, 139], [68, 104, 152], [145, 51, 45], [47, 78, 43], [33, 252, 5], [175, 102, 173], [28, 116, 114], [243, 225, 96], [98, 20, 232], [17, 240, 215], [155, 236, 78], [92, 96, 29], [74, 32, 237], [133, 201, 177], [214, 167, 55], [211, 213, 117], [102, 132, 251], [189, 91, 65], [206, 253, 169], [41, 247, 82], [209, 195, 175], [71, 53, 65], [81, 153, 7], [48, 83, 147], [178, 186, 116], [62, 112, 64], [14, 137, 132], [234, 179, 241], [84, 222, 64], [16, 54, 227], [159, 149, 111], [19, 18, 255], [171, 134, 26], [65, 211, 72], [232, 72, 185], [54, 234, 52], [79, 144, 6], [99, 27, 143], [182, 203, 90], [251, 87, 97], [84, 29, 139], [65, 35, 80], [78, 166, 126], [39, 15, 93], [189, 59, 155], [107, 125, 193], [51, 73, 221], [24, 37, 253], [56, 112, 88], [5, 151, 170], [162, 104, 71], [222, 156, 144], [7, 30, 107], [91, 1, 175], [135, 225, 237], [159, 66, 182], [123, 145, 141], [117, 160, 112], [251, 15, 122], [96, 148, 34], [94, 201, 14], [126, 242, 55], [67, 4, 112], [189, 154, 230], [53, 174, 45], [188, 206, 52], [202, 247, 25], [144, 175, 116], [186, 95, 141], [57, 145, 51], [241, 49, 16], [167, 254, 108], [157, 202, 89], [175, 84, 230], [169, 84, 156], [188, 64, 129], [126, 179, 221], [29, 48, 32], [154, 228, 111], [71, 89, 254], [172, 80, 173], [99, 192, 93], [69, 198, 223], [244, 90, 52], [98, 122, 123], [211, 151, 180], [238, 174, 81], [131, 90, 66], [91, 129, 210], [183, 72, 49], [146, 144, 64], [157, 11, 248], [115, 148, 83], [254, 58, 0], [64, 14, 8], [143, 204, 243], [79, 182, 41], [215, 151, 87], [80, 187, 37], [222, 145, 202], [109, 9, 235], [42, 185, 245], [100, 194, 103], [91, 91, 44], [130, 250, 73], [248, 60, 11], [250, 177, 35], [171, 79, 127], [154, 189, 113], [38, 214, 140], [94, 25, 224], [251, 210, 185], [120, 131, 128], [17, 85, 239], [79, 190, 215], [173, 179, 239], [245, 6, 5], [171, 54, 161], [165, 213, 176], [148, 95, 43], [207, 3, 109], [69, 19, 41], [28, 95, 241], [59, 73, 59], [32, 96, 58], [91, 68, 191], [41, 111, 210], [232, 56, 80], [188, 0, 225], [239, 87, 27], [43, 97, 40], [208, 72, 149], [8, 157, 20], [48, 214, 112], [151, 105, 165], [17, 23, 124], [151, 250, 234], [84, 86, 217], [132, 178, 161], [240, 130, 81], [103, 211, 1], [159, 254, 136], [237, 25, 123], [68, 154, 106], [41, 134, 155], [218, 57, 2], [9, 193, 15], [181, 38, 213], [189, 209, 248], [199, 110, 192], [98, 212, 66], [9, 127, 249], [43, 60, 190], [139, 89, 106], [242, 74, 117], [150, 118, 114], [208, 213, 110], [121, 225, 51], [235, 202, 165], [210, 79, 217], [103, 205, 19], [85, 65, 95], [90, 174, 253], [3, 219, 100], [224, 2, 1], [164, 39, 145], [41, 27, 142], [2, 3, 199], [142, 122, 211], [185, 116, 111], [43, 168, 195], [165, 94, 240], [152, 124, 243], [5, 163, 215], [13, 250, 161], [138, 143, 243], [146, 104, 245], [177, 206, 172], [15, 70, 127], [1, 175, 120], [162, 106, 9], [118, 124, 160], [95, 27, 224], [189, 133, 143], [133, 194, 80], [87, 59, 49], [204, 113, 180], [21, 148, 154], [82, 182, 68], [21, 100, 118], [47, 190, 181], [211, 80, 52], [14, 94, 222], [46, 166, 51], [128, 78, 92], [217, 39, 110], [14, 184, 191], [34, 13, 223], [93, 88, 112], [42, 130, 89], [223, 40, 153], [29, 180, 127], [172, 160, 98], [169, 125, 215], [146, 107, 138], [195, 114, 19], [42, 206, 68], [62, 145, 204], [93, 196, 99], [63, 58, 101], [69, 96, 132], [40, 78, 50], [209, 121, 208], [56, 212, 195], [126, 42, 117], [12, 198, 202], [98, 237, 160], [158, 139, 166], [152, 109, 200], [78, 141, 103], [32, 58, 49], [173, 24, 61], [50, 240, 93], [194, 43, 22], [245, 108, 13], [81, 220, 128], [44, 224, 87], [121, 113, 158], [254, 231, 254], [105, 33, 183], [88, 150, 50], [106, 191, 229], [94, 154, 207], [88, 85, 246], [101, 207, 113], [86, 150, 120], [227, 25, 21], [210, 124, 229], [29, 21, 196], [71, 39, 92], [252, 69, 94], [180, 20, 51], [185, 217, 45], [215, 129, 0], [90, 164, 219], [12, 225, 150], [1, 120, 190], [198, 165, 85], [194, 190, 86], [64, 163, 114], [217, 241, 236], [246, 75, 229], [204, 185, 106], [99, 11, 60], [206, 122, 230], [246, 97, 225], [133, 76, 196], [235, 26, 168], [71, 0, 238], [230, 215, 84], [234, 34, 19], [93, 158, 180], [2, 100, 231], [18, 92, 207], [35, 126, 69], [207, 12, 132], [178, 129, 120], [119, 151, 157], [182, 44, 164], [16, 150, 27], [150, 231, 195], [229, 247, 153], [95, 185, 128], [18, 160, 206], [114, 4, 18], [5, 77, 36], [246, 54, 24], [240, 221, 252], [207, 191, 49], [78, 237, 30], [86, 50, 176], [135, 203, 78], [26, 115, 248], [55, 31, 204], [92, 224, 127], [50, 53, 174], [38, 70, 243], [114, 154, 2], [158, 2, 161], [120, 91, 208], [134, 171, 58], [80, 58, 204], [60, 154, 152], [254, 141, 129], [245, 239, 59], [8, 181, 43], [193, 104, 126], [155, 207, 166], [167, 87, 18], [137, 168, 15], [20, 91, 58], [177, 208, 161], [45, 33, 78], [32, 20, 123], [173, 75, 224], [0, 47, 109], [118, 248, 144], [84, 236, 186], [200, 243, 230], [41, 69, 93], [26, 52, 28], [130, 13, 191], [238, 76, 51], [115, 59, 167], [241, 136, 151], [212, 249, 77], [79, 125, 53], [0, 107, 195], [164, 182, 67], [162, 169, 48], [34, 125, 206], [200, 34, 144], [240, 106, 2], [103, 64, 197], [8, 56, 39], [184, 219, 171], [213, 234, 70], [157, 189, 58], [115, 93, 232], [87, 81, 183], [73, 21, 82], [41, 128, 7], [37, 178, 187], [208, 56, 6], [101, 53, 246], [178, 7, 73], [100, 235, 26], [113, 76, 103], [139, 167, 225], [236, 27, 178], [97, 36, 78], [80, 112, 16], [218, 110, 42], [67, 25, 234], [199, 237, 57], [242, 18, 112], [31, 151, 173], [42, 193, 61], [76, 147, 65], [172, 57, 214], [58, 14, 122], [179, 23, 220], [234, 234, 12], [131, 140, 55], [121, 149, 123], [158, 98, 6], [17, 179, 136], [90, 65, 159], [19, 147, 22], [153, 21, 132], [104, 209, 169], [197, 54, 9], [22, 55, 51], [90, 150, 161], [86, 24, 254], [150, 216, 210], [48, 81, 31], [159, 117, 77], [213, 210, 225], [185, 51, 15], [228, 26, 162], [154, 232, 20], [255, 137, 144], [107, 228, 8], [191, 210, 81], [153, 246, 131], [103, 90, 97], [71, 174, 175], [46, 235, 33], [161, 201, 209], [175, 136, 230], [75, 41, 175], [183, 144, 181], [0, 61, 163], [180, 65, 236], [150, 212, 209], [125, 229, 39], [165, 68, 76], [164, 231, 121], [30, 123, 212], [89, 59, 203], [66, 187, 245], [53, 63, 60], [242, 195, 137], [2, 178, 16], [85, 75, 197], [215, 199, 75], [9, 249, 197], [9, 161, 206], [226, 90, 51], [130, 90, 159], [33, 211, 67], [226, 142, 73], [189, 153, 98], [73, 66, 0], [216, 41, 113], [33, 83, 94], [84, 108, 69], [204, 253, 108], [185, 51, 105], [173, 232, 63], [224, 15, 155], [30, 99, 204], [157, 46, 105], [235, 223, 60], [39, 150, 0], [211, 198, 157], [204, 50, 209], [243, 87, 175], [162, 128, 242], [147, 143, 107], [60, 69, 178], [32, 71, 102], [108, 30, 29], [204, 244, 9], [225, 164, 208], [138, 101, 249], [46, 93, 124], [2, 102, 2], [77, 232, 190], [193, 94, 142], [242, 85, 190], [147, 214, 42], [240, 186, 66], [47, 87, 49], [77, 89, 154], [218, 228, 114], [217, 255, 33], [110, 165, 232], [69, 156, 158], [108, 154, 206], [171, 195, 187], [89, 181, 28], [40, 191, 121], [8, 108, 81], [68, 120, 236], [65, 174, 70], [25, 248, 199], [23, 66, 53], [62, 93, 93], [177, 240, 118], [214, 190, 175], [209, 102, 203], [195, 183, 211], [155, 21, 234], [242, 96, 26], [76, 113, 83], [3, 38, 55], [202, 76, 40], [117, 20, 198], [42, 204, 211], [120, 26, 222], [18, 166, 232], [154, 209, 183], [18, 155, 142], [20, 68, 114], [17, 51, 17], [223, 93, 58], [90, 127, 161], [189, 148, 230], [42, 0, 27], [116, 136, 56], [15, 165, 160], [164, 187, 3], [223, 33, 205], [176, 255, 146], [155, 39, 198], [211, 229, 226], [158, 202, 179], [207, 148, 66], [28, 25, 213], [179, 100, 53], [32, 71, 166], [194, 69, 193], [144, 5, 207], [170, 172, 242], [206, 52, 16], [231, 202, 7], [77, 109, 32], [166, 252, 134], [37, 68, 81], [173, 122, 51], [220, 51, 112], [17, 162, 2], [40, 119, 84], [177, 106, 117], [26, 190, 150], [121, 70, 128], [165, 255, 152], [83, 68, 206], [32, 88, 58], [253, 175, 230], [99, 223, 143], [31, 83, 137], [237, 73, 22], [248, 22, 128], [126, 92, 2], [238, 90, 42], [219, 175, 132], [155, 11, 10], [90, 160, 221], [83, 27, 102], [1, 54, 105], [116, 102, 65], [148, 65, 91], [55, 195, 10], [83, 128, 129], [112, 214, 164], [206, 253, 189], [253, 36, 176], [105, 223, 245], [200, 130, 84], [70, 37, 83], [82, 240, 169], [65, 77, 79], [108, 103, 45], [70, 65, 204], [239, 140, 2], [64, 4, 29], [146, 128, 6], [163, 0, 96], [114, 40, 136], [100, 105, 4], [166, 58, 90], [25, 255, 175], [13, 108, 251], [93, 104, 62], [104, 185, 129], [144, 234, 236], [205, 239, 175], [171, 151, 182], [70, 253, 39], [102, 220, 29], [46, 250, 153], [178, 155, 34], [189, 152, 244], [61, 175, 194], [94, 201, 228], [187, 247, 81], [175, 133, 7], [168, 145, 37], [193, 242, 10], [182, 4, 206], [79, 235, 111], [73, 114, 171], [221, 85, 118], [92, 45, 175], [79, 45, 220], [63, 42, 147], [7, 61, 196], [142, 224, 235], [87, 221, 242], [142, 105, 218], [84, 217, 199], [156, 167, 231], [107, 155, 23], [232, 247, 83], [162, 115, 116], [96, 14, 180], [69, 121, 40], [233, 94, 219], [254, 205, 202], [233, 175, 58], [71, 193, 85], [55, 1, 17], [67, 1, 28], [114, 205, 205], [120, 216, 150], [16, 251, 33], [163, 38, 196], [228, 100, 252], [93, 161, 45], [106, 43, 121], [152, 163, 142], [46, 115, 63], [217, 181, 172], [232, 212, 66], [244, 22, 27], [252, 116, 189], [131, 184, 64], [27, 80, 41], [124, 167, 147], [181, 229, 138], [106, 150, 99], [73, 17, 73], [246, 80, 35], [83, 70, 178], [6, 250, 50], [46, 21, 237], [47, 150, 49], [184, 145, 78], [50, 233, 225], [78, 174, 174], [21, 124, 186], [92, 156, 13], [169, 200, 173], [168, 223, 177], [244, 45, 55], [108, 227, 30], [104, 187, 121], [147, 249, 10], [128, 75, 138], [79, 27, 67], [59, 66, 212], [224, 44, 122], [229, 177, 104], [60, 192, 250], [46, 203, 224], [242, 177, 224], [156, 146, 60], [78, 249, 201], [30, 214, 138], [121, 125, 159], [86, 7, 83], [247, 177, 155], [87, 223, 103], [243, 159, 174], [245, 118, 193], [211, 12, 78], [112, 120, 209], [175, 56, 47], [179, 10, 189], [63, 178, 113], [151, 253, 222], [243, 64, 87], [223, 3, 156], [221, 151, 11], [153, 238, 157], [219, 198, 156], [249, 178, 172], [179, 252, 93], [183, 62, 77], [117, 0, 206], [222, 224, 195], [45, 103, 82], [30, 106, 51], [184, 229, 188], [64, 59, 32], [254, 56, 56], [138, 91, 198], [137, 119, 191], [93, 91, 3], [207, 219, 35], [223, 122, 88], [227, 69, 175], [101, 168, 27], [92, 255, 182], [243, 73, 126], [155, 224, 12], [115, 215, 77], [82, 198, 42], [208, 21, 8], [9, 113, 25], [4, 118, 27], [98, 220, 29], [184, 103, 68], [3, 82, 41], [22, 113, 77], [146, 224, 42], [85, 71, 180], [125, 112, 180], [82, 86, 13], [37, 239, 10], [197, 59, 181], [208, 154, 101], [119, 233, 151], [203, 177, 216], [29, 85, 39], [221, 43, 210], [74, 104, 127], [155, 126, 59], [164, 113, 242], [48, 50, 219], [37, 236, 153], [215, 255, 139], [202, 195, 216], [146, 67, 41], [200, 79, 12], [168, 251, 240], [182, 90, 208], [111, 45, 22], [212, 106, 41], [36, 106, 11], [106, 83, 104], [160, 42, 75], [244, 84, 163], [174, 63, 144], [152, 196, 116], [22, 106, 52], [115, 201, 87], [243, 25, 119], [106, 238, 155], [176, 40, 43], [51, 175, 97], [126, 72, 226], [78, 125, 76], [234, 83, 215], [34, 22, 224], [229, 158, 213], [37, 2, 227], [244, 92, 50], [159, 241, 129], [38, 159, 197], [8, 216, 248], [127, 166, 55], [246, 155, 53], [163, 124, 212], [187, 172, 163], [116, 217, 219], [241, 178, 9], [235, 228, 211], [141, 41, 172], [238, 74, 239], [235, 181, 221], [146, 53, 145], [249, 181, 71], [96, 67, 164], [178, 109, 163], [32, 129, 147], [217, 135, 198], [149, 196, 138], [224, 27, 101], [7, 191, 224], [220, 175, 151], [7, 61, 53], [208, 92, 84], [126, 142, 85], [100, 82, 111], [179, 228, 39], [50, 57, 64], [45, 154, 237], [111, 69, 11], [29, 11, 109], [15, 103, 72], [221, 127, 25], [32, 160, 16], [203, 60, 103], [93, 143, 39], [96, 212, 241], [61, 16, 222], [34, 188, 190], [250, 47, 6], [171, 41, 218], [231, 132, 210], [37, 176, 171], [27, 76, 232], [21, 243, 104], [217, 227, 120], [48, 56, 145], [194, 219, 22], [190, 45, 70], [97, 240, 161], [244, 216, 154], [124, 80, 171], [95, 183, 162], [239, 83, 87], [164, 36, 190], [102, 173, 236], [152, 150, 95], [222, 20, 123], [8, 18, 50], [209, 60, 48], [34, 101, 51], [127, 225, 204], [199, 4, 250], [218, 177, 105], [189, 235, 182], [10, 87, 68], [23, 201, 150], [148, 147, 208], [61, 165, 80], [44, 217, 225], [218, 58, 158], [184, 75, 29], [229, 182, 59], [185, 174, 55], [180, 82, 98], [247, 211, 210], [35, 187, 133], [170, 241, 252], [71, 154, 174], [20, 9, 163], [217, 158, 130], [206, 231, 116], [30, 156, 104], [204, 115, 47], [198, 194, 26], [164, 235, 200], [54, 17, 245], [31, 208, 255], [12, 3, 225], [29, 108, 35], [221, 193, 190], [254, 134, 181], [158, 194, 95], [48, 239, 224], [253, 188, 12], [184, 73, 167], [73, 27, 200], [189, 3, 182], [121, 100, 34], [56, 158, 198], [92, 76, 27], [156, 40, 16], [67, 38, 133], [135, 17, 65], [248, 223, 230], [101, 22, 255], [189, 122, 137], [62, 247, 12], [18, 235, 219], [220, 157, 27], [69, 157, 25], [215, 5, 208], [232, 61, 65], [121, 26, 66], [108, 6, 27], [90, 14, 238], [75, 113, 53], [198, 109, 92], [85, 76, 85], [64, 83, 158], [5, 153, 99], [33, 183, 94], [217, 120, 21], [88, 38, 233], [144, 189, 241], [59, 144, 181], [28, 87, 114], [93, 154, 44], [68, 253, 186], [121, 172, 37], [118, 55, 154], [60, 63, 104], [162, 245, 11], [188, 71, 246], [107, 190, 173], [129, 15, 7], [45, 142, 50], [230, 140, 247], [253, 117, 174], [245, 118, 53], [58, 132, 56], [230, 37, 56], [137, 244, 39], [231, 31, 82], [81, 101, 93], [4, 255, 28], [134, 55, 92], [98, 164, 204], [109, 197, 245], [152, 97, 22], [215, 233, 211], [70, 235, 168], [82, 245, 151], [209, 218, 170], [242, 51, 119], [64, 104, 98], [138, 158, 233], [157, 162, 234], [230, 11, 174], [23, 45, 36], [216, 46, 81], [142, 100, 140], [58, 140, 143], [129, 217, 62], [38, 46, 33], [102, 102, 164], [60, 160, 25], [123, 147, 56], [74, 78, 189], [90, 247, 8], [114, 207, 183], [60, 167, 97], [210, 38, 217], [174, 201, 175], [59, 106, 246], [180, 176, 5], [52, 191, 120], [69, 241, 91], [203, 48, 222], [62, 144, 217], [110, 208, 137], [114, 160, 242], [213, 114, 176], [99, 80, 170], [206, 129, 230], [239, 212, 184], [0, 79, 29], [107, 159, 175], [215, 90, 28], [142, 168, 0], [191, 59, 30], [28, 122, 93], [201, 21, 88], [21, 97, 155], [189, 167, 15], [54, 183, 133], [255, 114, 8], [252, 29, 244], [12, 151, 49], [58, 171, 115], [191, 133, 99], [155, 132, 30], [153, 229, 25], [1, 212, 41], [239, 82, 103], [22, 108, 100], [129, 163, 196], [36, 118, 2], [176, 76, 59], [149, 77, 172], [250, 56, 177], [199, 188, 207], [157, 21, 120], [178, 236, 236], [22, 151, 97], [150, 79, 234], [179, 249, 77], [8, 96, 35], [98, 67, 37], [4, 144, 78], [71, 12, 107], [27, 65, 132], [205, 146, 91], [204, 172, 15], [42, 160, 228], [215, 6, 49], [179, 34, 22], [211, 67, 163], [217, 222, 13], [105, 127, 157], [1, 227, 12], [225, 0, 71], [218, 227, 110], [39, 22, 25], [187, 8, 131], [116, 151, 213], [41, 99, 252], [212, 121, 232], [23, 206, 234], [167, 150, 160], [67, 199, 49], [31, 222, 202], [20, 85, 91], [117, 38, 61], [91, 65, 2], [128, 77, 126], [157, 148, 150], [12, 197, 32], [47, 210, 177], [15, 15, 42], [25, 71, 64], [99, 78, 115], [101, 90, 74], [242, 239, 152], [251, 183, 77], [14, 186, 41], [4, 56, 21], [68, 15, 68], [226, 239, 226], [19, 5, 116], [209, 166, 12], [120, 26, 122], [200, 51, 177], [127, 212, 230], [181, 226, 149], [250, 197, 218], [205, 70, 89], [221, 30, 12], [137, 222, 118], [70, 148, 197], [185, 159, 98], [202, 41, 74], [79, 84, 194], [122, 37, 70], [20, 218, 190], [171, 87, 133], [210, 64, 41], [142, 94, 149], [68, 177, 35], [36, 131, 225], [175, 102, 238], [103, 205, 231], [55, 24, 116], [159, 186, 14], [182, 62, 229], [172, 30, 62], [30, 202, 111], [119, 7, 174], [1, 207, 208], [244, 32, 21], [114, 218, 25], [129, 141, 94], [209, 154, 157], [113, 218, 100], [58, 163, 187], [177, 245, 93], [106, 28, 45], [134, 161, 69], [79, 36, 103], [134, 184, 93], [248, 150, 234], [64, 118, 78], [250, 1, 137], [207, 82, 100], [214, 129, 134], [83, 25, 130], [232, 216, 188], [181, 113, 27], [47, 17, 214], [112, 16, 179], [219, 34, 66], [166, 248, 219], [171, 198, 193], [56, 35, 15], [35, 5, 87], [62, 135, 38], [65, 80, 199], [128, 111, 147], [8, 218, 207], [96, 81, 253], [92, 231, 108], [161, 227, 196], [223, 229, 100], [78, 202, 52], [135, 174, 175], [50, 132, 3], [177, 22, 97], [232, 49, 145], [35, 149, 67], [154, 85, 4], [33, 21, 106], [164, 170, 91], [78, 185, 40], [37, 12, 154], [250, 144, 239], [249, 48, 35], [97, 209, 152], [221, 174, 44], [216, 176, 201], [125, 156, 176], [130, 176, 255], [163, 59, 68], [45, 207, 250], [45, 134, 163], [168, 204, 115], [56, 80, 62], [75, 90, 138], [9, 239, 142], [57, 218, 150], [111, 104, 83], [208, 32, 77], [250, 197, 255], [23, 105, 38], [116, 185, 92], [46, 141, 210], [26, 246, 92], [114, 16, 243], [186, 223, 93], [210, 182, 151], [188, 149, 70], [130, 135, 39], [205, 41, 186], [115, 219, 163], [198, 73, 122], [144, 17, 220], [188, 227, 48], [93, 5, 197], [84, 163, 55], [235, 166, 238], [81, 247, 91], [18, 226, 167], [68, 10, 80], [87, 100, 175], [11, 176, 97], [46, 254, 62], [211, 159, 168], [26, 77, 181], [124, 136, 138], [17, 28, 131], [63, 101, 248], [78, 3, 244], [229, 42, 81], [7, 171, 51], [217, 216, 222], [44, 228, 66], [231, 17, 154], [187, 36, 214], [121, 218, 240], [4, 181, 193], [73, 165, 188], [106, 145, 40], [189, 178, 124], [164, 104, 34], [55, 21, 164], [233, 102, 159], [188, 120, 143], [253, 99, 243], [7, 210, 211], [29, 214, 205], [96, 165, 89], [234, 34, 23], [27, 18, 175], [103, 212, 248], [255, 3, 196], [228, 101, 144], [176, 17, 253], [122, 62, 183], [216, 84, 95], [214, 187, 202], [214, 100, 151], [174, 66, 19], [81, 165, 165], [144, 51, 129], [82, 24, 31], [9, 168, 147], [46, 193, 40], [127, 45, 249], [129, 2, 33], [104, 186, 54], [27, 212, 91], [190, 236, 82], [69, 120, 171], [119, 33, 181], [169, 105, 124], [219, 159, 169], [220, 128, 155], [84, 245, 159], [34, 152, 196], [230, 85, 36], [59, 64, 171], [78, 226, 180], [247, 55, 190], [65, 153, 209], [146, 123, 158], [116, 199, 35], [156, 227, 89], [159, 50, 247], [224, 153, 69], [214, 205, 136], [93, 65, 242], [47, 166, 164], [103, 169, 176], [200, 93, 234], [139, 237, 196], [224, 228, 97], [144, 63, 182], [34, 204, 199], [75, 227, 165], [213, 42, 115], [205, 46, 197], [138, 184, 7], [94, 150, 90], [201, 13, 69], [184, 172, 28], [218, 104, 186], [41, 28, 70], [105, 135, 254], [211, 2, 60], [165, 246, 115], [65, 108, 213], [89, 26, 213], [154, 21, 8], [51, 126, 137], [101, 118, 202], [17, 102, 215], [26, 46, 3], [249, 119, 181], [109, 124, 60], [8, 86, 251], [19, 112, 10], [247, 57, 208], [119, 110, 175], [97, 15, 60], [164, 39, 248], [38, 98, 238], [67, 212, 192], [55, 39, 100], [229, 119, 121], [246, 159, 250], [235, 213, 119], [13, 148, 18], [38, 251, 209], [173, 27, 234], [166, 95, 7], [37, 4, 77], [107, 34, 45], [199, 89, 91], [35, 27, 132], [13, 228, 140], [33, 208, 34], [110, 66, 220], [125, 39, 214], [14, 47, 51], [181, 20, 181], [207, 60, 223], [150, 115, 102], [24, 131, 37], [235, 29, 202], [150, 188, 129], [137, 173, 60], [244, 58, 208], [150, 198, 149], [179, 255, 32], [152, 163, 33], [19, 82, 89], [214, 213, 173], [154, 131, 138], [57, 211, 3], [96, 49, 169], [158, 250, 124], [156, 211, 176], [69, 3, 16], [15, 223, 242], [123, 181, 91], [183, 107, 48], [115, 29, 77], [161, 199, 93], [251, 52, 206], [219, 81, 13], [77, 216, 202], [159, 111, 128], [61, 143, 111], [132, 33, 83], [49, 65, 52], [17, 82, 197], [175, 251, 14], [116, 181, 243], [26, 47, 166], [65, 59, 75], [90, 68, 78], [132, 98, 246], [244, 129, 4], [222, 214, 220], [156, 29, 103], [45, 146, 115], [29, 98, 128], [226, 171, 40], [27, 192, 135], [203, 254, 71], [12, 224, 210], [52, 98, 221], [34, 220, 190], [71, 191, 252], [187, 199, 63], [214, 157, 55], [194, 248, 120], [181, 162, 89], [62, 101, 125], [16, 18, 107], [31, 234, 190], [65, 9, 97], [177, 123, 79], [25, 154, 136], [169, 237, 100], [126, 89, 120], [37, 181, 19], [23, 85, 127], [182, 60, 92], [83, 38, 252], [188, 167, 157], [172, 143, 171], [252, 250, 16], [48, 157, 200], [13, 232, 129], [223, 110, 231], [235, 68, 121], [67, 244, 16], [24, 92, 180], [187, 97, 144], [206, 222, 145], [67, 108, 91], [180, 144, 218], [253, 127, 112], [62, 183, 43], [44, 235, 4], [246, 209, 237], [254, 240, 70], [36, 69, 46], [157, 129, 212], [229, 186, 79], [145, 159, 182], [131, 82, 69], [37, 48, 209], [249, 138, 25], [247, 18, 136], [145, 93, 117], [134, 128, 212], [40, 152, 16], [72, 121, 175], [96, 51, 254], [24, 85, 136], [90, 54, 216], [64, 72, 227], [230, 142, 213], [159, 152, 42], [190, 93, 151], [43, 33, 81], [37, 126, 172], [190, 59, 127], [102, 3, 38], [9, 80, 65], [44, 110, 212], [125, 133, 89], [126, 182, 175], [160, 18, 113], [237, 118, 144], [35, 39, 110], [18, 139, 55], [89, 30, 187], [46, 121, 193], [4, 92, 49], [151, 204, 142], [10, 201, 188], [17, 18, 91], [42, 108, 117], [151, 191, 196], [69, 21, 145], [252, 235, 198], [111, 204, 172], [6, 0, 132], [228, 107, 15], [55, 32, 82], [53, 125, 241], [159, 143, 116], [117, 120, 252], [244, 215, 19], [156, 91, 5], [178, 151, 119], [207, 236, 168], [126, 132, 109], [53, 209, 57], [131, 95, 70], [6, 162, 180], [91, 251, 115], [245, 1, 134], [217, 54, 1], [243, 24, 210], [91, 81, 248], [31, 93, 8], [135, 11, 48], [125, 171, 79], [137, 47, 176], [207, 193, 72], [5, 87, 59], [144, 53, 51], [4, 156, 27], [96, 142, 162], [173, 34, 254], [137, 243, 37], [117, 137, 209], [49, 251, 245], [33, 142, 170], [197, 57, 64], [139, 163, 32], [201, 53, 155], [53, 147, 85], [108, 116, 115], [120, 7, 180], [216, 249, 219], [217, 41, 107], [73, 67, 220], [217, 66, 76], [71, 246, 10], [84, 45, 206], [153, 76, 201], [13, 243, 10], [109, 158, 249], [57, 89, 96], [24, 49, 153], [230, 63, 193], [156, 191, 251], [188, 199, 183], [46, 89, 254], [28, 191, 186], [197, 15, 52], [113, 153, 167], [0, 54, 225], [213, 179, 22], [195, 147, 63], [32, 70, 52], [78, 59, 24], [245, 83, 244], [112, 219, 130], [79, 204, 99], [86, 49, 102], [31, 141, 27], [185, 23, 248], [29, 16, 148], [159, 41, 145], [212, 127, 128], [130, 85, 13], [248, 179, 237], [179, 72, 249], [157, 89, 16], [47, 27, 117], [195, 146, 249], [159, 80, 213], [126, 186, 70], [208, 144, 240], [174, 239, 32], [95, 222, 83], [182, 220, 54], [7, 164, 19], [99, 220, 160], [165, 247, 27], [10, 53, 83], [159, 250, 119], [106, 114, 239], [17, 129, 221], [144, 95, 98], [190, 210, 112], [174, 114, 152], [180, 228, 152], [6, 62, 46], [52, 106, 81], [244, 245, 164], [96, 203, 121], [171, 40, 144], [176, 189, 28], [10, 178, 138], [34, 97, 32], [170, 82, 88], [227, 198, 250], [194, 138, 5], [99, 6, 91], [43, 201, 200], [122, 73, 170], [176, 29, 127], [148, 143, 102], [88, 103, 44], [203, 66, 160], [188, 156, 30], [41, 137, 255], [243, 190, 237], [247, 216, 0], [52, 158, 12], [139, 184, 1], [64, 131, 236], [222, 184, 236], [181, 75, 92], [186, 190, 246], [130, 159, 223], [72, 115, 62], [209, 12, 34], [79, 172, 240], [67, 189, 155], [221, 53, 20], [18, 7, 140], [33, 113, 25], [178, 163, 149], [208, 171, 194], [203, 96, 70], [130, 104, 92], [13, 112, 136], [176, 78, 129], [111, 36, 38], [22, 17, 126], [213, 142, 117], [153, 103, 6], [204, 147, 193], [227, 39, 12], [159, 239, 175], [110, 209, 154], [71, 147, 34], [157, 230, 55], [234, 219, 223], [13, 1, 82], [42, 102, 242], [61, 116, 95], [244, 3, 142], [106, 247, 228], [179, 80, 28], [232, 80, 105], [102, 86, 128], [181, 77, 179], [78, 30, 93], [119, 173, 36], [73, 157, 120], [159, 16, 163], [23, 90, 4], [112, 17, 199], [33, 73, 18], [144, 71, 76], [28, 182, 240], [85, 167, 64], [160, 193, 193], [59, 97, 13], [67, 62, 7], [65, 211, 216], [243, 32, 207], [95, 69, 126], [86, 201, 112], [67, 243, 9], [109, 90, 198], [65, 49, 155], [59, 38, 86], [78, 31, 26], [126, 158, 140], [32, 7, 146], [179, 202, 225], [98, 209, 149], [82, 23, 16], [152, 233, 116], [85, 177, 29], [0, 129, 228], [9, 86, 135], [227, 94, 154], [197, 44, 191], [63, 184, 226], [243, 39, 108], [185, 152, 133], [89, 254, 151], [104, 189, 238], [220, 251, 219], [51, 32, 110], [141, 155, 106], [21, 169, 14], [38, 117, 127], [5, 29, 229], [126, 6, 43], [197, 235, 168], [171, 105, 144], [138, 6, 80], [214, 187, 208], [63, 203, 134], [55, 138, 0], [172, 146, 87], [43, 235, 113], [13, 41, 202], [22, 183, 8], [53, 74, 159], [189, 65, 93], [16, 236, 107], [156, 157, 88], [50, 42, 149], [70, 71, 217], [214, 202, 192], [247, 175, 89], [155, 12, 205], [124, 25, 26], [188, 6, 64], [135, 169, 142], [211, 89, 124], [174, 65, 92], [135, 200, 35], [90, 55, 46], [186, 135, 5], [177, 255, 173], [198, 198, 39], [214, 224, 124], [220, 76, 124], [40, 92, 125], [26, 186, 243], [63, 8, 122], [109, 196, 92], [103, 64, 194], [29, 103, 166], [75, 36, 95], [223, 95, 28], [71, 158, 190], [212, 39, 33], [96, 225, 69], [4, 240, 130], [48, 132, 155], [200, 150, 84], [80, 81, 189], [237, 35, 143], [156, 18, 131], [240, 218, 11], [50, 189, 248], [138, 139, 103], [230, 27, 58], [54, 74, 97], [157, 77, 130], [180, 242, 55], [251, 18, 195], [81, 91, 22], [45, 57, 226], [20, 143, 225], [244, 111, 206], [131, 124, 102], [211, 193, 32], [45, 158, 31], [140, 238, 253], [194, 248, 43], [191, 64, 237], [164, 99, 121], [138, 233, 24], [245, 97, 125], [123, 97, 93], [216, 51, 55], [114, 31, 40], [243, 103, 141], [57, 222, 184], [230, 161, 127], [117, 58, 81], [224, 172, 77], [21, 66, 251], [240, 254, 111], [229, 140, 90], [248, 161, 193], [215, 247, 63], [61, 129, 97], [124, 40, 137], [20, 24, 17], [210, 44, 255], [197, 224, 12], [77, 238, 41], [190, 9, 142], [136, 159, 240], [146, 115, 201], [92, 171, 249], [27, 136, 221], [100, 204, 42], [34, 154, 33], [234, 17, 97], [173, 38, 207], [14, 20, 242], [238, 65, 153], [148, 161, 216], [169, 1, 133], [12, 7, 28], [49, 123, 156], [17, 225, 72], [187, 198, 214], [213, 42, 248], [78, 87, 75], [11, 67, 161], [22, 61, 114], [91, 229, 77], [212, 204, 109], [105, 25, 56], [63, 152, 60], [155, 167, 89], [103, 176, 162], [249, 75, 174], [19, 176, 174], [77, 93, 154], [175, 47, 81], [34, 9, 160], [56, 162, 191], [65, 122, 16], [20, 229, 202], [75, 136, 208], [8, 152, 47], [232, 40, 22], [109, 72, 205], [3, 247, 31], [35, 228, 240], [191, 86, 166], [224, 85, 68], [52, 70, 147], [42, 25, 90], [253, 241, 205], [168, 139, 103], [8, 94, 188], [82, 168, 156], [115, 36, 37], [66, 104, 149], [153, 30, 184], [252, 153, 110], [5, 107, 197], [63, 51, 106], [144, 104, 154], [14, 238, 243], [70, 118, 144], [61, 243, 31], [1, 210, 236], [41, 202, 161], [34, 57, 57], [145, 132, 136], [222, 96, 62], [6, 145, 134], [99, 235, 49], [237, 247, 137], [59, 72, 251], [12, 14, 210], [98, 14, 151], [11, 160, 91], [160, 65, 111], [235, 87, 138], [231, 71, 226], [197, 4, 142], [86, 167, 118], [84, 196, 211], [71, 99, 62], [114, 127, 165], [193, 92, 254], [44, 241, 99], [163, 15, 191], [18, 151, 175], [100, 183, 34], [57, 142, 156], [248, 133, 103], [0, 55, 23], [78, 76, 242], [225, 166, 237], [217, 26, 32], [8, 166, 162], [35, 31, 33], [144, 125, 25], [7, 230, 66], [193, 2, 124], [222, 86, 42], [105, 90, 33], [152, 77, 191], [150, 109, 224], [34, 54, 181], [0, 3, 58], [123, 157, 91], [76, 64, 242], [114, 225, 136], [2, 78, 3], [211, 85, 241], [38, 167, 63], [155, 216, 155], [148, 147, 115], [46, 191, 214], [147, 191, 192], [54, 149, 77], [139, 247, 129], [244, 185, 209], [209, 247, 158], [125, 182, 53], [210, 138, 186], [203, 110, 253], [117, 228, 80], [37, 123, 98], [27, 76, 82], [39, 39, 115], [181, 32, 184], [120, 122, 44], [19, 80, 50], [114, 64, 168], [61, 134, 221], [41, 74, 165], [130, 73, 225], [97, 185, 244], [112, 35, 240], [6, 180, 210], [232, 101, 185], [118, 176, 158], [166, 221, 142], [228, 4, 77], [129, 89, 35], [213, 128, 43], [54, 82, 79], [183, 83, 233], [193, 167, 108], [46, 176, 252], [2, 62, 34], [179, 219, 114], [105, 107, 157], [222, 208, 226], [45, 103, 143], [170, 118, 31], [34, 245, 149], [31, 101, 74], [64, 171, 43], [186, 66, 50], [10, 219, 197], [138, 46, 136], [215, 146, 25], [105, 7, 235], [44, 76, 178], [111, 230, 98], [109, 193, 24], [0, 246, 150], [37, 3, 183], [105, 90, 158], [154, 30, 214], [125, 139, 37], [179, 25, 251], [2, 38, 75], [158, 243, 62], [252, 117, 222], [40, 161, 219], [120, 35, 73], [149, 0, 147], [70, 97, 74], [218, 109, 223], [175, 236, 229], [64, 49, 198], [94, 173, 39], [44, 92, 189], [59, 242, 53], [54, 2, 197], [114, 43, 46], [19, 95, 221], [153, 26, 251], [54, 29, 57], [190, 95, 210], [217, 135, 145], [190, 148, 59], [143, 113, 70], [195, 168, 45], [227, 182, 166], [230, 22, 209], [73, 218, 75], [141, 121, 5], [39, 189, 45], [95, 245, 99], [164, 117, 253], [56, 213, 103], [240, 118, 250], [120, 52, 95], [26, 69, 145], [229, 207, 50], [242, 191, 157], [185, 74, 255], [134, 216, 84], [244, 148, 137], [143, 68, 236], [181, 251, 83], [41, 22, 32], [163, 200, 90], [223, 22, 106], [178, 255, 1], [69, 238, 161], [194, 201, 255], [136, 175, 249], [132, 40, 77], [117, 91, 97], [131, 208, 25], [139, 160, 236], [116, 237, 12], [237, 146, 73], [9, 249, 25], [159, 212, 208], [150, 133, 80], [19, 117, 37], [172, 49, 68], [117, 18, 225], [117, 241, 51], [195, 252, 96], [105, 91, 124], [166, 50, 229], [144, 248, 157], [158, 147, 15], [5, 158, 92], [237, 154, 1], [91, 213, 25], [77, 186, 164], [95, 48, 10], [121, 21, 213], [247, 156, 141], [99, 23, 214], [201, 130, 175], [7, 131, 63], [194, 238, 63], [213, 25, 45], [61, 156, 37], [5, 190, 9], [2, 118, 2], [115, 130, 86], [192, 31, 160], [64, 255, 75], [240, 146, 18], [40, 140, 32], [47, 69, 132], [220, 228, 93], [23, 167, 155], [41, 104, 222], [61, 82, 171], [200, 208, 13], [248, 50, 114], [201, 47, 82], [144, 22, 33], [141, 124, 25], [214, 43, 11], [196, 231, 239], [56, 131, 166], [133, 76, 179], [175, 80, 59], [252, 191, 114], [255, 156, 165], [166, 254, 103], [151, 250, 225], [180, 227, 102], [133, 94, 222], [138, 142, 142], [68, 153, 120], [135, 132, 137], [154, 188, 30], [179, 130, 86], [236, 223, 188], [72, 139, 94], [91, 219, 31], [71, 20, 93], [30, 65, 26], [248, 218, 93], [33, 58, 98], [200, 194, 246], [63, 13, 161], [216, 22, 207], [230, 70, 102], [204, 39, 145], [160, 64, 3], [6, 201, 218], [221, 14, 182], [78, 151, 244], [2, 5, 146], [40, 59, 181], [164, 5, 56], [89, 198, 184], [69, 196, 156], [19, 229, 68], [88, 123, 29], [102, 183, 202], [16, 187, 112], [141, 159, 93], [117, 231, 137], [228, 161, 120], [154, 185, 191], [152, 61, 142], [109, 192, 59], [16, 136, 85], [194, 55, 56], [225, 187, 225], [153, 81, 172], [226, 202, 237], [255, 237, 125], [127, 92, 248], [25, 29, 209], [100, 47, 85], [199, 219, 158], [243, 249, 98], [8, 121, 2], [236, 183, 162], [70, 95, 223], [204, 242, 89], [4, 112, 132], [173, 48, 242], [177, 27, 185], [171, 252, 88], [238, 168, 143], [217, 25, 54], [144, 1, 191], [168, 63, 49], [253, 99, 80], [36, 110, 30], [96, 90, 177], [9, 22, 135], [65, 148, 19], [42, 43, 95], [203, 74, 18], [171, 6, 99], [166, 17, 211], [26, 243, 138], [132, 92, 251], [132, 39, 44], [97, 210, 105], [237, 4, 114], [68, 143, 54], [230, 246, 152], [156, 189, 180], [84, 76, 21], [219, 82, 61], [52, 102, 202], [217, 64, 73], [68, 162, 146], [251, 42, 84], [45, 56, 137], [229, 25, 70], [150, 96, 182], [110, 210, 238], [170, 121, 36], [99, 15, 53], [250, 92, 193], [150, 194, 128], [71, 109, 18], [218, 54, 35], [51, 141, 51], [210, 74, 138], [163, 181, 26], [119, 79, 28], [83, 205, 253], [151, 230, 127], [233, 73, 177], [17, 221, 139], [110, 244, 50], [80, 193, 138], [138, 66, 248], [1, 33, 231], [224, 39, 221], [201, 27, 252], [14, 63, 195], [188, 130, 248], [84, 98, 72], [133, 230, 209], [108, 69, 94], [158, 15, 86], [10, 8, 103], [2, 249, 166], [142, 70, 120], [92, 30, 178], [167, 220, 84], [40, 204, 230], [235, 90, 119], [48, 147, 1], [171, 87, 73], [151, 124, 188], [121, 150, 113], [151, 95, 75], [71, 161, 159], [244, 31, 241], [50, 23, 56], [243, 85, 224], [4, 2, 96], [196, 250, 224], [234, 192, 41], [22, 3, 222], [227, 107, 108], [16, 75, 197], [93, 36, 44], [96, 144, 115], [16, 2, 0], [208, 23, 223], [254, 241, 112], [240, 175, 9], [231, 158, 155], [240, 160, 150], [77, 58, 164], [4, 32, 185], [144, 237, 88], [200, 25, 114], [49, 253, 81], [78, 121, 228], [68, 63, 40], [8, 163, 9], [241, 103, 217], [32, 105, 244], [5, 157, 154], [58, 20, 7], [19, 50, 202], [119, 36, 69], [37, 30, 128], [80, 31, 113], [230, 17, 81], [90, 119, 17], [23, 60, 182], [61, 202, 30], [75, 183, 115], [215, 143, 96], [64, 233, 41], [54, 5, 229], [72, 29, 26], [196, 143, 237], [35, 119, 168], [148, 81, 77], [80, 213, 49], [215, 37, 73], [104, 35, 223], [137, 134, 48], [168, 6, 223], [25, 116, 9], [19, 74, 83], [23, 74, 68], [234, 78, 180], [154, 225, 3], [113, 207, 180], [203, 164, 13], [185, 108, 159], [18, 70, 62], [239, 247, 143], [110, 39, 88], [112, 229, 219], [30, 32, 224], [4, 57, 56], [242, 30, 119], [152, 121, 0], [211, 253, 143], [6, 109, 193], [124, 190, 56], [7, 7, 206], [137, 148, 207], [117, 4, 194], [108, 208, 76], [193, 59, 4], [220, 74, 8], [124, 206, 134], [94, 150, 21], [38, 46, 72], [8, 200, 154], [255, 110, 246], [6, 146, 191], [155, 202, 113], [253, 57, 34], [121, 254, 181], [89, 59, 77], [194, 141, 191], [178, 42, 248], [202, 5, 211], [70, 211, 232], [71, 107, 106], [95, 130, 224], [1, 28, 4], [188, 245, 2], [207, 8, 197], [156, 141, 234], [32, 95, 156], [243, 33, 231], [111, 69, 10], [235, 34, 45], [169, 188, 13], [92, 213, 38], [160, 133, 173], [242, 214, 173], [240, 205, 44], [178, 34, 120], [144, 248, 3], [134, 41, 79], [254, 252, 255], [234, 177, 17], [46, 26, 183], [181, 214, 64], [28, 32, 65], [169, 145, 217], [179, 88, 104], [233, 63, 13], [158, 104, 229], [106, 104, 145], [38, 83, 252], [30, 255, 47], [145, 113, 106], [141, 0, 219], [149, 114, 120], [166, 249, 45], [19, 218, 45], [142, 141, 172], [173, 226, 103], [163, 200, 66], [143, 28, 102], [15, 125, 25], [192, 235, 231], [67, 42, 110], [247, 177, 44], [241, 110, 110], [111, 183, 158], [197, 124, 144], [78, 53, 244], [46, 108, 118], [6, 222, 83], [71, 249, 37], [58, 241, 109], [71, 43, 237], [82, 18, 101], [36, 229, 172], [229, 94, 153], [194, 235, 77], [88, 122, 113], [92, 67, 194], [18, 227, 109], [75, 146, 138], [205, 145, 9], [33, 162, 181], [6, 109, 45], [114, 98, 37], [30, 88, 24], [20, 72, 181], [174, 189, 195], [142, 226, 11], [88, 47, 184], [172, 105, 208], [42, 189, 192], [107, 35, 145], [20, 33, 146], [69, 115, 37], [238, 180, 116], [95, 44, 103], [34, 204, 238], [185, 67, 1], [221, 179, 211], [189, 204, 0], [195, 190, 209], [171, 134, 124], [240, 108, 10], [156, 161, 223], [143, 69, 100], [65, 79, 9], [82, 142, 119], [204, 197, 135], [220, 189, 84], [82, 0, 83], [140, 203, 76], [99, 12, 2], [63, 96, 209], [142, 36, 186], [189, 172, 200], [71, 252, 206], [111, 135, 182], [52, 217, 153], [53, 98, 41], [231, 213, 7], [226, 142, 51], [170, 242, 59], [15, 193, 49], [38, 177, 247], [151, 36, 108], [125, 0, 209], [118, 99, 106], [230, 27, 218], [85, 45, 76], [150, 129, 96], [167, 52, 57], [23, 115, 247], [7, 149, 233], [150, 148, 210], [242, 109, 135], [20, 250, 183], [73, 199, 110], [204, 2, 67], [107, 163, 5], [75, 121, 87], [172, 3, 14], [5, 100, 159], [24, 41, 121], [181, 207, 70], [51, 163, 61], [29, 166, 241], [64, 232, 21], [126, 143, 205], [15, 236, 114], [214, 62, 53], [134, 0, 201], [224, 65, 112], [82, 191, 158], [205, 151, 6], [91, 69, 230], [69, 91, 236], [26, 212, 0], [191, 203, 199], [123, 200, 161], [118, 85, 26], [196, 8, 90], [134, 96, 130], [25, 189, 15], [15, 132, 88], [140, 46, 39], [127, 180, 89], [200, 200, 51], [106, 242, 39], [229, 70, 75], [21, 196, 130], [189, 204, 61], [140, 38, 240], [123, 208, 224], [219, 195, 66], [114, 107, 59], [48, 153, 73], [92, 209, 33], [255, 40, 125], [109, 210, 7], [184, 185, 68], [231, 57, 40], [103, 38, 212], [66, 100, 250], [164, 215, 120], [179, 97, 3], [17, 33, 88], [17, 198, 22], [199, 243, 69], [182, 151, 144], [187, 83, 169], [245, 65, 62], [15, 15, 186], [246, 142, 253], [121, 181, 79], [123, 195, 115], [201, 233, 74], [219, 104, 134], [77, 2, 207], [52, 16, 36], [111, 40, 158], [243, 28, 240], [39, 172, 5], [159, 21, 196], [246, 32, 8], [173, 97, 23], [78, 132, 95], [162, 202, 183], [11, 93, 187], [104, 227, 28], [105, 94, 37], [17, 31, 124], [178, 181, 160], [40, 19, 234], [172, 67, 245], [45, 220, 93], [249, 173, 0], [119, 168, 188], [10, 20, 20], [176, 67, 255], [19, 106, 150], [174, 75, 18], [180, 247, 176], [133, 184, 141], [71, 69, 54], [240, 113, 49], [250, 201, 13], [202, 235, 248], [66, 210, 195], [109, 104, 82], [90, 140, 154], [59, 203, 37], [75, 202, 120], [36, 21, 103], [36, 221, 150], [68, 203, 19], [140, 107, 216], [128, 220, 174], [132, 198, 119], [172, 240, 215], [252, 152, 102], [54, 4, 82], [20, 13, 69], [34, 194, 246], [163, 250, 11], [119, 103, 121], [58, 95, 61], [83, 50, 37], [115, 158, 20], [155, 142, 110], [8, 129, 148], [42, 60, 190], [164, 51, 155], [10, 160, 134], [246, 178, 254], [43, 44, 92], [28, 67, 160], [245, 188, 13], [172, 221, 232], [196, 37, 176], [170, 233, 119], [50, 31, 145], [148, 164, 24], [7, 81, 93], [240, 152, 244], [155, 37, 149], [121, 46, 183], [59, 99, 30], [43, 188, 3], [115, 185, 98], [141, 51, 22], [108, 206, 246], [124, 130, 20], [61, 109, 184], [241, 60, 184], [144, 31, 198], [215, 165, 51], [76, 232, 4], [154, 105, 166], [21, 34, 148], [10, 48, 15], [8, 112, 27], [9, 39, 127], [136, 83, 91], [245, 254, 89], [195, 139, 247], [31, 94, 90], [177, 149, 132], [101, 92, 251], [231, 164, 215], [116, 226, 122], [182, 170, 17], [27, 231, 106], [51, 206, 77], [227, 222, 231], [125, 224, 244], [252, 117, 125], [231, 155, 211], [178, 58, 26], [205, 142, 130], [133, 145, 199], [136, 18, 53], [77, 40, 90], [198, 62, 115], [109, 235, 239], [14, 138, 8], [35, 134, 79], [40, 102, 87], [179, 39, 161], [253, 121, 211], [187, 163, 58], [144, 210, 98], [100, 57, 151], [116, 145, 115], [211, 152, 245], [110, 3, 180], [17, 110, 58], [46, 74, 63], [223, 172, 251], [209, 85, 185], [117, 136, 108], [16, 229, 41], [7, 43, 28], [108, 204, 62], [10, 27, 203], [221, 162, 164], [233, 135, 120], [199, 174, 53], [220, 46, 193], [90, 108, 231], [30, 217, 34], [249, 235, 227], [25, 126, 114], [95, 181, 4], [46, 25, 26], [152, 164, 109], [80, 157, 145], [244, 222, 1], [98, 14, 167], [244, 243, 143], [241, 174, 65], [43, 131, 60], [118, 234, 27], [129, 7, 97], [156, 190, 185], [229, 69, 158], [11, 135, 124], [214, 7, 22], [85, 201, 27], [188, 165, 173], [76, 40, 228], [184, 216, 125], [32, 253, 236], [144, 19, 89], [192, 175, 20], [129, 91, 36], [235, 243, 132], [237, 36, 139], [155, 20, 162], [241, 135, 140], [51, 199, 68], [142, 171, 93], [118, 48, 200], [90, 196, 140], [174, 199, 195], [131, 255, 220], [55, 12, 115], [184, 32, 75], [29, 171, 37], [242, 245, 254], [210, 212, 58], [206, 62, 225], [157, 200, 173], [210, 117, 71], [40, 90, 146], [40, 176, 103], [245, 138, 239], [181, 34, 246], [132, 17, 184], [202, 8, 229], [91, 24, 9], [167, 120, 195], [12, 179, 6], [22, 33, 103], [30, 18, 223], [31, 181, 99], [20, 237, 22], [197, 192, 49], [143, 110, 92], [171, 67, 228], [18, 31, 129], [173, 170, 54], [194, 87, 109], [27, 22, 52], [61, 175, 28], [63, 150, 107], [190, 207, 138], [242, 196, 178], [190, 22, 89], [0, 130, 23], [40, 255, 36], [117, 93, 38], [141, 102, 243], [174, 90, 194], [247, 144, 235], [180, 58, 1], [49, 158, 41], [156, 199, 216], [153, 136, 243], [106, 177, 89], [116, 13, 197], [146, 11, 252], [195, 175, 56], [146, 129, 158], [205, 156, 67], [236, 89, 177], [13, 198, 143], [246, 49, 154], [232, 155, 192], [1, 145, 65], [166, 111, 188], [118, 161, 106], [98, 45, 14], [146, 94, 26], [193, 35, 183], [199, 37, 152], [233, 8, 88], [108, 107, 111], [215, 38, 91], [157, 42, 201], [222, 184, 53], [122, 220, 143], [127, 82, 54], [236, 209, 248], [74, 218, 133], [114, 145, 166], [226, 82, 9], [16, 84, 155], [170, 165, 30], [129, 250, 253], [119, 91, 15], [34, 185, 193], [15, 73, 2], [115, 210, 29], [14, 31, 70], [230, 117, 83], [48, 64, 67], [223, 157, 182], [23, 197, 57], [58, 130, 87], [142, 87, 131], [167, 240, 187], [178, 12, 217], [198, 76, 41], [209, 160, 108], [248, 9, 73], [196, 241, 148], [140, 97, 220], [223, 138, 188], [135, 229, 16], [152, 133, 239], [47, 198, 86], [56, 156, 223], [12, 106, 187], [75, 163, 24], [195, 50, 147], [133, 123, 162], [198, 0, 219], [188, 177, 225], [222, 63, 175], [22, 69, 108], [249, 129, 253], [34, 52, 207], [139, 104, 93], [183, 85, 117], [91, 87, 172], [6, 246, 157], [20, 154, 119], [186, 186, 214], [106, 81, 78], [199, 146, 238], [122, 0, 234], [193, 65, 153], [155, 29, 38], [199, 126, 22], [197, 149, 28], [96, 129, 16], [255, 73, 133], [120, 233, 212], [228, 52, 57], [202, 84, 144], [113, 98, 83], [164, 2, 244], [137, 203, 108], [40, 99, 129], [206, 29, 194], [216, 77, 0], [133, 230, 188], [75, 83, 15], [111, 78, 151], [102, 245, 146], [120, 111, 55], [234, 165, 15], [136, 89, 65], [47, 66, 44], [124, 8, 128], [224, 238, 33], [114, 24, 153], [155, 75, 7], [145, 223, 169], [93, 176, 216], [64, 43, 207], [29, 80, 239], [175, 190, 4], [244, 199, 129], [254, 145, 96], [45, 37, 244], [26, 184, 254], [218, 222, 113], [69, 137, 229], [204, 236, 102], [109, 132, 170], [86, 247, 178], [173, 189, 130], [160, 175, 21], [87, 103, 11], [137, 97, 249], [226, 88, 170], [50, 231, 228], [75, 211, 65], [8, 84, 61], [167, 121, 128], [146, 6, 201], [236, 136, 163], [35, 14, 49], [145, 27, 40], [147, 161, 99], [54, 183, 115], [57, 205, 164], [171, 101, 89], [16, 176, 219], [179, 168, 77], [6, 42, 8], [178, 124, 173], [6, 209, 141], [100, 89, 4], [58, 97, 205], [211, 188, 73], [76, 160, 95], [153, 181, 194], [241, 119, 255], [14, 226, 133], [51, 186, 114], [225, 52, 160], [105, 152, 64], [137, 133, 210], [243, 238, 82], [73, 135, 203], [224, 253, 160], [52, 78, 165], [165, 36, 81], [135, 23, 144], [82, 248, 227], [174, 71, 187], [63, 57, 94], [62, 23, 230], [215, 161, 72], [25, 135, 215], [32, 218, 149], [170, 46, 176], [77, 106, 194], [160, 150, 85], [107, 24, 243], [94, 73, 24], [123, 77, 120], [207, 211, 247], [57, 110, 82], [99, 11, 59], [38, 204, 110], [29, 175, 204], [67, 190, 106], [25, 232, 170], [14, 169, 30], [128, 27, 6], [165, 150, 24], [41, 71, 172], [137, 24, 0], [191, 123, 248], [225, 204, 218], [89, 26, 131], [184, 168, 90], [137, 32, 91], [144, 242, 90], [114, 94, 230], [79, 114, 93], [104, 200, 214], [162, 101, 137], [164, 204, 122], [181, 18, 180], [96, 215, 166], [103, 75, 34], [69, 118, 95], [241, 156, 90], [53, 54, 126], [110, 29, 7], [70, 190, 241], [241, 218, 11], [182, 12, 97], [91, 185, 150], [230, 198, 94], [153, 33, 207], [122, 40, 30], [209, 219, 13], [12, 198, 182], [135, 67, 184], [156, 167, 158], [240, 90, 31], [182, 113, 107], [159, 243, 102], [182, 166, 37], [174, 14, 134], [109, 169, 116], [247, 89, 221], [195, 4, 113], [46, 60, 31], [34, 22, 237], [95, 119, 225], [231, 54, 104], [165, 241, 128], [9, 38, 2], [242, 252, 35], [153, 33, 55], [237, 27, 151], [49, 153, 4], [179, 137, 134], [172, 200, 180], [70, 164, 3], [208, 150, 67], [181, 173, 211], [214, 129, 58], [164, 109, 198], [117, 4, 214], [128, 176, 149], [85, 145, 45], [114, 73, 137], [221, 43, 21], [82, 11, 64], [197, 80, 246], [122, 205, 129], [64, 187, 93], [120, 44, 62], [144, 216, 206], [49, 10, 163], [160, 115, 29], [99, 125, 103], [14, 170, 15], [178, 127, 130], [106, 185, 40], [11, 223, 32], [227, 104, 74], [39, 153, 14], [153, 203, 172], [177, 169, 69], [1, 117, 241], [175, 226, 180], [153, 232, 10], [66, 23, 166], [22, 172, 125], [122, 18, 35], [113, 64, 9], [240, 154, 45], [0, 1, 165], [45, 163, 189], [33, 203, 38], [124, 89, 34], [65, 9, 56], [230, 254, 68], [208, 186, 166], [199, 1, 45], [204, 231, 139], [203, 61, 156], [170, 85, 116], [46, 63, 180], [169, 173, 186], [104, 160, 234], [179, 177, 39], [94, 240, 213], [96, 171, 214], [121, 59, 40], [87, 113, 1], [216, 83, 173], [19, 202, 18], [228, 235, 146], [150, 173, 48], [104, 162, 240], [87, 59, 230], [124, 21, 3], [5, 162, 233], [27, 251, 254], [243, 144, 197], [138, 102, 85], [0, 20, 30], [132, 164, 130], [157, 52, 112], [243, 162, 62], [42, 249, 217], [6, 64, 155], [62, 155, 249], [146, 120, 227], [223, 216, 90], [92, 247, 184], [213, 144, 100], [14, 20, 172], [65, 130, 155], [220, 85, 115], [181, 185, 76], [249, 55, 33], [11, 6, 60], [195, 58, 243], [201, 141, 140], [131, 109, 242], [146, 3, 102], [148, 87, 50], [12, 77, 217], [40, 108, 118], [29, 45, 78], [233, 2, 37], [240, 55, 21], [153, 64, 62], [162, 197, 120], [144, 63, 125], [73, 209, 223], [201, 62, 14], [246, 72, 84], [107, 67, 45], [174, 54, 217], [201, 250, 229], [41, 152, 197], [107, 1, 83], [39, 197, 25], [18, 101, 243], [106, 114, 107], [153, 50, 195], [44, 44, 109], [148, 87, 95], [222, 33, 24], [235, 239, 110], [123, 153, 32], [245, 41, 95], [141, 150, 90], [238, 0, 161], [75, 180, 58], [127, 159, 222], [168, 254, 47], [10, 224, 19], [147, 17, 165], [240, 248, 179], [48, 118, 255], [23, 33, 252], [112, 238, 191], [65, 61, 224], [224, 148, 81], [193, 225, 234], [1, 17, 49], [5, 203, 153], [129, 62, 114], [47, 220, 194], [120, 240, 154], [43, 163, 58], [150, 16, 220], [105, 17, 188], [41, 55, 146], [37, 32, 55], [145, 57, 43], [242, 124, 202], [45, 124, 126], [246, 229, 227], [6, 62, 62], [19, 170, 97], [71, 127, 147], [87, 118, 162], [140, 150, 193], [185, 54, 72], [134, 170, 243], [121, 224, 184], [219, 103, 177], [95, 41, 205], [111, 252, 181], [175, 159, 56], [159, 190, 48], [135, 112, 3], [181, 244, 26], [29, 74, 5], [228, 98, 0], [184, 69, 208], [114, 174, 162], [143, 116, 85], [222, 85, 162], [169, 140, 31], [213, 128, 195], [163, 57, 196], [81, 95, 112], [154, 197, 66], [2, 34, 8], [25, 172, 225], [55, 76, 91], [207, 235, 158], [77, 236, 248], [140, 140, 72], [6, 127, 132], [206, 149, 22], [44, 248, 123], [20, 170, 7], [67, 34, 128], [232, 233, 219], [195, 127, 44], [221, 91, 123], [229, 55, 153], [165, 24, 33], [248, 248, 237], [203, 103, 10], [144, 24, 70], [156, 207, 109], [128, 37, 222], [223, 143, 70], [8, 215, 11], [192, 47, 33], [28, 75, 239], [103, 145, 59], [3, 234, 247], [244, 123, 27], [144, 34, 51], [23, 133, 44], [96, 20, 93], [246, 57, 195], [110, 170, 79], [199, 68, 144], [212, 232, 167], [83, 53, 224], [93, 92, 171], [21, 108, 245], [17, 81, 133], [87, 119, 32], [163, 63, 68], [107, 152, 16], [98, 59, 118], [34, 120, 184], [221, 149, 252], [34, 50, 75], [0, 11, 60], [103, 167, 149], [208, 58, 97], [53, 142, 147], [118, 59, 212], [229, 25, 130], [168, 199, 232], [98, 68, 14], [244, 225, 32], [22, 179, 142], [39, 137, 203], [136, 114, 178], [153, 44, 117], [225, 23, 143], [8, 199, 254], [160, 144, 51], [147, 76, 39], [209, 95, 113], [102, 213, 25], [95, 163, 185], [63, 28, 135], [217, 7, 244], [50, 197, 115], [187, 183, 230], [72, 139, 207], [162, 223, 113], [77, 44, 164], [169, 132, 27], [173, 77, 219], [241, 27, 16], [12, 87, 182], [82, 88, 170], [132, 201, 93], [50, 73, 176], [181, 102, 253], [166, 118, 100], [19, 102, 203], [24, 38, 203], [155, 230, 221], [211, 10, 58], [138, 92, 19], [87, 154, 150], [230, 2, 175], [114, 65, 13], [218, 215, 105], [204, 239, 103], [204, 19, 244], [13, 12, 129], [253, 6, 114], [45, 238, 117], [236, 15, 10], [240, 209, 59], [199, 99, 106], [59, 161, 204], [166, 112, 195], [83, 86, 174], [93, 141, 142], [63, 0, 23], [162, 214, 221], [3, 23, 10], [255, 165, 178], [173, 75, 2], [84, 6, 117], [242, 194, 93], [141, 172, 6], [236, 163, 170], [11, 39, 237], [220, 151, 252], [35, 104, 161], [42, 88, 200], [66, 187, 187], [81, 9, 231], [100, 0, 109], [102, 12, 53], [91, 130, 62], [132, 123, 234], [113, 128, 124], [65, 46, 237], [37, 236, 111], [254, 52, 42], [9, 7, 192], [3, 159, 20], [132, 144, 46], [143, 102, 232], [212, 117, 14], [40, 127, 206], [117, 66, 197], [88, 142, 138], [192, 136, 135], [200, 195, 52], [102, 150, 168], [149, 127, 169], [244, 240, 189], [29, 10, 80], [124, 247, 184], [130, 252, 100], [229, 225, 33], [19, 27, 147], [63, 64, 158], [165, 133, 137], [153, 222, 249], [33, 9, 177], [252, 62, 164], [225, 158, 255], [78, 189, 53], [20, 19, 117], [205, 219, 242], [88, 235, 239], [52, 211, 172], [149, 197, 101], [117, 235, 126], [25, 112, 233], [66, 180, 212], [162, 193, 49], [250, 228, 149], [120, 160, 69], [77, 115, 245], [69, 93, 86], [105, 92, 255], [139, 55, 33], [142, 29, 207], [193, 175, 123], [248, 202, 178], [220, 241, 248], [166, 52, 18], [138, 148, 211], [76, 183, 79], [28, 136, 109], [10, 247, 128], [200, 57, 215], [216, 149, 135], [107, 56, 124], [134, 2, 125], [136, 18, 92], [105, 237, 11], [76, 216, 242], [87, 196, 230], [52, 58, 140], [58, 24, 239], [172, 18, 74], [154, 39, 64], [159, 92, 160], [23, 19, 94], [75, 110, 14], [110, 231, 79], [95, 67, 200], [58, 86, 4], [73, 243, 93], [162, 8, 188], [215, 66, 45], [41, 79, 249], [245, 79, 145], [188, 213, 255], [223, 249, 241], [76, 3, 2], [138, 130, 66], [139, 26, 54], [234, 245, 219], [171, 64, 219], [26, 167, 11], [102, 199, 47], [238, 214, 226], [106, 225, 243], [214, 56, 179], [124, 232, 168], [86, 68, 52], [243, 132, 187], [196, 137, 133], [71, 12, 82], [99, 43, 205], [16, 226, 232], [159, 99, 186], [26, 6, 24], [16, 163, 225], [34, 55, 95], [122, 131, 124], [194, 201, 8], [225, 182, 250], [248, 152, 30], [59, 47, 94], [168, 193, 57], [26, 118, 226], [211, 149, 166], [79, 165, 171], [93, 130, 84], [101, 149, 67], [80, 76, 25], [46, 43, 237], [196, 133, 87], [44, 181, 195], [43, 106, 37], [214, 179, 183], [92, 206, 111], [84, 118, 249], [55, 13, 217], [161, 28, 203], [212, 229, 99], [42, 162, 15], [182, 115, 251], [85, 248, 168], [168, 216, 161], [26, 171, 59], [181, 121, 63], [129, 44, 204], [194, 221, 89], [233, 126, 107], [69, 171, 173], [49, 84, 84], [164, 96, 230], [252, 172, 154], [154, 136, 136], [156, 13, 117], [112, 182, 96], [248, 165, 191], [162, 218, 67], [95, 150, 6], [99, 124, 59], [133, 97, 207], [169, 125, 223], [103, 23, 72], [241, 164, 116], [202, 7, 46], [143, 47, 47], [202, 42, 102], [88, 97, 87], [216, 36, 8], [221, 223, 252], [240, 252, 115], [175, 125, 112], [237, 42, 2], [78, 167, 199], [116, 34, 247], [62, 178, 156], [6, 129, 212], [85, 17, 191], [139, 50, 237], [155, 118, 29], [93, 113, 166], [64, 65, 20], [144, 77, 67], [22, 48, 73], [18, 90, 155], [77, 76, 26], [186, 14, 182], [113, 99, 67], [151, 86, 39], [34, 150, 134], [2, 127, 67], [49, 160, 233], [201, 104, 3], [247, 198, 179], [131, 179, 238], [70, 213, 38], [243, 162, 232], [124, 96, 182], [160, 33, 200], [53, 153, 205], [255, 149, 145], [28, 23, 129], [74, 69, 50], [40, 147, 96], [17, 226, 207], [116, 147, 160], [120, 233, 187], [234, 172, 3], [137, 255, 164], [241, 94, 196], [232, 186, 44], [139, 23, 196], [155, 126, 0], [92, 199, 80], [32, 124, 91], [176, 148, 98], [234, 106, 183], [172, 110, 110], [170, 249, 181], [72, 242, 22], [80, 253, 0], [133, 110, 76], [242, 191, 79], [119, 248, 73], [187, 196, 14], [130, 236, 127], [53, 135, 66], [217, 200, 235], [15, 75, 153], [2, 115, 137], [11, 123, 27], [135, 224, 138], [222, 2, 77], [96, 79, 74], [175, 78, 184], [234, 176, 161], [224, 185, 10], [78, 76, 66], [7, 177, 39], [6, 6, 220], [33, 236, 247], [121, 203, 45], [29, 254, 68], [146, 244, 19], [219, 246, 79], [204, 49, 158], [6, 237, 56], [240, 118, 116], [63, 97, 165], [83, 175, 143], [120, 5, 62], [154, 87, 206], [42, 234, 235], [186, 129, 61], [134, 225, 218], [134, 100, 53], [26, 255, 169], [195, 217, 195], [166, 91, 180], [189, 208, 135], [181, 229, 158], [231, 30, 128], [190, 211, 114], [203, 127, 22], [183, 217, 162], [198, 41, 32], [128, 0, 47], [105, 122, 139], [26, 9, 172], [109, 4, 206], [223, 48, 32], [244, 249, 238], [171, 244, 211], [192, 180, 147], [47, 154, 226], [0, 52, 211], [230, 207, 94], [224, 212, 233], [70, 103, 172], [66, 11, 183], [74, 220, 254], [252, 149, 136], [86, 186, 238], [214, 14, 26], [56, 44, 128], [169, 188, 16], [189, 79, 81], [191, 113, 29], [139, 64, 156], [118, 196, 1], [84, 188, 100], [119, 11, 133], [80, 42, 5], [189, 93, 33], [32, 41, 150], [163, 68, 93], [129, 216, 13], [105, 102, 16], [123, 133, 187], [234, 246, 187], [3, 214, 41], [174, 193, 49], [6, 50, 85], [211, 77, 30], [243, 127, 7], [199, 4, 78], [118, 176, 14], [172, 117, 137], [55, 178, 174], [65, 43, 121], [108, 209, 148], [39, 171, 255], [166, 121, 41], [101, 247, 167], [207, 51, 229], [127, 244, 251], [141, 254, 11], [224, 188, 141], [237, 76, 165], [124, 75, 165], [110, 14, 140], [169, 138, 95], [125, 161, 239], [26, 81, 238], [223, 58, 182], [33, 182, 168], [226, 47, 81], [242, 95, 143], [211, 65, 126], [67, 92, 149], [31, 224, 220], [215, 245, 136], [15, 187, 172], [144, 236, 46], [137, 174, 183], [117, 34, 51], [185, 67, 185], [129, 213, 228], [21, 250, 16], [84, 238, 122], [206, 7, 69], [61, 113, 64], [249, 25, 139], [84, 9, 53], [248, 216, 49], [232, 81, 43], [34, 223, 89], [76, 124, 11], [222, 226, 34], [135, 109, 66], [163, 220, 168], [92, 224, 219], [150, 243, 178], [193, 241, 205], [4, 163, 65], [234, 84, 235], [99, 83, 108], [31, 65, 30], [94, 14, 212], [20, 115, 242], [226, 216, 186], [90, 34, 37], [193, 110, 72], [62, 185, 162], [161, 125, 27], [153, 193, 137], [159, 163, 170], [197, 16, 52], [13, 222, 161], [63, 237, 229], [106, 94, 78], [84, 136, 85], [194, 2, 42], [3, 26, 100], [170, 47, 41], [5, 6, 16], [180, 65, 165], [249, 66, 178], [137, 44, 81], [126, 228, 133], [245, 145, 59], [158, 187, 45], [208, 65, 77], [232, 70, 213], [239, 212, 100], [105, 119, 211], [116, 38, 34], [201, 8, 238], [136, 9, 120], [88, 22, 51], [113, 197, 65], [206, 23, 60], [123, 241, 249], [201, 187, 50], [204, 166, 141], [70, 217, 239], [242, 75, 100], [228, 175, 234], [39, 245, 29], [163, 53, 172], [241, 210, 27], [243, 86, 204], [13, 41, 158], [95, 31, 11], [104, 186, 159], [12, 227, 100], [47, 36, 70], [254, 29, 84], [57, 73, 244], [14, 156, 223], [153, 6, 111], [231, 87, 15], [77, 89, 56], [6, 197, 123], [82, 48, 128], [2, 82, 81], [138, 246, 176], [119, 205, 193], [148, 3, 215], [21, 26, 192], [193, 212, 83], [185, 38, 206], [107, 48, 60], [39, 123, 138], [101, 24, 19], [73, 238, 243], [99, 211, 55], [159, 207, 129], [77, 42, 247], [63, 181, 209], [212, 119, 200], [126, 46, 205], [226, 233, 39], [16, 129, 10], [245, 208, 119], [134, 177, 248], [106, 242, 134], [0, 239, 243], [140, 165, 165], [158, 196, 24], [102, 9, 113], [93, 75, 241], [0, 229, 249], [165, 166, 160], [132, 62, 173], [64, 45, 206], [104, 145, 52], [170, 139, 33], [129, 57, 81], [108, 75, 88], [1, 113, 80], [217, 202, 218], [132, 156, 61], [195, 246, 141], [35, 91, 134], [57, 80, 119], [118, 57, 51], [30, 172, 18], [220, 81, 169], [135, 245, 188], [208, 9, 159], [16, 236, 49], [75, 124, 240], [154, 40, 86], [30, 140, 131], [68, 196, 101], [57, 109, 25], [209, 152, 201], [172, 93, 103], [76, 188, 142], [140, 176, 110], [44, 165, 28], [128, 55, 154], [188, 239, 233], [228, 17, 198], [237, 58, 210], [112, 112, 19], [77, 89, 41], [46, 229, 177], [187, 52, 163], [242, 7, 248], [219, 149, 191], [57, 19, 197], [217, 117, 181], [183, 24, 247], [213, 139, 79], [129, 13, 139], [221, 20, 68], [122, 140, 168], [225, 108, 106], [43, 228, 39], [138, 94, 180], [118, 1, 216], [88, 146, 238], [1, 51, 137], [175, 244, 8], [212, 181, 145], [41, 245, 8], [111, 75, 112], [171, 83, 48], [232, 87, 151], [54, 237, 203], [159, 92, 23], [11, 65, 90], [55, 44, 253], [148, 232, 185], [52, 203, 49], [34, 15, 27], [101, 112, 192], [158, 76, 214], [128, 148, 110], [213, 115, 196], [246, 242, 91], [117, 123, 71], [17, 133, 136], [13, 30, 226], [75, 172, 213], [134, 57, 148], [160, 98, 137], [209, 221, 100], [241, 141, 148], [35, 39, 165], [13, 228, 29], [144, 88, 30], [49, 136, 16], [219, 144, 66], [238, 79, 138], [117, 110, 153], [50, 21, 177], [239, 247, 242], [123, 235, 246], [72, 164, 5], [198, 201, 126], [68, 121, 114], [238, 136, 233], [83, 185, 50], [181, 185, 1], [183, 181, 192], [110, 88, 239], [35, 163, 200], [124, 253, 98], [9, 250, 116], [218, 123, 134], [129, 193, 198], [65, 158, 254], [165, 158, 148], [71, 72, 99], [111, 44, 247], [27, 182, 95], [245, 156, 244], [162, 189, 147], [128, 181, 235], [189, 240, 204], [76, 162, 92], [249, 212, 125], [110, 79, 162], [251, 52, 123], [117, 153, 51], [37, 97, 53], [172, 26, 51], [78, 34, 66], [40, 88, 36], [70, 175, 101], [164, 47, 124], [238, 182, 193], [229, 182, 205], [36, 170, 68], [176, 238, 119], [24, 61, 227], [90, 251, 202], [137, 29, 208], [138, 29, 61], [128, 155, 152], [112, 11, 138], [23, 45, 31], [87, 76, 73], [60, 145, 40], [173, 96, 129], [206, 209, 104], [23, 3, 210], [202, 121, 92], [57, 98, 233], [138, 168, 61], [77, 162, 131], [73, 50, 54], [99, 73, 131], [89, 60, 25], [192, 141, 102], [187, 175, 109], [97, 253, 207], [107, 96, 182], [49, 213, 226], [37, 123, 69], [6, 158, 252], [169, 57, 176], [90, 153, 210], [153, 160, 138], [67, 168, 105], [173, 26, 203], [209, 229, 31], [188, 11, 182], [26, 179, 111], [142, 138, 159], [253, 234, 102], [88, 41, 46], [182, 73, 182], [13, 225, 247], [225, 25, 237], [144, 113, 229], [104, 124, 191], [227, 205, 191], [208, 245, 81], [216, 136, 212], [18, 247, 73], [223, 10, 91], [131, 17, 40], [182, 223, 7], [36, 66, 172], [56, 50, 232], [52, 15, 158], [54, 51, 83], [113, 124, 45], [137, 139, 47], [237, 63, 192], [33, 15, 118], [35, 199, 113], [242, 82, 227], [30, 113, 73], [18, 65, 220], [73, 141, 95], [137, 225, 60], [190, 16, 85], [7, 219, 250], [155, 175, 55], [87, 35, 148], [176, 105, 92], [2, 40, 34], [80, 173, 237], [189, 249, 179], [171, 217, 165], [137, 153, 207], [234, 164, 6], [159, 124, 86], [120, 33, 86], [131, 187, 28], [38, 0, 132], [65, 169, 77], [136, 224, 233], [91, 95, 157], [221, 122, 161], [110, 149, 236], [2, 95, 134], [156, 192, 35], [175, 162, 215], [188, 237, 191], [56, 32, 101], [121, 204, 226], [254, 111, 244], [255, 162, 118], [61, 246, 61], [218, 208, 185], [115, 151, 193], [101, 90, 48], [67, 226, 32], [163, 242, 243], [213, 54, 19], [74, 194, 81], [192, 120, 214], [223, 199, 185], [157, 108, 230], [149, 131, 252], [89, 127, 222], [133, 99, 250], [211, 41, 169], [252, 180, 233], [150, 241, 47], [130, 126, 30], [29, 215, 199], [235, 75, 239], [83, 69, 33], [37, 41, 50], [41, 195, 233], [254, 13, 210], [124, 65, 167], [142, 18, 217], [64, 213, 155], [7, 152, 185], [179, 246, 15], [182, 26, 183], [110, 11, 144], [24, 246, 206], [241, 167, 169], [143, 163, 128], [143, 64, 22], [66, 59, 188], [183, 189, 60], [84, 148, 178], [25, 245, 21], [35, 106, 161], [86, 191, 139], [148, 10, 132], [66, 3, 137], [230, 32, 95], [185, 89, 161], [161, 210, 13], [97, 20, 46], [79, 235, 136], [125, 144, 227], [86, 110, 91], [155, 134, 82], [61, 250, 21], [99, 252, 150], [27, 158, 12], [77, 179, 148], [170, 221, 180], [145, 166, 79], [123, 21, 147], [163, 50, 251], [11, 1, 195], [174, 43, 48], [122, 196, 197], [253, 83, 13], [54, 120, 29], [116, 202, 171], [172, 144, 206], [170, 79, 169], [115, 28, 233], [39, 117, 119], [57, 76, 92], [182, 194, 206], [219, 253, 249], [99, 23, 120], [49, 92, 51], [145, 66, 119], [140, 75, 48], [114, 110, 209], [67, 154, 205], [125, 203, 238], [157, 234, 17], [67, 135, 32], [242, 96, 143], [79, 202, 22], [61, 125, 213], [105, 115, 9], [14, 8, 235], [118, 57, 203], [180, 92, 193], [242, 175, 198], [213, 83, 19], [244, 6, 50], [177, 150, 115], [82, 200, 190], [165, 34, 175], [71, 33, 249], [248, 27, 239], [79, 152, 82], [249, 142, 66], [96, 59, 253], [70, 213, 98], [24, 254, 88], [21, 142, 132], [65, 173, 234], [166, 42, 115], [167, 47, 175], [75, 59, 145], [146, 81, 211], [4, 230, 43], [228, 110, 82], [195, 249, 128], [73, 135, 226], [19, 2, 226], [166, 64, 71], [4, 144, 202], [209, 254, 159], [26, 212, 216], [243, 27, 137], [9, 156, 33], [93, 166, 156], [196, 44, 71], [190, 54, 185], [89, 23, 199], [85, 187, 226], [75, 197, 205], [210, 231, 177], [205, 80, 10], [13, 102, 97], [140, 220, 234], [120, 53, 161], [86, 98, 69], [137, 4, 3], [150, 12, 35], [68, 4, 129], [49, 196, 217], [115, 180, 52], [81, 171, 240], [178, 153, 46], [46, 20, 94], [17, 166, 165], [139, 172, 236], [226, 57, 177], [136, 193, 184], [113, 5, 205], [87, 34, 120], [216, 20, 26], [130, 14, 206], [255, 34, 54], [11, 103, 6], [94, 64, 241], [11, 42, 52], [11, 242, 68], [60, 164, 52], [85, 71, 224], [84, 180, 168], [86, 50, 245], [7, 150, 178], [165, 148, 7], [62, 24, 151], [20, 206, 35], [183, 153, 155], [169, 70, 56], [212, 39, 117], [157, 39, 200], [208, 66, 166], [26, 5, 50], [248, 232, 60], [240, 116, 160], [106, 124, 68], [118, 116, 251], [150, 117, 93], [86, 35, 67], [39, 30, 239], [217, 70, 39], [74, 254, 161], [219, 69, 184], [93, 145, 74], [141, 95, 163], [161, 233, 195], [159, 239, 255], [187, 78, 5], [22, 91, 67], [70, 252, 220], [23, 141, 218], [210, 207, 184], [207, 200, 231], [102, 23, 181], [230, 5, 173], [8, 29, 63], [109, 81, 157], [101, 203, 10], [184, 9, 191], [183, 215, 95], [235, 201, 190], [156, 51, 143], [0, 141, 179], [128, 54, 41], [42, 94, 116], [121, 240, 238], [37, 16, 186], [97, 139, 249], [139, 58, 143], [185, 76, 87], [82, 36, 214], [230, 107, 91], [17, 86, 207], [40, 174, 90], [215, 52, 187], [57, 57, 246], [87, 120, 42], [186, 241, 133], [161, 166, 255], [174, 97, 212], [142, 118, 100], [59, 135, 236], [25, 196, 198], [63, 41, 169], [49, 91, 22], [241, 250, 157], [66, 6, 25], [151, 239, 73], [199, 232, 198], [95, 163, 72], [158, 34, 252], [5, 239, 173], [255, 94, 78], [80, 12, 10], [31, 253, 177], [117, 209, 119], [78, 68, 241], [95, 50, 35], [233, 237, 16], [131, 13, 18], [124, 105, 125], [171, 56, 128], [237, 44, 207], [105, 68, 173], [169, 234, 99], [107, 191, 185], [112, 192, 105], [244, 191, 227], [90, 248, 184], [93, 138, 192], [246, 49, 1], [110, 40, 7], [48, 182, 205], [183, 228, 66], [183, 205, 9], [34, 200, 241], [29, 67, 212], [16, 122, 51], [33, 88, 149], [202, 70, 34], [179, 60, 163], [4, 79, 233], [75, 32, 123], [66, 203, 180], [160, 55, 244], [250, 171, 211], [193, 93, 40], [137, 36, 45], [197, 118, 250], [99, 169, 33], [6, 160, 98], [145, 47, 185], [20, 212, 80], [189, 170, 243], [237, 228, 143], [86, 223, 188], [206, 19, 67], [19, 60, 221], [98, 99, 56], [239, 50, 196], [141, 52, 184], [16, 18, 94], [71, 254, 93], [228, 26, 61], [201, 165, 3], [31, 11, 53], [5, 61, 49], [180, 109, 176], [25, 249, 56], [77, 138, 234], [245, 93, 207], [188, 219, 48], [14, 53, 233], [3, 54, 217], [109, 236, 85], [75, 156, 241], [34, 75, 56], [118, 106, 87], [135, 227, 211], [166, 161, 16], [225, 192, 221], [76, 12, 30], [173, 33, 160], [230, 221, 147], [89, 200, 208], [174, 248, 222], [124, 238, 205], [89, 32, 100], [132, 5, 222], [59, 159, 170], [64, 171, 179], [152, 138, 245], [21, 18, 16], [46, 83, 192], [106, 92, 118], [168, 179, 125], [107, 138, 199], [36, 193, 122], [186, 62, 201], [169, 230, 169], [9, 94, 217], [248, 232, 22], [48, 186, 110], [203, 8, 205], [95, 26, 142], [85, 38, 152], [86, 19, 171], [89, 115, 247], [150, 2, 109], [139, 13, 47], [55, 200, 104], [32, 189, 219], [17, 45, 231], [129, 51, 46], [172, 162, 138], [105, 110, 70], [135, 49, 51], [170, 190, 210], [89, 60, 90], [15, 42, 243], [243, 197, 53], [240, 49, 118], [251, 104, 54], [12, 14, 226], [245, 46, 76], [91, 140, 181], [186, 63, 1], [49, 200, 100], [79, 170, 80], [160, 24, 202], [227, 167, 252], [113, 161, 36], [73, 16, 53], [34, 238, 21], [54, 203, 106], [124, 37, 77], [82, 136, 21], [71, 172, 252], [116, 214, 6], [22, 235, 240], [242, 127, 54], [217, 250, 208], [138, 60, 73], [183, 192, 208], [128, 101, 59], [77, 124, 72], [15, 57, 72], [100, 135, 137], [216, 77, 245], [26, 73, 115], [214, 49, 64], [193, 15, 3], [4, 254, 143], [215, 76, 155], [82, 160, 60], [211, 107, 24], [190, 37, 207], [237, 150, 31], [56, 182, 243], [248, 165, 226], [151, 250, 211], [195, 216, 59], [126, 43, 202], [25, 169, 0], [77, 142, 15], [63, 132, 229], [14, 0, 59], [146, 5, 91], [234, 52, 141], [177, 190, 163], [47, 169, 150], [127, 90, 87], [158, 125, 185], [27, 149, 122], [161, 52, 6], [84, 73, 205], [189, 79, 36], [95, 242, 19], [103, 230, 36], [248, 22, 62], [220, 226, 130], [99, 85, 191], [235, 131, 59], [43, 180, 212], [68, 19, 71], [6, 83, 136], [105, 202, 214], [26, 217, 127], [58, 34, 192], [32, 140, 99], [6, 243, 241], [6, 131, 139], [55, 147, 187], [191, 249, 86], [24, 27, 42], [179, 246, 249], [163, 103, 151], [75, 175, 236], [84, 246, 38], [186, 222, 165], [207, 168, 196], [242, 179, 132], [114, 233, 3], [240, 139, 104], [101, 240, 55], [147, 181, 64], [83, 186, 231], [53, 67, 71], [165, 213, 157], [120, 165, 13], [222, 43, 44], [13, 205, 212], [167, 151, 13], [186, 71, 242], [113, 87, 255], [8, 251, 34], [59, 125, 222], [239, 38, 249], [1, 165, 189], [241, 38, 219], [187, 75, 254], [124, 3, 108], [61, 219, 166], [245, 59, 24], [126, 89, 141], [97, 30, 253], [253, 28, 11], [182, 216, 50], [62, 122, 8], [213, 175, 43], [12, 49, 39], [191, 243, 194], [83, 179, 140], [7, 66, 200], [168, 104, 189], [221, 8, 223], [109, 110, 58], [154, 203, 84], [24, 67, 87], [222, 52, 245], [124, 16, 191], [221, 123, 1], [75, 236, 123], [248, 173, 64], [181, 46, 142], [103, 20, 198], [20, 213, 207], [16, 203, 42], [241, 126, 182], [16, 138, 227], [120, 204, 192], [144, 111, 88], [165, 174, 191], [174, 22, 108], [90, 35, 33], [188, 101, 97], [131, 86, 212], [171, 184, 127], [235, 248, 5], [51, 101, 36], [79, 143, 229], [101, 125, 146], [243, 107, 87], [230, 112, 194], [233, 253, 21], [207, 120, 102], [214, 144, 148], [187, 89, 67], [7, 109, 213], [108, 159, 216], [205, 73, 143], [120, 39, 72], [156, 96, 23], [29, 115, 0], [52, 16, 23], [142, 252, 209], [7, 76, 45], [190, 246, 210], [118, 130, 250], [224, 40, 149], [161, 254, 129], [37, 110, 245], [142, 22, 178], [129, 94, 107], [88, 209, 254], [76, 239, 215], [73, 105, 112], [241, 46, 132], [8, 176, 146], [242, 164, 5], [5, 18, 186], [252, 162, 117], [114, 73, 216], [205, 14, 189], [211, 138, 58], [58, 12, 176], [77, 127, 118], [139, 241, 25], [252, 234, 12], [164, 112, 174], [107, 89, 51], [165, 207, 109], [147, 107, 190], [12, 106, 98], [102, 144, 114], [185, 149, 178], [96, 25, 53], [70, 206, 79], [52, 201, 138], [214, 228, 34], [228, 89, 159], [248, 91, 79], [116, 169, 26], [104, 203, 128], [69, 16, 72], [4, 79, 49], [99, 19, 101], [9, 192, 107], [81, 87, 196], [243, 156, 49], [186, 28, 71], [217, 167, 172], [50, 203, 186], [154, 16, 42], [151, 73, 26], [136, 59, 97], [147, 21, 184], [99, 32, 24], [224, 14, 148], [175, 1, 45], [77, 81, 205], [159, 164, 219], [168, 180, 142], [255, 75, 75], [153, 72, 34], [15, 130, 79], [90, 178, 159], [112, 148, 9], [162, 132, 30], [214, 14, 37], [194, 64, 144], [17, 58, 110], [137, 171, 64], [154, 120, 200], [72, 243, 174], [59, 189, 72], [48, 209, 173], [221, 117, 125], [23, 191, 64], [13, 30, 135], [217, 26, 224], [152, 230, 229], [14, 81, 178], [116, 152, 39], [172, 139, 49], [92, 165, 230], [63, 208, 78], [205, 148, 229], [85, 97, 102], [15, 39, 197], [23, 23, 189], [76, 151, 23], [25, 225, 48], [247, 16, 149], [120, 138, 166], [38, 72, 159], [200, 242, 221], [136, 201, 13], [54, 200, 27], [255, 104, 66], [177, 66, 219], [248, 207, 1], [92, 4, 130], [212, 226, 228], [143, 220, 191], [102, 41, 75], [172, 120, 247], [86, 15, 43], [82, 2, 245], [79, 191, 30], [170, 106, 26], [166, 84, 74], [235, 217, 130], [137, 96, 152], [25, 121, 253], [22, 161, 236], [200, 245, 253], [195, 177, 130], [153, 175, 199], [20, 226, 46], [37, 36, 46], [68, 14, 119], [111, 40, 183], [255, 180, 161], [157, 198, 94], [75, 188, 114], [162, 90, 166], [147, 181, 28], [108, 145, 34], [168, 72, 92], [195, 10, 74], [238, 24, 54], [246, 183, 209], [249, 124, 219], [33, 142, 136], [88, 165, 235], [126, 125, 241], [207, 142, 249], [176, 184, 14], [161, 247, 31], [3, 160, 36], [192, 246, 187], [172, 1, 59], [248, 138, 25], [19, 216, 58], [198, 240, 32], [162, 155, 78], [11, 92, 186], [61, 229, 168], [204, 58, 97], [90, 4, 210], [164, 200, 251], [227, 158, 44], [24, 140, 14], [156, 189, 37], [1, 40, 240], [159, 180, 36], [28, 171, 106], [235, 235, 140], [37, 62, 55], [5, 254, 220], [108, 58, 111], [222, 121, 145], [57, 72, 199], [25, 199, 216], [74, 107, 134], [17, 132, 248], [97, 231, 190], [85, 75, 207], [29, 116, 205], [15, 189, 238], [193, 177, 116], [201, 41, 2], [233, 204, 27], [138, 84, 112], [145, 11, 239], [10, 195, 164], [167, 75, 196], [208, 231, 39], [50, 160, 242], [126, 120, 57], [202, 248, 74], [213, 129, 41], [229, 233, 163], [25, 107, 52], [33, 240, 149], [51, 192, 63], [14, 122, 249], [23, 145, 121], [211, 103, 108], [169, 79, 117], [84, 118, 212], [237, 93, 144], [221, 63, 151], [35, 49, 22], [26, 228, 236], [108, 143, 245], [182, 239, 208], [244, 91, 208], [253, 114, 55], [151, 71, 167], [245, 201, 124], [50, 190, 32], [15, 70, 113], [209, 193, 157], [212, 154, 34], [154, 2, 46], [86, 217, 39], [57, 153, 70], [50, 1, 27], [117, 227, 96], [71, 151, 113], [220, 47, 152], [174, 152, 166], [60, 115, 81], [166, 240, 113], [58, 53, 30], [32, 200, 89], [195, 37, 95], [212, 90, 243], [192, 196, 72], [171, 251, 142], [240, 143, 47], [39, 69, 106], [147, 162, 1], [246, 108, 30], [253, 28, 216], [145, 160, 163], [174, 29, 22], [212, 24, 254], [89, 17, 17], [125, 201, 248], [81, 58, 16], [62, 217, 25], [172, 181, 93], [114, 236, 177], [69, 167, 120], [18, 108, 16], [80, 180, 14], [223, 214, 237], [136, 43, 50], [3, 37, 16], [81, 80, 245], [34, 68, 21], [210, 43, 21], [51, 128, 91], [71, 5, 155], [77, 249, 248], [104, 222, 222], [245, 147, 106], [177, 240, 174], [139, 76, 158], [173, 203, 42], [183, 45, 227], [21, 247, 244], [26, 165, 23], [16, 253, 75], [48, 235, 45], [172, 240, 19], [154, 100, 211], [157, 147, 11], [242, 252, 116], [29, 225, 129], [13, 126, 147], [238, 36, 186], [37, 143, 205], [229, 230, 158], [8, 0, 159], [155, 143, 4], [143, 170, 165], [192, 177, 66], [94, 65, 218], [242, 97, 82], [40, 116, 73], [110, 211, 63], [146, 102, 242], [46, 143, 210], [163, 235, 55], [107, 107, 182], [245, 81, 156], [174, 145, 232], [169, 26, 251], [104, 33, 246], [16, 128, 252], [36, 190, 98], [172, 133, 33], [143, 215, 196], [251, 120, 231], [246, 129, 0], [222, 250, 47], [115, 240, 232], [124, 118, 162], [126, 40, 9], [188, 147, 130], [234, 145, 105], [144, 228, 189], [247, 122, 225], [154, 173, 87], [197, 98, 255], [0, 42, 51], [123, 35, 90], [142, 178, 153], [88, 88, 252], [28, 201, 59], [192, 44, 214], [122, 84, 55], [232, 121, 32], [81, 8, 173], [239, 157, 56], [17, 240, 15], [174, 113, 7], [141, 143, 116], [241, 204, 132], [202, 208, 76], [246, 143, 130], [100, 163, 171], [160, 243, 214], [219, 241, 192], [82, 100, 38], [27, 109, 200], [18, 88, 149], [56, 182, 64], [61, 94, 230], [237, 105, 126], [4, 243, 163], [240, 97, 114], [102, 248, 59], [118, 166, 70], [67, 187, 177], [194, 15, 248], [12, 73, 104], [188, 126, 55], [133, 147, 66], [45, 32, 106], [93, 20, 41], [139, 86, 140], [13, 87, 75], [83, 105, 91], [140, 113, 194], [144, 98, 232], [214, 76, 155], [210, 2, 243], [20, 5, 159], [112, 148, 52], [115, 98, 81], [28, 248, 146], [74, 170, 165], [68, 62, 114], [40, 73, 59], [251, 91, 4], [138, 252, 151], [58, 253, 158], [102, 20, 16], [145, 241, 198], [244, 252, 152], [131, 35, 7], [96, 26, 37], [27, 51, 185], [129, 153, 65], [182, 104, 45], [153, 117, 249], [172, 104, 168], [170, 252, 165], [128, 167, 245], [187, 252, 137], [222, 143, 32], [155, 170, 68], [20, 254, 217], [159, 141, 78], [74, 234, 137], [111, 224, 3], [159, 252, 9], [246, 131, 249], [220, 53, 46], [195, 193, 57], [163, 85, 84], [93, 245, 241], [96, 201, 221], [225, 124, 195], [198, 67, 247], [225, 196, 65], [221, 99, 168], [236, 94, 55], [38, 115, 142], [22, 22, 52], [60, 255, 40], [138, 193, 170], [168, 79, 199], [228, 105, 104], [41, 204, 59], [239, 144, 203], [198, 182, 20], [55, 180, 42], [14, 221, 3], [189, 196, 97], [53, 146, 193], [38, 251, 213], [15, 186, 167], [216, 103, 219], [40, 182, 121], [224, 8, 108], [176, 216, 188], [69, 243, 70], [244, 233, 186], [47, 123, 167], [231, 81, 36], [160, 172, 254], [53, 42, 86], [121, 249, 239], [174, 126, 228], [179, 212, 251], [0, 62, 182], [239, 93, 230], [156, 248, 34], [193, 119, 50], [169, 37, 134], [96, 131, 146], [155, 63, 105], [172, 120, 59], [43, 177, 20], [248, 254, 247], [120, 165, 217], [156, 243, 225], [228, 39, 251], [1, 164, 173], [206, 217, 107], [14, 52, 232], [150, 65, 88], [134, 4, 205], [18, 160, 95], [51, 204, 148], [72, 165, 27], [11, 250, 114], [230, 48, 201], [56, 243, 228], [174, 27, 231], [139, 14, 105], [146, 160, 215], [188, 174, 79], [168, 33, 16], [132, 222, 246], [150, 10, 87], [122, 56, 79], [100, 234, 210], [116, 137, 37], [103, 165, 192], [143, 132, 161], [211, 121, 233], [57, 5, 20], [58, 228, 31], [93, 163, 28], [230, 185, 123], [184, 240, 37], [186, 83, 233], [252, 67, 65], [168, 82, 37], [109, 211, 174], [23, 175, 4], [141, 33, 29], [18, 73, 202], [175, 140, 204], [175, 152, 231], [65, 0, 194], [116, 124, 194], [81, 173, 98], [112, 148, 134], [71, 6, 133], [89, 225, 134], [177, 32, 94], [171, 156, 49], [176, 59, 234], [135, 40, 194], [39, 135, 58], [216, 220, 59], [226, 234, 164], [2, 192, 6], [208, 220, 86], [184, 77, 159], [89, 188, 230], [16, 87, 14], [212, 72, 64], [198, 197, 149], [195, 181, 136], [134, 88, 123], [97, 58, 63], [127, 181, 67], [232, 18, 18], [119, 151, 75], [32, 202, 0], [10, 70, 127], [137, 246, 228], [159, 76, 49], [158, 242, 112], [69, 71, 26], [22, 118, 99], [188, 133, 40], [173, 30, 92], [46, 192, 163], [167, 47, 107], [208, 116, 145], [158, 207, 107], [60, 134, 61], [217, 69, 223], [117, 139, 255], [244, 254, 189], [241, 232, 165], [62, 94, 41], [22, 131, 228], [123, 143, 64], [203, 62, 242], [193, 153, 145], [217, 106, 162], [0, 253, 25], [147, 247, 96], [57, 116, 51], [117, 41, 147], [123, 185, 107], [85, 249, 107], [200, 230, 35], [243, 156, 120], [13, 84, 43], [195, 145, 34], [230, 77, 21], [67, 176, 79], [207, 119, 249], [174, 137, 72], [222, 29, 213], [140, 195, 180], [127, 186, 25], [14, 121, 83], [133, 150, 167], [113, 161, 31], [168, 114, 99], [187, 154, 172], [226, 71, 19], [190, 238, 124], [204, 117, 111], [114, 197, 146], [233, 167, 244], [1, 51, 45], [81, 105, 240], [151, 87, 131], [158, 190, 84], [81, 135, 59], [44, 249, 102], [34, 94, 57], [5, 121, 141], [148, 169, 35], [228, 121, 54], [235, 243, 173], [17, 240, 111], [65, 121, 194], [51, 162, 74], [154, 123, 19], [90, 82, 145], [93, 117, 195], [241, 173, 186], [46, 168, 237], [145, 50, 211], [13, 3, 187], [43, 2, 89], [161, 190, 193], [223, 101, 248], [78, 20, 221], [191, 123, 33], [247, 15, 253], [243, 46, 109], [2, 154, 88], [34, 176, 151], [169, 36, 163], [89, 116, 56], [62, 38, 7], [198, 134, 172], [237, 185, 201], [185, 38, 133], [124, 255, 119], [65, 174, 133], [112, 53, 42], [166, 239, 245], [204, 217, 8], [129, 72, 125], [53, 234, 230], [159, 35, 57], [156, 236, 16], [36, 128, 6], [230, 116, 120], [129, 39, 221], [144, 143, 55], [181, 244, 64], [107, 225, 224], [229, 225, 66], [2, 152, 164], [17, 11, 56], [30, 44, 90], [94, 44, 64], [92, 171, 217], [161, 154, 240], [22, 68, 128], [72, 215, 209], [116, 97, 24], [204, 55, 89], [51, 231, 93], [191, 118, 102], [168, 52, 182], [222, 162, 22], [40, 125, 107], [63, 79, 189], [136, 179, 103], [50, 93, 15], [112, 198, 235], [214, 220, 75], [118, 101, 251], [137, 136, 147], [239, 144, 98], [27, 113, 185], [172, 35, 119], [202, 160, 76], [48, 182, 125], [145, 16, 62], [76, 40, 22], [1, 249, 220], [162, 234, 110], [110, 102, 113], [205, 158, 235], [143, 137, 221], [112, 235, 195], [152, 65, 100], [248, 48, 76], [186, 8, 157], [184, 231, 109], [156, 251, 244], [68, 235, 127], [198, 10, 107], [109, 150, 231], [221, 97, 199], [235, 110, 143], [169, 45, 244], [107, 178, 212], [85, 151, 250], [217, 245, 226], [106, 200, 127], [18, 196, 20], [38, 78, 129], [152, 2, 30], [100, 25, 232], [12, 235, 143], [27, 150, 23], [249, 224, 139], [18, 198, 21], [254, 121, 181], [170, 250, 61], [185, 90, 241], [0, 111, 107], [88, 46, 233], [149, 24, 215], [65, 56, 23], [225, 84, 53], [88, 35, 176], [219, 231, 137], [137, 24, 169], [68, 185, 31], [79, 116, 92], [31, 209, 2], [145, 137, 32], [79, 120, 139], [29, 106, 15], [182, 135, 249], [112, 65, 19], [186, 70, 113], [241, 252, 39], [150, 76, 85], [180, 147, 3], [151, 209, 7], [128, 140, 204], [149, 197, 254], [114, 55, 134], [168, 227, 241], [146, 70, 44], [6, 176, 87], [25, 220, 217], [83, 164, 109], [65, 151, 215], [123, 50, 7], [175, 32, 111], [217, 195, 22], [56, 171, 170], [193, 134, 232], [30, 23, 107], [153, 20, 203], [194, 83, 36], [51, 220, 104], [155, 61, 117], [115, 137, 170], [180, 227, 252], [210, 110, 190], [139, 234, 192], [162, 132, 16], [114, 5, 248], [188, 46, 142], [44, 182, 173], [33, 138, 57], [34, 174, 111], [15, 71, 67], [83, 221, 115], [176, 8, 245], [224, 12, 27], [6, 163, 196], [17, 53, 211], [123, 83, 136], [105, 178, 240], [42, 204, 54], [180, 165, 84], [65, 21, 200], [98, 16, 225], [38, 15, 145], [207, 23, 14], [129, 210, 65], [24, 93, 184], [248, 128, 103], [96, 81, 113], [124, 55, 252], [106, 77, 237], [24, 102, 210], [11, 22, 100], [58, 150, 27], [243, 16, 214], [40, 144, 122], [47, 29, 138], [255, 72, 148], [247, 112, 112], [219, 236, 5], [22, 19, 141], [108, 147, 192], [96, 217, 27], [196, 158, 213], [224, 93, 146], [148, 67, 154], [131, 236, 85], [159, 235, 50], [176, 68, 192], [164, 216, 254], [176, 61, 251], [86, 97, 101], [134, 164, 83], [194, 217, 25], [218, 88, 71], [201, 220, 184], [213, 177, 167], [105, 116, 232], [224, 143, 165], [231, 21, 149], [132, 158, 133], [99, 76, 37], [172, 78, 234], [254, 125, 221], [218, 45, 165], [43, 8, 235], [153, 162, 249], [190, 251, 134], [197, 34, 247], [43, 144, 200], [142, 204, 234], [178, 241, 247], [102, 198, 118], [142, 194, 175], [235, 151, 5], [204, 61, 36], [16, 15, 67], [71, 42, 178], [92, 194, 97], [10, 139, 48], [39, 0, 135], [252, 21, 131], [4, 15, 128], [0, 119, 10], [232, 187, 171], [73, 160, 88], [254, 84, 158], [220, 236, 40], [208, 167, 55], [246, 5, 18], [87, 77, 115], [153, 207, 141], [239, 254, 144], [160, 242, 116], [116, 219, 230], [90, 57, 229], [244, 241, 114], [94, 127, 160], [33, 159, 229], [37, 113, 205], [171, 110, 189], [138, 33, 192], [163, 86, 207], [81, 12, 172], [145, 203, 247], [78, 163, 0], [157, 7, 111], [217, 110, 248], [156, 205, 100], [55, 65, 157], [111, 134, 49], [118, 189, 216], [134, 40, 136], [247, 100, 81], [7, 150, 248], [214, 71, 105], [10, 70, 97], [147, 138, 120], [102, 103, 66], [80, 136, 240], [221, 51, 158], [31, 105, 167], [116, 131, 218], [243, 240, 116], [185, 113, 173], [160, 110, 197], [74, 177, 68], [233, 41, 90], [158, 214, 12], [206, 80, 213], [155, 48, 111], [227, 116, 187], [83, 246, 172], [130, 152, 101], [71, 185, 111], [41, 158, 201], [31, 2, 177], [97, 104, 246], [144, 114, 104], [186, 223, 242], [232, 20, 143], [246, 200, 160], [32, 5, 231], [205, 36, 78], [30, 230, 114], [74, 223, 215], [219, 4, 185], [103, 164, 189], [217, 240, 232], [215, 239, 78], [121, 126, 47], [181, 119, 59], [3, 164, 15], [73, 39, 42], [180, 124, 27], [198, 83, 150], [61, 234, 224], [122, 117, 52], [19, 239, 70], [177, 138, 205], [38, 126, 79], [101, 25, 28], [219, 177, 121], [151, 139, 199], [137, 58, 164], [174, 215, 183], [141, 200, 198], [46, 155, 43], [5, 29, 172], [24, 11, 225], [166, 29, 219], [184, 139, 116], [146, 246, 166], [201, 80, 28], [234, 27, 92], [239, 168, 250], [186, 171, 167], [132, 251, 144], [82, 124, 129], [175, 198, 188], [234, 75, 169], [187, 144, 249], [150, 7, 202], [97, 234, 149], [54, 246, 30], [160, 24, 147], [147, 190, 149], [210, 63, 86], [97, 191, 49], [127, 208, 31], [164, 43, 32], [205, 118, 163], [251, 128, 238], [71, 140, 241], [152, 221, 249], [89, 8, 4], [38, 219, 44], [172, 30, 116], [111, 40, 108], [171, 213, 57], [146, 190, 171], [248, 117, 213], [104, 32, 66], [187, 75, 144], [9, 252, 174], [149, 122, 7], [104, 64, 26], [210, 94, 94], [238, 211, 55], [151, 75, 184], [221, 130, 163], [160, 14, 19], [10, 204, 31], [5, 46, 75], [174, 201, 63], [63, 23, 204], [176, 39, 47], [166, 171, 161], [135, 227, 91], [78, 179, 202], [182, 116, 177], [222, 141, 53], [234, 65, 89], [68, 80, 109], [233, 168, 59], [5, 60, 127], [84, 239, 125], [146, 132, 237], [154, 97, 195], [30, 113, 121], [174, 57, 81], [159, 57, 196], [153, 219, 80], [127, 48, 68], [247, 213, 95], [59, 63, 152], [179, 76, 248], [145, 75, 221], [24, 139, 100], [176, 246, 215], [109, 190, 28], [97, 154, 142], [197, 152, 116], [254, 134, 111], [200, 87, 64], [175, 129, 21], [135, 105, 217], [136, 188, 223], [121, 158, 100], [229, 192, 124], [12, 102, 78], [192, 251, 95], [66, 95, 89], [149, 175, 96], [34, 213, 123], [37, 234, 91], [185, 186, 11], [212, 174, 90], [162, 58, 134], [109, 215, 78], [119, 108, 181], [142, 24, 247], [222, 141, 35], [7, 132, 19], [74, 127, 225], [140, 160, 92], [182, 96, 47], [250, 239, 238], [220, 246, 16], [83, 83, 138], [202, 210, 45], [247, 86, 72], [7, 81, 94], [117, 13, 202], [248, 97, 137], [4, 88, 90], [245, 68, 157], [187, 27, 253], [116, 127, 127], [227, 127, 59], [225, 116, 12], [105, 20, 103], [105, 234, 16], [193, 110, 169], [50, 108, 179], [49, 89, 52], [208, 85, 201], [132, 217, 118], [92, 178, 172], [57, 155, 59], [60, 195, 82], [240, 58, 1], [162, 143, 193], [75, 3, 54], [201, 214, 187], [153, 103, 169], [226, 95, 240], [115, 156, 209], [184, 112, 31], [105, 220, 67], [214, 188, 215], [176, 77, 252], [63, 42, 230], [16, 16, 8], [132, 84, 82], [4, 20, 217], [187, 190, 226], [183, 182, 126], [247, 4, 217], [38, 245, 197], [124, 221, 183], [77, 131, 176], [102, 170, 205], [82, 194, 27], [5, 228, 36], [245, 162, 176], [148, 50, 130], [152, 148, 141], [163, 93, 134], [203, 219, 67], [8, 80, 59], [51, 96, 53], [66, 95, 141], [243, 141, 191], [51, 38, 232], [156, 245, 31], [80, 215, 237], [42, 80, 244], [95, 108, 22], [53, 94, 131], [0, 182, 27], [117, 219, 72], [123, 37, 224], [22, 19, 121], [66, 163, 221], [155, 139, 24], [166, 205, 34], [63, 144, 93], [163, 210, 202], [165, 250, 89], [238, 179, 188], [147, 141, 76], [231, 253, 31], [136, 93, 226], [121, 179, 8], [179, 85, 59], [83, 2, 152], [124, 25, 111], [210, 173, 45], [8, 173, 221], [125, 126, 28], [170, 19, 99], [251, 5, 75], [126, 84, 170], [202, 32, 25], [104, 242, 135], [105, 86, 214], [241, 150, 99], [150, 0, 61], [55, 154, 236], [144, 246, 70], [51, 36, 142], [205, 0, 79], [54, 2, 46], [191, 197, 87], [14, 209, 135], [126, 221, 23], [183, 224, 2], [147, 53, 110], [167, 80, 253], [38, 152, 103], [184, 198, 175], [177, 106, 103], [249, 112, 168], [208, 252, 22], [163, 230, 158], [131, 59, 183], [156, 148, 119], [54, 233, 223], [138, 30, 22], [108, 68, 62], [178, 9, 85], [96, 191, 246], [178, 248, 2], [238, 224, 162], [207, 70, 205], [250, 129, 161], [46, 62, 146], [107, 153, 89], [39, 213, 179], [37, 97, 249], [55, 53, 161], [238, 143, 28], [151, 217, 41], [45, 90, 123], [145, 160, 219], [119, 142, 17], [175, 228, 161], [102, 148, 17], [136, 84, 77], [5, 190, 166], [24, 99, 53], [168, 191, 116], [16, 46, 219], [135, 31, 49], [165, 126, 15], [202, 234, 231], [227, 27, 133], [221, 173, 40], [80, 222, 52], [247, 110, 79], [144, 33, 130], [84, 25, 202], [154, 221, 199], [105, 31, 72], [229, 202, 51], [109, 189, 73], [220, 73, 12], [99, 190, 251], [239, 24, 53], [127, 16, 0], [172, 211, 92], [231, 184, 253], [77, 89, 235], [26, 49, 226], [37, 71, 110], [103, 201, 84], [72, 220, 206], [74, 140, 237], [130, 223, 238], [166, 149, 248], [140, 163, 150], [148, 148, 29], [229, 209, 102], [146, 226, 54], [144, 141, 119], [128, 170, 170], [196, 128, 239], [96, 9, 172], [146, 155, 177], [186, 70, 243], [219, 46, 231], [143, 31, 224], [107, 230, 19], [117, 224, 225], [93, 24, 157], [101, 196, 43], [219, 78, 180], [118, 61, 219], [32, 243, 170], [150, 216, 182], [61, 34, 9], [245, 165, 24], [77, 241, 83], [230, 201, 242], [121, 34, 127], [161, 242, 213], [49, 43, 90], [69, 18, 20], [21, 209, 85], [133, 244, 216], [203, 182, 188], [109, 29, 116], [26, 181, 208], [121, 132, 11], [66, 39, 106], [59, 105, 135], [8, 137, 178], [118, 16, 68], [108, 99, 41], [123, 158, 57], [225, 39, 8], [237, 161, 43], [235, 142, 13], [34, 135, 36], [59, 69, 57], [176, 31, 183], [7, 73, 110], [151, 253, 73], [120, 214, 193], [93, 234, 172], [6, 40, 112], [212, 120, 113], [197, 48, 106], [181, 168, 50], [92, 118, 247], [58, 111, 162], [162, 191, 134], [59, 211, 14], [248, 150, 23], [250, 54, 238], [159, 94, 105], [203, 135, 123], [160, 189, 195], [44, 232, 137], [145, 207, 224], [249, 217, 109], [208, 237, 30], [237, 220, 117], [228, 2, 40], [118, 172, 80], [79, 211, 102], [106, 180, 16], [83, 57, 43], [240, 187, 132], [188, 214, 18], [217, 32, 120], [174, 204, 76], [29, 101, 12], [210, 175, 0], [253, 131, 226], [86, 66, 164], [194, 25, 193], [10, 85, 179], [244, 219, 62], [184, 64, 33], [24, 17, 197], [1, 36, 67], [170, 202, 110], [223, 18, 111], [109, 251, 169], [43, 66, 92], [194, 224, 16], [132, 107, 251], [93, 48, 100], [154, 43, 216], [29, 216, 88], [55, 146, 151], [76, 245, 34], [92, 216, 169], [24, 38, 172], [223, 13, 97], [75, 248, 152], [233, 109, 64], [168, 245, 9], [185, 218, 83], [202, 69, 187], [88, 84, 193], [178, 183, 70], [196, 183, 38], [54, 54, 58], [51, 166, 137], [66, 196, 60], [153, 103, 153], [233, 227, 255], [252, 11, 93], [146, 197, 55], [72, 124, 77], [124, 144, 214], [238, 232, 254], [83, 197, 71], [148, 217, 168], [207, 100, 140], [87, 137, 146], [15, 251, 240], [66, 160, 154], [202, 109, 101], [99, 34, 15], [105, 1, 177], [36, 62, 99], [191, 202, 253], [83, 53, 68], [219, 86, 202], [158, 20, 9], [117, 143, 168], [42, 254, 61], [6, 68, 236], [130, 123, 199], [114, 29, 27], [107, 223, 193], [247, 138, 132], [24, 124, 84], [19, 253, 31], [191, 255, 194], [234, 159, 114], [242, 206, 112], [142, 56, 22], [145, 84, 178], [173, 134, 174], [92, 248, 9], [93, 183, 40], [134, 105, 133], [215, 194, 217], [119, 27, 55], [134, 140, 211], [116, 61, 239], [48, 233, 134], [92, 112, 31], [89, 128, 242], [7, 49, 107], [50, 111, 98], [30, 206, 112], [214, 52, 204], [179, 88, 35], [225, 159, 191], [189, 120, 135], [33, 224, 130], [68, 98, 57], [27, 198, 235], [196, 227, 138], [110, 154, 244], [159, 226, 128], [208, 23, 87], [98, 13, 247], [203, 143, 219], [208, 5, 127], [37, 20, 6], [10, 202, 151], [72, 197, 126], [72, 34, 5], [128, 50, 175], [156, 67, 196], [222, 110, 78], [229, 198, 52], [206, 153, 207], [241, 247, 142], [5, 148, 83], [189, 8, 238], [190, 50, 173], [230, 16, 49], [201, 14, 19], [242, 189, 236], [1, 93, 115], [188, 46, 254], [129, 153, 134], [115, 67, 129], [89, 242, 195], [190, 50, 92], [205, 72, 43], [68, 153, 159], [74, 223, 50], [3, 87, 230], [156, 139, 9], [174, 239, 151], [35, 106, 242], [64, 71, 107], [20, 180, 81], [76, 188, 83], [66, 10, 156], [226, 56, 225], [221, 182, 171], [67, 78, 238], [179, 217, 213], [46, 229, 160], [205, 144, 108], [111, 90, 74], [129, 162, 209], [53, 5, 50], [58, 140, 230], [73, 131, 68], [225, 253, 207], [193, 46, 29], [111, 170, 147], [30, 147, 77], [51, 147, 84], [132, 101, 17], [76, 161, 3], [206, 207, 226], [190, 200, 238], [128, 133, 163], [200, 234, 102], [191, 2, 5], [3, 70, 117], [166, 171, 178], [141, 247, 30], [106, 175, 142], [80, 43, 127], [1, 185, 147], [177, 255, 168], [99, 94, 73], [243, 102, 107], [87, 154, 219], [125, 150, 160], [86, 57, 24], [118, 36, 214], [215, 47, 18], [54, 116, 66], [107, 251, 207], [29, 67, 52], [130, 112, 165], [116, 128, 54], [66, 51, 68], [60, 240, 77], [122, 41, 2], [200, 66, 35], [40, 63, 148], [195, 56, 161], [135, 160, 210], [26, 203, 219], [108, 5, 79], [63, 26, 196], [0, 216, 149], [117, 165, 152], [56, 221, 233], [211, 206, 220], [9, 237, 235], [219, 20, 251], [233, 190, 84], [136, 164, 78], [40, 159, 128], [229, 109, 254], [102, 100, 126], [253, 115, 207], [194, 49, 59], [9, 244, 83], [8, 145, 21], [54, 17, 148], [68, 250, 178], [233, 239, 58], [86, 178, 107], [55, 88, 109], [235, 9, 154], [42, 208, 157], [137, 150, 78], [34, 250, 61], [158, 231, 197], [157, 192, 184], [97, 68, 224], [198, 174, 33], [130, 134, 112], [40, 39, 16], [253, 31, 217], [211, 246, 88], [85, 124, 231], [64, 15, 8], [44, 46, 237], [249, 102, 176], [83, 236, 49], [195, 229, 123], [137, 135, 200], [208, 117, 52], [43, 208, 115], [117, 39, 22], [16, 143, 142], [175, 43, 182], [230, 207, 94], [205, 209, 254], [251, 147, 110], [236, 217, 2], [69, 128, 159], [91, 129, 223], [30, 97, 144], [228, 250, 200], [126, 4, 234], [88, 16, 96], [115, 237, 8], [11, 215, 211], [228, 1, 150], [238, 60, 232], [232, 213, 171], [135, 150, 55], [57, 242, 55], [11, 213, 17], [42, 137, 70], [4, 180, 166], [33, 111, 163], [49, 94, 232], [217, 30, 27], [214, 181, 73], [47, 191, 14], [116, 172, 129], [122, 239, 244], [65, 149, 30], [33, 25, 195], [225, 126, 201], [113, 85, 4], [197, 249, 172], [34, 51, 184], [97, 89, 118], [208, 217, 216], [156, 116, 49], [40, 150, 230], [60, 139, 118], [25, 4, 117], [24, 118, 37], [205, 196, 90], [92, 169, 21], [62, 97, 37], [112, 231, 80], [187, 214, 4], [143, 83, 56], [210, 171, 200], [176, 207, 250], [211, 237, 140], [60, 32, 224], [20, 11, 76], [2, 142, 155], [82, 155, 16], [4, 77, 93], [64, 219, 61], [230, 170, 228], [1, 112, 228], [187, 103, 89], [252, 207, 196], [125, 35, 243], [255, 173, 87], [135, 182, 120], [60, 16, 62], [63, 40, 163], [121, 105, 17], [152, 0, 226], [11, 19, 54], [90, 170, 110], [116, 180, 125], [126, 233, 55], [94, 26, 58], [229, 148, 161], [105, 119, 174], [137, 171, 111], [6, 132, 110], [230, 162, 221], [252, 242, 6], [240, 158, 70], [77, 35, 130], [116, 185, 221], [35, 194, 35], [243, 9, 254], [56, 56, 147], [93, 219, 227], [0, 52, 18], [154, 63, 21], [72, 30, 140], [164, 79, 244], [197, 228, 123], [158, 19, 6], [142, 89, 33], [201, 137, 85], [89, 178, 104], [180, 235, 38], [145, 200, 131], [169, 69, 212], [193, 186, 0], [94, 219, 69], [186, 214, 51], [27, 81, 23], [164, 74, 209], [144, 40, 214], [98, 14, 252], [66, 201, 198], [181, 209, 86], [211, 64, 194], [119, 155, 44], [55, 70, 40], [161, 138, 100], [179, 82, 107], [39, 188, 150], [37, 55, 199], [82, 200, 69], [181, 144, 149], [103, 184, 54], [173, 213, 202], [10, 167, 107], [75, 53, 170], [136, 58, 131], [199, 147, 238], [8, 23, 55], [37, 240, 8], [36, 238, 220], [162, 81, 170], [69, 150, 244], [115, 184, 208], [243, 208, 70], [183, 139, 87], [49, 0, 98], [6, 214, 58], [3, 159, 173], [61, 40, 1], [129, 18, 128], [11, 241, 232], [152, 180, 92], [248, 118, 210], [127, 211, 100], [106, 0, 96], [71, 149, 96], [226, 0, 245], [105, 130, 118], [251, 62, 17], [100, 108, 241], [207, 149, 182], [162, 208, 48], [11, 129, 51], [188, 197, 60], [148, 234, 76], [171, 172, 214], [79, 141, 45], [87, 106, 76], [166, 211, 63], [229, 56, 212], [59, 158, 199], [151, 157, 155], [120, 17, 115], [61, 55, 69], [236, 104, 151], [128, 70, 242], [0, 144, 54], [9, 33, 134], [129, 2, 163], [204, 42, 43], [95, 136, 166], [106, 240, 92], [37, 247, 102], [30, 45, 55], [185, 28, 97], [97, 200, 162], [53, 88, 66], [26, 126, 120], [231, 173, 47], [255, 44, 145], [58, 107, 203], [107, 111, 51], [246, 79, 118], [253, 216, 121], [211, 120, 90], [32, 149, 198], [182, 251, 172], [109, 55, 209], [108, 101, 147], [214, 108, 167], [14, 132, 246], [122, 221, 120], [8, 210, 200], [211, 219, 203], [102, 208, 114], [70, 122, 68], [190, 66, 114], [213, 197, 165], [149, 199, 139], [16, 250, 234], [62, 132, 95], [109, 236, 22], [202, 184, 89], [91, 43, 231], [15, 245, 188], [244, 195, 25], [111, 172, 252], [192, 51, 248], [32, 172, 56], [69, 14, 138], [176, 114, 176], [152, 98, 82], [153, 241, 67], [253, 147, 7], [185, 16, 49], [90, 86, 27], [72, 152, 1], [15, 42, 52], [115, 146, 232], [47, 143, 190], [149, 92, 176], [103, 224, 33], [218, 93, 220], [28, 132, 172], [118, 10, 53], [169, 107, 251], [39, 13, 98], [213, 234, 125], [131, 88, 85], [248, 199, 66], [255, 63, 66], [19, 251, 27], [147, 80, 8], [78, 74, 169], [174, 66, 5], [1, 162, 76], [63, 47, 218], [23, 80, 4], [132, 46, 51], [163, 243, 247], [4, 243, 146], [109, 107, 240], [78, 200, 90], [98, 150, 187], [235, 69, 174], [134, 58, 175], [30, 109, 221], [123, 61, 223], [234, 75, 195], [224, 60, 253], [99, 11, 223], [144, 97, 206], [3, 64, 36], [102, 118, 21], [75, 253, 24], [87, 74, 84], [76, 229, 30], [29, 212, 235], [79, 242, 128], [209, 206, 141], [117, 184, 46], [60, 240, 211], [9, 14, 99], [50, 73, 32], [134, 10, 90], [217, 177, 24], [2, 225, 47], [20, 37, 158], [155, 78, 243], [163, 1, 150], [14, 123, 40], [7, 91, 14], [14, 211, 34], [200, 120, 6], [215, 239, 186], [233, 228, 87], [40, 28, 140], [198, 135, 252], [205, 45, 123], [26, 35, 80], [77, 85, 3], [184, 77, 11], [252, 224, 166], [12, 104, 28], [28, 3, 149], [120, 238, 130], [205, 6, 129], [107, 184, 219], [109, 42, 114], [156, 27, 140], [160, 98, 191], [247, 127, 167], [106, 236, 134], [12, 247, 88], [124, 194, 25], [132, 154, 184], [195, 238, 153], [33, 35, 35], [229, 27, 250], [52, 185, 20], [226, 216, 27], [132, 225, 140], [235, 105, 33], [165, 127, 240], [119, 253, 0], [219, 143, 99], [24, 91, 148], [185, 55, 18], [183, 217, 210], [0, 192, 131], [165, 192, 93], [26, 58, 69], [75, 169, 140], [65, 220, 248], [161, 246, 103], [146, 71, 70], [47, 2, 148], [80, 42, 64], [0, 71, 140], [0, 190, 132], [6, 62, 112], [150, 44, 92], [191, 10, 7], [84, 220, 50], [225, 229, 234], [112, 8, 20], [210, 49, 206], [64, 61, 200], [246, 139, 34], [241, 138, 76], [200, 251, 13], [65, 174, 197], [92, 119, 105], [55, 232, 224], [251, 42, 46], [126, 34, 130], [172, 230, 87], [68, 119, 240], [227, 84, 166], [1, 181, 125], [29, 201, 33], [217, 102, 153], [115, 104, 199], [192, 255, 61], [197, 194, 125], [206, 32, 219], [17, 148, 17], [128, 235, 231], [236, 239, 10], [92, 245, 65], [182, 153, 22], [68, 192, 116], [139, 246, 55], [179, 204, 206], [195, 245, 255], [87, 86, 97], [198, 8, 159], [172, 87, 220], [208, 157, 217], [233, 100, 197], [121, 34, 32], [111, 130, 168], [103, 221, 14], [229, 125, 225], [40, 231, 12], [146, 76, 232], [30, 234, 228], [252, 52, 249], [236, 157, 172], [22, 86, 190], [9, 36, 191], [160, 22, 223], [232, 132, 252], [68, 231, 39], [95, 106, 86], [103, 57, 210], [241, 14, 105], [64, 186, 217], [238, 254, 218], [133, 197, 206], [35, 179, 78], [189, 157, 61], [172, 176, 39], [157, 214, 155], [150, 250, 10], [20, 101, 183], [64, 126, 16], [145, 183, 105], [14, 157, 94], [201, 32, 14], [207, 116, 57], [230, 237, 202], [44, 12, 147], [114, 14, 20], [254, 247, 107], [99, 87, 216], [146, 49, 149], [27, 163, 1], [56, 102, 119], [134, 50, 169], [19, 158, 69], [205, 185, 236], [173, 96, 117], [116, 32, 42], [58, 129, 106], [14, 233, 203], [181, 24, 41], [82, 82, 79], [114, 24, 30], [111, 126, 41], [29, 181, 97], [114, 120, 44], [190, 154, 152], [111, 193, 10], [178, 51, 224], [0, 43, 184], [234, 27, 227], [206, 153, 33], [106, 67, 87], [190, 12, 223], [207, 131, 59], [208, 159, 96], [56, 80, 12], [151, 191, 162], [108, 165, 134], [130, 201, 248], [67, 216, 4], [50, 111, 147], [125, 253, 226], [212, 28, 46], [241, 189, 191], [143, 133, 0], [239, 19, 57], [5, 55, 152], [162, 138, 117], [191, 108, 110], [98, 5, 134], [24, 193, 213], [168, 5, 91], [1, 84, 217], [193, 171, 159], [185, 146, 27], [225, 80, 21], [85, 12, 149], [184, 203, 147], [149, 229, 7], [233, 26, 141], [114, 212, 89], [177, 223, 5], [153, 82, 180], [151, 71, 0], [57, 0, 99], [48, 30, 196], [103, 202, 194], [159, 51, 243], [7, 22, 158], [68, 165, 219], [187, 28, 89], [248, 90, 232], [233, 94, 67], [128, 15, 20], [38, 196, 39], [28, 80, 219], [232, 1, 173], [210, 96, 3], [234, 137, 245], [122, 48, 110], [148, 181, 230], [62, 208, 236], [175, 216, 100], [227, 244, 89], [57, 219, 171], [43, 12, 166], [94, 47, 119], [137, 70, 35], [135, 59, 131], [46, 23, 64], [184, 22, 143], [115, 70, 64], [211, 234, 61], [82, 223, 230], [16, 43, 161], [97, 216, 49], [105, 189, 227], [125, 212, 17], [244, 94, 216], [226, 47, 60], [32, 52, 17], [130, 138, 226], [127, 138, 74], [50, 14, 158], [54, 68, 216], [113, 138, 95], [19, 202, 203], [109, 218, 54], [186, 105, 161], [137, 21, 233], [104, 78, 67], [111, 50, 124], [141, 28, 9], [66, 111, 55], [93, 121, 10], [168, 221, 63], [41, 210, 249], [222, 54, 237], [219, 56, 14], [193, 22, 200], [232, 216, 118], [8, 108, 221], [15, 148, 130], [235, 107, 64], [123, 19, 77], [83, 141, 71], [38, 14, 220], [33, 229, 8], [136, 200, 10], [255, 215, 215], [8, 23, 117], [194, 166, 82], [194, 63, 45], [56, 77, 31], [130, 185, 193], [205, 186, 193], [133, 103, 189], [58, 157, 154], [242, 164, 30], [254, 220, 197], [246, 217, 4], [213, 104, 66], [4, 202, 167], [114, 213, 244], [81, 200, 31], [148, 79, 176], [17, 233, 228], [138, 195, 168], [233, 50, 223], [6, 45, 4], [62, 191, 148], [253, 254, 202], [200, 248, 7], [190, 131, 173], [146, 79, 197], [42, 23, 97], [71, 101, 101], [42, 117, 97], [238, 65, 215], [150, 6, 46], [163, 174, 104], [219, 121, 30], [177, 212, 145], [254, 37, 110], [40, 217, 204], [132, 171, 159], [75, 137, 238], [210, 25, 246], [68, 24, 167], [104, 13, 74], [201, 173, 75], [40, 237, 147], [213, 115, 146], [168, 59, 219], [50, 58, 79], [171, 176, 38], [3, 127, 245], [44, 119, 105], [84, 126, 41], [171, 162, 121], [242, 115, 164], [83, 126, 223], [252, 179, 234], [142, 216, 137], [24, 214, 90], [141, 128, 168], [84, 134, 70], [133, 62, 228], [177, 75, 144], [183, 48, 50], [12, 76, 98], [186, 4, 246], [103, 125, 64], [136, 138, 135], [152, 87, 13], [44, 19, 170], [71, 84, 143], [203, 14, 41], [126, 138, 102], [188, 183, 3], [249, 152, 227], [218, 116, 227], [78, 25, 151], [47, 139, 65], [131, 200, 119], [58, 23, 34], [63, 86, 84], [151, 119, 248], [57, 159, 168], [9, 224, 160], [244, 187, 62], [70, 31, 133], [193, 121, 36], [8, 162, 102], [151, 3, 229], [140, 21, 92], [180, 13, 63], [128, 155, 66], [207, 121, 200], [131, 243, 2], [16, 117, 115], [69, 4, 201], [63, 222, 127], [126, 187, 149], [158, 134, 255], [121, 16, 67], [125, 197, 82], [222, 245, 146], [92, 15, 133], [181, 201, 189], [78, 201, 174], [0, 133, 57], [241, 30, 95], [253, 202, 91], [39, 155, 237], [78, 17, 209], [23, 85, 243], [221, 39, 71], [154, 136, 245], [21, 56, 233], [200, 164, 104], [80, 109, 239], [68, 177, 197], [173, 57, 145], [21, 77, 22], [30, 67, 65], [78, 25, 102], [242, 10, 229], [5, 208, 251], [33, 35, 95], [173, 213, 228], [31, 255, 232], [227, 155, 67], [183, 153, 177], [140, 76, 199], [202, 69, 23], [251, 236, 102], [156, 95, 156], [10, 95, 57], [57, 156, 110], [182, 18, 11], [169, 29, 3], [142, 58, 54], [139, 1, 236], [175, 69, 95], [197, 207, 4], [250, 119, 119], [217, 170, 229], [157, 154, 123], [229, 71, 48], [225, 154, 107], [54, 21, 23], [15, 9, 207], [97, 175, 6], [132, 98, 143], [14, 98, 106], [100, 72, 169], [250, 222, 199], [161, 202, 62], [125, 213, 235], [13, 43, 191], [130, 103, 134], [171, 71, 33], [72, 23, 155], [190, 18, 92], [54, 82, 26], [207, 21, 98], [18, 116, 170], [38, 120, 184], [122, 5, 9], [205, 202, 192], [156, 87, 44], [173, 143, 208], [77, 31, 112], [98, 13, 6], [245, 141, 203], [242, 20, 30], [137, 45, 48], [106, 63, 156], [41, 121, 39], [129, 250, 233], [187, 227, 81], [121, 147, 142], [97, 110, 203], [231, 214, 196], [210, 84, 18], [90, 239, 128], [44, 71, 224], [201, 150, 71], [195, 182, 191], [124, 216, 126], [217, 207, 158], [162, 126, 50], [185, 119, 8], [148, 150, 166], [187, 196, 30], [250, 13, 230], [152, 237, 174], [68, 71, 21], [139, 191, 192], [231, 5, 166], [22, 2, 203], [223, 105, 205], [14, 229, 24], [167, 118, 97], [171, 26, 227], [139, 191, 221], [207, 84, 118], [226, 196, 214], [170, 235, 142], [184, 160, 150], [158, 86, 155], [191, 222, 156], [189, 131, 230], [101, 219, 55], [234, 34, 111], [35, 243, 148], [147, 26, 221], [154, 194, 130], [253, 36, 169], [54, 29, 223], [238, 219, 6], [172, 198, 205], [98, 80, 239], [229, 113, 193], [218, 205, 52], [27, 155, 7], [173, 169, 156], [192, 217, 229], [92, 100, 68], [55, 188, 237], [252, 58, 60], [99, 61, 68], [190, 210, 107], [12, 45, 85], [88, 32, 117], [125, 15, 38], [21, 91, 156], [143, 225, 212], [111, 159, 52], [48, 85, 224], [250, 183, 154], [227, 46, 98], [254, 29, 201], [242, 110, 110], [154, 21, 154], [61, 130, 227], [20, 99, 206], [79, 157, 11], [51, 232, 243], [20, 171, 178], [116, 213, 213], [21, 197, 106], [119, 118, 32], [194, 194, 65], [80, 154, 198], [84, 31, 254], [125, 38, 30], [121, 212, 48], [12, 43, 225], [52, 46, 63], [215, 88, 21], [83, 123, 228], [56, 94, 49], [141, 142, 111], [24, 82, 232], [128, 129, 159], [151, 161, 114], [158, 197, 182], [186, 44, 98], [184, 5, 220], [4, 210, 36], [45, 232, 206], [168, 97, 159], [180, 18, 71], [229, 35, 112], [199, 74, 232], [107, 230, 157], [33, 72, 59], [228, 47, 217], [178, 84, 35], [76, 45, 117], [126, 20, 99], [244, 153, 131], [173, 87, 77], [13, 38, 106], [72, 116, 99], [160, 170, 116], [230, 236, 73], [169, 18, 23], [189, 130, 49], [75, 192, 60], [239, 173, 61], [17, 89, 0], [142, 132, 234], [68, 9, 148], [34, 178, 255], [116, 110, 210], [32, 174, 206], [227, 121, 111], [180, 120, 91], [130, 137, 247], [80, 148, 143], [128, 46, 185], [91, 241, 188], [225, 97, 169], [132, 237, 237], [52, 123, 235], [226, 168, 163], [138, 246, 58], [179, 123, 13], [246, 102, 214], [194, 112, 10], [0, 24, 183], [162, 220, 207], [245, 125, 208], [161, 152, 74], [30, 138, 190], [112, 34, 112], [175, 104, 205], [127, 144, 104], [80, 194, 189], [26, 181, 225], [112, 34, 119], [12, 242, 234], [146, 154, 0], [253, 200, 9], [48, 182, 46], [65, 43, 134], [114, 113, 32], [38, 65, 181], [138, 96, 247], [9, 74, 124], [34, 178, 77], [51, 0, 142], [201, 69, 69], [203, 19, 2], [45, 246, 175], [82, 190, 69], [68, 112, 130], [241, 28, 244], [199, 190, 217], [112, 168, 108], [240, 110, 184], [248, 12, 36], [73, 45, 179], [223, 104, 141], [32, 204, 216], [33, 119, 206], [153, 159, 176], [41, 88, 89], [6, 233, 159], [97, 70, 109], [26, 197, 180], [6, 161, 106], [234, 233, 86], [199, 176, 38], [211, 39, 169], [150, 21, 0], [114, 166, 149], [255, 70, 149], [201, 16, 247], [150, 70, 164], [212, 178, 63], [88, 191, 223], [87, 241, 90], [68, 230, 177], [144, 151, 61], [110, 29, 13], [92, 23, 177], [223, 182, 21], [47, 100, 126], [92, 33, 25], [154, 33, 151], [236, 219, 247], [81, 224, 180], [156, 226, 206], [173, 102, 231], [50, 190, 194], [70, 46, 75], [109, 229, 30], [10, 172, 189], [57, 180, 222], [234, 167, 152], [26, 155, 118], [130, 27, 219], [151, 21, 74], [251, 184, 84], [71, 29, 63], [18, 80, 119], [216, 163, 198], [53, 71, 231], [13, 75, 21], [102, 63, 229], [14, 159, 145], [34, 24, 86], [94, 131, 131], [235, 123, 192], [75, 151, 46], [155, 53, 229], [81, 105, 91], [130, 50, 79], [243, 3, 188], [104, 98, 25], [147, 106, 189], [67, 57, 155], [218, 99, 249], [68, 164, 230], [231, 253, 108], [90, 42, 253], [226, 42, 151], [63, 135, 116], [25, 99, 107], [5, 128, 21], [126, 96, 72], [248, 159, 17], [136, 157, 197], [223, 37, 9], [79, 36, 13], [106, 239, 113], [187, 36, 26], [7, 150, 70], [218, 155, 93], [16, 245, 121], [216, 173, 53], [105, 35, 209], [64, 49, 15], [215, 48, 148], [84, 197, 233], [101, 91, 30], [177, 229, 244], [148, 197, 116], [113, 161, 108], [183, 255, 199], [181, 192, 195], [55, 114, 55], [52, 31, 110], [92, 159, 54], [164, 36, 122], [88, 197, 11], [196, 28, 154], [91, 82, 225], [30, 52, 172], [25, 152, 245], [159, 248, 47], [115, 193, 213], [76, 84, 166], [232, 114, 249], [236, 104, 108], [187, 43, 170], [83, 130, 29], [17, 8, 41], [35, 47, 18], [106, 186, 174], [159, 149, 244], [2, 45, 197], [65, 148, 223], [124, 42, 173], [2, 92, 111], [222, 76, 161], [234, 62, 89], [8, 22, 151], [178, 56, 108], [85, 106, 36], [6, 236, 98], [252, 67, 37], [27, 86, 79], [209, 184, 228], [115, 132, 15], [175, 61, 71], [65, 70, 244], [250, 37, 207], [172, 238, 24], [50, 82, 57], [168, 253, 191], [240, 39, 187], [168, 76, 121], [160, 249, 1], [80, 194, 27], [92, 218, 63], [228, 9, 69], [240, 219, 79], [202, 240, 121], [100, 113, 12], [124, 37, 205], [103, 227, 222], [227, 51, 192], [198, 150, 48], [212, 207, 13], [130, 35, 13], [123, 196, 114], [83, 235, 156], [90, 252, 181], [170, 168, 185], [151, 25, 88], [104, 155, 153], [201, 0, 13], [85, 30, 72], [77, 171, 141], [213, 163, 206], [213, 70, 230], [126, 164, 69], [177, 47, 152], [154, 190, 16], [252, 6, 16], [157, 97, 161], [90, 239, 12], [6, 106, 221], [16, 42, 225], [246, 145, 139], [68, 54, 45], [232, 179, 199], [174, 99, 142], [27, 39, 249], [80, 3, 86], [229, 127, 185], [231, 17, 66], [148, 94, 222], [31, 250, 178], [12, 137, 214], [173, 228, 47], [22, 147, 32], [248, 250, 118], [55, 196, 123], [46, 40, 222], [187, 29, 215], [70, 168, 71], [4, 176, 75], [33, 43, 6], [51, 212, 249], [30, 15, 61], [232, 172, 146], [110, 154, 254], [205, 121, 154], [71, 161, 227], [45, 41, 187], [64, 69, 44], [32, 186, 95], [45, 142, 115], [132, 221, 189], [80, 166, 179], [67, 12, 54], [26, 166, 224], [110, 225, 111], [86, 126, 44], [227, 155, 106], [0, 116, 219], [36, 147, 228], [149, 191, 217], [9, 230, 215], [9, 13, 192], [182, 252, 230], [80, 165, 3], [116, 169, 242], [254, 25, 97], [207, 141, 197], [49, 109, 213], [167, 200, 241], [236, 114, 160], [113, 99, 248], [200, 176, 38], [94, 46, 192], [127, 190, 142], [176, 117, 240], [174, 57, 204], [185, 219, 244], [20, 135, 117], [61, 199, 160], [205, 6, 158], [164, 239, 202], [141, 122, 177], [114, 154, 52], [148, 161, 47], [154, 114, 45], [225, 228, 216], [38, 81, 76], [13, 59, 52], [156, 34, 232], [106, 229, 215], [180, 78, 146], [90, 139, 214], [251, 44, 180], [246, 143, 89], [110, 27, 240], [79, 133, 147], [77, 254, 144], [201, 92, 112], [157, 54, 222], [181, 244, 64], [56, 225, 138], [112, 178, 183], [70, 251, 117], [180, 79, 104], [8, 55, 216], [250, 222, 253], [131, 124, 172], [140, 210, 99], [254, 189, 118], [233, 107, 80], [229, 72, 181], [6, 132, 222], [113, 66, 199], [132, 140, 66], [163, 245, 186], [17, 128, 151], [221, 8, 200], [171, 171, 27], [90, 47, 172], [48, 150, 52], [169, 232, 51], [221, 253, 92], [95, 101, 120], [252, 92, 76], [41, 243, 146], [169, 136, 210], [225, 82, 175], [97, 144, 91], [23, 212, 83], [131, 104, 95], [31, 231, 32], [142, 94, 193], [216, 201, 216], [104, 26, 206], [235, 186, 170], [154, 109, 232], [216, 192, 116], [97, 116, 67], [210, 129, 87], [147, 52, 236], [216, 114, 191], [172, 231, 34], [150, 231, 104], [154, 240, 162], [167, 133, 18], [74, 173, 54], [224, 189, 176], [208, 100, 93], [185, 192, 56], [123, 33, 198], [163, 251, 250], [162, 179, 186], [209, 157, 192], [166, 7, 100], [116, 104, 73], [181, 191, 4], [61, 224, 139], [15, 168, 115], [116, 252, 162], [126, 111, 168], [226, 237, 162], [210, 56, 96], [81, 179, 163], [203, 72, 47], [32, 230, 81], [143, 222, 15], [1, 10, 136], [179, 159, 100], [11, 106, 206], [47, 168, 127], [158, 201, 204], [14, 44, 117], [50, 111, 137], [195, 248, 175], [170, 156, 45], [95, 8, 196], [87, 70, 124], [179, 110, 70], [101, 0, 73], [168, 178, 17], [34, 225, 11], [202, 163, 157], [77, 80, 35], [26, 68, 54], [170, 249, 46], [84, 216, 202], [168, 138, 243], [218, 247, 34], [61, 143, 253], [227, 177, 138], [119, 157, 83], [139, 84, 169], [87, 112, 141], [86, 132, 206], [60, 213, 27], [138, 175, 45], [30, 205, 43], [32, 51, 128], [190, 32, 140], [195, 251, 27], [218, 97, 244], [242, 20, 139], [41, 224, 194], [15, 57, 153], [161, 49, 155], [123, 77, 0], [151, 32, 105], [87, 175, 112], [20, 125, 33], [207, 48, 182], [190, 17, 187], [12, 149, 223], [183, 10, 244], [127, 195, 163], [119, 67, 24], [167, 104, 84], [30, 49, 36], [210, 240, 68], [61, 96, 232], [117, 165, 68], [148, 66, 4], [168, 248, 181], [137, 122, 129], [4, 43, 239], [69, 232, 176], [175, 143, 48], [23, 203, 64], [26, 84, 188], [96, 170, 47], [96, 167, 155], [59, 124, 254], [116, 36, 211], [208, 139, 36], [42, 254, 132], [10, 164, 23], [95, 180, 22], [145, 149, 102], [164, 70, 241], [240, 48, 165], [34, 136, 85], [183, 241, 11], [78, 79, 194], [87, 124, 47], [34, 27, 179], [173, 255, 7], [89, 252, 98], [169, 185, 241], [23, 244, 85], [45, 31, 120], [85, 80, 247], [130, 171, 104], [146, 75, 239], [30, 41, 212], [228, 37, 51], [161, 245, 193], [13, 32, 245], [75, 238, 149], [22, 77, 19], [230, 96, 224], [10, 195, 176], [189, 78, 40], [138, 207, 80], [186, 254, 40], [142, 114, 198], [20, 151, 6], [156, 97, 29], [205, 201, 158], [205, 116, 130], [103, 113, 175], [129, 36, 25], [225, 195, 143], [42, 134, 187], [234, 206, 80], [69, 38, 248], [20, 185, 151], [13, 117, 230], [190, 213, 43], [157, 67, 22], [44, 29, 225], [37, 194, 7], [78, 165, 122], [71, 225, 20], [108, 246, 237], [222, 65, 21], [247, 89, 88], [50, 63, 74], [142, 61, 180], [115, 11, 156], [87, 156, 54], [103, 175, 9], [8, 239, 153], [169, 134, 179], [212, 129, 132], [152, 157, 214], [232, 31, 208], [147, 59, 242], [168, 30, 62], [184, 131, 247], [66, 49, 160], [200, 79, 86], [91, 205, 9], [71, 44, 250], [177, 143, 35], [47, 222, 176], [100, 63, 214], [165, 75, 155], [248, 87, 84], [181, 31, 33], [148, 143, 106], [119, 247, 205], [250, 73, 27], [168, 161, 239], [35, 209, 131], [227, 169, 211], [213, 189, 181], [192, 192, 156], [138, 165, 180], [45, 125, 43], [168, 63, 186], [159, 145, 223], [89, 146, 253], [66, 101, 220], [139, 140, 231], [75, 229, 223], [152, 105, 101], [158, 199, 173], [198, 200, 166], [243, 95, 248], [209, 68, 252], [154, 99, 109], [35, 180, 204], [124, 91, 139], [198, 240, 45], [38, 193, 236], [58, 236, 129], [81, 174, 130], [226, 31, 83], [96, 10, 118], [162, 215, 228], [153, 184, 210], [215, 40, 220], [129, 88, 102], [29, 79, 172], [84, 208, 158], [212, 200, 28], [243, 226, 5], [243, 84, 67], [192, 82, 193], [103, 86, 62], [226, 245, 58], [16, 1, 209], [152, 193, 178], [121, 123, 192], [125, 169, 182], [159, 123, 91], [107, 194, 157], [251, 78, 251], [255, 125, 137], [3, 213, 173], [181, 7, 15], [18, 184, 54], [32, 234, 95], [158, 254, 245], [252, 146, 40], [191, 194, 131], [110, 25, 50], [102, 133, 73], [212, 249, 115], [34, 89, 72], [165, 30, 21], [74, 48, 52], [225, 145, 85], [73, 161, 52], [240, 108, 154], [136, 142, 242], [233, 72, 217], [238, 124, 53], [75, 200, 131], [36, 61, 131], [57, 186, 94], [162, 82, 72], [74, 45, 250], [244, 145, 73], [57, 98, 71], [206, 110, 144], [215, 28, 159], [140, 202, 215], [9, 200, 242], [169, 78, 32], [190, 221, 207], [175, 147, 186], [74, 49, 218], [211, 8, 169], [168, 36, 121], [208, 200, 38], [106, 125, 24], [55, 88, 179], [242, 4, 37], [128, 169, 206], [134, 199, 204], [46, 123, 191], [100, 123, 190], [117, 168, 159], [121, 177, 185], [255, 125, 253], [192, 127, 181], [8, 220, 156], [9, 77, 135], [205, 109, 64], [100, 220, 197], [211, 96, 199], [246, 87, 6], [137, 234, 84], [5, 44, 245], [95, 88, 82], [174, 233, 12], [255, 185, 118], [126, 27, 51], [225, 35, 48], [156, 205, 40], [62, 190, 254], [180, 99, 185], [236, 128, 27], [6, 85, 73], [8, 213, 246], [212, 189, 85], [233, 65, 65], [241, 248, 3], [62, 173, 133], [18, 27, 7], [81, 171, 115], [110, 220, 85], [49, 36, 130], [153, 121, 199], [243, 135, 2], [110, 246, 222], [58, 86, 209], [196, 206, 115], [89, 6, 57], [255, 23, 30], [53, 16, 127], [40, 120, 71], [233, 140, 127], [133, 241, 18], [82, 113, 60], [184, 46, 216], [166, 53, 9], [228, 149, 7], [96, 182, 1], [101, 126, 227], [202, 108, 147], [85, 241, 233], [122, 161, 42], [97, 48, 231], [13, 80, 199], [7, 112, 0], [208, 110, 69], [226, 44, 197], [26, 23, 149], [187, 200, 210], [166, 187, 89], [244, 199, 181], [35, 46, 188], [0, 49, 237], [4, 1, 73], [212, 94, 38], [236, 61, 224], [106, 158, 85], [82, 220, 116], [101, 106, 192], [94, 132, 159], [200, 173, 223], [152, 250, 131], [73, 121, 107], [212, 255, 75], [202, 64, 15], [136, 162, 47], [180, 244, 45], [175, 44, 248], [11, 154, 37], [150, 244, 44], [15, 11, 128], [232, 31, 238], [31, 243, 138], [203, 98, 215], [103, 124, 219], [30, 15, 180], [183, 238, 39], [175, 93, 76], [199, 246, 172], [64, 14, 109], [9, 48, 105], [27, 250, 238], [207, 24, 205], [74, 2, 111], [80, 127, 255], [114, 202, 31], [129, 184, 19], [96, 29, 243], [79, 173, 118], [104, 225, 15], [141, 91, 255], [60, 199, 67], [69, 207, 74], [129, 254, 144], [22, 40, 165], [105, 174, 49], [76, 91, 191], [35, 117, 89], [214, 114, 240], [184, 233, 169], [184, 194, 194], [173, 181, 144], [58, 126, 207], [95, 69, 220], [42, 35, 220], [171, 40, 1], [233, 55, 105], [25, 95, 208], [34, 115, 132], [89, 2, 82], [8, 103, 66], [15, 141, 3], [105, 125, 31], [252, 220, 164], [17, 145, 54], [205, 69, 253], [207, 2, 147], [87, 243, 11], [33, 46, 158], [71, 226, 107], [108, 192, 128], [28, 163, 196], [80, 98, 15], [189, 45, 207], [230, 58, 27], [143, 225, 192], [94, 22, 11], [93, 211, 68], [98, 129, 120], [27, 199, 68], [194, 148, 109], [162, 94, 75], [36, 2, 80], [33, 239, 174], [7, 114, 56], [236, 234, 129], [21, 109, 158], [5, 95, 90], [134, 216, 223], [24, 204, 228], [245, 44, 3], [111, 195, 72], [223, 87, 212], [116, 35, 33], [100, 20, 169], [5, 249, 7], [29, 20, 242], [1, 24, 240], [164, 79, 180], [13, 47, 46], [40, 169, 135], [21, 51, 145], [37, 174, 24], [143, 172, 142], [162, 127, 129], [234, 214, 189], [116, 126, 103], [0, 221, 254], [181, 83, 90], [227, 232, 95], [199, 129, 8], [90, 226, 233], [80, 118, 90], [50, 111, 188], [105, 164, 189], [210, 165, 179], [144, 90, 190], [122, 89, 131], [34, 92, 234], [208, 234, 58], [182, 132, 41], [174, 128, 216], [38, 200, 201], [170, 165, 9], [81, 153, 169], [109, 76, 105], [152, 17, 124], [206, 3, 163], [246, 200, 21], [206, 206, 250], [190, 2, 170], [219, 210, 214], [0, 143, 105], [200, 176, 244], [28, 250, 65], [129, 29, 227], [45, 61, 96], [169, 38, 170], [65, 69, 70], [202, 192, 65], [159, 106, 231], [247, 52, 93], [216, 194, 170], [40, 138, 6], [174, 163, 51], [80, 237, 78], [15, 44, 183], [178, 19, 146], [202, 92, 74], [16, 251, 113], [140, 192, 144], [223, 203, 255], [94, 11, 173], [162, 203, 36], [213, 225, 95], [226, 28, 164], [35, 175, 118], [167, 158, 47], [122, 85, 113], [208, 237, 2], [121, 28, 197], [43, 46, 179], [143, 91, 199], [144, 114, 25], [2, 32, 207], [255, 241, 176], [213, 107, 52], [115, 212, 109], [96, 75, 198], [92, 32, 210], [37, 244, 49], [130, 228, 178], [232, 110, 170], [26, 100, 189], [100, 160, 203], [9, 197, 253], [122, 183, 23], [85, 49, 154], [135, 49, 219], [26, 12, 63], [102, 146, 22], [8, 190, 52], [226, 207, 8], [157, 9, 184], [231, 14, 112], [80, 1, 119], [165, 57, 106], [32, 0, 200], [247, 92, 42], [186, 161, 158], [211, 141, 152], [6, 223, 231], [77, 38, 35], [142, 126, 59], [205, 11, 39], [8, 61, 171], [17, 177, 238], [217, 177, 46], [129, 246, 219], [122, 46, 161], [106, 193, 232], [32, 17, 77], [54, 95, 37], [102, 54, 12], [22, 5, 48], [55, 230, 122], [199, 225, 229], [232, 156, 234], [128, 123, 103], [48, 209, 201], [148, 34, 245], [143, 244, 19], [151, 47, 73], [151, 11, 65], [6, 25, 155], [111, 106, 182], [105, 241, 128], [229, 188, 72], [122, 119, 95], [240, 114, 197], [116, 208, 164], [197, 30, 136], [150, 167, 193], [247, 22, 45], [22, 198, 23], [33, 152, 58], [87, 198, 140], [118, 205, 27], [133, 55, 202], [191, 85, 170], [199, 190, 179], [81, 22, 194], [96, 58, 245], [140, 229, 98], [239, 27, 89], [66, 212, 132], [196, 171, 137], [234, 187, 157], [210, 163, 96], [237, 189, 228], [60, 3, 81], [48, 105, 135], [178, 160, 57], [188, 191, 81], [164, 72, 26], [132, 132, 229], [125, 255, 113], [107, 25, 132], [196, 177, 2], [17, 66, 142], [237, 233, 181], [126, 24, 184], [70, 126, 168], [182, 93, 167], [172, 53, 245], [130, 25, 33], [250, 61, 138], [17, 82, 177], [249, 242, 95], [114, 25, 182], [149, 148, 187], [91, 95, 196], [215, 156, 207], [75, 119, 26], [70, 217, 121], [175, 60, 93], [157, 89, 235], [201, 78, 203], [246, 176, 204], [98, 182, 103], [19, 191, 183], [210, 120, 158], [108, 172, 238], [95, 219, 200], [20, 64, 12], [239, 209, 103], [195, 201, 94], [40, 76, 164], [180, 81, 10], [146, 121, 0], [208, 171, 55], [8, 121, 186], [142, 116, 28], [114, 18, 3], [223, 60, 148], [121, 89, 250], [211, 243, 223], [87, 132, 129], [38, 32, 133], [123, 24, 232], [94, 40, 137], [69, 233, 215], [25, 161, 50], [187, 240, 43], [215, 246, 8], [250, 195, 194], [178, 215, 105], [50, 81, 148], [146, 88, 234], [99, 166, 183], [10, 67, 36], [33, 249, 18], [106, 152, 59], [194, 219, 191], [243, 109, 43], [0, 152, 63], [255, 138, 21], [167, 98, 105], [235, 181, 143], [86, 31, 96], [31, 149, 6], [247, 81, 117], [160, 250, 105], [109, 139, 178], [151, 154, 234], [78, 93, 59], [225, 114, 112], [140, 207, 86], [119, 12, 150], [19, 33, 73], [222, 40, 163], [215, 123, 26], [251, 90, 222], [25, 156, 237], [19, 147, 33], [26, 167, 136], [97, 24, 231], [185, 157, 123], [199, 209, 16], [15, 52, 163], [92, 40, 213], [129, 50, 32], [140, 111, 250], [85, 101, 35], [234, 32, 135], [147, 1, 152], [56, 232, 167], [81, 140, 142], [114, 117, 100], [103, 5, 58], [229, 181, 48], [199, 130, 41], [251, 155, 33], [74, 117, 199], [109, 15, 72], [168, 71, 35], [114, 248, 48], [189, 8, 130], [240, 13, 255], [219, 203, 170], [134, 149, 116], [253, 13, 96], [18, 11, 105], [87, 1, 78], [10, 143, 115], [125, 157, 129], [219, 196, 221], [144, 74, 145], [183, 92, 170], [177, 196, 192], [27, 225, 195], [206, 17, 134], [181, 106, 156], [47, 155, 23], [30, 70, 153], [52, 27, 0], [58, 233, 3], [174, 238, 17], [30, 36, 230], [149, 212, 111], [74, 0, 191], [188, 174, 207], [55, 174, 69], [35, 36, 254], [33, 71, 130], [170, 132, 34], [128, 14, 219], [175, 187, 225], [14, 14, 19], [14, 85, 191], [21, 159, 6], [152, 179, 20], [83, 101, 162], [168, 70, 111], [176, 109, 110], [138, 147, 38], [64, 193, 27], [47, 20, 97], [77, 28, 192], [72, 233, 159], [109, 79, 249], [71, 125, 66], [185, 200, 167], [104, 146, 18], [159, 135, 140], [199, 203, 174], [23, 30, 103], [24, 127, 196], [47, 246, 119], [177, 254, 29], [115, 125, 72], [101, 57, 151], [109, 161, 129], [95, 60, 179], [80, 33, 0], [181, 35, 216], [237, 235, 172], [124, 195, 223], [240, 220, 55], [68, 198, 162], [50, 113, 172], [30, 22, 43], [163, 79, 20], [191, 125, 198], [164, 166, 149], [180, 200, 220], [253, 234, 178], [52, 82, 105], [58, 177, 111], [158, 180, 125], [163, 171, 186], [183, 108, 234], [111, 188, 129], [90, 127, 246], [178, 234, 70], [166, 21, 228], [146, 127, 226], [72, 213, 111], [42, 76, 253], [108, 132, 37], [194, 156, 94], [150, 35, 11], [54, 195, 201], [131, 113, 103], [16, 59, 189], [196, 65, 87], [23, 200, 162], [220, 56, 57], [143, 150, 23], [80, 156, 18], [235, 195, 59], [84, 158, 70], [60, 209, 23], [255, 50, 209], [139, 249, 56], [193, 134, 212], [201, 35, 101], [187, 129, 140], [28, 1, 35], [155, 98, 103], [92, 33, 254], [135, 136, 110], [63, 213, 172], [90, 123, 18], [33, 80, 73], [175, 11, 196], [104, 178, 196], [53, 33, 141], [112, 237, 248], [220, 129, 159], [228, 44, 214], [117, 142, 200], [111, 195, 151], [37, 251, 160], [203, 149, 121], [204, 21, 189], [186, 120, 70], [205, 188, 224], [81, 149, 23], [135, 80, 222], [190, 145, 150], [10, 98, 82], [38, 215, 161], [169, 186, 87], [46, 3, 185], [139, 210, 185], [125, 2, 55], [123, 177, 79], [188, 28, 216], [123, 105, 11], [40, 242, 106], [50, 212, 13], [146, 201, 129], [85, 161, 184], [49, 141, 39], [38, 14, 230], [172, 132, 60], [107, 86, 142], [249, 53, 16], [0, 0, 139], [90, 112, 18], [238, 187, 118], [242, 234, 237], [142, 147, 193], [79, 2, 149], [209, 71, 192], [143, 140, 93], [205, 252, 67], [8, 7, 200], [68, 248, 173], [217, 65, 211], [166, 53, 246], [109, 195, 27], [19, 40, 247], [16, 5, 14], [234, 73, 44], [223, 37, 222], [198, 26, 176], [237, 219, 111], [195, 38, 220], [230, 78, 174], [208, 162, 133], [39, 133, 122], [82, 190, 208], [107, 172, 172], [33, 109, 180], [205, 89, 92], [50, 9, 80], [148, 147, 217], [79, 124, 8], [4, 46, 172], [157, 237, 13], [207, 20, 107], [184, 17, 113], [108, 126, 192], [47, 201, 8], [71, 217, 75], [76, 229, 1], [134, 171, 170], [177, 142, 232], [125, 54, 240], [63, 184, 214], [133, 45, 164], [251, 123, 26], [122, 231, 247], [82, 13, 231], [166, 248, 54], [6, 220, 91], [105, 211, 224], [8, 195, 182], [192, 160, 55], [83, 31, 209], [159, 73, 157], [176, 93, 187], [99, 230, 160], [13, 58, 64], [122, 139, 134], [254, 237, 144], [241, 65, 3], [57, 102, 151], [107, 241, 96], [41, 236, 212], [145, 54, 41], [56, 153, 48], [118, 91, 7], [142, 41, 114], [27, 166, 136], [166, 204, 221], [231, 170, 34], [56, 78, 248], [79, 225, 208], [128, 224, 251], [146, 32, 24], [161, 225, 206], [104, 144, 140], [89, 233, 248], [44, 4, 129], [28, 111, 224], [112, 38, 155], [53, 72, 218], [84, 83, 147], [204, 231, 4], [75, 243, 66], [239, 23, 224], [225, 195, 148], [135, 248, 204], [48, 111, 226], [149, 220, 14], [102, 255, 60], [42, 31, 170], [102, 242, 199], [63, 169, 119], [115, 28, 113], [151, 195, 35], [29, 144, 72], [195, 175, 182], [78, 115, 47], [0, 13, 37], [147, 205, 134], [100, 234, 149], [179, 237, 135], [7, 183, 3], [134, 92, 116], [156, 177, 205], [220, 35, 71], [52, 45, 118], [166, 186, 159], [181, 91, 75], [207, 174, 237], [222, 177, 175], [179, 162, 86], [172, 123, 6], [62, 94, 98], [5, 135, 94], [76, 185, 71], [129, 76, 86], [45, 8, 83], [186, 53, 77], [19, 36, 123], [208, 76, 41], [45, 213, 170], [42, 21, 47], [252, 0, 91], [140, 110, 193], [9, 115, 173], [230, 25, 196], [20, 61, 54], [119, 119, 178], [34, 224, 107], [142, 38, 86], [81, 187, 123], [10, 122, 139], [133, 154, 89], [145, 47, 111], [100, 194, 179], [122, 57, 53], [3, 244, 70], [108, 231, 236], [221, 98, 163], [10, 184, 12], [90, 197, 84], [6, 20, 239], [224, 50, 3], [137, 96, 211], [188, 150, 125], [195, 79, 78], [220, 249, 129], [221, 156, 164], [112, 142, 95], [47, 222, 56], [26, 225, 252], [120, 246, 68], [98, 3, 123], [243, 36, 9], [221, 103, 100], [116, 214, 78], [248, 244, 159], [138, 168, 242], [46, 224, 198], [85, 10, 225], [97, 157, 48], [8, 141, 113], [74, 3, 100], [108, 129, 74], [138, 159, 205], [59, 6, 157], [198, 37, 19], [56, 67, 166], [10, 41, 26], [130, 1, 155], [189, 223, 226], [151, 198, 65], [151, 106, 237], [75, 48, 253], [159, 193, 218], [78, 57, 138], [131, 139, 92], [138, 24, 165], [58, 101, 4], [84, 71, 242], [57, 59, 214], [109, 150, 212], [159, 124, 55], [102, 110, 52], [71, 113, 59], [44, 96, 37], [194, 221, 156], [255, 6, 150], [171, 191, 178], [40, 127, 7], [133, 40, 110], [100, 21, 90], [97, 69, 59], [20, 239, 118], [17, 222, 111], [166, 24, 70], [186, 25, 244], [119, 21, 58], [78, 247, 22], [29, 157, 175], [76, 108, 237], [145, 0, 39], [59, 41, 210], [215, 216, 133], [20, 182, 95], [135, 110, 145], [204, 103, 60], [136, 64, 149], [209, 64, 140], [147, 49, 27], [77, 162, 49], [166, 117, 124], [212, 109, 204], [242, 165, 84], [252, 17, 81], [210, 105, 155], [186, 218, 171], [35, 116, 5], [143, 46, 55], [4, 173, 153], [121, 80, 86], [88, 74, 65], [126, 255, 142], [192, 97, 225], [239, 137, 206], [6, 107, 155], [178, 38, 28], [97, 44, 117], [81, 109, 173], [192, 38, 193], [141, 176, 76], [166, 252, 108], [200, 239, 95], [124, 220, 115], [20, 234, 28], [169, 159, 32], [210, 2, 185], [219, 25, 19], [234, 26, 242], [60, 96, 88], [238, 235, 253], [61, 148, 7], [247, 54, 67], [161, 245, 78], [34, 20, 53], [18, 251, 56], [85, 53, 237], [57, 223, 219], [57, 193, 218], [39, 68, 144], [74, 7, 156], [101, 83, 20], [14, 192, 47], [158, 77, 113], [133, 47, 240], [149, 22, 210], [211, 172, 83], [135, 187, 222], [150, 134, 10], [70, 254, 102], [130, 43, 86], [229, 215, 91], [92, 55, 20], [143, 247, 138], [65, 97, 19], [174, 217, 76], [108, 198, 150], [117, 44, 93], [10, 125, 109], [161, 132, 255], [82, 76, 217], [11, 73, 235], [216, 202, 185], [110, 14, 33], [195, 255, 128], [107, 28, 186], [29, 47, 44], [149, 41, 56], [195, 188, 242], [140, 74, 87], [90, 29, 116], [159, 210, 174], [79, 237, 157], [124, 149, 188], [222, 116, 173], [35, 214, 148], [103, 146, 131], [51, 181, 119], [40, 57, 65], [117, 131, 13], [198, 181, 133], [33, 250, 89], [251, 135, 34], [150, 134, 62], [57, 88, 17], [140, 230, 113], [159, 97, 20], [207, 36, 236], [103, 46, 107], [221, 236, 237], [208, 111, 75], [54, 73, 178], [157, 32, 194], [25, 251, 16], [175, 76, 139], [153, 137, 126], [254, 67, 146], [103, 183, 141], [223, 181, 95], [76, 9, 0], [47, 10, 147], [107, 126, 63], [200, 180, 94], [55, 125, 124], [131, 183, 20], [74, 137, 148], [164, 5, 161], [207, 198, 122], [195, 158, 61], [18, 207, 55], [122, 248, 193], [189, 223, 118], [217, 220, 130], [246, 22, 77], [176, 189, 211], [181, 41, 242], [101, 131, 190], [123, 37, 246], [135, 116, 52], [1, 184, 137], [118, 92, 13], [247, 42, 134], [13, 144, 186], [176, 143, 228], [142, 25, 133], [29, 109, 148], [190, 57, 217], [227, 244, 120], [221, 235, 153], [121, 153, 101], [113, 104, 83], [172, 164, 146], [71, 24, 19], [231, 114, 72], [227, 4, 169], [104, 76, 113], [84, 175, 96], [150, 187, 79], [249, 108, 86], [181, 223, 147], [149, 142, 112], [95, 5, 164], [243, 146, 94], [43, 55, 29], [211, 84, 11], [120, 88, 80], [124, 115, 139], [142, 181, 60], [210, 173, 174], [137, 45, 200], [88, 202, 214], [89, 211, 74], [239, 243, 148], [5, 117, 42], [68, 171, 128], [54, 90, 255], [6, 35, 63], [131, 116, 125], [201, 154, 17], [44, 92, 234], [83, 131, 25], [67, 128, 162], [153, 155, 215], [78, 67, 136], [45, 227, 183], [142, 43, 45], [43, 39, 58], [121, 173, 131], [183, 88, 55], [179, 138, 120], [59, 252, 125], [126, 76, 116], [98, 119, 150], [163, 168, 100], [78, 33, 247], [112, 227, 76], [116, 179, 36], [26, 24, 255], [244, 182, 202], [167, 27, 95], [144, 166, 51], [180, 124, 29], [119, 8, 81], [80, 124, 43], [210, 56, 39], [102, 223, 63], [242, 10, 173], [146, 229, 251], [37, 51, 7], [115, 138, 100], [76, 126, 185], [239, 222, 168], [250, 74, 234], [84, 236, 181], [72, 185, 31], [197, 173, 55], [114, 30, 206], [25, 95, 174], [47, 121, 173], [102, 243, 197], [62, 229, 225], [72, 189, 249], [200, 165, 104], [65, 41, 243], [36, 105, 99], [173, 144, 237], [3, 26, 124], [32, 68, 97], [57, 173, 101], [57, 204, 243], [174, 94, 141], [192, 238, 49], [108, 12, 110], [147, 132, 135], [1, 208, 252], [60, 250, 70], [75, 28, 168], [37, 172, 221], [92, 144, 180], [76, 207, 72], [70, 195, 139], [123, 202, 146], [7, 66, 68], [53, 120, 154], [227, 60, 44], [204, 193, 244], [63, 162, 149], [253, 145, 172], [165, 126, 82], [210, 59, 129], [202, 99, 141], [106, 242, 17], [19, 79, 84], [88, 247, 11], [98, 234, 10], [17, 65, 30], [104, 115, 161], [216, 224, 223], [62, 217, 31], [161, 37, 45], [228, 129, 189], [1, 86, 167], [105, 104, 186], [83, 102, 77], [166, 56, 74], [149, 203, 28], [40, 173, 183], [90, 226, 249], [171, 8, 31], [87, 174, 118], [235, 239, 152], [121, 95, 112], [41, 206, 127], [79, 31, 28], [230, 189, 200], [219, 4, 200], [197, 144, 70], [179, 235, 43], [240, 101, 210], [249, 50, 171], [168, 136, 136], [138, 50, 0], [100, 7, 217], [193, 123, 228], [52, 182, 6], [136, 216, 157], [188, 222, 127], [3, 236, 17], [161, 11, 138], [247, 169, 95], [114, 167, 243], [3, 232, 119], [196, 249, 130], [12, 109, 190], [201, 241, 46], [60, 74, 37], [112, 13, 26], [86, 55, 40], [41, 57, 111], [93, 60, 136], [220, 207, 164], [129, 78, 12], [121, 79, 232], [199, 118, 74], [211, 232, 182], [120, 31, 21], [194, 232, 108], [220, 51, 207], [204, 134, 242], [181, 155, 180], [138, 48, 134], [152, 65, 192], [3, 147, 27], [41, 197, 178], [94, 121, 131], [38, 113, 145], [226, 59, 207], [215, 140, 178], [143, 220, 46], [173, 137, 220], [72, 140, 96], [213, 66, 98], [43, 54, 155], [23, 171, 62], [180, 71, 109], [67, 41, 152], [222, 183, 49], [35, 222, 48], [216, 36, 243], [196, 204, 52], [179, 187, 239], [213, 185, 198], [9, 65, 55], [181, 232, 85], [239, 167, 93], [62, 111, 112], [18, 62, 151], [151, 140, 35], [252, 207, 40], [60, 246, 24], [112, 230, 22], [62, 103, 237], [205, 24, 242], [192, 206, 161], [117, 93, 57], [162, 60, 133], [126, 253, 194], [232, 138, 85], [83, 69, 239], [179, 59, 88], [69, 176, 125], [6, 23, 129], [233, 1, 218], [7, 123, 149], [121, 45, 63], [82, 117, 119], [119, 126, 90], [42, 31, 73], [9, 96, 191], [74, 20, 95], [21, 227, 206], [92, 76, 117], [51, 167, 246], [20, 110, 26], [102, 242, 205], [150, 45, 60], [20, 250, 40], [8, 211, 15], [203, 237, 24], [135, 157, 7], [246, 149, 140], [75, 173, 8], [98, 85, 248], [154, 182, 202], [11, 149, 113], [227, 73, 116], [199, 86, 32], [231, 114, 46], [243, 146, 194], [119, 97, 58], [117, 99, 224], [232, 77, 21], [150, 142, 106], [225, 138, 160], [34, 87, 142], [60, 183, 46], [228, 9, 157], [17, 170, 158], [147, 8, 194], [220, 129, 130], [53, 35, 132], [173, 1, 146], [29, 93, 229], [101, 50, 115], [127, 22, 25], [13, 200, 73], [157, 165, 151], [254, 55, 136], [203, 251, 7], [129, 55, 67], [140, 4, 219], [113, 208, 6], [177, 123, 80], [35, 112, 32], [55, 238, 60], [198, 36, 124], [70, 95, 196], [249, 243, 154], [111, 198, 83], [249, 177, 106], [128, 73, 8], [95, 140, 249], [164, 113, 28], [235, 75, 179], [78, 208, 157], [67, 16, 171], [211, 68, 198], [53, 144, 119], [26, 251, 194], [160, 137, 163], [173, 183, 194], [148, 187, 49], [0, 189, 183], [142, 189, 59], [227, 96, 219], [207, 67, 252], [39, 231, 40], [59, 132, 54], [207, 238, 208], [222, 56, 124], [48, 212, 241], [56, 133, 175], [87, 29, 218], [31, 231, 88], [82, 170, 244], [112, 144, 141], [133, 155, 241], [163, 132, 100], [14, 95, 233], [183, 97, 138], [15, 181, 211], [131, 225, 126], [134, 165, 134], [38, 227, 195], [46, 27, 178], [65, 131, 8], [149, 32, 213], [119, 222, 10], [173, 252, 50], [34, 141, 57], [110, 226, 207], [144, 95, 117], [192, 187, 118], [44, 243, 251], [66, 73, 184], [119, 55, 130], [9, 219, 100], [80, 144, 129], [254, 72, 12], [95, 244, 97], [108, 235, 77], [245, 21, 106], [76, 124, 216], [97, 136, 87], [64, 104, 55], [83, 206, 56], [76, 11, 255], [20, 206, 214], [112, 192, 19], [59, 21, 154], [59, 65, 252], [100, 13, 36], [26, 110, 57], [35, 197, 99], [94, 49, 130], [188, 34, 82], [227, 67, 165], [59, 75, 114], [127, 99, 174], [112, 187, 198], [52, 198, 227], [140, 227, 96], [76, 240, 12], [190, 20, 176], [39, 209, 43], [171, 142, 146], [82, 91, 146], [145, 2, 215], [177, 207, 80], [152, 62, 58], [105, 83, 130], [20, 184, 182], [85, 171, 2], [208, 190, 184], [233, 23, 239], [102, 64, 68], [136, 81, 92], [195, 5, 244], [145, 189, 138], [66, 75, 207], [149, 18, 142], [153, 119, 28], [206, 67, 175], [30, 85, 142], [147, 228, 123], [185, 20, 151], [158, 223, 174], [71, 142, 161], [254, 63, 73], [0, 157, 171], [226, 159, 217], [145, 78, 245], [53, 244, 155], [71, 208, 89], [87, 143, 157], [127, 179, 29], [124, 206, 24], [174, 104, 15], [135, 245, 175], [154, 40, 160], [171, 88, 138], [49, 92, 205], [225, 246, 153], [247, 155, 202], [204, 136, 49], [250, 19, 177], [151, 187, 230], [207, 36, 186], [148, 168, 159], [253, 186, 55], [231, 148, 170], [161, 131, 227], [5, 41, 17], [233, 114, 171], [140, 34, 52], [235, 86, 10], [147, 98, 169], [53, 236, 182], [148, 129, 67], [236, 173, 53], [237, 244, 43], [164, 26, 121], [85, 151, 173], [84, 253, 150], [114, 248, 178], [231, 168, 69], [246, 212, 116], [252, 154, 112], [216, 215, 40], [187, 55, 2], [165, 77, 10], [244, 149, 177], [214, 160, 218], [52, 202, 106], [216, 20, 218], [69, 28, 94], [197, 59, 18], [233, 127, 222], [240, 12, 189], [34, 184, 194], [143, 249, 75], [96, 154, 177], [130, 181, 24], [83, 232, 32], [93, 71, 85], [253, 7, 31], [208, 40, 137], [32, 46, 179], [164, 221, 4], [146, 32, 83], [168, 122, 142], [212, 242, 206], [122, 34, 248], [72, 41, 65], [73, 246, 232], [141, 204, 250], [139, 73, 130], [77, 236, 52], [118, 174, 183], [45, 161, 98], [57, 198, 91], [168, 185, 55], [222, 12, 143], [211, 149, 255], [244, 225, 176], [239, 38, 88], [59, 86, 106], [17, 30, 191], [170, 81, 200], [243, 50, 238], [75, 29, 244], [185, 158, 188], [144, 104, 26], [167, 255, 187], [129, 116, 13], [94, 1, 71], [245, 6, 175], [106, 36, 77], [42, 96, 14], [111, 26, 158], [25, 88, 210], [223, 11, 148], [143, 51, 32], [209, 240, 48], [18, 238, 127], [29, 254, 63], [228, 55, 226], [15, 241, 209], [56, 111, 96], [30, 196, 147], [43, 246, 209], [247, 200, 245], [219, 230, 42], [64, 134, 134], [45, 22, 49], [114, 94, 142], [158, 252, 91], [185, 141, 101], [114, 11, 255], [243, 42, 84], [232, 88, 187], [225, 174, 7], [232, 65, 143], [6, 73, 64], [55, 252, 105], [130, 207, 252], [28, 28, 252], [127, 4, 166], [91, 215, 214], [157, 40, 125], [121, 18, 124], [168, 87, 95], [49, 161, 125], [136, 179, 100], [53, 112, 117], [101, 169, 136], [90, 75, 236], [170, 175, 222], [229, 173, 118], [125, 86, 124], [197, 4, 205], [11, 171, 109], [30, 70, 5], [132, 89, 112], [146, 54, 10], [95, 209, 55], [38, 124, 166], [213, 17, 191], [215, 28, 84], [179, 136, 34], [7, 59, 106], [119, 190, 107], [233, 160, 179], [109, 199, 167], [79, 213, 210], [139, 252, 186], [92, 149, 120], [131, 215, 86], [156, 42, 53], [3, 129, 64], [167, 105, 25], [132, 221, 197], [108, 131, 24], [100, 122, 10], [144, 239, 52], [139, 235, 102], [245, 104, 246], [43, 202, 93], [218, 116, 135], [232, 239, 124], [99, 84, 96], [211, 195, 53], [171, 178, 170], [135, 172, 236], [139, 168, 106], [216, 71, 104], [38, 170, 118], [208, 178, 8], [191, 3, 199], [131, 23, 193], [130, 97, 1], [157, 116, 127], [197, 9, 218], [125, 110, 118], [19, 163, 137], [56, 14, 91], [19, 154, 123], [149, 202, 137], [19, 194, 52], [206, 102, 164], [25, 127, 255], [67, 86, 233], [229, 71, 147], [36, 74, 12], [175, 97, 179], [125, 34, 75], [196, 217, 156], [240, 210, 205], [68, 181, 18], [62, 77, 16], [196, 210, 126], [79, 81, 170], [140, 134, 199], [243, 119, 85], [190, 1, 118], [178, 195, 131], [42, 34, 89], [86, 252, 236], [123, 115, 168], [86, 98, 124], [52, 4, 102], [68, 100, 55], [227, 141, 233], [113, 236, 156], [69, 26, 130], [111, 82, 241], [89, 177, 76], [74, 229, 241], [73, 127, 37], [28, 127, 165], [98, 84, 255], [206, 120, 122], [132, 73, 64], [22, 78, 60], [111, 161, 173], [201, 209, 64], [128, 171, 249], [100, 149, 103], [174, 172, 187], [153, 119, 86], [132, 179, 215], [244, 251, 114], [128, 187, 46], [130, 252, 142], [130, 93, 241], [160, 245, 36], [85, 44, 129], [255, 158, 141], [73, 109, 252], [56, 247, 59], [49, 195, 25], [177, 66, 148], [66, 105, 196], [48, 6, 159], [77, 74, 231], [230, 225, 58], [73, 16, 4], [110, 133, 246], [127, 253, 62], [130, 105, 182], [6, 248, 248], [200, 150, 148], [206, 247, 135], [70, 109, 207], [43, 212, 194], [199, 10, 72], [117, 160, 158], [235, 69, 14], [8, 160, 161], [45, 147, 90], [175, 226, 70], [31, 16, 255], [168, 5, 64], [193, 94, 105], [115, 143, 65], [174, 67, 79], [157, 207, 140], [204, 239, 199], [246, 227, 13], [21, 191, 9], [217, 53, 32], [64, 156, 80], [174, 126, 148], [217, 147, 220], [213, 131, 57], [129, 70, 153], [17, 20, 138], [189, 140, 161], [151, 36, 12], [69, 57, 110], [241, 45, 240], [148, 33, 252], [159, 234, 155], [228, 41, 57], [16, 92, 140], [242, 208, 149], [86, 115, 69], [244, 83, 95], [231, 115, 9], [71, 112, 152], [1, 245, 227], [223, 15, 92], [136, 186, 132], [199, 178, 48], [223, 106, 159], [228, 28, 231], [104, 8, 201], [214, 7, 215], [102, 29, 55], [79, 140, 117], [225, 245, 208], [100, 163, 193], [203, 130, 175], [173, 12, 96], [77, 60, 45], [72, 220, 162], [229, 74, 173], [235, 60, 96], [64, 77, 44], [214, 99, 190], [115, 138, 129], [174, 93, 31], [100, 1, 197], [146, 168, 206], [217, 149, 93], [28, 100, 183], [41, 225, 16], [150, 235, 49], [24, 162, 222], [69, 35, 196], [182, 13, 94], [205, 138, 170], [97, 141, 224], [71, 138, 173], [222, 71, 114], [226, 124, 245], [159, 235, 89], [49, 63, 23], [107, 203, 187], [169, 162, 173], [67, 199, 206], [112, 234, 250], [195, 28, 91], [7, 218, 3], [6, 219, 243], [19, 169, 134], [207, 103, 118], [123, 250, 234], [64, 214, 98], [61, 216, 226], [213, 103, 165], [195, 219, 149], [247, 70, 73], [166, 234, 146], [247, 207, 7], [142, 187, 25], [183, 70, 23], [148, 97, 33], [41, 185, 55], [132, 247, 90], [64, 148, 6], [241, 216, 195], [79, 66, 153], [130, 168, 58], [189, 56, 51], [43, 46, 21], [87, 104, 149], [134, 34, 214], [81, 198, 34], [18, 230, 244], [83, 148, 68], [17, 235, 80], [150, 5, 141], [80, 39, 1], [194, 153, 172], [7, 29, 176], [68, 44, 91], [27, 253, 175], [184, 86, 226], [243, 86, 1], [243, 125, 174], [208, 153, 157], [155, 71, 0], [97, 213, 89], [191, 150, 135], [11, 119, 134], [149, 241, 73], [153, 169, 42], [138, 224, 178], [73, 201, 52], [173, 115, 120], [211, 175, 113], [4, 88, 107], [169, 91, 154], [121, 162, 48], [193, 221, 221], [4, 241, 25], [41, 15, 38], [218, 20, 224], [108, 133, 37], [224, 43, 204], [59, 47, 3], [162, 150, 23], [118, 156, 3], [100, 14, 186], [17, 242, 67], [86, 250, 175], [214, 18, 137], [27, 155, 104], [243, 142, 60], [215, 103, 113], [112, 203, 250], [157, 214, 102], [34, 112, 50], [129, 13, 215], [252, 96, 3], [135, 183, 69], [166, 83, 226], [8, 107, 228], [90, 165, 181], [106, 182, 127], [166, 245, 248], [19, 116, 153], [30, 101, 169], [174, 249, 143], [176, 198, 65], [246, 169, 103], [49, 242, 132], [151, 15, 193], [248, 238, 20], [63, 223, 158], [150, 236, 216], [2, 151, 162], [211, 230, 55], [209, 45, 243], [21, 238, 210], [93, 241, 37], [105, 65, 37], [60, 25, 205], [10, 71, 102], [101, 134, 222], [57, 160, 160], [252, 244, 204], [122, 10, 38], [34, 180, 95], [87, 23, 151], [8, 38, 76], [55, 157, 28], [231, 139, 161], [194, 180, 80], [222, 8, 79], [9, 48, 176], [228, 53, 42], [249, 0, 137], [159, 109, 172], [104, 52, 204], [36, 190, 29], [250, 117, 181], [194, 125, 225], [185, 222, 164], [38, 177, 235], [251, 105, 217], [55, 159, 61], [47, 60, 125], [113, 249, 145], [116, 194, 38], [224, 9, 226], [229, 248, 124], [123, 55, 5], [181, 114, 15], [8, 170, 137], [11, 3, 99], [168, 27, 10], [226, 149, 236], [28, 156, 37], [178, 1, 231], [82, 208, 41], [226, 104, 140], [142, 123, 243], [237, 21, 203], [227, 186, 118], [213, 160, 42], [145, 213, 167], [209, 19, 73], [9, 253, 27], [151, 110, 52], [243, 177, 96], [217, 135, 77], [214, 149, 115], [19, 54, 162], [124, 224, 123], [47, 97, 179], [120, 123, 199], [216, 1, 180], [128, 60, 188], [45, 144, 92], [233, 183, 128], [248, 128, 144], [2, 34, 181], [41, 223, 214], [130, 30, 254], [101, 233, 58], [40, 188, 23], [178, 108, 169], [190, 143, 249], [61, 51, 115], [142, 177, 174], [217, 79, 196], [121, 155, 47], [230, 247, 26], [238, 139, 82], [9, 233, 251], [229, 59, 196], [25, 16, 142], [157, 185, 195], [121, 83, 45], [127, 104, 70], [243, 130, 174], [187, 177, 168], [113, 157, 180], [104, 210, 31], [107, 166, 117], [44, 113, 65], [110, 219, 220], [52, 85, 16], [176, 224, 121], [141, 8, 178], [213, 39, 16], [158, 211, 6], [80, 225, 236], [146, 215, 17], [138, 90, 13], [158, 77, 220], [165, 141, 234], [226, 3, 148], [171, 216, 234], [30, 14, 196], [44, 223, 198], [52, 234, 44], [107, 33, 216], [222, 76, 247], [151, 233, 28], [138, 216, 55], [99, 145, 247], [240, 211, 31], [161, 154, 203], [88, 167, 252], [243, 128, 70], [241, 234, 87], [45, 199, 25], [100, 197, 17], [214, 28, 215], [43, 70, 244], [39, 99, 101], [223, 229, 165], [107, 98, 10], [115, 159, 254], [196, 124, 237], [211, 143, 148], [63, 57, 244], [243, 143, 51], [159, 215, 92], [142, 255, 157], [86, 242, 224], [142, 106, 221], [254, 75, 54], [139, 243, 214], [140, 192, 199], [128, 254, 79], [170, 160, 138], [30, 196, 90], [224, 94, 225], [33, 225, 67], [48, 148, 145], [130, 25, 41], [100, 222, 30], [125, 198, 75], [79, 212, 4], [171, 39, 44], [240, 140, 37], [192, 243, 82], [255, 104, 72], [227, 249, 241], [223, 201, 229], [216, 6, 102], [212, 122, 252], [226, 143, 127], [86, 200, 2], [247, 62, 193], [83, 204, 196], [144, 165, 56], [232, 218, 179], [251, 112, 236], [33, 122, 220], [18, 96, 253], [169, 6, 86], [8, 24, 131], [99, 136, 227], [38, 126, 127], [33, 241, 138], [16, 43, 182], [30, 8, 131], [68, 175, 93], [155, 100, 72], [110, 199, 58], [102, 33, 100], [35, 186, 240], [37, 144, 52], [249, 162, 150], [135, 106, 216], [173, 60, 176], [41, 82, 42], [252, 237, 172], [139, 176, 8], [243, 11, 70], [191, 29, 191], [112, 142, 199], [209, 104, 203], [40, 92, 193], [165, 202, 97], [223, 171, 123], [156, 20, 7], [112, 237, 26], [77, 166, 68], [161, 8, 241], [222, 234, 208], [109, 116, 194], [166, 97, 86], [242, 187, 100], [10, 148, 114], [46, 41, 203], [245, 48, 241], [103, 195, 89], [128, 148, 186], [129, 180, 215], [210, 5, 37], [0, 87, 182], [118, 243, 238], [97, 208, 187], [100, 71, 139], [160, 191, 230], [167, 0, 10], [214, 118, 166], [71, 152, 124], [206, 161, 116], [6, 227, 227], [153, 67, 158], [32, 105, 118], [221, 62, 12], [143, 156, 230], [43, 6, 101], [206, 143, 16], [167, 120, 188], [79, 95, 17], [149, 58, 41], [221, 16, 1], [51, 155, 255], [16, 65, 82], [103, 242, 42], [9, 126, 135], [216, 120, 85], [29, 217, 175], [15, 128, 152], [20, 232, 179], [6, 23, 68], [94, 4, 146], [222, 81, 19], [30, 237, 238], [68, 104, 75], [65, 102, 130], [26, 152, 44], [123, 81, 253], [3, 82, 167], [31, 181, 43], [88, 212, 89], [8, 133, 102], [208, 117, 243], [163, 6, 104], [101, 14, 81], [67, 199, 20], [147, 143, 135], [232, 161, 80], [176, 123, 32], [34, 164, 56], [57, 85, 68], [117, 250, 3], [135, 0, 7], [214, 131, 0], [213, 173, 15], [22, 70, 133], [200, 215, 176], [17, 53, 58], [138, 103, 2], [66, 102, 206], [8, 237, 110], [232, 44, 236], [52, 153, 213], [179, 80, 119], [26, 241, 76], [68, 180, 233], [24, 39, 164], [96, 19, 235], [19, 84, 32], [69, 146, 207], [150, 200, 184], [97, 165, 13], [13, 132, 76], [105, 180, 174], [245, 125, 132], [103, 223, 48], [243, 169, 55], [194, 232, 208], [112, 90, 202], [16, 111, 251], [36, 193, 177], [174, 195, 173], [245, 171, 96], [72, 26, 216], [137, 97, 114], [184, 36, 132], [101, 71, 102], [37, 232, 188], [130, 86, 239], [207, 165, 123], [79, 179, 145], [156, 52, 86], [42, 168, 131], [141, 179, 152], [112, 223, 21], [109, 197, 152], [240, 191, 122], [4, 155, 47], [129, 127, 234], [71, 77, 237], [148, 114, 9], [146, 220, 134], [232, 242, 214], [234, 200, 82], [32, 111, 95], [84, 136, 210], [110, 21, 1], [92, 158, 51], [248, 226, 224], [26, 191, 160], [37, 30, 174], [206, 11, 130], [102, 143, 84], [156, 211, 137], [86, 10, 128], [45, 19, 9], [53, 57, 168], [138, 141, 134], [255, 232, 122], [229, 147, 207], [58, 186, 208], [120, 237, 195], [116, 203, 101], [182, 218, 158], [252, 8, 75], [144, 94, 38], [105, 58, 5], [242, 44, 109], [53, 188, 72], [40, 54, 146], [237, 127, 103], [212, 105, 21], [31, 253, 108], [202, 233, 2], [120, 127, 42], [193, 93, 232], [142, 187, 208], [96, 83, 8], [187, 75, 59], [118, 147, 210], [232, 79, 145], [6, 6, 159], [63, 50, 227], [108, 90, 114], [252, 106, 168], [76, 147, 215], [114, 7, 70], [19, 172, 55], [242, 18, 12], [49, 85, 3], [46, 1, 159], [161, 45, 251], [255, 112, 229], [169, 169, 183], [32, 144, 52], [111, 90, 120], [140, 102, 172], [233, 51, 242], [75, 126, 86], [213, 206, 130], [17, 6, 98], [174, 17, 187], [146, 158, 114], [43, 11, 27], [143, 150, 93], [174, 26, 185], [244, 147, 248], [95, 227, 13], [15, 220, 154], [205, 175, 236], [141, 45, 92], [155, 205, 137], [248, 16, 31], [17, 194, 110], [131, 185, 142], [106, 93, 168], [73, 224, 252], [179, 32, 64], [180, 166, 91], [55, 53, 122], [35, 108, 193], [36, 175, 192], [205, 92, 188], [43, 13, 214], [107, 147, 6], [211, 14, 238], [0, 6, 126], [255, 240, 8], [40, 140, 217], [197, 170, 196], [155, 45, 208], [58, 115, 208], [199, 150, 58], [69, 74, 92], [49, 95, 226], [4, 147, 144], [145, 120, 237], [224, 196, 230], [214, 51, 237], [221, 233, 201], [31, 108, 62], [134, 68, 189], [156, 226, 57], [10, 253, 65], [170, 25, 89], [246, 113, 189], [239, 216, 243], [141, 117, 139], [163, 210, 48], [162, 96, 181], [91, 150, 78], [238, 140, 251], [79, 186, 168], [55, 66, 14], [53, 172, 64], [147, 244, 19], [39, 225, 105], [12, 169, 165], [120, 61, 179], [232, 167, 45], [125, 180, 21], [36, 136, 246], [248, 200, 254], [227, 209, 254], [24, 171, 14], [229, 106, 28], [37, 167, 36], [180, 131, 100], [223, 229, 12], [209, 151, 121], [251, 5, 250], [19, 234, 252], [240, 78, 158], [124, 9, 37], [86, 43, 236], [179, 191, 208], [142, 105, 231], [226, 86, 187], [28, 250, 35], [224, 112, 181], [252, 0, 42], [253, 86, 103], [9, 13, 139], [122, 93, 172], [68, 133, 162], [95, 170, 249], [215, 34, 89], [130, 199, 78], [147, 140, 67], [245, 178, 100], [197, 91, 0], [31, 161, 153], [192, 17, 86], [177, 241, 241], [117, 45, 220], [175, 247, 229], [36, 242, 174], [50, 49, 62], [12, 57, 85], [61, 37, 112], [69, 43, 117], [41, 175, 199], [42, 96, 99], [192, 241, 85], [54, 70, 147], [172, 43, 216], [166, 248, 131], [156, 121, 93], [89, 158, 43], [170, 147, 131], [252, 131, 218], [26, 203, 207], [54, 136, 103], [126, 105, 189], [81, 200, 32], [213, 41, 64], [56, 155, 171], [219, 150, 91], [213, 34, 115], [32, 225, 229], [179, 138, 154], [176, 226, 209], [47, 84, 96], [64, 38, 69], [137, 178, 115], [121, 198, 181], [145, 125, 255], [158, 8, 29], [122, 213, 96], [210, 206, 30], [209, 249, 183], [24, 59, 4], [51, 200, 77], [254, 69, 143], [44, 114, 61], [248, 104, 28], [101, 6, 74], [23, 8, 1], [123, 197, 32], [193, 6, 211], [106, 89, 107], [165, 217, 175], [76, 41, 154], [25, 76, 164], [16, 171, 129], [69, 208, 184], [196, 71, 93], [100, 252, 86], [12, 251, 75], [158, 80, 212], [1, 5, 227], [158, 120, 169], [206, 121, 168], [138, 73, 207], [54, 205, 128], [186, 47, 147], [222, 73, 121], [200, 171, 179], [164, 65, 85], [70, 6, 215], [45, 12, 174], [17, 164, 173], [7, 66, 8], [42, 211, 47], [3, 18, 251], [125, 106, 103], [38, 187, 6], [247, 34, 216], [53, 61, 248], [95, 254, 155], [129, 18, 194], [134, 172, 216], [124, 12, 10], [219, 146, 101], [236, 20, 93], [255, 177, 57], [86, 6, 32], [153, 78, 220], [4, 114, 87], [179, 107, 10], [103, 19, 21], [89, 133, 136], [159, 116, 47], [178, 202, 70], [92, 13, 33], [115, 138, 126], [16, 178, 133], [193, 36, 44], [237, 48, 1], [19, 56, 26], [85, 191, 240], [130, 56, 185], [143, 232, 98], [55, 194, 230], [189, 6, 196], [178, 43, 108], [203, 90, 154], [23, 174, 217], [245, 64, 252], [241, 117, 216], [129, 6, 250], [4, 28, 24], [128, 104, 117], [165, 230, 166], [3, 169, 148], [176, 84, 80], [186, 31, 68], [238, 180, 103], [28, 61, 215], [160, 233, 104], [116, 81, 124], [177, 3, 174], [221, 51, 147], [136, 70, 249], [236, 6, 80], [78, 236, 38], [216, 40, 72], [221, 74, 135], [246, 201, 210], [243, 190, 180], [56, 235, 62], [40, 236, 150], [34, 67, 14], [226, 89, 227], [160, 21, 104], [185, 103, 19], [214, 35, 39], [179, 231, 94], [145, 222, 71], [197, 130, 251], [28, 22, 197], [88, 103, 98], [107, 108, 12], [3, 3, 42], [123, 131, 208], [58, 204, 88], [215, 171, 76], [126, 219, 0], [16, 182, 12], [24, 158, 186], [152, 191, 2], [4, 126, 166], [243, 28, 90], [213, 73, 52], [222, 29, 192], [210, 176, 196], [86, 34, 233], [181, 228, 21], [10, 212, 197], [237, 9, 27], [30, 239, 69], [177, 32, 198], [251, 17, 83], [102, 61, 72], [29, 46, 245], [64, 14, 16], [156, 127, 201], [231, 187, 231], [161, 81, 194], [133, 185, 140], [71, 158, 101], [219, 75, 107], [170, 38, 232], [205, 225, 135], [243, 43, 53], [174, 157, 2], [75, 30, 8], [187, 51, 137], [23, 205, 61], [217, 21, 61], [45, 98, 169], [49, 65, 244], [122, 108, 211], [27, 71, 27], [118, 55, 57], [244, 213, 85], [189, 6, 231], [254, 188, 210], [7, 157, 63], [151, 221, 54], [107, 11, 191], [143, 144, 1], [6, 176, 4], [216, 55, 64], [4, 78, 236], [191, 109, 156], [93, 172, 221], [89, 126, 90], [147, 97, 63], [36, 252, 2], [243, 206, 137], [93, 222, 81], [136, 160, 87], [18, 186, 178], [146, 95, 185], [5, 190, 186], [86, 13, 15], [233, 224, 253], [155, 197, 108], [214, 239, 118], [224, 67, 156], [166, 156, 91], [134, 43, 203], [187, 229, 252], [133, 237, 223], [191, 74, 224], [220, 155, 7], [141, 171, 79], [176, 201, 58], [83, 122, 139], [215, 7, 180], [9, 18, 120], [31, 95, 47], [98, 112, 95], [230, 29, 59], [227, 217, 168], [172, 67, 252], [22, 118, 95], [31, 130, 92], [246, 80, 40], [177, 194, 24], [123, 162, 111], [43, 95, 246], [72, 21, 78], [170, 217, 103], [39, 167, 238], [235, 236, 31], [9, 143, 47], [126, 28, 56], [128, 180, 169], [102, 236, 85], [124, 137, 16], [103, 161, 183], [70, 175, 211], [79, 28, 111], [188, 221, 224], [32, 50, 136], [31, 9, 27], [138, 10, 214], [164, 67, 183], [80, 223, 185], [214, 4, 243], [183, 205, 48], [52, 21, 115], [82, 53, 175], [24, 201, 22], [239, 177, 117], [236, 47, 15], [182, 247, 216], [27, 172, 5], [82, 216, 63], [14, 119, 137], [169, 217, 13], [198, 148, 190], [54, 247, 14], [76, 209, 15], [186, 236, 21], [104, 112, 128], [244, 240, 3], [132, 94, 176], [30, 102, 139], [227, 153, 242], [126, 212, 177], [210, 56, 198], [226, 52, 254], [188, 186, 209], [95, 59, 0], [231, 244, 102], [35, 105, 77], [255, 37, 143], [136, 239, 1], [207, 226, 164], [130, 114, 167], [154, 62, 52], [151, 185, 169], [144, 246, 254], [250, 185, 117], [192, 19, 179], [155, 135, 22], [142, 179, 150], [137, 19, 187], [140, 161, 205], [248, 221, 68], [41, 179, 236], [7, 185, 47], [234, 172, 92], [151, 249, 224], [185, 9, 85], [18, 172, 236], [172, 237, 24], [99, 113, 96], [211, 158, 251], [207, 3, 253], [125, 151, 205], [202, 106, 60], [207, 113, 216], [210, 134, 36], [121, 250, 48], [243, 201, 27], [56, 90, 61], [109, 137, 136], [175, 111, 87], [56, 72, 103], [82, 202, 217], [36, 93, 17], [125, 233, 27], [71, 126, 169], [242, 220, 156], [21, 103, 113], [217, 181, 75], [17, 187, 29], [209, 219, 116], [93, 199, 180], [63, 39, 166], [139, 76, 220], [86, 206, 173], [223, 62, 116], [49, 210, 240], [140, 64, 135], [20, 204, 121], [196, 38, 241], [181, 57, 189], [30, 108, 40], [204, 251, 130], [36, 113, 72], [57, 70, 215], [84, 160, 198], [188, 215, 85], [107, 189, 152], [235, 208, 128], [66, 183, 24], [107, 221, 31], [253, 113, 41], [184, 206, 131], [58, 16, 28], [249, 31, 135], [34, 6, 192], [83, 107, 169], [160, 51, 196], [218, 103, 209], [230, 127, 248], [72, 168, 202], [182, 23, 91], [45, 85, 231], [165, 52, 134], [114, 157, 16], [13, 116, 72], [193, 168, 110], [203, 69, 159], [163, 52, 178], [70, 23, 137], [97, 56, 56], [245, 136, 219], [0, 57, 183], [243, 124, 46], [48, 180, 176], [45, 13, 81], [33, 96, 132], [221, 125, 163], [77, 230, 185], [103, 93, 217], [180, 98, 133], [210, 115, 38], [241, 195, 103], [230, 26, 114], [239, 0, 133], [54, 12, 233], [157, 204, 64], [2, 77, 163], [185, 76, 255], [209, 143, 8], [212, 40, 191], [36, 167, 165], [59, 212, 109], [13, 228, 119], [9, 242, 66], [204, 26, 177], [128, 85, 47], [9, 149, 247], [214, 27, 109], [254, 68, 162], [42, 91, 42], [150, 117, 210], [51, 182, 49], [6, 111, 9], [151, 13, 202], [119, 44, 129], [44, 1, 246], [56, 248, 36], [246, 189, 205], [1, 163, 200], [132, 108, 208], [156, 243, 120], [9, 72, 19], [100, 26, 84], [252, 230, 190], [64, 138, 127], [27, 78, 202], [243, 156, 56], [239, 199, 144], [40, 139, 88], [130, 108, 97], [17, 118, 208], [116, 28, 58], [219, 249, 131], [130, 242, 7], [148, 194, 234], [141, 223, 151], [67, 199, 156], [116, 162, 84], [216, 220, 79], [208, 185, 252], [205, 20, 28], [177, 181, 114], [44, 83, 150], [210, 1, 240], [153, 142, 58], [169, 251, 206], [84, 139, 42], [140, 227, 240], [23, 143, 29], [196, 17, 116], [129, 52, 37], [186, 251, 227], [192, 144, 140], [101, 97, 245], [223, 182, 116], [100, 217, 196], [35, 222, 31], [61, 79, 132], [60, 238, 193], [94, 227, 221], [40, 10, 85], [246, 87, 118], [127, 75, 236], [197, 168, 253], [114, 32, 207], [5, 165, 90], [78, 54, 236], [161, 130, 77], [239, 27, 76], [75, 143, 194], [205, 53, 41], [65, 27, 44], [234, 84, 246], [66, 189, 248], [104, 3, 239], [76, 245, 109], [237, 208, 78], [201, 188, 231], [52, 174, 106], [104, 93, 5], [9, 14, 255], [226, 19, 223], [103, 62, 69], [68, 41, 117], [141, 86, 15], [176, 40, 253], [89, 118, 190], [85, 14, 38], [164, 23, 196], [40, 121, 71], [229, 131, 115], [121, 35, 50], [70, 184, 119], [13, 251, 206], [183, 38, 18], [202, 120, 156], [80, 95, 163], [187, 233, 218], [189, 197, 254], [127, 114, 184], [91, 233, 143], [84, 86, 111], [193, 201, 19], [131, 74, 90], [255, 216, 44], [111, 15, 120], [202, 80, 172], [104, 17, 113], [128, 133, 222], [94, 199, 184], [21, 234, 128], [235, 186, 52], [232, 237, 240], [166, 220, 108], [152, 19, 59], [249, 69, 69], [49, 172, 65], [40, 221, 43], [249, 116, 255], [228, 42, 211], [136, 176, 89], [213, 228, 18], [128, 53, 177], [33, 203, 224], [64, 98, 40], [143, 148, 51], [174, 102, 97], [41, 19, 98], [148, 42, 118], [62, 47, 52], [170, 22, 86], [135, 104, 182], [136, 4, 129], [83, 191, 46], [53, 20, 144], [79, 53, 236], [189, 49, 223], [77, 13, 93], [68, 229, 204], [82, 24, 254], [194, 125, 225], [234, 156, 55], [67, 112, 180], [22, 205, 221], [216, 172, 205], [32, 134, 111], [15, 90, 130], [22, 210, 119], [195, 139, 41], [189, 67, 127], [166, 66, 163], [171, 88, 157], [54, 164, 196], [126, 229, 142], [237, 153, 34], [45, 239, 225], [146, 94, 56], [214, 79, 138], [107, 255, 8], [10, 111, 74], [191, 217, 114], [159, 96, 87], [135, 93, 220], [3, 22, 48], [210, 183, 76], [39, 52, 119], [214, 239, 6], [55, 166, 234], [134, 122, 17], [251, 17, 19], [70, 239, 146], [216, 76, 101], [185, 210, 133], [152, 139, 190], [254, 89, 13], [179, 209, 47], [158, 212, 193], [85, 214, 132], [39, 30, 148], [189, 147, 41], [75, 171, 193], [212, 54, 47], [157, 162, 192], [251, 108, 255], [176, 2, 84], [52, 84, 144], [102, 95, 140], [63, 131, 230], [193, 128, 196], [191, 204, 168], [116, 149, 15], [230, 236, 94], [164, 50, 94], [253, 193, 60], [85, 83, 95], [26, 221, 204], [158, 248, 101], [67, 64, 17], [26, 154, 8], [49, 92, 158], [58, 158, 38], [198, 243, 53], [198, 202, 6], [50, 43, 14], [168, 189, 198], [183, 220, 239], [250, 196, 41], [111, 169, 244], [210, 214, 157], [0, 204, 80], [57, 166, 149], [178, 234, 113], [98, 45, 115], [252, 149, 25], [52, 185, 163], [73, 52, 108], [65, 47, 22], [155, 22, 252], [98, 146, 104], [81, 118, 124], [221, 36, 26], [179, 190, 108], [39, 199, 97], [121, 89, 142], [121, 30, 83], [104, 37, 110], [196, 250, 228], [234, 122, 10], [213, 20, 222], [175, 210, 72], [69, 148, 20], [174, 164, 68], [81, 238, 2], [14, 154, 151], [114, 232, 26], [205, 48, 243], [27, 132, 215], [194, 71, 92], [19, 136, 5], [174, 103, 95], [49, 17, 68], [50, 35, 67], [107, 21, 238], [130, 234, 157], [212, 249, 145], [21, 175, 153], [213, 101, 73], [167, 182, 179], [67, 34, 123], [249, 100, 159], [244, 159, 154], [59, 215, 10], [74, 146, 39], [0, 142, 163], [152, 197, 186], [190, 194, 189], [29, 204, 15], [231, 242, 43], [42, 67, 164], [250, 149, 58], [153, 62, 201], [169, 238, 46], [144, 218, 55], [8, 201, 30], [118, 61, 34], [251, 251, 218], [252, 19, 89], [41, 148, 166], [222, 203, 10], [43, 47, 119], [20, 233, 35], [16, 10, 18], [95, 166, 2], [69, 213, 195], [91, 33, 80], [154, 99, 255], [51, 195, 16], [243, 188, 0], [182, 161, 235], [220, 200, 207], [75, 160, 246], [10, 250, 100], [173, 0, 72], [84, 14, 241], [46, 16, 12], [32, 3, 23], [181, 195, 58], [89, 151, 164], [140, 71, 221], [14, 12, 54], [233, 176, 235], [54, 149, 193], [46, 160, 94], [167, 68, 123], [116, 74, 183], [239, 171, 153], [27, 218, 219], [0, 97, 81], [23, 189, 142], [191, 113, 74], [41, 174, 25], [242, 145, 216], [192, 197, 104], [153, 139, 54], [126, 56, 199], [15, 31, 55], [61, 168, 42], [230, 147, 59], [41, 90, 47], [166, 95, 183], [87, 72, 161], [159, 51, 118], [165, 168, 62], [13, 111, 45], [56, 22, 18], [167, 100, 97], [17, 207, 70], [130, 57, 213], [197, 128, 84], [156, 95, 68], [235, 205, 139], [200, 42, 67], [5, 221, 15], [219, 150, 144], [9, 25, 104], [229, 210, 55], [23, 77, 54], [63, 19, 24], [45, 3, 130], [125, 22, 218], [203, 50, 155], [92, 100, 7], [177, 35, 135], [239, 86, 27], [192, 105, 255], [3, 55, 213], [76, 22, 127], [184, 18, 49], [11, 226, 117], [145, 208, 77], [65, 50, 189], [60, 41, 176], [57, 116, 217], [72, 171, 10], [36, 133, 226], [111, 243, 104], [59, 24, 69], [85, 118, 140], [136, 144, 35], [77, 55, 138], [46, 220, 132], [25, 104, 124], [155, 20, 211], [32, 133, 146], [198, 10, 74], [98, 111, 149], [115, 185, 150], [40, 182, 214], [104, 1, 218], [100, 200, 138], [210, 123, 84], [193, 249, 15], [27, 191, 160], [77, 151, 85], [133, 177, 128], [34, 185, 206], [109, 161, 77], [102, 192, 166], [112, 53, 113], [145, 138, 103], [214, 134, 182], [133, 111, 52], [18, 26, 198], [196, 179, 191], [99, 76, 106], [49, 118, 28], [69, 64, 3], [51, 107, 217], [99, 211, 190], [134, 143, 238], [71, 175, 109], [84, 57, 214], [52, 138, 184], [51, 234, 199], [253, 64, 108], [121, 55, 203], [90, 127, 63], [177, 243, 141], [130, 181, 29], [194, 219, 30], [24, 160, 189], [205, 229, 42], [46, 50, 231], [134, 134, 130], [166, 67, 139], [67, 132, 157], [252, 232, 242], [224, 11, 59], [218, 193, 204], [182, 229, 132], [204, 253, 3], [244, 186, 2], [152, 201, 212], [52, 93, 251], [141, 233, 235], [124, 94, 63], [130, 101, 103], [52, 68, 159], [3, 223, 58], [171, 178, 179], [35, 31, 128], [137, 202, 12], [66, 69, 34], [66, 100, 193], [217, 159, 147], [149, 202, 118], [239, 34, 34], [130, 239, 155], [192, 56, 39], [115, 16, 210], [220, 109, 67], [197, 255, 85], [133, 105, 232], [212, 24, 77], [135, 45, 59], [42, 201, 189], [210, 236, 173], [79, 242, 40], [194, 23, 58], [213, 237, 138], [26, 132, 215], [91, 219, 116], [104, 5, 85], [3, 129, 106], [179, 108, 90], [93, 76, 34], [238, 133, 59], [107, 27, 208], [163, 172, 249], [100, 159, 253], [72, 232, 106], [153, 225, 69], [196, 92, 152], [85, 22, 117], [154, 180, 70], [26, 67, 44], [204, 241, 245], [223, 95, 122], [15, 164, 65], [137, 91, 229], [154, 84, 77], [126, 182, 176], [123, 233, 43], [156, 190, 87], [75, 169, 172], [37, 191, 120], [123, 214, 96], [27, 88, 211], [163, 45, 92], [135, 170, 7], [76, 71, 155], [229, 55, 197], [146, 118, 253], [15, 151, 21], [229, 222, 252], [8, 28, 53], [129, 211, 152], [108, 224, 175], [80, 128, 69], [162, 223, 140], [233, 229, 112], [2, 169, 138], [158, 14, 39], [103, 163, 40], [145, 148, 206], [101, 166, 26], [4, 114, 29], [128, 109, 228], [230, 5, 97], [128, 139, 8], [132, 149, 191], [240, 112, 210], [187, 241, 179], [39, 93, 9], [103, 98, 111], [119, 140, 78], [49, 249, 103], [38, 35, 203], [141, 79, 43], [155, 30, 171], [166, 183, 157], [29, 227, 129], [214, 180, 247], [120, 223, 121], [19, 159, 198], [20, 129, 222], [173, 66, 123], [117, 214, 183], [185, 13, 120], [52, 98, 203], [210, 15, 162], [242, 198, 113], [26, 227, 77], [156, 76, 121], [103, 213, 158], [216, 27, 167], [166, 130, 117], [87, 66, 57], [188, 183, 85], [168, 107, 228], [79, 40, 92], [21, 231, 38], [232, 84, 72], [250, 179, 232], [129, 174, 43], [94, 115, 33], [12, 143, 105], [227, 24, 107], [58, 207, 233], [22, 210, 109], [8, 34, 165], [7, 236, 184], [80, 206, 51], [110, 195, 101], [67, 190, 255], [191, 200, 240], [109, 213, 206], [72, 123, 40], [205, 105, 72], [193, 211, 139], [89, 19, 68], [188, 98, 87], [114, 161, 161], [231, 151, 80], [33, 227, 23], [167, 73, 94], [79, 115, 193], [82, 19, 172], [48, 140, 137], [20, 26, 135], [130, 213, 140], [166, 68, 9], [139, 226, 239], [152, 31, 251], [15, 156, 245], [49, 143, 63], [226, 7, 28], [178, 71, 78], [10, 17, 52], [230, 102, 108], [147, 114, 150], [204, 89, 183], [171, 37, 205], [52, 220, 238], [149, 96, 17], [145, 205, 84], [12, 255, 85], [150, 106, 44], [83, 73, 201], [189, 28, 185], [119, 69, 212], [140, 122, 193], [234, 185, 95], [184, 168, 45], [33, 198, 142], [58, 172, 250], [222, 248, 235], [64, 142, 240], [33, 111, 231], [139, 56, 220], [249, 61, 21], [184, 176, 101], [214, 59, 161], [101, 236, 112], [52, 124, 84], [180, 42, 92], [215, 9, 102], [106, 135, 153], [154, 15, 163], [183, 240, 191], [179, 161, 101], [48, 157, 238], [29, 220, 243], [7, 176, 39], [152, 96, 207], [127, 157, 132], [111, 8, 223], [248, 127, 239], [235, 251, 174], [28, 106, 129], [120, 194, 250], [92, 222, 33], [135, 142, 152], [99, 56, 134], [88, 123, 171], [254, 168, 42], [51, 172, 118], [126, 81, 127], [20, 180, 220], [255, 129, 194], [20, 124, 125], [95, 145, 5], [198, 125, 148], [178, 223, 94], [213, 28, 67], [218, 116, 38], [238, 89, 218], [106, 34, 172], [108, 152, 76], [104, 3, 241], [250, 60, 83], [58, 12, 9], [224, 133, 119], [27, 88, 227], [53, 29, 98], [112, 148, 154], [170, 40, 14], [34, 67, 195], [180, 181, 146], [65, 228, 63], [36, 233, 172], [203, 222, 173], [82, 13, 219], [85, 241, 136], [132, 32, 64], [8, 152, 109], [217, 202, 40], [122, 33, 132], [4, 91, 35], [90, 252, 202], [193, 56, 243], [187, 47, 137], [216, 14, 40], [124, 181, 213], [124, 213, 7], [116, 66, 53], [204, 19, 183], [218, 114, 11], [130, 18, 154], [60, 103, 16], [100, 125, 88], [148, 177, 215], [22, 90, 246], [8, 216, 178], [228, 149, 172], [232, 151, 216], [198, 35, 161], [157, 79, 145], [105, 99, 162], [147, 118, 87], [86, 170, 220], [77, 207, 101], [243, 231, 201], [251, 167, 32], [150, 202, 73], [167, 106, 139], [172, 121, 180], [48, 201, 103], [141, 93, 246], [20, 118, 214], [90, 54, 134], [248, 30, 121], [99, 170, 208], [135, 176, 172], [168, 28, 159], [221, 37, 16], [59, 183, 178], [126, 189, 129], [224, 168, 154], [128, 193, 58], [64, 120, 187], [8, 156, 106], [237, 106, 233], [153, 102, 252], [206, 231, 211], [168, 36, 249], [203, 217, 121], [226, 237, 245], [175, 8, 167], [98, 63, 189], [206, 182, 179], [46, 67, 92], [18, 96, 189], [238, 255, 206], [184, 134, 205], [163, 199, 41], [12, 61, 173], [243, 64, 27], [59, 120, 225], [170, 218, 103], [232, 33, 120], [213, 32, 224], [186, 106, 169], [17, 56, 76], [240, 33, 248], [50, 152, 99], [93, 156, 116], [155, 178, 194], [20, 240, 114], [153, 45, 159], [98, 198, 203], [121, 19, 135], [233, 195, 139], [248, 84, 10], [62, 84, 100], [238, 180, 105], [120, 26, 17], [217, 238, 61], [78, 184, 33], [3, 48, 145], [136, 135, 21], [229, 81, 236], [78, 147, 6], [140, 59, 174], [193, 168, 127], [33, 204, 156], [48, 255, 100], [158, 189, 1], [28, 118, 118], [5, 83, 9], [149, 91, 53], [1, 32, 208], [102, 207, 227], [94, 41, 32], [116, 33, 214], [139, 172, 57], [145, 22, 131], [59, 167, 83], [233, 7, 228], [118, 129, 90], [142, 15, 227], [254, 112, 192], [208, 151, 57], [185, 55, 54], [63, 73, 249], [246, 243, 203], [83, 166, 171], [209, 223, 244], [40, 28, 4], [44, 121, 196], [205, 130, 243], [226, 29, 252], [44, 60, 208], [25, 185, 125], [100, 78, 9], [244, 181, 178], [160, 205, 124], [9, 89, 129], [218, 213, 3], [158, 47, 220], [116, 86, 32], [64, 201, 146], [166, 61, 99], [101, 113, 122], [227, 252, 95], [129, 1, 66], [206, 81, 88], [199, 223, 40], [29, 87, 150], [224, 69, 26], [192, 48, 240], [178, 212, 148], [30, 67, 5], [57, 38, 173], [174, 17, 94], [247, 90, 13], [22, 56, 134], [42, 39, 122], [54, 103, 38], [226, 146, 41], [70, 134, 73], [248, 40, 98], [111, 47, 253], [120, 136, 48], [83, 40, 95], [200, 104, 228], [78, 12, 183], [6, 179, 129], [160, 42, 132], [62, 216, 244], [42, 166, 252], [131, 51, 171], [153, 227, 168], [154, 157, 38], [5, 121, 120], [157, 161, 255], [142, 61, 73], [122, 56, 131], [1, 44, 95], [103, 215, 131], [6, 105, 247], [87, 188, 193], [62, 62, 166], [186, 66, 130], [147, 205, 211], [203, 0, 249], [210, 77, 108], [28, 224, 44], [148, 62, 19], [190, 189, 122], [208, 37, 23], [105, 147, 195], [164, 21, 172], [68, 202, 240], [208, 41, 219], [117, 175, 119], [179, 80, 63], [86, 89, 78], [159, 190, 244], [3, 146, 150], [230, 43, 2], [213, 9, 50], [11, 49, 141], [246, 77, 164], [54, 143, 5], [179, 68, 91], [28, 191, 118], [169, 9, 164], [214, 230, 90], [203, 149, 16], [12, 171, 1], [19, 25, 231], [191, 5, 190], [78, 25, 65], [229, 214, 201], [42, 12, 12], [49, 37, 29], [255, 43, 102], [7, 87, 8], [73, 203, 3], [174, 250, 4], [28, 33, 249], [39, 48, 7], [61, 31, 139], [71, 206, 189], [144, 52, 12], [170, 43, 25], [243, 96, 215], [112, 56, 204], [25, 117, 19], [160, 246, 77], [105, 133, 152], [242, 176, 15], [213, 216, 66], [2, 51, 198], [112, 235, 72], [122, 207, 32], [101, 85, 178], [169, 117, 115], [242, 14, 226], [199, 19, 45], [81, 201, 99], [251, 89, 148], [214, 50, 46], [37, 123, 125], [148, 1, 114], [197, 97, 201], [89, 8, 123], [202, 224, 167], [252, 185, 241], [223, 15, 213], [254, 133, 62], [181, 254, 20], [168, 115, 181], [116, 131, 25], [200, 108, 79], [173, 127, 8], [139, 53, 173], [99, 233, 142], [154, 76, 108], [70, 69, 183], [255, 178, 252], [101, 138, 149], [178, 124, 202], [133, 33, 53], [117, 191, 128], [247, 193, 2], [253, 244, 127], [115, 228, 234], [106, 21, 154], [0, 17, 109], [140, 85, 212], [229, 118, 243], [141, 238, 164], [58, 160, 91], [36, 10, 109], [219, 47, 38], [200, 118, 12], [139, 188, 98], [14, 213, 239], [73, 153, 29], [29, 186, 217], [64, 193, 10], [190, 99, 2], [171, 28, 71], [43, 187, 145], [225, 247, 204], [132, 206, 251], [34, 188, 215], [94, 8, 73], [205, 31, 12], [176, 98, 91], [44, 106, 152], [21, 4, 180], [228, 19, 174], [78, 211, 231], [232, 39, 50], [82, 52, 166], [74, 44, 85], [8, 53, 3], [158, 24, 104], [95, 244, 191], [230, 230, 68], [14, 108, 173], [143, 21, 236], [84, 100, 24], [194, 111, 18], [180, 196, 137], [90, 156, 1], [84, 19, 121], [179, 24, 0], [156, 218, 216], [126, 28, 148], [176, 80, 233], [1, 55, 228], [26, 147, 228], [33, 184, 12], [237, 232, 202], [241, 189, 124], [130, 28, 154], [232, 76, 17], [175, 204, 153], [174, 35, 25], [6, 122, 206], [156, 194, 16], [185, 50, 4], [176, 245, 38], [192, 127, 237], [125, 204, 164], [82, 206, 157], [199, 47, 222], [43, 228, 188], [73, 248, 5], [69, 94, 199], [89, 18, 113], [88, 87, 127], [190, 52, 103], [240, 144, 126], [167, 128, 57], [169, 246, 120], [29, 14, 48], [155, 74, 131], [220, 235, 92], [254, 204, 170], [70, 100, 21], [115, 250, 215], [217, 173, 24], [43, 11, 152], [130, 80, 37], [115, 26, 72], [101, 95, 48], [84, 126, 115], [39, 216, 160], [59, 24, 50], [39, 10, 121], [46, 221, 6], [9, 92, 81], [193, 153, 72], [58, 237, 221], [222, 34, 134], [9, 183, 3], [58, 111, 88], [59, 35, 224], [68, 195, 160], [167, 190, 195], [250, 190, 233], [224, 158, 85], [141, 234, 195], [68, 30, 100], [168, 69, 21], [60, 23, 169], [97, 92, 36], [174, 114, 124], [204, 164, 183], [24, 47, 165], [54, 184, 119], [147, 31, 20], [95, 77, 56], [142, 71, 65], [16, 253, 96], [196, 28, 151], [156, 106, 42], [47, 76, 58], [40, 219, 217], [93, 143, 69], [107, 225, 95], [33, 21, 106], [235, 229, 214], [140, 110, 175], [245, 195, 65], [204, 75, 244], [34, 75, 204], [252, 0, 248], [183, 139, 58], [83, 49, 150], [241, 201, 34], [149, 80, 244], [197, 5, 168], [73, 183, 232], [82, 240, 221], [143, 133, 36], [240, 181, 176], [124, 17, 75], [97, 18, 111], [127, 7, 142], [10, 3, 50], [84, 137, 80], [77, 103, 42], [155, 154, 4], [242, 208, 209], [0, 109, 250], [117, 33, 44], [76, 75, 127], [115, 79, 25], [91, 216, 126], [206, 249, 66], [169, 218, 216], [131, 164, 147], [188, 167, 167], [71, 77, 144], [59, 188, 76], [187, 193, 5], [172, 6, 1], [75, 223, 107], [74, 26, 202], [107, 107, 161], [145, 253, 204], [240, 83, 136], [21, 107, 11], [240, 114, 39], [159, 16, 199], [245, 26, 63], [102, 24, 183], [33, 180, 169], [14, 212, 164], [73, 108, 253], [175, 163, 51], [209, 189, 21], [168, 51, 207], [133, 126, 217], [133, 215, 197], [42, 242, 9], [248, 125, 79], [216, 43, 13], [127, 63, 39], [63, 175, 204], [50, 35, 154], [26, 215, 168], [185, 6, 171], [166, 252, 174], [147, 126, 35], [146, 194, 227], [128, 171, 0], [42, 126, 179], [2, 239, 231], [146, 176, 76], [52, 34, 186], [248, 15, 67], [101, 130, 1], [100, 45, 249], [99, 156, 64], [208, 214, 180], [2, 1, 206], [5, 92, 191], [224, 40, 224], [24, 19, 237], [198, 139, 239], [166, 56, 187], [193, 80, 25], [30, 108, 204], [218, 80, 104], [109, 171, 83], [186, 41, 147], [16, 176, 253], [208, 129, 127], [66, 218, 24], [4, 144, 227], [64, 116, 225], [86, 4, 41], [107, 56, 212], [81, 216, 137], [44, 101, 39], [87, 44, 36], [111, 126, 172], [168, 119, 136], [160, 47, 244], [91, 221, 193], [200, 87, 43], [215, 50, 52], [173, 13, 87], [4, 124, 245], [60, 141, 219], [255, 71, 141], [146, 73, 87], [67, 66, 137], [220, 79, 157], [154, 217, 42], [39, 96, 224], [109, 22, 43], [102, 144, 232], [35, 20, 63], [73, 98, 154], [94, 113, 194], [17, 64, 157], [127, 118, 65], [197, 173, 224], [171, 208, 47], [15, 162, 71], [33, 158, 135], [159, 31, 115], [35, 63, 207], [150, 149, 227], [183, 47, 199], [226, 120, 14], [12, 188, 6], [149, 244, 29], [165, 26, 253], [25, 237, 252], [215, 235, 199], [34, 167, 224], [58, 101, 176], [66, 35, 22], [18, 153, 239], [18, 95, 146], [202, 5, 28], [198, 51, 45], [194, 45, 109], [42, 220, 142], [174, 29, 76], [61, 160, 249], [29, 129, 174], [190, 34, 69], [78, 26, 59], [141, 110, 165], [237, 53, 81], [174, 141, 143], [204, 123, 101], [55, 42, 140], [62, 53, 7], [197, 109, 41], [224, 184, 110], [115, 189, 255], [175, 85, 178], [0, 45, 128], [125, 85, 24], [224, 168, 50], [217, 64, 119], [16, 57, 59], [195, 71, 51], [189, 4, 198], [82, 156, 30], [150, 177, 210], [185, 6, 117], [114, 152, 65], [241, 213, 217], [145, 140, 63], [189, 2, 98], [243, 216, 109], [47, 125, 208], [203, 23, 70], [79, 73, 103], [52, 216, 169], [240, 13, 100], [65, 168, 199], [154, 176, 169], [77, 103, 67], [13, 223, 89], [247, 53, 123], [136, 164, 34], [106, 34, 211], [220, 82, 31], [41, 60, 197], [173, 234, 95], [215, 102, 212], [165, 223, 138], [206, 124, 241], [110, 158, 170], [254, 205, 157], [215, 42, 245], [93, 65, 22], [100, 49, 196], [118, 213, 218], [90, 90, 152], [151, 82, 234], [176, 123, 102], [143, 241, 121], [93, 52, 27], [180, 29, 97], [115, 68, 252], [59, 57, 47], [145, 90, 9], [126, 153, 247], [202, 204, 173], [18, 79, 20], [46, 107, 191], [246, 146, 132], [72, 176, 221], [174, 19, 255], [255, 149, 216], [199, 98, 249], [182, 209, 174], [19, 21, 119], [36, 65, 169], [202, 52, 221], [108, 56, 212], [109, 37, 134], [166, 132, 178], [183, 39, 2], [238, 205, 215], [216, 35, 61], [84, 9, 152], [240, 220, 53], [140, 36, 134], [227, 231, 34], [238, 72, 21], [43, 206, 122], [26, 45, 82], [46, 243, 92], [213, 93, 217], [232, 4, 249], [122, 7, 22], [211, 123, 122], [102, 66, 10], [211, 239, 92], [235, 201, 181], [237, 173, 39], [248, 157, 145], [166, 180, 82], [137, 164, 183], [74, 18, 115], [122, 12, 105], [239, 192, 235], [84, 205, 223], [35, 233, 229], [100, 74, 253], [167, 123, 87], [14, 163, 21], [84, 169, 221], [187, 82, 203], [235, 132, 173], [200, 152, 37], [0, 220, 128], [37, 207, 167], [226, 161, 133], [242, 22, 198], [28, 8, 40], [148, 50, 41], [65, 177, 222], [219, 223, 16], [173, 237, 50], [115, 214, 136], [104, 207, 59], [13, 150, 155], [137, 31, 161], [54, 95, 96], [41, 21, 246], [206, 125, 26], [128, 204, 124], [113, 89, 16], [242, 3, 16], [119, 3, 147], [175, 55, 31], [116, 52, 197], [99, 178, 113], [153, 87, 151], [119, 253, 37], [70, 51, 224], [32, 82, 11], [74, 52, 165], [242, 161, 29], [56, 14, 58], [167, 105, 43], [180, 206, 165], [165, 121, 193], [228, 60, 20], [27, 254, 164], [19, 195, 19], [210, 42, 115], [114, 64, 237], [188, 225, 218], [4, 75, 136], [143, 192, 152], [174, 237, 0], [226, 157, 243], [247, 203, 207], [229, 160, 20], [151, 129, 64], [154, 19, 44], [39, 190, 129], [82, 229, 242], [114, 231, 169], [182, 150, 158], [58, 63, 171], [142, 159, 207], [3, 232, 74], [35, 206, 25], [93, 123, 20], [110, 2, 199], [84, 139, 120], [224, 74, 119], [112, 199, 16], [124, 215, 242], [219, 131, 243], [35, 127, 237], [137, 202, 111], [186, 234, 36], [247, 33, 70], [237, 244, 79], [50, 158, 237], [54, 45, 102], [237, 51, 251], [190, 20, 204], [115, 68, 33], [169, 158, 216], [215, 246, 47], [96, 69, 152], [13, 104, 68], [138, 145, 2], [65, 34, 154], [219, 98, 121], [240, 255, 227], [126, 196, 201], [157, 106, 237], [79, 94, 208], [116, 142, 122], [2, 216, 113], [166, 144, 230], [48, 40, 125], [179, 66, 141], [114, 146, 121], [219, 145, 114], [12, 65, 99], [75, 49, 52], [24, 121, 27], [70, 138, 238], [13, 17, 220], [111, 11, 253], [62, 168, 156], [122, 125, 137], [48, 10, 171], [43, 243, 248], [204, 36, 41], [254, 16, 240], [42, 55, 135], [157, 107, 7], [59, 110, 219], [8, 10, 50], [3, 89, 159], [138, 39, 95], [162, 254, 101], [222, 239, 4], [236, 116, 4], [148, 0, 1], [47, 210, 14], [55, 185, 13], [149, 31, 20], [122, 199, 252], [189, 33, 249], [208, 121, 29], [234, 147, 211], [137, 109, 15], [79, 47, 82], [114, 92, 33], [219, 36, 145], [128, 175, 184], [45, 142, 161], [20, 126, 111], [115, 214, 185], [79, 35, 37], [161, 86, 208], [111, 158, 41], [165, 28, 202], [118, 51, 239], [189, 25, 41], [22, 27, 166], [168, 132, 65], [148, 204, 133], [110, 167, 133], [115, 20, 48], [7, 229, 237], [22, 96, 124], [92, 222, 71], [40, 33, 135], [196, 200, 90], [98, 34, 69], [26, 10, 105], [112, 96, 181], [191, 246, 18], [222, 251, 219], [41, 156, 162], [15, 46, 220], [71, 107, 204], [155, 155, 247], [205, 201, 192], [62, 34, 167], [218, 228, 6], [123, 115, 162], [168, 158, 144], [170, 121, 41], [37, 175, 186], [87, 149, 25], [184, 204, 60], [3, 243, 126], [171, 23, 148], [102, 241, 60], [152, 122, 212], [38, 250, 181], [120, 84, 254], [206, 221, 58], [254, 77, 75], [17, 128, 165], [79, 237, 125], [55, 14, 176], [51, 104, 156], [111, 171, 132], [11, 142, 113], [87, 109, 28], [135, 171, 21], [21, 108, 239], [158, 111, 125], [27, 182, 157], [206, 113, 103], [91, 120, 137], [68, 171, 11], [172, 78, 110], [94, 205, 86], [28, 135, 58], [46, 237, 85], [61, 180, 99], [214, 59, 117], [173, 144, 70], [154, 124, 167], [130, 24, 39], [188, 142, 112], [86, 248, 98], [167, 61, 242], [71, 65, 74], [193, 180, 30], [50, 93, 156], [137, 170, 174], [131, 248, 234], [245, 226, 217], [18, 79, 111], [23, 221, 137], [46, 46, 203], [89, 254, 148], [135, 175, 206], [63, 233, 68], [123, 238, 67], [33, 20, 91], [3, 241, 242], [228, 205, 118], [69, 1, 69], [156, 164, 77], [119, 181, 60], [9, 14, 243], [64, 11, 119], [153, 239, 111], [102, 25, 156], [89, 186, 34], [95, 29, 196], [184, 85, 3], [98, 139, 226], [225, 138, 142], [172, 18, 22], [65, 78, 61], [227, 72, 81], [114, 231, 175], [178, 32, 159], [152, 161, 65], [168, 185, 204], [6, 105, 115], [167, 199, 167], [236, 234, 93], [10, 160, 51], [65, 192, 173], [65, 120, 71], [4, 114, 135], [249, 172, 232], [139, 169, 45], [240, 25, 106], [249, 173, 8], [24, 113, 37], [75, 118, 75], [68, 94, 160], [254, 50, 238], [120, 196, 37], [212, 113, 90], [151, 166, 164], [89, 207, 48], [202, 160, 139], [147, 14, 172], [93, 176, 114], [158, 198, 162], [42, 182, 205], [5, 251, 42], [134, 221, 47], [41, 134, 58], [184, 77, 13], [191, 240, 188], [236, 71, 66], [196, 159, 73], [25, 90, 156], [154, 140, 205], [188, 5, 205], [61, 65, 63], [214, 226, 156], [40, 117, 126], [154, 56, 12], [172, 82, 195], [39, 4, 147], [63, 56, 251], [228, 188, 241], [255, 129, 43], [134, 46, 42], [46, 104, 22], [110, 69, 133], [77, 107, 18], [48, 200, 160], [238, 214, 107], [178, 195, 147], [19, 201, 46], [135, 251, 85], [160, 183, 20], [41, 10, 209], [186, 185, 21], [18, 82, 46], [217, 228, 163], [176, 214, 149], [113, 201, 143], [132, 13, 179], [223, 241, 242], [144, 23, 126], [145, 63, 105], [134, 238, 218], [154, 33, 155], [70, 131, 175], [129, 223, 203], [58, 83, 10], [62, 142, 155], [59, 67, 83], [168, 99, 141], [188, 196, 142], [4, 243, 139], [76, 120, 60], [164, 31, 171], [97, 21, 227], [34, 61, 201], [42, 221, 42], [242, 39, 6], [102, 174, 186], [253, 245, 210], [174, 84, 14], [190, 31, 155], [168, 174, 155], [145, 204, 95], [114, 28, 114], [22, 138, 124], [103, 223, 254], [187, 37, 213], [38, 55, 69], [36, 228, 177], [116, 172, 57], [85, 45, 254], [168, 209, 235], [3, 129, 6], [67, 251, 224], [114, 95, 51], [219, 159, 250], [126, 36, 123], [87, 110, 9], [203, 92, 176], [240, 71, 182], [35, 196, 121], [123, 22, 68], [12, 72, 182], [91, 86, 98], [116, 225, 231], [255, 58, 58], [83, 3, 140], [182, 133, 34], [73, 175, 28], [218, 195, 27], [207, 123, 34], [147, 10, 13], [106, 228, 162], [193, 174, 132], [239, 181, 249], [43, 222, 98], [77, 196, 199], [104, 68, 17], [12, 244, 49], [82, 202, 153], [133, 218, 19], [175, 59, 33], [159, 101, 68], [112, 111, 113], [16, 172, 232], [74, 215, 202], [19, 222, 229], [91, 86, 236], [211, 188, 131], [53, 233, 119], [189, 154, 134], [136, 228, 164], [32, 84, 57], [63, 84, 165], [110, 48, 171], [190, 240, 70], [141, 65, 231], [116, 166, 228], [181, 209, 81], [192, 187, 103], [85, 196, 57], [163, 100, 115], [140, 131, 1], [87, 82, 235], [20, 125, 79], [233, 1, 129], [93, 200, 231], [240, 48, 142], [151, 29, 92], [81, 194, 15], [236, 3, 198], [69, 228, 30], [64, 111, 71], [185, 240, 67], [233, 111, 202], [238, 90, 135], [50, 145, 231], [222, 248, 44], [86, 159, 108], [152, 119, 92], [101, 198, 67], [179, 215, 212], [170, 223, 156], [140, 100, 136], [108, 236, 246], [62, 32, 54], [154, 35, 192], [74, 113, 60], [208, 121, 68], [224, 58, 151], [250, 243, 236], [177, 175, 172], [21, 98, 252], [117, 40, 90], [180, 27, 130], [209, 70, 138], [215, 17, 29], [190, 235, 180], [178, 187, 168], [136, 50, 194], [130, 120, 43], [75, 25, 194], [191, 178, 167], [38, 174, 177], [65, 0, 201], [127, 205, 167], [24, 46, 253], [8, 3, 10], [4, 110, 3], [147, 234, 243], [130, 146, 152], [174, 22, 143], [195, 158, 235], [169, 81, 193], [152, 76, 124], [187, 176, 237], [239, 145, 233], [162, 176, 234], [9, 243, 16], [149, 77, 188], [50, 75, 56], [171, 75, 163], [13, 182, 197], [240, 59, 31], [131, 217, 2], [109, 120, 25], [196, 118, 84], [230, 182, 158], [203, 136, 200], [250, 55, 80], [43, 147, 215], [54, 226, 214], [134, 132, 220], [175, 168, 40], [4, 122, 155], [46, 159, 24], [118, 32, 226], [199, 243, 120], [183, 123, 179], [195, 14, 6], [159, 180, 222], [165, 251, 107], [235, 109, 71], [191, 252, 131], [166, 95, 55], [29, 199, 49], [124, 83, 190], [33, 5, 142], [166, 214, 226], [198, 130, 47], [6, 23, 101], [231, 25, 253], [124, 138, 6], [57, 242, 25], [136, 193, 186], [97, 70, 184], [121, 255, 196], [240, 227, 74], [36, 244, 196], [168, 150, 65], [235, 127, 248], [47, 178, 12], [217, 230, 253], [206, 224, 162], [172, 169, 165], [250, 194, 224], [16, 87, 97], [83, 153, 213], [94, 104, 53], [227, 33, 99], [187, 132, 95], [22, 99, 14], [80, 93, 27], [25, 172, 218], [4, 247, 80], [104, 71, 122], [91, 151, 155], [215, 2, 19], [118, 141, 92], [36, 91, 123], [228, 30, 132], [255, 118, 153], [244, 166, 245], [18, 50, 28], [32, 196, 80], [89, 31, 124], [37, 162, 18], [241, 203, 99], [54, 209, 150], [9, 7, 196], [101, 91, 194], [150, 173, 93], [157, 70, 222], [193, 67, 134], [251, 154, 179], [4, 23, 228], [244, 179, 118], [30, 245, 30], [175, 109, 81], [31, 79, 246], [129, 51, 244], [144, 60, 95], [16, 135, 158], [126, 49, 80], [54, 19, 160], [115, 24, 223], [235, 141, 252], [179, 67, 239], [235, 60, 116], [108, 191, 88], [210, 119, 49], [29, 212, 11], [254, 175, 173], [220, 11, 197], [160, 218, 156], [16, 237, 54], [151, 68, 252], [203, 110, 109], [47, 203, 217], [246, 212, 211], [121, 226, 178], [80, 126, 200], [110, 158, 118], [34, 254, 34], [83, 146, 173], [68, 184, 194], [74, 45, 234], [37, 157, 137], [134, 224, 71], [57, 59, 149], [196, 169, 86], [202, 159, 91], [93, 31, 89], [142, 98, 193], [11, 96, 157], [237, 2, 93], [202, 132, 12], [63, 228, 208], [175, 111, 90], [34, 250, 213], [64, 95, 147], [242, 67, 251], [251, 182, 40], [198, 254, 192], [5, 23, 75], [128, 170, 58], [118, 254, 133], [52, 74, 99], [172, 5, 194], [35, 212, 191], [224, 190, 248], [90, 0, 172], [86, 81, 128], [57, 86, 247], [48, 60, 56], [183, 96, 141], [230, 147, 196], [130, 35, 94], [95, 238, 48], [214, 56, 34], [126, 245, 1], [196, 132, 39], [182, 13, 253], [170, 125, 15], [16, 48, 78], [148, 215, 175], [123, 251, 144], [118, 139, 214], [253, 126, 193], [128, 211, 186], [250, 68, 88], [128, 74, 202], [232, 140, 157], [153, 134, 0], [210, 15, 41], [50, 206, 189], [155, 3, 154], [219, 73, 181], [181, 130, 13], [253, 124, 149], [182, 113, 199], [182, 115, 201], [148, 230, 0], [67, 203, 26], [128, 75, 113], [97, 118, 109], [23, 95, 219], [163, 67, 217], [238, 144, 53], [22, 230, 206], [214, 206, 99], [160, 160, 45], [53, 126, 225], [122, 40, 125], [144, 82, 127], [205, 236, 157], [135, 134, 102], [33, 8, 35], [90, 230, 14], [158, 100, 188], [209, 50, 173], [254, 151, 3], [240, 160, 131], [58, 206, 14], [70, 111, 38], [217, 121, 79], [130, 120, 89], [108, 122, 140], [170, 65, 13], [41, 87, 126], [0, 3, 26], [6, 8, 177], [213, 205, 167], [243, 96, 255], [235, 116, 103], [238, 240, 233], [248, 162, 45], [92, 172, 227], [222, 155, 9], [206, 240, 38], [5, 49, 169], [56, 246, 81], [72, 197, 100], [211, 127, 93], [190, 121, 112], [114, 114, 182], [217, 26, 131], [255, 53, 30], [31, 19, 91], [220, 118, 225], [208, 85, 93], [154, 43, 57], [160, 210, 186], [14, 94, 146], [231, 82, 44], [24, 80, 34], [50, 1, 149], [240, 204, 152], [210, 198, 206], [85, 90, 106], [182, 95, 45], [224, 69, 167], [101, 87, 253], [83, 222, 22], [78, 26, 73], [250, 104, 132], [85, 127, 57], [16, 221, 7], [25, 47, 138], [160, 235, 96], [142, 233, 130], [16, 96, 174], [136, 25, 139], [148, 139, 231], [131, 224, 234], [60, 235, 118], [83, 204, 110], [22, 45, 81], [32, 89, 156], [101, 5, 167], [66, 160, 196], [57, 44, 118], [200, 235, 205], [15, 146, 144], [235, 167, 242], [31, 69, 159], [44, 40, 62], [10, 79, 190], [80, 76, 71], [221, 16, 247], [93, 1, 25], [12, 137, 232], [49, 63, 222], [224, 23, 72], [120, 163, 239], [203, 188, 251], [181, 54, 193], [32, 32, 233], [89, 169, 199], [224, 216, 229], [179, 199, 76], [12, 82, 26], [97, 223, 200], [47, 203, 188], [185, 63, 214], [189, 68, 102], [225, 149, 81], [223, 147, 161], [202, 39, 1], [60, 5, 16], [248, 239, 107], [208, 126, 87], [146, 211, 98], [161, 17, 64], [177, 5, 197], [144, 52, 20], [254, 195, 126], [201, 182, 227], [131, 22, 227], [215, 53, 105], [19, 188, 6], [51, 69, 55], [235, 220, 198], [253, 67, 104], [159, 146, 97], [14, 242, 173], [182, 1, 133], [78, 82, 59], [220, 252, 82], [149, 6, 168], [199, 202, 9], [211, 163, 77], [205, 78, 127], [131, 114, 18], [163, 124, 87], [153, 85, 140], [201, 53, 205], [113, 91, 74], [137, 154, 152], [186, 74, 141], [137, 58, 76], [47, 241, 170], [208, 29, 233], [69, 66, 60], [202, 166, 204], [45, 131, 132], [32, 180, 173], [35, 85, 81], [32, 2, 132], [117, 78, 37], [23, 212, 190], [229, 86, 141], [153, 211, 230], [18, 190, 108], [147, 11, 86], [90, 250, 31], [230, 55, 26], [189, 216, 121], [106, 34, 181], [13, 139, 176], [146, 70, 16], [96, 189, 9], [22, 93, 30], [19, 253, 217], [111, 105, 183], [243, 62, 27], [142, 144, 122], [113, 170, 206], [75, 62, 202], [82, 196, 204], [5, 133, 198], [208, 78, 252], [208, 147, 250], [55, 64, 42], [245, 31, 163], [105, 179, 147], [27, 184, 244], [125, 156, 46], [34, 25, 221], [232, 13, 99], [139, 207, 153], [16, 251, 24], [184, 61, 184], [60, 61, 87], [166, 148, 153], [228, 169, 0], [115, 129, 225], [235, 29, 130], [123, 134, 190], [70, 26, 46], [14, 79, 129], [82, 123, 12], [217, 0, 234], [197, 251, 69], [78, 57, 50], [186, 213, 82], [176, 163, 15], [142, 87, 248], [106, 18, 99], [207, 24, 199], [162, 246, 80], [212, 116, 72], [179, 122, 71], [181, 96, 170], [50, 10, 184], [99, 11, 3], [219, 106, 191], [208, 156, 83], [102, 199, 3], [23, 82, 58], [84, 173, 50], [16, 107, 187], [158, 100, 223], [12, 83, 84], [99, 17, 228], [198, 25, 151], [79, 254, 197], [130, 244, 158], [234, 218, 57], [25, 30, 208], [29, 156, 50], [193, 76, 27], [212, 19, 240], [212, 241, 59], [142, 204, 130], [179, 162, 101], [240, 83, 238], [236, 223, 49], [139, 68, 166], [32, 70, 120], [83, 8, 28], [235, 196, 130], [51, 28, 75], [186, 21, 5], [104, 88, 212], [69, 0, 247], [241, 176, 150], [170, 205, 7], [28, 63, 106], [226, 164, 247], [204, 17, 87], [142, 68, 114], [68, 142, 76], [44, 4, 211], [103, 199, 51], [7, 231, 223], [177, 156, 171], [205, 115, 116], [171, 110, 133], [252, 93, 99], [178, 88, 114], [73, 193, 243], [219, 22, 187], [29, 231, 28], [24, 7, 182], [179, 187, 234], [187, 212, 33], [110, 215, 75], [217, 90, 38], [169, 123, 163], [143, 42, 129], [47, 12, 167], [60, 0, 80], [97, 75, 40], [242, 34, 182], [8, 24, 59], [7, 33, 194], [15, 172, 218], [34, 25, 162], [226, 169, 187], [7, 66, 181], [154, 61, 212], [122, 49, 243], [144, 210, 183], [108, 44, 103], [55, 52, 149], [81, 44, 217], [78, 59, 145], [251, 106, 238], [88, 127, 145], [91, 109, 139], [140, 177, 156], [48, 201, 172], [214, 243, 55], [174, 183, 243], [132, 47, 236], [227, 173, 130], [12, 131, 230], [107, 77, 98], [194, 167, 33], [232, 122, 247], [49, 76, 57], [64, 221, 140], [95, 243, 152], [205, 1, 10], [202, 50, 98], [79, 181, 198], [237, 126, 255], [211, 59, 160], [217, 183, 62], [115, 228, 59], [81, 124, 12], [104, 143, 91], [200, 186, 12], [136, 85, 165], [56, 45, 223], [190, 244, 60], [201, 35, 244], [110, 32, 49], [58, 93, 223], [249, 163, 175], [244, 125, 196], [10, 48, 23], [157, 31, 147], [177, 244, 31], [96, 131, 216], [169, 72, 66], [232, 70, 104], [127, 95, 168], [128, 175, 198], [209, 6, 189], [223, 202, 60], [31, 28, 92], [162, 176, 151], [192, 142, 69], [205, 242, 71], [249, 164, 87], [246, 56, 127], [142, 213, 144], [228, 119, 137], [75, 82, 193], [105, 43, 64], [187, 237, 112], [75, 182, 173], [149, 130, 14], [202, 239, 64], [170, 46, 47], [29, 12, 55], [137, 239, 216], [22, 53, 173], [171, 18, 225], [23, 225, 204], [24, 183, 151], [203, 33, 148], [216, 75, 141], [109, 195, 9], [113, 56, 187], [57, 249, 208], [199, 33, 63], [157, 189, 141], [134, 19, 35], [60, 25, 229], [14, 3, 208], [224, 84, 227], [58, 205, 206], [183, 58, 91], [86, 234, 124], [1, 234, 185], [13, 117, 195], [189, 97, 125], [115, 41, 252], [90, 58, 106], [154, 131, 175], [114, 163, 227], [125, 250, 231], [224, 122, 241], [27, 164, 105], [16, 181, 96], [183, 165, 238], [180, 93, 3], [241, 139, 1], [96, 218, 44], [186, 219, 131], [184, 227, 210], [43, 44, 151], [192, 90, 0], [26, 115, 201], [197, 131, 35], [101, 244, 196], [25, 178, 109], [219, 243, 22], [194, 128, 32], [56, 36, 165], [201, 220, 136], [199, 110, 226], [194, 179, 216], [21, 155, 69], [13, 32, 161], [204, 85, 229], [221, 199, 89], [178, 13, 4], [78, 81, 241], [137, 222, 197], [91, 70, 99], [114, 59, 209], [30, 108, 184], [182, 174, 75], [194, 226, 223], [35, 161, 4], [43, 12, 210], [81, 45, 233], [191, 180, 60], [77, 157, 206], [2, 127, 185], [235, 96, 66], [60, 227, 76], [57, 32, 175], [16, 105, 27], [190, 130, 12], [229, 184, 186], [92, 143, 66], [78, 248, 38], [58, 201, 40], [175, 188, 80], [7, 96, 68], [94, 124, 191], [219, 237, 162], [72, 127, 71], [145, 161, 192], [237, 84, 120], [215, 160, 5], [144, 20, 78], [178, 191, 193], [236, 52, 118], [142, 157, 158], [118, 190, 146], [26, 89, 232], [205, 16, 70], [69, 57, 227], [188, 169, 204], [231, 131, 119], [73, 28, 34], [105, 58, 134], [95, 198, 79], [22, 14, 29], [105, 137, 28], [240, 35, 246], [6, 254, 203], [84, 43, 245], [89, 124, 246], [88, 55, 115], [192, 144, 118], [233, 187, 212], [44, 180, 178], [113, 152, 50], [172, 45, 52], [17, 62, 218], [221, 197, 178], [148, 46, 196], [55, 93, 228], [39, 130, 228], [171, 48, 16], [45, 68, 131], [57, 208, 102], [127, 230, 151], [76, 11, 140], [25, 208, 255], [57, 222, 166], [116, 185, 82], [16, 100, 229], [51, 108, 107], [40, 169, 127], [171, 163, 27], [72, 154, 229], [134, 124, 37], [102, 71, 43], [239, 191, 143], [179, 125, 230], [47, 24, 250], [121, 23, 185], [94, 129, 135], [87, 205, 234], [160, 12, 239], [78, 223, 41], [223, 143, 42], [59, 0, 54], [227, 86, 18], [179, 11, 129], [106, 102, 79], [7, 100, 240], [69, 18, 67], [34, 171, 187], [171, 69, 5], [100, 231, 198], [174, 109, 121], [248, 155, 184], [139, 190, 235], [203, 195, 181], [221, 250, 99], [117, 91, 27], [168, 72, 194], [251, 162, 223], [184, 146, 96], [252, 177, 95], [148, 222, 80], [230, 19, 253], [144, 228, 241], [113, 82, 26], [149, 189, 134], [139, 250, 172], [182, 143, 171], [25, 158, 130], [23, 79, 232], [91, 123, 143], [224, 158, 99], [125, 205, 6], [71, 61, 155], [74, 253, 73], [206, 70, 231], [43, 47, 53], [8, 34, 122], [110, 190, 135], [209, 178, 56], [44, 47, 94], [44, 3, 230], [80, 47, 115], [45, 91, 114], [10, 25, 122], [147, 109, 199], [103, 97, 181], [69, 81, 212], [51, 63, 230], [211, 57, 216], [194, 138, 86], [72, 188, 241], [154, 190, 192], [74, 56, 58], [152, 229, 103], [228, 162, 237], [247, 132, 34], [161, 221, 120], [250, 66, 92], [245, 129, 179], [181, 26, 197], [78, 236, 51], [224, 171, 29], [235, 31, 103], [31, 51, 222], [49, 131, 44], [81, 29, 53], [19, 30, 192], [171, 84, 11], [201, 162, 89], [90, 126, 221], [49, 74, 203], [211, 180, 124], [116, 57, 145], [69, 24, 10], [180, 6, 235], [25, 114, 14], [203, 175, 56], [172, 158, 178], [172, 132, 192], [149, 227, 85], [162, 173, 144], [115, 184, 60], [129, 151, 80], [80, 221, 222], [90, 206, 40], [234, 59, 70], [53, 66, 51], [107, 40, 90], [109, 141, 58], [35, 24, 249], [162, 45, 227], [198, 175, 95], [162, 220, 189], [212, 185, 81], [35, 6, 25], [203, 100, 201], [132, 240, 247], [41, 93, 132], [153, 88, 139], [147, 231, 59], [201, 137, 57], [31, 52, 243], [202, 213, 138], [70, 11, 70], [59, 12, 49], [151, 101, 51], [35, 9, 194], [253, 49, 237], [238, 96, 4], [181, 53, 171], [111, 57, 85], [160, 101, 213], [199, 170, 192], [137, 210, 24], [230, 98, 184], [136, 55, 203], [5, 212, 139], [172, 185, 219], [109, 59, 52], [104, 45, 231], [14, 143, 20], [139, 42, 17], [140, 100, 78], [183, 219, 118], [211, 85, 193], [23, 191, 128], [108, 147, 139], [130, 43, 125], [163, 111, 32], [138, 170, 58], [67, 228, 84], [90, 177, 202], [210, 183, 174], [70, 135, 237], [97, 145, 16], [154, 188, 106], [255, 77, 42], [224, 64, 152], [163, 174, 79], [189, 20, 177], [69, 208, 141], [126, 35, 31], [223, 136, 85], [27, 3, 14], [235, 171, 22], [33, 180, 41], [196, 229, 209], [180, 17, 27], [161, 29, 128], [59, 80, 72], [109, 143, 89], [107, 134, 156], [114, 65, 188], [172, 110, 122], [135, 22, 6], [162, 211, 154], [167, 81, 149], [13, 75, 251], [216, 50, 82], [192, 75, 144], [194, 94, 92], [233, 94, 240], [8, 14, 156], [154, 22, 45], [140, 114, 173], [115, 106, 101], [161, 236, 144], [75, 241, 104], [250, 2, 139], [213, 70, 67], [233, 212, 198], [124, 79, 87], [250, 172, 247], [69, 116, 26], [115, 15, 230], [101, 199, 101], [242, 139, 242], [24, 243, 214], [44, 54, 81], [135, 38, 169], [118, 231, 179], [254, 84, 162], [234, 143, 57], [211, 44, 133], [152, 147, 96], [94, 114, 233], [222, 190, 45], [169, 112, 11], [139, 50, 153], [150, 182, 223], [182, 90, 34], [56, 208, 66], [125, 104, 219], [180, 59, 15], [233, 239, 120], [150, 215, 227], [222, 188, 6], [78, 99, 235], [187, 169, 164], [11, 115, 135], [250, 93, 167], [230, 67, 106], [182, 22, 91], [233, 143, 32], [185, 19, 222], [58, 41, 13], [113, 122, 186], [125, 75, 229], [36, 177, 76], [120, 46, 83], [74, 194, 149], [255, 41, 97], [228, 0, 7], [38, 167, 158], [42, 221, 167], [102, 161, 8], [179, 107, 239], [249, 80, 243], [209, 22, 241], [118, 130, 214], [21, 91, 48], [71, 228, 154], [86, 96, 59], [189, 220, 42], [193, 221, 149], [207, 46, 240], [30, 45, 25], [30, 191, 253], [127, 110, 64], [23, 100, 66], [12, 1, 126], [199, 164, 48], [212, 47, 148], [149, 66, 121], [23, 20, 230], [201, 152, 5], [218, 20, 62], [249, 81, 26], [107, 128, 206], [201, 108, 232], [123, 188, 249], [172, 68, 239], [235, 129, 55], [39, 161, 242], [40, 64, 159], [99, 172, 114], [7, 69, 42], [109, 113, 16], [19, 47, 67], [33, 17, 188], [12, 56, 229], [115, 39, 144], [88, 222, 127], [29, 58, 13], [69, 131, 229], [104, 131, 146], [211, 68, 194], [157, 43, 121], [241, 250, 147], [228, 54, 96], [189, 154, 15], [141, 192, 88], [170, 196, 137], [163, 244, 184], [1, 177, 241], [197, 237, 138], [68, 224, 111], [249, 39, 20], [26, 217, 217], [74, 48, 124], [89, 116, 70], [24, 210, 189], [45, 77, 54], [190, 184, 242], [5, 190, 248], [224, 46, 200], [167, 125, 197], [247, 15, 219], [240, 101, 157], [50, 118, 53], [19, 129, 152], [102, 70, 206], [67, 203, 180], [226, 170, 141], [137, 162, 96], [30, 185, 57], [195, 7, 98], [185, 238, 216], [144, 108, 47], [175, 127, 193], [214, 92, 62], [165, 105, 78], [184, 14, 249], [36, 190, 155], [252, 220, 165], [252, 32, 117], [152, 132, 88], [132, 24, 64], [14, 157, 180], [193, 99, 119], [192, 204, 94], [248, 127, 182], [89, 74, 136], [180, 49, 92], [128, 242, 55], [151, 167, 104], [74, 238, 66], [25, 172, 48], [112, 76, 211], [41, 79, 58], [185, 17, 126], [140, 225, 180], [165, 175, 34], [211, 129, 63], [7, 158, 240], [176, 113, 7], [158, 234, 41], [191, 22, 240], [17, 213, 35], [230, 72, 88], [66, 162, 24], [99, 28, 2], [175, 116, 170], [217, 25, 124], [175, 90, 213], [222, 115, 138], [151, 27, 134], [180, 254, 108], [93, 168, 93], [94, 203, 105], [95, 69, 28], [125, 56, 233], [227, 107, 162], [181, 95, 253], [56, 120, 228], [163, 84, 51], [177, 5, 226], [132, 190, 144], [105, 59, 184], [202, 124, 167], [62, 193, 65], [218, 52, 128], [37, 94, 171], [45, 125, 189], [226, 217, 118], [37, 211, 86], [106, 85, 229], [125, 213, 158], [154, 23, 241], [121, 211, 21], [94, 127, 135], [147, 249, 132], [0, 222, 187], [115, 91, 22], [126, 225, 250], [32, 202, 246], [248, 83, 224], [87, 17, 146], [227, 103, 248], [194, 141, 186], [124, 75, 173], [39, 178, 35], [53, 54, 134], [16, 159, 193], [50, 77, 9], [127, 254, 232], [199, 1, 36], [218, 169, 168], [93, 109, 143], [193, 218, 231], [171, 119, 4], [151, 223, 30], [222, 140, 225], [100, 114, 201], [147, 252, 20], [187, 53, 99], [73, 94, 75], [161, 11, 153], [196, 88, 30], [249, 129, 129], [125, 206, 113], [219, 228, 163], [63, 53, 16], [215, 107, 5], [144, 102, 213], [25, 75, 93], [42, 121, 224], [239, 96, 248], [158, 21, 157], [168, 193, 215], [165, 16, 54], [203, 47, 139], [104, 93, 48], [33, 133, 144], [23, 105, 1], [44, 67, 148], [56, 45, 64], [229, 25, 103], [70, 146, 141], [24, 134, 128], [59, 204, 187], [153, 98, 218], [172, 161, 185], [221, 31, 183], [20, 32, 246], [41, 171, 106], [133, 95, 158], [125, 131, 203], [45, 17, 243], [87, 66, 197], [186, 117, 56], [143, 47, 66], [53, 55, 8], [99, 218, 219], [95, 140, 61], [115, 184, 77], [175, 169, 95], [16, 172, 90], [206, 139, 202], [33, 229, 143], [38, 156, 68], [204, 60, 164], [185, 147, 197], [100, 115, 79], [230, 3, 15], [164, 153, 205], [17, 198, 114], [206, 191, 145], [99, 178, 209], [121, 54, 211], [110, 178, 211], [207, 86, 108], [245, 230, 28], [242, 66, 18], [169, 116, 161], [137, 154, 87], [92, 72, 26], [132, 0, 33], [33, 199, 155], [49, 129, 36], [37, 197, 234], [7, 162, 43], [157, 102, 19], [165, 63, 191], [77, 19, 242], [126, 16, 169], [255, 170, 218], [141, 190, 174], [80, 170, 216], [92, 103, 200], [237, 14, 100], [207, 162, 124], [36, 192, 119], [117, 71, 107], [92, 136, 76], [200, 208, 225], [128, 102, 47], [241, 157, 115], [195, 12, 26], [136, 229, 167], [124, 129, 69], [152, 104, 60], [79, 188, 248], [47, 168, 244], [121, 225, 186], [119, 183, 218], [221, 171, 248], [105, 90, 39], [91, 50, 137], [222, 92, 120], [61, 5, 64], [47, 43, 202], [46, 139, 116], [36, 180, 47], [16, 115, 33], [64, 186, 208], [142, 20, 63], [225, 120, 57], [141, 170, 66], [99, 64, 149], [17, 50, 222], [241, 49, 105], [86, 94, 196], [175, 56, 228], [17, 240, 20], [44, 17, 37], [192, 173, 55], [78, 9, 123], [13, 56, 80], [216, 25, 160], [120, 37, 109], [149, 177, 132], [115, 232, 183], [130, 91, 191], [61, 27, 169], [46, 52, 197], [252, 127, 199], [101, 235, 218], [149, 32, 193], [82, 123, 90], [136, 39, 145], [15, 152, 105], [41, 231, 213], [212, 220, 243], [224, 146, 184], [12, 198, 4], [65, 33, 253], [110, 39, 194], [73, 144, 237], [147, 134, 46], [149, 107, 57], [247, 254, 34], [99, 115, 81], [47, 89, 255], [192, 239, 196], [160, 112, 223], [255, 216, 124], [229, 133, 23], [74, 155, 58], [245, 174, 87], [4, 91, 17], [186, 78, 228], [100, 14, 57], [225, 29, 211], [18, 10, 125], [60, 234, 243], [238, 224, 84], [212, 208, 61], [30, 42, 14], [163, 203, 16], [123, 36, 87], [133, 132, 171], [105, 94, 227], [52, 64, 188], [102, 195, 54], [20, 228, 231], [200, 45, 149], [179, 11, 202], [186, 148, 100], [171, 130, 207], [85, 198, 20], [150, 149, 253], [230, 111, 33], [29, 22, 218], [168, 83, 196], [119, 121, 165], [85, 238, 110], [187, 250, 102], [195, 139, 182], [132, 146, 126], [45, 63, 17], [125, 3, 240], [68, 205, 107], [204, 38, 5], [106, 38, 225], [78, 137, 204], [220, 136, 34], [14, 176, 12], [200, 75, 76], [232, 250, 236], [106, 84, 84], [45, 27, 108], [86, 38, 17], [9, 209, 101], [239, 198, 80], [249, 148, 60], [128, 124, 52], [83, 129, 30], [108, 244, 78], [202, 237, 147], [7, 233, 169], [169, 5, 86], [254, 234, 125], [44, 189, 71], [171, 88, 125], [208, 77, 67], [8, 144, 179], [41, 168, 112], [85, 63, 95], [78, 221, 86], [179, 188, 21], [53, 243, 33], [203, 72, 182], [171, 214, 78], [46, 86, 72], [232, 213, 221], [151, 251, 22], [32, 144, 65], [112, 65, 55], [92, 137, 98], [191, 168, 133], [153, 176, 187], [222, 24, 164], [86, 123, 83], [2, 133, 201], [9, 1, 14], [189, 227, 82], [157, 87, 64], [172, 190, 61], [71, 150, 181], [44, 118, 86], [224, 47, 126], [169, 206, 247], [143, 241, 115], [183, 204, 236], [207, 45, 107], [115, 50, 178], [50, 124, 45], [119, 206, 122], [145, 233, 208], [118, 43, 151], [131, 224, 190], [113, 198, 200], [31, 89, 118], [91, 37, 115], [79, 66, 166], [33, 0, 239], [95, 17, 51], [181, 73, 20], [179, 145, 1], [44, 248, 169], [160, 212, 4], [197, 87, 37], [39, 28, 36], [161, 206, 2], [179, 188, 0], [143, 124, 165], [105, 29, 32], [99, 140, 152], [125, 188, 50], [187, 157, 244], [88, 127, 182], [190, 129, 32], [143, 35, 33], [132, 50, 31], [243, 236, 203], [89, 151, 176], [122, 226, 166], [44, 141, 105], [239, 124, 66], [118, 218, 125], [18, 34, 199], [48, 195, 253], [226, 231, 75], [210, 16, 151], [136, 3, 189], [151, 36, 191], [147, 103, 240], [19, 95, 191], [138, 40, 228], [1, 117, 64], [176, 224, 217], [204, 145, 234], [219, 141, 182], [241, 66, 234], [96, 138, 192], [185, 225, 230], [109, 123, 14], [0, 23, 34], [183, 154, 199], [92, 191, 44], [64, 32, 203], [53, 18, 158], [134, 102, 236], [119, 154, 102], [175, 47, 177], [94, 194, 229], [44, 110, 165], [150, 152, 50], [244, 142, 34], [41, 207, 56], [31, 206, 108], [167, 91, 50], [214, 27, 58], [209, 180, 194], [77, 226, 109], [106, 203, 133], [26, 72, 198], [60, 138, 206], [225, 247, 167], [156, 245, 132], [93, 229, 159], [54, 246, 215], [137, 105, 18], [175, 149, 11], [133, 57, 92], [115, 135, 206], [33, 51, 2], [56, 168, 154], [144, 197, 207], [5, 124, 48], [233, 217, 149], [185, 52, 145], [136, 32, 240], [29, 251, 250], [50, 160, 231], [62, 13, 120], [70, 231, 157], [41, 29, 142], [152, 168, 11], [84, 245, 126], [36, 226, 118], [106, 156, 73], [18, 193, 147], [251, 158, 205], [170, 106, 215], [156, 36, 51], [189, 170, 204], [183, 74, 16], [206, 165, 194], [23, 37, 182], [173, 103, 231], [42, 66, 153], [119, 20, 115], [151, 81, 116], [24, 191, 224], [150, 232, 88], [232, 218, 201], [118, 200, 107], [134, 174, 244], [193, 147, 43], [59, 153, 202], [25, 131, 160], [186, 121, 59], [119, 120, 233], [46, 3, 217], [240, 68, 168], [37, 188, 2], [73, 54, 54], [19, 179, 142], [4, 127, 69], [219, 72, 34], [151, 40, 165], [205, 130, 243], [141, 204, 138], [10, 52, 89], [63, 6, 174], [157, 80, 121], [166, 157, 73], [130, 21, 189], [218, 110, 199], [41, 38, 119], [115, 95, 166], [119, 200, 10], [183, 2, 117], [224, 179, 38], [2, 162, 174], [136, 15, 17], [51, 30, 52], [99, 163, 150], [141, 235, 129], [241, 23, 123], [41, 50, 38], [88, 212, 151], [224, 85, 73], [217, 133, 117], [145, 39, 118], [8, 68, 161], [11, 112, 41], [197, 1, 97], [57, 111, 238], [33, 174, 52], [110, 56, 9], [13, 20, 51], [238, 69, 60], [171, 193, 99], [232, 165, 213], [70, 74, 189], [25, 17, 107], [111, 89, 110], [18, 198, 179], [124, 107, 29], [11, 210, 41], [174, 217, 112], [41, 171, 12], [213, 178, 93], [41, 184, 243], [95, 221, 22], [104, 200, 122], [254, 55, 95], [171, 104, 237], [51, 178, 139], [151, 89, 54], [182, 18, 89], [23, 49, 113], [79, 82, 207], [138, 194, 234], [127, 155, 79], [83, 177, 32], [124, 41, 213], [135, 19, 113], [10, 181, 211], [109, 231, 118], [3, 190, 154], [131, 4, 221], [76, 140, 226], [191, 172, 173], [35, 52, 196], [209, 213, 124], [250, 250, 217], [75, 209, 70], [110, 78, 115], [59, 159, 87], [201, 172, 107], [163, 119, 192], [53, 10, 42], [207, 241, 117], [22, 73, 153], [162, 176, 129], [60, 27, 102], [160, 18, 118], [174, 215, 69], [162, 18, 251], [132, 117, 173], [16, 248, 100], [238, 167, 218], [65, 55, 249], [184, 108, 234], [134, 161, 10], [154, 86, 226], [42, 70, 191], [29, 177, 94], [225, 199, 27], [51, 245, 80], [21, 247, 193], [72, 179, 246], [225, 145, 39], [242, 230, 43], [144, 55, 12], [7, 70, 29], [152, 215, 175], [155, 83, 130], [254, 75, 78], [90, 83, 197], [71, 231, 66], [216, 160, 174], [65, 160, 138], [165, 48, 135], [176, 123, 36], [153, 171, 90], [188, 227, 58], [5, 199, 107], [108, 90, 189], [188, 49, 122], [142, 145, 24], [94, 169, 54], [30, 183, 152], [53, 204, 101], [185, 51, 204], [16, 180, 253], [137, 251, 251], [122, 112, 222], [140, 75, 124], [125, 174, 151], [171, 121, 48], [125, 240, 138], [176, 6, 53], [161, 225, 167], [133, 214, 80], [215, 24, 61], [146, 19, 172], [241, 38, 120], [13, 231, 99], [71, 31, 250], [154, 242, 120], [187, 26, 86], [21, 67, 231], [116, 23, 140], [18, 139, 179], [114, 65, 39], [28, 140, 68], [105, 175, 150], [203, 249, 30], [95, 127, 240], [66, 174, 242], [239, 221, 81], [235, 102, 145], [169, 254, 2], [109, 171, 116], [72, 32, 121], [185, 94, 138], [130, 249, 119], [136, 3, 96], [208, 16, 255], [223, 184, 151], [85, 215, 227], [195, 236, 253], [130, 241, 148], [174, 171, 157], [236, 173, 68], [35, 226, 104], [68, 172, 205], [255, 9, 159], [75, 199, 114], [209, 143, 67], [171, 51, 47], [246, 224, 164], [185, 162, 225], [252, 23, 57], [100, 177, 73], [13, 156, 156], [126, 177, 154], [6, 173, 126], [149, 220, 214], [233, 131, 74], [33, 249, 38], [105, 169, 219], [154, 118, 45], [8, 31, 195], [253, 5, 140], [61, 216, 212], [146, 60, 135], [198, 95, 183], [234, 223, 127], [216, 168, 253], [244, 67, 30], [182, 52, 180], [187, 185, 0], [218, 145, 187], [149, 210, 89], [251, 168, 133], [179, 197, 95], [153, 68, 117], [32, 71, 62], [194, 255, 191], [114, 18, 176], [84, 158, 209], [98, 223, 188], [109, 223, 52], [152, 78, 252], [44, 133, 215], [115, 91, 76], [211, 44, 14], [110, 167, 17], [221, 126, 90], [249, 66, 135], [231, 137, 65], [244, 144, 158], [244, 246, 34], [12, 135, 226], [83, 214, 154], [229, 234, 254], [112, 210, 217], [6, 127, 165], [171, 238, 37], [52, 204, 212], [11, 200, 138], [120, 148, 30], [194, 49, 232], [37, 69, 82], [249, 121, 92], [228, 45, 69], [99, 121, 161], [156, 103, 61], [177, 201, 30], [147, 9, 22], [61, 116, 97], [178, 104, 87], [209, 250, 114], [25, 217, 79], [139, 79, 229], [210, 143, 194], [191, 69, 50], [218, 140, 36], [24, 250, 0], [248, 8, 218], [130, 184, 95], [185, 45, 46], [117, 126, 192], [191, 226, 127], [57, 53, 238], [21, 131, 210], [101, 222, 216], [162, 88, 28], [17, 189, 247], [28, 80, 148], [33, 110, 226], [170, 198, 119], [223, 133, 17], [177, 120, 118], [216, 231, 154], [17, 85, 208], [37, 221, 5], [221, 211, 156], [231, 33, 12], [17, 255, 22], [8, 167, 223], [71, 141, 63], [108, 245, 125], [127, 216, 84], [39, 161, 254], [44, 137, 60], [226, 6, 103], [43, 26, 22], [22, 60, 10], [49, 146, 157], [74, 128, 28], [245, 38, 60], [199, 61, 114], [144, 32, 157], [122, 174, 23], [15, 13, 145], [107, 84, 250], [143, 221, 218], [252, 4, 173], [199, 189, 15], [17, 127, 19], [20, 15, 50], [76, 81, 229], [159, 113, 96], [104, 81, 168], [205, 7, 150], [6, 65, 49], [220, 166, 232], [244, 100, 20], [162, 98, 237], [67, 190, 97], [230, 10, 233], [89, 141, 156], [12, 2, 146], [103, 139, 82], [125, 119, 235], [6, 111, 89], [181, 37, 48], [75, 234, 209], [81, 109, 153], [115, 56, 194], [112, 141, 231], [34, 43, 85], [125, 191, 244], [98, 104, 240], [182, 125, 42], [207, 229, 130], [150, 109, 98], [18, 192, 251], [174, 7, 16], [136, 239, 23], [78, 22, 140], [242, 80, 225], [146, 3, 11], [180, 119, 125], [39, 209, 160], [254, 184, 153], [76, 198, 141], [152, 117, 114], [215, 76, 3], [181, 202, 248], [10, 192, 207], [206, 40, 158], [152, 44, 90], [167, 44, 188], [94, 177, 105], [209, 3, 160], [40, 101, 18], [81, 45, 125], [190, 19, 44], [88, 53, 91], [212, 8, 178], [234, 134, 197], [249, 88, 36], [84, 54, 84], [152, 240, 181], [253, 26, 207], [208, 210, 216], [186, 55, 118], [167, 97, 205], [201, 71, 125], [87, 77, 89], [15, 66, 240], [184, 211, 134], [180, 38, 220], [158, 84, 213], [242, 107, 134], [240, 240, 39], [61, 154, 26], [199, 127, 0], [124, 110, 103], [179, 214, 249], [192, 133, 146], [148, 198, 208], [216, 83, 196], [148, 65, 21], [27, 6, 123], [226, 74, 50], [188, 74, 64], [180, 163, 84], [245, 128, 175], [58, 59, 116], [147, 193, 33], [108, 105, 166], [144, 35, 109], [250, 7, 26], [152, 26, 175], [40, 140, 204], [243, 138, 60], [202, 75, 17], [105, 26, 29], [194, 21, 56], [23, 47, 123], [10, 230, 76], [53, 255, 34], [91, 136, 189], [23, 16, 161], [32, 24, 108], [50, 13, 95], [114, 15, 109], [194, 35, 154], [25, 207, 27], [69, 96, 93], [213, 31, 234], [21, 50, 217], [247, 25, 160], [230, 143, 154], [243, 72, 179], [104, 211, 210], [78, 118, 0], [200, 116, 192], [77, 206, 119], [238, 79, 18], [191, 171, 233], [213, 201, 102], [77, 37, 131], [96, 169, 129], [100, 150, 65], [241, 228, 34], [113, 153, 60], [30, 216, 23], [61, 69, 21], [47, 120, 19], [110, 227, 18], [40, 81, 114], [13, 253, 178], [110, 63, 151], [66, 45, 5], [121, 212, 122], [220, 218, 250], [180, 145, 35], [24, 140, 67], [181, 60, 89], [10, 116, 10], [172, 4, 33], [108, 52, 189], [199, 211, 24], [49, 150, 58], [230, 157, 133], [220, 173, 57], [213, 186, 53], [54, 77, 55], [137, 190, 245], [36, 53, 221], [132, 55, 130], [219, 9, 15], [97, 30, 92], [76, 108, 37], [42, 84, 232], [236, 52, 16], [70, 115, 243], [56, 128, 105], [12, 252, 209], [38, 120, 129], [46, 41, 125], [164, 74, 180], [59, 208, 74], [49, 62, 148], [217, 97, 179], [177, 67, 84], [160, 39, 144], [89, 182, 82], [59, 246, 235], [73, 41, 245], [43, 83, 20], [245, 22, 237], [137, 226, 207], [233, 110, 78], [143, 195, 2], [171, 112, 12], [25, 147, 6], [98, 118, 82], [88, 239, 88], [79, 141, 207], [242, 50, 239], [172, 170, 134], [245, 82, 220], [86, 110, 186], [163, 164, 36], [185, 163, 114], [52, 97, 208], [22, 29, 243], [180, 203, 182], [88, 57, 168], [82, 18, 23], [133, 101, 231], [197, 47, 45], [75, 226, 91], [48, 147, 42], [29, 239, 212], [205, 204, 4], [156, 189, 34], [226, 96, 80], [131, 69, 59], [3, 12, 234], [60, 108, 146], [1, 32, 160], [198, 71, 59], [22, 6, 14], [107, 137, 121], [202, 251, 249], [62, 97, 136], [172, 144, 191], [244, 79, 13], [118, 96, 108], [78, 3, 199], [66, 61, 64], [208, 31, 137], [208, 42, 245], [153, 65, 16], [127, 98, 26], [21, 19, 59], [198, 59, 48], [105, 15, 211], [131, 60, 102], [156, 47, 153], [210, 101, 107], [123, 71, 0], [26, 85, 46], [1, 45, 120], [93, 234, 230], [41, 12, 72], [226, 183, 68], [0, 182, 100], [51, 129, 9], [220, 160, 212], [150, 132, 76], [15, 193, 142], [38, 220, 124], [169, 86, 57], [51, 181, 192], [124, 208, 184], [103, 176, 119], [10, 207, 21], [174, 43, 129], [43, 74, 31], [12, 193, 1], [108, 2, 229], [210, 207, 135], [28, 243, 219], [172, 122, 122], [177, 129, 136], [91, 181, 204], [203, 82, 203], [22, 129, 246], [162, 254, 213], [109, 68, 22], [15, 210, 35], [56, 250, 60], [58, 137, 24], [37, 214, 62], [72, 175, 65], [61, 74, 175], [168, 57, 176], [97, 215, 27], [100, 242, 71], [29, 32, 16], [219, 93, 87], [159, 90, 228], [231, 38, 193], [191, 129, 130], [4, 197, 125], [191, 233, 146], [35, 143, 216], [109, 127, 118], [106, 227, 46], [17, 48, 2], [159, 115, 210], [93, 91, 160], [38, 117, 253], [33, 148, 208], [241, 247, 247], [113, 12, 23], [171, 2, 65], [45, 159, 220], [147, 112, 28], [38, 231, 147], [131, 126, 234], [13, 44, 90], [242, 58, 6], [196, 165, 128], [15, 135, 196], [105, 202, 184], [160, 101, 89], [100, 189, 90], [181, 102, 95], [244, 52, 189], [242, 176, 128], [198, 232, 112], [205, 172, 180], [113, 43, 75], [229, 227, 174], [13, 168, 212], [33, 8, 36], [71, 237, 191], [238, 6, 63], [150, 13, 192], [57, 21, 24], [8, 48, 83], [68, 194, 21], [184, 223, 255], [146, 85, 234], [1, 65, 8], [96, 161, 35], [74, 46, 106], [52, 32, 15], [52, 166, 194], [59, 235, 8], [97, 56, 49], [71, 223, 113], [101, 59, 93], [135, 61, 134], [89, 156, 160], [138, 189, 29], [161, 105, 211], [17, 213, 248], [60, 43, 239], [64, 97, 229], [58, 63, 247], [3, 255, 215], [71, 66, 27], [233, 153, 75], [99, 141, 228], [213, 242, 255], [51, 207, 76], [184, 228, 195], [118, 98, 102], [117, 34, 47], [83, 16, 33], [215, 62, 252], [238, 243, 30], [197, 65, 155], [236, 109, 232], [159, 159, 185], [244, 118, 255], [109, 48, 3], [165, 184, 137], [165, 247, 210], [16, 174, 166], [167, 157, 207], [122, 199, 63], [18, 167, 33], [89, 239, 179], [246, 88, 21], [92, 140, 176], [49, 203, 134], [95, 61, 180], [108, 68, 254], [35, 253, 252], [220, 108, 136], [242, 187, 65], [253, 200, 171], [27, 116, 76], [70, 71, 202], [188, 49, 219], [72, 152, 218], [86, 37, 63], [7, 242, 141], [219, 235, 187], [162, 61, 58], [37, 201, 112], [212, 96, 195], [155, 191, 230], [152, 12, 96], [104, 97, 48], [61, 30, 170], [67, 244, 133], [146, 72, 133], [45, 5, 22], [4, 56, 197], [21, 55, 115], [185, 166, 185], [101, 4, 151], [165, 196, 158], [149, 126, 240], [206, 70, 28], [251, 253, 66], [105, 47, 125], [168, 216, 8], [47, 228, 171], [87, 53, 241], [77, 136, 161], [13, 170, 240], [98, 180, 189], [195, 149, 200], [166, 113, 177], [16, 232, 248], [85, 203, 70], [109, 92, 87], [93, 98, 28], [22, 52, 234], [105, 195, 73], [82, 226, 130], [27, 227, 196], [194, 36, 10], [220, 50, 118], [175, 157, 194], [34, 67, 165], [63, 148, 98], [95, 61, 88], [107, 70, 36], [14, 98, 169], [251, 243, 191], [157, 105, 178], [101, 6, 11], [22, 177, 206], [219, 210, 94], [170, 212, 174], [79, 8, 94], [228, 109, 82], [159, 217, 64], [16, 110, 201], [229, 134, 87], [194, 8, 47], [246, 38, 127], [230, 51, 79], [213, 165, 54], [173, 197, 129], [191, 240, 117], [193, 228, 147], [76, 139, 250], [211, 78, 4], [165, 185, 19], [23, 124, 63], [219, 84, 58], [253, 132, 155], [104, 168, 20], [166, 127, 72], [242, 143, 129], [64, 11, 85], [57, 186, 33], [164, 230, 78], [123, 50, 89], [212, 154, 123], [147, 106, 190], [151, 241, 109], [118, 33, 62], [154, 155, 148], [54, 253, 165], [111, 83, 112], [119, 247, 106], [28, 101, 79], [170, 23, 47], [69, 223, 203], [69, 112, 99], [64, 27, 247], [144, 87, 144], [133, 28, 9], [89, 100, 183], [131, 30, 240], [89, 234, 174], [140, 223, 8], [73, 186, 39], [100, 234, 186], [190, 7, 145], [202, 98, 208], [138, 232, 105], [197, 76, 143], [220, 124, 105], [233, 3, 98], [58, 11, 113], [9, 126, 60], [182, 131, 13], [28, 104, 234], [71, 137, 11], [94, 171, 142], [194, 22, 38], [129, 213, 14], [116, 29, 179], [194, 122, 138], [53, 48, 79], [194, 123, 241], [132, 158, 92], [34, 66, 37], [59, 85, 131], [183, 166, 47], [248, 167, 139], [132, 17, 103], [174, 42, 1], [37, 109, 50], [63, 138, 187], [149, 64, 81], [240, 127, 161], [190, 71, 177], [58, 226, 14], [209, 14, 195], [136, 83, 56], [225, 171, 217], [233, 173, 181], [61, 181, 15], [107, 197, 5], [91, 250, 162], [111, 200, 244], [65, 19, 155], [152, 14, 161], [174, 71, 171], [88, 154, 216], [33, 59, 190], [193, 43, 247], [166, 17, 91], [207, 42, 220], [78, 88, 41], [31, 81, 163], [72, 37, 229], [63, 79, 77], [60, 206, 20], [148, 126, 62], [73, 68, 89], [3, 66, 19], [77, 23, 234], [190, 237, 227], [84, 134, 176], [141, 223, 167], [192, 242, 250], [4, 112, 98], [172, 106, 126], [187, 82, 223], [70, 145, 30], [39, 231, 124], [204, 28, 99], [126, 57, 55], [60, 190, 93], [179, 90, 154], [24, 87, 235], [102, 162, 6], [35, 71, 177], [113, 66, 9], [198, 54, 164], [227, 34, 70], [218, 95, 11], [53, 126, 51], [208, 74, 144], [145, 246, 244], [204, 186, 166], [208, 127, 209], [208, 32, 168], [130, 23, 8], [177, 163, 159], [217, 116, 60], [204, 204, 142], [8, 174, 176], [71, 109, 219], [214, 3, 202], [60, 54, 29], [192, 9, 221], [150, 133, 243], [160, 55, 145], [222, 162, 74], [106, 45, 169], [162, 37, 121], [89, 33, 143], [118, 66, 42], [23, 215, 179], [66, 152, 254], [53, 30, 253], [23, 233, 147], [165, 226, 156], [236, 159, 10], [81, 9, 54], [10, 214, 49], [226, 65, 9], [196, 17, 81], [60, 189, 32], [176, 89, 177], [38, 160, 107], [63, 222, 244], [55, 223, 96], [70, 76, 150], [144, 22, 198], [229, 40, 52], [32, 5, 91], [128, 126, 63], [241, 8, 138], [139, 91, 198], [247, 7, 217], [193, 62, 243], [143, 74, 20], [38, 189, 127], [22, 158, 205], [154, 173, 152], [78, 191, 132], [10, 80, 38], [178, 162, 195], [160, 128, 163], [225, 191, 179], [61, 224, 185], [149, 192, 130], [231, 115, 221], [175, 139, 112], [164, 182, 83], [61, 193, 63], [13, 197, 234], [79, 32, 1], [73, 6, 94], [207, 87, 18], [100, 163, 227], [215, 176, 70], [71, 114, 124], [122, 126, 127], [96, 144, 27], [119, 60, 60], [219, 75, 22], [52, 47, 31], [18, 89, 96], [185, 128, 122], [137, 138, 127], [136, 2, 235], [40, 95, 60], [57, 218, 96], [113, 246, 36], [55, 80, 247], [245, 200, 134], [194, 238, 138], [239, 38, 9], [232, 150, 102], [36, 25, 143], [107, 232, 57], [158, 0, 209], [94, 92, 41], [89, 112, 155], [36, 21, 140], [212, 41, 191], [108, 108, 69], [161, 155, 213], [102, 55, 159], [19, 134, 153], [59, 120, 104], [61, 61, 51], [123, 23, 30], [28, 243, 105], [88, 97, 23], [116, 248, 119], [17, 232, 253], [37, 100, 228], [17, 62, 150], [244, 243, 113], [122, 41, 178], [98, 245, 209], [62, 195, 54], [158, 33, 67], [79, 89, 51], [105, 192, 91], [37, 228, 140], [108, 184, 229], [244, 35, 99], [174, 239, 140], [169, 127, 88], [135, 23, 140], [134, 205, 178], [157, 57, 4], [117, 238, 206], [207, 102, 84], [109, 20, 155], [123, 136, 4], [99, 109, 13], [177, 246, 19], [30, 174, 123], [47, 39, 174], [100, 73, 202], [228, 82, 222], [38, 174, 15], [1, 71, 217], [223, 119, 249], [132, 26, 80], [3, 214, 242], [65, 63, 28], [167, 106, 119], [123, 120, 176], [29, 43, 209], [226, 43, 76], [183, 69, 211], [81, 252, 1], [204, 53, 11], [224, 60, 171], [87, 132, 51], [150, 142, 42], [69, 157, 32], [50, 196, 57], [226, 78, 136], [163, 63, 146], [226, 161, 184], [199, 102, 145], [100, 119, 4], [230, 45, 143], [174, 193, 15], [182, 138, 246], [107, 157, 38], [206, 76, 121], [99, 147, 254], [146, 106, 220], [36, 136, 40], [157, 30, 244], [176, 224, 45], [146, 26, 165], [23, 14, 242], [177, 224, 233], [247, 144, 65], [199, 69, 49], [161, 88, 138], [225, 108, 134], [2, 78, 93], [94, 14, 116], [41, 17, 119], [203, 11, 119], [33, 106, 56], [235, 17, 100], [90, 134, 222], [11, 3, 4], [109, 54, 247], [20, 79, 105], [110, 239, 163], [149, 247, 88], [187, 80, 48], [59, 166, 159], [115, 45, 137], [15, 168, 69], [115, 248, 156], [158, 57, 62], [95, 105, 204], [105, 126, 64], [254, 203, 213], [5, 252, 104], [38, 152, 161], [191, 246, 155], [36, 233, 120], [85, 85, 31], [169, 230, 109], [211, 71, 27], [105, 209, 58], [20, 21, 134], [179, 0, 77], [76, 253, 120], [155, 172, 150], [184, 66, 59], [220, 170, 95], [119, 224, 106], [215, 173, 66], [139, 252, 175], [123, 16, 155], [114, 228, 44], [246, 120, 106], [238, 75, 212], [200, 110, 240], [174, 121, 240], [58, 125, 65], [149, 71, 105], [31, 62, 91], [204, 120, 41], [229, 71, 59], [42, 88, 241], [184, 13, 152], [101, 66, 168], [82, 53, 101], [244, 233, 93], [40, 69, 119], [184, 31, 39], [222, 47, 253], [13, 18, 66], [178, 249, 253], [102, 80, 72], [186, 210, 31], [34, 150, 35], [4, 221, 22], [42, 158, 22], [3, 43, 237], [23, 16, 84], [89, 26, 142], [50, 160, 233], [131, 47, 144], [114, 139, 51], [55, 123, 86], [14, 62, 146], [234, 229, 58], [45, 157, 228], [200, 143, 76], [108, 216, 161], [37, 181, 132], [224, 155, 96], [160, 220, 208], [68, 21, 219], [211, 232, 32], [99, 53, 134], [23, 154, 192], [17, 23, 82], [32, 199, 219], [134, 79, 208], [143, 123, 61], [119, 74, 63], [128, 73, 194], [104, 94, 7], [206, 22, 212], [209, 30, 76], [77, 167, 53], [104, 122, 237], [99, 149, 177], [56, 57, 26], [20, 121, 54], [215, 238, 42], [199, 29, 71], [210, 190, 240], [152, 145, 169], [160, 183, 242], [240, 246, 29], [48, 96, 78], [85, 193, 192], [242, 96, 68], [227, 231, 217], [145, 31, 196], [67, 200, 184], [233, 15, 223], [12, 25, 114], [51, 65, 71], [103, 38, 242], [241, 8, 189], [69, 56, 76], [24, 178, 138], [137, 243, 120], [54, 17, 7], [81, 69, 20], [60, 17, 107], [145, 68, 252], [168, 184, 161], [161, 248, 166], [4, 190, 169], [13, 193, 200], [172, 58, 144], [218, 153, 212], [185, 174, 194], [118, 41, 213], [214, 251, 55], [35, 252, 21], [199, 194, 140], [212, 145, 32], [200, 102, 4], [226, 19, 108], [221, 151, 58], [135, 143, 15], [81, 225, 23], [131, 226, 45], [165, 134, 83], [63, 32, 212], [33, 82, 168], [223, 0, 14], [100, 217, 241], [146, 212, 178], [63, 215, 193], [154, 3, 70], [64, 62, 136], [15, 105, 228], [249, 27, 50], [134, 126, 166], [6, 165, 160], [75, 220, 200], [7, 150, 142], [74, 255, 160], [9, 173, 152], [185, 153, 228], [21, 216, 51], [35, 153, 80], [192, 210, 106], [209, 143, 55], [232, 181, 201], [23, 101, 79], [246, 235, 248], [99, 139, 26], [23, 81, 128], [8, 202, 197], [242, 124, 77], [29, 81, 12], [168, 219, 244], [169, 252, 15], [66, 68, 242], [90, 84, 106], [244, 45, 35], [93, 59, 204], [134, 202, 251], [129, 20, 169], [90, 207, 79], [209, 38, 23], [63, 224, 160], [1, 49, 202], [90, 105, 188], [223, 167, 94], [235, 237, 192], [141, 25, 79], [34, 181, 112], [147, 18, 162], [130, 22, 217], [157, 221, 223], [54, 165, 123], [71, 207, 239], [27, 142, 204], [252, 73, 112], [167, 151, 204], [126, 104, 101], [104, 203, 72], [114, 213, 231], [197, 118, 116], [142, 166, 233], [167, 14, 144], [186, 129, 137], [2, 156, 190], [160, 138, 153], [137, 50, 177], [156, 10, 173], [78, 176, 122], [212, 180, 224], [162, 245, 43], [224, 86, 139], [178, 107, 198], [134, 32, 58], [42, 111, 237], [238, 42, 14], [104, 252, 137], [115, 92, 185], [179, 37, 41], [114, 17, 86], [77, 88, 61], [90, 28, 242], [46, 143, 18], [225, 216, 95], [84, 207, 216], [229, 133, 24], [128, 65, 18], [157, 78, 92], [3, 52, 208], [11, 189, 89], [86, 28, 245], [11, 248, 145], [210, 13, 94], [85, 79, 177], [62, 229, 12], [75, 204, 185], [145, 163, 14], [249, 57, 143], [184, 111, 243], [109, 77, 33], [230, 145, 181], [175, 224, 215], [209, 125, 121], [178, 221, 47], [215, 146, 105], [196, 173, 255], [233, 142, 143], [221, 62, 141], [193, 159, 212], [126, 106, 166], [131, 8, 20], [43, 237, 169], [76, 97, 15], [87, 155, 6], [48, 4, 242], [63, 199, 93], [31, 130, 161], [217, 66, 95], [170, 145, 236], [25, 102, 10], [162, 145, 104], [5, 250, 45], [171, 170, 143], [92, 201, 137], [240, 45, 115], [158, 6, 101], [90, 194, 135], [148, 229, 201], [211, 83, 5], [202, 72, 145], [17, 124, 184], [148, 58, 113], [117, 97, 90], [216, 34, 108], [72, 149, 148], [253, 204, 56], [241, 248, 64], [95, 18, 132], [167, 223, 249], [176, 246, 166], [18, 30, 39], [14, 107, 191], [122, 79, 245], [5, 61, 198], [34, 44, 28], [91, 214, 101], [174, 79, 25], [172, 89, 238], [29, 204, 60], [203, 252, 25], [108, 5, 190], [9, 118, 211], [83, 41, 182], [237, 227, 156], [74, 169, 40], [37, 44, 64], [196, 37, 145], [132, 197, 60], [213, 14, 148], [33, 164, 246], [250, 172, 219], [98, 219, 10], [249, 234, 39], [120, 67, 171], [181, 27, 245], [168, 190, 63], [194, 69, 205], [250, 250, 107], [38, 249, 145], [168, 81, 185], [66, 144, 0], [220, 109, 242], [241, 38, 132], [135, 152, 178], [169, 17, 3], [20, 171, 142], [141, 44, 145], [147, 150, 23], [98, 131, 129], [251, 159, 93], [3, 175, 112], [10, 30, 172], [146, 157, 160], [45, 73, 214], [224, 211, 33], [114, 130, 158], [135, 191, 203], [234, 1, 25], [13, 171, 96], [220, 207, 138], [22, 138, 238], [65, 50, 88], [160, 80, 69], [136, 36, 223], [231, 124, 165], [197, 60, 239], [118, 41, 56], [8, 13, 11], [249, 236, 173], [251, 49, 149], [133, 4, 41], [208, 70, 33], [239, 155, 179], [145, 92, 1], [176, 138, 244], [237, 223, 140], [19, 114, 58], [167, 1, 8], [116, 184, 63], [218, 194, 39], [162, 188, 245], [52, 97, 59], [104, 13, 20], [103, 181, 48], [97, 244, 69], [126, 14, 67], [15, 165, 193], [175, 75, 116], [245, 17, 108], [199, 107, 153], [139, 209, 194], [102, 3, 149], [219, 68, 149], [232, 41, 0], [228, 137, 179], [54, 241, 238], [109, 125, 35], [172, 135, 48], [246, 52, 131], [192, 14, 69], [169, 20, 64], [86, 229, 39], [67, 217, 197], [14, 50, 7], [223, 171, 90], [57, 242, 63], [215, 148, 28], [47, 176, 51], [180, 176, 193], [103, 50, 60], [124, 85, 146], [233, 180, 202], [211, 187, 220], [115, 28, 51], [188, 231, 25], [185, 42, 162], [241, 212, 204], [87, 164, 126], [245, 47, 202], [169, 160, 250], [9, 156, 153], [101, 27, 200], [224, 220, 253], [224, 146, 217], [75, 217, 153], [22, 75, 70], [147, 235, 110], [97, 164, 36], [163, 88, 27], [29, 81, 222], [240, 196, 186], [30, 41, 56], [145, 162, 38], [70, 117, 241], [225, 111, 233], [176, 18, 146], [204, 52, 204], [172, 68, 163], [29, 197, 192], [43, 210, 24], [129, 147, 34], [152, 86, 192], [144, 124, 75], [86, 107, 12], [119, 68, 240], [242, 112, 188], [96, 198, 111], [253, 57, 18], [180, 164, 116], [109, 88, 128], [221, 172, 254], [73, 81, 87], [3, 219, 127], [61, 89, 20], [141, 48, 219], [157, 130, 84], [136, 137, 142], [114, 173, 149], [168, 147, 179], [103, 180, 127], [60, 53, 151], [227, 157, 121], [173, 146, 194], [23, 173, 53], [156, 183, 168], [57, 2, 72], [131, 62, 21], [87, 23, 33], [223, 66, 165], [27, 53, 182], [183, 234, 241], [221, 38, 238], [145, 192, 249], [12, 127, 132], [10, 219, 235], [5, 95, 60], [87, 66, 119], [177, 57, 253], [132, 6, 127], [107, 71, 180], [156, 237, 54], [77, 120, 98], [39, 120, 171], [8, 6, 119], [214, 84, 83], [47, 236, 208], [65, 229, 203], [169, 149, 87], [211, 249, 183], [155, 154, 181], [117, 117, 94], [177, 3, 166], [50, 111, 26], [213, 124, 167], [174, 219, 207], [232, 21, 47], [113, 137, 199], [196, 220, 172], [171, 2, 82], [19, 4, 47], [1, 86, 36], [26, 246, 98], [247, 46, 77], [223, 67, 233], [209, 232, 106], [77, 57, 65], [198, 130, 59], [24, 88, 254], [37, 1, 153], [181, 172, 173], [183, 31, 137], [215, 160, 38], [203, 144, 3], [18, 21, 90], [9, 73, 121], [208, 163, 234], [105, 216, 67], [210, 37, 54], [180, 149, 81], [186, 184, 212], [29, 73, 143], [199, 252, 124], [145, 69, 152], [182, 93, 21], [161, 102, 243], [204, 51, 196], [131, 90, 132], [93, 190, 110], [71, 48, 169], [162, 14, 236], [84, 211, 16], [208, 252, 197], [242, 81, 241], [159, 14, 130], [196, 99, 72], [123, 65, 56], [84, 250, 154], [211, 161, 95], [107, 227, 152], [222, 121, 18], [112, 158, 92], [178, 84, 144], [30, 230, 237], [138, 0, 208], [83, 176, 245], [30, 131, 88], [168, 63, 160], [30, 125, 156], [182, 113, 143], [101, 36, 32], [160, 248, 47], [183, 218, 66], [34, 84, 116], [68, 37, 239], [236, 111, 188], [93, 128, 45], [141, 249, 132], [31, 31, 161], [250, 29, 141], [2, 22, 27], [123, 3, 0], [147, 154, 242], [106, 191, 148], [22, 84, 13], [208, 97, 122], [133, 81, 161], [251, 34, 6], [197, 170, 183], [177, 118, 85], [116, 255, 111], [223, 226, 164], [4, 85, 98], [95, 213, 88], [34, 57, 210], [142, 185, 225], [87, 174, 133], [157, 208, 72], [142, 55, 197], [238, 54, 114], [107, 144, 5], [101, 69, 219], [185, 214, 120], [228, 128, 157], [211, 43, 60], [29, 54, 228], [242, 243, 58], [101, 144, 90], [194, 40, 96], [155, 103, 3], [167, 85, 99], [97, 69, 238], [188, 38, 141], [218, 177, 38], [90, 127, 132], [46, 12, 112], [152, 80, 175], [64, 111, 22], [167, 150, 76], [130, 1, 209], [211, 248, 45], [95, 202, 135], [205, 125, 112], [200, 134, 68], [142, 60, 233], [56, 109, 203], [1, 175, 65], [102, 164, 203], [30, 18, 192], [2, 7, 213], [188, 185, 69], [247, 240, 92], [166, 95, 102], [57, 92, 34], [89, 152, 166], [111, 136, 21], [195, 230, 238], [0, 39, 254], [18, 110, 80], [176, 220, 184], [155, 45, 98], [93, 74, 214], [108, 127, 13], [50, 5, 195], [55, 211, 86], [66, 214, 38], [194, 191, 27], [220, 148, 78], [132, 186, 67], [29, 62, 33], [238, 214, 194], [136, 77, 153], [241, 246, 122], [210, 47, 213], [59, 230, 51], [196, 158, 165], [190, 100, 219], [135, 197, 145], [253, 228, 38], [63, 168, 100], [136, 221, 83], [39, 217, 82], [202, 116, 206], [169, 131, 219], [213, 185, 71], [212, 2, 223], [168, 130, 145], [47, 23, 101], [201, 149, 244], [98, 126, 29], [253, 186, 88], [78, 30, 155], [68, 130, 158], [247, 141, 198], [26, 128, 64], [74, 56, 175], [138, 95, 77], [79, 67, 196], [201, 34, 153], [187, 242, 231], [177, 24, 162], [36, 2, 144], [113, 155, 158], [47, 117, 247], [89, 209, 144], [218, 8, 109], [106, 178, 244], [193, 9, 79], [51, 14, 85], [164, 205, 46], [90, 203, 147], [45, 219, 132], [44, 185, 200], [212, 169, 134], [214, 122, 95], [35, 125, 189], [230, 252, 71], [195, 160, 185], [4, 91, 162], [0, 50, 147], [58, 99, 193], [128, 165, 194], [22, 44, 186], [158, 66, 198], [143, 16, 129], [201, 17, 105], [147, 138, 128], [254, 238, 73], [253, 233, 4], [5, 115, 117], [88, 253, 195], [152, 155, 144], [106, 238, 185], [12, 71, 218], [222, 237, 7], [48, 28, 121], [159, 98, 143], [4, 66, 78], [246, 214, 194], [204, 243, 154], [129, 53, 130], [126, 96, 168], [81, 93, 44], [91, 105, 152], [237, 4, 138], [175, 15, 161], [152, 68, 80], [24, 117, 51], [212, 28, 98], [245, 56, 255], [144, 29, 213], [104, 244, 33], [83, 88, 176], [78, 17, 61], [69, 125, 128], [230, 144, 207], [29, 203, 86], [94, 226, 21], [130, 10, 31], [118, 153, 227], [104, 234, 132], [149, 130, 186], [43, 195, 116], [140, 167, 242], [98, 114, 170], [91, 9, 47], [25, 176, 164], [183, 189, 114], [39, 221, 67], [149, 237, 34], [5, 208, 237], [21, 116, 231], [67, 35, 159], [213, 10, 150], [109, 35, 171], [247, 119, 209], [194, 160, 100], [17, 213, 33], [248, 227, 51], [249, 131, 153], [194, 208, 170], [240, 250, 118], [67, 14, 143], [119, 51, 243], [175, 226, 213], [236, 67, 99], [51, 34, 224], [11, 187, 179], [141, 179, 247], [154, 147, 104], [15, 201, 39], [125, 144, 123], [230, 108, 210], [53, 235, 171], [120, 57, 57], [112, 166, 152], [21, 40, 221], [120, 117, 202], [230, 112, 228], [48, 29, 192], [240, 65, 237], [154, 174, 214], [41, 159, 159], [208, 63, 12], [192, 29, 86], [85, 5, 112], [9, 100, 234], [101, 68, 185], [235, 214, 61], [136, 229, 202], [179, 114, 67], [194, 232, 153], [196, 8, 137], [127, 3, 244], [241, 29, 248], [154, 66, 249], [70, 177, 252], [152, 122, 127], [151, 49, 149], [223, 213, 237], [131, 188, 183], [186, 162, 72], [118, 227, 8], [24, 215, 176], [244, 171, 186], [10, 164, 231], [82, 157, 142], [154, 50, 211], [69, 216, 21], [2, 2, 193], [128, 151, 125], [208, 245, 163], [17, 243, 244], [118, 94, 194], [100, 67, 78], [188, 149, 239], [195, 139, 101], [82, 25, 132], [87, 129, 231], [245, 106, 53], [198, 52, 212], [26, 214, 27], [213, 146, 104], [188, 86, 233], [242, 148, 137], [10, 81, 232], [55, 159, 248], [75, 91, 23], [191, 1, 92], [5, 121, 203], [9, 192, 145], [87, 240, 123], [47, 231, 116], [117, 31, 203], [183, 44, 199], [186, 93, 82], [18, 212, 193], [206, 118, 157], [77, 77, 24], [187, 92, 217], [249, 24, 201], [65, 139, 77], [208, 167, 5], [17, 221, 175], [22, 77, 122], [236, 67, 145], [228, 27, 181], [209, 67, 209], [62, 35, 148], [56, 99, 84], [134, 4, 189], [158, 48, 190], [23, 91, 216], [40, 211, 242], [166, 209, 235], [142, 155, 44], [105, 65, 190], [176, 88, 28], [73, 64, 130], [36, 209, 26], [59, 38, 161], [21, 167, 149], [146, 88, 141], [67, 58, 62], [66, 33, 247], [84, 254, 64], [88, 178, 14], [232, 14, 193], [43, 67, 130], [253, 231, 252], [77, 20, 112], [227, 186, 193], [46, 122, 29], [57, 236, 149], [68, 124, 103], [67, 38, 125], [134, 88, 88], [162, 33, 119], [175, 19, 185], [153, 198, 14], [26, 135, 142], [76, 78, 5], [68, 248, 116], [44, 217, 141], [20, 8, 250], [59, 182, 229], [104, 159, 52], [202, 24, 25], [223, 37, 246], [80, 53, 125], [166, 86, 66], [26, 103, 171], [222, 161, 229], [114, 24, 143], [123, 144, 139], [192, 117, 183], [116, 208, 19], [65, 24, 111], [228, 234, 149], [15, 228, 194], [198, 32, 72], [19, 151, 160], [104, 49, 50], [172, 72, 123], [124, 99, 16], [203, 53, 112], [115, 27, 7], [240, 185, 6], [103, 240, 235], [238, 184, 183], [224, 146, 68], [137, 166, 31], [2, 212, 197], [212, 25, 241], [2, 127, 238], [254, 4, 147], [8, 161, 220], [91, 139, 135], [185, 30, 120], [233, 220, 234], [154, 40, 23], [33, 21, 185], [25, 209, 76], [240, 28, 70], [98, 180, 173], [194, 72, 151], [102, 57, 30], [144, 60, 69], [213, 78, 167], [222, 138, 207], [108, 127, 245], [233, 127, 190], [140, 18, 29], [177, 42, 21], [189, 154, 70], [26, 230, 254], [88, 92, 178], [170, 26, 57], [72, 167, 165], [165, 5, 155], [102, 145, 167], [233, 68, 165], [185, 63, 218], [116, 9, 229], [240, 205, 255], [221, 239, 9], [160, 145, 28], [243, 226, 161], [239, 87, 230], [45, 132, 253], [232, 29, 142], [58, 251, 233], [147, 170, 221], [16, 98, 213], [161, 90, 218], [201, 35, 42], [167, 189, 101], [156, 53, 225], [49, 32, 211], [27, 131, 32], [171, 188, 188], [170, 201, 81], [7, 5, 24], [10, 202, 247], [230, 99, 20], [34, 69, 182], [32, 21, 103], [84, 195, 61], [28, 156, 91], [56, 211, 9], [221, 232, 78], [52, 96, 133], [13, 118, 223], [46, 18, 143], [92, 34, 182], [236, 34, 92], [235, 77, 158], [47, 26, 238], [139, 238, 90], [211, 128, 183], [249, 233, 125], [244, 131, 153], [191, 73, 138], [72, 226, 209], [168, 183, 217], [65, 203, 95], [176, 50, 106], [4, 167, 52], [82, 14, 235], [55, 252, 18], [123, 16, 109], [134, 9, 80], [46, 240, 36], [148, 50, 139], [125, 168, 7], [203, 86, 69], [143, 26, 154], [183, 222, 222], [226, 184, 31], [134, 82, 34], [234, 187, 29], [165, 227, 225], [151, 238, 46], [143, 0, 9], [147, 27, 152], [0, 140, 215], [238, 16, 161], [71, 7, 209], [247, 18, 144], [77, 96, 33], [255, 60, 243], [121, 14, 167], [218, 149, 227], [95, 204, 107], [230, 14, 161], [83, 159, 21], [142, 185, 226], [206, 177, 29], [116, 51, 229], [57, 187, 15], [226, 250, 193], [34, 29, 228], [27, 245, 134], [206, 11, 171], [251, 42, 114], [93, 183, 201], [109, 174, 98], [239, 181, 100], [39, 0, 141], [229, 36, 226], [75, 72, 83], [151, 150, 90], [195, 68, 193], [64, 184, 134], [69, 210, 34], [5, 80, 97], [166, 7, 109], [159, 235, 180], [144, 138, 4], [145, 169, 11], [241, 40, 42], [205, 91, 251], [98, 133, 193], [9, 122, 11], [4, 198, 197], [69, 106, 186], [15, 3, 130], [118, 188, 250], [253, 31, 55], [153, 186, 9], [121, 207, 236], [83, 21, 88], [14, 139, 243], [160, 45, 163], [128, 159, 99], [95, 188, 128], [222, 167, 80], [194, 171, 230], [157, 67, 68], [200, 200, 218], [33, 60, 68], [216, 62, 109], [246, 98, 201], [234, 249, 53], [242, 101, 9], [187, 13, 203], [82, 179, 89], [137, 193, 242], [74, 129, 121], [204, 218, 161], [226, 224, 82], [185, 101, 57], [213, 157, 229], [234, 244, 107], [108, 38, 127], [4, 251, 109], [5, 65, 169], [20, 112, 131], [5, 127, 180], [161, 210, 194], [63, 6, 218], [188, 50, 161], [102, 70, 21], [251, 252, 219], [50, 124, 225], [41, 253, 32], [34, 92, 102], [60, 248, 145], [65, 89, 204], [53, 99, 175], [20, 151, 38], [209, 75, 189], [232, 210, 31], [62, 233, 46], [104, 225, 226], [109, 58, 170], [201, 72, 55], [162, 204, 18], [195, 76, 160], [79, 215, 167], [170, 38, 207], [54, 55, 192], [174, 140, 101], [48, 239, 53], [166, 168, 95], [31, 26, 7], [99, 160, 189], [57, 3, 7], [52, 33, 95], [219, 167, 194], [159, 195, 6], [167, 64, 5], [54, 111, 19], [182, 135, 20], [18, 9, 95], [74, 41, 40], [222, 1, 136], [199, 123, 73], [201, 122, 114], [161, 63, 110], [54, 242, 133], [53, 203, 239], [100, 185, 9], [52, 157, 57], [166, 215, 254], [93, 235, 127], [230, 145, 118], [222, 115, 176], [64, 5, 68], [163, 9, 22], [2, 70, 99], [101, 1, 27], [16, 103, 42], [91, 79, 65], [106, 33, 163], [146, 190, 41], [203, 222, 16], [164, 8, 210], [54, 166, 35], [207, 85, 40], [93, 4, 178], [107, 219, 7], [174, 181, 229], [198, 42, 249], [13, 218, 55], [52, 37, 115], [237, 120, 169], [142, 120, 147], [143, 16, 11], [250, 156, 120], [96, 76, 52], [176, 174, 27], [242, 51, 113], [52, 171, 129], [36, 168, 122], [155, 218, 139], [139, 156, 215], [214, 155, 196], [86, 103, 249], [160, 120, 179], [48, 119, 50], [133, 182, 47], [16, 237, 82], [67, 55, 160], [40, 155, 233], [95, 203, 149], [47, 62, 74], [62, 151, 151], [92, 26, 138], [12, 229, 145], [211, 118, 127], [25, 191, 127], [136, 72, 87], [23, 91, 205], [243, 42, 189], [70, 241, 144], [207, 16, 241], [243, 42, 160], [73, 210, 158], [19, 90, 175], [0, 145, 105], [250, 16, 87], [73, 54, 164], [75, 232, 236], [24, 124, 13], [22, 30, 121], [114, 42, 115], [155, 95, 217], [204, 160, 111], [72, 113, 44], [138, 158, 113], [248, 201, 223], [248, 217, 93], [91, 73, 25], [229, 242, 106], [91, 150, 179], [0, 100, 163], [101, 46, 91], [121, 246, 72], [162, 246, 6], [89, 168, 171], [201, 30, 59], [232, 26, 38], [194, 88, 245], [225, 151, 95], [252, 195, 173], [109, 198, 190], [242, 241, 37], [208, 209, 84], [10, 128, 112], [231, 120, 152], [112, 244, 79], [195, 93, 232], [122, 148, 31], [251, 138, 7], [246, 42, 9], [190, 216, 6], [219, 244, 159], [18, 94, 196], [226, 176, 26], [165, 86, 171], [23, 180, 107], [85, 87, 125], [131, 91, 247], [55, 78, 190], [164, 7, 112], [18, 218, 179], [22, 181, 222], [119, 52, 92], [54, 115, 85], [105, 170, 243], [48, 85, 24], [127, 41, 185], [170, 177, 29], [182, 84, 117], [124, 216, 135], [255, 204, 225], [52, 26, 124], [129, 159, 82], [41, 216, 217], [110, 17, 217], [239, 233, 193], [124, 66, 129], [5, 30, 80], [179, 165, 99], [221, 175, 182], [116, 28, 144], [48, 55, 102], [196, 104, 47], [10, 150, 202], [129, 133, 135], [140, 137, 1], [15, 199, 33], [128, 43, 156], [34, 218, 150], [81, 10, 197], [57, 133, 9], [83, 28, 190], [140, 140, 239], [155, 1, 83], [161, 103, 236], [107, 94, 230], [222, 179, 249], [134, 92, 16], [131, 131, 43], [237, 156, 141], [99, 91, 81], [80, 237, 111], [201, 250, 76], [255, 151, 132], [29, 207, 84], [247, 112, 16], [30, 215, 70], [74, 145, 104], [46, 161, 19], [139, 165, 224], [16, 41, 206], [16, 39, 114], [164, 181, 28], [212, 70, 188], [2, 218, 237], [249, 213, 213], [197, 47, 102], [9, 180, 113], [254, 244, 219], [95, 44, 5], [161, 209, 204], [36, 79, 227], [108, 120, 91], [119, 141, 236], [61, 160, 194], [197, 105, 253], [59, 113, 95], [221, 80, 140], [149, 218, 90], [182, 238, 17], [104, 156, 170], [38, 86, 235], [188, 161, 64], [14, 27, 214], [182, 75, 185], [215, 50, 91], [26, 112, 106], [92, 255, 7], [137, 232, 242], [52, 63, 64], [7, 93, 23], [209, 73, 86], [50, 45, 130], [231, 186, 85], [22, 241, 167], [38, 118, 205], [99, 181, 52], [206, 179, 199], [156, 14, 41], [76, 163, 1], [105, 235, 7], [250, 195, 57], [5, 101, 18], [150, 74, 124], [5, 235, 234], [90, 209, 51], [112, 119, 5], [179, 107, 76], [142, 227, 164], [175, 251, 123], [33, 23, 164], [120, 23, 104], [153, 223, 91], [82, 169, 236], [174, 204, 144], [214, 15, 197], [5, 167, 255], [254, 196, 87], [23, 225, 152], [213, 188, 152], [155, 185, 159], [146, 173, 131], [247, 122, 107], [122, 158, 246], [188, 13, 67], [255, 18, 21], [110, 62, 48], [15, 76, 175], [128, 229, 49], [128, 101, 157], [168, 136, 225], [217, 205, 231], [64, 57, 165], [99, 134, 217], [197, 178, 171], [186, 247, 130], [88, 8, 182], [3, 114, 213], [187, 92, 63], [245, 110, 1], [118, 158, 89], [89, 173, 179], [38, 166, 34], [237, 9, 92], [25, 133, 253], [171, 189, 149], [189, 104, 236], [35, 241, 247], [128, 46, 231], [109, 229, 106], [79, 115, 77], [59, 238, 55], [63, 247, 165], [80, 176, 34], [193, 99, 160], [35, 109, 98], [131, 232, 191], [133, 64, 10], [106, 136, 201], [55, 21, 4], [84, 22, 173], [128, 179, 3], [157, 64, 178], [131, 186, 215], [211, 113, 9], [8, 121, 130], [41, 66, 162], [19, 233, 55], [69, 117, 194], [2, 9, 62], [228, 235, 11], [169, 168, 97], [68, 16, 112], [58, 126, 32], [247, 121, 78], [195, 93, 27], [51, 148, 195], [105, 247, 13], [111, 54, 92], [155, 48, 39], [28, 38, 163], [126, 195, 116], [183, 119, 178], [150, 99, 22], [92, 107, 248], [76, 117, 55], [188, 115, 127], [132, 70, 128], [77, 30, 68], [27, 69, 33], [197, 6, 139], [213, 178, 82], [110, 30, 156], [172, 21, 235], [16, 210, 42], [102, 181, 55], [56, 3, 5], [110, 136, 230], [239, 220, 118], [213, 94, 147], [208, 244, 58], [10, 133, 248], [67, 111, 17], [184, 57, 130], [251, 205, 47], [45, 113, 15], [226, 93, 124], [185, 130, 29], [91, 83, 162], [145, 19, 90], [210, 16, 110], [151, 36, 71], [179, 128, 237], [48, 54, 54], [29, 101, 112], [115, 39, 120], [166, 41, 243], [192, 83, 190], [145, 27, 59], [196, 74, 178], [228, 126, 33], [50, 116, 50], [200, 44, 16], [230, 21, 87], [206, 170, 119], [18, 231, 37], [78, 115, 177], [77, 187, 74], [175, 244, 206], [116, 68, 105], [179, 13, 171], [244, 60, 192], [217, 163, 145], [208, 236, 249], [232, 111, 232], [243, 53, 43], [123, 118, 215], [22, 250, 218], [198, 17, 43], [4, 52, 83], [9, 185, 187], [222, 220, 125], [60, 113, 122], [68, 164, 57], [57, 197, 30], [150, 135, 201], [19, 145, 55], [240, 107, 7], [189, 84, 252], [187, 195, 23], [120, 66, 104], [130, 87, 95], [127, 57, 114], [171, 112, 42], [45, 19, 57], [212, 32, 61], [206, 112, 153], [106, 249, 25], [26, 104, 148], [2, 128, 246], [12, 172, 113], [67, 163, 239], [209, 138, 152], [9, 163, 78], [194, 214, 10], [134, 184, 88], [154, 91, 106], [132, 191, 22], [227, 128, 173], [207, 105, 68], [102, 167, 206], [6, 90, 129], [179, 142, 239], [223, 151, 83], [87, 65, 69], [214, 247, 82], [100, 129, 23], [36, 117, 93], [69, 7, 178], [55, 103, 30], [187, 44, 137], [69, 108, 74], [43, 23, 128], [18, 42, 30], [112, 88, 162], [2, 205, 128], [153, 84, 102], [46, 54, 203], [174, 39, 217], [6, 10, 62], [57, 13, 238], [45, 70, 62], [70, 153, 191], [233, 80, 133], [207, 28, 41], [191, 254, 43], [130, 9, 180], [13, 206, 211], [210, 215, 226], [223, 82, 138], [57, 152, 153], [204, 219, 254], [132, 189, 139], [222, 223, 159], [26, 192, 223], [75, 134, 126], [101, 70, 234], [72, 230, 87], [58, 40, 203], [243, 97, 146], [80, 22, 229], [145, 101, 40], [226, 144, 203], [21, 192, 103], [42, 128, 160], [185, 80, 62], [230, 198, 108], [182, 21, 95], [88, 7, 58], [29, 186, 186], [39, 121, 152], [79, 47, 25], [108, 187, 68], [201, 246, 246], [22, 223, 129], [145, 44, 238], [44, 237, 71], [170, 86, 63], [204, 190, 97], [25, 48, 126], [179, 242, 108], [230, 76, 235], [81, 38, 105], [36, 227, 120], [242, 166, 17], [91, 180, 159], [107, 99, 110], [18, 237, 139], [246, 81, 8], [23, 48, 214], [109, 178, 35], [220, 121, 100], [56, 40, 31], [158, 255, 80], [76, 151, 13], [227, 186, 91], [94, 45, 221], [81, 108, 43], [164, 217, 182], [112, 227, 8], [17, 142, 253], [100, 108, 212], [161, 100, 76], [202, 160, 62], [24, 225, 11], [155, 230, 60], [150, 197, 8], [16, 68, 62], [114, 106, 102], [65, 172, 176], [195, 93, 29], [211, 116, 197], [158, 229, 122], [253, 217, 23], [31, 7, 188], [84, 142, 91], [232, 8, 101], [100, 48, 190], [84, 206, 50], [163, 69, 85], [146, 227, 69], [224, 7, 68], [73, 1, 253], [187, 31, 83], [144, 240, 128], [238, 173, 81], [245, 233, 83], [171, 165, 134], [199, 66, 187], [197, 134, 38], [62, 205, 122], [25, 22, 247], [225, 42, 48], [71, 30, 177], [153, 73, 167], [75, 131, 54], [179, 43, 151], [32, 10, 147], [255, 110, 101], [22, 69, 225], [138, 134, 1], [137, 184, 227], [223, 26, 198], [26, 157, 68], [123, 204, 238], [202, 102, 8], [151, 71, 74], [116, 194, 130], [59, 86, 230], [158, 20, 81], [229, 189, 238], [5, 88, 19], [145, 29, 47], [92, 47, 192], [52, 248, 2], [113, 30, 182], [119, 107, 179], [116, 99, 37], [187, 163, 149], [222, 194, 86], [110, 231, 126], [22, 93, 232], [232, 125, 138], [151, 221, 48], [238, 56, 145], [43, 100, 11], [144, 45, 77], [137, 11, 87], [159, 121, 138], [234, 62, 115], [239, 47, 255], [144, 54, 136], [178, 91, 180], [45, 128, 57], [71, 16, 168], [223, 89, 202], [123, 120, 182], [91, 233, 84], [249, 136, 153], [194, 145, 187], [14, 221, 90], [119, 54, 205], [78, 241, 18], [151, 206, 150], [210, 44, 108], [18, 32, 255], [147, 109, 95], [128, 18, 19], [255, 140, 79], [44, 82, 174], [52, 63, 87], [128, 155, 38], [158, 108, 37], [55, 208, 55], [74, 35, 111], [176, 45, 122], [1, 128, 243], [186, 56, 57], [214, 206, 0], [65, 60, 144], [183, 211, 184], [215, 41, 187], [121, 143, 181], [155, 136, 20], [103, 48, 76], [27, 137, 136], [78, 6, 226], [193, 209, 133], [176, 74, 187], [190, 107, 16], [35, 168, 182], [38, 39, 173], [20, 75, 243], [43, 95, 160], [56, 255, 35], [184, 250, 175], [114, 223, 163], [59, 76, 122], [34, 152, 6], [46, 68, 59], [209, 192, 169], [125, 136, 139], [195, 161, 99], [175, 182, 223], [230, 198, 17], [71, 86, 13], [87, 27, 160], [35, 29, 93], [113, 126, 90], [252, 68, 9], [114, 8, 180], [191, 127, 105], [32, 173, 32], [203, 52, 37], [189, 225, 44], [156, 63, 134], [138, 139, 22], [163, 92, 197], [34, 251, 143], [196, 102, 11], [96, 142, 58], [49, 115, 115], [90, 223, 31], [97, 180, 176], [143, 205, 29], [153, 119, 55], [149, 123, 45], [228, 193, 217], [31, 86, 171], [216, 38, 109], [228, 80, 122], [229, 31, 53], [49, 115, 195], [192, 239, 37], [62, 231, 84], [69, 37, 241], [116, 234, 224], [249, 84, 184], [165, 26, 221], [156, 98, 242], [190, 85, 159], [11, 75, 10], [156, 247, 0], [222, 55, 96], [242, 70, 88], [79, 159, 60], [121, 40, 100], [169, 157, 32], [35, 110, 5], [75, 216, 45], [41, 96, 70], [157, 57, 90], [167, 239, 132], [164, 226, 23], [193, 201, 82], [216, 168, 218], [75, 249, 62], [120, 82, 220], [170, 107, 35], [253, 22, 53], [128, 85, 199], [254, 164, 4], [108, 211, 79], [3, 200, 92], [88, 195, 215], [110, 57, 139], [238, 194, 225], [13, 251, 1], [80, 131, 38], [89, 128, 36], [78, 27, 7], [245, 16, 183], [207, 221, 211], [56, 111, 4], [138, 157, 41], [215, 54, 83], [227, 28, 145], [26, 140, 86], [43, 217, 89], [8, 216, 145], [85, 246, 163], [74, 75, 209], [170, 39, 157], [132, 179, 194], [144, 136, 242], [63, 22, 65], [134, 208, 223], [64, 69, 83], [2, 242, 146], [236, 32, 249], [39, 174, 113], [2, 38, 84], [68, 192, 140], [231, 247, 196], [49, 250, 53], [151, 67, 84], [110, 77, 14], [195, 239, 72], [38, 117, 25], [230, 202, 76], [7, 64, 54], [3, 218, 37], [81, 60, 64], [101, 9, 48], [132, 123, 182], [117, 187, 211], [134, 205, 40], [46, 207, 184], [62, 190, 208], [41, 102, 240], [74, 54, 37], [108, 168, 100], [41, 139, 201], [202, 115, 218], [230, 184, 82], [213, 86, 132], [63, 80, 167], [51, 116, 139], [212, 101, 154], [66, 168, 21], [11, 42, 134], [184, 245, 125], [103, 49, 165], [105, 135, 237], [8, 151, 10], [157, 187, 166], [206, 133, 202], [236, 75, 91], [49, 154, 61], [180, 189, 53], [182, 202, 78], [25, 154, 221], [114, 172, 108], [199, 255, 38], [149, 219, 181], [196, 127, 4], [65, 39, 208], [2, 126, 184], [152, 165, 141], [36, 204, 169], [95, 24, 101], [146, 193, 110], [235, 15, 173], [197, 49, 7], [178, 92, 96], [154, 71, 214], [165, 97, 171], [15, 176, 211], [240, 36, 123], [129, 103, 32], [122, 115, 250], [63, 165, 10], [146, 200, 3], [91, 231, 69], [170, 101, 42], [65, 180, 3], [154, 31, 32], [36, 202, 135], [37, 141, 97], [130, 88, 143], [170, 148, 219], [17, 227, 24], [66, 12, 187], [7, 230, 51], [121, 147, 43], [230, 174, 181], [98, 209, 25], [40, 29, 189], [239, 231, 150], [83, 122, 253], [230, 102, 84], [132, 174, 148], [186, 161, 218], [113, 90, 200], [54, 200, 112], [118, 171, 45], [117, 74, 133], [120, 232, 21], [82, 159, 230], [142, 67, 9], [226, 190, 151], [244, 86, 132], [217, 172, 153], [55, 172, 16], [52, 226, 180], [254, 122, 149], [34, 54, 32], [236, 156, 133], [169, 31, 103], [127, 69, 54], [107, 130, 50], [146, 60, 55], [121, 193, 3], [51, 219, 80], [46, 155, 60], [5, 58, 88], [29, 42, 147], [2, 62, 193], [30, 54, 35], [215, 52, 72], [251, 190, 247], [244, 247, 13], [98, 78, 36], [70, 52, 19], [209, 47, 124], [117, 29, 122], [196, 38, 44], [9, 71, 71], [114, 138, 73], [123, 227, 2], [205, 245, 23], [105, 108, 117], [35, 91, 37], [59, 133, 149], [79, 188, 15], [226, 192, 145], [67, 107, 197], [16, 6, 139], [255, 129, 10], [99, 85, 171], [200, 55, 214], [88, 3, 34], [219, 252, 83], [152, 13, 54], [97, 145, 156], [83, 198, 218], [171, 233, 90], [158, 121, 25], [118, 63, 148], [4, 159, 13], [175, 136, 159], [204, 194, 194], [187, 233, 224], [16, 88, 232], [38, 11, 195], [213, 93, 59], [104, 84, 178], [237, 91, 27], [141, 142, 34], [247, 46, 13], [188, 97, 162], [219, 189, 147], [123, 99, 179], [99, 206, 142], [78, 209, 239], [13, 245, 233], [190, 149, 89], [73, 97, 243], [55, 247, 144], [230, 221, 165], [161, 240, 34], [246, 251, 126], [30, 100, 72], [102, 27, 57], [249, 144, 198], [29, 105, 229], [148, 130, 64], [56, 49, 135], [111, 49, 203], [27, 156, 160], [133, 37, 2], [98, 4, 212], [16, 209, 204], [85, 240, 239], [14, 24, 254], [199, 169, 10], [213, 197, 160], [196, 82, 169], [126, 69, 120], [186, 114, 101], [75, 209, 165], [124, 3, 119], [165, 187, 152], [129, 114, 152], [244, 150, 29], [178, 137, 119], [143, 28, 5], [189, 252, 120], [23, 127, 90], [1, 80, 159], [109, 164, 215], [75, 213, 154], [62, 126, 173], [107, 118, 154], [180, 248, 73], [190, 36, 249], [83, 146, 247], [30, 164, 75], [102, 25, 119], [187, 12, 7], [155, 231, 3], [171, 81, 194], [238, 56, 240], [70, 14, 47], [248, 189, 12], [123, 242, 41], [13, 129, 100], [174, 234, 52], [189, 250, 245], [128, 236, 3], [117, 150, 209], [16, 188, 134], [18, 254, 193], [232, 198, 13], [190, 233, 186], [76, 101, 174], [169, 123, 189], [21, 139, 105], [152, 224, 94], [31, 100, 14], [95, 122, 33], [208, 141, 35], [94, 235, 159], [197, 252, 23], [213, 149, 130], [149, 223, 24], [211, 130, 204], [188, 216, 47], [88, 77, 165], [230, 192, 204], [240, 12, 111], [131, 123, 50], [235, 194, 123], [138, 127, 92], [0, 151, 60], [154, 131, 42], [101, 95, 169], [195, 161, 176], [62, 120, 27], [244, 126, 79], [122, 169, 187], [118, 3, 130], [153, 192, 111], [115, 171, 246], [211, 50, 245], [95, 48, 218], [7, 48, 198], [90, 167, 72], [169, 138, 221], [143, 28, 124], [224, 128, 187], [175, 247, 135], [186, 190, 74], [138, 30, 183], [246, 59, 121], [49, 89, 233], [116, 60, 231], [140, 245, 118], [71, 88, 119], [112, 61, 208], [129, 233, 28], [114, 70, 161], [147, 108, 138], [186, 161, 15], [43, 44, 193], [221, 77, 80], [60, 243, 164], [144, 79, 31], [68, 138, 65], [69, 180, 167], [0, 92, 168], [33, 67, 169], [78, 47, 235], [29, 21, 41], [124, 229, 37], [222, 136, 43], [106, 158, 187], [234, 91, 250], [51, 180, 141], [126, 77, 216], [4, 237, 169], [10, 60, 111], [76, 240, 85], [191, 187, 248], [190, 99, 55], [29, 194, 6], [98, 75, 158], [241, 102, 83], [57, 76, 122], [164, 183, 3], [33, 65, 27], [222, 89, 157], [252, 110, 194], [40, 105, 227], [205, 20, 204], [24, 222, 174], [101, 79, 207], [221, 97, 184], [186, 123, 112], [134, 228, 57], [231, 59, 48], [125, 22, 90], [227, 111, 136], [115, 243, 232], [228, 124, 18], [68, 95, 138], [17, 147, 105], [248, 23, 159], [29, 97, 124], [193, 75, 223], [89, 52, 157], [136, 200, 69], [59, 168, 134], [226, 89, 31], [173, 196, 252], [73, 234, 217], [150, 173, 4], [152, 227, 51], [23, 233, 144], [165, 225, 174], [116, 151, 35], [119, 130, 149], [27, 244, 204], [11, 140, 203], [85, 76, 243], [190, 247, 235], [46, 52, 65], [158, 251, 107], [121, 220, 32], [33, 179, 126], [40, 33, 119], [60, 222, 197], [202, 0, 249], [69, 153, 238], [201, 157, 33], [16, 42, 214], [118, 136, 205], [225, 51, 51], [221, 25, 71], [242, 91, 152], [169, 168, 59], [141, 219, 251], [67, 22, 247], [166, 213, 79], [28, 202, 142], [217, 147, 36], [115, 122, 67], [205, 196, 151], [222, 47, 1], [203, 128, 65], [173, 217, 146], [10, 93, 86], [160, 183, 235], [175, 75, 52], [213, 135, 233], [116, 221, 49], [211, 143, 9], [118, 203, 91], [124, 9, 112], [14, 220, 89], [128, 54, 50], [138, 28, 103], [113, 69, 31], [104, 218, 185], [31, 231, 6], [135, 202, 42], [84, 109, 95], [19, 126, 231], [128, 32, 177], [213, 168, 30], [116, 123, 139], [166, 44, 241], [28, 198, 239], [227, 224, 157], [107, 42, 160], [253, 252, 202], [65, 70, 80], [110, 80, 39], [175, 50, 24], [216, 122, 149], [74, 226, 85], [145, 84, 106], [95, 192, 17], [72, 7, 63], [232, 228, 123], [129, 64, 61], [174, 169, 200], [121, 59, 209], [143, 50, 67], [43, 119, 222], [240, 102, 84], [245, 252, 154], [192, 186, 110], [111, 16, 224], [43, 19, 219], [191, 134, 254], [177, 229, 75], [47, 77, 224], [88, 7, 86], [51, 71, 78], [36, 85, 125], [219, 58, 111], [146, 0, 194], [245, 82, 130], [149, 2, 69], [52, 37, 115], [147, 251, 115], [33, 196, 82], [101, 59, 32], [91, 173, 44], [39, 16, 154], [208, 223, 171], [204, 149, 152], [5, 12, 236], [3, 168, 87], [155, 173, 34], [236, 18, 104], [152, 154, 42], [123, 151, 149], [217, 164, 77], [106, 140, 129], [251, 158, 96], [172, 248, 199], [148, 103, 24], [50, 244, 251], [59, 136, 75], [93, 109, 208], [149, 94, 209], [209, 229, 125], [81, 69, 72], [89, 5, 2], [18, 184, 171], [21, 235, 33], [2, 246, 182], [206, 1, 108], [108, 237, 5], [31, 160, 39], [169, 136, 75], [196, 187, 126], [170, 203, 36], [121, 17, 83], [177, 20, 142], [57, 89, 114], [244, 149, 102], [182, 12, 232], [186, 71, 130], [28, 55, 58], [96, 109, 51], [210, 10, 105], [96, 32, 239], [27, 176, 82], [204, 109, 79], [128, 151, 69], [94, 29, 220], [80, 4, 79], [79, 99, 42], [13, 190, 32], [255, 207, 181], [64, 252, 246], [65, 119, 185], [152, 140, 207], [208, 58, 162], [137, 179, 169], [197, 201, 25], [85, 205, 160], [132, 51, 216], [15, 86, 114], [130, 201, 81], [60, 44, 20], [199, 122, 106], [106, 177, 51], [116, 52, 241], [81, 106, 231], [184, 64, 49], [34, 99, 35], [58, 160, 65], [66, 236, 213], [21, 123, 129], [125, 217, 60], [123, 86, 125], [229, 166, 165], [245, 56, 52], [93, 148, 60], [51, 137, 234], [73, 23, 146], [11, 10, 98], [231, 22, 148], [237, 53, 226], [200, 6, 18], [18, 104, 214], [188, 56, 172], [32, 1, 58], [26, 128, 87], [154, 44, 130], [231, 248, 112], [99, 109, 178], [218, 46, 1], [50, 234, 153], [51, 140, 208], [147, 127, 102], [18, 227, 122], [115, 91, 26], [230, 14, 188], [36, 188, 247], [66, 47, 224], [244, 120, 39], [227, 23, 97], [60, 194, 39], [127, 123, 89], [231, 174, 107], [25, 238, 120], [168, 74, 53], [97, 219, 33], [233, 193, 160], [213, 35, 221], [212, 54, 192], [156, 162, 12], [178, 107, 154], [65, 32, 171], [98, 190, 37], [94, 21, 54], [40, 205, 41], [246, 234, 18], [247, 156, 230], [90, 194, 117], [136, 82, 28], [228, 194, 197], [20, 51, 182], [63, 222, 141], [84, 169, 116], [189, 45, 137], [234, 214, 197], [64, 60, 140], [149, 93, 131], [238, 118, 99], [246, 18, 124], [77, 82, 14], [177, 12, 59], [171, 84, 186], [80, 14, 166], [236, 191, 204], [211, 214, 19], [117, 119, 150], [20, 238, 183], [36, 82, 104], [201, 98, 190], [73, 0, 227], [103, 136, 165], [208, 61, 126], [52, 167, 187], [128, 150, 62], [207, 47, 158], [59, 212, 115], [5, 211, 186], [87, 34, 0], [246, 112, 42], [50, 86, 80], [88, 105, 151], [122, 137, 112], [225, 48, 179], [184, 156, 231], [82, 119, 1], [177, 4, 63], [105, 185, 191], [229, 15, 109], [13, 121, 99], [78, 33, 105], [61, 190, 11], [169, 156, 96], [170, 84, 68], [206, 234, 0], [85, 111, 199], [188, 106, 116], [216, 40, 61], [159, 55, 41], [224, 39, 81], [238, 93, 6], [210, 248, 97], [86, 28, 109], [59, 40, 191], [172, 24, 226], [54, 17, 189], [73, 69, 109], [42, 172, 16], [102, 61, 130], [19, 217, 27], [58, 249, 121], [6, 8, 180], [213, 193, 69], [16, 170, 61], [59, 187, 163], [33, 160, 35], [239, 120, 208], [95, 253, 69], [113, 214, 248], [49, 137, 99], [209, 48, 203], [41, 101, 252], [53, 148, 72], [207, 168, 26], [89, 97, 213], [221, 111, 168], [165, 77, 121], [86, 131, 45], [66, 167, 163], [229, 62, 174], [160, 142, 23], [34, 203, 59], [84, 238, 138], [153, 30, 21], [128, 143, 191], [188, 41, 171], [152, 209, 217], [148, 87, 224], [47, 44, 10], [118, 229, 27], [219, 18, 104], [246, 96, 107], [142, 90, 131], [87, 250, 158], [119, 124, 42], [95, 154, 7], [50, 245, 253], [147, 218, 158], [48, 234, 238], [132, 109, 165], [181, 162, 194], [129, 12, 164], [77, 123, 131], [154, 164, 164], [184, 205, 234], [187, 71, 208], [33, 32, 64], [253, 228, 50], [76, 99, 131], [201, 21, 45], [219, 67, 57], [111, 190, 147], [72, 90, 193], [58, 138, 211], [218, 198, 209], [93, 57, 138], [4, 244, 41], [190, 91, 26], [14, 44, 30], [68, 94, 201], [44, 147, 132], [60, 105, 193], [31, 129, 70], [77, 149, 113], [136, 67, 49], [85, 11, 110], [146, 153, 118], [174, 205, 210], [11, 68, 138], [156, 57, 252], [197, 80, 206], [144, 72, 112], [141, 119, 8], [156, 205, 22], [12, 5, 40], [67, 127, 173], [244, 180, 58], [47, 219, 223], [60, 32, 241], [212, 113, 92], [182, 18, 122], [88, 150, 224], [32, 243, 101], [159, 79, 162], [35, 67, 250], [35, 197, 40], [111, 142, 191], [222, 246, 176], [70, 169, 214], [91, 45, 57], [102, 135, 27], [172, 134, 12], [10, 208, 103], [200, 84, 136], [241, 74, 190], [118, 97, 12], [243, 76, 73], [235, 173, 77], [118, 70, 113], [124, 243, 211], [26, 98, 237], [135, 240, 29], [144, 241, 19], [179, 246, 180], [174, 135, 10], [226, 198, 190], [13, 201, 194], [1, 177, 197], [87, 230, 197], [126, 152, 39], [193, 214, 180], [22, 100, 18], [195, 196, 66], [189, 242, 91], [108, 26, 128], [113, 158, 94], [193, 56, 12], [12, 164, 23], [48, 88, 237], [70, 103, 143], [14, 212, 203], [58, 76, 9], [195, 236, 106], [240, 100, 245], [38, 42, 177], [235, 155, 64], [221, 58, 21], [41, 166, 193], [236, 209, 210], [94, 87, 190], [233, 255, 98], [215, 188, 211], [172, 93, 159], [231, 17, 148], [244, 78, 231], [137, 241, 254], [60, 41, 111], [10, 240, 51], [163, 131, 186], [243, 59, 235], [109, 254, 24], [204, 0, 132], [91, 189, 251], [197, 88, 178], [208, 119, 4], [172, 163, 141], [120, 146, 188], [58, 136, 225], [120, 102, 187], [220, 151, 185], [144, 55, 77], [202, 210, 155], [141, 227, 39], [125, 168, 213], [140, 52, 16], [211, 4, 30], [86, 63, 229], [37, 58, 243], [14, 143, 83], [237, 78, 194], [252, 213, 36], [161, 134, 143], [229, 249, 198], [33, 14, 67], [74, 144, 215], [222, 58, 105], [1, 108, 8], [46, 49, 16], [119, 66, 228], [186, 239, 124], [234, 19, 177], [66, 226, 68], [215, 156, 59], [179, 100, 178], [193, 251, 242], [121, 158, 172], [252, 145, 157], [46, 91, 188], [13, 233, 242], [124, 244, 247], [53, 84, 70], [195, 248, 82], [159, 245, 84], [175, 132, 245], [252, 46, 206], [213, 82, 208], [180, 125, 216], [224, 206, 38], [255, 140, 81], [102, 94, 150], [146, 38, 50], [108, 151, 89], [162, 173, 203], [8, 120, 143], [214, 67, 24], [211, 252, 54], [96, 168, 244], [96, 111, 10], [254, 85, 7], [236, 132, 249], [42, 220, 36], [8, 145, 36], [210, 6, 96], [113, 87, 105], [32, 27, 171], [75, 72, 158], [117, 58, 199], [199, 32, 161], [80, 40, 150], [240, 50, 76], [122, 22, 122], [147, 112, 195], [33, 169, 198], [79, 212, 183], [148, 185, 162], [32, 229, 121], [213, 79, 44], [218, 15, 41], [31, 62, 196], [81, 232, 57], [140, 239, 173], [13, 214, 118], [104, 108, 139], [201, 26, 22], [146, 23, 144], [149, 120, 193], [190, 153, 70], [126, 234, 47], [163, 52, 100], [67, 187, 84], [34, 65, 121], [92, 57, 235], [197, 213, 19], [154, 152, 85], [6, 82, 172], [35, 200, 176], [113, 253, 105], [222, 249, 184], [49, 196, 248], [72, 187, 158], [97, 117, 210], [176, 242, 193], [58, 212, 93], [157, 36, 52], [8, 157, 232], [34, 213, 1], [73, 37, 76], [118, 7, 165], [230, 39, 114], [11, 14, 156], [39, 199, 163], [66, 17, 224], [188, 143, 162], [109, 64, 214], [248, 191, 95], [88, 226, 137], [3, 100, 93], [186, 181, 22], [196, 182, 11], [34, 63, 96], [207, 32, 45], [238, 157, 53], [104, 181, 246], [160, 243, 212], [89, 165, 35], [228, 21, 206], [61, 179, 51], [71, 3, 133], [125, 58, 87], [216, 232, 17], [75, 182, 210], [78, 180, 190], [161, 206, 209], [216, 222, 140], [211, 1, 194], [243, 42, 138], [80, 72, 249], [27, 203, 160], [231, 106, 35], [184, 176, 156], [186, 226, 219], [49, 230, 241], [44, 181, 81], [125, 187, 242], [112, 181, 73], [48, 28, 100], [186, 93, 188], [201, 246, 34], [171, 172, 103], [244, 213, 152], [121, 219, 223], [144, 133, 219], [215, 17, 227], [69, 237, 152], [26, 224, 60], [105, 178, 244], [0, 185, 119], [176, 91, 8], [49, 195, 189], [240, 120, 80], [69, 128, 188], [85, 136, 91], [14, 135, 44], [98, 73, 145], [107, 165, 53], [9, 42, 162], [177, 210, 6], [51, 98, 100], [208, 94, 171], [25, 81, 248], [242, 189, 201], [158, 148, 93], [244, 142, 140], [30, 172, 158], [120, 220, 125], [247, 225, 246], [117, 43, 105], [58, 193, 129], [111, 139, 187], [221, 75, 249], [0, 197, 100], [225, 123, 42], [193, 175, 221], [37, 147, 33], [182, 173, 163], [232, 3, 159], [10, 84, 7], [3, 220, 191], [222, 192, 195], [66, 130, 252], [141, 107, 83], [164, 201, 149], [106, 91, 193], [162, 226, 162], [179, 120, 173], [72, 189, 212], [56, 67, 133], [236, 44, 13], [121, 52, 3], [164, 221, 34], [129, 16, 6], [207, 216, 215], [166, 137, 112], [124, 246, 154], [4, 132, 55], [170, 118, 50], [215, 246, 146], [95, 8, 151], [122, 21, 29], [219, 9, 135], [129, 197, 166], [81, 161, 204], [12, 107, 55], [65, 219, 86], [110, 87, 47], [23, 48, 115], [34, 216, 178], [36, 140, 176], [155, 181, 118], [89, 46, 217], [79, 58, 126], [133, 166, 134], [208, 138, 65], [121, 8, 179], [40, 142, 183], [181, 76, 160], [10, 24, 240], [107, 182, 149], [251, 176, 203], [249, 189, 157], [22, 4, 54], [39, 223, 127], [254, 235, 8], [153, 153, 44], [179, 59, 36], [145, 43, 151], [110, 23, 180], [53, 176, 107], [35, 99, 140], [78, 119, 70], [29, 127, 247], [234, 32, 96], [60, 160, 168], [122, 107, 127], [65, 192, 111], [165, 254, 166], [45, 23, 134], [13, 15, 255], [208, 207, 165], [40, 157, 154], [43, 162, 3], [205, 17, 253], [106, 228, 29], [207, 37, 89], [25, 13, 97], [206, 3, 86], [75, 143, 221], [15, 164, 84], [158, 101, 82], [37, 187, 122], [26, 148, 111], [222, 157, 187], [249, 138, 99], [19, 77, 147], [10, 239, 90], [19, 65, 36], [152, 180, 158], [239, 18, 205], [179, 61, 118], [217, 200, 191], [157, 203, 117], [120, 189, 114], [248, 202, 25], [98, 242, 220], [104, 107, 56], [169, 112, 227], [218, 221, 76], [10, 120, 78], [226, 2, 230], [129, 247, 243], [244, 32, 48], [245, 242, 92], [100, 74, 209], [131, 219, 214], [79, 170, 15], [125, 117, 176], [152, 58, 202], [17, 21, 36], [116, 54, 70], [175, 143, 125], [144, 9, 41], [156, 117, 52], [173, 157, 223], [133, 249, 218], [110, 188, 6], [108, 105, 48], [247, 243, 93], [220, 167, 165], [179, 51, 214], [137, 156, 170], [193, 18, 18], [230, 194, 110], [183, 116, 20], [222, 40, 226], [70, 224, 27], [130, 175, 10], [161, 150, 161], [138, 197, 200], [4, 128, 231], [255, 182, 174], [170, 65, 83], [181, 240, 168], [3, 124, 105], [97, 250, 180], [242, 23, 98], [67, 208, 100], [221, 38, 143], [217, 224, 115], [20, 102, 86], [119, 153, 135], [59, 184, 37], [171, 144, 54], [2, 38, 59], [5, 94, 250], [58, 161, 25], [248, 52, 143], [185, 112, 112], [190, 228, 177], [21, 204, 252], [237, 34, 148], [250, 208, 252], [209, 30, 90], [246, 207, 4], [58, 197, 159], [239, 63, 26], [84, 163, 0], [122, 217, 110], [46, 151, 252], [42, 225, 230], [119, 179, 120], [24, 26, 68], [234, 39, 219], [226, 72, 61], [190, 58, 237], [147, 209, 92], [130, 223, 148], [214, 114, 177], [19, 133, 253], [86, 144, 248], [226, 65, 182], [103, 20, 116], [38, 249, 32], [253, 5, 232], [99, 44, 124], [186, 216, 69], [213, 115, 241], [206, 153, 254], [86, 222, 42], [145, 169, 204], [140, 142, 199], [63, 195, 32], [95, 146, 35], [82, 136, 90], [101, 153, 229], [119, 46, 180], [152, 158, 238], [203, 156, 238], [243, 117, 160], [114, 59, 33], [109, 249, 112], [60, 42, 64], [193, 50, 58], [149, 60, 182], [243, 67, 91], [65, 36, 249], [230, 72, 168], [39, 219, 100], [99, 74, 70], [252, 184, 65], [228, 3, 50], [132, 20, 15], [110, 12, 213], [172, 124, 210], [99, 239, 197], [87, 227, 122], [146, 219, 219], [254, 161, 254], [136, 198, 171], [114, 160, 147], [87, 255, 246], [8, 3, 4], [225, 84, 143], [208, 144, 153], [30, 109, 26], [104, 255, 52], [85, 32, 250], [187, 107, 75], [2, 62, 251], [42, 132, 83], [180, 201, 8], [113, 222, 242], [253, 174, 131], [208, 172, 103], [39, 205, 237], [170, 192, 28], [200, 224, 123], [184, 108, 199], [133, 50, 109], [100, 244, 237], [187, 16, 188], [219, 14, 112], [248, 237, 76], [142, 56, 125], [57, 4, 42], [202, 195, 190], [196, 121, 30], [230, 106, 40], [166, 172, 174], [102, 84, 77], [182, 0, 5], [55, 224, 155], [136, 68, 230], [140, 153, 81], [40, 195, 144], [156, 79, 71], [243, 35, 104], [30, 237, 87], [181, 149, 211], [207, 183, 125], [167, 60, 203], [42, 224, 51], [226, 141, 48], [252, 37, 223], [72, 218, 145], [225, 181, 31], [166, 193, 197], [84, 18, 208], [46, 17, 46], [201, 207, 22], [210, 30, 127], [118, 145, 47], [78, 241, 185], [255, 113, 201], [166, 69, 52], [110, 214, 182], [135, 244, 23], [138, 72, 5], [126, 178, 234], [246, 50, 117], [54, 113, 136], [88, 222, 4], [205, 56, 45], [216, 32, 204], [211, 110, 181], [49, 126, 99], [95, 180, 212], [153, 17, 205], [99, 129, 186], [118, 84, 118], [113, 76, 39], [122, 241, 84], [151, 33, 115], [38, 55, 126], [218, 76, 172], [59, 34, 78], [45, 231, 109], [17, 189, 118], [219, 250, 150], [20, 119, 89], [28, 155, 139], [237, 134, 53], [162, 200, 113], [135, 110, 117], [181, 73, 90], [109, 159, 12], [40, 198, 41], [183, 91, 188], [189, 0, 81], [128, 22, 218], [225, 142, 69], [55, 134, 240], [200, 174, 109], [174, 107, 57], [43, 197, 113], [112, 11, 156], [57, 78, 90], [165, 95, 247], [39, 53, 98], [58, 131, 51], [231, 238, 253], [42, 213, 25], [37, 23, 143], [221, 152, 151], [184, 111, 115], [235, 177, 49], [161, 135, 166], [204, 244, 74], [247, 52, 50], [243, 167, 37], [247, 156, 169], [28, 37, 240], [209, 31, 69], [240, 35, 99], [247, 170, 106], [1, 229, 170], [121, 64, 82], [73, 202, 251], [104, 236, 219], [58, 42, 196], [147, 120, 240], [122, 108, 90], [14, 70, 192], [40, 75, 117], [110, 101, 34], [121, 110, 57], [162, 210, 202], [37, 235, 54], [177, 115, 93], [23, 143, 79], [48, 204, 232], [226, 251, 214], [52, 80, 35], [54, 44, 26], [144, 176, 220], [166, 27, 66], [232, 103, 140], [154, 117, 233], [86, 123, 127], [69, 86, 2], [123, 241, 109], [119, 64, 118], [105, 208, 229], [211, 94, 229], [32, 65, 52], [194, 98, 0], [63, 5, 187], [134, 107, 198], [177, 119, 8], [194, 221, 5], [116, 189, 127], [186, 58, 168], [80, 192, 69], [208, 102, 251], [82, 223, 142], [18, 137, 49], [241, 111, 205], [142, 197, 195], [16, 99, 119], [192, 246, 171], [198, 146, 118], [249, 175, 188], [63, 123, 58], [148, 236, 5], [158, 148, 235], [101, 233, 0], [217, 98, 38], [109, 0, 166], [84, 195, 168], [135, 14, 25], [20, 171, 223], [205, 105, 176], [102, 40, 107], [238, 153, 222], [24, 83, 6], [241, 17, 62], [94, 95, 160], [50, 2, 68], [62, 63, 208], [64, 97, 215], [28, 104, 182], [111, 70, 45], [192, 133, 162], [210, 26, 168], [45, 58, 128], [183, 57, 39], [130, 182, 185], [151, 170, 64], [66, 254, 30], [110, 188, 60], [0, 111, 219], [162, 15, 172], [160, 151, 183], [209, 7, 234], [162, 39, 212], [44, 230, 4], [252, 227, 211], [86, 224, 30], [227, 152, 63], [36, 229, 209], [53, 249, 219], [158, 230, 174], [115, 184, 214], [143, 15, 244], [106, 76, 93], [46, 20, 55], [213, 204, 34], [244, 35, 33], [55, 244, 247], [72, 45, 109], [58, 182, 207], [99, 182, 180], [50, 251, 34], [63, 249, 64], [251, 95, 104], [178, 242, 90], [209, 243, 211], [132, 74, 6], [255, 178, 240], [199, 196, 154], [255, 128, 158], [235, 207, 114], [38, 2, 203], [98, 162, 102], [33, 167, 245], [230, 224, 173], [246, 92, 200], [11, 191, 167], [216, 50, 9], [244, 85, 213], [171, 33, 246], [69, 148, 120], [200, 107, 227], [212, 212, 140], [39, 233, 135], [79, 112, 167], [123, 188, 44], [129, 194, 81], [5, 242, 124], [69, 158, 164], [48, 134, 224], [96, 207, 224], [71, 172, 145], [160, 118, 170], [146, 5, 11], [85, 229, 32], [58, 249, 124], [25, 217, 4], [80, 110, 85], [105, 68, 230], [201, 122, 110], [206, 127, 252], [222, 68, 186], [128, 32, 156], [220, 182, 54], [230, 19, 118], [180, 219, 204], [98, 243, 122], [25, 199, 212], [253, 233, 125], [13, 187, 62], [30, 92, 145], [166, 107, 187], [236, 245, 43], [173, 199, 201], [94, 121, 23], [250, 189, 188], [26, 123, 154], [246, 54, 95], [128, 38, 137], [27, 129, 136], [145, 61, 235], [204, 235, 92], [232, 187, 61], [193, 133, 103], [48, 54, 56], [188, 10, 190], [108, 69, 23], [164, 54, 121], [170, 77, 70], [186, 120, 30], [78, 202, 211], [23, 156, 77], [50, 243, 121], [98, 173, 31], [98, 38, 206], [152, 140, 157], [16, 255, 134], [86, 49, 57], [55, 124, 80], [146, 4, 131], [152, 90, 179], [255, 138, 234], [83, 61, 116], [16, 163, 143], [63, 104, 204], [97, 158, 40], [216, 248, 192], [38, 63, 170], [77, 50, 241], [85, 42, 227], [164, 147, 228], [174, 31, 172], [71, 12, 41], [60, 9, 74], [175, 71, 58], [197, 128, 16], [144, 7, 156], [225, 248, 16], [49, 51, 69], [36, 254, 28], [120, 103, 214], [250, 105, 58], [184, 115, 23], [149, 73, 124], [41, 212, 154], [76, 232, 26], [130, 177, 37], [155, 98, 237], [0, 113, 247], [174, 36, 19], [255, 65, 71], [251, 199, 74], [20, 246, 111], [196, 127, 248], [91, 97, 228], [149, 36, 218], [85, 168, 101], [255, 175, 137], [153, 234, 173], [174, 172, 227], [48, 12, 90], [117, 144, 161], [141, 240, 158], [118, 186, 242], [102, 236, 85], [32, 14, 21], [17, 35, 195], [210, 208, 43], [58, 164, 90], [132, 217, 116], [17, 30, 95], [190, 248, 218], [224, 241, 50], [42, 81, 149], [24, 27, 191], [62, 221, 181], [168, 112, 188], [92, 47, 212], [180, 74, 48], [98, 67, 186], [237, 159, 93], [174, 170, 27], [178, 121, 216], [183, 248, 143], [6, 33, 129], [109, 24, 234], [50, 218, 8], [118, 241, 38], [67, 91, 19], [88, 166, 221], [67, 216, 211], [198, 198, 25], [144, 232, 134], [84, 103, 220], [173, 231, 20], [125, 33, 142], [240, 41, 81], [106, 137, 25], [138, 36, 66], [200, 241, 130], [14, 88, 228], [59, 176, 185], [178, 150, 90], [139, 5, 219], [114, 210, 216], [222, 253, 33], [54, 85, 210], [4, 184, 125], [99, 83, 135], [87, 11, 154], [86, 225, 21], [35, 237, 153], [47, 14, 63], [177, 209, 33], [131, 173, 251], [83, 230, 71], [101, 107, 10], [168, 4, 183], [140, 227, 11], [86, 251, 121], [70, 143, 28], [171, 228, 125], [50, 162, 97], [49, 179, 61], [113, 170, 159], [98, 133, 198], [77, 96, 0], [78, 17, 217], [72, 221, 23], [135, 147, 240], [207, 187, 149], [210, 239, 90], [223, 233, 17], [48, 223, 78], [213, 17, 38], [216, 31, 43], [36, 88, 93], [200, 26, 228], [106, 44, 196], [96, 67, 197], [54, 231, 127], [135, 137, 37], [83, 41, 46], [173, 66, 168], [45, 128, 159], [96, 227, 248], [93, 241, 139], [67, 84, 162], [192, 105, 194], [201, 156, 189], [94, 13, 73], [27, 247, 33], [56, 100, 120], [162, 90, 170], [165, 215, 63], [244, 159, 180], [115, 80, 58], [253, 181, 93], [74, 15, 141], [104, 53, 23], [152, 60, 124], [220, 97, 45], [176, 41, 217], [42, 9, 122], [149, 146, 222], [185, 161, 147], [5, 37, 160], [142, 94, 204], [64, 103, 102], [200, 127, 89], [215, 88, 167], [243, 120, 195], [194, 238, 78], [188, 58, 123], [195, 242, 209], [207, 119, 54], [129, 73, 175], [254, 61, 15], [45, 37, 118], [111, 35, 76], [179, 73, 174], [135, 138, 223], [67, 101, 201], [250, 8, 230], [43, 88, 77], [53, 178, 77], [6, 51, 157], [86, 208, 35], [173, 225, 214], [238, 208, 141], [89, 207, 19], [65, 224, 101], [224, 175, 37], [217, 102, 64], [60, 156, 181], [157, 136, 126], [193, 249, 216], [70, 223, 12], [159, 43, 84], [48, 41, 207], [241, 165, 62], [211, 78, 120], [8, 83, 75], [249, 238, 220], [117, 244, 143], [221, 1, 175], [109, 6, 85], [38, 50, 176], [79, 23, 75], [166, 26, 2], [129, 61, 78], [73, 223, 102], [1, 87, 252], [70, 232, 27], [108, 188, 135], [31, 184, 204], [119, 7, 246], [5, 85, 148], [3, 158, 49], [176, 63, 226], [84, 40, 92], [77, 184, 57], [143, 169, 239], [112, 137, 236], [54, 57, 173], [21, 221, 230], [1, 236, 69], [62, 225, 186], [198, 192, 129], [76, 240, 247], [177, 204, 52], [133, 250, 28], [158, 42, 102], [242, 42, 241], [187, 26, 157], [138, 11, 170], [203, 53, 137], [80, 63, 127], [214, 88, 180], [26, 215, 199], [108, 65, 254], [35, 183, 229], [123, 56, 83], [92, 229, 97], [101, 195, 190], [134, 109, 194], [209, 44, 31], [242, 130, 115], [90, 197, 154], [177, 133, 203], [176, 60, 246], [188, 142, 100], [253, 185, 252], [170, 183, 17], [75, 177, 70], [38, 148, 138], [155, 120, 89], [88, 78, 150], [175, 103, 153], [10, 117, 19], [182, 215, 163], [174, 253, 120], [4, 249, 15], [184, 111, 147], [145, 190, 35], [8, 8, 214], [31, 140, 150], [180, 250, 154], [74, 27, 23], [95, 181, 4], [111, 26, 110], [24, 102, 166], [214, 51, 228], [16, 195, 208], [218, 77, 71], [136, 32, 188], [114, 145, 92], [41, 14, 16], [241, 253, 162], [65, 100, 68], [87, 225, 121], [127, 205, 191], [138, 57, 78], [112, 191, 17], [178, 107, 79], [228, 23, 148], [120, 238, 9], [106, 244, 143], [28, 19, 108], [178, 96, 175], [200, 54, 248], [239, 183, 203], [36, 220, 227], [0, 67, 94], [230, 137, 99], [84, 129, 104], [217, 70, 194], [132, 0, 169], [206, 93, 14], [20, 149, 152], [107, 207, 25], [223, 53, 202], [191, 10, 212], [87, 27, 228], [86, 5, 113], [56, 26, 26], [250, 168, 234], [229, 79, 85], [48, 101, 75], [163, 193, 81], [204, 96, 89], [197, 58, 72], [254, 70, 136], [42, 209, 227], [181, 104, 59], [118, 30, 175], [82, 21, 229], [176, 161, 148], [21, 61, 32], [181, 43, 243], [133, 87, 103], [220, 95, 106], [129, 233, 191], [91, 149, 69], [40, 241, 35], [208, 42, 11], [87, 249, 229], [182, 30, 107], [156, 105, 144], [114, 171, 210], [196, 233, 113], [99, 33, 38], [73, 101, 120], [150, 13, 147], [165, 213, 200], [127, 140, 209], [246, 52, 92], [218, 201, 251], [71, 228, 158], [143, 218, 133], [44, 167, 115], [131, 132, 223], [107, 255, 233], [244, 242, 173], [235, 204, 211], [126, 215, 231], [73, 127, 241], [89, 179, 188], [33, 216, 84], [4, 146, 61], [129, 155, 197], [34, 219, 239], [15, 38, 66], [15, 46, 35], [219, 163, 107], [183, 49, 247], [215, 124, 57], [110, 33, 30], [120, 138, 183], [224, 57, 153], [35, 206, 25], [58, 200, 135], [78, 245, 158], [141, 155, 166], [204, 20, 173], [223, 166, 3], [205, 117, 201], [152, 160, 218], [12, 137, 172], [218, 229, 155], [147, 160, 63], [154, 108, 18], [194, 220, 165], [146, 95, 44], [250, 21, 182], [27, 46, 146], [94, 189, 86], [2, 208, 228], [143, 178, 188], [206, 34, 56], [246, 149, 45], [59, 137, 1], [145, 10, 11], [240, 83, 217], [148, 148, 114], [212, 179, 247], [128, 197, 107], [232, 236, 42], [97, 16, 161], [219, 167, 184], [97, 157, 169], [139, 52, 27], [176, 127, 159], [33, 144, 48], [146, 71, 5], [206, 161, 186], [26, 17, 15], [37, 72, 0], [72, 242, 140], [54, 184, 141], [36, 34, 65], [117, 127, 216], [125, 108, 236], [178, 7, 218], [206, 223, 208], [84, 177, 66], [102, 244, 55], [73, 17, 48], [82, 64, 162], [156, 183, 25], [116, 113, 223], [249, 133, 68], [70, 253, 230], [211, 174, 240], [2, 24, 78], [47, 118, 233], [174, 164, 118], [8, 173, 253], [10, 148, 16], [98, 32, 8], [9, 90, 245], [129, 250, 83], [5, 5, 145], [131, 98, 202], [182, 25, 210], [238, 108, 152], [234, 104, 109], [91, 29, 126], [255, 144, 107], [248, 84, 28], [139, 249, 138], [210, 22, 96], [80, 215, 42], [212, 160, 207], [237, 105, 120], [165, 153, 33], [167, 14, 60], [213, 15, 104], [103, 39, 175], [180, 61, 230], [36, 206, 48], [27, 169, 107], [129, 88, 158], [103, 33, 206], [237, 94, 38], [225, 200, 22], [101, 40, 212], [245, 176, 132], [95, 119, 94], [236, 243, 77], [64, 104, 163], [98, 118, 219], [10, 228, 247], [15, 207, 202], [158, 14, 144], [101, 113, 48], [129, 244, 246], [222, 224, 53], [218, 3, 229], [173, 98, 189], [232, 229, 91], [71, 5, 102], [131, 31, 158], [80, 222, 19], [224, 15, 231], [81, 113, 103], [224, 178, 129], [186, 41, 200], [126, 64, 227], [86, 185, 213], [185, 204, 59], [144, 223, 17], [246, 38, 21], [118, 118, 0], [132, 96, 56], [244, 240, 224], [15, 60, 183], [32, 109, 26], [58, 120, 132], [103, 250, 244], [4, 18, 144], [103, 163, 246], [13, 111, 183], [58, 79, 0], [37, 243, 210], [34, 1, 59], [221, 56, 242], [30, 85, 118], [126, 239, 84], [145, 37, 16], [112, 35, 242], [145, 175, 252], [173, 143, 147], [34, 58, 40], [12, 107, 222], [70, 241, 77], [212, 69, 195], [232, 229, 250], [171, 97, 126], [112, 68, 52], [212, 151, 205], [78, 45, 197], [101, 213, 254], [230, 135, 43], [62, 72, 226], [32, 91, 96], [137, 13, 208], [73, 205, 254], [3, 78, 250], [183, 206, 184], [167, 66, 164], [208, 147, 52], [122, 27, 106], [68, 111, 155], [1, 2, 206], [10, 130, 48], [82, 147, 86], [121, 168, 236], [231, 168, 182], [225, 4, 33], [161, 149, 251], [125, 210, 49], [64, 35, 174], [229, 138, 197], [144, 45, 221], [203, 147, 145], [223, 114, 242], [138, 20, 129], [39, 253, 198], [46, 192, 177], [113, 121, 122], [253, 77, 52], [58, 14, 228], [62, 41, 91], [191, 169, 102], [63, 94, 181], [20, 48, 127], [175, 177, 116], [241, 248, 253], [92, 47, 246], [72, 176, 2], [171, 66, 208], [9, 110, 30], [92, 214, 206], [170, 203, 116], [99, 240, 126], [124, 63, 170], [156, 123, 125], [245, 137, 35], [200, 123, 222], [254, 117, 219], [49, 168, 112], [203, 121, 171], [34, 24, 2], [217, 95, 88], [76, 123, 134], [118, 253, 137], [190, 225, 240], [141, 76, 98], [55, 145, 171], [83, 235, 76], [12, 194, 192], [220, 231, 141], [69, 188, 249], [90, 208, 17], [192, 142, 117], [165, 154, 2], [33, 79, 73], [177, 208, 179], [28, 164, 109], [30, 241, 181], [181, 62, 22], [48, 154, 94], [78, 87, 202], [104, 197, 7], [26, 246, 85], [3, 206, 47], [43, 168, 151], [236, 155, 184], [84, 126, 15], [103, 209, 24], [58, 129, 3], [63, 130, 13], [93, 221, 235], [158, 208, 13], [153, 222, 192], [22, 95, 172], [138, 79, 20], [162, 24, 44], [23, 250, 142], [29, 143, 182], [161, 194, 139], [21, 11, 9], [19, 196, 123], [83, 110, 142], [6, 1, 122], [15, 91, 115], [26, 194, 59], [24, 221, 11], [117, 31, 30], [65, 39, 83], [48, 34, 237], [63, 81, 175], [204, 137, 64], [47, 210, 136], [88, 109, 147], [227, 101, 56], [158, 7, 61], [83, 109, 161], [172, 155, 107], [231, 1, 192], [11, 26, 182], [9, 203, 133], [50, 198, 131], [65, 2, 181], [30, 107, 197], [105, 138, 209], [94, 127, 29], [130, 226, 74], [7, 9, 129], [79, 91, 253], [107, 129, 206], [191, 240, 94], [250, 247, 143], [18, 124, 174], [108, 201, 9], [82, 235, 162], [15, 217, 93], [88, 157, 243], [182, 2, 53], [222, 193, 254], [108, 2, 72], [98, 229, 10], [56, 49, 48], [166, 162, 39], [170, 28, 126], [140, 59, 122], [13, 161, 240], [120, 161, 80], [82, 244, 23], [217, 220, 115], [97, 118, 164], [16, 11, 172], [188, 253, 99], [250, 122, 12], [100, 23, 223], [28, 5, 244], [48, 210, 3], [244, 204, 222], [103, 5, 213], [196, 29, 16], [214, 242, 69], [94, 95, 59], [140, 154, 234], [98, 45, 172], [108, 146, 229], [1, 174, 214], [217, 169, 26], [118, 241, 63], [107, 178, 118], [215, 222, 36], [162, 63, 193], [84, 13, 13], [46, 189, 42], [113, 247, 1], [202, 61, 4], [72, 228, 45], [215, 101, 18], [41, 65, 250], [76, 75, 60], [214, 18, 210], [161, 214, 48], [138, 209, 152], [124, 83, 179], [131, 9, 23], [170, 92, 125], [102, 158, 127], [247, 166, 235], [216, 212, 108], [112, 130, 24], [1, 143, 233], [248, 148, 230], [56, 167, 179], [242, 214, 44], [244, 177, 17], [246, 241, 59], [234, 78, 213], [124, 50, 223], [69, 69, 127], [186, 224, 36], [220, 42, 34], [233, 19, 108], [243, 114, 167], [126, 184, 111], [35, 81, 78], [182, 97, 245], [89, 133, 82], [110, 230, 110], [149, 143, 9], [70, 70, 226], [142, 171, 123], [80, 206, 241], [3, 207, 161], [57, 26, 164], [17, 56, 16], [229, 195, 182], [42, 208, 130], [9, 18, 249], [156, 117, 12], [54, 101, 255], [214, 17, 75], [15, 157, 37], [47, 10, 243], [60, 212, 27], [106, 209, 114], [148, 45, 64], [124, 206, 112], [159, 32, 217], [50, 39, 203], [101, 193, 87], [27, 75, 7], [11, 185, 224], [74, 105, 102], [136, 143, 122], [110, 2, 164], [198, 136, 143], [50, 109, 119], [15, 138, 157], [53, 249, 224], [157, 195, 60], [186, 202, 213], [179, 217, 241], [179, 118, 234], [220, 142, 172], [13, 227, 48], [236, 205, 232], [193, 20, 39], [162, 136, 63], [46, 53, 54], [191, 200, 255], [98, 84, 240], [222, 117, 7], [210, 46, 156], [226, 114, 24], [161, 27, 71], [106, 115, 203], [216, 50, 116], [143, 227, 117], [48, 35, 239], [148, 222, 48], [54, 226, 235], [133, 236, 242], [217, 224, 160], [84, 154, 155], [40, 51, 184], [65, 39, 30], [200, 234, 188], [8, 83, 148], [125, 72, 62], [53, 94, 147], [83, 187, 77], [219, 145, 37], [4, 2, 235], [217, 33, 169], [123, 124, 102], [210, 11, 7], [181, 81, 13], [106, 161, 31], [12, 83, 79], [155, 221, 34], [107, 132, 135], [116, 109, 11], [112, 119, 18], [89, 60, 196], [179, 127, 230], [177, 225, 57], [78, 96, 116], [136, 56, 131], [237, 189, 166], [203, 87, 180], [36, 199, 202], [232, 193, 11], [192, 29, 203], [168, 183, 228], [188, 132, 193], [89, 250, 245], [84, 164, 109], [144, 188, 155], [161, 54, 128], [15, 208, 125], [190, 172, 197], [178, 190, 209], [129, 58, 17], [185, 118, 10], [3, 214, 189], [60, 242, 40], [53, 11, 216], [0, 237, 35], [121, 203, 100], [201, 23, 205], [189, 236, 235], [254, 96, 242], [83, 237, 235], [51, 220, 92], [197, 188, 96], [182, 43, 127], [71, 101, 192], [77, 41, 129], [196, 85, 212], [238, 149, 46], [3, 132, 82], [130, 163, 84], [233, 155, 253], [120, 134, 75], [125, 18, 142], [122, 168, 32], [195, 54, 159], [109, 21, 246], [60, 50, 80], [32, 74, 51], [167, 176, 98], [96, 176, 0], [100, 175, 169], [172, 236, 134], [203, 36, 136], [133, 253, 201], [16, 91, 59], [85, 157, 77], [92, 25, 93], [1, 240, 143], [5, 233, 103], [114, 201, 143], [161, 110, 68], [73, 94, 222], [47, 38, 218], [34, 62, 203], [115, 107, 116], [21, 139, 179], [74, 195, 108], [88, 70, 249], [246, 32, 189], [76, 248, 199], [92, 211, 40], [154, 245, 2], [43, 73, 141], [126, 120, 51], [26, 178, 192], [244, 80, 15], [40, 87, 63], [163, 35, 131], [72, 151, 165], [37, 58, 196], [117, 33, 104], [78, 56, 7], [192, 227, 57], [20, 17, 237], [42, 210, 30], [0, 249, 60], [94, 244, 138], [210, 20, 39], [128, 39, 21], [215, 25, 16], [122, 240, 112], [120, 210, 248], [116, 6, 54], [67, 190, 42], [95, 235, 232], [193, 247, 225], [121, 132, 14], [212, 230, 23], [90, 51, 5], [192, 63, 22], [185, 229, 2], [134, 174, 34], [86, 60, 129], [230, 77, 240], [186, 157, 247], [197, 145, 20], [168, 63, 9], [153, 39, 53], [94, 92, 41], [242, 151, 61], [174, 224, 249], [70, 240, 64], [91, 74, 235], [143, 9, 123], [133, 64, 150], [21, 165, 36], [48, 29, 29], [86, 124, 74], [8, 181, 193], [126, 140, 151], [154, 3, 113], [81, 55, 93], [226, 122, 144], [41, 151, 107], [250, 42, 6], [38, 33, 141], [138, 72, 25], [131, 150, 127], [15, 189, 190], [71, 20, 219], [44, 51, 190], [169, 141, 89], [75, 126, 153], [64, 76, 113], [163, 243, 230], [234, 163, 27], [4, 71, 213], [221, 204, 61], [96, 63, 81], [215, 246, 72], [222, 222, 76], [129, 170, 225], [71, 240, 3], [117, 142, 224], [138, 113, 250], [239, 146, 177], [236, 23, 197], [243, 6, 221], [170, 99, 199], [227, 196, 9], [172, 169, 159], [253, 89, 40], [50, 199, 62], [222, 254, 233], [93, 138, 62], [254, 166, 208], [202, 129, 8], [242, 56, 30], [74, 203, 154], [21, 253, 99], [222, 149, 25], [46, 229, 52], [73, 132, 178], [104, 151, 183], [237, 203, 75], [98, 36, 139], [19, 163, 86], [6, 199, 39], [117, 205, 86], [103, 68, 44], [189, 2, 239], [76, 231, 2], [88, 12, 250], [242, 75, 42], [107, 24, 161], [241, 150, 75], [85, 19, 53], [4, 83, 67], [108, 28, 28], [212, 49, 221], [95, 220, 18], [247, 177, 27], [156, 220, 41], [49, 161, 249], [242, 188, 152], [141, 127, 225], [100, 82, 245], [94, 179, 75], [207, 90, 64], [59, 86, 21], [108, 54, 25], [179, 137, 38], [146, 11, 110], [83, 121, 72], [63, 51, 19], [90, 95, 173], [247, 155, 165], [66, 245, 154], [164, 118, 66], [133, 55, 60], [211, 116, 193], [193, 85, 154], [171, 217, 178], [22, 192, 170], [39, 38, 25], [54, 32, 159], [137, 240, 30], [232, 29, 50], [115, 194, 240], [48, 81, 212], [105, 156, 106], [140, 223, 68], [228, 96, 76], [198, 105, 73], [42, 16, 108], [123, 95, 182], [46, 93, 215], [170, 132, 152], [156, 142, 14], [62, 196, 170], [68, 102, 150], [152, 3, 172], [73, 25, 139], [102, 34, 141], [201, 255, 64], [67, 75, 58], [69, 157, 213], [54, 178, 136], [58, 150, 244], [234, 21, 170], [86, 245, 179], [55, 166, 38], [10, 237, 115], [65, 23, 136], [95, 38, 202], [15, 245, 151], [141, 28, 134], [28, 25, 180], [117, 146, 174], [107, 61, 58], [231, 235, 226], [157, 125, 126], [193, 33, 97], [203, 62, 40], [210, 59, 95], [83, 49, 255], [110, 30, 68], [162, 74, 92], [110, 225, 2], [185, 128, 101], [172, 48, 106], [129, 93, 167], [219, 18, 97], [49, 13, 175], [211, 75, 58], [234, 70, 148], [49, 112, 78], [254, 206, 245], [199, 99, 223], [48, 93, 148], [2, 34, 198], [230, 157, 83], [19, 6, 158], [26, 162, 61], [88, 152, 245], [67, 85, 161], [114, 117, 50], [167, 34, 49], [231, 178, 251], [155, 211, 253], [197, 248, 191], [37, 7, 204], [138, 190, 30], [158, 37, 7], [253, 253, 254], [244, 200, 44], [115, 23, 24], [195, 190, 253], [6, 123, 232], [197, 89, 1], [92, 23, 245], [144, 237, 19], [204, 74, 187], [199, 150, 147], [213, 114, 171], [231, 179, 48], [222, 19, 1], [118, 125, 115], [21, 117, 12], [139, 221, 222], [228, 161, 233], [142, 42, 89], [122, 135, 119], [240, 238, 173], [140, 90, 130], [136, 226, 162], [241, 7, 205], [69, 187, 32], [117, 38, 126], [29, 217, 98], [109, 87, 227], [27, 20, 0], [121, 1, 187], [12, 251, 99], [213, 190, 177], [135, 33, 254], [151, 100, 195], [104, 35, 144], [105, 168, 80], [5, 116, 227], [96, 154, 11], [84, 238, 138], [103, 78, 205], [136, 144, 87], [93, 98, 238], [90, 30, 220], [180, 99, 91], [52, 205, 90], [171, 142, 147], [102, 206, 23], [79, 229, 56], [52, 136, 96], [12, 79, 26], [173, 229, 151], [242, 203, 11], [4, 186, 158], [76, 103, 217], [159, 112, 159], [131, 12, 81], [177, 12, 164], [218, 173, 15], [93, 217, 74], [11, 136, 64], [226, 45, 137], [199, 30, 188], [114, 6, 211], [164, 142, 26], [242, 73, 17], [205, 92, 210], [180, 237, 196], [29, 147, 199], [39, 124, 34], [127, 19, 179], [127, 109, 62], [167, 242, 172], [211, 36, 21], [255, 83, 117], [3, 78, 90], [86, 198, 36], [48, 119, 178], [118, 122, 79], [241, 63, 152], [193, 39, 74], [159, 63, 205], [184, 30, 86], [8, 99, 125], [95, 117, 45], [231, 114, 96], [38, 122, 58], [169, 185, 3], [37, 87, 119], [27, 89, 156], [227, 182, 174], [241, 223, 3], [1, 229, 219], [36, 213, 62], [111, 33, 212], [22, 28, 138], [208, 89, 198], [12, 177, 68], [98, 220, 15], [22, 32, 177], [54, 41, 138], [226, 213, 206], [133, 147, 119], [0, 1, 225], [211, 3, 233], [80, 221, 106], [103, 187, 137], [155, 72, 248], [120, 254, 151], [116, 72, 93], [158, 190, 93], [113, 156, 190], [158, 103, 24], [15, 5, 72], [242, 105, 85], [15, 95, 10], [214, 217, 94], [220, 114, 75], [73, 82, 63], [6, 8, 176], [187, 114, 98], [239, 135, 114], [8, 125, 77], [195, 117, 75], [22, 20, 151], [7, 187, 21], [244, 171, 129], [25, 82, 216], [228, 13, 86], [7, 205, 16], [144, 74, 170], [140, 141, 240], [53, 174, 166], [65, 254, 76], [97, 247, 188], [73, 66, 150], [56, 188, 215], [167, 95, 42], [25, 224, 66], [219, 116, 187], [31, 210, 202], [242, 187, 137], [2, 243, 94], [25, 244, 129], [57, 208, 224], [198, 6, 73], [182, 197, 144], [192, 103, 134], [83, 10, 236], [79, 29, 158], [65, 218, 104], [34, 102, 150], [142, 162, 65], [152, 172, 134], [139, 26, 100], [251, 3, 142], [14, 26, 33], [24, 119, 22], [27, 23, 223], [134, 69, 13], [248, 240, 206], [227, 194, 146], [119, 1, 151], [41, 121, 18], [105, 3, 218], [60, 56, 240], [100, 94, 32], [231, 40, 236], [145, 161, 43], [170, 66, 26], [165, 53, 249], [178, 75, 247], [2, 254, 32], [145, 238, 188], [7, 65, 73], [119, 133, 82], [171, 47, 121], [40, 79, 113], [92, 161, 91], [207, 213, 80], [169, 82, 69], [77, 225, 235], [128, 166, 157], [128, 217, 130], [51, 76, 176], [31, 229, 89], [199, 60, 222], [197, 124, 45], [72, 200, 156], [188, 235, 17], [214, 88, 190], [59, 91, 89], [70, 155, 128], [50, 161, 167], [148, 119, 166], [160, 101, 54], [113, 122, 245], [67, 119, 110], [107, 212, 194], [106, 77, 44], [180, 35, 215], [60, 62, 92], [225, 54, 8], [232, 223, 67], [150, 152, 73], [141, 151, 102], [116, 129, 66], [205, 165, 164], [13, 114, 237], [160, 61, 11], [250, 179, 22], [221, 87, 89], [213, 14, 150], [128, 49, 10], [24, 143, 26], [143, 145, 189], [54, 173, 38], [185, 208, 154], [37, 6, 64], [177, 233, 139], [197, 129, 69], [56, 26, 187], [152, 64, 142], [142, 154, 2], [141, 140, 252], [147, 200, 168], [104, 70, 19], [49, 195, 93], [79, 44, 148], [169, 58, 89], [244, 102, 222], [117, 41, 2], [183, 91, 233], [171, 155, 80], [42, 130, 238], [174, 90, 125], [99, 110, 18], [75, 255, 51], [254, 13, 124], [21, 141, 171], [24, 3, 239], [217, 190, 213], [54, 216, 179], [75, 20, 0], [249, 81, 238], [107, 214, 211], [45, 92, 25], [173, 45, 60], [126, 253, 7], [31, 175, 33], [136, 249, 30], [203, 164, 70], [175, 19, 132], [96, 223, 97], [204, 58, 47], [228, 34, 209], [121, 2, 186], [75, 34, 152], [18, 255, 208], [50, 181, 184], [41, 234, 192], [32, 51, 15], [90, 202, 199], [221, 157, 160], [12, 137, 68], [43, 155, 40], [190, 223, 130], [141, 225, 99], [200, 126, 93], [165, 30, 84], [108, 6, 170], [135, 22, 246], [74, 13, 136], [25, 249, 209], [38, 112, 188], [121, 195, 213], [8, 59, 62], [154, 116, 193], [22, 70, 25], [40, 13, 33], [183, 244, 155], [4, 248, 58], [244, 190, 37], [226, 152, 73], [8, 2, 137], [194, 94, 183], [206, 182, 207], [19, 167, 7], [121, 203, 203], [199, 77, 144], [163, 22, 206], [222, 55, 91], [241, 175, 76], [252, 9, 165], [71, 122, 28], [194, 254, 219], [37, 44, 37], [180, 152, 15], [141, 30, 53], [55, 205, 16], [232, 212, 82], [175, 25, 165], [155, 8, 184], [17, 234, 199], [136, 215, 210], [156, 84, 53], [166, 211, 247], [57, 67, 247], [31, 53, 163], [111, 206, 13], [112, 56, 72], [63, 225, 74], [148, 96, 13], [140, 228, 218], [133, 53, 40], [100, 85, 218], [101, 7, 146], [119, 135, 115], [195, 237, 7], [205, 103, 183], [38, 127, 104], [222, 56, 132], [173, 247, 87], [217, 185, 134], [190, 207, 199], [199, 154, 253], [218, 127, 234], [234, 188, 139], [173, 217, 174], [92, 84, 68], [172, 189, 145], [126, 78, 208], [81, 52, 161], [37, 250, 88], [73, 225, 16], [73, 197, 2], [182, 7, 101], [141, 167, 214], [133, 174, 56], [230, 237, 180], [55, 93, 103], [242, 9, 158], [167, 29, 53], [248, 17, 19], [3, 154, 104], [175, 233, 236], [120, 118, 12], [241, 177, 106], [239, 76, 204], [2, 239, 223], [232, 104, 241], [245, 92, 28], [48, 85, 252], [60, 105, 219], [143, 175, 240], [29, 250, 199], [133, 187, 140], [59, 145, 191], [246, 84, 183], [203, 51, 244], [108, 203, 238], [63, 151, 18], [97, 188, 184], [58, 214, 219], [50, 56, 173], [16, 48, 5], [226, 26, 203], [68, 107, 131], [103, 44, 229], [26, 66, 88], [219, 190, 125], [54, 45, 11], [179, 212, 136], [16, 29, 113], [116, 79, 23], [163, 92, 62], [124, 204, 62], [57, 198, 36], [246, 200, 213], [200, 199, 172], [190, 13, 235], [52, 20, 12], [109, 145, 169], [108, 136, 102], [175, 232, 189], [90, 171, 34], [67, 173, 86], [27, 197, 62], [79, 60, 44], [186, 25, 147], [147, 216, 115], [1, 55, 205], [249, 230, 228], [34, 112, 218], [19, 251, 252], [201, 193, 55], [136, 162, 1], [53, 191, 236], [88, 40, 236], [223, 144, 73], [195, 76, 207], [103, 248, 75], [213, 2, 203], [184, 215, 42], [224, 177, 124], [249, 153, 221], [199, 234, 35], [152, 251, 136], [118, 38, 145], [106, 156, 85], [59, 25, 152], [223, 169, 117], [20, 193, 37], [131, 237, 143], [206, 73, 59], [90, 81, 73], [152, 183, 181], [227, 83, 23], [207, 172, 35], [186, 36, 11], [46, 39, 5], [239, 146, 237], [212, 243, 243], [216, 199, 204], [83, 65, 14], [133, 25, 200], [62, 238, 238], [131, 149, 218], [71, 199, 229], [49, 90, 25], [167, 93, 249], [134, 84, 76], [20, 19, 223], [12, 206, 28], [39, 31, 38], [184, 195, 227], [12, 173, 189], [162, 203, 193], [7, 243, 38], [184, 75, 60], [83, 100, 66], [67, 185, 156], [140, 75, 151], [20, 129, 239], [121, 124, 9], [153, 179, 75], [242, 81, 246], [206, 25, 207], [246, 150, 2], [178, 43, 90], [26, 11, 114], [51, 133, 16], [67, 156, 79], [170, 216, 5], [91, 206, 49], [194, 198, 216], [230, 2, 49], [163, 159, 10], [35, 185, 145], [54, 187, 27], [88, 171, 16], [17, 147, 112], [194, 215, 104], [65, 255, 157], [215, 111, 94], [132, 245, 172], [254, 105, 172], [177, 48, 91], [163, 94, 120], [77, 28, 206], [154, 43, 215], [101, 190, 29], [103, 191, 172], [3, 240, 93], [142, 14, 161], [174, 81, 82], [94, 97, 16], [113, 109, 32], [156, 25, 124], [37, 245, 84], [245, 25, 236], [235, 40, 88], [91, 128, 62], [140, 224, 69], [133, 59, 108], [76, 65, 229], [205, 185, 71], [225, 83, 113], [160, 7, 60], [8, 123, 165], [152, 108, 225], [84, 4, 97], [210, 136, 92], [65, 61, 142], [90, 127, 209], [150, 64, 200], [132, 24, 190], [47, 175, 48], [254, 172, 35], [198, 84, 126], [50, 243, 181], [186, 201, 121], [11, 113, 194], [49, 78, 111], [143, 160, 81], [249, 143, 124], [148, 32, 107], [130, 99, 2], [41, 62, 108], [9, 179, 108], [106, 251, 38], [45, 219, 249], [99, 206, 245], [16, 132, 219], [23, 184, 220], [74, 73, 172], [79, 98, 37], [73, 65, 122], [130, 28, 42], [77, 173, 227], [53, 77, 214], [207, 34, 222], [36, 244, 229], [54, 40, 223], [14, 5, 166], [57, 154, 144], [70, 135, 48], [68, 171, 185], [29, 50, 112], [232, 212, 143], [218, 160, 83], [9, 179, 216], [98, 91, 23], [175, 71, 190], [36, 8, 95], [63, 218, 86], [211, 239, 181], [17, 28, 126], [151, 89, 101], [34, 124, 3], [59, 110, 150], [243, 255, 121], [165, 174, 22], [53, 39, 199], [187, 17, 204], [22, 144, 76], [214, 186, 1], [246, 209, 244], [140, 120, 114], [225, 18, 56], [224, 239, 87], [39, 43, 250], [53, 44, 96], [117, 230, 0], [147, 207, 98], [220, 48, 154], [36, 187, 131], [103, 129, 56], [141, 226, 154], [115, 122, 164], [81, 182, 42], [243, 33, 160], [166, 82, 195], [200, 16, 43], [168, 131, 54], [201, 5, 138], [222, 24, 197], [152, 214, 217], [83, 113, 146], [165, 197, 216], [194, 229, 58], [237, 42, 221], [247, 174, 191], [47, 126, 216], [48, 148, 1], [151, 65, 220], [114, 154, 60], [102, 103, 51], [36, 50, 187], [10, 173, 84], [55, 101, 203], [182, 197, 173], [194, 224, 29], [200, 248, 190], [199, 51, 138], [113, 181, 109], [192, 27, 66], [245, 120, 193], [71, 33, 150], [8, 186, 95], [224, 15, 221], [230, 2, 21], [119, 50, 208], [160, 123, 146], [153, 228, 135], [120, 251, 238], [74, 93, 20], [207, 3, 175], [212, 173, 51], [57, 163, 128], [40, 181, 29], [23, 56, 172], [129, 82, 62], [175, 15, 20], [10, 46, 115], [229, 1, 188], [100, 149, 3], [24, 152, 78], [224, 225, 129], [27, 93, 102], [237, 46, 41], [47, 159, 116], [231, 158, 21], [47, 156, 55], [179, 162, 100], [248, 75, 161], [193, 53, 36], [110, 131, 62], [44, 96, 179], [227, 114, 143], [213, 107, 214], [93, 171, 36], [10, 237, 119], [31, 116, 216], [75, 129, 62], [98, 86, 193], [165, 216, 206], [201, 231, 129], [54, 154, 39], [131, 74, 10], [175, 44, 163], [231, 105, 118], [207, 142, 95], [238, 22, 40], [134, 68, 124], [230, 146, 50], [133, 68, 75], [154, 85, 119], [136, 12, 215], [161, 136, 194], [59, 249, 33], [61, 81, 23], [162, 25, 188], [244, 156, 29], [59, 110, 18], [220, 60, 55], [154, 97, 179], [5, 51, 236], [0, 141, 103], [16, 76, 60], [167, 251, 30], [192, 165, 184], [185, 222, 26], [163, 223, 8], [58, 89, 25], [245, 236, 44], [49, 144, 244], [101, 45, 9], [16, 6, 254], [254, 46, 251], [159, 37, 255], [138, 109, 197], [116, 165, 203], [175, 173, 85], [151, 15, 155], [21, 131, 206], [151, 60, 136], [28, 158, 109], [67, 3, 25], [206, 234, 241], [18, 250, 77], [219, 23, 230], [12, 175, 198], [3, 133, 149], [245, 50, 252], [71, 3, 162], [241, 116, 90], [24, 185, 20], [48, 56, 209], [182, 126, 224], [251, 24, 3], [184, 161, 243], [183, 3, 168], [145, 85, 187], [210, 128, 242], [146, 100, 67], [70, 225, 121], [229, 203, 161], [109, 172, 66], [29, 104, 147], [170, 1, 92], [72, 24, 107], [8, 127, 27], [69, 76, 123], [215, 146, 29], [159, 52, 60], [157, 35, 233], [137, 240, 66], [73, 176, 196], [169, 115, 119], [5, 140, 201], [132, 244, 37], [105, 38, 146], [201, 196, 172], [6, 187, 161], [27, 76, 165], [157, 39, 55], [43, 132, 253], [176, 201, 57], [71, 233, 14], [27, 50, 129], [77, 115, 212], [166, 173, 92], [202, 93, 249], [186, 72, 163], [193, 209, 216], [105, 161, 74], [47, 167, 253], [100, 127, 12], [13, 202, 82], [15, 201, 119], [43, 71, 255], [237, 130, 246], [16, 175, 108], [200, 5, 182], [13, 149, 153], [1, 224, 46], [151, 42, 226], [12, 27, 86], [156, 39, 182], [208, 101, 249], [31, 182, 134], [126, 212, 161], [143, 138, 149], [74, 20, 82], [159, 203, 136], [91, 30, 165], [3, 7, 3], [99, 58, 8], [144, 20, 1], [72, 218, 185], [141, 50, 146], [234, 110, 57], [143, 251, 219], [159, 196, 57], [223, 182, 191], [195, 59, 25], [140, 231, 237], [252, 72, 114], [156, 218, 206], [123, 239, 185], [224, 194, 194], [187, 79, 131], [225, 233, 236], [186, 213, 24], [46, 71, 165], [194, 182, 219], [204, 116, 152], [110, 27, 23], [29, 65, 113], [126, 53, 0], [111, 245, 132], [215, 214, 246], [250, 191, 159], [27, 13, 167], [83, 142, 114], [67, 126, 39], [100, 110, 213], [221, 151, 9], [124, 66, 41], [134, 110, 9], [2, 154, 246], [117, 88, 81], [167, 72, 5], [154, 73, 232], [180, 109, 53], [253, 188, 130], [196, 189, 215], [215, 12, 128], [168, 245, 125], [143, 210, 159], [79, 98, 155], [98, 70, 112], [107, 202, 87], [128, 123, 4], [92, 194, 10], [151, 157, 61], [41, 244, 254], [79, 148, 65], [5, 74, 135], [2, 171, 221], [161, 56, 113], [23, 16, 1], [209, 1, 49], [152, 35, 112], [166, 26, 123], [237, 94, 150], [250, 31, 116], [0, 195, 78], [240, 230, 253], [244, 147, 37], [181, 81, 250], [121, 17, 233], [84, 126, 89], [240, 192, 84], [193, 42, 123], [12, 57, 99], [127, 208, 175], [158, 253, 218], [23, 28, 76], [222, 16, 243], [145, 95, 210], [148, 65, 53], [41, 186, 184], [111, 199, 30], [236, 46, 51], [13, 127, 196], [114, 80, 197], [86, 13, 196], [138, 125, 70], [210, 98, 121], [156, 206, 153], [246, 0, 189], [195, 200, 226], [51, 58, 3], [249, 154, 20], [209, 85, 200], [196, 166, 9], [116, 193, 138], [16, 74, 12], [139, 114, 96], [30, 115, 217], [33, 51, 163], [55, 254, 242], [157, 242, 217], [85, 16, 140], [239, 145, 31], [227, 3, 201], [37, 178, 111], [65, 101, 224], [214, 7, 138], [114, 65, 125], [26, 216, 239], [62, 12, 42], [152, 225, 19], [107, 208, 57], [52, 69, 248], [104, 33, 181], [127, 28, 153], [116, 94, 201], [83, 89, 13], [99, 220, 22], [5, 4, 62], [67, 226, 218], [107, 81, 227], [223, 97, 89], [66, 196, 60], [155, 39, 96], [188, 64, 49], [100, 158, 223], [41, 202, 119], [130, 205, 227], [46, 11, 236], [165, 82, 113], [79, 203, 142], [42, 154, 40], [238, 177, 226], [210, 25, 29], [239, 163, 12], [176, 196, 149], [111, 138, 42], [47, 240, 138], [113, 109, 141], [121, 74, 88], [108, 102, 2], [51, 236, 229], [165, 253, 181], [13, 249, 193], [62, 216, 85], [228, 133, 143], [178, 151, 155], [22, 74, 235], [4, 212, 233], [99, 7, 236], [169, 24, 148], [178, 220, 110], [159, 157, 101], [50, 150, 188], [52, 93, 245], [204, 167, 23], [20, 150, 128], [218, 142, 142], [134, 95, 146], [136, 84, 48], [91, 180, 106], [97, 124, 228], [153, 104, 161], [168, 122, 17], [78, 78, 144], [246, 170, 19], [21, 19, 160], [16, 198, 0], [54, 145, 171], [21, 89, 182], [1, 112, 175], [238, 28, 119], [7, 130, 134], [220, 161, 20], [78, 101, 8], [27, 51, 118], [9, 246, 62], [239, 249, 19], [83, 9, 242], [93, 175, 208], [17, 151, 21], [58, 186, 116], [96, 234, 124], [246, 199, 146], [204, 22, 213], [149, 64, 195], [219, 103, 151], [71, 153, 219], [178, 130, 32], [27, 113, 28], [99, 126, 113], [93, 175, 248], [6, 180, 16], [192, 13, 106], [20, 2, 240], [192, 218, 47], [146, 27, 45], [246, 219, 131], [146, 190, 207], [186, 232, 204], [113, 109, 199], [246, 79, 221], [11, 254, 98], [1, 73, 150], [79, 206, 4], [64, 51, 195], [177, 248, 204], [251, 225, 20], [43, 234, 178], [148, 57, 219], [157, 80, 191], [215, 208, 101], [204, 168, 48], [99, 99, 236], [142, 25, 137], [137, 111, 200], [24, 170, 134], [27, 177, 17], [95, 54, 66], [122, 172, 114], [171, 29, 56], [91, 138, 43], [106, 122, 147], [106, 130, 38], [111, 53, 144], [252, 111, 51], [36, 17, 33], [10, 198, 238], [174, 226, 200], [118, 202, 122], [210, 88, 50], [233, 152, 167], [166, 101, 59], [8, 36, 158], [150, 43, 207], [214, 96, 118], [25, 100, 245], [190, 35, 188], [135, 88, 84], [241, 15, 187], [177, 188, 196], [237, 216, 138], [220, 31, 188], [212, 7, 17], [103, 238, 116], [46, 104, 58], [110, 193, 96], [236, 168, 233], [6, 20, 113], [119, 21, 180], [109, 45, 91], [134, 176, 89], [217, 28, 248], [202, 232, 207], [191, 61, 34], [132, 253, 171], [139, 12, 228], [176, 49, 105], [189, 46, 60], [170, 103, 199], [235, 132, 255], [175, 14, 195], [115, 242, 222], [2, 10, 145], [5, 118, 198], [181, 142, 238], [253, 2, 84], [149, 253, 47], [214, 196, 46], [232, 229, 189], [250, 201, 13], [213, 133, 103], [116, 18, 119], [144, 14, 45], [98, 68, 114], [158, 56, 215], [155, 1, 219], [3, 120, 117], [3, 129, 251], [153, 228, 48], [117, 208, 202], [56, 231, 35], [191, 211, 166], [214, 84, 43], [34, 10, 191], [15, 217, 234], [23, 175, 173], [42, 148, 33], [158, 25, 84], [74, 67, 107], [142, 10, 123], [77, 146, 26], [158, 131, 252], [185, 40, 20], [225, 197, 62], [61, 52, 240], [98, 195, 56], [35, 87, 141], [235, 166, 221], [57, 101, 198], [119, 29, 22], [204, 110, 162], [91, 215, 137], [208, 191, 32], [175, 40, 61], [96, 155, 70], [3, 124, 27], [100, 54, 81], [88, 17, 126], [73, 224, 90], [186, 28, 173], [164, 227, 249], [125, 25, 0], [63, 237, 165], [51, 78, 36], [170, 86, 14], [142, 142, 231], [211, 115, 43], [8, 136, 88], [218, 74, 134], [160, 212, 184], [252, 70, 253], [115, 161, 191], [226, 255, 30], [23, 111, 242], [218, 162, 103], [243, 228, 129], [228, 186, 6], [164, 238, 177], [52, 109, 228], [200, 72, 10], [104, 184, 51], [53, 61, 74], [61, 15, 132], [192, 172, 168], [233, 209, 70], [45, 149, 167], [125, 19, 186], [33, 16, 40], [169, 173, 132], [241, 231, 93], [146, 220, 36], [34, 190, 168], [85, 118, 159], [56, 186, 111], [15, 186, 92], [4, 222, 153], [10, 220, 46], [212, 184, 168], [193, 242, 138], [137, 183, 216], [52, 193, 199], [39, 38, 222], [187, 182, 150], [139, 71, 45], [39, 235, 7], [154, 205, 108], [175, 175, 121], [220, 129, 194], [195, 233, 141], [191, 71, 209], [191, 60, 189], [184, 211, 166], [210, 119, 208], [85, 5, 224], [216, 173, 240], [219, 182, 207], [148, 119, 98], [252, 73, 95], [47, 48, 245], [141, 139, 248], [90, 92, 62], [101, 92, 162], [216, 211, 93], [62, 69, 34], [68, 113, 14], [159, 28, 58], [159, 173, 52], [237, 12, 17], [108, 24, 187], [161, 234, 99], [173, 161, 220], [48, 180, 12], [60, 182, 37], [4, 8, 160], [49, 163, 168], [166, 160, 156], [163, 65, 40], [17, 222, 26], [244, 148, 170], [49, 126, 177], [72, 68, 46], [211, 66, 155], [237, 182, 29], [121, 190, 105], [85, 148, 20], [95, 224, 99], [2, 214, 52], [19, 133, 157], [17, 134, 108], [160, 141, 142], [126, 169, 64], [242, 70, 168], [204, 192, 72], [34, 154, 101], [18, 12, 82], [255, 22, 125], [6, 74, 247], [31, 32, 236], [212, 55, 109], [197, 206, 42], [218, 141, 159], [132, 92, 85], [255, 107, 103], [213, 239, 215], [38, 26, 53], [150, 31, 200], [164, 163, 104], [36, 117, 137], [239, 4, 116], [49, 169, 107], [121, 167, 127], [194, 201, 176], [146, 40, 114], [158, 100, 194], [217, 115, 84], [70, 105, 115], [29, 152, 199], [83, 154, 247], [94, 29, 233], [153, 27, 15], [85, 253, 240], [9, 4, 47], [5, 224, 163], [250, 115, 243], [159, 249, 225], [180, 67, 47], [142, 124, 192], [45, 212, 35], [243, 208, 138], [40, 12, 240], [4, 206, 177], [82, 16, 205], [59, 34, 235], [159, 101, 43], [141, 213, 134], [191, 219, 127], [234, 142, 65], [185, 122, 221], [2, 135, 96], [127, 188, 26], [159, 25, 51], [179, 206, 163], [133, 56, 55], [32, 245, 96], [160, 223, 41], [94, 179, 133], [230, 224, 253], [141, 81, 226], [204, 231, 197], [145, 16, 142], [67, 235, 24], [235, 70, 31], [214, 82, 94], [218, 109, 216], [110, 36, 217], [40, 222, 74], [219, 86, 247], [120, 13, 222], [248, 91, 221], [41, 23, 145], [165, 56, 16], [195, 209, 46], [205, 21, 218], [178, 33, 208], [70, 49, 6], [2, 223, 109], [171, 241, 134], [208, 159, 21], [186, 2, 242], [39, 205, 33], [0, 148, 43], [91, 135, 130], [246, 116, 134], [74, 122, 68], [60, 22, 42], [212, 161, 120], [242, 160, 23], [9, 96, 68], [30, 250, 234], [35, 124, 22], [139, 15, 224], [203, 54, 165], [47, 52, 68], [215, 102, 167], [165, 65, 16], [196, 227, 202], [145, 236, 101], [221, 34, 50], [241, 127, 74], [229, 177, 86], [167, 194, 186], [149, 101, 173], [161, 121, 110], [108, 75, 88], [248, 119, 84], [229, 14, 48], [147, 44, 249], [145, 75, 107], [115, 166, 52], [243, 99, 126], [36, 121, 254], [34, 115, 38], [254, 100, 105], [103, 230, 164], [182, 81, 8], [82, 231, 186], [138, 164, 14], [33, 29, 9], [240, 78, 130], [62, 23, 255], [170, 13, 35], [47, 202, 246], [153, 251, 51], [40, 69, 199], [164, 84, 13], [181, 221, 75], [178, 53, 220], [106, 36, 25], [23, 122, 163], [116, 239, 242], [148, 207, 5], [120, 33, 70], [202, 75, 239], [246, 72, 221], [102, 14, 97], [131, 137, 255], [81, 182, 37], [180, 4, 128], [41, 59, 154], [188, 44, 20], [237, 123, 20], [11, 158, 154], [159, 232, 79], [161, 192, 130], [166, 200, 179], [47, 95, 161], [248, 42, 133], [182, 173, 6], [66, 63, 12], [14, 95, 243], [96, 215, 26], [98, 209, 167], [185, 38, 9], [109, 6, 212], [42, 5, 180], [221, 240, 29], [29, 209, 198], [142, 38, 103], [42, 210, 114], [118, 142, 166], [117, 102, 8], [43, 0, 25], [181, 22, 147], [148, 159, 161], [57, 255, 169], [113, 57, 105], [173, 98, 102], [237, 171, 236], [236, 54, 4], [174, 206, 42], [157, 195, 210], [96, 28, 91], [171, 199, 142], [39, 189, 89], [105, 144, 197], [84, 217, 176], [181, 29, 168], [32, 142, 146], [195, 167, 165], [196, 185, 70], [87, 97, 252], [61, 129, 190], [23, 174, 138], [170, 245, 27], [55, 139, 138], [237, 7, 18], [8, 178, 246], [22, 235, 76], [133, 122, 36], [80, 156, 216], [81, 10, 232], [90, 125, 207], [49, 21, 224], [165, 173, 204], [156, 133, 123], [244, 187, 195], [226, 137, 64], [22, 83, 209], [187, 176, 56], [134, 97, 240], [255, 165, 249], [55, 130, 235], [182, 14, 74], [248, 66, 95], [240, 178, 222], [193, 100, 201], [197, 233, 224], [11, 78, 210], [61, 197, 194], [208, 198, 140], [225, 101, 35], [149, 70, 39], [224, 63, 203], [52, 216, 197], [2, 249, 187], [97, 136, 19], [232, 37, 215], [220, 229, 156], [129, 96, 135], [177, 96, 173], [219, 39, 209], [62, 96, 172], [32, 76, 39], [124, 66, 198], [228, 174, 19], [97, 155, 216], [235, 108, 23], [195, 154, 167], [42, 82, 5], [94, 111, 223], [136, 254, 245], [216, 38, 66], [1, 159, 76], [36, 219, 0], [160, 12, 30], [35, 94, 150], [178, 181, 116], [72, 104, 43], [26, 153, 202], [21, 188, 88], [11, 80, 206], [229, 203, 55], [5, 221, 95], [221, 102, 197], [182, 36, 234], [71, 180, 227], [128, 18, 170], [142, 217, 218], [85, 169, 51], [25, 24, 185], [206, 123, 11], [212, 187, 86], [64, 40, 25], [91, 25, 73], [121, 211, 100], [123, 189, 140], [197, 197, 86], [34, 250, 241], [254, 167, 16], [243, 86, 146], [141, 164, 116], [119, 113, 2], [57, 212, 58], [250, 60, 85], [183, 57, 148], [207, 103, 85], [150, 124, 128], [167, 154, 85], [4, 30, 119], [64, 168, 250], [89, 144, 208], [62, 47, 28], [187, 128, 243], [115, 132, 99], [29, 89, 31], [243, 132, 8], [80, 13, 17], [85, 242, 130], [186, 107, 203], [175, 92, 128], [46, 223, 85], [191, 14, 149], [127, 95, 31], [109, 82, 60], [237, 168, 196], [136, 124, 197], [22, 24, 39], [59, 66, 226], [40, 253, 83], [175, 26, 194], [77, 214, 72], [166, 153, 18], [51, 28, 20], [232, 78, 90], [105, 89, 115], [164, 207, 34], [56, 34, 56], [119, 126, 21], [2, 141, 207], [38, 208, 149], [253, 235, 37], [180, 176, 232], [120, 106, 229], [190, 18, 40], [127, 139, 231], [91, 181, 177], [182, 183, 73], [72, 51, 190], [44, 41, 144], [178, 151, 69], [192, 91, 237], [225, 106, 69], [99, 155, 8], [65, 150, 133], [163, 243, 17], [121, 49, 211], [195, 251, 211], [9, 222, 44], [58, 86, 0], [182, 113, 123], [98, 93, 151], [4, 14, 50], [86, 204, 225], [26, 222, 39], [99, 98, 62], [126, 1, 242], [55, 216, 174], [216, 163, 166], [97, 196, 48], [253, 130, 158], [231, 165, 9], [188, 248, 86], [122, 247, 139], [44, 201, 189], [107, 228, 186], [234, 139, 16], [183, 138, 60], [177, 148, 62], [180, 13, 182], [84, 49, 221], [42, 171, 107], [243, 54, 108], [88, 165, 175], [19, 103, 15], [143, 248, 88], [18, 142, 73], [31, 132, 88], [148, 251, 139], [144, 159, 205], [207, 238, 40], [94, 238, 59], [135, 150, 183], [181, 204, 232], [110, 226, 13], [218, 147, 112], [138, 148, 137], [170, 137, 162], [90, 189, 160], [240, 123, 92], [142, 133, 107], [19, 208, 209], [173, 61, 180], [50, 124, 199], [46, 32, 12], [93, 3, 99], [224, 200, 154], [104, 26, 71], [108, 2, 76], [235, 66, 43], [223, 185, 249], [40, 80, 177], [105, 79, 109], [148, 84, 137], [144, 38, 37], [148, 29, 188], [134, 78, 197], [19, 254, 89], [178, 28, 81], [25, 198, 228], [191, 133, 50], [110, 79, 218], [42, 142, 185], [5, 33, 125], [219, 237, 135], [126, 5, 145], [101, 234, 19], [102, 193, 95], [30, 165, 36], [158, 124, 150], [66, 85, 229], [255, 31, 107], [67, 140, 103], [80, 25, 198], [200, 193, 183], [108, 176, 196], [118, 34, 223], [239, 239, 36], [216, 13, 132], [143, 182, 30], [53, 250, 59], [215, 50, 58], [209, 71, 5], [53, 253, 136], [54, 83, 6], [152, 203, 47], [115, 225, 246], [145, 234, 70], [239, 97, 189], [127, 110, 136], [127, 30, 212], [142, 192, 86], [245, 111, 70], [145, 34, 184], [153, 253, 25], [187, 53, 219], [136, 224, 194], [100, 117, 68], [210, 132, 112], [98, 158, 180], [161, 254, 53], [179, 154, 152], [226, 36, 214], [44, 96, 184], [216, 244, 27], [16, 95, 17], [23, 218, 82], [162, 58, 155], [62, 68, 164], [224, 71, 69], [36, 111, 50], [239, 141, 111], [145, 224, 36], [127, 190, 23], [132, 69, 248], [167, 174, 88], [65, 26, 63], [153, 145, 99], [220, 146, 190], [91, 79, 6], [88, 201, 251], [138, 252, 6], [152, 169, 178], [41, 219, 112], [129, 75, 30], [176, 149, 214], [180, 125, 7], [197, 60, 50], [191, 135, 67], [115, 32, 149], [245, 79, 161], [196, 138, 246], [57, 184, 100], [189, 146, 199], [22, 148, 239], [213, 28, 213], [193, 69, 122], [126, 192, 246], [3, 136, 251], [63, 45, 104], [14, 78, 187], [222, 51, 242], [89, 229, 191], [5, 155, 136], [239, 97, 146], [228, 190, 211], [37, 109, 126], [159, 138, 200], [250, 20, 63], [40, 166, 124], [182, 39, 234], [200, 136, 142], [139, 231, 37], [121, 197, 213], [187, 214, 41], [84, 66, 138], [152, 122, 65], [103, 44, 67], [153, 178, 171], [199, 157, 212], [35, 90, 137], [169, 79, 201], [62, 1, 10], [78, 104, 179], [193, 220, 109], [79, 149, 1], [188, 125, 100], [41, 68, 255], [222, 252, 156], [20, 21, 167], [250, 138, 141], [162, 54, 65], [199, 55, 144], [35, 213, 28], [149, 171, 203], [129, 22, 252], [55, 231, 11], [46, 163, 96], [136, 207, 241], [199, 89, 12], [108, 244, 211], [4, 235, 236], [66, 202, 181], [10, 85, 236], [169, 129, 96], [241, 69, 201], [150, 128, 146], [81, 233, 237], [59, 2, 197], [39, 69, 164], [194, 99, 194], [195, 196, 102], [208, 131, 76], [240, 144, 192], [167, 94, 33], [124, 180, 185], [12, 112, 228], [22, 109, 155], [133, 125, 26], [96, 70, 153], [237, 9, 97], [29, 104, 159], [61, 156, 130], [245, 32, 210], [207, 85, 184], [21, 34, 156], [7, 63, 232], [123, 207, 160], [20, 82, 100], [200, 225, 120], [253, 36, 24], [2, 170, 40], [230, 109, 113], [207, 80, 125], [50, 121, 229], [119, 18, 7], [59, 184, 246], [203, 217, 72], [187, 238, 133], [159, 160, 50], [239, 138, 234], [142, 109, 229], [212, 136, 190], [45, 50, 154], [59, 254, 87], [80, 213, 76], [202, 50, 79], [197, 86, 94], [216, 87, 223], [159, 184, 2], [192, 206, 159], [159, 108, 64], [185, 57, 228], [214, 18, 248], [138, 0, 45], [224, 163, 254], [41, 132, 69], [70, 90, 126], [53, 244, 247], [121, 38, 40], [54, 67, 149], [34, 165, 190], [95, 237, 196], [251, 53, 219], [106, 14, 129], [234, 163, 28], [247, 245, 106], [129, 182, 160], [146, 108, 186], [74, 218, 9], [154, 87, 49], [71, 242, 233], [31, 253, 52], [55, 190, 174], [24, 153, 155], [83, 158, 1], [54, 60, 22], [122, 31, 245], [159, 250, 169], [32, 106, 52], [214, 90, 135], [92, 56, 46], [149, 201, 196], [87, 220, 139], [212, 126, 82], [179, 76, 121], [57, 195, 235], [248, 50, 70], [139, 20, 211], [117, 53, 63], [15, 124, 74], [148, 177, 96], [187, 49, 31], [75, 159, 43], [157, 148, 174], [13, 150, 11], [133, 32, 55], [190, 150, 54], [65, 163, 216], [137, 193, 27], [30, 165, 102], [10, 143, 83], [245, 194, 187], [38, 136, 144], [29, 88, 71], [56, 226, 60], [13, 88, 186], [26, 12, 187], [236, 3, 81], [101, 61, 104], [166, 81, 138], [32, 146, 186], [23, 36, 247], [170, 133, 49], [68, 155, 64], [73, 194, 97], [85, 199, 3], [244, 93, 24], [251, 65, 32], [244, 22, 204], [197, 90, 72], [24, 21, 84], [190, 165, 23], [124, 132, 8], [67, 39, 0], [104, 184, 69], [186, 39, 66], [239, 13, 236], [23, 109, 154], [3, 225, 196], [133, 187, 145], [141, 217, 163], [94, 249, 143], [90, 164, 36], [95, 120, 75], [52, 140, 237], [200, 93, 48], [62, 141, 184], [165, 148, 193], [152, 70, 242], [56, 145, 38], [131, 43, 67], [231, 199, 45], [5, 11, 65], [87, 40, 146], [24, 143, 178], [214, 22, 104], [156, 78, 20], [115, 109, 155], [75, 200, 215], [252, 123, 6], [18, 113, 28], [21, 27, 99], [190, 159, 10], [230, 60, 137], [255, 155, 26], [72, 50, 172], [23, 49, 102], [173, 220, 225], [160, 42, 39], [171, 66, 196], [235, 96, 109], [16, 70, 187], [157, 119, 185], [191, 205, 158], [214, 112, 46], [225, 179, 45], [213, 33, 115], [21, 65, 169], [239, 206, 37], [117, 186, 153], [229, 220, 168], [210, 69, 224], [53, 191, 114], [248, 196, 145], [163, 171, 142], [243, 255, 10], [247, 123, 159], [3, 252, 68], [244, 238, 242], [195, 100, 10], [98, 183, 192], [6, 216, 232], [139, 6, 75], [133, 200, 166], [39, 13, 220], [182, 20, 47], [95, 185, 88], [135, 179, 2], [75, 197, 129], [59, 147, 137], [115, 109, 156], [59, 200, 62], [201, 197, 150], [252, 103, 100], [11, 6, 37], [146, 199, 238], [8, 178, 173], [155, 254, 47], [147, 150, 191], [51, 32, 211], [171, 243, 9], [60, 35, 247], [97, 238, 131], [212, 178, 93], [58, 120, 201], [1, 56, 98], [218, 71, 51], [183, 190, 217], [47, 165, 4], [59, 200, 95], [116, 98, 149], [185, 52, 242], [48, 34, 143], [50, 107, 184], [125, 135, 168], [35, 25, 246], [156, 165, 118], [129, 160, 177], [44, 48, 170], [18, 99, 156], [164, 3, 4], [124, 13, 112], [245, 157, 96], [215, 25, 75], [178, 172, 42], [232, 131, 49], [164, 173, 64], [46, 126, 220], [156, 201, 154], [77, 191, 112], [163, 44, 226], [199, 224, 134], [106, 46, 54], [255, 71, 217], [204, 156, 105], [68, 192, 4], [113, 191, 53], [174, 127, 153], [235, 244, 22], [151, 174, 44], [54, 243, 241], [140, 87, 248], [85, 190, 196], [105, 82, 172], [54, 24, 129], [106, 102, 143], [138, 117, 244], [250, 199, 225], [189, 42, 213], [148, 151, 202], [24, 133, 30], [125, 170, 65], [196, 25, 190], [14, 74, 227], [244, 1, 188], [225, 189, 53], [235, 84, 228], [147, 160, 46], [251, 249, 77], [8, 17, 39], [45, 173, 247], [176, 214, 68], [100, 112, 155], [162, 136, 34], [240, 6, 20], [88, 124, 250], [79, 139, 51], [76, 121, 170], [36, 126, 153], [208, 9, 35], [157, 150, 221], [234, 7, 33], [252, 207, 125], [127, 22, 254], [113, 253, 24], [75, 5, 231], [82, 246, 55], [209, 83, 86], [37, 8, 238], [58, 252, 80], [143, 7, 84], [200, 13, 101], [227, 130, 126], [43, 230, 158], [46, 156, 36], [226, 87, 183], [143, 185, 219], [9, 228, 124], [33, 113, 177], [44, 97, 16], [78, 102, 194], [1, 89, 78], [198, 113, 69], [122, 223, 216], [135, 139, 252], [254, 162, 252], [227, 169, 110], [57, 201, 217], [13, 154, 44], [89, 231, 85], [187, 253, 251], [67, 120, 197], [116, 213, 42], [43, 59, 255], [141, 177, 109], [232, 212, 167], [174, 123, 53], [200, 235, 105], [172, 89, 112], [224, 132, 22], [30, 161, 94], [183, 224, 240], [127, 203, 204], [11, 202, 127], [153, 255, 147], [103, 4, 166], [102, 128, 46], [180, 116, 182], [108, 191, 191], [157, 67, 168], [159, 212, 65], [201, 28, 122], [157, 114, 158], [225, 22, 168], [246, 251, 155], [171, 54, 171], [255, 120, 144], [45, 203, 193], [176, 227, 178], [114, 100, 245], [247, 130, 16], [72, 76, 187], [157, 108, 62], [5, 178, 52], [104, 226, 195], [221, 76, 230], [23, 206, 86], [125, 178, 255], [17, 164, 184], [34, 144, 120], [62, 182, 66], [115, 17, 194], [195, 148, 254], [147, 189, 162], [61, 250, 135], [224, 72, 250], [65, 211, 229], [240, 16, 40], [165, 39, 133], [134, 150, 204], [250, 208, 67], [48, 191, 177], [178, 144, 170], [180, 63, 40], [36, 118, 194], [177, 66, 3], [150, 46, 101], [13, 117, 219], [100, 158, 40], [9, 74, 0], [58, 179, 104], [165, 130, 83], [73, 119, 129], [49, 149, 17], [229, 96, 93], [73, 196, 230], [236, 34, 142], [78, 58, 245], [94, 107, 202], [255, 198, 63], [151, 30, 249], [103, 82, 229], [26, 57, 148], [195, 62, 173], [226, 11, 204], [252, 34, 146], [8, 151, 135], [170, 196, 71], [27, 121, 250], [183, 41, 83], [20, 136, 248], [85, 98, 170], [196, 133, 193], [70, 117, 98], [58, 65, 88], [219, 96, 17], [39, 155, 185], [122, 22, 186], [154, 90, 28], [252, 44, 202], [137, 232, 47], [235, 243, 210], [160, 12, 217], [133, 111, 54], [132, 71, 54], [44, 190, 72], [47, 46, 203], [146, 161, 228], [251, 40, 161], [109, 214, 209], [197, 176, 65], [90, 154, 198], [221, 178, 124], [212, 84, 233], [220, 27, 181], [65, 227, 84], [198, 132, 172], [51, 33, 45], [79, 112, 168], [191, 241, 254], [79, 110, 131], [201, 50, 202], [172, 225, 117], [158, 255, 185], [91, 17, 131], [216, 90, 163], [179, 99, 66], [187, 201, 188], [93, 82, 255], [151, 216, 95], [32, 19, 63], [75, 198, 18], [147, 243, 238], [232, 190, 32], [196, 203, 43], [18, 8, 208], [17, 25, 9], [95, 87, 164], [92, 63, 189], [21, 147, 204], [145, 97, 192], [248, 56, 83], [101, 195, 165], [131, 34, 50], [40, 157, 184], [142, 11, 51], [204, 191, 157], [16, 80, 186], [78, 99, 252], [114, 235, 240], [97, 225, 153], [100, 42, 212], [27, 207, 186], [37, 48, 126], [7, 159, 144], [90, 228, 221], [95, 60, 104], [241, 29, 1], [247, 11, 50], [3, 61, 44], [109, 184, 238], [139, 167, 137], [115, 107, 178], [103, 231, 250], [2, 49, 205], [150, 206, 123], [111, 163, 147], [5, 81, 177], [107, 150, 97], [210, 166, 163], [40, 30, 110], [247, 102, 60], [227, 10, 101], [204, 1, 15], [144, 207, 71], [0, 251, 86], [120, 189, 41], [150, 14, 74], [170, 167, 163], [182, 95, 121], [147, 73, 65], [46, 243, 56], [53, 169, 21], [129, 95, 200], [83, 26, 157], [201, 60, 194], [99, 23, 94], [66, 91, 196], [226, 183, 131], [104, 32, 43], [99, 176, 171], [205, 185, 153], [248, 193, 80], [65, 150, 180], [142, 207, 111], [106, 16, 32], [137, 98, 200], [70, 212, 215], [247, 89, 25], [56, 172, 189], [22, 146, 19], [88, 9, 6], [244, 235, 206], [198, 167, 252], [180, 184, 99], [62, 143, 30], [90, 170, 166], [110, 186, 154], [123, 59, 196], [14, 1, 243], [41, 173, 137], [24, 102, 210], [157, 211, 5], [82, 13, 141], [117, 28, 169], [24, 97, 94], [204, 70, 37], [250, 18, 36], [107, 78, 190], [46, 224, 11], [204, 214, 237], [25, 35, 199], [224, 31, 170], [206, 77, 48], [137, 67, 171], [22, 143, 241], [56, 98, 50], [48, 204, 118], [197, 152, 105], [212, 44, 204], [39, 54, 69], [221, 241, 191], [34, 146, 200], [162, 38, 98], [141, 106, 15], [227, 72, 30], [21, 99, 56], [121, 47, 210], [188, 42, 184], [150, 182, 174], [1, 253, 116], [26, 177, 47], [238, 111, 100], [124, 167, 71], [199, 235, 153], [45, 129, 123], [91, 231, 39], [22, 40, 160], [68, 112, 240], [168, 35, 139], [3, 3, 64], [88, 254, 66], [209, 161, 85], [167, 227, 86], [93, 102, 202], [218, 135, 252], [39, 81, 154], [191, 249, 218], [151, 103, 62], [1, 93, 72], [38, 93, 198], [115, 15, 194], [2, 255, 119], [104, 111, 148], [133, 120, 211], [246, 220, 171], [14, 3, 117], [85, 186, 208], [181, 71, 34], [206, 198, 47], [93, 193, 153], [171, 28, 65], [249, 111, 240], [83, 214, 38], [100, 90, 114], [124, 53, 180], [114, 17, 141], [144, 225, 188], [26, 226, 13], [182, 164, 147], [180, 46, 90], [147, 2, 15], [23, 177, 131], [144, 234, 183], [235, 92, 214], [149, 68, 50], [74, 141, 223], [246, 211, 160], [124, 27, 75], [231, 21, 171], [17, 198, 10], [172, 75, 212], [46, 208, 96], [189, 125, 65], [60, 215, 11], [187, 43, 144], [162, 254, 160], [166, 246, 54], [11, 90, 64], [124, 9, 34], [67, 112, 204], [102, 70, 57], [228, 217, 137], [124, 238, 58], [90, 16, 66], [204, 110, 32], [149, 204, 54], [43, 102, 222], [246, 111, 90], [132, 18, 13], [93, 19, 132], [185, 250, 103], [68, 69, 176], [74, 18, 33], [0, 149, 202], [27, 171, 249], [9, 22, 249], [137, 34, 188], [236, 134, 1], [183, 158, 199], [64, 12, 125], [21, 8, 109], [124, 108, 242], [214, 101, 121], [47, 164, 101], [244, 179, 175], [69, 169, 198], [74, 87, 9], [11, 4, 137], [72, 204, 191], [56, 49, 115], [117, 192, 180], [241, 13, 123], [86, 250, 6], [132, 112, 82], [255, 97, 69], [31, 141, 162], [189, 1, 82], [115, 234, 239], [244, 106, 157], [162, 41, 208], [244, 141, 12], [13, 231, 206], [172, 186, 189], [30, 6, 157], [40, 96, 156], [97, 249, 16], [140, 115, 227], [139, 43, 143], [129, 101, 49], [9, 63, 136], [36, 134, 166], [196, 16, 119], [59, 47, 110], [122, 71, 244], [210, 75, 129], [169, 0, 241], [52, 211, 166], [3, 42, 92], [2, 25, 67], [15, 81, 135], [73, 204, 216], [234, 236, 99], [244, 22, 248], [138, 198, 201], [148, 103, 237], [225, 174, 134], [187, 247, 234], [6, 15, 65], [57, 138, 116], [237, 192, 9], [137, 98, 213], [190, 68, 121], [58, 205, 39], [115, 72, 210], [80, 83, 54], [20, 83, 242], [128, 100, 251], [208, 57, 196], [130, 247, 154], [105, 98, 109], [233, 22, 108], [41, 148, 222], [34, 202, 55], [167, 63, 99], [4, 164, 92], [246, 142, 22], [195, 41, 149], [151, 199, 250], [241, 63, 6], [255, 153, 71], [67, 184, 136], [42, 237, 14], [176, 3, 39], [193, 149, 7], [255, 15, 165], [21, 80, 66], [171, 19, 153], [89, 235, 194], [146, 65, 94], [41, 147, 93], [88, 10, 226], [114, 83, 86], [62, 209, 205], [162, 99, 136], [98, 208, 225], [55, 53, 158], [198, 72, 60], [24, 44, 105], [236, 137, 96], [8, 243, 47], [126, 110, 219], [1, 162, 229], [243, 131, 105], [15, 103, 59], [55, 108, 75], [45, 26, 84], [13, 241, 27], [77, 50, 147], [208, 14, 27], [199, 189, 231], [225, 46, 31], [80, 96, 187], [134, 252, 97], [144, 213, 219], [38, 171, 238], [200, 111, 195], [92, 90, 79], [173, 106, 106], [219, 6, 245], [205, 220, 140], [17, 164, 91], [204, 18, 116], [18, 159, 203], [203, 182, 212], [233, 75, 203], [110, 208, 175], [74, 123, 255], [254, 70, 129], [95, 70, 23], [214, 64, 29], [3, 85, 189], [239, 255, 102], [187, 5, 106], [248, 158, 107], [240, 216, 5], [41, 208, 175], [108, 93, 59], [73, 99, 99], [11, 30, 48], [178, 253, 42], [38, 49, 7], [218, 39, 57], [48, 148, 246], [8, 146, 170], [110, 177, 66], [65, 106, 67], [82, 146, 115], [167, 178, 180], [49, 158, 118], [57, 224, 129], [248, 217, 162], [226, 130, 214], [177, 69, 181], [132, 25, 249], [120, 163, 179], [162, 69, 183], [160, 50, 30], [67, 88, 80], [222, 216, 35], [161, 104, 22], [43, 210, 146], [59, 182, 158], [153, 157, 111], [87, 147, 39], [153, 7, 238], [21, 207, 25], [113, 254, 97], [129, 226, 142], [110, 236, 203], [15, 161, 99], [199, 203, 12], [15, 227, 249], [86, 102, 52], [49, 168, 127], [103, 100, 75], [229, 238, 169], [98, 13, 10], [245, 193, 59], [164, 147, 228], [125, 33, 52], [85, 104, 112], [39, 125, 41], [145, 51, 200], [113, 178, 52], [155, 167, 161], [49, 85, 245], [122, 253, 43], [58, 82, 56], [51, 84, 223], [97, 114, 148], [212, 129, 211], [78, 126, 119], [52, 172, 104], [86, 29, 217], [80, 69, 40], [133, 68, 237], [9, 49, 34], [84, 142, 185], [229, 58, 154], [93, 115, 104], [146, 253, 66], [159, 88, 80], [162, 15, 98], [250, 193, 21], [168, 250, 227], [172, 172, 40], [226, 58, 253], [166, 57, 214], [78, 115, 88], [70, 100, 107], [74, 178, 221], [134, 158, 124], [26, 232, 139], [210, 12, 98], [210, 202, 226], [152, 19, 206], [194, 2, 35], [104, 48, 41], [31, 17, 249], [208, 88, 116], [227, 66, 242], [84, 213, 17], [97, 251, 9], [203, 37, 157], [213, 109, 138], [133, 168, 182], [158, 154, 136], [130, 116, 9], [228, 55, 81], [20, 45, 64], [95, 247, 117], [187, 172, 62], [61, 206, 104], [253, 120, 182], [114, 163, 130], [223, 18, 136], [135, 213, 147], [234, 40, 150], [205, 255, 152], [183, 0, 68], [237, 158, 154], [134, 64, 116], [103, 29, 165], [192, 116, 106], [36, 36, 95], [25, 90, 196], [103, 137, 79], [207, 62, 108], [251, 235, 183], [158, 104, 192], [248, 22, 135], [37, 220, 128], [84, 97, 250], [237, 37, 86], [132, 58, 77], [206, 66, 114], [192, 31, 104], [249, 82, 18], [92, 114, 34], [237, 39, 5], [126, 194, 23], [48, 211, 170], [219, 182, 254], [235, 53, 103], [217, 89, 200], [70, 47, 26], [115, 247, 150], [225, 78, 153], [21, 179, 230], [226, 112, 45], [26, 97, 39], [30, 62, 63], [61, 12, 1], [170, 252, 121], [153, 188, 191], [219, 191, 129], [13, 233, 103], [182, 78, 213], [116, 176, 130], [97, 56, 118], [88, 69, 86], [144, 247, 81], [97, 247, 88], [102, 33, 141], [219, 76, 77], [152, 176, 76], [252, 132, 100], [45, 56, 130], [35, 100, 11], [28, 133, 97], [76, 237, 71], [181, 200, 237], [245, 180, 131], [157, 11, 88], [110, 4, 38], [134, 17, 10], [210, 209, 82], [71, 28, 57], [224, 187, 86], [36, 139, 109], [227, 220, 229], [22, 59, 164], [17, 255, 180], [64, 54, 165], [190, 107, 25], [155, 174, 46], [219, 88, 160], [170, 155, 159], [117, 208, 86], [206, 75, 147], [224, 178, 242], [89, 35, 239], [205, 148, 103], [92, 131, 27], [96, 121, 138], [175, 213, 17], [108, 221, 162], [80, 98, 251], [190, 53, 188], [242, 63, 23], [227, 34, 204], [180, 112, 105], [58, 249, 77], [14, 132, 167], [73, 119, 112], [248, 245, 180], [103, 122, 253], [220, 243, 112], [195, 180, 75], [0, 118, 77], [188, 239, 205], [235, 201, 172], [27, 23, 16], [38, 157, 226], [176, 247, 250], [147, 120, 253], [9, 164, 238], [68, 144, 167], [26, 38, 237], [214, 224, 134], [207, 209, 86], [73, 131, 116], [92, 217, 77], [71, 57, 15], [10, 101, 168], [75, 63, 48], [234, 9, 143], [77, 117, 194], [17, 78, 185], [176, 29, 75], [73, 161, 130], [200, 55, 42], [235, 24, 131], [70, 94, 130], [191, 101, 62], [251, 242, 19], [144, 122, 99], [219, 22, 207], [234, 234, 189], [6, 107, 18], [159, 214, 15], [10, 120, 214], [133, 220, 33], [103, 196, 131], [136, 198, 192], [111, 151, 232], [199, 21, 95], [129, 169, 200], [186, 134, 42], [20, 69, 187], [78, 97, 128], [192, 75, 172], [113, 43, 206], [156, 19, 34], [159, 89, 183], [198, 235, 217], [57, 12, 196], [197, 179, 166], [230, 199, 208], [11, 102, 76], [255, 248, 188], [153, 220, 117], [215, 135, 211], [201, 229, 217], [183, 101, 35], [104, 143, 193], [85, 20, 65], [46, 21, 158], [101, 137, 128], [22, 34, 106], [117, 192, 55], [236, 164, 226], [244, 99, 12], [111, 87, 190], [19, 108, 71], [112, 169, 31], [131, 185, 142], [137, 127, 50], [202, 88, 68], [196, 75, 4], [253, 237, 77], [239, 84, 247], [112, 151, 13], [144, 206, 114], [217, 82, 46], [83, 137, 192], [43, 36, 218], [155, 41, 62], [99, 127, 90], [186, 219, 104], [16, 166, 219], [152, 168, 53], [18, 0, 223], [41, 13, 126], [124, 8, 152], [118, 149, 78], [57, 100, 157], [82, 255, 219], [245, 212, 191], [116, 16, 24], [40, 139, 94], [68, 118, 203], [118, 239, 46], [125, 80, 133], [226, 245, 151], [188, 239, 96], [96, 228, 248], [157, 84, 142], [157, 38, 225], [30, 116, 13], [111, 24, 250], [171, 35, 103], [200, 167, 214], [5, 131, 89], [251, 14, 216], [56, 7, 86], [152, 246, 187], [67, 98, 95], [0, 209, 80], [248, 151, 181], [90, 201, 144], [53, 154, 30], [22, 106, 225], [210, 161, 33], [175, 150, 5], [180, 169, 190], [68, 142, 73], [217, 91, 246], [195, 138, 134], [188, 137, 78], [45, 73, 146], [76, 100, 66], [130, 100, 55], [219, 248, 36], [227, 176, 58], [71, 142, 105], [12, 245, 63], [84, 87, 224], [126, 77, 81], [236, 68, 8], [29, 212, 28], [143, 145, 48], [231, 147, 194], [90, 183, 135], [141, 61, 114], [226, 72, 131], [85, 1, 226], [178, 135, 27], [9, 132, 112], [16, 172, 44], [86, 238, 35], [146, 159, 188], [207, 43, 156], [179, 19, 5], [104, 193, 190], [131, 253, 245], [242, 68, 20], [203, 22, 104], [83, 54, 111], [245, 248, 103], [91, 58, 7], [160, 219, 132], [45, 62, 135], [183, 216, 98], [254, 122, 225], [231, 218, 235], [223, 167, 188], [61, 8, 163], [218, 12, 15], [247, 49, 163], [119, 163, 0], [19, 212, 65], [149, 0, 245], [206, 222, 0], [226, 199, 226], [220, 16, 211], [174, 182, 46], [45, 209, 168], [134, 51, 152], [61, 131, 204], [234, 235, 81], [106, 59, 33], [117, 173, 97], [246, 88, 94], [186, 176, 22], [25, 57, 215], [123, 222, 169], [24, 229, 163], [166, 85, 245], [43, 4, 194], [77, 162, 194], [219, 31, 103], [191, 129, 230], [47, 197, 156], [131, 246, 238], [182, 227, 142], [137, 157, 161], [206, 251, 57], [222, 152, 14], [178, 140, 190], [31, 153, 88], [115, 218, 21], [73, 112, 73], [183, 5, 79], [49, 155, 141], [226, 149, 91], [228, 116, 42], [109, 11, 29], [94, 15, 53], [94, 226, 16], [103, 41, 154], [34, 144, 112], [62, 241, 128], [10, 131, 52], [209, 77, 240], [239, 223, 118], [234, 183, 22], [88, 13, 121], [199, 235, 177], [152, 21, 211], [67, 147, 93], [41, 243, 119], [158, 163, 131], [196, 6, 57], [14, 19, 133], [227, 54, 13], [169, 255, 54], [200, 231, 120], [183, 203, 213], [26, 170, 62], [197, 183, 110], [87, 11, 190], [221, 168, 52], [38, 226, 55], [219, 153, 198], [85, 239, 51], [112, 66, 87], [192, 198, 126], [149, 70, 134], [90, 235, 245], [177, 117, 245], [196, 174, 166], [165, 135, 58], [19, 18, 222], [159, 175, 200], [16, 179, 213], [193, 12, 31], [29, 99, 130], [199, 142, 168], [72, 103, 154], [207, 182, 175], [207, 181, 170], [80, 181, 67], [55, 217, 235], [209, 177, 162], [69, 89, 83], [84, 25, 10], [247, 164, 33], [205, 224, 49], [71, 9, 22], [130, 5, 231], [197, 234, 236], [124, 1, 90], [23, 228, 239], [74, 53, 89], [67, 212, 18], [39, 78, 159], [7, 99, 217], [44, 153, 153], [243, 18, 116], [55, 240, 82], [79, 68, 192], [169, 94, 31], [123, 171, 34], [167, 98, 175], [8, 146, 153], [14, 60, 184], [90, 67, 175], [235, 141, 206], [99, 13, 183], [190, 37, 152], [169, 235, 27], [197, 246, 10], [103, 212, 228], [13, 77, 79], [246, 142, 237], [29, 195, 155], [6, 239, 152], [6, 164, 119], [23, 75, 182], [127, 75, 247], [191, 110, 132], [80, 178, 75], [3, 162, 223], [95, 215, 64], [19, 67, 95], [219, 251, 61], [25, 217, 149], [135, 23, 149], [126, 235, 250], [13, 62, 118], [223, 182, 7], [180, 169, 57], [15, 65, 131], [181, 87, 25], [2, 138, 40], [238, 246, 182], [5, 23, 230], [26, 185, 178], [177, 12, 28], [198, 170, 222], [209, 217, 63], [125, 118, 76], [253, 66, 172], [182, 134, 251], [249, 41, 166], [145, 62, 195], [75, 211, 126], [51, 153, 11], [189, 115, 144], [165, 31, 228], [24, 109, 148], [107, 243, 175], [185, 109, 3], [94, 195, 73], [147, 110, 145], [3, 39, 254], [102, 245, 36], [1, 155, 148], [196, 165, 189], [158, 11, 67], [161, 198, 89], [2, 15, 43], [136, 157, 152], [81, 18, 6], [171, 72, 82], [125, 59, 193], [28, 235, 172], [101, 221, 236], [2, 150, 195], [115, 89, 133], [235, 118, 2], [156, 52, 191], [115, 174, 142], [156, 199, 198], [238, 31, 231], [63, 171, 0], [51, 120, 69], [40, 15, 212], [139, 190, 102], [52, 221, 19], [229, 46, 252], [39, 7, 169], [63, 37, 128], [158, 168, 194], [99, 160, 185], [114, 178, 33], [249, 177, 31], [159, 13, 189], [113, 201, 51], [72, 104, 58], [248, 36, 252], [53, 156, 240], [131, 112, 246], [135, 187, 232], [119, 76, 189], [86, 37, 151], [99, 182, 198], [137, 129, 170], [34, 206, 51], [249, 203, 249], [11, 120, 148], [72, 97, 47], [246, 178, 193], [133, 65, 128], [175, 20, 196], [213, 125, 142], [106, 214, 231], [39, 23, 75], [211, 4, 189], [128, 119, 145], [178, 76, 107], [149, 141, 34], [207, 198, 207], [160, 67, 206], [190, 173, 102], [129, 210, 11], [155, 123, 168], [32, 69, 215], [232, 24, 49], [100, 6, 161], [65, 90, 71], [57, 210, 141], [186, 185, 78], [111, 204, 61], [139, 80, 159], [93, 215, 150], [118, 224, 255], [125, 254, 18], [163, 222, 204], [20, 85, 222], [74, 247, 202], [163, 192, 86], [249, 120, 192], [160, 157, 230], [124, 169, 79], [133, 255, 141], [92, 196, 188], [227, 73, 75], [197, 24, 149], [244, 63, 240], [124, 75, 30], [254, 131, 86], [68, 214, 162], [194, 250, 47], [223, 84, 166], [97, 65, 171], [163, 80, 210], [243, 40, 88], [101, 47, 195], [241, 253, 111], [191, 104, 93], [173, 122, 168], [85, 197, 205], [107, 156, 224], [206, 154, 18], [110, 119, 178], [15, 233, 79], [37, 58, 185], [219, 239, 77], [45, 191, 251], [162, 174, 177], [94, 248, 189], [100, 222, 226], [163, 74, 189], [75, 23, 24], [146, 148, 235], [254, 203, 119], [31, 106, 200], [197, 111, 216], [130, 66, 131], [167, 253, 132], [149, 164, 219], [34, 7, 246], [232, 155, 99], [189, 14, 94], [230, 23, 233], [71, 144, 70], [214, 97, 142], [190, 29, 218], [59, 73, 83], [132, 187, 128], [164, 166, 5], [139, 237, 40], [232, 219, 96], [6, 204, 12], [156, 186, 110], [179, 165, 177], [139, 20, 73], [82, 93, 197], [16, 110, 21], [217, 135, 144], [27, 76, 210], [91, 101, 159], [225, 44, 98], [73, 139, 148], [229, 24, 120], [192, 34, 109], [170, 120, 91], [140, 108, 251], [51, 136, 144], [172, 203, 186], [33, 113, 145], [152, 97, 46], [174, 137, 107], [136, 70, 43], [49, 197, 188], [174, 79, 112], [156, 183, 73], [125, 133, 241], [217, 67, 72], [79, 252, 98], [155, 120, 148], [2, 47, 112], [62, 97, 146], [15, 241, 137], [189, 243, 82], [138, 231, 22], [190, 148, 87], [111, 193, 134], [204, 35, 119], [1, 60, 203], [169, 217, 221], [67, 73, 42], [31, 37, 113], [141, 255, 243], [219, 153, 224], [31, 123, 15], [84, 225, 58], [59, 254, 206], [160, 197, 22], [1, 130, 200], [234, 43, 167], [134, 125, 107], [183, 155, 38], [141, 75, 163], [22, 48, 78], [54, 45, 104], [48, 208, 165], [168, 28, 217], [163, 213, 90], [52, 196, 50], [230, 97, 118], [9, 0, 202], [38, 121, 242], [176, 13, 150], [22, 141, 111], [31, 93, 6], [50, 3, 9], [6, 13, 35], [224, 91, 201], [203, 109, 110], [143, 81, 82], [169, 20, 166], [131, 55, 164], [139, 66, 222], [204, 95, 95], [194, 181, 254], [251, 103, 218], [152, 115, 183], [58, 104, 166], [148, 131, 226], [204, 159, 145], [50, 5, 13], [153, 222, 128], [140, 146, 27], [243, 114, 77], [110, 100, 97], [36, 215, 204], [122, 184, 189], [17, 115, 129], [183, 118, 181], [134, 89, 61], [83, 147, 26], [68, 219, 254], [255, 19, 129], [99, 223, 111], [156, 150, 253], [229, 151, 246], [228, 115, 68], [177, 203, 223], [11, 223, 222], [165, 3, 211], [0, 113, 24], [205, 2, 117], [96, 39, 211], [161, 189, 176], [161, 138, 98], [142, 251, 47], [80, 58, 19], [124, 112, 48], [114, 62, 78], [140, 140, 21], [121, 71, 0], [46, 143, 184], [148, 251, 95], [178, 22, 45], [154, 237, 254], [38, 226, 249], [213, 114, 230], [127, 85, 157], [223, 186, 57], [219, 143, 244], [212, 101, 94], [90, 213, 214], [138, 236, 68], [198, 117, 127], [104, 226, 14], [142, 147, 60], [80, 163, 36], [93, 85, 91], [57, 169, 111], [171, 54, 117], [29, 241, 72], [35, 119, 51], [223, 17, 1], [178, 182, 10], [180, 206, 80], [59, 0, 197], [35, 234, 55], [4, 247, 122], [97, 212, 103], [101, 129, 87], [163, 200, 154], [174, 203, 115], [188, 29, 84], [232, 147, 86], [109, 71, 240], [0, 94, 185], [206, 167, 164], [157, 156, 22], [239, 93, 174], [177, 36, 83], [17, 250, 40], [120, 249, 242], [203, 5, 178], [219, 91, 177], [87, 51, 89], [67, 114, 25], [191, 185, 97], [91, 133, 194], [145, 53, 108], [249, 198, 206], [170, 43, 178], [70, 158, 143], [46, 41, 72], [249, 250, 159], [214, 34, 236], [37, 153, 116], [214, 216, 197], [49, 74, 10], [229, 57, 76], [74, 103, 15], [224, 201, 102], [36, 246, 99], [171, 114, 234], [4, 46, 145], [17, 9, 74], [142, 209, 190], [89, 81, 31], [207, 125, 45], [179, 228, 18], [235, 90, 201], [204, 204, 64], [180, 117, 161], [203, 196, 255], [124, 206, 136], [217, 26, 180], [30, 9, 39], [180, 192, 82], [142, 0, 240], [32, 151, 178], [241, 222, 103], [127, 114, 41], [133, 101, 147], [107, 66, 137], [214, 230, 101], [56, 197, 199], [241, 226, 89], [163, 124, 199], [50, 235, 124], [110, 134, 148], [209, 163, 105], [204, 218, 224], [241, 213, 238], [123, 33, 71], [108, 226, 87], [187, 150, 254], [18, 27, 113], [231, 136, 21], [230, 136, 159], [30, 150, 188], [74, 102, 232], [8, 197, 85], [205, 47, 41], [214, 96, 66], [54, 53, 254], [36, 240, 22], [113, 22, 8], [140, 14, 216], [113, 174, 33], [71, 248, 135], [6, 91, 186], [137, 81, 231], [189, 74, 153], [107, 14, 65], [18, 105, 38], [81, 20, 36], [206, 243, 58], [243, 236, 37], [127, 99, 209], [98, 251, 111], [195, 21, 142], [154, 156, 26], [61, 0, 143], [59, 60, 91], [10, 142, 49], [163, 192, 119], [54, 247, 10], [175, 53, 32], [234, 3, 32], [213, 221, 59], [209, 104, 77], [94, 65, 177], [91, 70, 36], [207, 236, 50], [181, 42, 230], [235, 183, 26], [23, 125, 143], [65, 186, 110], [100, 199, 112], [109, 111, 65], [157, 203, 158], [229, 174, 104], [113, 28, 16], [62, 150, 47], [222, 245, 91], [102, 28, 153], [181, 116, 1], [129, 158, 122], [1, 68, 90], [69, 117, 244], [60, 101, 177], [213, 44, 245], [134, 129, 209], [109, 18, 177], [183, 234, 75], [11, 107, 222], [42, 84, 220], [181, 108, 85], [10, 131, 227], [114, 151, 160], [175, 8, 103], [45, 35, 26], [253, 211, 178], [31, 113, 78], [148, 162, 212], [234, 90, 23], [253, 50, 74], [240, 214, 214], [62, 67, 216], [192, 158, 95], [251, 92, 30], [81, 107, 17], [251, 52, 176], [34, 121, 48], [104, 202, 182], [63, 209, 20], [63, 125, 227], [83, 242, 31], [102, 132, 72], [55, 242, 252], [144, 235, 55], [34, 65, 147], [116, 160, 195], [173, 54, 32], [138, 82, 76], [34, 145, 39], [32, 86, 154], [205, 138, 109], [100, 24, 132], [112, 140, 234], [39, 199, 204], [214, 147, 125], [55, 160, 145], [172, 179, 7], [11, 223, 167], [69, 215, 242], [170, 152, 77], [20, 167, 1], [135, 170, 147], [58, 36, 169], [69, 130, 224], [43, 145, 14], [139, 131, 21], [91, 208, 13], [38, 52, 55], [241, 216, 62], [231, 30, 160], [192, 218, 200], [123, 88, 29], [14, 225, 14], [247, 136, 213], [77, 140, 0], [168, 163, 156], [73, 160, 115], [93, 202, 202], [203, 130, 203], [73, 220, 208], [79, 35, 163], [3, 243, 9], [85, 215, 233], [65, 108, 73], [36, 191, 123], [36, 99, 106], [143, 126, 24], [13, 93, 85], [84, 142, 165], [169, 155, 109], [105, 129, 157], [84, 117, 91], [156, 55, 190], [89, 209, 230], [74, 194, 111], [185, 27, 45], [5, 132, 252], [148, 160, 85], [118, 157, 66], [2, 188, 190], [168, 198, 20], [236, 19, 105], [24, 248, 40], [164, 181, 4], [204, 59, 237], [58, 73, 242], [241, 124, 102], [160, 119, 188], [217, 13, 151], [82, 19, 169], [202, 49, 103], [1, 233, 16], [136, 213, 180], [41, 13, 76], [136, 169, 171], [227, 4, 246], [152, 162, 185], [18, 70, 110], [40, 159, 200], [82, 48, 169], [115, 138, 48], [55, 37, 203], [224, 18, 71], [152, 58, 171], [226, 201, 107], [91, 52, 104], [169, 80, 17], [118, 1, 47], [193, 241, 2], [29, 125, 24], [180, 119, 202], [19, 157, 169], [91, 125, 84], [61, 255, 152], [77, 126, 153], [95, 89, 100], [45, 176, 218], [150, 35, 53], [248, 64, 75], [17, 172, 69], [185, 57, 154], [255, 55, 38], [233, 223, 87], [158, 24, 19], [111, 82, 22], [3, 12, 64], [113, 57, 221], [159, 141, 90], [27, 241, 159], [20, 3, 167], [254, 247, 223], [227, 138, 164], [20, 20, 16], [212, 79, 62], [80, 161, 150], [99, 183, 193], [180, 52, 91], [168, 152, 46], [122, 231, 219], [200, 99, 7], [22, 174, 75], [162, 208, 237], [168, 35, 145], [111, 255, 170], [49, 41, 66], [234, 40, 158], [66, 24, 181], [179, 211, 159], [117, 52, 127], [193, 187, 126], [19, 66, 175], [194, 93, 224], [176, 245, 67], [230, 24, 182], [27, 124, 53], [14, 155, 236], [81, 171, 251], [3, 62, 158], [23, 102, 61], [53, 156, 87], [206, 79, 56], [17, 227, 159], [25, 248, 88], [141, 228, 216], [144, 57, 13], [133, 254, 34], [131, 171, 225], [146, 37, 253], [126, 115, 91], [96, 27, 14], [53, 182, 186], [48, 157, 14], [31, 28, 26], [32, 213, 63], [103, 45, 7], [12, 25, 43], [42, 81, 114], [200, 100, 141], [215, 198, 82], [210, 82, 187], [1, 138, 173], [89, 213, 77], [170, 129, 243], [149, 45, 254], [109, 121, 237], [126, 158, 7], [235, 95, 231], [6, 22, 93], [114, 233, 200], [168, 39, 111], [53, 114, 157], [115, 60, 218], [186, 104, 11], [102, 76, 194], [189, 223, 83], [101, 45, 196], [247, 96, 67], [11, 31, 113], [235, 138, 98], [140, 222, 180], [241, 49, 175], [166, 219, 181], [220, 145, 38], [140, 85, 90], [251, 196, 193], [247, 113, 167], [89, 107, 39], [2, 117, 215], [255, 10, 188], [109, 163, 39], [104, 22, 114], [63, 9, 214], [152, 224, 250], [104, 201, 152], [178, 106, 131], [187, 156, 229], [129, 189, 93], [205, 116, 67], [198, 23, 72], [196, 190, 128], [166, 92, 167], [154, 208, 7], [71, 91, 170], [27, 56, 123], [165, 183, 79], [117, 130, 83], [128, 195, 85], [1, 183, 50], [155, 245, 149], [132, 119, 34], [90, 234, 77], [228, 209, 87], [115, 2, 183], [158, 154, 205], [69, 31, 226], [87, 136, 40], [102, 48, 215], [82, 156, 194], [39, 182, 93], [128, 45, 171], [152, 30, 146], [82, 238, 122], [125, 115, 76], [157, 231, 227], [245, 199, 44], [163, 243, 204], [191, 47, 116], [26, 13, 6], [21, 230, 43], [107, 55, 181], [155, 147, 162], [77, 29, 71], [120, 149, 199], [51, 254, 233], [238, 244, 229], [14, 99, 230], [119, 132, 77], [197, 143, 36], [37, 216, 255], [117, 252, 5], [207, 16, 219], [196, 183, 53], [202, 144, 68], [194, 58, 45], [163, 185, 236], [128, 154, 70], [221, 146, 42], [45, 172, 115], [158, 35, 195], [202, 116, 214], [240, 178, 205], [38, 31, 104], [98, 6, 149], [10, 218, 67], [148, 158, 237], [245, 172, 76], [47, 179, 187], [105, 175, 28], [142, 169, 67], [35, 161, 129], [244, 230, 20], [173, 9, 234], [128, 251, 206], [29, 187, 54], [253, 147, 238], [29, 215, 179], [139, 221, 250], [67, 158, 141], [51, 245, 31], [242, 34, 210], [11, 77, 133], [134, 69, 205], [87, 96, 92], [42, 103, 194], [233, 35, 171], [3, 173, 109], [32, 148, 87], [160, 91, 3], [228, 3, 237], [76, 30, 106], [233, 39, 193], [230, 226, 216], [103, 217, 213], [182, 77, 165], [173, 198, 249], [48, 1, 127], [128, 106, 18], [243, 234, 219], [21, 232, 72], [200, 179, 186], [51, 47, 46], [143, 11, 253], [139, 126, 88], [231, 54, 118], [188, 150, 201], [246, 228, 56], [35, 12, 144], [172, 22, 227], [231, 19, 249], [225, 245, 9], [160, 149, 120], [87, 12, 54], [175, 252, 250], [237, 213, 208], [214, 134, 242], [145, 172, 131], [139, 86, 253], [12, 124, 22], [87, 48, 179], [138, 225, 37], [172, 197, 118], [186, 229, 126], [185, 180, 232], [22, 187, 92], [70, 199, 105], [95, 55, 57], [188, 238, 141], [96, 180, 179], [170, 187, 6], [243, 229, 67], [193, 100, 77], [78, 41, 163], [94, 66, 73], [40, 197, 169], [92, 96, 115], [73, 29, 76], [210, 75, 55], [29, 178, 66], [51, 14, 14], [193, 167, 230], [114, 127, 48], [82, 137, 41], [174, 226, 1], [251, 29, 18], [21, 34, 202], [128, 172, 134], [245, 2, 51], [211, 199, 141], [33, 129, 88], [223, 214, 235], [1, 173, 164], [109, 153, 19], [36, 45, 27], [143, 213, 15], [28, 52, 177], [151, 238, 93], [141, 200, 21], [50, 70, 208], [58, 231, 160], [180, 43, 115], [66, 168, 14], [250, 252, 198], [23, 77, 22], [152, 44, 158], [168, 152, 208], [209, 57, 67], [186, 212, 152], [107, 249, 161], [120, 49, 249], [146, 83, 35], [215, 140, 149], [193, 202, 49], [157, 49, 59], [132, 88, 150], [184, 214, 59], [166, 231, 162], [225, 116, 158], [32, 62, 55], [233, 35, 99], [165, 192, 42], [71, 80, 143], [193, 248, 202], [103, 9, 148], [65, 138, 67], [10, 155, 149], [90, 134, 149], [85, 106, 119], [246, 32, 245], [41, 83, 239], [76, 58, 15], [159, 147, 201], [138, 212, 154], [217, 89, 173], [178, 0, 200], [250, 34, 179], [73, 181, 178], [170, 168, 173], [208, 154, 161], [41, 41, 6], [223, 78, 40], [182, 101, 80], [15, 69, 215], [145, 114, 187], [96, 107, 234], [87, 162, 97], [90, 29, 32], [190, 15, 99], [166, 201, 107], [78, 202, 188], [106, 232, 181], [190, 54, 168], [149, 50, 84], [83, 3, 3], [193, 199, 203], [246, 166, 222], [162, 52, 187], [105, 216, 124], [152, 22, 204], [225, 108, 181], [17, 228, 192], [14, 179, 162], [7, 57, 208], [86, 156, 54], [1, 42, 46], [64, 4, 246], [113, 122, 52], [104, 13, 88], [98, 181, 210], [115, 12, 147], [238, 224, 150], [97, 183, 30], [28, 62, 103], [180, 221, 73], [252, 137, 86], [18, 164, 131], [70, 29, 67], [240, 60, 224], [66, 150, 180], [237, 184, 131], [83, 218, 138], [18, 104, 132], [38, 226, 60], [231, 236, 37], [60, 34, 250], [153, 79, 228], [169, 227, 42], [189, 189, 84], [93, 165, 130], [52, 203, 247], [136, 20, 204], [171, 254, 119], [142, 246, 250], [41, 249, 233], [11, 112, 19], [241, 45, 24], [177, 141, 238], [50, 40, 62], [55, 176, 189], [243, 56, 86], [5, 148, 196], [130, 10, 95], [77, 161, 118], [103, 166, 183], [115, 99, 24], [57, 110, 80], [159, 91, 223], [70, 232, 226], [184, 233, 13], [154, 69, 189], [130, 155, 173], [157, 38, 12], [192, 17, 216], [197, 16, 28], [146, 169, 59], [182, 46, 37], [115, 164, 5], [217, 237, 247], [148, 9, 4], [46, 43, 222], [159, 119, 63], [255, 19, 22], [124, 79, 241], [151, 100, 149], [183, 120, 247], [114, 48, 224], [196, 205, 142], [156, 41, 173], [47, 112, 66], [196, 148, 155], [38, 251, 128], [216, 5, 85], [153, 11, 235], [9, 252, 60], [90, 194, 125], [167, 85, 80], [163, 35, 155], [66, 149, 186], [68, 163, 180], [200, 188, 159], [11, 62, 50], [53, 1, 83], [173, 158, 85], [178, 101, 220], [187, 120, 103], [188, 118, 92], [136, 43, 59], [96, 160, 94], [255, 118, 79], [13, 103, 137], [246, 56, 245], [174, 177, 146], [20, 191, 147], [176, 240, 198], [46, 115, 168], [86, 120, 152], [226, 194, 169], [99, 149, 140], [74, 184, 22], [241, 70, 11], [187, 129, 44], [169, 129, 222], [37, 2, 206], [240, 196, 174], [131, 97, 175], [185, 219, 37], [154, 87, 244], [84, 255, 18], [32, 155, 78], [226, 108, 31], [122, 40, 23], [229, 74, 152], [238, 153, 155], [209, 117, 118], [37, 249, 195], [56, 52, 134], [191, 215, 159], [88, 113, 172], [130, 171, 235], [191, 147, 146], [172, 53, 14], [5, 106, 93], [2, 246, 149], [126, 115, 190], [145, 172, 184], [3, 20, 93], [138, 148, 30], [135, 25, 240], [164, 10, 232], [223, 60, 35], [108, 250, 11], [63, 43, 189], [229, 168, 148], [235, 159, 176], [116, 34, 196], [102, 0, 9], [6, 217, 21], [187, 126, 79], [53, 253, 76], [184, 71, 214], [253, 126, 2], [86, 28, 121], [223, 125, 198], [215, 53, 208], [88, 5, 34], [175, 64, 13], [208, 245, 170], [212, 249, 167], [156, 204, 112], [194, 164, 245], [234, 24, 148], [6, 209, 26], [204, 16, 209], [137, 66, 170], [20, 144, 247], [249, 104, 70], [164, 80, 75], [7, 156, 242], [232, 251, 229], [228, 19, 93], [53, 152, 171], [163, 235, 149], [170, 100, 75], [166, 97, 3], [117, 189, 137], [247, 78, 21], [3, 212, 34], [99, 198, 131], [249, 252, 104], [242, 220, 22], [96, 216, 250], [221, 80, 97], [12, 80, 100], [113, 130, 68], [110, 223, 78], [232, 175, 191], [61, 91, 140], [179, 183, 178], [161, 84, 251], [153, 146, 228], [250, 162, 148], [139, 124, 129], [126, 214, 35], [115, 154, 97], [236, 0, 138], [20, 3, 102], [239, 169, 3], [216, 130, 181], [173, 83, 112], [85, 163, 134], [175, 138, 110], [56, 116, 21], [207, 95, 235], [164, 212, 237], [74, 88, 54], [37, 205, 167], [218, 197, 118], [92, 115, 68], [26, 245, 38], [133, 239, 25], [116, 250, 43], [185, 109, 190], [24, 54, 106], [63, 233, 73], [219, 227, 244], [253, 240, 86], [44, 147, 233], [224, 139, 147], [149, 195, 117], [177, 187, 3], [200, 93, 208], [7, 245, 71], [212, 98, 137], [242, 26, 114], [72, 241, 144], [154, 255, 231], [200, 147, 253], [152, 156, 207], [180, 175, 42], [77, 80, 77], [97, 73, 113], [1, 175, 19], [30, 7, 19], [134, 18, 173], [137, 69, 203], [244, 169, 226], [208, 211, 196], [186, 223, 191], [189, 71, 230], [95, 173, 190], [204, 105, 145], [174, 139, 93], [157, 73, 162], [176, 123, 124], [253, 102, 150], [66, 41, 116], [97, 133, 159], [192, 156, 177], [82, 242, 226], [240, 99, 10], [239, 47, 225], [232, 42, 40], [157, 207, 182], [77, 174, 99], [145, 179, 34], [21, 227, 189], [125, 182, 123], [236, 33, 4], [103, 186, 78], [199, 226, 141], [141, 195, 3], [154, 38, 223], [215, 107, 225], [8, 30, 204], [221, 54, 106], [175, 71, 115], [105, 231, 22], [222, 190, 207], [247, 176, 62], [56, 52, 138], [53, 41, 208], [212, 64, 162], [206, 52, 60], [37, 116, 223], [36, 70, 46], [180, 212, 85], [207, 34, 183], [100, 105, 220], [80, 87, 159], [129, 228, 119], [119, 29, 2], [144, 247, 117], [232, 195, 159], [182, 210, 50], [164, 213, 72], [176, 204, 28], [167, 30, 45], [80, 17, 47], [254, 0, 89], [107, 95, 106], [252, 16, 86], [125, 57, 203], [211, 217, 165], [124, 117, 18], [166, 153, 47], [162, 182, 0], [197, 131, 144], [6, 76, 93], [145, 140, 213], [67, 152, 41], [219, 221, 105], [32, 170, 124], [8, 36, 108], [236, 129, 216], [122, 161, 6], [159, 87, 166], [243, 60, 45], [124, 248, 12], [237, 93, 156], [140, 163, 100], [65, 214, 111], [118, 152, 34], [199, 82, 10], [56, 59, 184], [182, 158, 45], [68, 240, 209], [219, 7, 106], [147, 101, 71], [173, 127, 75], [208, 146, 42], [103, 183, 80], [157, 191, 52], [104, 151, 58], [196, 180, 226], [229, 235, 12], [109, 137, 41], [150, 70, 4], [146, 131, 186], [90, 190, 227], [113, 45, 20], [164, 76, 26], [253, 118, 225], [139, 184, 112], [229, 73, 144], [82, 91, 111], [149, 69, 29], [26, 193, 48], [205, 219, 253], [207, 191, 140], [124, 63, 96], [246, 178, 207], [112, 99, 186], [136, 241, 58], [119, 143, 217], [153, 222, 12], [67, 220, 237], [61, 58, 188], [223, 117, 123], [249, 233, 47], [143, 254, 221], [223, 1, 166], [106, 129, 76], [216, 204, 52], [213, 66, 2], [236, 123, 162], [40, 229, 170], [152, 179, 104], [237, 230, 207], [40, 107, 9], [92, 71, 235], [141, 221, 230], [120, 5, 93], [225, 61, 127], [244, 18, 136], [76, 128, 177], [12, 81, 100], [54, 103, 179], [33, 90, 122], [207, 23, 70], [239, 224, 15], [27, 167, 101], [80, 194, 217], [16, 121, 151], [17, 47, 44], [211, 66, 220], [254, 119, 172], [29, 43, 131], [153, 108, 216], [219, 109, 108], [77, 69, 44], [65, 217, 11], [248, 77, 1], [53, 230, 226], [176, 115, 46], [10, 75, 88], [181, 235, 48], [167, 67, 152], [221, 173, 149], [144, 170, 156], [193, 212, 26], [154, 83, 155], [175, 172, 126], [75, 77, 55], [210, 173, 9], [24, 240, 196], [189, 105, 196], [166, 63, 179], [79, 117, 59], [187, 132, 186], [198, 12, 75], [208, 162, 253], [28, 239, 107], [175, 249, 220], [211, 166, 171], [192, 79, 17], [182, 134, 217], [79, 96, 19], [226, 19, 67], [200, 186, 219], [99, 175, 60], [61, 205, 121], [84, 162, 114], [236, 68, 153], [13, 76, 193], [149, 165, 176], [157, 221, 33], [100, 250, 90], [18, 254, 76], [214, 201, 140], [234, 18, 42], [33, 129, 166], [225, 161, 121], [133, 10, 24], [29, 51, 201], [139, 63, 184], [216, 65, 49], [43, 67, 155], [23, 254, 173], [80, 254, 99], [136, 69, 148], [113, 91, 152], [51, 11, 235], [28, 165, 153], [97, 126, 7], [205, 87, 105], [77, 208, 116], [169, 34, 213], [70, 124, 63], [182, 148, 208], [206, 1, 139], [4, 109, 46], [97, 89, 13], [196, 43, 100], [122, 58, 171], [26, 113, 38], [232, 244, 58], [177, 95, 217], [69, 176, 184], [213, 191, 255], [15, 47, 152], [188, 64, 37], [81, 32, 213], [98, 70, 241], [158, 126, 60], [123, 189, 179], [126, 206, 194], [75, 200, 156], [114, 111, 255], [89, 134, 145], [241, 61, 160], [200, 242, 244], [249, 141, 200], [134, 109, 131], [132, 129, 154], [210, 190, 80], [137, 250, 28], [152, 235, 96], [4, 120, 174], [152, 159, 84], [92, 6, 247], [157, 175, 167], [219, 24, 114], [122, 90, 132], [192, 85, 183], [251, 66, 132], [69, 99, 63], [118, 189, 65], [205, 77, 154], [189, 130, 203], [26, 171, 14], [54, 219, 62], [145, 254, 240], [120, 6, 125], [117, 91, 215], [17, 7, 98], [59, 112, 90], [216, 10, 221], [102, 148, 116], [166, 23, 179], [234, 182, 196], [172, 202, 37], [76, 63, 215], [147, 168, 251], [84, 112, 39], [7, 138, 93], [165, 247, 101], [133, 167, 97], [130, 45, 122], [115, 200, 188], [250, 94, 15], [166, 85, 42], [170, 45, 39], [19, 121, 214], [181, 227, 217], [179, 243, 3], [203, 60, 21], [22, 197, 33], [103, 224, 82], [20, 70, 145], [244, 16, 79], [22, 166, 155], [58, 82, 252], [181, 10, 134], [205, 5, 238], [74, 117, 181], [114, 57, 29], [139, 71, 8], [126, 221, 148], [8, 140, 130], [102, 237, 217], [235, 38, 240], [155, 152, 173], [44, 16, 230], [37, 53, 186], [171, 236, 237], [73, 58, 213], [0, 8, 111], [132, 25, 31], [49, 252, 224], [158, 112, 128], [52, 232, 136], [83, 136, 118], [251, 219, 31], [151, 18, 197], [94, 142, 123], [181, 114, 2], [221, 121, 83], [153, 24, 17], [85, 24, 11], [112, 188, 112], [216, 48, 42], [52, 247, 212], [53, 237, 69], [222, 47, 116], [15, 248, 86], [117, 81, 153], [22, 233, 0], [31, 130, 114], [90, 158, 179], [157, 80, 74], [132, 74, 7], [244, 174, 14], [63, 70, 54], [188, 200, 105], [35, 169, 161], [41, 202, 94], [70, 171, 213], [111, 219, 193], [25, 239, 36], [19, 224, 30], [27, 122, 213], [176, 149, 38], [4, 158, 252], [102, 115, 207], [37, 174, 159], [89, 123, 57], [198, 144, 182], [190, 69, 214], [136, 194, 51], [18, 214, 209], [128, 78, 230], [196, 8, 6], [60, 50, 34], [65, 140, 61], [78, 219, 35], [38, 77, 75], [42, 120, 208], [58, 8, 143], [223, 209, 72], [123, 169, 250], [176, 25, 187], [74, 145, 207], [33, 200, 33], [8, 188, 200], [116, 69, 14], [39, 39, 92], [99, 199, 230], [64, 126, 23], [187, 168, 192], [190, 79, 92], [78, 186, 158], [41, 39, 245], [119, 143, 9], [129, 130, 204], [206, 229, 191], [149, 233, 172], [242, 74, 141], [96, 184, 171], [17, 116, 92], [239, 21, 239], [213, 162, 63], [121, 56, 140], [123, 7, 184], [38, 137, 183], [146, 180, 54], [213, 207, 85], [46, 16, 75], [4, 23, 150], [120, 186, 194], [172, 5, 70], [6, 123, 240], [234, 101, 185], [78, 169, 56], [32, 70, 8], [143, 152, 53], [30, 55, 17], [146, 56, 126], [159, 81, 168], [231, 62, 181], [68, 212, 185], [207, 122, 136], [80, 107, 249], [67, 208, 250], [114, 229, 171], [54, 108, 227], [80, 184, 47], [4, 255, 247], [248, 88, 28], [124, 243, 36], [218, 152, 173], [155, 187, 145], [8, 109, 179], [145, 122, 173], [19, 70, 240], [57, 88, 249], [37, 55, 242], [134, 33, 1], [43, 21, 133], [171, 251, 9], [123, 53, 80], [221, 158, 32], [113, 120, 0], [200, 82, 236], [94, 218, 27], [63, 116, 162], [180, 103, 170], [223, 177, 89], [21, 145, 38], [16, 1, 166], [174, 130, 175], [94, 252, 126], [197, 41, 154], [52, 197, 187], [129, 188, 229], [13, 143, 170], [235, 22, 131], [34, 8, 85], [255, 182, 171], [32, 93, 137], [164, 101, 243], [97, 1, 26], [96, 171, 61], [194, 51, 172], [251, 91, 41], [48, 233, 112], [223, 98, 71], [62, 119, 183], [132, 44, 48], [105, 149, 233], [246, 13, 83], [1, 160, 215], [182, 46, 1], [172, 226, 71], [238, 17, 87], [149, 227, 246], [61, 167, 253], [185, 121, 42], [151, 48, 226], [13, 176, 179], [6, 19, 44], [72, 14, 182], [139, 135, 84], [170, 20, 102], [31, 210, 171], [39, 82, 249], [188, 139, 9], [24, 207, 22], [105, 213, 247], [248, 101, 144], [118, 116, 41], [206, 246, 7], [106, 241, 39], [1, 140, 37], [255, 18, 48], [111, 126, 39], [229, 202, 52], [132, 173, 252], [160, 1, 66], [18, 220, 37], [40, 29, 93], [190, 254, 196], [183, 110, 22], [155, 129, 247], [138, 251, 142], [166, 231, 106], [109, 248, 114], [177, 204, 119], [247, 83, 249], [137, 216, 95], [110, 3, 63], [187, 153, 126], [117, 169, 31], [69, 245, 124], [110, 49, 88], [142, 154, 101], [37, 169, 143], [212, 64, 115], [147, 120, 172], [107, 47, 146], [125, 79, 10], [18, 218, 60], [90, 205, 100], [61, 83, 9], [45, 41, 201], [214, 24, 137], [218, 62, 39], [220, 78, 29], [30, 249, 99], [109, 244, 155], [195, 176, 234], [117, 212, 122], [162, 151, 152], [184, 192, 103], [49, 73, 224], [220, 216, 224], [114, 147, 197], [120, 40, 10], [62, 69, 119], [28, 0, 171], [175, 135, 149], [33, 166, 224], [153, 193, 24], [214, 249, 135], [227, 12, 145], [167, 120, 182], [29, 239, 70], [114, 26, 203], [253, 179, 153], [231, 44, 132], [181, 197, 137], [201, 111, 136], [220, 54, 41], [119, 173, 117], [147, 100, 73], [198, 74, 10], [30, 219, 222], [94, 144, 118], [228, 229, 143], [126, 101, 200], [6, 226, 108], [22, 190, 163], [24, 191, 250], [4, 109, 9], [130, 167, 33], [190, 173, 14], [128, 173, 148], [116, 92, 85], [65, 165, 167], [93, 191, 163], [185, 107, 157], [244, 191, 63], [17, 211, 136], [54, 22, 46], [43, 98, 179], [142, 178, 186], [53, 251, 140], [200, 248, 137], [86, 177, 35], [168, 39, 251], [10, 160, 54], [4, 172, 43], [129, 26, 184], [251, 75, 252], [125, 2, 37], [63, 30, 209], [166, 38, 53], [122, 246, 70], [227, 113, 135], [26, 16, 204], [196, 42, 221], [97, 180, 74], [151, 118, 90], [188, 100, 78], [186, 89, 207], [54, 84, 23], [223, 2, 118], [66, 213, 42], [139, 237, 233], [13, 167, 199], [167, 252, 81], [245, 5, 140], [6, 225, 86], [170, 197, 144], [16, 116, 116], [187, 134, 64], [146, 61, 94], [90, 219, 70], [213, 155, 189], [37, 54, 6], [220, 173, 171], [243, 169, 68], [233, 209, 196], [202, 160, 241], [40, 136, 219], [153, 225, 174], [99, 97, 185], [230, 159, 158], [94, 125, 108], [158, 9, 246], [167, 1, 121], [75, 15, 188], [20, 227, 220], [239, 246, 84], [208, 154, 106], [172, 38, 147], [162, 189, 19], [35, 213, 170], [221, 156, 79], [86, 171, 182], [8, 215, 171], [101, 93, 3], [191, 115, 203], [249, 103, 139], [153, 177, 39], [185, 245, 79], [30, 163, 113], [58, 252, 141], [211, 28, 138], [16, 18, 121], [56, 193, 27], [2, 13, 20], [4, 61, 111], [95, 98, 80], [248, 35, 245], [73, 246, 133], [217, 29, 39], [84, 4, 215], [169, 205, 211], [184, 184, 67], [118, 104, 26], [190, 198, 22], [14, 211, 214], [194, 122, 16], [16, 233, 250], [27, 123, 154], [46, 190, 229], [7, 45, 113], [33, 199, 169], [50, 219, 97], [151, 13, 171], [203, 133, 53], [164, 129, 202], [47, 203, 55], [243, 31, 147], [53, 157, 79], [241, 245, 175], [231, 210, 102], [210, 132, 17], [141, 106, 84], [173, 173, 40], [201, 188, 151], [175, 49, 113], [165, 42, 237], [82, 208, 54], [239, 73, 227], [137, 232, 141], [224, 107, 38], [178, 19, 7], [98, 27, 42], [152, 187, 127], [49, 10, 87], [18, 205, 191], [187, 63, 86], [99, 164, 98], [211, 124, 102], [216, 86, 138], [197, 135, 0], [62, 99, 40], [158, 128, 114], [225, 58, 121], [220, 224, 162], [213, 209, 225], [52, 220, 24], [201, 74, 196], [116, 121, 53], [100, 56, 61], [239, 196, 168], [73, 143, 78], [141, 89, 173], [209, 136, 95], [21, 71, 117], [159, 155, 185], [72, 64, 254], [90, 40, 81], [128, 184, 150], [151, 119, 120], [122, 191, 8], [119, 114, 22], [168, 222, 233], [139, 224, 240], [113, 35, 57], [15, 6, 223], [220, 70, 35], [117, 102, 192], [190, 128, 248], [161, 184, 39], [53, 196, 144], [93, 69, 73], [5, 52, 178], [108, 21, 59], [207, 249, 182], [116, 152, 189], [51, 52, 79], [229, 230, 183], [39, 125, 91], [13, 88, 175], [92, 80, 90], [93, 90, 213], [10, 37, 96], [189, 113, 176], [160, 209, 242], [106, 18, 11], [162, 81, 28], [48, 19, 219], [212, 187, 20], [87, 72, 87], [250, 92, 92], [105, 8, 61], [40, 15, 158], [113, 195, 83], [94, 133, 8], [38, 99, 128], [139, 5, 193], [163, 40, 15], [175, 46, 193], [216, 239, 110], [14, 211, 106], [13, 19, 216], [111, 24, 9], [114, 38, 240], [61, 11, 193], [37, 243, 204], [122, 236, 87], [213, 141, 85], [137, 154, 0], [170, 233, 135], [94, 48, 216], [218, 157, 166], [166, 8, 128], [153, 255, 180], [207, 142, 254], [87, 209, 176], [222, 149, 4], [189, 153, 121], [101, 1, 195], [47, 240, 76], [83, 7, 123], [84, 215, 227], [74, 85, 117], [148, 1, 124], [221, 234, 244], [52, 234, 228], [105, 220, 1], [175, 84, 155], [42, 2, 41], [75, 171, 147], [37, 64, 178], [141, 23, 222], [218, 231, 177], [246, 125, 249], [20, 235, 132], [217, 93, 167], [188, 171, 4], [118, 132, 114], [137, 108, 60], [76, 197, 181], [151, 211, 84], [11, 26, 82], [79, 234, 164], [6, 252, 247], [124, 164, 164], [242, 69, 141], [71, 20, 7], [21, 10, 112], [191, 148, 135], [41, 205, 73], [195, 183, 239], [224, 144, 146], [237, 97, 231], [0, 234, 232], [15, 179, 196], [230, 167, 228], [134, 14, 153], [2, 176, 158], [69, 211, 158], [143, 79, 156], [169, 207, 104], [8, 80, 130], [24, 182, 22], [57, 40, 193], [183, 57, 207], [47, 179, 141], [43, 100, 13], [233, 215, 148], [24, 108, 94], [64, 179, 3], [113, 157, 214], [237, 208, 88], [45, 123, 13], [126, 189, 198], [150, 195, 61], [37, 36, 182], [39, 128, 244], [100, 215, 186], [122, 70, 1], [87, 140, 39], [159, 193, 231], [100, 248, 178], [33, 123, 147], [224, 115, 252], [231, 182, 102], [199, 115, 53], [12, 44, 143], [106, 45, 85], [230, 167, 179], [52, 244, 221], [112, 13, 126], [93, 241, 68], [51, 107, 229], [71, 233, 138], [123, 76, 128], [239, 212, 31], [251, 231, 30], [189, 127, 22], [168, 10, 229], [211, 177, 41], [201, 54, 98], [20, 27, 219], [220, 97, 6], [214, 186, 168], [122, 100, 132], [64, 143, 141], [139, 23, 20], [160, 2, 191], [147, 109, 167], [9, 85, 151], [107, 69, 180], [168, 204, 252], [67, 184, 6], [22, 200, 9], [174, 247, 81], [222, 238, 56], [24, 172, 178], [223, 217, 155], [164, 246, 163], [51, 124, 159], [156, 183, 241], [92, 208, 116], [187, 196, 98], [119, 154, 72], [29, 139, 117], [9, 103, 215], [130, 59, 81], [119, 27, 126], [211, 179, 50], [107, 238, 243], [93, 108, 208], [79, 53, 147], [221, 131, 55], [128, 149, 245], [90, 129, 167], [220, 238, 246], [129, 112, 232], [144, 83, 112], [54, 5, 117], [16, 41, 180], [161, 161, 105], [86, 87, 215], [209, 166, 31], [199, 18, 54], [192, 69, 108], [247, 129, 86], [45, 117, 176], [58, 211, 60], [131, 62, 130], [41, 172, 201], [118, 3, 213], [44, 23, 140], [151, 95, 161], [188, 136, 129], [154, 254, 128], [211, 133, 26], [240, 149, 158], [219, 242, 62], [234, 119, 9], [96, 84, 167], [87, 233, 190], [1, 54, 120], [31, 254, 112], [167, 111, 190], [72, 64, 50], [119, 21, 131], [162, 224, 51], [36, 159, 8], [218, 202, 9], [132, 14, 146], [48, 117, 203], [243, 69, 160], [104, 111, 122], [157, 20, 236], [118, 39, 28], [88, 191, 56], [138, 255, 71], [150, 170, 182], [110, 158, 160], [93, 7, 69], [217, 228, 111], [139, 213, 189], [63, 170, 10], [42, 117, 194], [235, 226, 129], [250, 164, 123], [93, 251, 53], [121, 14, 118], [221, 222, 133], [150, 23, 90], [100, 202, 118], [142, 149, 104], [90, 164, 14], [74, 227, 238], [43, 208, 164], [242, 58, 133], [126, 165, 208], [226, 95, 178], [1, 48, 100], [85, 177, 44], [249, 100, 6], [85, 183, 157], [191, 206, 210], [100, 46, 250], [72, 212, 122], [118, 112, 205], [122, 143, 132], [2, 3, 222], [196, 23, 255], [104, 159, 41], [71, 6, 38], [223, 221, 64], [132, 157, 100], [204, 78, 50], [133, 111, 188], [140, 123, 182], [25, 236, 170], [225, 189, 181], [51, 27, 210], [128, 162, 26], [224, 128, 25], [189, 133, 152], [172, 150, 31], [196, 165, 130], [62, 58, 133], [169, 3, 222], [107, 118, 96], [56, 173, 153], [142, 202, 31], [130, 201, 83], [138, 243, 198], [106, 191, 148], [51, 124, 159], [154, 33, 181], [202, 14, 7], [184, 118, 232], [93, 53, 184], [219, 225, 47], [34, 8, 210], [91, 157, 187], [161, 89, 11], [142, 33, 1], [32, 126, 30], [224, 161, 177], [124, 211, 14], [131, 21, 42], [179, 176, 156], [110, 217, 12], [152, 222, 218], [80, 99, 249], [104, 114, 62], [73, 214, 53], [33, 155, 127], [175, 4, 206], [215, 67, 222], [249, 219, 174], [20, 25, 52], [83, 205, 73], [183, 55, 218], [210, 213, 136], [34, 4, 109], [140, 12, 200], [137, 4, 100], [72, 206, 192], [148, 95, 7], [101, 33, 9], [98, 242, 126], [250, 10, 176], [194, 217, 16], [112, 47, 159], [233, 76, 47], [252, 215, 243], [176, 53, 133], [254, 18, 155], [54, 187, 254], [2, 99, 84], [27, 126, 198], [36, 250, 244], [175, 14, 85], [60, 114, 131], [8, 250, 73], [113, 240, 123], [144, 18, 26], [148, 12, 43], [125, 5, 34], [238, 57, 36], [203, 101, 93], [153, 216, 147], [179, 216, 183], [151, 89, 46], [158, 209, 7], [221, 37, 10], [16, 117, 26], [33, 92, 192], [114, 200, 53], [130, 209, 120], [119, 130, 244], [28, 66, 189], [26, 171, 33], [185, 90, 74], [94, 40, 236], [50, 230, 247], [168, 147, 199], [221, 206, 245], [241, 224, 200], [223, 248, 142], [42, 80, 12], [89, 157, 215], [165, 27, 129], [28, 120, 150], [202, 48, 102], [135, 33, 152], [110, 39, 166], [187, 242, 149], [170, 182, 72], [176, 129, 179], [120, 129, 58], [61, 226, 1], [217, 152, 23], [189, 229, 242], [174, 59, 245], [77, 22, 253], [179, 90, 182], [1, 98, 127], [109, 219, 174], [156, 39, 246], [145, 170, 109], [139, 87, 166], [156, 160, 78], [2, 174, 200], [22, 81, 4], [3, 114, 149], [152, 65, 208], [87, 96, 230], [216, 140, 48], [113, 21, 152], [243, 105, 110], [36, 100, 123], [188, 204, 224], [60, 131, 72], [91, 29, 28], [28, 225, 127], [140, 118, 103], [98, 127, 61], [148, 63, 244], [159, 89, 253], [38, 32, 53], [83, 161, 55], [241, 170, 194], [158, 91, 183], [43, 47, 30], [16, 11, 18], [73, 61, 177], [99, 88, 214], [98, 98, 208], [212, 244, 12], [181, 141, 90], [25, 55, 218], [59, 36, 139], [181, 248, 166], [206, 74, 227], [86, 153, 104], [63, 46, 206], [60, 190, 50], [118, 30, 98], [50, 231, 121], [136, 132, 177], [144, 32, 15], [64, 77, 58], [16, 156, 118], [143, 151, 245], [152, 71, 57], [201, 42, 24], [77, 34, 2], [13, 87, 237], [122, 221, 97], [65, 255, 120], [116, 74, 166], [229, 180, 10], [145, 196, 129], [0, 62, 42], [33, 110, 216], [55, 96, 103], [128, 113, 166], [175, 37, 206], [21, 255, 203], [120, 21, 109], [112, 94, 99], [144, 229, 130], [83, 40, 235], [136, 96, 49], [76, 158, 155], [26, 106, 208], [120, 168, 220], [74, 166, 113], [59, 64, 192], [101, 104, 85], [150, 86, 239], [24, 215, 232], [131, 209, 164], [76, 3, 71], [45, 161, 24], [206, 181, 175], [227, 19, 56], [236, 182, 243], [198, 110, 135], [108, 208, 37], [163, 99, 28], [9, 60, 166], [88, 69, 46], [232, 82, 242], [176, 176, 133], [139, 83, 40], [35, 100, 154], [113, 41, 10], [20, 104, 131], [61, 155, 155], [47, 236, 95], [99, 45, 182], [69, 243, 176], [29, 253, 132], [220, 111, 11], [149, 5, 91], [43, 176, 100], [39, 23, 236], [215, 52, 159], [206, 69, 86], [64, 37, 3], [147, 180, 253], [203, 170, 229], [43, 175, 159], [65, 238, 173], [208, 39, 10], [139, 41, 49], [80, 6, 138], [25, 0, 127], [155, 125, 64], [189, 42, 125], [120, 87, 69], [213, 70, 110], [145, 222, 142], [65, 40, 207], [0, 45, 131], [232, 167, 254], [241, 39, 92], [199, 95, 98], [253, 245, 0], [173, 81, 186], [212, 64, 8], [181, 186, 44], [32, 169, 50], [255, 173, 232], [148, 98, 237], [56, 151, 3], [61, 49, 10], [84, 172, 176], [123, 132, 224], [149, 62, 227], [223, 154, 217], [65, 172, 126], [15, 37, 250], [107, 219, 17], [70, 28, 120], [201, 65, 190], [105, 227, 94], [158, 196, 202], [229, 101, 86], [85, 144, 254], [13, 153, 38], [138, 199, 250], [239, 159, 197], [205, 92, 27], [250, 25, 196], [139, 12, 218], [133, 88, 102], [212, 168, 150], [93, 160, 41], [80, 143, 144], [125, 97, 246], [190, 67, 229], [83, 29, 197], [249, 76, 1], [173, 173, 23], [97, 36, 120], [1, 55, 164], [177, 98, 7], [15, 31, 104], [124, 90, 155], [142, 92, 71], [208, 167, 243], [74, 39, 78], [12, 190, 218], [243, 213, 239], [243, 251, 71], [88, 42, 140], [252, 17, 204], [199, 12, 147], [167, 143, 124], [0, 140, 163], [137, 39, 166], [152, 227, 112], [230, 26, 35], [76, 123, 112], [219, 3, 80], [25, 241, 42], [237, 223, 46], [5, 143, 55], [223, 27, 103], [219, 84, 243], [112, 235, 198], [164, 177, 255], [67, 126, 193], [45, 121, 247], [194, 11, 6], [198, 142, 86], [66, 14, 195], [160, 235, 27], [197, 13, 107], [178, 52, 137], [101, 244, 126], [14, 57, 6], [128, 9, 41], [175, 23, 55], [228, 46, 12], [181, 255, 150], [180, 142, 55], [132, 115, 70], [231, 223, 207], [75, 20, 251], [72, 98, 127], [30, 168, 93], [126, 177, 49], [140, 134, 174], [254, 155, 173], [88, 39, 203], [132, 196, 145], [180, 238, 255], [86, 244, 99], [250, 226, 188], [175, 50, 203], [39, 66, 131], [238, 68, 243], [44, 32, 198], [220, 2, 177], [28, 135, 103], [227, 4, 105], [225, 158, 195], [70, 20, 182], [114, 98, 120], [218, 67, 125], [216, 65, 224], [253, 35, 39], [149, 198, 67], [72, 68, 194], [209, 215, 194], [211, 145, 69], [156, 164, 199], [67, 227, 233], [179, 42, 86], [25, 98, 60], [183, 67, 240], [159, 212, 197], [3, 71, 158], [200, 236, 32], [121, 109, 246], [89, 53, 164], [166, 141, 107], [244, 51, 64], [150, 227, 141], [164, 215, 169], [59, 220, 250], [11, 99, 123], [215, 202, 239], [146, 145, 94], [85, 55, 147], [50, 140, 150], [147, 119, 16], [72, 22, 26], [226, 137, 43], [180, 98, 186], [151, 152, 249], [201, 138, 203], [89, 61, 187], [190, 85, 24], [249, 17, 81], [147, 185, 17], [205, 57, 239], [58, 198, 93], [251, 42, 15], [219, 154, 50], [93, 6, 211], [107, 95, 43], [190, 165, 177], [32, 159, 167], [71, 57, 6], [174, 211, 55], [23, 25, 155], [91, 217, 233], [240, 97, 205], [125, 95, 166], [155, 5, 221], [19, 128, 178], [101, 196, 208], [98, 55, 35], [8, 178, 174], [148, 169, 35], [1, 37, 28], [47, 37, 12], [131, 166, 205], [242, 173, 145], [243, 204, 11], [240, 104, 6], [200, 241, 228], [10, 66, 41], [40, 89, 243], [148, 180, 171], [121, 241, 152], [172, 169, 74], [220, 197, 45], [129, 255, 247], [243, 234, 203], [203, 210, 104], [108, 196, 5], [240, 3, 159], [36, 92, 50], [230, 141, 223], [122, 43, 21], [178, 96, 242], [37, 7, 101], [5, 68, 97], [180, 104, 89], [157, 129, 175], [130, 23, 48], [238, 206, 64], [250, 193, 195], [49, 219, 172], [170, 129, 223], [229, 93, 216], [71, 77, 178], [172, 89, 26], [12, 143, 84], [235, 165, 221], [107, 223, 193], [138, 25, 203], [173, 78, 15], [36, 124, 44], [251, 237, 168], [138, 75, 159], [104, 99, 45], [159, 187, 171], [114, 82, 168], [192, 26, 165], [79, 143, 61], [125, 239, 226], [29, 142, 244], [245, 187, 17], [246, 173, 198], [107, 224, 121], [195, 200, 71], [112, 244, 144], [135, 55, 243], [212, 68, 67], [154, 35, 136], [230, 246, 94], [108, 127, 37], [48, 20, 134], [177, 34, 170], [90, 155, 219], [181, 208, 179], [141, 104, 189], [231, 113, 37], [83, 61, 169], [153, 254, 5], [242, 54, 76], [104, 52, 0], [108, 72, 220], [133, 61, 146], [237, 21, 27], [204, 46, 237], [88, 206, 127], [39, 246, 105], [215, 128, 188], [137, 182, 246], [106, 223, 174], [75, 250, 150], [176, 106, 120], [84, 136, 90], [164, 172, 164], [87, 92, 139], [207, 245, 64], [147, 102, 2], [66, 188, 111], [10, 213, 207], [117, 9, 119], [151, 101, 22], [232, 172, 240], [192, 10, 191], [251, 107, 148], [215, 236, 18], [207, 129, 19], [80, 161, 25], [218, 52, 103], [42, 134, 57], [183, 153, 183], [94, 72, 216], [206, 3, 214], [208, 0, 203], [16, 148, 73], [139, 230, 66], [237, 188, 45], [21, 190, 28], [101, 179, 114], [115, 7, 233], [59, 10, 159], [216, 3, 72], [160, 208, 156], [153, 220, 168], [62, 67, 105], [179, 37, 159], [113, 81, 37], [177, 188, 53], [157, 118, 0], [79, 144, 122], [170, 177, 130], [124, 150, 43], [183, 33, 141], [86, 52, 241], [46, 193, 246], [226, 254, 1], [41, 69, 154], [96, 45, 22], [85, 7, 14], [132, 150, 239], [121, 191, 80], [141, 66, 237], [118, 27, 101], [104, 126, 48], [251, 161, 193], [43, 249, 105], [53, 74, 30], [183, 224, 135], [130, 107, 134], [141, 74, 37], [73, 202, 91], [68, 80, 103], [191, 113, 240], [118, 138, 135], [145, 27, 152], [1, 250, 208], [132, 74, 213], [161, 203, 211], [46, 189, 219], [70, 59, 174], [71, 53, 52], [7, 43, 184], [35, 195, 130], [57, 128, 165], [20, 10, 221], [245, 135, 246], [142, 69, 6], [237, 99, 87], [101, 196, 111], [129, 120, 239], [135, 25, 198], [173, 255, 139], [56, 194, 82], [246, 134, 110], [209, 173, 67], [106, 77, 74], [182, 20, 180], [6, 175, 108], [91, 100, 140], [157, 35, 92], [109, 187, 155], [110, 194, 0], [126, 76, 135], [189, 3, 132], [0, 255, 18], [114, 191, 218], [243, 224, 140], [37, 227, 157], [20, 202, 55], [6, 73, 42], [220, 35, 227], [229, 229, 96], [248, 42, 68], [224, 203, 237], [184, 146, 136], [53, 249, 63], [6, 194, 83], [3, 36, 217], [192, 226, 5], [41, 173, 8], [58, 118, 207], [22, 83, 39], [10, 212, 165], [61, 69, 107], [250, 48, 149], [238, 254, 163], [69, 89, 214], [94, 33, 220], [88, 69, 85], [35, 253, 74], [64, 39, 17], [133, 162, 218], [142, 182, 218], [221, 19, 228], [149, 210, 203], [232, 95, 213], [22, 205, 1], [4, 147, 67], [194, 126, 131], [244, 239, 168], [43, 19, 49], [54, 109, 131], [183, 225, 78], [210, 25, 250], [156, 108, 29], [217, 113, 240], [124, 164, 94], [75, 215, 251], [114, 193, 219], [24, 225, 117], [238, 205, 180], [160, 128, 111], [64, 149, 218], [2, 158, 218], [24, 90, 204], [58, 150, 101], [5, 125, 6], [50, 9, 232], [250, 204, 29], [209, 93, 47], [49, 165, 197], [86, 255, 139], [248, 43, 12], [31, 161, 243], [244, 202, 178], [220, 60, 218], [196, 42, 227], [169, 217, 1], [105, 202, 149], [139, 244, 162], [73, 207, 181], [25, 13, 189], [111, 221, 255], [58, 248, 146], [183, 50, 78], [229, 117, 151], [20, 184, 240], [55, 81, 202], [176, 56, 154], [14, 71, 126], [167, 123, 90], [109, 186, 232], [211, 193, 95], [249, 55, 184], [58, 89, 7], [217, 80, 191], [176, 29, 111], [163, 112, 17], [165, 61, 152], [199, 85, 120], [218, 180, 226], [145, 196, 182], [125, 186, 251], [32, 81, 164], [127, 104, 97], [89, 176, 180], [112, 71, 97], [225, 49, 122], [166, 95, 74], [229, 27, 104], [235, 194, 97], [178, 44, 231], [251, 57, 167], [53, 156, 123], [35, 209, 187], [53, 141, 161], [47, 76, 146], [16, 188, 133], [16, 41, 14], [143, 38, 143], [54, 172, 63], [13, 16, 114], [17, 114, 11], [204, 87, 245], [61, 244, 63], [155, 208, 159], [138, 69, 169], [193, 137, 173], [52, 26, 222], [42, 123, 79], [213, 146, 164], [175, 32, 59], [158, 253, 107], [173, 55, 243], [85, 143, 245], [125, 101, 92], [28, 82, 254], [45, 95, 20], [243, 155, 208], [107, 90, 21], [53, 154, 188], [41, 63, 25], [160, 19, 167], [5, 224, 241], [244, 169, 23], [195, 88, 78], [114, 87, 32], [64, 228, 242], [10, 241, 195], [116, 185, 105], [255, 116, 221], [54, 53, 78], [231, 67, 114], [81, 137, 81], [136, 136, 28], [189, 31, 117], [140, 72, 139], [214, 162, 240], [192, 64, 162], [234, 37, 24], [217, 86, 199], [194, 24, 42], [112, 79, 119], [191, 199, 56], [252, 215, 247], [155, 173, 151], [22, 167, 242], [27, 68, 113], [245, 110, 35], [198, 61, 74], [131, 165, 76], [170, 227, 123], [89, 122, 80], [114, 205, 21], [90, 209, 55], [23, 126, 230], [84, 52, 96], [130, 70, 180], [146, 234, 74], [229, 39, 11], [81, 33, 6], [248, 193, 46], [95, 29, 139], [50, 228, 60], [88, 218, 36], [8, 192, 110], [242, 17, 133], [239, 208, 141], [132, 218, 169], [141, 99, 40], [49, 228, 227], [207, 126, 13], [6, 215, 176], [117, 123, 15], [230, 5, 206], [227, 109, 50], [141, 212, 152], [244, 184, 132], [40, 27, 109], [44, 159, 13], [199, 85, 42], [127, 104, 28], [11, 222, 231], [99, 31, 39], [214, 40, 135], [140, 239, 160], [106, 143, 153], [227, 214, 116], [179, 47, 43], [88, 13, 152], [135, 16, 169], [8, 189, 189], [154, 251, 145], [27, 148, 30], [25, 69, 62], [20, 201, 189], [62, 32, 176], [154, 37, 162], [112, 193, 112], [240, 163, 212], [120, 249, 114], [254, 25, 186], [139, 119, 192], [100, 89, 85], [174, 217, 216], [230, 67, 32], [115, 94, 192], [82, 204, 83], [76, 164, 142], [169, 70, 111], [10, 121, 168], [83, 84, 122], [87, 127, 101], [228, 35, 103], [133, 183, 175], [132, 42, 196], [168, 1, 238], [2, 113, 83], [190, 212, 125], [91, 219, 77], [160, 186, 167], [147, 48, 222], [36, 63, 251], [198, 7, 91], [148, 107, 107], [178, 137, 24], [52, 18, 189], [240, 171, 80], [245, 106, 232], [43, 254, 180], [205, 38, 49], [8, 72, 125], [106, 188, 225], [75, 73, 143], [3, 95, 33], [72, 117, 78], [11, 150, 220], [220, 28, 157], [204, 193, 222], [27, 23, 0], [61, 238, 20], [53, 19, 187], [170, 118, 123], [43, 8, 201], [58, 125, 39], [81, 252, 57], [47, 215, 6], [174, 245, 134], [151, 4, 219], [216, 55, 70], [72, 224, 151], [228, 5, 28], [232, 215, 190], [12, 223, 195], [238, 144, 85], [72, 60, 224], [142, 179, 84], [78, 135, 92], [105, 19, 149], [45, 0, 145], [171, 240, 239], [62, 55, 214], [182, 46, 255], [240, 245, 114], [37, 176, 235], [247, 102, 120], [73, 202, 100], [179, 162, 81], [216, 220, 12], [215, 3, 154], [246, 109, 214], [222, 15, 87], [186, 15, 18], [218, 132, 2], [233, 163, 168], [88, 249, 115], [125, 249, 81], [223, 56, 198], [250, 41, 170], [56, 229, 74], [172, 55, 209], [217, 113, 140], [203, 35, 238], [152, 223, 79], [8, 117, 182], [155, 117, 215], [25, 194, 68], [231, 43, 69], [36, 74, 55], [199, 191, 93], [65, 140, 11], [41, 208, 117], [213, 209, 118], [248, 208, 17], [146, 192, 13], [247, 5, 1], [69, 74, 220], [225, 58, 126], [233, 151, 146], [201, 74, 102], [150, 56, 138], [2, 28, 4], [0, 244, 101], [240, 139, 56], [41, 177, 123], [152, 164, 222], [175, 45, 7], [62, 85, 175], [86, 30, 52], [107, 47, 222], [37, 234, 143], [58, 239, 203], [82, 162, 190], [222, 26, 42], [141, 51, 145], [126, 202, 21], [76, 171, 217], [185, 71, 229], [236, 187, 169], [10, 240, 218], [122, 115, 26], [32, 157, 30], [251, 222, 97], [130, 73, 184], [44, 151, 5], [156, 79, 120], [199, 122, 133], [197, 28, 32], [54, 98, 175], [239, 245, 111], [147, 53, 147], [150, 9, 64], [140, 119, 82], [228, 123, 218], [106, 159, 140], [138, 181, 120], [130, 251, 157], [95, 95, 233], [106, 7, 124], [222, 139, 21], [185, 153, 52], [40, 101, 119], [186, 51, 192], [86, 227, 157], [62, 14, 238], [168, 128, 147], [249, 88, 143], [69, 34, 216], [100, 130, 204], [175, 173, 76], [185, 152, 38], [88, 236, 153], [215, 55, 67], [24, 150, 13], [206, 126, 198], [207, 160, 105], [127, 42, 203], [13, 234, 231], [48, 146, 105], [230, 176, 93], [82, 77, 51], [82, 137, 207], [198, 103, 14], [169, 72, 192], [200, 184, 46], [129, 70, 206], [68, 236, 42], [211, 140, 20], [46, 146, 233], [253, 18, 68], [226, 222, 253], [179, 39, 58], [167, 7, 93], [54, 148, 144], [94, 99, 130], [9, 36, 47], [108, 249, 195], [116, 39, 83], [138, 68, 174], [78, 109, 167], [202, 37, 146], [138, 143, 35], [220, 79, 87], [62, 92, 29], [62, 224, 242], [37, 228, 52], [110, 12, 239], [154, 251, 21], [196, 142, 21], [219, 153, 72], [172, 178, 195], [80, 67, 207], [41, 206, 179], [92, 95, 77], [208, 58, 194], [200, 90, 42], [83, 84, 1], [140, 235, 159], [21, 6, 2], [235, 191, 237], [142, 218, 54], [185, 211, 37], [167, 43, 194], [214, 42, 240], [216, 160, 22], [181, 241, 69], [197, 164, 185], [180, 6, 224], [5, 32, 100], [218, 159, 51], [240, 140, 60], [63, 27, 74], [117, 58, 5], [167, 43, 139], [245, 180, 66], [171, 209, 101], [60, 139, 253], [69, 82, 18], [100, 163, 193], [179, 253, 203], [90, 188, 136], [22, 123, 251], [25, 140, 35], [140, 248, 219], [12, 163, 20], [89, 98, 82], [150, 134, 190], [196, 95, 15], [72, 114, 228], [105, 74, 241], [119, 106, 55], [215, 88, 40], [5, 62, 221], [221, 21, 44], [244, 60, 149], [143, 42, 191], [143, 53, 80], [195, 58, 175], [150, 204, 168], [117, 109, 192], [251, 69, 254], [34, 113, 4], [224, 77, 149], [26, 0, 225], [189, 140, 3], [226, 146, 163], [246, 87, 176], [12, 120, 140], [154, 196, 246], [76, 190, 34], [155, 121, 48], [240, 153, 254], [195, 168, 107], [177, 25, 42], [132, 59, 220], [165, 231, 109], [117, 38, 244], [1, 165, 51], [164, 107, 117], [45, 154, 82], [78, 123, 111], [204, 137, 15], [77, 140, 186], [51, 214, 125], [249, 156, 236], [186, 101, 198], [24, 129, 107], [84, 46, 53], [49, 130, 246], [99, 25, 227], [31, 178, 148], [128, 157, 102], [252, 21, 166], [73, 42, 200], [175, 135, 3], [109, 40, 157], [49, 191, 226], [108, 80, 211], [110, 11, 24], [34, 91, 89], [187, 125, 252], [197, 201, 214], [194, 202, 120], [35, 94, 211], [174, 212, 32], [52, 230, 188], [69, 74, 183], [17, 89, 243], [83, 101, 143], [149, 83, 30], [6, 116, 181], [170, 74, 232], [35, 222, 2], [241, 63, 119], [144, 177, 77], [14, 28, 246], [107, 249, 219], [52, 82, 21], [3, 83, 146], [202, 49, 215], [52, 117, 177], [230, 106, 13], [225, 193, 71], [118, 5, 25], [187, 32, 78], [22, 81, 187], [215, 120, 43], [171, 238, 94], [29, 164, 123], [159, 70, 192], [27, 217, 28], [109, 34, 115], [98, 183, 62], [17, 156, 53], [32, 141, 166], [211, 76, 108], [221, 49, 171], [243, 118, 130], [239, 247, 205], [252, 68, 103], [69, 226, 73], [119, 125, 221], [158, 171, 124], [241, 194, 13], [204, 182, 109], [30, 232, 181], [124, 210, 144], [183, 75, 236], [29, 253, 209], [213, 207, 31], [74, 136, 163], [43, 19, 91], [86, 85, 245], [59, 182, 232], [241, 221, 99], [46, 82, 240], [62, 102, 138], [174, 10, 142], [241, 217, 205], [25, 145, 251], [189, 81, 230], [228, 175, 80], [235, 88, 226], [247, 56, 150], [84, 228, 255], [96, 151, 55], [124, 31, 178], [31, 252, 254], [70, 122, 193], [55, 113, 224], [183, 160, 243], [165, 66, 27], [212, 26, 165], [7, 142, 48], [196, 218, 92], [57, 172, 157], [26, 72, 218], [224, 241, 130], [230, 190, 206], [119, 241, 78], [119, 245, 157], [13, 241, 92], [117, 70, 106], [64, 15, 148], [191, 158, 44], [96, 154, 162], [187, 8, 155], [157, 34, 253], [129, 110, 97], [75, 244, 72], [182, 166, 104], [93, 58, 84], [200, 96, 3], [85, 60, 68], [186, 136, 49], [77, 37, 57], [216, 69, 173], [132, 54, 167], [100, 138, 0], [8, 35, 151], [169, 63, 242], [232, 174, 78], [107, 124, 35], [97, 100, 52], [24, 209, 43], [150, 183, 15], [48, 192, 74], [212, 125, 251], [255, 29, 223], [56, 34, 73], [115, 6, 254], [252, 104, 139], [143, 95, 228], [230, 201, 123], [193, 195, 232], [43, 17, 123], [223, 175, 192], [53, 110, 98], [116, 34, 199], [168, 70, 148], [108, 164, 181], [181, 5, 218], [171, 66, 220], [108, 190, 234], [59, 174, 197], [59, 168, 63], [208, 99, 72], [234, 15, 76], [93, 123, 133], [133, 202, 61], [100, 183, 167], [140, 94, 243], [120, 161, 17], [44, 178, 120], [207, 197, 197], [134, 93, 70], [201, 215, 10], [253, 164, 237], [131, 251, 8], [59, 236, 6], [26, 124, 239], [216, 110, 82], [5, 93, 98], [53, 3, 109], [214, 56, 129], [217, 101, 226], [15, 16, 83], [7, 21, 74], [129, 75, 183], [130, 3, 54], [221, 252, 73], [192, 62, 151], [90, 159, 124], [143, 45, 18], [55, 12, 1], [77, 28, 88], [223, 186, 0], [79, 116, 153], [66, 22, 228], [189, 246, 45], [149, 152, 171], [54, 188, 1], [206, 227, 80], [103, 139, 69], [177, 127, 219], [146, 15, 174], [242, 73, 254], [97, 172, 71], [207, 204, 246], [171, 179, 86], [175, 13, 81], [246, 159, 237], [113, 8, 73], [207, 247, 172], [134, 74, 157], [179, 29, 73], [19, 100, 249], [113, 67, 254], [225, 251, 69], [117, 195, 54], [127, 114, 7], [56, 102, 239], [237, 168, 55], [194, 112, 219], [61, 20, 165], [147, 80, 175], [154, 28, 243], [57, 3, 208], [49, 181, 24], [33, 110, 134], [225, 15, 76], [194, 168, 32], [21, 228, 38], [253, 3, 86], [222, 73, 28], [177, 229, 149], [205, 79, 185], [171, 155, 87], [95, 190, 219], [184, 29, 89], [194, 188, 73], [196, 10, 170], [15, 249, 50], [152, 32, 183], [51, 228, 117], [125, 190, 0], [200, 184, 195], [40, 26, 225], [55, 55, 6], [80, 171, 33], [58, 67, 82], [92, 139, 181], [186, 177, 59], [218, 238, 99], [131, 187, 249], [122, 153, 134], [244, 28, 115], [38, 73, 226], [20, 166, 242], [110, 6, 107], [160, 135, 55], [8, 158, 247], [5, 69, 213], [244, 141, 173], [93, 36, 228], [207, 159, 94], [57, 61, 231], [203, 217, 117], [23, 25, 38], [235, 215, 251], [106, 214, 28], [185, 108, 8], [52, 46, 14], [113, 170, 181], [122, 122, 135], [224, 111, 25], [13, 114, 121], [69, 40, 16], [66, 227, 48], [176, 188, 111], [119, 224, 170], [118, 47, 193], [25, 132, 62], [137, 162, 233], [199, 111, 40], [110, 60, 195], [236, 58, 244], [86, 154, 140], [8, 238, 17], [71, 79, 61], [49, 61, 14], [245, 187, 165], [73, 154, 252], [64, 21, 219], [173, 102, 118], [112, 63, 101], [168, 253, 201], [75, 39, 227], [74, 151, 97], [192, 121, 253], [109, 148, 56], [78, 28, 43], [149, 130, 126], [243, 240, 216], [151, 42, 51], [247, 72, 193], [72, 46, 82], [36, 19, 6], [84, 50, 159], [245, 118, 230], [52, 119, 113], [168, 23, 6], [100, 63, 186], [131, 165, 108], [200, 138, 162], [116, 92, 185], [68, 183, 100], [175, 144, 113], [254, 131, 80], [73, 52, 3], [241, 148, 189], [40, 148, 84], [89, 206, 19], [3, 156, 104], [180, 29, 215], [169, 249, 129], [147, 205, 255], [88, 147, 20], [35, 239, 72], [178, 211, 239], [214, 9, 20], [187, 201, 163], [105, 47, 44], [10, 118, 222], [76, 199, 157], [68, 77, 0], [108, 223, 3], [220, 43, 56], [202, 98, 28], [186, 57, 140], [221, 200, 159], [63, 11, 105], [24, 180, 119], [13, 86, 29], [153, 214, 192], [109, 73, 107], [117, 139, 152], [112, 211, 37], [254, 102, 173], [220, 182, 189], [61, 169, 61], [209, 56, 147], [254, 108, 14], [227, 142, 160], [14, 253, 255], [142, 191, 48], [87, 17, 141], [202, 212, 193], [156, 21, 239], [117, 82, 250], [4, 254, 228], [137, 247, 172], [168, 150, 236], [27, 124, 176], [132, 95, 234], [218, 58, 36], [191, 224, 182], [243, 188, 222], [62, 197, 187], [55, 222, 2], [86, 110, 70], [91, 43, 224], [32, 243, 241], [75, 91, 185], [180, 68, 96], [84, 60, 122], [5, 137, 72], [137, 45, 38], [241, 137, 226], [93, 234, 16], [212, 129, 243], [204, 86, 40], [234, 242, 245], [203, 56, 167], [37, 52, 30], [41, 207, 159], [236, 75, 227], [199, 216, 16], [242, 139, 146], [239, 90, 149], [15, 107, 17], [129, 172, 136], [23, 57, 148], [190, 162, 44], [148, 72, 183], [142, 154, 180], [227, 121, 32], [85, 192, 25], [134, 27, 37], [88, 140, 74], [133, 131, 202], [228, 212, 84], [86, 237, 152], [218, 79, 74], [115, 38, 145], [76, 93, 232], [189, 192, 84], [41, 149, 32], [217, 9, 244], [150, 250, 49], [205, 26, 121], [33, 206, 134], [86, 49, 46], [236, 142, 87], [139, 191, 87], [254, 67, 164], [65, 19, 73], [137, 138, 139], [114, 20, 187], [71, 128, 75], [249, 192, 118], [197, 140, 126], [212, 167, 66], [203, 139, 13], [167, 173, 53], [166, 106, 197], [94, 134, 138], [206, 69, 6], [199, 36, 10], [169, 224, 220], [101, 164, 202], [136, 205, 43], [134, 75, 110], [57, 151, 177], [203, 128, 227], [188, 84, 172], [168, 213, 172], [40, 149, 128], [223, 1, 56], [104, 199, 89], [164, 242, 224], [234, 166, 207], [35, 41, 17], [100, 65, 153], [104, 41, 242], [229, 7, 54], [126, 52, 169], [251, 86, 106], [39, 94, 16], [248, 81, 181], [66, 159, 226], [221, 172, 70], [99, 162, 136], [195, 24, 19], [92, 3, 61], [179, 251, 80], [64, 166, 63], [224, 35, 68], [44, 36, 222], [57, 254, 232], [216, 53, 111], [226, 227, 11], [238, 142, 3], [215, 238, 238], [242, 118, 71], [239, 236, 50], [117, 209, 48], [162, 95, 77], [163, 115, 138], [133, 247, 51], [204, 193, 58], [90, 9, 244], [200, 36, 68], [113, 108, 62], [138, 152, 159], [92, 15, 51], [189, 82, 67], [45, 202, 27], [1, 116, 25], [81, 165, 194], [67, 216, 161], [185, 119, 186], [224, 113, 254], [161, 176, 182], [148, 248, 212], [22, 252, 122], [164, 240, 100], [254, 15, 202], [135, 76, 141], [31, 229, 191], [128, 128, 111], [67, 150, 121], [194, 98, 64], [248, 88, 122], [197, 70, 163], [116, 140, 64], [107, 255, 181], [40, 26, 180], [28, 39, 30], [90, 64, 219], [80, 217, 188], [65, 21, 168], [55, 251, 221], [54, 71, 236], [68, 38, 5], [40, 32, 30], [114, 218, 255], [207, 204, 219], [239, 12, 252], [94, 91, 234], [91, 184, 202], [54, 54, 190], [37, 73, 38], [143, 76, 185], [76, 52, 200], [23, 18, 59], [125, 176, 144], [158, 3, 147], [188, 253, 145], [171, 10, 161], [128, 133, 161], [250, 250, 24], [64, 51, 138], [95, 123, 230], [83, 44, 107], [107, 165, 126], [171, 140, 9], [58, 113, 61], [79, 87, 218], [109, 88, 247], [119, 209, 193], [37, 57, 157], [177, 187, 250], [52, 79, 33], [254, 195, 53], [169, 34, 220], [34, 33, 44], [240, 208, 105], [135, 157, 161], [8, 165, 63], [56, 11, 27], [97, 115, 205], [171, 143, 143], [55, 25, 9], [228, 82, 60], [91, 224, 243], [179, 45, 28], [231, 12, 153], [183, 223, 201], [26, 112, 116], [17, 120, 179], [95, 216, 209], [159, 96, 66], [7, 89, 50], [133, 141, 167], [230, 45, 97], [146, 14, 23], [93, 226, 146], [189, 62, 111], [231, 127, 173], [53, 227, 218], [45, 106, 175], [14, 118, 19], [2, 180, 239], [130, 164, 173], [44, 250, 80], [235, 60, 162], [251, 17, 28], [193, 234, 211], [32, 231, 116], [197, 143, 210], [226, 88, 232], [4, 136, 204], [196, 113, 209], [203, 190, 43], [188, 186, 164], [63, 77, 183], [140, 157, 193], [209, 183, 247], [7, 153, 185], [54, 192, 48], [29, 165, 3], [82, 131, 102], [92, 228, 6], [254, 126, 211], [200, 119, 167], [116, 181, 161], [221, 242, 191], [138, 139, 175], [41, 233, 47], [181, 6, 185], [177, 96, 152], [17, 24, 85], [120, 68, 138], [118, 103, 34], [28, 161, 245], [56, 89, 14], [55, 123, 57], [125, 27, 15], [102, 146, 225], [32, 198, 22], [157, 103, 247], [158, 202, 59], [18, 186, 222], [178, 198, 169], [216, 50, 102], [50, 228, 183], [248, 158, 72], [75, 138, 208], [97, 89, 102], [4, 255, 96], [212, 252, 112], [189, 185, 99], [37, 170, 241], [162, 45, 24], [245, 184, 224], [54, 248, 5], [46, 146, 227], [80, 19, 90], [106, 73, 51], [41, 107, 126], [209, 82, 230], [140, 210, 12], [214, 7, 59], [73, 82, 232], [201, 171, 112], [147, 231, 204], [54, 63, 14], [219, 178, 178], [193, 138, 31], [171, 211, 63], [164, 149, 219], [205, 42, 230], [115, 113, 244], [217, 233, 31], [42, 252, 78], [133, 107, 35], [255, 109, 195], [100, 242, 224], [46, 119, 238], [96, 140, 85], [143, 138, 56], [229, 230, 199], [199, 90, 15], [24, 221, 81], [200, 106, 177], [191, 184, 185], [164, 18, 240], [81, 141, 6], [41, 63, 198], [238, 34, 73], [140, 66, 203], [221, 113, 40], [162, 0, 91], [14, 191, 157], [54, 169, 178], [34, 114, 65], [111, 250, 41], [16, 135, 88], [231, 35, 16], [120, 252, 67], [163, 30, 216], [66, 76, 71], [177, 192, 75], [36, 196, 238], [167, 175, 178], [236, 220, 92], [93, 246, 143], [148, 99, 141], [238, 152, 214], [242, 95, 87], [211, 146, 121], [124, 81, 211], [203, 150, 245], [175, 122, 243], [50, 145, 39], [72, 253, 144], [172, 53, 195], [247, 180, 98], [226, 144, 233], [239, 9, 26], [198, 126, 176], [252, 147, 92], [158, 146, 97], [141, 189, 221], [191, 226, 128], [70, 156, 52], [138, 8, 174], [29, 31, 44], [235, 198, 138], [243, 175, 99], [50, 106, 181], [228, 55, 116], [231, 237, 110], [183, 167, 52], [189, 183, 92], [135, 235, 167], [216, 3, 141], [162, 131, 99], [245, 80, 181], [70, 178, 32], [54, 35, 178], [219, 203, 64], [18, 107, 168], [164, 16, 239], [25, 167, 136], [28, 183, 170], [152, 212, 38], [200, 247, 150], [45, 157, 171], [159, 56, 146], [100, 249, 56], [165, 21, 107], [136, 202, 142], [17, 73, 62], [29, 251, 199], [10, 104, 249], [23, 196, 143], [73, 69, 161], [163, 212, 46], [254, 80, 29], [13, 100, 76], [119, 55, 39], [56, 185, 130], [58, 93, 15], [62, 73, 215], [189, 143, 107], [66, 191, 116], [40, 31, 14], [74, 118, 184], [134, 227, 128], [113, 29, 169], [28, 58, 12], [191, 238, 50], [49, 220, 69], [197, 82, 18], [191, 5, 198], [130, 42, 250], [48, 112, 23], [49, 165, 35], [108, 201, 148], [227, 43, 214], [90, 5, 130], [64, 155, 151], [207, 216, 78], [210, 198, 118], [196, 157, 210], [140, 38, 42], [194, 243, 36], [55, 10, 93], [153, 26, 199], [156, 213, 118], [36, 14, 8], [150, 61, 116], [172, 104, 71], [255, 39, 85], [51, 36, 153], [146, 113, 189], [18, 237, 9], [136, 126, 118], [189, 248, 253], [59, 129, 247], [100, 37, 232], [214, 101, 159], [169, 70, 209], [124, 124, 119], [244, 85, 130], [251, 243, 159], [106, 194, 26], [167, 26, 169], [24, 49, 188], [132, 61, 175], [234, 216, 8], [18, 80, 252], [61, 129, 107], [61, 118, 215], [58, 22, 136], [132, 49, 185], [184, 71, 148], [232, 239, 135], [207, 105, 159], [173, 137, 88], [255, 225, 1], [78, 174, 46], [126, 148, 8], [87, 175, 72], [206, 131, 161], [173, 102, 4], [131, 152, 143], [65, 43, 220], [130, 36, 39], [76, 223, 199], [3, 178, 102], [29, 237, 173], [217, 66, 93], [97, 121, 104], [80, 50, 139], [172, 237, 16], [191, 49, 213], [25, 243, 137], [58, 74, 93], [222, 61, 246], [222, 242, 46], [219, 39, 200], [221, 230, 176], [243, 167, 18], [77, 182, 229], [169, 162, 227], [220, 73, 43], [25, 234, 140], [139, 232, 25], [133, 61, 203], [104, 53, 226], [166, 251, 178], [99, 117, 132], [198, 179, 249], [125, 91, 109], [235, 229, 144], [76, 140, 190], [19, 36, 204], [159, 98, 27], [66, 37, 225], [33, 139, 130], [191, 69, 245], [128, 221, 234], [224, 221, 254], [151, 25, 104], [120, 172, 90], [31, 54, 43], [106, 8, 121], [21, 117, 2], [211, 135, 212], [9, 19, 186], [112, 204, 245], [53, 65, 133], [9, 9, 233], [223, 191, 18], [2, 226, 137], [103, 81, 89], [107, 7, 28], [214, 184, 186], [47, 216, 32], [103, 247, 102], [178, 173, 166], [212, 153, 78], [30, 156, 83], [190, 190, 251], [55, 143, 152], [146, 204, 156], [86, 237, 149], [65, 118, 179], [114, 69, 184], [203, 105, 88], [147, 228, 81], [170, 246, 204], [146, 179, 222], [205, 47, 117], [49, 218, 213], [205, 86, 192], [199, 192, 122], [135, 11, 28], [48, 64, 229], [232, 166, 14], [157, 164, 131], [188, 16, 190], [134, 194, 206], [254, 75, 118], [14, 170, 78], [76, 77, 159], [183, 77, 170], [119, 246, 196], [53, 20, 251], [52, 170, 201], [120, 225, 12], [71, 200, 22], [124, 14, 73], [140, 198, 183], [53, 159, 153], [0, 1, 25], [155, 64, 90], [21, 211, 105], [12, 138, 13], [195, 55, 56], [203, 22, 85], [1, 212, 81], [39, 249, 106], [73, 89, 152], [191, 204, 213], [91, 156, 202], [229, 76, 111], [191, 218, 33], [117, 153, 151], [18, 94, 177], [243, 51, 87], [70, 215, 17], [94, 149, 87], [150, 118, 154], [167, 156, 164], [14, 72, 253], [127, 164, 239], [138, 51, 227], [157, 177, 155], [197, 105, 175], [44, 112, 96], [77, 89, 221], [184, 235, 155], [66, 49, 140], [181, 201, 233], [109, 206, 191], [122, 46, 233], [155, 50, 119], [61, 139, 16], [158, 27, 120], [0, 74, 227], [23, 145, 69], [7, 34, 65], [132, 114, 196], [159, 137, 225], [90, 12, 185], [100, 188, 196], [100, 188, 239], [189, 141, 124], [193, 245, 123], [177, 174, 57], [185, 77, 79], [92, 160, 125], [204, 250, 38], [175, 134, 245], [248, 4, 67], [248, 74, 223], [85, 71, 247], [177, 73, 86], [103, 86, 183], [44, 55, 27], [197, 97, 73], [146, 54, 18], [38, 215, 85], [105, 235, 87], [66, 195, 205], [64, 146, 43], [165, 158, 72], [167, 200, 110], [165, 112, 109], [177, 223, 109], [78, 177, 56], [19, 224, 236], [182, 183, 133], [4, 187, 74], [81, 150, 55], [215, 4, 108], [189, 228, 111], [62, 49, 234], [86, 63, 190], [55, 108, 7], [113, 4, 128], [85, 189, 60], [195, 116, 159], [134, 182, 154], [159, 79, 91], [208, 233, 218], [223, 164, 151], [244, 56, 38], [65, 183, 248], [173, 252, 212], [91, 141, 76], [203, 126, 22], [251, 233, 54], [248, 90, 134], [118, 249, 55], [89, 74, 19], [164, 140, 218], [216, 41, 10], [154, 51, 220], [145, 225, 35], [153, 174, 241], [235, 115, 73], [29, 136, 165], [101, 53, 152], [101, 139, 108], [75, 97, 121], [236, 4, 172], [229, 94, 150], [177, 185, 166], [6, 32, 8], [141, 32, 240], [29, 111, 211], [81, 238, 26], [96, 189, 26], [30, 144, 47], [211, 7, 109], [24, 252, 236], [238, 75, 255], [156, 101, 161], [94, 210, 188], [112, 70, 176], [13, 0, 202], [193, 100, 135], [243, 83, 33], [151, 15, 163], [100, 35, 91], [79, 201, 0], [20, 121, 63], [66, 71, 202], [126, 99, 243], [35, 191, 146], [88, 193, 159], [170, 234, 190], [175, 180, 16], [14, 20, 136], [184, 114, 248], [184, 4, 30], [51, 121, 142], [120, 137, 255], [155, 48, 150], [27, 42, 21], [8, 213, 50], [179, 139, 66], [108, 98, 4], [222, 172, 222], [52, 72, 19], [66, 210, 126], [72, 195, 222], [52, 117, 67], [28, 251, 95], [16, 31, 25], [83, 227, 23], [36, 232, 137], [184, 34, 5], [183, 3, 21], [182, 155, 93], [101, 145, 108], [111, 2, 235], [224, 126, 31], [168, 26, 199], [33, 190, 27], [134, 21, 127], [145, 7, 221], [12, 248, 114], [169, 246, 30], [138, 140, 128], [143, 54, 11], [212, 247, 162], [185, 69, 83], [190, 21, 157], [50, 61, 252], [178, 248, 233], [34, 236, 102], [98, 155, 232], [230, 135, 204], [54, 208, 174], [195, 138, 222], [77, 41, 172], [231, 74, 220], [29, 151, 197], [97, 156, 174], [248, 243, 254], [189, 2, 253], [108, 154, 93], [59, 78, 78], [52, 232, 164], [199, 162, 69], [249, 135, 244], [247, 193, 66], [216, 153, 74], [132, 121, 221], [14, 112, 208], [163, 208, 56], [125, 252, 110], [128, 228, 29], [149, 33, 41], [2, 53, 166], [255, 113, 251], [187, 241, 243], [172, 215, 100], [51, 202, 201], [197, 4, 155], [136, 79, 186], [127, 120, 161], [6, 206, 216], [68, 35, 175], [13, 16, 86], [22, 237, 208], [176, 136, 74], [253, 15, 71], [61, 49, 223], [252, 72, 80], [44, 184, 213], [46, 220, 112], [229, 205, 140], [85, 44, 202], [167, 8, 246], [139, 116, 87], [133, 78, 58], [127, 53, 155], [209, 126, 182], [65, 36, 32], [208, 142, 254], [5, 26, 231], [186, 146, 247], [78, 229, 60], [223, 91, 132], [202, 30, 129], [32, 109, 231], [54, 93, 181], [51, 62, 159], [147, 221, 14], [193, 163, 182], [218, 245, 108], [188, 240, 155], [76, 248, 154], [176, 33, 73], [185, 179, 137], [149, 237, 59], [67, 106, 170], [89, 46, 43], [77, 205, 102], [120, 252, 122], [151, 217, 14], [50, 33, 80], [188, 94, 180], [45, 160, 208], [183, 120, 126], [148, 1, 135], [84, 143, 206], [3, 148, 202], [213, 166, 137], [146, 211, 82], [176, 63, 28], [16, 219, 13], [52, 39, 193], [119, 0, 89], [101, 165, 116], [200, 122, 255], [228, 12, 188], [217, 24, 244], [66, 35, 110], [216, 244, 7], [159, 0, 121], [84, 177, 122], [239, 50, 121], [110, 203, 221], [135, 84, 117], [129, 242, 91], [153, 31, 204], [156, 92, 80], [222, 196, 139], [65, 203, 234], [134, 164, 54], [62, 219, 122], [108, 157, 188], [96, 37, 97], [244, 125, 132], [32, 130, 155], [144, 187, 90], [25, 206, 42], [236, 134, 56], [252, 79, 179], [1, 226, 230], [126, 134, 74], [112, 188, 29], [173, 210, 169], [254, 106, 219], [70, 247, 64], [143, 226, 16], [74, 201, 192], [178, 27, 191], [148, 73, 6], [15, 201, 208], [211, 107, 252], [47, 123, 24], [225, 43, 123], [71, 105, 201], [237, 67, 212], [121, 31, 219], [81, 60, 164], [47, 194, 120], [162, 19, 195], [166, 153, 71], [51, 176, 192], [145, 244, 20], [49, 183, 245], [220, 104, 44], [229, 247, 113], [254, 148, 104], [203, 69, 217], [161, 187, 253], [82, 31, 219], [173, 239, 237], [147, 161, 206], [207, 62, 122], [208, 154, 176], [198, 94, 63], [173, 121, 60], [78, 96, 105], [39, 107, 44], [125, 87, 114], [136, 214, 146], [144, 103, 126], [155, 170, 135], [197, 48, 112], [69, 247, 17], [238, 7, 253], [145, 12, 195], [195, 97, 143], [87, 35, 32], [113, 103, 68], [175, 193, 231], [191, 254, 183], [14, 185, 211], [197, 133, 1], [244, 23, 102], [43, 213, 146], [203, 34, 120], [3, 245, 42], [84, 9, 99], [28, 121, 72], [132, 168, 60], [59, 161, 215], [172, 143, 61], [6, 220, 90], [28, 47, 133], [206, 8, 186], [170, 10, 236], [105, 180, 231], [25, 42, 76], [238, 67, 116], [17, 180, 244], [14, 254, 125], [212, 46, 57], [246, 79, 177], [138, 52, 82], [220, 155, 85], [234, 93, 180], [208, 202, 40], [103, 89, 195], [191, 127, 195], [169, 124, 18], [46, 208, 208], [140, 234, 248], [115, 173, 135], [154, 171, 72], [254, 137, 78], [189, 252, 170], [94, 211, 140], [209, 170, 100], [4, 212, 135], [109, 175, 211], [154, 235, 47], [15, 113, 66], [121, 63, 138], [64, 192, 39], [119, 220, 136], [187, 181, 169], [91, 94, 171], [41, 145, 29], [225, 1, 75], [36, 60, 209], [80, 119, 230], [121, 84, 167], [164, 112, 178], [221, 120, 56], [233, 205, 87], [37, 80, 184], [79, 132, 140], [36, 33, 107], [73, 97, 128], [253, 97, 168], [58, 219, 148], [232, 186, 9], [142, 254, 7], [200, 153, 176], [11, 220, 31], [48, 205, 141], [25, 179, 198], [156, 111, 178], [95, 136, 112], [144, 219, 23], [250, 79, 162], [50, 129, 211], [147, 251, 137], [62, 171, 184], [138, 199, 91], [188, 154, 165], [164, 241, 197], [92, 49, 145], [12, 89, 61], [81, 20, 158], [163, 17, 116], [155, 192, 10], [188, 222, 143], [6, 173, 15], [122, 181, 181], [8, 192, 80], [180, 143, 50], [232, 9, 237], [133, 0, 57], [37, 148, 38], [103, 170, 5], [157, 108, 245], [106, 87, 39], [142, 249, 88], [135, 227, 124], [76, 92, 16], [32, 11, 12], [15, 193, 26], [34, 70, 41], [142, 37, 53], [120, 244, 31], [42, 166, 190], [245, 121, 200], [194, 201, 184], [74, 23, 205], [124, 248, 107], [30, 52, 156], [151, 244, 133], [210, 201, 127], [119, 45, 182], [78, 32, 178], [102, 60, 197], [4, 6, 227], [172, 17, 118], [250, 116, 112], [82, 95, 147], [75, 156, 194], [65, 47, 250], [115, 90, 235], [153, 226, 32], [210, 18, 53], [211, 242, 77], [107, 14, 7], [90, 106, 87], [23, 137, 57], [136, 228, 88], [102, 191, 115], [38, 49, 83], [144, 93, 24], [76, 54, 85], [40, 149, 75], [17, 72, 150], [113, 4, 150], [39, 91, 68], [247, 42, 156], [50, 139, 222], [251, 228, 150], [201, 80, 183], [243, 8, 255], [44, 61, 168], [70, 184, 217], [131, 198, 217], [114, 168, 39], [66, 148, 116], [98, 83, 171], [28, 248, 65], [76, 166, 69], [176, 20, 38], [254, 205, 114], [20, 54, 179], [186, 243, 69], [197, 115, 123], [130, 12, 83], [179, 8, 208], [124, 207, 163], [38, 211, 50], [246, 201, 235], [45, 14, 75], [36, 100, 172], [244, 252, 181], [18, 105, 170], [171, 2, 115], [1, 69, 161], [242, 137, 89], [172, 228, 102], [43, 172, 137], [72, 67, 158], [38, 251, 218], [101, 242, 98], [89, 244, 83], [240, 54, 102], [252, 224, 103], [101, 22, 21], [189, 4, 205], [35, 211, 224], [108, 29, 118], [161, 65, 170], [131, 203, 246], [254, 129, 239], [24, 116, 130], [196, 25, 43], [32, 29, 115], [133, 61, 105], [52, 229, 143], [184, 238, 216], [241, 226, 252], [235, 125, 128], [100, 124, 178], [130, 88, 204], [7, 153, 24], [217, 34, 134], [57, 98, 149], [72, 62, 106], [155, 91, 78], [223, 120, 52], [5, 39, 202], [193, 174, 71], [250, 210, 115], [134, 26, 53], [80, 123, 179], [108, 13, 199], [26, 147, 237], [245, 159, 2], [148, 166, 154], [205, 212, 62], [207, 194, 98], [182, 15, 223], [155, 199, 27], [14, 233, 112], [22, 204, 41], [53, 240, 40], [180, 153, 121], [142, 177, 123], [123, 35, 182], [220, 99, 165], [180, 149, 88], [81, 158, 19], [7, 43, 157], [35, 121, 39], [135, 135, 135], [184, 142, 112], [55, 105, 148], [100, 21, 1], [20, 95, 117], [91, 177, 247], [152, 84, 61], [181, 33, 118], [21, 139, 157], [42, 92, 2], [168, 243, 162], [15, 119, 69], [177, 3, 50], [44, 118, 113], [19, 38, 229], [50, 148, 254], [183, 238, 89], [46, 115, 216], [246, 105, 185], [2, 108, 165], [166, 59, 36], [2, 21, 245], [172, 142, 159], [38, 255, 226], [50, 59, 113], [141, 173, 47], [130, 195, 25], [20, 182, 181], [74, 190, 155], [76, 109, 38], [96, 174, 112], [7, 135, 126], [65, 6, 183], [142, 11, 3], [179, 180, 231], [3, 121, 248], [59, 155, 179], [23, 48, 52], [184, 153, 112], [240, 57, 13], [143, 24, 124], [182, 226, 254], [39, 107, 245], [154, 23, 71], [142, 20, 3], [116, 54, 211], [3, 99, 205], [122, 132, 253], [184, 122, 79], [197, 37, 195], [45, 136, 202], [242, 189, 244], [137, 231, 28], [19, 223, 69], [33, 108, 124], [38, 68, 216], [183, 90, 12], [206, 137, 60], [214, 211, 209], [125, 157, 148], [236, 129, 90], [16, 154, 203], [52, 185, 150], [71, 83, 204], [242, 195, 166], [46, 239, 62], [248, 117, 207], [156, 178, 202], [123, 143, 81], [16, 219, 41], [141, 34, 41], [195, 163, 170], [148, 60, 189], [207, 231, 222], [56, 214, 52], [120, 15, 243], [211, 17, 84], [38, 121, 199], [236, 31, 0], [168, 117, 31], [22, 93, 249], [211, 224, 91], [143, 141, 98], [93, 70, 227], [231, 50, 43], [110, 206, 214], [80, 6, 129], [212, 58, 73], [157, 100, 151], [193, 200, 196], [74, 49, 200], [209, 178, 16], [58, 83, 82], [70, 120, 202], [15, 37, 21], [127, 25, 201], [107, 125, 154], [148, 154, 219], [252, 178, 2], [103, 227, 172], [76, 220, 21], [114, 188, 217], [81, 137, 29], [66, 201, 102], [166, 138, 240], [165, 173, 248], [209, 163, 192], [239, 116, 220], [185, 228, 169], [69, 226, 240], [5, 3, 20], [96, 40, 16], [107, 203, 3], [93, 74, 101], [142, 252, 219], [161, 14, 113], [230, 226, 173], [123, 71, 71], [131, 59, 228], [1, 216, 77], [55, 64, 175], [58, 161, 255], [150, 228, 171], [175, 252, 88], [235, 144, 16], [70, 232, 76], [0, 212, 149], [28, 90, 172], [96, 141, 199], [58, 175, 101], [81, 107, 143], [113, 3, 177], [13, 189, 162], [201, 64, 227], [156, 58, 58], [183, 246, 12], [243, 99, 179], [119, 17, 254], [106, 193, 38], [178, 247, 227], [158, 52, 154], [122, 167, 128], [41, 138, 223], [125, 176, 237], [96, 225, 115], [186, 34, 95], [170, 153, 199], [104, 186, 89], [22, 98, 131], [182, 94, 136], [226, 106, 200], [184, 59, 18], [25, 108, 4], [72, 194, 82], [39, 1, 51], [61, 241, 93], [24, 211, 151], [81, 176, 82], [45, 32, 194], [227, 171, 240], [234, 236, 62], [15, 107, 117], [130, 254, 210], [112, 244, 168], [59, 76, 39], [189, 173, 134], [52, 252, 248], [228, 65, 61], [244, 245, 68], [90, 177, 67], [78, 195, 0], [76, 194, 128], [226, 241, 25], [236, 178, 147], [203, 139, 124], [235, 251, 97], [49, 72, 40], [170, 57, 188], [157, 188, 119], [233, 254, 85], [32, 223, 219], [158, 197, 18], [109, 218, 207], [159, 33, 146], [177, 59, 157], [139, 244, 143], [20, 85, 206], [251, 87, 123], [81, 56, 115], [140, 245, 244], [142, 120, 180], [244, 165, 117], [77, 129, 131], [7, 62, 88], [190, 213, 49], [215, 161, 205], [36, 113, 163], [55, 18, 225], [204, 110, 63], [232, 249, 217], [233, 216, 240], [18, 163, 48], [76, 50, 50], [189, 174, 88], [99, 10, 236], [242, 100, 23], [231, 55, 214], [45, 73, 33], [135, 159, 191], [213, 244, 13], [136, 235, 252], [75, 56, 55], [119, 49, 49], [202, 9, 139], [107, 100, 214], [69, 35, 179], [50, 50, 125], [70, 129, 8], [72, 63, 133], [132, 35, 42], [187, 90, 192], [83, 232, 94], [30, 134, 159], [122, 3, 51], [60, 243, 155], [231, 57, 71], [10, 101, 68], [139, 78, 203], [216, 144, 200], [106, 60, 138], [201, 165, 8], [26, 96, 71], [249, 197, 35], [219, 198, 39], [24, 59, 253], [161, 131, 96], [211, 29, 246], [196, 253, 255], [253, 38, 127], [95, 243, 125], [29, 7, 131], [127, 190, 187], [102, 65, 28], [17, 12, 30], [63, 206, 75], [180, 231, 91], [211, 198, 204], [62, 107, 45], [153, 210, 140], [174, 173, 186], [161, 197, 191], [89, 18, 98], [46, 94, 172], [191, 11, 99], [97, 101, 99], [48, 128, 223], [166, 127, 187], [169, 58, 209], [33, 9, 226], [216, 212, 184], [41, 161, 56], [107, 100, 97], [75, 223, 234], [138, 12, 96], [242, 40, 29], [130, 78, 193], [64, 64, 22], [43, 33, 186], [246, 242, 15], [147, 242, 185], [212, 56, 83], [142, 123, 88], [134, 235, 46], [185, 38, 143], [48, 160, 199], [204, 150, 247], [209, 213, 105], [92, 162, 126], [131, 154, 63], [170, 62, 19], [35, 170, 140], [249, 205, 43], [243, 194, 35], [50, 37, 159], [105, 57, 229], [103, 114, 237], [87, 157, 133], [161, 64, 25], [236, 99, 161], [63, 73, 99], [137, 126, 88], [28, 108, 154], [241, 241, 188], [241, 43, 82], [179, 74, 161], [155, 225, 220], [67, 12, 218], [118, 144, 26], [159, 94, 142], [54, 43, 250], [76, 111, 17], [135, 50, 48], [222, 248, 5], [77, 97, 179], [22, 232, 34], [83, 248, 46], [166, 132, 60], [60, 148, 141], [52, 192, 157], [27, 20, 238], [204, 143, 26], [89, 136, 136], [56, 214, 216], [211, 174, 118], [243, 188, 141], [173, 69, 187], [14, 174, 29], [31, 224, 180], [177, 80, 201], [34, 106, 86], [228, 56, 57], [42, 212, 159], [174, 176, 172], [251, 107, 211], [212, 158, 59], [66, 66, 139], [129, 225, 189], [41, 136, 148], [66, 188, 213], [86, 112, 178], [146, 158, 135], [116, 229, 193], [140, 84, 200], [222, 13, 134], [184, 18, 84], [85, 127, 72], [202, 173, 119], [67, 201, 255], [12, 110, 250], [128, 218, 199], [101, 10, 239], [146, 15, 75], [129, 64, 123], [191, 50, 16], [148, 45, 250], [36, 124, 163], [141, 208, 39], [3, 77, 99], [240, 166, 20], [105, 132, 133], [175, 166, 8], [225, 59, 107], [138, 218, 181], [48, 5, 149], [37, 156, 38], [199, 135, 198], [6, 127, 194], [1, 94, 148], [85, 217, 119], [245, 242, 38], [24, 127, 23], [128, 251, 127], [179, 178, 16], [163, 183, 185], [123, 193, 10], [33, 48, 51], [180, 65, 185], [207, 4, 99], [121, 60, 170], [61, 196, 186], [30, 185, 105], [48, 202, 193], [244, 4, 4], [45, 128, 57], [34, 146, 156], [102, 36, 174], [43, 68, 114], [22, 153, 80], [57, 216, 189], [21, 35, 79], [254, 11, 177], [109, 193, 98], [93, 162, 103], [186, 241, 253], [87, 213, 170], [107, 131, 40], [124, 208, 106], [64, 67, 79], [126, 133, 208], [190, 162, 135], [88, 74, 80], [159, 193, 216], [243, 37, 163], [241, 249, 122], [178, 207, 84], [47, 88, 68], [108, 146, 193], [14, 54, 255], [56, 19, 91], [17, 178, 52], [176, 235, 144], [205, 241, 85], [223, 31, 81], [33, 2, 240], [83, 217, 8], [15, 18, 167], [255, 214, 43], [40, 197, 118], [45, 112, 189], [128, 110, 83], [219, 225, 225], [67, 81, 32], [52, 216, 5], [176, 29, 75], [190, 230, 238], [63, 91, 133], [22, 145, 154], [161, 127, 120], [58, 74, 108], [192, 48, 57], [75, 220, 105], [88, 140, 224], [174, 188, 118], [100, 139, 145], [87, 28, 7], [197, 221, 149], [76, 70, 62], [159, 17, 83], [140, 58, 9], [75, 228, 216], [127, 65, 23], [230, 220, 147], [236, 132, 111], [64, 66, 180], [68, 85, 32], [57, 151, 216], [167, 124, 192], [14, 105, 110], [233, 54, 158], [221, 171, 23], [156, 36, 46], [2, 65, 58], [226, 144, 195], [14, 173, 249], [19, 245, 119], [61, 225, 160], [5, 108, 50], [252, 185, 105], [67, 27, 97], [122, 189, 127], [52, 58, 192], [52, 166, 112], [199, 24, 128], [2, 245, 76], [247, 194, 159], [190, 17, 139], [171, 98, 144], [116, 65, 8], [106, 60, 34], [71, 138, 220], [152, 212, 47], [127, 88, 79], [158, 130, 63], [81, 86, 172], [40, 215, 208], [227, 24, 64], [12, 72, 207], [50, 251, 45], [125, 249, 6], [196, 166, 114], [48, 77, 129], [76, 119, 196], [136, 159, 63], [108, 56, 20], [208, 8, 129], [134, 30, 206], [197, 230, 192], [16, 94, 174], [204, 235, 189], [198, 43, 50], [209, 171, 74], [53, 109, 115], [128, 155, 28], [188, 72, 52], [52, 236, 242], [239, 118, 49], [169, 129, 39], [38, 175, 139], [104, 181, 5], [250, 77, 168], [222, 143, 206], [85, 239, 11], [87, 17, 78], [178, 237, 171], [65, 187, 176], [90, 84, 3], [108, 222, 207], [77, 215, 230], [69, 238, 99], [240, 189, 34], [7, 174, 161], [196, 174, 93], [99, 119, 84], [68, 74, 164], [177, 123, 125], [235, 166, 75], [204, 178, 254], [18, 204, 15], [46, 22, 123], [198, 99, 7], [134, 222, 152], [189, 30, 74], [158, 120, 58], [144, 174, 90], [213, 203, 33], [102, 109, 132], [168, 59, 30], [19, 132, 76], [145, 219, 23], [58, 148, 5], [228, 135, 193], [88, 161, 125], [205, 95, 171], [194, 166, 42], [204, 98, 209], [192, 51, 223], [18, 204, 231], [108, 105, 17], [44, 184, 5], [63, 46, 78], [189, 39, 82], [251, 229, 55], [233, 132, 162], [99, 83, 161], [145, 238, 39], [221, 38, 142], [214, 197, 41], [248, 111, 190], [154, 124, 255], [5, 209, 121], [68, 80, 207], [214, 105, 251], [134, 118, 126], [132, 245, 190], [45, 115, 6], [254, 116, 190], [87, 26, 214], [19, 134, 81], [172, 64, 204], [23, 73, 91], [229, 234, 102], [221, 245, 44], [75, 155, 31], [241, 138, 225], [241, 94, 171], [169, 206, 121], [44, 7, 16], [141, 108, 98], [101, 104, 183], [223, 255, 243], [15, 170, 98], [106, 185, 188], [32, 231, 128], [77, 172, 49], [79, 73, 47], [161, 110, 102], [105, 214, 183], [151, 128, 236], [60, 160, 104], [48, 150, 35], [97, 58, 187], [64, 143, 42], [192, 86, 61], [75, 118, 63], [68, 236, 181], [231, 233, 56], [161, 95, 39], [107, 248, 97], [166, 250, 154], [29, 104, 249], [56, 44, 158], [0, 74, 200], [76, 90, 203], [138, 105, 213], [19, 122, 213], [135, 185, 138], [38, 138, 114], [231, 144, 61], [210, 244, 134], [132, 7, 198], [200, 135, 30], [52, 158, 252], [251, 249, 100], [231, 109, 140], [239, 10, 36], [219, 24, 214], [82, 111, 128], [144, 179, 151], [186, 147, 58], [89, 162, 254], [228, 226, 85], [103, 121, 216], [197, 250, 197], [84, 76, 232], [152, 43, 0], [11, 105, 38], [92, 202, 239], [238, 235, 245], [26, 226, 36], [85, 187, 144], [211, 134, 251], [226, 33, 177], [197, 177, 135], [5, 178, 225], [225, 83, 133], [132, 214, 229], [76, 114, 159], [169, 4, 23], [96, 153, 45], [44, 7, 126], [73, 19, 12], [118, 126, 237], [252, 75, 202], [240, 240, 0], [184, 114, 47], [181, 250, 177], [27, 177, 68], [34, 25, 65], [97, 83, 89], [7, 23, 31], [52, 28, 106], [152, 48, 135], [70, 223, 39], [124, 108, 244], [123, 201, 63], [68, 50, 80], [47, 187, 173], [22, 231, 167], [86, 140, 87], [32, 118, 104], [153, 38, 178], [146, 73, 64], [77, 47, 212], [178, 176, 110], [243, 114, 166], [186, 78, 122], [126, 181, 78], [57, 96, 214], [212, 222, 23], [208, 168, 53], [58, 155, 165], [69, 143, 195], [60, 161, 139], [214, 189, 49], [22, 200, 148], [15, 158, 136], [173, 55, 106], [72, 252, 51], [9, 239, 29], [184, 93, 220], [26, 56, 129], [41, 68, 102], [152, 117, 218], [170, 133, 89], [170, 5, 164], [204, 10, 38], [108, 140, 67], [185, 59, 132], [39, 130, 187], [109, 24, 3], [20, 0, 167], [87, 21, 13], [17, 238, 147], [32, 46, 89], [29, 137, 255], [113, 194, 166], [156, 40, 65], [42, 81, 143], [127, 90, 223], [77, 26, 190], [62, 208, 37], [143, 190, 40], [201, 226, 20], [60, 20, 249], [253, 91, 174], [45, 109, 121], [142, 143, 3], [185, 117, 189], [222, 60, 49], [171, 143, 101], [66, 159, 227], [179, 89, 6], [164, 208, 216], [34, 242, 80], [96, 141, 163], [27, 136, 194], [219, 112, 35], [50, 63, 228], [147, 107, 12], [170, 119, 65], [173, 150, 61], [174, 10, 251], [109, 177, 184], [29, 101, 184], [76, 220, 16], [51, 13, 204], [17, 238, 8], [185, 28, 248], [249, 151, 229], [151, 110, 210], [86, 104, 0], [113, 253, 139], [128, 6, 93], [180, 86, 39], [176, 74, 236], [191, 3, 20], [184, 13, 114], [14, 205, 8], [97, 232, 79], [114, 95, 118], [228, 42, 209], [227, 68, 115], [150, 82, 85], [250, 138, 201], [122, 148, 65], [172, 242, 226], [184, 101, 124], [231, 124, 216], [241, 124, 115], [242, 10, 108], [34, 82, 47], [96, 106, 16], [172, 135, 133], [44, 71, 85], [133, 89, 127], [150, 247, 89], [125, 207, 113], [133, 61, 6], [194, 54, 236], [185, 157, 124], [29, 29, 230], [139, 95, 182], [183, 18, 246], [191, 189, 209], [12, 113, 194], [228, 198, 243], [67, 196, 12], [82, 57, 131], [194, 146, 181], [124, 225, 115], [76, 237, 148], [182, 34, 125], [105, 11, 11], [175, 242, 235], [159, 1, 52], [213, 180, 45], [183, 182, 62], [209, 6, 166], [153, 166, 79], [155, 85, 6], [41, 196, 237], [99, 236, 38], [40, 251, 225], [105, 234, 116], [131, 0, 216], [142, 122, 211], [36, 216, 36], [246, 215, 10], [179, 114, 153], [76, 71, 108], [138, 173, 225], [134, 210, 201], [252, 13, 101], [241, 117, 65], [160, 161, 53], [241, 4, 31], [4, 26, 174], [35, 7, 89], [7, 61, 64], [53, 8, 138], [89, 37, 121], [119, 234, 30], [204, 59, 153], [12, 163, 70], [60, 181, 97], [174, 241, 157], [24, 108, 187], [238, 24, 181], [18, 100, 52], [33, 215, 150], [221, 14, 40], [233, 118, 203], [31, 109, 73], [51, 139, 182], [63, 97, 231], [129, 174, 234], [242, 145, 40], [32, 124, 150], [109, 200, 114], [181, 75, 33], [136, 209, 124], [49, 28, 184], [245, 209, 40], [6, 150, 179], [238, 54, 45], [194, 147, 68], [251, 240, 52], [111, 147, 38], [32, 126, 40], [186, 237, 187], [237, 98, 105], [176, 92, 97], [150, 16, 75], [142, 190, 189], [216, 184, 18], [150, 178, 166], [248, 199, 156], [19, 55, 11], [126, 79, 185], [11, 29, 199], [187, 125, 139], [18, 214, 61], [201, 86, 202], [178, 72, 183], [116, 194, 143], [74, 69, 139], [116, 110, 169], [203, 21, 46], [109, 34, 120], [92, 69, 91], [104, 190, 169], [173, 153, 27], [243, 127, 6], [51, 125, 138], [196, 143, 236], [185, 234, 95], [23, 245, 107], [217, 222, 103], [252, 213, 151], [74, 4, 181], [20, 156, 59], [6, 152, 183], [167, 85, 191], [226, 169, 200], [47, 3, 228], [97, 90, 70], [23, 177, 194], [228, 151, 23], [222, 36, 184], [88, 92, 214], [86, 152, 179], [245, 201, 174], [73, 135, 100], [77, 186, 180], [196, 156, 117], [123, 127, 92], [233, 177, 99], [102, 13, 11], [51, 74, 26], [157, 87, 68], [149, 196, 149], [155, 51, 217], [142, 228, 79], [43, 102, 120], [84, 217, 118], [56, 226, 84], [136, 33, 242], [105, 203, 235], [169, 115, 96], [120, 16, 62], [147, 255, 202], [210, 238, 148], [182, 136, 3], [169, 167, 56], [92, 174, 146], [240, 116, 214], [207, 84, 22], [242, 213, 172], [71, 147, 208], [73, 8, 223], [157, 244, 216], [223, 99, 224], [76, 219, 129], [37, 227, 67], [53, 192, 117], [194, 204, 79], [52, 25, 177], [90, 26, 243], [45, 115, 152], [141, 80, 77], [5, 241, 226], [116, 170, 201], [117, 225, 194], [123, 198, 10], [227, 251, 79], [27, 143, 39], [199, 171, 129], [231, 189, 179], [158, 236, 13], [222, 76, 165], [150, 85, 197], [37, 110, 87], [250, 66, 143], [173, 60, 76], [162, 49, 254], [129, 176, 190], [170, 225, 176], [147, 244, 38], [7, 43, 236], [246, 30, 54], [90, 119, 38], [112, 142, 80], [166, 36, 5], [9, 144, 201], [44, 132, 2], [212, 49, 9], [72, 193, 129], [170, 185, 198], [66, 75, 64], [189, 252, 159], [226, 7, 189], [183, 5, 188], [241, 30, 236], [253, 138, 88], [187, 183, 60], [178, 135, 93], [28, 29, 127], [68, 11, 124], [19, 204, 202], [28, 104, 142], [202, 160, 17], [138, 14, 136], [153, 141, 43], [198, 110, 162], [76, 95, 34], [97, 25, 122], [81, 16, 159], [111, 67, 89], [158, 64, 44], [209, 75, 32], [146, 203, 34], [241, 52, 87], [128, 130, 25], [237, 213, 194], [217, 147, 7], [25, 205, 114], [95, 230, 124], [66, 90, 106], [196, 132, 202], [35, 130, 120], [129, 11, 98], [79, 252, 139], [238, 200, 228], [78, 62, 163], [230, 147, 230], [123, 239, 228], [209, 227, 251], [252, 114, 7], [4, 75, 20], [179, 190, 116], [76, 82, 186], [244, 230, 163], [226, 142, 189], [90, 101, 46], [95, 40, 121], [244, 17, 206], [188, 107, 189], [129, 108, 140], [181, 99, 63], [79, 44, 251], [58, 149, 198], [14, 184, 76], [114, 187, 190], [246, 197, 89], [91, 151, 53], [113, 49, 197], [132, 38, 189], [223, 32, 240], [239, 139, 157], [120, 35, 234], [52, 36, 106], [204, 157, 20], [231, 3, 76], [251, 115, 38], [186, 130, 227], [111, 81, 244], [184, 251, 122], [235, 130, 255], [222, 33, 86], [77, 80, 208], [151, 136, 160], [96, 93, 6], [26, 74, 107], [72, 92, 253], [15, 246, 124], [32, 31, 239], [215, 174, 96], [253, 250, 186], [23, 60, 151], [154, 31, 164], [29, 7, 246], [29, 141, 99], [146, 138, 168], [130, 46, 156], [219, 52, 86], [61, 112, 66], [247, 198, 155], [165, 86, 174], [31, 220, 197], [8, 188, 9], [81, 71, 145], [213, 137, 236], [183, 141, 65], [216, 105, 224], [81, 102, 177], [130, 219, 52], [85, 242, 188], [40, 203, 215], [236, 154, 186], [191, 190, 23], [174, 131, 64], [175, 23, 188], [246, 195, 94], [131, 104, 170], [250, 56, 75], [227, 27, 60], [247, 136, 34], [161, 78, 232], [159, 230, 21], [230, 207, 27], [0, 161, 228], [12, 48, 185], [46, 226, 42], [20, 143, 13], [70, 221, 195], [112, 11, 224], [151, 9, 33], [156, 113, 136], [155, 25, 136], [205, 126, 150], [72, 251, 225], [173, 207, 201], [56, 246, 226], [173, 218, 178], [128, 210, 56], [168, 105, 197], [234, 95, 155], [14, 145, 159], [76, 94, 15], [251, 3, 253], [1, 101, 42], [205, 132, 202], [101, 149, 81], [61, 150, 23], [48, 108, 188], [173, 235, 6], [201, 208, 15], [2, 190, 51], [212, 21, 83], [195, 159, 191], [236, 63, 120], [123, 202, 9], [239, 119, 249], [213, 229, 181], [111, 98, 126], [45, 192, 91], [186, 90, 219], [162, 242, 6], [140, 28, 59], [216, 110, 172], [51, 37, 54], [175, 251, 46], [76, 81, 101], [229, 98, 232], [214, 119, 5], [111, 48, 115], [87, 186, 114], [24, 217, 245], [179, 2, 20], [133, 181, 99], [93, 85, 97], [45, 163, 204], [214, 137, 22], [184, 98, 185], [227, 20, 129], [225, 219, 100], [159, 208, 84], [52, 144, 221], [138, 53, 210], [200, 149, 249], [216, 37, 24], [125, 2, 47], [75, 44, 49], [49, 229, 80], [145, 10, 70], [5, 58, 16], [19, 105, 28], [171, 69, 72], [247, 143, 174], [143, 216, 168], [58, 228, 147], [3, 6, 88], [173, 190, 225], [12, 220, 68], [97, 202, 250], [190, 209, 151], [235, 134, 136], [108, 247, 91], [106, 234, 93], [204, 87, 122], [213, 60, 204], [161, 212, 81], [57, 215, 59], [125, 7, 237], [111, 72, 88], [246, 118, 119], [178, 120, 253], [80, 95, 147], [33, 163, 153], [248, 3, 199], [140, 32, 37], [197, 69, 197], [220, 71, 38], [24, 211, 52], [137, 39, 69], [153, 6, 61], [236, 170, 105], [176, 174, 65], [123, 251, 118], [79, 200, 56], [205, 76, 112], [150, 173, 64], [241, 189, 24], [137, 39, 158], [31, 251, 129], [56, 3, 26], [253, 101, 17], [99, 19, 151], [225, 211, 158], [251, 46, 236], [151, 55, 106], [26, 208, 93], [30, 221, 92], [139, 104, 239], [219, 51, 181], [224, 0, 230], [64, 221, 28], [218, 197, 251], [113, 249, 136], [188, 141, 15], [130, 93, 121], [202, 37, 241], [217, 124, 172], [110, 174, 46], [122, 195, 139], [166, 49, 196], [86, 217, 79], [59, 22, 44], [246, 35, 69], [220, 206, 218], [67, 33, 176], [74, 242, 37], [184, 220, 143], [84, 99, 220], [217, 75, 249], [136, 64, 2], [251, 149, 54], [61, 133, 162], [56, 42, 242], [109, 241, 241], [207, 64, 182], [206, 139, 208], [87, 119, 140], [12, 128, 73], [159, 193, 47], [154, 137, 164], [57, 91, 255], [221, 155, 157], [172, 78, 65], [8, 249, 189], [93, 94, 197], [14, 82, 201], [64, 115, 108], [44, 246, 110], [90, 145, 119], [129, 26, 216], [131, 158, 156], [179, 148, 82], [143, 227, 3], [29, 38, 145], [133, 213, 82], [50, 91, 9], [29, 88, 229], [242, 94, 4], [233, 41, 150], [218, 248, 171], [182, 205, 37], [64, 19, 53], [244, 156, 63], [101, 158, 182], [244, 150, 153], [9, 80, 26], [107, 28, 74], [250, 92, 156], [253, 214, 211], [204, 113, 28], [142, 91, 170], [185, 103, 243], [86, 155, 117], [6, 173, 4], [220, 32, 58], [107, 29, 253], [30, 215, 181], [151, 201, 207], [85, 102, 190], [171, 198, 205], [69, 71, 74], [228, 52, 21], [106, 161, 168], [20, 187, 10], [66, 208, 79], [110, 132, 105], [27, 138, 148], [175, 7, 160], [247, 110, 34], [159, 146, 204], [222, 127, 95], [218, 144, 70], [220, 187, 223], [149, 5, 139], [66, 68, 171], [215, 185, 163], [179, 161, 162], [214, 47, 126], [199, 112, 215], [55, 136, 70], [179, 176, 149], [190, 106, 2], [7, 57, 89], [52, 105, 72], [218, 214, 213], [90, 171, 105], [192, 139, 245], [207, 53, 57], [212, 111, 254], [254, 144, 239], [222, 149, 143], [223, 139, 242], [87, 45, 77], [232, 252, 70], [53, 92, 91], [21, 158, 226], [29, 14, 184], [213, 205, 91], [253, 191, 40], [135, 101, 253], [201, 234, 46], [100, 246, 224], [5, 128, 247], [130, 147, 249], [217, 188, 236], [155, 209, 55], [153, 182, 156], [10, 195, 47], [137, 162, 30], [203, 230, 126], [113, 227, 13], [50, 68, 2], [92, 88, 243], [223, 46, 34], [87, 32, 79], [160, 135, 34], [252, 112, 119], [210, 97, 145], [36, 21, 90], [97, 69, 81], [5, 189, 36], [152, 238, 159], [82, 206, 204], [183, 47, 72], [253, 49, 198], [16, 159, 111], [190, 127, 197], [228, 60, 84], [154, 29, 243], [125, 111, 140], [151, 219, 211], [4, 181, 46], [110, 149, 41], [214, 174, 178], [79, 12, 185], [107, 157, 195], [156, 183, 110], [137, 116, 255], [186, 189, 222], [173, 159, 78], [161, 84, 12], [236, 1, 207], [44, 19, 98], [59, 254, 21], [49, 226, 211], [146, 22, 148], [0, 172, 25], [96, 188, 18], [147, 252, 233], [129, 209, 151], [249, 251, 215], [61, 178, 85], [225, 250, 192], [17, 44, 63], [200, 25, 111], [204, 65, 41], [200, 245, 33], [127, 146, 109], [124, 89, 235], [10, 195, 206], [184, 252, 172], [242, 102, 53], [190, 115, 175], [196, 162, 19], [161, 38, 90], [12, 220, 208], [206, 166, 81], [227, 17, 71], [111, 182, 54], [95, 129, 153], [166, 58, 136], [185, 152, 246], [139, 102, 60], [91, 135, 103], [240, 16, 98], [17, 180, 113], [189, 190, 131], [39, 136, 185], [179, 60, 222], [180, 201, 247], [117, 178, 125], [175, 75, 184], [218, 55, 180], [97, 12, 186], [187, 36, 65], [10, 42, 247], [25, 42, 121], [45, 160, 248], [32, 182, 157], [5, 209, 72], [47, 183, 57], [126, 44, 25], [227, 153, 16], [153, 40, 45], [193, 225, 223], [244, 134, 173], [156, 25, 202], [254, 242, 203], [54, 169, 9], [92, 255, 108], [108, 137, 104], [86, 139, 46], [62, 115, 113], [226, 133, 152], [100, 69, 208], [184, 167, 81], [203, 246, 124], [17, 38, 38], [113, 82, 139], [27, 111, 172], [255, 31, 57], [205, 117, 82], [131, 36, 35], [226, 166, 66], [185, 9, 66], [252, 15, 95], [216, 191, 193], [238, 6, 123], [11, 190, 110], [80, 86, 237], [119, 16, 191], [55, 68, 179], [163, 218, 213], [237, 117, 48], [15, 104, 191], [203, 237, 68], [223, 25, 100], [30, 19, 226], [95, 30, 34], [225, 119, 239], [8, 66, 139], [77, 138, 185], [200, 171, 156], [81, 18, 251], [135, 106, 143], [65, 88, 137], [253, 158, 181], [168, 167, 229], [193, 135, 67], [62, 39, 138], [63, 208, 45], [199, 131, 64], [69, 9, 174], [5, 8, 121], [85, 155, 206], [127, 132, 23], [154, 209, 185], [189, 217, 12], [138, 217, 204], [191, 7, 124], [107, 77, 48], [227, 242, 72], [6, 137, 223], [194, 148, 181], [234, 140, 38], [153, 199, 244], [40, 41, 148], [58, 126, 82], [93, 40, 122], [80, 76, 49], [73, 245, 181], [113, 85, 29], [249, 115, 25], [30, 7, 245], [199, 125, 194], [32, 223, 109], [171, 86, 214], [146, 140, 120], [64, 122, 94], [61, 47, 194], [167, 203, 103], [52, 105, 3], [141, 53, 50], [160, 188, 144], [135, 147, 194], [24, 206, 35], [189, 243, 106], [210, 110, 160], [129, 209, 214], [246, 74, 94], [135, 194, 44], [97, 166, 203], [251, 223, 135], [73, 169, 185], [190, 25, 91], [120, 124, 212], [87, 15, 128], [40, 122, 224], [79, 222, 91], [150, 66, 137], [79, 204, 88], [11, 189, 169], [82, 210, 167], [140, 32, 123], [4, 174, 246], [253, 221, 4], [149, 131, 225], [191, 210, 85], [222, 176, 148], [53, 47, 224], [93, 134, 217], [26, 99, 220], [190, 128, 140], [184, 100, 13], [146, 143, 234], [233, 37, 127], [249, 77, 72], [175, 113, 55], [111, 254, 25], [140, 234, 12], [183, 72, 56], [179, 58, 123], [243, 91, 165], [145, 251, 40], [81, 138, 224], [178, 70, 108], [15, 38, 128], [101, 189, 143], [99, 240, 126], [30, 168, 90], [105, 172, 133], [201, 179, 31], [200, 48, 210], [18, 0, 138], [39, 157, 119], [180, 137, 122], [15, 23, 165], [99, 205, 212], [196, 68, 182], [97, 148, 166], [255, 175, 205], [141, 231, 15], [45, 226, 184], [1, 154, 135], [124, 219, 148], [55, 139, 44], [161, 63, 46], [110, 93, 121], [120, 149, 35], [98, 91, 86], [158, 245, 73], [222, 46, 105], [157, 194, 178], [40, 117, 34], [251, 138, 130], [35, 40, 159], [72, 42, 225], [25, 102, 227], [187, 10, 206], [22, 202, 175], [67, 215, 230], [30, 12, 144], [223, 252, 69], [75, 245, 199], [166, 208, 233], [230, 224, 11], [44, 248, 32], [24, 206, 44], [146, 74, 16], [204, 48, 247], [95, 254, 10], [141, 50, 0], [99, 246, 153], [131, 255, 105], [134, 207, 33], [37, 52, 13], [8, 57, 65], [205, 243, 153], [8, 65, 76], [97, 204, 217], [152, 215, 220], [75, 28, 88], [250, 207, 210], [51, 107, 110], [176, 118, 185], [209, 17, 23], [16, 143, 15], [211, 241, 170], [183, 96, 126], [122, 49, 106], [84, 25, 81], [120, 123, 121], [166, 17, 130], [254, 119, 47], [135, 213, 249], [39, 148, 160], [15, 11, 97], [249, 186, 251], [207, 243, 166], [108, 88, 222], [114, 2, 189], [124, 186, 14], [74, 175, 246], [118, 89, 63], [51, 97, 12], [153, 249, 36], [5, 223, 234], [129, 225, 254], [2, 107, 194], [42, 98, 176], [2, 110, 117], [36, 119, 201], [58, 230, 39], [101, 140, 214], [203, 208, 41], [140, 218, 166], [128, 178, 124], [29, 247, 94], [134, 164, 245], [115, 100, 102], [158, 210, 98], [231, 158, 111], [18, 217, 201], [28, 220, 211], [239, 179, 21], [112, 98, 233], [108, 107, 170], [26, 118, 191], [40, 172, 151], [162, 108, 129], [234, 90, 201], [144, 7, 180], [192, 41, 19], [29, 146, 198], [8, 73, 165], [228, 200, 84], [27, 68, 38], [35, 204, 19], [103, 47, 131], [139, 54, 232], [113, 25, 99], [41, 3, 92], [162, 74, 66], [240, 99, 2], [12, 3, 90], [159, 105, 137], [30, 106, 56], [21, 43, 90], [181, 105, 78], [164, 129, 196], [206, 19, 218], [19, 158, 77], [129, 68, 52], [49, 99, 5], [68, 176, 20], [80, 199, 225], [6, 103, 118], [182, 120, 24], [55, 236, 33], [163, 75, 130], [110, 143, 141], [88, 183, 83], [184, 186, 205], [81, 231, 41], [117, 183, 192], [137, 83, 128], [19, 228, 217], [57, 101, 173], [219, 113, 4], [224, 24, 50], [152, 192, 29], [14, 134, 139], [165, 116, 77], [84, 133, 45], [138, 158, 167], [221, 224, 59], [46, 61, 44], [98, 206, 132], [254, 94, 102], [229, 11, 241], [234, 31, 79], [169, 215, 194], [198, 213, 250], [97, 190, 115], [104, 60, 185], [200, 152, 16], [215, 77, 177], [52, 12, 55], [82, 128, 240], [67, 108, 28], [115, 141, 182], [7, 204, 3], [70, 138, 16], [65, 97, 7], [234, 39, 221], [66, 196, 115], [203, 110, 216], [60, 29, 188], [115, 72, 110], [110, 159, 86], [85, 25, 16], [248, 152, 136], [242, 120, 57], [126, 98, 132], [66, 24, 103], [138, 194, 126], [127, 131, 50], [112, 188, 122], [37, 35, 234], [226, 106, 43], [23, 229, 226], [82, 32, 168], [210, 164, 53], [216, 45, 209], [17, 104, 74], [173, 148, 33], [112, 163, 87], [60, 193, 182], [31, 222, 145], [1, 25, 50], [135, 173, 88], [64, 162, 58], [59, 179, 240], [159, 156, 2], [50, 117, 183], [54, 213, 216], [181, 220, 251], [226, 31, 127], [47, 202, 104], [126, 198, 65], [126, 211, 131], [155, 36, 241], [218, 254, 148], [70, 41, 244], [195, 64, 84], [67, 16, 94], [32, 103, 127], [13, 227, 174], [125, 230, 167], [196, 133, 62], [13, 155, 119], [148, 18, 201], [194, 244, 170], [113, 199, 71], [35, 25, 21], [93, 253, 225], [99, 22, 77], [9, 138, 32], [49, 49, 102], [252, 39, 88], [35, 158, 141], [137, 250, 172], [180, 100, 110], [103, 236, 110], [149, 162, 129], [113, 79, 113], [105, 98, 46], [84, 4, 236], [121, 234, 217], [118, 165, 192], [104, 228, 20], [164, 42, 143], [135, 222, 170], [144, 133, 174], [38, 203, 104], [183, 236, 78], [52, 80, 90], [106, 155, 180], [149, 81, 79], [221, 159, 157], [245, 34, 254], [30, 92, 174], [62, 221, 210], [44, 92, 219], [69, 11, 39], [97, 172, 23], [69, 228, 104], [150, 130, 101], [113, 64, 18], [5, 228, 95], [233, 136, 3], [247, 122, 166], [104, 138, 81], [46, 229, 34], [243, 221, 86], [25, 189, 180], [245, 140, 111], [108, 139, 103], [11, 29, 120], [75, 26, 80], [51, 210, 40], [22, 158, 103], [158, 164, 4], [69, 163, 3], [247, 111, 19], [30, 193, 172], [250, 184, 129], [51, 236, 139], [201, 59, 67], [89, 23, 2], [243, 97, 239], [8, 31, 55], [4, 23, 57], [99, 231, 226], [104, 67, 43], [101, 224, 119], [67, 42, 173], [254, 71, 154], [73, 212, 158], [183, 29, 191], [79, 23, 5], [220, 73, 239], [48, 123, 215], [255, 12, 245], [224, 162, 123], [230, 167, 73], [20, 69, 103], [149, 208, 100], [131, 197, 102], [195, 0, 115], [154, 250, 89], [127, 150, 230], [135, 29, 188], [93, 117, 134], [242, 216, 29], [96, 235, 152], [163, 149, 205], [197, 79, 255], [224, 178, 48], [97, 119, 9], [19, 161, 78], [32, 139, 236], [164, 117, 104], [161, 114, 9], [183, 45, 17], [55, 227, 211], [186, 116, 235], [83, 45, 71], [124, 219, 214], [27, 25, 90], [227, 75, 162], [64, 85, 220], [209, 130, 214], [173, 227, 9], [177, 219, 47], [242, 31, 57], [28, 64, 251], [94, 203, 110], [201, 116, 77], [199, 240, 25], [13, 156, 104], [183, 47, 142], [192, 93, 51], [86, 230, 134], [81, 7, 26], [150, 222, 58], [76, 90, 76], [5, 103, 248], [182, 176, 170], [227, 133, 182], [153, 234, 248], [207, 44, 164], [112, 13, 195], [137, 132, 179], [119, 22, 23], [54, 172, 87], [50, 228, 179], [120, 127, 242], [49, 32, 96], [108, 246, 89], [5, 35, 77], [178, 174, 204], [92, 62, 86], [2, 253, 214], [205, 129, 37], [216, 243, 171], [29, 85, 46], [141, 221, 116], [198, 89, 221], [67, 40, 130], [124, 197, 165], [201, 202, 245], [159, 84, 16], [30, 4, 224], [32, 58, 70], [77, 190, 155], [43, 160, 120], [81, 115, 164], [193, 120, 196], [213, 255, 235], [190, 78, 135], [137, 187, 211], [129, 39, 55], [48, 169, 181], [133, 171, 5], [182, 70, 128], [193, 15, 244], [53, 58, 14], [101, 167, 220], [112, 63, 26], [189, 102, 96], [136, 17, 238], [45, 186, 166], [93, 232, 139], [117, 111, 236], [218, 133, 114], [59, 16, 217], [112, 44, 171], [25, 108, 165], [61, 39, 110], [94, 16, 133], [229, 32, 75], [109, 136, 214], [251, 242, 64], [179, 183, 163], [127, 192, 69], [166, 42, 81], [144, 116, 29], [0, 70, 108], [150, 142, 8], [161, 130, 39], [57, 80, 230], [244, 114, 24], [97, 236, 251], [116, 89, 32], [211, 189, 236], [3, 122, 211], [190, 157, 195], [172, 186, 200], [46, 244, 239], [91, 23, 221], [37, 205, 197], [212, 172, 133], [141, 46, 246], [126, 15, 25], [191, 204, 155], [213, 4, 29], [99, 87, 247], [7, 83, 136], [182, 230, 96], [61, 93, 251], [137, 251, 63], [244, 181, 27], [17, 235, 144], [239, 116, 141], [187, 230, 199], [18, 177, 80], [19, 156, 50], [128, 23, 92], [28, 0, 247], [212, 147, 251], [154, 209, 201], [127, 75, 149], [145, 153, 87], [66, 136, 190], [13, 150, 113], [166, 162, 65], [2, 243, 228], [202, 198, 80], [36, 216, 255], [181, 94, 147], [76, 134, 136], [143, 156, 252], [147, 34, 114], [124, 134, 104], [160, 50, 157], [160, 32, 117], [164, 135, 137], [151, 177, 81], [160, 253, 85], [216, 233, 12], [114, 231, 224], [92, 168, 230], [49, 34, 21], [10, 239, 162], [167, 77, 122], [47, 34, 108], [146, 113, 64], [7, 160, 32], [235, 64, 58], [153, 102, 152], [120, 25, 157], [35, 229, 124], [68, 172, 146], [198, 119, 4], [104, 52, 0], [81, 107, 237], [192, 91, 94], [198, 235, 108], [216, 79, 217], [190, 39, 26], [97, 172, 8], [72, 228, 48], [69, 174, 154], [114, 107, 144], [87, 80, 3], [1, 254, 13], [127, 111, 91], [184, 123, 222], [153, 203, 117], [100, 173, 248], [255, 41, 180], [94, 7, 188], [221, 153, 152], [234, 30, 153], [105, 134, 241], [148, 96, 177], [105, 16, 159], [156, 29, 77], [137, 224, 236], [187, 204, 87], [142, 73, 8], [69, 194, 77], [202, 99, 109], [154, 9, 171], [14, 124, 3], [235, 142, 235], [141, 194, 244], [87, 70, 105], [62, 235, 93], [115, 254, 191], [17, 125, 227], [135, 127, 227], [137, 60, 201], [134, 105, 23], [112, 188, 76], [165, 132, 249], [118, 8, 85], [62, 244, 200], [86, 47, 62], [202, 14, 75], [142, 2, 50], [132, 8, 203], [189, 195, 152], [113, 48, 238], [37, 99, 80], [101, 93, 188], [85, 226, 169], [109, 96, 166], [114, 16, 139], [184, 52, 180], [242, 29, 89], [242, 193, 194], [10, 69, 201], [166, 234, 80], [81, 26, 176], [216, 122, 140], [51, 79, 110], [59, 39, 123], [57, 153, 191], [90, 35, 37], [49, 55, 45], [164, 52, 75], [8, 143, 125], [199, 169, 103], [126, 79, 134], [18, 41, 86], [90, 198, 252], [144, 25, 156], [134, 223, 240], [80, 118, 47], [103, 6, 138], [95, 30, 192], [113, 71, 100], [236, 29, 212], [28, 158, 189], [193, 147, 234], [213, 219, 227], [219, 6, 114], [104, 66, 206], [147, 90, 29], [40, 35, 100], [125, 232, 162], [185, 170, 19], [140, 141, 246], [60, 157, 55], [168, 126, 202], [139, 31, 17], [193, 28, 203], [187, 5, 165], [175, 215, 47], [98, 52, 216], [34, 132, 227], [211, 106, 0], [80, 189, 18], [97, 172, 99], [251, 237, 3], [195, 238, 230], [108, 238, 243], [67, 131, 127], [33, 116, 220], [214, 193, 115], [131, 35, 196], [70, 183, 228], [8, 238, 142], [151, 115, 197], [189, 8, 153], [105, 42, 183], [251, 95, 183], [21, 210, 231], [147, 190, 24], [170, 44, 143], [184, 132, 176], [18, 159, 181], [175, 230, 19], [228, 214, 6], [109, 119, 241], [208, 154, 131], [64, 134, 231], [122, 106, 190], [87, 223, 136], [117, 61, 84], [136, 4, 84], [239, 43, 143], [107, 186, 183], [128, 133, 187], [145, 252, 231], [52, 254, 169], [168, 200, 22], [96, 13, 150], [94, 202, 123], [230, 209, 47], [16, 239, 135], [130, 177, 236], [162, 238, 134], [21, 99, 22], [65, 82, 126], [135, 33, 171], [45, 15, 183], [239, 247, 157], [183, 143, 165], [128, 186, 161], [145, 219, 121], [99, 211, 164], [166, 224, 147], [248, 206, 231], [185, 73, 51], [146, 88, 167], [55, 64, 96], [136, 45, 130], [170, 219, 93], [168, 202, 101], [83, 208, 249], [126, 46, 11], [238, 134, 131], [7, 240, 15], [159, 238, 250], [114, 238, 161], [52, 74, 55], [154, 192, 83], [30, 16, 70], [33, 59, 160], [19, 127, 63], [81, 22, 13], [237, 240, 229], [101, 208, 155], [217, 20, 43], [146, 120, 48], [77, 69, 250], [170, 29, 134], [113, 218, 185], [94, 246, 87], [105, 198, 148], [233, 94, 228], [151, 166, 68], [240, 174, 56], [175, 15, 122], [120, 87, 163], [211, 157, 113], [218, 3, 107], [225, 148, 7], [3, 26, 210], [76, 121, 192], [233, 27, 217], [21, 142, 19], [149, 121, 17], [219, 162, 128], [225, 64, 101], [174, 5, 21], [104, 39, 54], [167, 35, 84], [197, 226, 53], [56, 245, 98], [85, 56, 152], [221, 80, 232], [47, 31, 38], [37, 210, 228], [93, 117, 206], [177, 179, 119], [120, 89, 167], [11, 14, 128], [10, 113, 178], [70, 99, 68], [145, 48, 136], [201, 80, 48], [57, 156, 77], [231, 40, 58], [244, 232, 246], [21, 164, 0], [66, 43, 96], [70, 161, 212], [144, 113, 1], [59, 224, 138], [198, 176, 132], [93, 172, 163], [28, 255, 144], [80, 18, 204], [218, 130, 4], [105, 229, 15], [116, 118, 4], [9, 222, 10], [200, 32, 63], [117, 214, 212], [233, 18, 122], [175, 236, 89], [146, 65, 234], [71, 241, 241], [14, 204, 247], [204, 215, 28], [234, 170, 207], [125, 17, 164], [132, 8, 203], [207, 15, 12], [9, 249, 137], [217, 175, 134], [129, 127, 244], [11, 71, 4], [124, 163, 71], [49, 202, 138], [119, 104, 59], [96, 65, 74], [177, 245, 224], [192, 174, 25], [199, 97, 36], [56, 148, 33], [7, 102, 186], [61, 146, 229], [151, 7, 137], [202, 199, 190], [212, 242, 161], [170, 57, 252], [30, 137, 189], [43, 111, 251], [255, 93, 242], [128, 231, 169], [141, 164, 50], [182, 221, 139], [168, 228, 241], [107, 34, 225], [101, 80, 174], [196, 228, 226], [58, 147, 151], [53, 96, 51], [50, 104, 148], [129, 27, 21], [130, 23, 114], [99, 8, 222], [208, 90, 149], [72, 95, 137], [9, 213, 21], [52, 182, 85], [240, 38, 80], [179, 212, 154], [109, 138, 223], [161, 44, 179], [178, 167, 59], [123, 88, 242], [8, 88, 12], [253, 229, 221], [61, 10, 2], [168, 84, 84], [37, 102, 75], [122, 114, 191], [110, 90, 57], [138, 207, 197], [72, 71, 123], [197, 202, 82], [33, 254, 111], [95, 108, 22], [76, 66, 140], [226, 123, 122], [116, 154, 195], [62, 103, 1], [249, 185, 232], [182, 203, 198], [156, 124, 78], [49, 149, 6], [196, 97, 230], [238, 18, 2], [238, 248, 180], [173, 159, 173], [181, 144, 118], [247, 220, 158], [209, 180, 194], [254, 60, 58], [204, 162, 54], [176, 44, 216], [206, 164, 107], [99, 58, 202], [53, 170, 0], [231, 247, 121], [169, 155, 110], [52, 98, 243], [21, 6, 147], [190, 68, 210], [224, 137, 33], [79, 229, 126], [11, 223, 63], [79, 180, 161], [30, 138, 90], [73, 195, 247], [97, 223, 112], [185, 44, 113], [77, 191, 202], [66, 207, 145], [108, 83, 183], [43, 112, 77], [16, 61, 13], [195, 205, 193], [163, 71, 214], [0, 242, 72], [37, 114, 6], [128, 17, 41], [196, 169, 135], [101, 190, 37], [71, 10, 206], [29, 54, 165], [97, 75, 42], [228, 13, 155], [225, 193, 91], [44, 31, 146], [130, 165, 188], [104, 151, 54], [64, 26, 18], [167, 117, 213], [56, 45, 226], [124, 148, 216], [185, 103, 33], [158, 106, 25], [65, 91, 135], [66, 16, 119], [255, 1, 107], [150, 230, 162], [134, 38, 187], [166, 100, 52], [200, 69, 152], [136, 54, 159], [58, 213, 211], [162, 138, 198], [234, 160, 46], [190, 126, 115], [98, 224, 168], [195, 214, 116], [78, 170, 65], [169, 102, 3], [27, 2, 118], [181, 93, 242], [192, 137, 159], [195, 128, 150], [228, 108, 246], [81, 20, 94], [253, 58, 232], [119, 0, 182], [229, 165, 86], [148, 127, 28], [75, 47, 17], [174, 106, 134], [240, 139, 227], [242, 73, 71], [61, 13, 113], [247, 144, 20], [204, 17, 34], [198, 73, 28], [247, 96, 229], [90, 67, 252], [51, 51, 244], [123, 31, 161], [71, 65, 128], [65, 111, 13], [146, 95, 179], [22, 167, 247], [38, 225, 82], [46, 94, 78], [53, 112, 233], [240, 159, 224], [89, 46, 196], [237, 68, 253], [60, 47, 89], [64, 61, 99], [81, 239, 151], [197, 234, 26], [12, 194, 186], [170, 174, 163], [204, 9, 239], [106, 169, 176], [194, 213, 199], [45, 192, 216], [158, 128, 94], [97, 61, 238], [63, 65, 56], [155, 124, 127], [7, 32, 247], [104, 137, 97], [183, 116, 82], [17, 20, 202], [197, 115, 37], [218, 191, 20], [244, 211, 168], [97, 69, 23], [11, 95, 147], [121, 234, 151], [73, 207, 107], [181, 200, 247], [97, 99, 145], [79, 11, 91], [134, 35, 104], [15, 144, 211], [106, 164, 235], [66, 145, 35], [165, 79, 87], [159, 89, 207], [180, 246, 31], [147, 212, 144], [124, 181, 68], [129, 118, 28], [53, 234, 158], [188, 245, 180], [217, 114, 43], [139, 138, 201], [31, 45, 103], [64, 80, 121], [225, 193, 251], [159, 12, 190], [93, 42, 113], [203, 122, 92], [242, 62, 182], [215, 169, 7], [97, 199, 134], [109, 77, 137], [235, 203, 53], [247, 92, 247], [30, 96, 248], [218, 72, 210], [190, 44, 33], [156, 222, 175], [159, 98, 196], [179, 88, 164], [174, 124, 208], [109, 22, 66], [201, 165, 182], [20, 196, 138], [250, 127, 59], [81, 105, 53], [247, 41, 8], [187, 2, 84], [254, 233, 118], [232, 19, 227], [54, 245, 63], [67, 53, 254], [245, 55, 254], [131, 70, 204], [78, 85, 78], [48, 151, 117], [170, 63, 28], [116, 182, 7], [157, 128, 78], [212, 191, 128], [176, 136, 114], [200, 23, 2], [233, 253, 79], [183, 52, 23], [175, 189, 167], [215, 201, 90], [179, 179, 130], [167, 90, 92], [28, 95, 20], [175, 61, 177], [206, 56, 97], [248, 16, 247], [179, 155, 98], [147, 93, 159], [236, 24, 71], [188, 164, 142], [134, 7, 183], [222, 216, 32], [215, 119, 79], [58, 185, 44], [174, 240, 68], [161, 50, 205], [230, 98, 22], [133, 128, 190], [97, 101, 111], [187, 16, 176], [252, 49, 10], [125, 104, 80], [119, 228, 94], [200, 49, 33], [174, 238, 80], [59, 249, 239], [47, 65, 196], [37, 232, 207], [127, 207, 78], [198, 69, 249], [27, 199, 187], [218, 22, 85], [44, 146, 74], [4, 206, 21], [213, 244, 20], [229, 251, 41], [157, 229, 134], [71, 37, 191], [109, 80, 151], [171, 206, 61], [41, 127, 248], [67, 67, 196], [18, 69, 194], [149, 3, 67], [25, 99, 21], [48, 96, 241], [149, 198, 135], [40, 190, 196], [190, 151, 4], [80, 196, 10], [235, 250, 61], [220, 43, 217], [185, 128, 187], [54, 48, 101], [255, 95, 67], [73, 72, 1], [231, 190, 229], [104, 210, 8], [67, 167, 130], [97, 61, 121], [31, 170, 58], [143, 189, 139], [98, 243, 240], [51, 67, 221], [16, 233, 9], [25, 169, 20], [7, 185, 147], [158, 218, 113], [134, 166, 68], [199, 147, 152], [201, 224, 165], [211, 15, 4], [191, 222, 43], [106, 202, 30], [255, 36, 13], [246, 120, 10], [156, 156, 120], [157, 79, 208], [176, 99, 49], [242, 132, 68], [211, 255, 77], [228, 7, 178], [85, 118, 80], [242, 242, 100], [75, 46, 101], [25, 188, 59], [182, 210, 212], [54, 157, 119], [62, 214, 50], [128, 167, 11], [254, 126, 236], [18, 183, 248], [208, 210, 217], [89, 55, 123], [14, 91, 27], [204, 193, 230], [158, 48, 225], [238, 72, 113], [225, 16, 252], [25, 235, 154], [61, 85, 234], [230, 42, 117], [17, 28, 129], [63, 71, 102], [38, 51, 252], [123, 92, 56], [167, 163, 247], [57, 201, 157], [215, 190, 167], [78, 112, 239], [253, 61, 144], [255, 105, 236], [15, 220, 208], [27, 216, 74], [23, 132, 137], [156, 58, 221], [239, 201, 72], [222, 34, 67], [76, 77, 160], [215, 101, 108], [108, 28, 174], [204, 178, 115], [153, 81, 12], [241, 74, 236], [100, 93, 238], [48, 176, 94], [233, 83, 127], [196, 131, 176], [11, 115, 97], [0, 28, 230], [125, 115, 123], [60, 68, 239], [19, 49, 209], [69, 65, 221], [129, 99, 119], [193, 225, 117], [9, 70, 184], [159, 0, 237], [78, 128, 120], [190, 28, 26], [159, 80, 209], [26, 235, 51], [82, 169, 178], [129, 25, 127], [155, 201, 245], [44, 122, 215], [44, 75, 33], [18, 165, 242], [139, 182, 211], [79, 120, 162], [111, 255, 151], [108, 149, 179], [192, 115, 179], [201, 217, 81], [83, 97, 129], [73, 227, 46], [101, 31, 24], [206, 224, 237], [67, 141, 244], [168, 214, 56], [67, 100, 87], [101, 247, 221], [163, 94, 199], [230, 157, 248], [59, 222, 181], [155, 43, 130], [40, 244, 209], [233, 121, 71], [190, 83, 39], [74, 118, 39], [80, 83, 209], [73, 162, 239], [111, 93, 230], [213, 211, 115], [242, 177, 135], [21, 62, 13], [242, 39, 142], [24, 191, 76], [161, 130, 133], [122, 214, 24], [105, 119, 204], [17, 133, 160], [49, 123, 101], [48, 24, 106], [186, 206, 87], [242, 186, 40], [169, 87, 190], [177, 168, 149], [97, 25, 169], [141, 36, 113], [154, 115, 7], [217, 71, 118], [31, 195, 73], [176, 26, 141], [135, 34, 45], [30, 239, 86], [247, 90, 77], [220, 53, 88], [5, 59, 105], [3, 176, 230], [214, 174, 255], [101, 102, 138], [73, 24, 17], [131, 64, 121], [133, 198, 213], [63, 177, 6], [142, 136, 19], [41, 42, 169], [185, 204, 125], [188, 126, 122], [174, 78, 35], [1, 4, 75], [123, 153, 244], [42, 204, 200], [178, 154, 159], [247, 171, 227], [41, 140, 21], [126, 120, 110], [162, 68, 77], [77, 75, 168], [212, 205, 121], [137, 216, 120], [153, 206, 194], [138, 94, 109], [77, 74, 26], [250, 12, 187], [103, 93, 70], [43, 168, 185], [130, 77, 111], [234, 117, 150], [84, 22, 58], [172, 245, 14], [3, 42, 183], [228, 20, 169], [123, 186, 208], [81, 157, 120], [254, 88, 27], [192, 231, 121], [115, 178, 75], [211, 136, 203], [6, 93, 252], [44, 146, 10], [146, 187, 40], [53, 187, 88], [200, 173, 186], [30, 196, 211], [96, 111, 72], [127, 149, 124], [26, 197, 89], [35, 215, 248], [148, 85, 216], [144, 16, 19], [144, 205, 161], [190, 162, 64], [54, 38, 102], [150, 25, 152], [211, 183, 173], [248, 138, 85], [98, 12, 171], [254, 164, 253], [40, 31, 61], [79, 12, 22], [102, 8, 226], [242, 39, 64], [234, 224, 77], [200, 236, 77], [49, 163, 175], [220, 88, 164], [220, 251, 102], [117, 153, 207], [103, 1, 50], [191, 232, 180], [91, 164, 89], [245, 74, 238], [185, 172, 89], [209, 2, 84], [193, 156, 47], [123, 6, 149], [125, 169, 12], [17, 152, 205], [187, 229, 66], [209, 83, 94], [234, 10, 71], [238, 130, 186], [3, 120, 188], [3, 49, 37], [20, 248, 188], [231, 215, 135], [67, 158, 100], [214, 199, 168], [145, 127, 96], [61, 119, 4], [71, 93, 32], [46, 208, 44], [208, 221, 9], [142, 250, 207], [143, 200, 91], [100, 18, 126], [104, 25, 5], [104, 40, 246], [100, 23, 166], [240, 243, 24], [11, 127, 91], [222, 178, 18], [141, 130, 16], [173, 170, 146], [67, 111, 56], [61, 48, 13], [4, 37, 8], [181, 5, 209], [248, 238, 19], [227, 56, 53], [37, 166, 238], [218, 142, 1], [4, 148, 241], [90, 83, 158], [142, 63, 173], [135, 140, 75], [126, 120, 243], [21, 239, 116], [238, 232, 30], [173, 85, 10], [124, 80, 236], [103, 246, 95], [0, 128, 28], [205, 89, 8], [155, 176, 97], [117, 55, 140], [164, 134, 19], [72, 35, 9], [192, 139, 193], [176, 66, 168], [183, 166, 157], [17, 245, 88], [10, 17, 168], [122, 37, 119], [75, 143, 157], [131, 198, 237], [225, 235, 45], [141, 180, 225], [177, 60, 223], [59, 96, 148], [46, 76, 61], [249, 217, 88], [231, 101, 202], [61, 219, 65], [232, 3, 48], [137, 172, 48], [31, 183, 54], [78, 175, 62], [153, 38, 32], [122, 10, 66], [50, 118, 21], [117, 168, 27], [171, 19, 239], [108, 88, 209], [112, 73, 208], [11, 246, 123], [235, 126, 5], [52, 147, 54], [211, 238, 162], [90, 92, 219], [51, 110, 34], [239, 102, 10], [166, 198, 92], [35, 151, 47], [114, 106, 99], [164, 203, 238], [25, 249, 11], [22, 98, 69], [143, 211, 33], [100, 169, 30], [6, 3, 208], [114, 234, 152], [49, 153, 18], [227, 19, 201], [23, 91, 242], [146, 195, 27], [67, 159, 106], [94, 164, 162], [48, 90, 165], [11, 39, 15], [123, 38, 26], [121, 26, 47], [108, 4, 209], [197, 76, 104], [223, 91, 233], [147, 48, 115], [41, 175, 139], [214, 181, 172], [114, 159, 66], [72, 225, 87], [25, 18, 207], [46, 162, 169], [239, 184, 111], [97, 225, 32], [84, 90, 128], [45, 97, 23], [134, 155, 218], [126, 62, 79], [7, 234, 113], [255, 1, 245], [177, 178, 131], [108, 25, 47], [161, 34, 68], [155, 75, 205], [230, 248, 163], [115, 242, 194], [199, 240, 247], [25, 133, 131], [204, 17, 11], [249, 125, 99], [132, 110, 248], [100, 63, 78], [43, 120, 169], [212, 32, 36], [227, 81, 233], [106, 86, 48], [21, 197, 38], [48, 230, 11], [184, 16, 202], [156, 50, 27], [17, 45, 20], [194, 73, 192], [120, 240, 129], [210, 88, 180], [192, 56, 210], [98, 191, 211], [154, 54, 91], [46, 67, 26], [163, 160, 187], [10, 107, 237], [109, 240, 111], [248, 160, 114], [50, 201, 253], [4, 68, 200], [162, 207, 164], [154, 250, 127], [216, 242, 19], [155, 253, 252], [200, 130, 229], [195, 248, 23], [49, 3, 68], [94, 43, 171], [27, 37, 113], [3, 60, 166], [215, 13, 10], [9, 216, 34], [108, 226, 214], [217, 49, 127], [227, 244, 226], [60, 203, 31], [60, 189, 16], [68, 56, 11], [107, 100, 230], [109, 77, 3], [93, 148, 199], [135, 109, 49], [88, 179, 242], [253, 5, 206], [254, 200, 171], [103, 195, 113], [128, 232, 172], [150, 41, 169], [32, 112, 255], [153, 148, 148], [94, 57, 29], [136, 177, 221], [178, 83, 168], [145, 42, 158], [117, 232, 219], [89, 122, 16], [210, 30, 76], [210, 206, 246], [110, 163, 226], [4, 162, 211], [26, 113, 47], [246, 213, 3], [53, 29, 249], [162, 0, 217], [15, 137, 153], [189, 56, 141], [234, 80, 228], [146, 210, 75], [255, 223, 111], [35, 254, 166], [90, 242, 210], [123, 235, 248], [93, 60, 93], [228, 154, 194], [203, 3, 159], [230, 217, 83], [15, 69, 104], [159, 247, 123], [195, 225, 246], [132, 51, 182], [72, 79, 105], [51, 224, 229], [158, 27, 111], [43, 206, 186], [229, 204, 46], [65, 5, 31], [223, 122, 178], [204, 26, 183], [136, 120, 170], [29, 250, 168], [158, 83, 216], [109, 73, 87], [115, 234, 62], [186, 8, 68], [177, 252, 113], [242, 83, 191], [62, 33, 93], [139, 140, 0], [164, 187, 116], [124, 132, 66], [23, 76, 135], [142, 236, 138], [3, 222, 91], [175, 146, 236], [43, 43, 65], [103, 204, 242], [142, 29, 178], [180, 232, 196], [206, 242, 156], [182, 229, 160], [78, 218, 211], [76, 40, 210], [213, 137, 124], [43, 54, 29], [205, 37, 122], [231, 20, 202], [252, 177, 165], [253, 161, 132], [217, 210, 226], [105, 196, 183], [105, 249, 174], [40, 58, 40], [27, 75, 124], [235, 47, 117], [248, 166, 39], [222, 187, 211], [56, 64, 7], [3, 99, 32], [254, 108, 128], [39, 72, 61], [76, 186, 247], [88, 160, 209], [178, 31, 93], [16, 34, 122], [183, 139, 153], [245, 106, 183], [35, 128, 20], [189, 187, 250], [27, 36, 119], [65, 146, 114], [219, 93, 186], [6, 174, 241], [101, 161, 124], [88, 47, 63], [158, 234, 194], [78, 88, 108], [171, 111, 0], [173, 83, 171], [242, 163, 228], [111, 149, 163], [219, 192, 29], [93, 131, 255], [107, 132, 202], [70, 163, 51], [110, 225, 138], [128, 202, 70], [8, 78, 216], [83, 89, 44], [250, 173, 232], [133, 215, 97], [84, 147, 196], [29, 134, 227], [136, 238, 208], [185, 81, 149], [221, 202, 164], [30, 167, 245], [45, 171, 54], [191, 249, 125], [234, 80, 25], [46, 160, 254], [131, 238, 107], [18, 94, 11], [144, 222, 212], [112, 11, 127], [5, 235, 150], [156, 67, 40], [80, 220, 110], [192, 167, 143], [214, 252, 197], [201, 40, 234], [11, 125, 222], [46, 104, 236], [64, 46, 47], [111, 252, 207], [83, 167, 165], [94, 36, 102], [56, 1, 89], [19, 160, 130], [86, 157, 145], [210, 241, 253], [60, 125, 8], [38, 52, 205], [150, 153, 38], [160, 54, 149], [6, 48, 183], [229, 29, 143], [88, 189, 49], [202, 87, 236], [57, 198, 132], [93, 226, 100], [16, 168, 98], [47, 129, 221], [178, 165, 161], [77, 179, 224], [163, 74, 167], [5, 59, 114], [67, 116, 18], [223, 59, 109], [123, 154, 240], [230, 61, 124], [160, 212, 156], [216, 186, 107], [146, 70, 13], [35, 238, 164], [13, 83, 150], [105, 249, 242], [10, 245, 156], [247, 76, 178], [217, 132, 8], [123, 253, 225], [26, 156, 104], [103, 186, 96], [90, 37, 221], [91, 76, 50], [72, 211, 114], [37, 49, 28], [192, 160, 7], [133, 103, 218], [67, 215, 212], [184, 185, 38], [172, 180, 240], [231, 54, 49], [146, 118, 77], [242, 2, 52], [184, 205, 15], [138, 227, 242], [98, 194, 185], [221, 160, 214], [165, 154, 223], [213, 188, 146], [160, 250, 153], [220, 16, 70], [13, 167, 91], [193, 78, 163], [153, 34, 135], [53, 77, 112], [222, 139, 219], [12, 121, 206], [12, 142, 65], [149, 226, 239], [56, 230, 184], [253, 97, 36], [34, 241, 156], [229, 79, 38], [185, 167, 211], [140, 184, 148], [235, 185, 162], [33, 56, 241], [192, 88, 16], [17, 2, 130], [235, 108, 200], [34, 211, 142], [79, 255, 209], [197, 91, 105], [234, 167, 92], [232, 28, 222], [36, 83, 106], [253, 41, 234], [42, 45, 222], [191, 79, 207], [170, 229, 250], [225, 139, 158], [181, 24, 169], [193, 94, 151], [84, 199, 0], [73, 239, 76], [215, 253, 9], [219, 73, 39], [66, 176, 143], [142, 165, 121], [11, 147, 45], [125, 112, 45], [111, 93, 110], [105, 252, 90], [69, 193, 31], [215, 40, 27], [190, 202, 165], [111, 186, 31], [59, 235, 172], [70, 142, 97], [142, 72, 173], [76, 154, 158], [36, 219, 107], [135, 26, 98], [152, 70, 144], [127, 21, 30], [250, 140, 76], [7, 25, 53], [5, 147, 219], [8, 209, 2], [232, 199, 102], [237, 181, 148], [111, 32, 129], [178, 35, 222], [145, 118, 56], [36, 242, 189], [73, 245, 246], [92, 45, 93], [24, 90, 187], [254, 21, 214], [4, 188, 15], [232, 146, 8], [144, 120, 65], [253, 65, 178], [156, 81, 216], [31, 3, 226], [231, 183, 104], [202, 104, 255], [178, 242, 176], [24, 105, 38], [109, 97, 138], [27, 170, 252], [68, 200, 66], [10, 148, 108], [41, 10, 32], [64, 151, 4], [6, 206, 225], [199, 212, 70], [179, 101, 161], [173, 112, 203], [207, 182, 211], [114, 83, 11], [205, 133, 44], [8, 139, 153], [194, 25, 254], [245, 45, 78], [34, 13, 105], [16, 252, 89], [7, 27, 153], [236, 230, 127], [92, 72, 81], [95, 89, 229], [142, 190, 16], [9, 206, 250], [34, 195, 79], [35, 239, 209], [245, 240, 39], [108, 197, 217], [226, 159, 64], [133, 181, 146], [246, 4, 93], [81, 40, 211], [227, 161, 250], [180, 146, 68], [244, 0, 167], [92, 34, 155], [115, 4, 62], [209, 68, 110], [178, 208, 18], [89, 252, 199], [110, 168, 37], [248, 41, 49], [232, 194, 231], [84, 77, 3], [150, 221, 121], [178, 217, 32], [204, 220, 113], [1, 59, 164], [20, 50, 172], [244, 109, 49], [13, 128, 217], [168, 123, 85], [145, 119, 228], [156, 78, 94], [23, 51, 79], [65, 70, 44], [32, 135, 121], [100, 90, 140], [248, 144, 201], [227, 120, 63], [19, 44, 161], [174, 51, 106], [122, 103, 224], [170, 139, 14], [77, 140, 111], [110, 136, 180], [126, 189, 151], [95, 207, 226], [32, 250, 97], [245, 85, 243], [132, 237, 106], [132, 207, 42], [222, 169, 40], [101, 137, 225], [10, 206, 61], [87, 93, 246], [216, 65, 120], [60, 116, 146], [191, 140, 241], [51, 17, 65], [3, 70, 144], [46, 29, 227], [23, 136, 140], [138, 250, 93], [12, 88, 254], [211, 45, 118], [247, 164, 61], [168, 215, 89], [29, 188, 235], [83, 53, 184], [84, 213, 72], [173, 92, 69], [125, 11, 54], [255, 37, 14], [120, 31, 18], [67, 142, 233], [243, 156, 183], [17, 221, 171], [116, 167, 122], [230, 105, 26], [218, 154, 177], [149, 240, 6], [23, 100, 103], [225, 188, 229], [170, 128, 90], [136, 240, 23], [175, 109, 189], [205, 116, 140], [237, 250, 163], [108, 6, 138], [68, 1, 60], [58, 133, 179], [2, 41, 58], [228, 156, 49], [236, 119, 146], [23, 0, 30], [206, 157, 246], [135, 197, 170], [152, 91, 17], [78, 178, 215], [47, 120, 250], [65, 30, 207], [245, 230, 53], [183, 225, 212], [60, 92, 95], [212, 85, 106], [116, 61, 158], [219, 211, 186], [27, 44, 40], [233, 111, 188], [161, 225, 183], [97, 187, 46], [183, 145, 172], [125, 232, 62], [227, 209, 230], [127, 237, 114], [158, 231, 32], [164, 249, 10], [126, 3, 0], [85, 166, 250], [105, 203, 179], [14, 225, 19], [86, 56, 37], [134, 156, 139], [79, 62, 50], [99, 59, 145], [76, 64, 29], [9, 155, 46], [60, 199, 240], [207, 156, 146], [80, 53, 61], [24, 95, 32], [174, 199, 121], [108, 11, 130], [6, 94, 45], [230, 214, 139], [185, 223, 81], [122, 73, 167], [156, 53, 203], [108, 171, 227], [174, 91, 51], [199, 137, 27], [244, 175, 61], [59, 70, 134], [9, 83, 46], [243, 131, 10], [134, 14, 69], [55, 158, 33], [247, 87, 79], [21, 210, 173], [22, 226, 69], [154, 198, 50], [147, 195, 201], [123, 208, 27], [39, 15, 220], [127, 65, 212], [103, 229, 39], [161, 183, 200], [155, 117, 88], [132, 231, 10], [199, 189, 172], [172, 119, 49], [252, 42, 51], [38, 238, 159], [19, 79, 16], [157, 83, 138], [39, 36, 240], [134, 120, 37], [14, 42, 0], [243, 196, 39], [72, 29, 100], [245, 90, 67], [123, 3, 87], [174, 203, 14], [168, 219, 91], [121, 69, 131], [9, 123, 117], [27, 13, 253], [147, 179, 87], [93, 220, 16], [11, 181, 88], [81, 182, 54], [142, 62, 171], [170, 254, 15], [100, 185, 3], [109, 37, 192], [98, 185, 120], [140, 133, 110], [186, 189, 248], [95, 39, 48], [230, 3, 195], [223, 188, 117], [246, 37, 226], [73, 189, 174], [135, 120, 185], [54, 220, 242], [22, 18, 242], [175, 68, 2], [28, 99, 101], [106, 116, 22], [202, 136, 36], [180, 207, 206], [98, 158, 108], [213, 72, 159], [139, 200, 227], [31, 180, 252], [112, 103, 233], [175, 100, 78], [142, 3, 46], [77, 65, 254], [237, 188, 172], [174, 209, 97], [233, 120, 171], [206, 47, 0], [22, 136, 104], [143, 230, 121], [67, 205, 171], [143, 70, 255], [114, 198, 0], [95, 1, 58], [171, 57, 232], [88, 10, 74], [56, 86, 200], [66, 102, 71], [182, 196, 133], [4, 186, 211], [111, 43, 96], [31, 222, 15], [245, 221, 181], [26, 104, 24], [38, 5, 91], [131, 9, 182], [163, 19, 248], [132, 156, 102], [7, 181, 14], [129, 49, 30], [32, 225, 214], [160, 48, 62], [67, 162, 37], [231, 110, 249], [21, 164, 178], [81, 46, 17], [215, 220, 75], [244, 232, 120], [169, 248, 37], [166, 76, 163], [168, 99, 230], [54, 43, 57], [88, 144, 204], [162, 7, 165], [160, 136, 172], [162, 244, 68], [124, 59, 186], [67, 26, 187], [87, 0, 172], [245, 245, 94], [247, 248, 222], [146, 49, 143], [152, 157, 48], [49, 235, 87], [66, 147, 197], [72, 162, 131], [58, 219, 165], [36, 217, 243], [139, 172, 56], [236, 170, 108], [26, 95, 205], [177, 36, 95], [117, 77, 85], [60, 22, 54], [168, 237, 194], [7, 3, 124], [152, 67, 213], [8, 217, 191], [10, 177, 254], [52, 166, 165], [13, 82, 12], [212, 39, 255], [253, 144, 63], [99, 49, 242], [29, 146, 96], [223, 80, 217], [73, 53, 201], [123, 215, 4], [226, 104, 210], [45, 36, 78], [96, 239, 60], [176, 197, 181], [245, 234, 136], [55, 158, 7], [9, 59, 174], [105, 56, 241], [71, 49, 160], [223, 59, 240], [106, 185, 10], [86, 149, 245], [166, 42, 145], [17, 26, 200], [59, 51, 65], [253, 170, 230], [113, 5, 193], [253, 41, 132], [249, 158, 126], [203, 127, 44], [155, 175, 20], [141, 213, 27], [122, 209, 58], [154, 3, 38], [232, 66, 76], [158, 111, 19], [46, 27, 253], [185, 140, 63], [185, 209, 208], [130, 68, 111], [106, 220, 143], [226, 97, 55], [98, 203, 0], [94, 252, 111], [197, 173, 154], [216, 84, 15], [44, 26, 81], [59, 118, 213], [208, 137, 130], [106, 248, 163], [146, 183, 79], [163, 216, 14], [169, 238, 203], [23, 70, 32], [102, 217, 212], [10, 99, 223], [127, 228, 28], [182, 230, 198], [245, 238, 155], [132, 111, 171], [66, 13, 205], [218, 239, 20], [132, 68, 141], [171, 199, 174], [32, 98, 142], [90, 255, 176], [151, 147, 194], [218, 152, 119], [208, 171, 175], [255, 234, 172], [185, 186, 230], [128, 182, 116], [244, 75, 154], [161, 20, 130], [40, 181, 5], [217, 198, 101], [196, 243, 155], [232, 248, 178], [52, 252, 148], [165, 56, 0], [146, 160, 117], [27, 7, 49], [198, 123, 100], [157, 101, 5], [12, 93, 83], [56, 84, 40], [53, 16, 244], [70, 84, 208], [61, 34, 201], [64, 152, 147], [203, 209, 58], [33, 75, 0], [191, 43, 3], [102, 54, 82], [133, 105, 212], [111, 35, 23], [113, 86, 177], [244, 116, 220], [135, 9, 27], [117, 125, 45], [83, 57, 190], [181, 88, 5], [73, 179, 0], [151, 19, 52], [166, 25, 252], [87, 74, 39], [190, 250, 153], [131, 62, 150], [158, 169, 32], [190, 221, 17], [61, 95, 208], [42, 206, 204], [127, 9, 226], [225, 73, 88], [229, 25, 23], [191, 45, 251], [250, 12, 66], [71, 103, 9], [7, 193, 155], [223, 132, 151], [97, 115, 166], [159, 165, 55], [204, 73, 225], [185, 197, 173], [50, 230, 90], [80, 252, 21], [96, 156, 96], [11, 180, 27], [102, 202, 25], [82, 10, 21], [222, 62, 101], [242, 129, 125], [108, 14, 103], [91, 178, 160], [149, 27, 216], [169, 153, 181], [50, 167, 47], [138, 104, 249], [123, 213, 56], [55, 102, 142], [159, 28, 156], [77, 91, 204], [95, 121, 116], [144, 180, 152], [81, 123, 238], [14, 237, 18], [93, 158, 157], [39, 213, 118], [159, 94, 91], [71, 174, 33], [155, 176, 253], [11, 50, 110], [231, 173, 129], [67, 205, 1], [198, 55, 192], [26, 202, 185], [21, 159, 104], [159, 52, 18], [35, 10, 9], [132, 56, 233], [123, 209, 27], [174, 181, 230], [170, 8, 161], [140, 185, 144], [233, 45, 59], [124, 235, 139], [230, 43, 237], [205, 130, 100], [181, 145, 33], [90, 126, 212], [191, 3, 133], [86, 241, 146], [197, 216, 71], [233, 138, 71], [106, 35, 142], [135, 173, 173], [167, 189, 203], [133, 244, 159], [16, 168, 199], [18, 190, 179], [154, 188, 32], [186, 59, 29], [143, 185, 119], [24, 71, 207], [9, 187, 56], [189, 207, 119], [219, 183, 251], [17, 163, 2], [209, 31, 184], [57, 65, 67], [119, 154, 162], [155, 60, 124], [58, 86, 129], [120, 241, 250], [102, 74, 79], [203, 129, 1], [13, 225, 154], [154, 158, 31], [141, 238, 34], [25, 137, 94], [61, 107, 56], [54, 204, 230], [30, 72, 224], [75, 25, 223], [50, 158, 28], [158, 23, 6], [156, 62, 11], [48, 47, 64], [237, 144, 132], [106, 124, 239], [57, 23, 135], [111, 246, 52], [143, 94, 13], [96, 162, 90], [26, 2, 45], [13, 154, 250], [43, 2, 167], [197, 125, 45], [77, 254, 99], [150, 23, 135], [122, 60, 181], [162, 121, 137], [152, 205, 31], [69, 67, 183], [118, 44, 174], [23, 88, 64], [155, 60, 35], [63, 99, 124], [231, 43, 146], [124, 19, 18], [62, 110, 22], [148, 20, 174], [242, 52, 55], [119, 247, 105], [135, 158, 61], [217, 44, 116], [188, 33, 113], [133, 68, 132], [126, 132, 236], [91, 5, 12], [246, 117, 208], [60, 108, 54], [27, 124, 208], [24, 15, 35], [129, 167, 175], [88, 58, 76], [20, 58, 34], [46, 12, 198], [220, 107, 3], [128, 171, 210], [138, 86, 255], [221, 146, 246], [115, 225, 225], [80, 147, 245], [212, 211, 165], [197, 245, 79], [52, 37, 12], [20, 140, 139], [16, 59, 127], [1, 127, 50], [131, 160, 61], [195, 152, 186], [145, 193, 13], [174, 83, 41], [104, 40, 181], [253, 75, 153], [35, 211, 86], [239, 209, 93], [241, 133, 192], [34, 27, 42], [215, 39, 50], [215, 129, 17], [248, 157, 109], [45, 79, 71], [200, 10, 14], [238, 220, 48], [123, 68, 114], [235, 104, 109], [155, 13, 37], [207, 167, 16], [107, 193, 60], [235, 204, 50], [41, 42, 234], [72, 222, 81], [128, 126, 25], [239, 8, 227], [219, 139, 160], [240, 61, 29], [179, 253, 255], [53, 159, 199], [164, 143, 33], [219, 10, 221], [159, 119, 5], [247, 57, 59], [175, 236, 134], [71, 51, 121], [116, 175, 94], [117, 95, 205], [116, 117, 72], [147, 130, 224], [219, 230, 13], [213, 169, 170], [164, 57, 123], [41, 121, 208], [229, 76, 254], [82, 188, 80], [49, 193, 188], [37, 22, 103], [178, 14, 70], [150, 132, 225], [192, 186, 74], [113, 106, 9], [223, 204, 15], [230, 226, 228], [141, 111, 155], [135, 73, 14], [117, 106, 113], [22, 183, 233], [206, 168, 222], [29, 240, 71], [207, 73, 181], [79, 166, 12], [198, 252, 14], [103, 209, 40], [218, 92, 135], [139, 68, 216], [15, 28, 52], [46, 215, 36], [52, 192, 204], [164, 215, 198], [62, 31, 12], [162, 132, 4], [21, 205, 187], [255, 157, 151], [106, 231, 214], [181, 250, 135], [155, 143, 19], [13, 4, 187], [233, 1, 230], [27, 197, 201], [188, 91, 53], [129, 48, 160], [160, 13, 205], [43, 199, 54], [20, 134, 24], [239, 10, 136], [91, 52, 209], [168, 28, 141], [102, 108, 232], [143, 247, 201], [33, 143, 182], [193, 51, 168], [160, 33, 91], [245, 50, 69], [37, 40, 212], [22, 188, 198], [111, 61, 237], [213, 247, 33], [191, 13, 29], [141, 197, 214], [208, 20, 100], [122, 150, 178], [103, 117, 122], [56, 250, 109], [71, 147, 111], [28, 139, 135], [33, 160, 218], [62, 245, 189], [67, 143, 153], [144, 254, 255], [101, 189, 80], [133, 224, 68], [244, 213, 11], [164, 78, 217], [90, 199, 18], [108, 221, 147], [52, 30, 145], [84, 93, 202], [13, 27, 68], [133, 50, 64], [14, 133, 178], [178, 171, 75], [3, 77, 112], [176, 230, 36], [84, 227, 185], [32, 76, 189], [9, 176, 135], [175, 230, 111], [176, 245, 182], [140, 200, 139], [255, 106, 58], [204, 196, 9], [173, 106, 28], [191, 10, 36], [53, 147, 235], [145, 91, 52], [177, 8, 108], [199, 134, 19], [53, 240, 39], [115, 229, 29], [200, 177, 68], [142, 51, 141], [13, 208, 22], [85, 61, 32], [204, 150, 245], [98, 183, 151], [28, 158, 24], [105, 131, 77], [81, 245, 86], [144, 105, 23], [147, 215, 225], [10, 84, 119], [57, 5, 80], [99, 123, 78], [226, 249, 175], [90, 195, 206], [170, 101, 236], [163, 135, 234], [69, 146, 248], [15, 142, 225], [240, 22, 149], [110, 176, 17], [242, 31, 18], [148, 152, 229], [165, 71, 194], [120, 249, 226], [209, 214, 1], [146, 220, 201], [69, 31, 174], [122, 238, 42], [95, 107, 245], [185, 164, 59], [200, 175, 154], [193, 122, 216], [178, 146, 106], [150, 26, 64], [168, 48, 58], [6, 113, 143], [17, 211, 54], [216, 9, 82], [94, 30, 210], [7, 86, 255], [80, 111, 225], [161, 91, 120], [135, 194, 200], [119, 231, 0], [57, 50, 179], [170, 158, 73], [46, 130, 220], [122, 255, 109], [3, 35, 18], [41, 9, 201], [201, 27, 119], [87, 124, 138], [39, 184, 154], [11, 152, 18], [143, 145, 86], [149, 161, 95], [88, 131, 147], [36, 16, 193], [164, 226, 172], [193, 85, 5], [78, 115, 132], [147, 199, 210], [200, 203, 53], [232, 79, 227], [9, 55, 154], [119, 176, 155], [130, 208, 237], [5, 55, 70], [107, 116, 145], [175, 244, 94], [8, 33, 89], [153, 49, 142], [100, 178, 50], [254, 136, 17], [147, 115, 147], [199, 85, 21], [237, 233, 139], [62, 85, 36], [147, 245, 215], [38, 23, 18], [203, 209, 18], [61, 168, 6], [229, 43, 17], [226, 78, 55], [253, 217, 0], [121, 149, 170], [105, 112, 84], [179, 109, 131], [2, 180, 117], [100, 7, 61], [237, 215, 120], [103, 237, 151], [125, 98, 18], [61, 40, 136], [134, 72, 44], [112, 132, 136], [169, 145, 18], [93, 216, 237], [230, 26, 71], [126, 119, 175], [47, 44, 210], [26, 249, 246], [239, 51, 77], [36, 77, 165], [232, 56, 135], [39, 227, 107], [174, 163, 16], [231, 168, 62], [165, 97, 145], [185, 40, 240], [32, 237, 248], [243, 68, 147], [248, 27, 182], [91, 4, 61], [136, 115, 108], [63, 238, 237], [153, 31, 150], [37, 184, 26], [186, 192, 220], [125, 83, 229], [116, 71, 186], [250, 255, 125], [91, 206, 24], [75, 70, 255], [22, 59, 243], [242, 59, 175], [120, 247, 25], [208, 22, 121], [250, 61, 16], [72, 67, 86], [27, 237, 107], [146, 5, 37], [253, 214, 240], [82, 216, 155], [185, 69, 249], [206, 173, 144], [125, 207, 151], [12, 9, 48], [52, 167, 190], [254, 142, 121], [177, 231, 62], [37, 198, 7], [143, 106, 0], [67, 28, 212], [4, 21, 83], [98, 56, 6], [0, 18, 155], [98, 184, 120], [92, 66, 255], [244, 123, 37], [90, 50, 110], [123, 135, 192], [26, 172, 21], [231, 113, 61], [234, 131, 50], [58, 207, 85], [173, 192, 22], [206, 12, 15], [64, 3, 118], [84, 5, 245], [126, 94, 149], [160, 194, 194], [155, 185, 13], [186, 74, 115], [22, 126, 10], [8, 212, 119], [132, 190, 101], [203, 156, 189], [96, 4, 178], [182, 160, 153], [10, 31, 197], [243, 252, 194], [43, 19, 39], [43, 219, 135], [61, 73, 115], [35, 150, 136], [78, 131, 105], [209, 159, 214], [235, 133, 218], [2, 140, 32], [166, 156, 216], [46, 1, 78], [238, 42, 145], [38, 15, 148], [117, 192, 216], [89, 143, 238], [86, 92, 168], [166, 148, 187], [209, 98, 85], [202, 45, 175], [184, 253, 251], [128, 241, 128], [13, 142, 135], [67, 74, 181], [205, 175, 84], [43, 29, 78], [2, 15, 168], [148, 98, 158], [249, 73, 168], [190, 151, 172], [185, 1, 156], [220, 139, 201], [161, 71, 95], [218, 100, 165], [112, 42, 150], [21, 190, 218], [162, 67, 88], [78, 161, 60], [193, 183, 42], [60, 75, 182], [198, 114, 133], [87, 255, 95], [56, 86, 27], [39, 52, 198], [212, 83, 157], [164, 192, 65], [125, 25, 210], [20, 205, 241], [135, 162, 108], [65, 120, 137], [154, 135, 238], [66, 24, 85], [198, 167, 218], [56, 100, 49], [68, 167, 77], [41, 6, 67], [131, 8, 230], [84, 253, 33], [66, 141, 166], [114, 55, 46], [133, 171, 103], [237, 64, 49], [222, 146, 247], [35, 135, 105], [20, 235, 37], [34, 27, 100], [19, 175, 87], [238, 176, 183], [46, 101, 44], [247, 200, 112], [5, 128, 145], [114, 210, 186], [115, 46, 219], [122, 115, 47], [199, 155, 221], [170, 228, 166], [200, 129, 87], [102, 175, 242], [153, 216, 192], [143, 236, 139], [107, 244, 24], [76, 148, 152], [152, 128, 65], [71, 103, 66], [203, 151, 239], [233, 101, 33], [235, 4, 47], [201, 184, 172], [222, 134, 38], [10, 33, 233], [67, 49, 119], [19, 177, 21], [224, 145, 166], [64, 226, 149], [70, 11, 104], [45, 241, 44], [1, 123, 29], [159, 39, 149], [94, 36, 255], [24, 46, 68], [136, 218, 84], [144, 195, 82], [170, 149, 161], [54, 222, 93], [133, 124, 8], [27, 216, 88], [199, 145, 129], [22, 74, 137], [12, 19, 225], [191, 31, 106], [15, 12, 158], [179, 87, 36], [59, 104, 12], [206, 56, 145], [168, 130, 53], [48, 136, 73], [123, 240, 254], [146, 233, 229], [239, 185, 93], [157, 243, 167], [35, 169, 57], [56, 36, 119], [186, 199, 21], [241, 61, 171], [136, 83, 45], [36, 160, 106], [34, 80, 226], [194, 40, 182], [85, 148, 161], [91, 25, 68], [143, 204, 156], [226, 163, 108], [72, 240, 235], [34, 19, 12], [54, 106, 114], [114, 243, 114], [73, 36, 175], [189, 27, 113], [80, 212, 230], [230, 55, 85], [55, 59, 55], [93, 173, 186], [168, 139, 167], [44, 45, 49], [55, 194, 26], [55, 27, 203], [28, 173, 96], [223, 111, 87], [131, 56, 245], [212, 125, 170], [41, 135, 2], [94, 126, 178], [90, 133, 214], [211, 162, 253], [155, 237, 155], [245, 89, 248], [50, 38, 146], [173, 119, 236], [107, 230, 95], [117, 219, 217], [157, 99, 213], [197, 24, 169], [249, 202, 41], [175, 131, 250], [103, 119, 186], [156, 134, 146], [246, 11, 29], [183, 108, 204], [10, 100, 35], [115, 16, 191], [122, 109, 96], [138, 204, 239], [58, 8, 31], [35, 160, 66], [35, 47, 109], [37, 127, 129], [219, 179, 35], [191, 40, 80], [193, 105, 22], [182, 81, 65], [218, 125, 3], [98, 101, 243], [36, 154, 139], [52, 247, 53], [225, 225, 17], [144, 218, 82], [200, 45, 165], [42, 28, 135], [61, 13, 158], [160, 162, 118], [173, 228, 114], [102, 246, 221], [58, 188, 206], [163, 140, 142], [241, 249, 239], [2, 163, 64], [42, 107, 6], [207, 99, 108], [5, 127, 247], [49, 182, 244], [153, 98, 81], [15, 253, 104], [111, 212, 196], [170, 228, 105], [154, 253, 236], [237, 84, 2], [96, 146, 12], [243, 152, 97], [220, 124, 141], [137, 212, 169], [165, 239, 20], [13, 206, 198], [58, 95, 88], [131, 141, 192], [128, 18, 102], [226, 10, 245], [135, 149, 188], [121, 180, 59], [183, 223, 217], [253, 55, 109], [201, 165, 99], [252, 33, 33], [229, 20, 55], [84, 216, 205], [150, 222, 125], [145, 242, 162], [17, 16, 221], [49, 33, 3], [144, 204, 207], [243, 18, 246], [39, 103, 4], [56, 147, 148], [65, 38, 172], [185, 69, 93], [155, 41, 217], [14, 211, 106], [92, 36, 196], [73, 141, 200], [206, 181, 126], [241, 126, 177], [100, 124, 190], [63, 162, 96], [82, 56, 131], [92, 141, 62], [254, 72, 45], [87, 160, 199], [15, 155, 12], [116, 117, 126], [140, 232, 2], [220, 20, 5], [180, 224, 55], [18, 112, 103], [103, 3, 44], [157, 89, 144], [56, 116, 186], [229, 54, 190], [202, 12, 232], [196, 11, 164], [149, 67, 116], [103, 106, 124], [201, 175, 29], [109, 235, 170], [63, 169, 29], [224, 239, 153], [222, 155, 193], [223, 199, 222], [45, 89, 220], [113, 237, 63], [67, 27, 151], [50, 35, 110], [36, 85, 134], [46, 45, 143], [24, 108, 149], [50, 75, 186], [134, 83, 44], [40, 48, 106], [241, 253, 253], [4, 250, 58], [25, 131, 215], [41, 80, 246], [136, 153, 157], [158, 98, 1], [33, 22, 251], [201, 97, 212], [132, 38, 174], [239, 179, 65], [161, 225, 200], [143, 65, 246], [5, 49, 241], [71, 38, 235], [69, 207, 150], [53, 50, 48], [192, 90, 221], [116, 144, 244], [57, 208, 112], [222, 110, 181], [74, 58, 120], [38, 77, 252], [232, 242, 213], [149, 126, 143], [134, 86, 114], [255, 24, 54], [243, 49, 119], [151, 202, 177], [68, 101, 255], [52, 65, 69], [47, 2, 218], [96, 60, 234], [107, 10, 3], [205, 93, 157], [234, 149, 44], [232, 92, 50], [202, 153, 103], [137, 53, 71], [125, 52, 98], [174, 209, 43], [234, 103, 219], [79, 160, 5], [111, 68, 32], [129, 104, 18], [247, 171, 134], [155, 80, 94], [121, 53, 181], [51, 111, 189], [194, 47, 48], [187, 249, 230], [50, 190, 45], [139, 209, 221], [120, 170, 97], [238, 163, 32], [18, 190, 153], [13, 137, 126], [176, 166, 143], [203, 167, 118], [236, 166, 102], [252, 246, 28], [250, 68, 206], [73, 84, 30], [48, 174, 159], [243, 233, 224], [155, 63, 19], [213, 121, 79], [246, 111, 62], [113, 3, 170], [146, 250, 26], [126, 240, 137], [166, 157, 214], [107, 81, 163], [107, 218, 196], [154, 127, 125], [17, 175, 104], [158, 195, 150], [107, 152, 59], [246, 42, 21], [10, 241, 16], [81, 188, 109], [156, 20, 74], [66, 131, 199], [83, 148, 78], [30, 11, 250], [144, 44, 201], [221, 49, 13], [39, 56, 75], [133, 191, 103], [226, 250, 131], [208, 90, 205], [31, 144, 56], [185, 111, 164], [129, 166, 107], [113, 212, 244], [32, 161, 91], [251, 211, 98], [48, 146, 25], [151, 75, 125], [107, 58, 39], [114, 137, 93], [36, 63, 35], [123, 217, 240], [64, 193, 28], [79, 109, 77], [196, 188, 105], [174, 133, 203], [128, 10, 29], [141, 130, 90], [95, 231, 86], [149, 158, 200], [163, 243, 233], [20, 36, 164], [67, 104, 241], [170, 111, 91], [100, 162, 99], [88, 52, 87], [164, 158, 65], [251, 112, 42], [215, 207, 94], [120, 22, 12], [36, 125, 104], [248, 29, 32], [244, 126, 65], [50, 243, 40], [78, 206, 52], [116, 112, 79], [1, 228, 207], [133, 124, 62], [62, 131, 8], [159, 232, 68], [196, 96, 110], [17, 139, 99], [53, 68, 11], [252, 102, 222], [26, 134, 136], [179, 127, 255], [182, 113, 65], [75, 237, 245], [10, 200, 103], [83, 58, 164], [5, 122, 146], [177, 132, 34], [139, 227, 20], [179, 201, 26], [77, 97, 116], [226, 189, 81], [61, 226, 49], [64, 72, 110], [25, 92, 27], [22, 232, 165], [48, 82, 94], [88, 139, 43], [149, 180, 253], [75, 151, 120], [250, 104, 188], [181, 222, 180], [162, 218, 40], [65, 181, 190], [206, 111, 88], [246, 237, 11], [71, 159, 83], [172, 15, 254], [249, 120, 212], [197, 102, 198], [47, 50, 18], [23, 152, 116], [77, 116, 51], [109, 79, 168], [49, 36, 183], [195, 177, 241], [132, 99, 100], [12, 174, 96], [163, 179, 221], [126, 147, 181], [47, 73, 8], [30, 146, 78], [149, 194, 56], [117, 228, 194], [186, 81, 164], [31, 18, 94], [204, 181, 154], [140, 225, 24], [99, 223, 24], [75, 183, 109], [148, 131, 236], [20, 159, 171], [99, 22, 132], [255, 188, 128], [194, 63, 184], [156, 89, 160], [202, 16, 182], [153, 16, 237], [88, 132, 130], [181, 33, 180], [38, 146, 205], [154, 250, 36], [145, 71, 6], [71, 166, 203], [81, 15, 53], [245, 106, 190], [246, 176, 217], [30, 64, 247], [15, 133, 186], [40, 240, 220], [87, 248, 235], [253, 69, 81], [130, 17, 232], [48, 24, 106], [16, 255, 253], [150, 116, 143], [139, 250, 222], [226, 52, 139], [205, 231, 244], [118, 212, 9], [231, 151, 155], [247, 152, 222], [110, 140, 40], [115, 15, 115], [21, 28, 72], [249, 187, 184], [186, 94, 206], [213, 193, 85], [156, 6, 222], [232, 127, 172], [109, 48, 21], [235, 110, 188], [45, 92, 44], [187, 44, 103], [60, 205, 183], [135, 87, 99], [232, 130, 24], [2, 102, 231], [154, 125, 19], [243, 222, 0], [131, 17, 202], [232, 228, 19], [226, 43, 50], [234, 73, 138], [138, 0, 172], [28, 154, 212], [137, 249, 133], [210, 122, 223], [148, 79, 156], [237, 209, 115], [51, 51, 191], [66, 56, 135], [249, 133, 167], [122, 30, 203], [186, 218, 244], [171, 165, 183], [37, 199, 163], [181, 172, 237], [7, 43, 128], [103, 149, 170], [219, 59, 171], [109, 244, 204], [87, 141, 142], [131, 251, 243], [205, 3, 81], [155, 78, 102], [156, 129, 212], [140, 164, 170], [223, 118, 85], [236, 67, 247], [76, 27, 221], [226, 190, 84], [4, 15, 163], [72, 183, 193], [225, 221, 177], [55, 247, 136], [200, 70, 105], [93, 223, 171], [20, 59, 98], [118, 188, 30], [67, 218, 9], [2, 224, 78], [11, 184, 60], [117, 83, 252], [19, 109, 110], [1, 187, 71], [203, 25, 33], [139, 175, 209], [202, 167, 26], [28, 83, 155], [80, 101, 105], [33, 234, 139], [115, 246, 122], [212, 218, 156], [235, 167, 153], [185, 83, 33], [71, 180, 71], [128, 92, 217], [63, 87, 232], [28, 237, 44], [93, 151, 228], [157, 137, 35], [209, 175, 109], [60, 58, 94], [16, 159, 211], [213, 23, 200], [12, 187, 196], [39, 180, 61], [23, 11, 235], [13, 85, 164], [155, 11, 136], [166, 195, 255], [21, 97, 37], [156, 162, 254], [149, 175, 66], [126, 59, 2], [61, 125, 81], [210, 238, 225], [232, 183, 136], [127, 179, 229], [87, 163, 244], [120, 9, 208], [248, 60, 1], [53, 30, 247], [254, 190, 10], [102, 79, 185], [171, 193, 59], [226, 45, 43], [109, 159, 30], [154, 165, 100], [27, 190, 123], [164, 148, 28], [1, 204, 81], [250, 31, 75], [44, 160, 198], [100, 111, 89], [96, 28, 236], [223, 19, 149], [50, 182, 232], [49, 244, 118], [211, 139, 28], [199, 187, 238], [36, 147, 219], [37, 93, 65], [33, 231, 229], [65, 137, 249], [11, 203, 203], [124, 146, 17], [132, 143, 83], [40, 86, 186], [27, 70, 220], [166, 243, 138], [4, 171, 144], [33, 45, 213], [253, 57, 59], [5, 50, 37], [194, 228, 171], [56, 58, 15], [198, 62, 163], [214, 220, 182], [89, 51, 185], [153, 105, 6], [151, 37, 203], [139, 102, 171], [177, 221, 27], [61, 243, 241], [11, 243, 202], [84, 25, 89], [60, 127, 239], [164, 94, 35], [186, 77, 102], [51, 108, 18], [204, 75, 164], [112, 225, 171], [130, 142, 99], [54, 3, 165], [198, 16, 249], [190, 72, 99], [35, 83, 58], [51, 88, 80], [123, 88, 251], [6, 133, 227], [117, 63, 206], [215, 127, 127], [14, 3, 236], [77, 194, 167], [181, 180, 209], [97, 120, 16], [194, 193, 159], [89, 232, 165], [248, 110, 151], [54, 167, 88], [184, 32, 12], [95, 118, 69], [219, 72, 119], [128, 53, 70], [29, 161, 77], [198, 205, 96], [233, 72, 62], [158, 248, 206], [190, 162, 134], [249, 96, 64], [184, 174, 198], [161, 158, 188], [64, 215, 219], [167, 245, 195], [76, 101, 64], [92, 214, 30], [41, 43, 224], [218, 87, 226], [148, 233, 151], [49, 72, 137], [142, 189, 232], [226, 135, 154], [107, 110, 53], [51, 83, 90], [56, 19, 20], [225, 176, 16], [64, 65, 98], [113, 226, 254], [232, 166, 173], [8, 157, 117], [164, 100, 25], [35, 167, 145], [192, 243, 186], [7, 57, 12], [140, 245, 179], [23, 25, 122], [30, 110, 0], [245, 40, 99], [254, 39, 232], [110, 94, 100], [132, 1, 183], [189, 47, 76], [95, 118, 71], [243, 4, 39], [193, 8, 162], [51, 12, 82], [134, 5, 106], [69, 1, 48], [223, 66, 36], [28, 248, 212], [165, 203, 48], [177, 127, 164], [201, 43, 182], [83, 156, 192], [177, 43, 253], [27, 104, 10], [206, 82, 164], [29, 37, 226], [140, 29, 55], [22, 133, 154], [21, 212, 87], [67, 128, 242], [98, 153, 78], [181, 79, 97], [69, 32, 28], [10, 4, 229], [9, 53, 142], [155, 70, 139], [206, 72, 187], [9, 120, 116], [181, 89, 235], [190, 160, 16], [94, 174, 149], [209, 153, 222], [157, 136, 229], [196, 158, 199], [232, 29, 87], [185, 133, 49], [92, 95, 231], [32, 128, 175], [6, 181, 114], [243, 49, 231], [113, 147, 82], [123, 46, 87], [105, 201, 87], [59, 226, 85], [128, 32, 13], [1, 116, 126], [91, 4, 77], [170, 193, 191], [192, 159, 66], [123, 120, 110], [169, 40, 233], [155, 204, 109], [64, 196, 54], [21, 201, 65], [167, 91, 63], [230, 96, 160], [205, 215, 172], [82, 205, 190], [127, 211, 77], [149, 211, 145], [162, 215, 222], [51, 40, 154], [212, 171, 123], [186, 37, 248], [41, 124, 252], [212, 166, 5], [120, 210, 131], [205, 14, 206], [138, 136, 94], [177, 75, 217], [46, 183, 148], [150, 63, 86], [95, 150, 206], [196, 182, 236], [235, 8, 111], [144, 25, 32], [159, 5, 241], [147, 205, 176], [219, 39, 190], [25, 3, 163], [51, 195, 227], [217, 15, 157], [152, 238, 101], [184, 134, 159], [103, 247, 111], [189, 123, 203], [203, 160, 133], [212, 128, 241], [135, 200, 235], [124, 185, 162], [43, 48, 33], [207, 129, 33], [201, 160, 106], [239, 36, 249], [185, 136, 113], [10, 134, 99], [91, 32, 241], [28, 239, 215], [192, 58, 54], [141, 65, 48], [200, 136, 255], [11, 249, 237], [198, 57, 57], [231, 252, 250], [48, 141, 97], [191, 10, 178], [192, 110, 10], [183, 129, 185], [57, 14, 99], [141, 194, 59], [76, 110, 245], [134, 197, 207], [247, 9, 188], [47, 122, 179], [164, 233, 181], [35, 147, 74], [135, 181, 81], [207, 232, 21], [158, 78, 85], [157, 182, 42], [200, 248, 82], [127, 37, 102], [204, 176, 21], [134, 107, 165], [47, 113, 48], [29, 119, 236], [232, 14, 74], [5, 250, 88], [175, 234, 115], [122, 45, 19], [93, 212, 82], [137, 210, 128], [133, 47, 10], [41, 177, 66], [147, 4, 103], [67, 80, 6], [201, 44, 193], [212, 216, 6], [63, 243, 65], [124, 200, 28], [61, 185, 26], [155, 235, 148], [30, 62, 18], [140, 88, 133], [252, 197, 110], [196, 158, 59], [119, 230, 162], [92, 53, 32], [249, 109, 123], [144, 180, 181], [95, 237, 27], [132, 81, 130], [158, 73, 218], [3, 226, 12], [40, 94, 22], [125, 95, 167], [29, 99, 43], [54, 222, 138], [92, 90, 231], [33, 221, 168], [196, 70, 182], [235, 151, 239], [49, 208, 217], [220, 88, 195], [113, 112, 70], [217, 136, 185], [166, 228, 216], [238, 103, 129], [177, 58, 146], [237, 8, 87], [173, 140, 112], [44, 45, 103], [162, 157, 71], [116, 108, 124], [201, 65, 209], [118, 160, 119], [56, 203, 59], [201, 229, 78], [90, 70, 179], [37, 235, 6], [17, 119, 192], [171, 69, 78], [108, 216, 105], [117, 149, 154], [58, 232, 172], [217, 202, 21], [140, 94, 111], [8, 12, 232], [107, 250, 91], [193, 54, 218], [230, 203, 50], [43, 188, 167], [61, 174, 162], [200, 161, 198], [116, 203, 5], [52, 192, 122], [176, 12, 195], [240, 100, 1], [214, 68, 130], [253, 136, 44], [11, 207, 135], [143, 209, 194], [205, 50, 119], [115, 201, 109], [5, 4, 231], [74, 143, 212], [161, 143, 45], [145, 188, 247], [39, 24, 213], [168, 105, 98], [26, 154, 10], [178, 213, 209], [199, 5, 190], [251, 151, 143], [121, 47, 160], [190, 226, 137], [185, 125, 91], [17, 15, 253], [79, 219, 143], [59, 143, 253], [183, 4, 58], [5, 184, 216], [16, 17, 98], [247, 251, 173], [78, 170, 169], [98, 245, 245], [79, 89, 69], [243, 77, 41], [210, 118, 118], [185, 191, 228], [157, 4, 205], [14, 94, 160], [100, 184, 115], [4, 136, 18], [103, 184, 229], [177, 12, 80], [126, 244, 95], [47, 23, 59], [163, 145, 159], [115, 190, 97], [70, 227, 27], [15, 100, 220], [51, 231, 147], [53, 20, 254], [70, 143, 136], [232, 58, 211], [99, 227, 219], [98, 26, 32], [254, 19, 57], [140, 48, 21], [18, 130, 249], [143, 124, 151], [170, 171, 74], [153, 207, 114], [64, 64, 91], [120, 107, 39], [114, 209, 35], [200, 70, 133], [236, 185, 76], [209, 117, 186], [21, 236, 224], [163, 165, 179], [13, 251, 241], [203, 1, 128], [74, 117, 140], [100, 190, 250], [225, 50, 245], [196, 91, 99], [228, 73, 227], [109, 23, 186], [5, 37, 47], [189, 210, 112], [62, 86, 197], [5, 49, 67], [106, 34, 31], [23, 68, 235], [228, 214, 255], [101, 80, 114], [90, 94, 16], [224, 76, 221], [102, 106, 177], [76, 240, 24], [18, 70, 204], [159, 236, 93], [41, 87, 33], [63, 65, 192], [50, 53, 13], [133, 76, 199], [60, 102, 18], [206, 121, 43], [3, 24, 227], [152, 229, 239], [76, 118, 153], [184, 195, 224], [5, 160, 186], [255, 236, 196], [74, 249, 223], [166, 147, 194], [131, 202, 38], [152, 242, 245], [244, 88, 133], [229, 229, 57], [7, 236, 82], [186, 108, 5], [47, 7, 177], [228, 12, 9], [120, 205, 20], [113, 139, 255], [84, 217, 251], [44, 97, 164], [22, 252, 117], [147, 140, 34], [251, 137, 189], [244, 174, 1], [117, 197, 231], [173, 124, 6], [44, 171, 160], [158, 137, 38], [219, 33, 75], [102, 24, 144], [179, 88, 254], [245, 88, 219], [127, 146, 83], [251, 15, 213], [127, 5, 227], [227, 36, 144], [92, 61, 16], [138, 221, 208], [183, 49, 56], [159, 124, 171], [191, 27, 84], [219, 8, 225], [76, 116, 158], [155, 111, 97], [95, 209, 83], [210, 64, 130], [209, 99, 125], [195, 119, 47], [234, 175, 222], [28, 149, 244], [203, 147, 105], [164, 120, 250], [95, 246, 168], [116, 207, 224], [129, 107, 211], [225, 129, 112], [153, 44, 108], [124, 213, 85], [125, 135, 196], [8, 52, 51], [70, 88, 73], [188, 84, 12], [196, 152, 8], [197, 83, 212], [182, 111, 167], [185, 81, 236], [116, 78, 209], [20, 152, 36], [245, 55, 143], [22, 96, 171], [167, 167, 21], [241, 158, 158], [87, 112, 188], [44, 41, 228], [60, 119, 237], [15, 182, 243], [146, 164, 40], [75, 21, 101], [147, 144, 103], [206, 30, 156], [112, 139, 20], [210, 187, 122], [204, 123, 170], [228, 90, 61], [151, 120, 203], [54, 46, 58], [128, 199, 228], [93, 244, 21], [89, 184, 90], [211, 29, 153], [133, 185, 70], [161, 31, 147], [142, 65, 183], [65, 5, 192], [177, 89, 61], [48, 135, 160], [165, 104, 199], [170, 25, 119], [135, 98, 199], [172, 111, 54], [53, 145, 45], [94, 61, 44], [103, 215, 43], [88, 146, 12], [103, 103, 250], [139, 82, 209], [22, 225, 79], [190, 63, 104], [117, 42, 13], [127, 128, 3], [46, 52, 140], [83, 100, 160], [128, 189, 143], [95, 35, 49], [36, 207, 144], [232, 191, 236], [83, 54, 93], [171, 176, 233], [187, 221, 231], [129, 113, 15], [213, 243, 14], [153, 196, 156], [210, 95, 29], [103, 154, 163], [110, 197, 222], [5, 41, 11], [26, 76, 160], [255, 44, 19], [94, 136, 182], [137, 191, 116], [9, 42, 89], [214, 239, 104], [67, 196, 40], [34, 145, 198], [4, 37, 209], [127, 245, 52], [93, 252, 64], [219, 133, 217], [195, 218, 185], [236, 56, 78], [72, 120, 174], [152, 74, 133], [131, 116, 152], [10, 186, 16], [10, 87, 207], [198, 53, 246], [219, 99, 153], [39, 130, 150], [137, 11, 100], [86, 16, 50], [252, 236, 77], [4, 7, 230], [44, 186, 84], [65, 57, 215], [61, 8, 96], [156, 165, 67], [174, 132, 31], [177, 42, 167], [242, 180, 79], [228, 34, 134], [110, 165, 22], [63, 14, 239], [194, 195, 27], [27, 88, 41], [217, 74, 65], [101, 139, 142], [72, 25, 58], [162, 92, 76], [112, 254, 163], [144, 31, 44], [7, 232, 114], [60, 197, 20], [131, 216, 149], [199, 31, 165], [198, 225, 95], [50, 84, 4], [68, 182, 227], [249, 210, 230], [177, 163, 5], [128, 33, 195], [6, 239, 235], [8, 49, 12], [214, 69, 255], [239, 162, 87], [126, 72, 109], [204, 223, 80], [194, 187, 175], [200, 71, 184], [11, 212, 137], [113, 78, 84], [90, 102, 214], [29, 53, 1], [161, 60, 15], [60, 99, 180], [133, 253, 242], [88, 127, 56], [118, 114, 108], [227, 82, 157], [172, 107, 228], [123, 50, 152], [158, 214, 65], [100, 212, 53], [83, 175, 69], [232, 110, 156], [249, 31, 143], [254, 254, 80], [164, 233, 133], [48, 215, 194], [118, 217, 73], [236, 99, 30], [159, 99, 5], [211, 169, 96], [9, 80, 225], [146, 64, 180], [80, 138, 173], [123, 247, 186], [69, 20, 4], [92, 243, 116], [123, 101, 2], [18, 25, 83], [228, 143, 223], [253, 31, 49], [198, 62, 82], [204, 94, 80], [117, 235, 16], [188, 106, 219], [178, 242, 106], [68, 170, 67], [48, 166, 11], [55, 233, 118], [78, 180, 147], [232, 161, 229], [28, 156, 99], [232, 251, 159], [189, 86, 158], [241, 24, 243], [224, 105, 116], [71, 214, 246], [244, 11, 145], [99, 237, 230], [244, 52, 4], [192, 191, 121], [89, 167, 246], [199, 177, 22], [179, 166, 182], [130, 202, 237], [222, 130, 147], [89, 97, 13], [76, 161, 122], [217, 241, 219], [28, 134, 34], [123, 9, 220], [185, 125, 176], [83, 51, 203], [186, 35, 133], [152, 11, 171], [15, 40, 46], [161, 192, 60], [100, 69, 222], [148, 244, 156], [38, 149, 161], [99, 65, 155], [230, 213, 17], [75, 176, 217], [49, 159, 246], [41, 140, 55], [130, 242, 221], [115, 77, 183], [76, 208, 105], [149, 25, 203], [44, 67, 197], [96, 2, 38], [119, 22, 221], [10, 30, 250], [203, 238, 58], [12, 95, 123], [156, 224, 163], [167, 230, 106], [82, 181, 148], [207, 96, 79], [55, 193, 154], [128, 207, 94], [186, 234, 253], [88, 106, 210], [43, 119, 158], [168, 237, 204], [81, 83, 211], [214, 168, 162], [138, 244, 247], [227, 224, 155], [91, 178, 219], [191, 177, 2], [62, 195, 199], [59, 226, 25], [128, 10, 237], [20, 29, 7], [219, 14, 219], [204, 91, 63], [6, 83, 215], [10, 120, 183], [135, 123, 221], [84, 96, 222], [219, 36, 80], [150, 233, 214], [202, 60, 23], [27, 146, 143], [87, 112, 1], [192, 179, 240], [176, 53, 26], [230, 12, 0], [82, 113, 225], [88, 184, 135], [75, 198, 80], [194, 37, 32], [29, 199, 81], [199, 87, 94], [157, 122, 146], [195, 37, 93], [81, 147, 24], [164, 42, 155], [122, 81, 5], [85, 150, 170], [48, 52, 46], [91, 91, 57], [205, 125, 123], [80, 156, 230], [245, 214, 60], [36, 117, 167], [75, 118, 104], [137, 200, 161], [75, 233, 139], [29, 142, 171], [241, 46, 254], [111, 83, 220], [109, 26, 226], [59, 70, 144], [172, 76, 228], [209, 66, 12], [66, 109, 114], [128, 90, 110], [103, 110, 228], [73, 126, 25], [91, 7, 210], [235, 35, 58], [131, 16, 88], [79, 232, 16], [243, 3, 175], [225, 255, 112], [154, 32, 55], [95, 190, 201], [91, 61, 195], [147, 235, 59], [125, 179, 40], [233, 241, 169], [222, 86, 125], [147, 65, 198], [50, 204, 78], [206, 234, 100], [191, 62, 236], [81, 115, 215], [219, 112, 199], [85, 138, 24], [238, 151, 124], [242, 131, 16], [68, 99, 250], [155, 113, 203], [82, 33, 186], [183, 226, 63], [185, 221, 120], [29, 191, 116], [177, 117, 243], [178, 47, 29], [248, 53, 248], [73, 20, 212], [25, 19, 179], [144, 158, 146], [67, 184, 66], [72, 239, 96], [206, 175, 25], [73, 98, 168], [196, 91, 120], [28, 206, 42], [172, 54, 216], [168, 118, 248], [245, 81, 215], [253, 48, 81], [127, 176, 201], [133, 161, 151], [235, 21, 207], [126, 39, 198], [67, 241, 89], [25, 161, 97], [252, 253, 74], [88, 52, 201], [83, 97, 0], [96, 37, 69], [39, 176, 214], [99, 86, 199], [83, 205, 27], [16, 65, 177], [249, 22, 95], [190, 12, 168], [144, 116, 142], [108, 153, 7], [165, 155, 60], [83, 155, 255], [216, 156, 10], [131, 209, 49], [105, 174, 31], [140, 248, 146], [253, 158, 88], [22, 2, 170], [3, 107, 99], [171, 163, 40], [226, 196, 191], [154, 26, 103], [244, 102, 136], [91, 116, 21], [139, 127, 220], [163, 191, 191], [21, 102, 255], [227, 210, 35], [27, 215, 149], [7, 114, 171], [27, 249, 128], [114, 40, 21], [146, 105, 12], [120, 33, 70], [172, 224, 224], [237, 27, 253], [240, 26, 151], [250, 14, 106], [154, 80, 180], [29, 70, 102], [129, 34, 0], [74, 185, 123], [205, 251, 204], [85, 217, 63], [130, 170, 187], [248, 155, 122], [248, 156, 70], [120, 46, 26], [130, 213, 201], [10, 244, 166], [142, 188, 103], [85, 130, 177], [64, 107, 229], [97, 161, 101], [134, 90, 216], [156, 164, 4], [244, 167, 65], [225, 251, 5], [142, 106, 166], [195, 234, 238], [199, 28, 60], [78, 151, 176], [210, 40, 222], [202, 185, 74], [216, 14, 14], [154, 119, 92], [89, 138, 23], [2, 2, 121], [84, 95, 99], [70, 159, 69], [250, 126, 67], [46, 53, 73], [26, 57, 103], [85, 73, 75], [20, 71, 32], [167, 83, 188], [48, 146, 119], [122, 223, 178], [194, 76, 38], [38, 86, 213], [215, 181, 112], [152, 180, 144], [224, 5, 215], [215, 51, 100], [9, 74, 44], [74, 205, 49], [60, 255, 50], [217, 48, 182], [149, 115, 233], [80, 182, 30], [143, 11, 92], [101, 211, 166], [245, 213, 32], [161, 245, 107], [228, 2, 195], [90, 36, 132], [232, 24, 144], [225, 180, 148], [229, 221, 81], [217, 227, 95], [27, 184, 148], [31, 40, 134], [104, 99, 53], [245, 135, 48], [101, 156, 145], [241, 36, 173], [8, 6, 217], [100, 245, 121], [147, 59, 93], [213, 134, 41], [102, 131, 10], [103, 133, 110], [158, 0, 16], [165, 53, 51], [201, 76, 195], [245, 118, 18], [222, 200, 183], [254, 82, 54], [112, 249, 205], [221, 105, 48], [116, 218, 62], [249, 244, 167], [148, 129, 62], [159, 6, 163], [245, 15, 124], [182, 63, 98], [238, 127, 165], [102, 131, 160], [214, 25, 31], [138, 149, 157], [210, 124, 0], [136, 18, 217], [34, 172, 190], [227, 176, 24], [112, 76, 176], [250, 200, 90], [28, 100, 61], [170, 212, 57], [198, 110, 118], [124, 4, 46], [173, 188, 188], [140, 26, 245], [13, 205, 235], [6, 78, 72], [192, 27, 238], [122, 61, 60], [152, 51, 108], [0, 102, 221], [143, 113, 180], [30, 88, 26], [140, 125, 245], [175, 86, 34], [91, 249, 50], [132, 169, 87], [20, 210, 241], [101, 194, 36], [235, 61, 89], [242, 111, 83], [83, 54, 180], [75, 45, 55], [37, 82, 53], [58, 223, 134], [62, 71, 105], [194, 129, 6], [17, 198, 166], [104, 99, 65], [239, 249, 248], [75, 227, 42], [41, 247, 88], [67, 117, 59], [117, 225, 29], [92, 95, 18], [174, 156, 42], [132, 74, 75], [18, 45, 3], [12, 98, 72], [162, 12, 181], [132, 248, 0], [35, 71, 161], [234, 77, 82], [111, 6, 26], [106, 127, 158], [197, 70, 195], [2, 201, 68], [229, 133, 124], [106, 101, 132], [246, 84, 23], [115, 180, 130], [99, 160, 128], [139, 77, 167], [254, 131, 120], [255, 147, 64], [102, 4, 178], [106, 20, 231], [125, 248, 115], [51, 80, 76], [72, 166, 85], [243, 47, 45], [136, 157, 83], [59, 49, 145], [120, 228, 75], [45, 51, 8], [63, 108, 244], [131, 10, 93], [184, 136, 93], [220, 55, 138], [19, 243, 142], [44, 210, 97], [249, 144, 76], [14, 53, 176], [192, 40, 158], [224, 80, 216], [26, 29, 237], [156, 190, 20], [215, 200, 10], [69, 81, 28], [168, 58, 130], [209, 231, 77], [49, 229, 51], [32, 205, 94], [250, 242, 46], [114, 239, 240], [3, 14, 191], [128, 152, 228], [8, 127, 204], [117, 215, 203], [140, 184, 8], [210, 202, 205], [111, 228, 49], [152, 42, 253], [240, 78, 110], [253, 251, 122], [111, 254, 62], [229, 101, 240], [197, 76, 209], [62, 61, 231], [121, 193, 255], [251, 115, 189], [88, 239, 136], [44, 181, 164], [45, 209, 69], [12, 105, 200], [191, 79, 140], [215, 61, 126], [250, 85, 49], [12, 193, 225], [9, 25, 148], [120, 94, 94], [8, 158, 151], [8, 206, 134], [86, 157, 45], [167, 198, 95], [13, 100, 59], [87, 195, 99], [47, 204, 36], [63, 254, 253], [61, 170, 107], [94, 196, 86], [0, 89, 142], [75, 57, 160], [203, 18, 36], [159, 213, 143], [199, 109, 234], [182, 106, 156], [251, 138, 250], [45, 137, 176], [62, 171, 176], [228, 78, 26], [21, 68, 58], [4, 19, 35], [185, 119, 19], [207, 91, 171], [70, 236, 105], [40, 149, 154], [102, 145, 29], [55, 9, 194], [138, 230, 115], [245, 104, 79], [156, 103, 187], [34, 177, 138], [246, 179, 211], [92, 69, 124], [79, 22, 118], [120, 221, 64], [129, 131, 254], [222, 216, 13], [72, 119, 239], [130, 26, 102], [31, 163, 131], [187, 31, 117], [74, 135, 79], [21, 115, 161], [100, 69, 66], [228, 154, 19], [165, 187, 34], [221, 205, 173], [253, 132, 206], [2, 140, 42], [254, 175, 168], [111, 15, 252], [87, 156, 36], [161, 27, 10], [217, 109, 24], [122, 67, 22], [185, 132, 88], [219, 137, 64], [137, 69, 9], [148, 23, 234], [19, 5, 75], [96, 239, 73], [171, 238, 136], [151, 12, 230], [98, 225, 23], [155, 100, 197], [162, 132, 49], [7, 218, 72], [253, 251, 241], [146, 92, 132], [57, 183, 206], [118, 60, 109], [141, 151, 227], [115, 18, 23], [155, 138, 52], [251, 78, 207], [38, 97, 200], [110, 22, 148], [209, 231, 152], [83, 71, 89], [124, 44, 113], [208, 231, 205], [150, 49, 114], [216, 64, 175], [148, 244, 82], [167, 57, 165], [38, 116, 78], [104, 207, 71], [204, 15, 136], [5, 164, 181], [208, 114, 60], [21, 181, 220], [110, 245, 234], [93, 224, 203], [153, 133, 120], [118, 11, 17], [92, 207, 66], [22, 53, 184], [244, 208, 124], [193, 106, 121], [6, 237, 179], [246, 141, 54], [146, 91, 158], [227, 59, 63], [122, 195, 124], [5, 118, 92], [66, 242, 174], [23, 203, 169], [228, 65, 77], [215, 123, 243], [27, 185, 105], [116, 53, 48], [159, 28, 188], [82, 61, 59], [56, 168, 106], [122, 240, 45], [134, 118, 70], [238, 194, 245], [199, 109, 205], [79, 177, 76], [111, 127, 116], [211, 238, 120], [9, 3, 190], [235, 6, 7], [238, 57, 249], [189, 110, 170], [234, 247, 129], [180, 152, 198], [182, 213, 140], [197, 85, 219], [39, 143, 86], [64, 197, 162], [81, 44, 67], [255, 97, 159], [106, 217, 9], [255, 201, 77], [107, 216, 125], [131, 73, 198], [111, 56, 255], [182, 48, 117], [55, 30, 204], [48, 105, 45], [84, 240, 135], [241, 18, 165], [19, 174, 142], [30, 208, 15], [6, 73, 99], [150, 208, 147], [249, 213, 82], [142, 169, 145], [94, 200, 64], [133, 246, 154], [5, 200, 60], [124, 179, 121], [138, 79, 78], [88, 147, 50], [154, 10, 182], [237, 213, 194], [150, 62, 240], [98, 109, 235], [182, 217, 80], [170, 81, 166], [7, 211, 15], [22, 95, 94], [107, 253, 235], [10, 153, 68], [199, 78, 59], [180, 255, 100], [51, 40, 243], [107, 41, 18], [23, 81, 120], [240, 14, 76], [121, 194, 70], [198, 26, 220], [57, 33, 63], [167, 181, 69], [103, 105, 81], [86, 151, 248], [149, 211, 18], [229, 90, 165], [202, 68, 249], [147, 121, 141], [36, 216, 233], [229, 213, 250], [19, 82, 208], [116, 167, 69], [61, 249, 238], [182, 11, 59], [86, 144, 204], [37, 212, 193], [18, 73, 194], [23, 33, 180], [78, 192, 246], [13, 55, 207], [155, 243, 165], [186, 39, 197], [145, 222, 254], [205, 108, 209], [33, 252, 213], [49, 246, 139], [100, 23, 115], [31, 6, 72], [248, 202, 194], [201, 134, 220], [153, 247, 228], [45, 12, 14], [164, 238, 46], [153, 252, 187], [220, 214, 244], [171, 84, 24], [174, 29, 178], [131, 238, 99], [38, 115, 73], [48, 221, 224], [18, 25, 106], [150, 125, 189], [73, 119, 201], [34, 8, 80], [174, 118, 93], [236, 3, 26], [118, 31, 187], [247, 122, 203], [53, 148, 128], [39, 37, 67], [141, 63, 220], [50, 215, 35], [49, 134, 100], [10, 191, 49], [4, 101, 121], [231, 160, 112], [136, 32, 29], [176, 16, 130], [203, 230, 151], [14, 110, 96], [194, 67, 244], [62, 45, 78], [214, 178, 20], [85, 49, 214], [170, 61, 58], [228, 107, 112], [18, 145, 193], [54, 125, 6], [160, 115, 191], [39, 231, 119], [109, 124, 78], [7, 29, 8], [35, 69, 213], [126, 33, 41], [198, 73, 92], [30, 255, 77], [20, 234, 59], [162, 169, 128], [165, 203, 130], [113, 193, 81], [219, 42, 81], [115, 87, 76], [188, 137, 66], [58, 73, 246], [176, 6, 48], [197, 36, 80], [147, 28, 9], [112, 200, 128], [83, 76, 175], [113, 63, 226], [61, 80, 168], [154, 11, 62], [195, 251, 25], [187, 137, 238], [222, 206, 189], [173, 45, 239], [85, 183, 182], [225, 28, 102], [27, 183, 175], [169, 47, 145], [42, 119, 44], [164, 204, 225], [224, 72, 200], [20, 27, 174], [72, 214, 36], [175, 26, 73], [83, 82, 69], [202, 192, 185], [92, 158, 186], [202, 18, 204], [14, 159, 97], [0, 159, 94], [163, 197, 185], [249, 212, 162], [194, 93, 97], [1, 36, 16], [115, 159, 177], [48, 43, 231], [247, 0, 137], [221, 20, 85], [159, 125, 245], [2, 198, 113], [56, 205, 231], [15, 230, 2], [80, 232, 194], [74, 220, 45], [0, 124, 31], [59, 95, 49], [248, 0, 173], [98, 25, 80], [146, 37, 235], [57, 240, 104], [126, 127, 237], [89, 196, 140], [59, 243, 236], [154, 248, 252], [74, 46, 136], [108, 250, 50], [110, 1, 131], [138, 61, 30], [110, 246, 67], [121, 5, 224], [161, 159, 24], [198, 220, 135], [187, 145, 197], [203, 238, 54], [250, 119, 234], [3, 19, 60], [156, 195, 28], [124, 218, 130], [216, 240, 75], [91, 46, 118], [168, 52, 125], [20, 97, 99], [217, 197, 49], [33, 137, 133], [40, 172, 93], [211, 80, 11], [5, 33, 192], [151, 75, 10], [181, 168, 8], [85, 122, 18], [124, 219, 175], [191, 30, 219], [249, 185, 70], [193, 159, 228], [29, 26, 46], [152, 149, 71], [83, 12, 171], [152, 235, 254], [178, 89, 180], [53, 40, 9], [243, 103, 254], [141, 169, 9], [193, 68, 46], [231, 104, 252], [216, 191, 230], [86, 18, 28], [106, 222, 200], [67, 7, 43], [246, 244, 99], [153, 170, 244], [74, 45, 192], [132, 82, 13], [252, 217, 248], [208, 29, 145], [144, 246, 95], [184, 115, 55], [253, 129, 172], [75, 215, 121], [3, 18, 64], [253, 81, 247], [63, 227, 126], [56, 203, 179], [47, 16, 3], [22, 76, 155], [142, 75, 47], [170, 185, 27], [3, 86, 91], [65, 233, 123], [43, 190, 22], [28, 16, 97], [43, 231, 44], [16, 223, 107], [245, 126, 131], [108, 94, 209], [37, 203, 183], [201, 198, 239], [101, 51, 52], [242, 13, 38], [243, 6, 39], [235, 96, 162], [186, 196, 129], [23, 198, 240], [35, 161, 11], [215, 187, 210], [26, 26, 165], [207, 155, 11], [52, 186, 159], [228, 161, 92], [222, 89, 135], [245, 30, 169], [181, 212, 150], [129, 14, 7], [179, 131, 5], [12, 33, 84], [242, 214, 63], [80, 36, 90], [70, 222, 122], [249, 46, 218], [209, 31, 222], [2, 34, 125], [51, 16, 43], [141, 67, 11], [64, 0, 77], [95, 194, 231], [219, 119, 118], [77, 125, 226], [118, 187, 189], [114, 56, 253], [153, 120, 75], [119, 141, 39], [16, 101, 11], [176, 247, 103], [163, 174, 121], [205, 31, 92], [19, 149, 44], [19, 58, 36], [81, 220, 255], [1, 124, 239], [138, 83, 127], [149, 129, 5], [8, 62, 75], [89, 208, 92], [167, 138, 14], [73, 242, 161], [123, 184, 146], [208, 204, 102], [86, 236, 20], [188, 136, 196], [71, 88, 238], [34, 183, 220], [91, 61, 165], [155, 192, 200], [26, 240, 1], [185, 82, 236], [77, 67, 91], [119, 58, 25], [138, 146, 47], [133, 126, 251], [9, 53, 136], [119, 226, 169], [3, 185, 25], [184, 241, 79], [44, 86, 27], [53, 59, 205], [238, 43, 167], [184, 217, 16], [120, 47, 81], [24, 107, 172], [141, 185, 174], [121, 127, 232], [225, 204, 213], [122, 210, 175], [231, 40, 169], [190, 200, 179], [37, 16, 71], [217, 235, 175], [17, 244, 50], [197, 85, 61], [206, 197, 145], [226, 78, 143], [132, 202, 189], [221, 38, 36], [175, 107, 104], [240, 217, 3], [217, 76, 30], [106, 210, 100], [226, 46, 254], [201, 182, 208], [127, 206, 112], [106, 140, 69], [91, 59, 207], [72, 62, 11], [63, 82, 63], [135, 202, 225], [214, 34, 168], [77, 231, 147], [54, 25, 248], [157, 65, 140], [17, 195, 65], [251, 40, 14], [174, 142, 22], [211, 110, 59], [228, 109, 47], [35, 113, 187], [83, 197, 59], [105, 45, 31], [214, 120, 144], [194, 0, 247], [34, 151, 195], [185, 65, 135], [227, 51, 4], [23, 122, 230], [225, 148, 85], [47, 19, 100], [245, 150, 16], [7, 101, 127], [93, 181, 246], [81, 64, 172], [23, 180, 103], [15, 71, 202], [81, 217, 138], [229, 2, 156], [106, 88, 161], [1, 59, 147], [84, 114, 176], [99, 115, 103], [222, 59, 192], [49, 184, 68], [86, 249, 112], [78, 46, 61], [33, 155, 121], [68, 245, 7], [43, 184, 172], [56, 175, 18], [208, 0, 171], [85, 226, 131], [97, 163, 12], [107, 16, 180], [45, 167, 218], [250, 178, 88], [235, 38, 32], [72, 196, 174], [63, 189, 93], [56, 174, 80], [57, 63, 71], [176, 104, 135], [8, 82, 55], [75, 226, 162], [127, 227, 187], [165, 12, 245], [127, 68, 123], [165, 130, 191], [207, 186, 65], [123, 91, 213], [53, 174, 254], [23, 1, 255], [236, 99, 40], [223, 75, 31], [0, 4, 215], [18, 255, 161], [237, 155, 218], [127, 103, 64], [111, 225, 36], [125, 210, 125], [177, 120, 96], [199, 58, 163], [169, 67, 171], [56, 171, 179], [251, 246, 141], [220, 60, 68], [254, 244, 214], [77, 210, 194], [206, 112, 184], [126, 231, 111], [171, 165, 115], [234, 77, 163], [169, 188, 205], [38, 186, 216], [81, 36, 33], [198, 65, 81], [214, 138, 80], [253, 231, 142], [202, 124, 143], [85, 192, 128], [4, 119, 173], [44, 250, 48], [242, 69, 10], [62, 148, 73], [12, 254, 178], [202, 215, 164], [168, 28, 62], [150, 157, 93], [109, 105, 118], [212, 190, 184], [27, 158, 9], [152, 155, 201], [220, 115, 222], [127, 10, 143], [195, 6, 175], [179, 112, 254], [140, 250, 113], [46, 194, 180], [88, 229, 129], [141, 175, 217], [175, 193, 232], [249, 197, 185], [89, 137, 52], [197, 253, 60], [25, 106, 174], [82, 176, 133], [187, 120, 146], [4, 231, 213], [88, 147, 212], [115, 33, 215], [20, 154, 198], [78, 252, 229], [82, 152, 252], [158, 250, 149], [56, 202, 250], [2, 185, 239], [122, 117, 26], [31, 125, 242], [119, 238, 75], [68, 135, 163], [63, 94, 70], [87, 67, 77], [121, 79, 113], [246, 226, 10], [237, 214, 201], [102, 94, 178], [93, 118, 218], [16, 94, 219], [212, 42, 215], [148, 111, 241], [233, 178, 113], [141, 40, 123], [237, 90, 211], [163, 3, 205], [235, 136, 233], [116, 243, 43], [2, 103, 186], [84, 158, 45], [158, 173, 52], [223, 33, 3], [162, 81, 172], [184, 217, 220], [14, 207, 92], [1, 239, 112], [31, 34, 54], [119, 49, 226], [13, 12, 89], [169, 188, 78], [51, 106, 4], [72, 64, 54], [244, 255, 162], [75, 38, 36], [99, 85, 98], [208, 87, 104], [247, 83, 115], [61, 78, 250], [162, 219, 101], [214, 43, 93], [169, 18, 75], [195, 65, 1], [88, 83, 48], [148, 35, 204], [107, 163, 64], [125, 117, 83], [143, 129, 4], [81, 70, 91], [78, 115, 116], [27, 160, 50], [125, 72, 232], [10, 66, 177], [34, 212, 188], [130, 83, 63], [19, 65, 167], [251, 218, 245], [208, 227, 164], [255, 193, 231], [224, 118, 44], [18, 59, 145], [46, 194, 63], [21, 85, 5], [117, 141, 124], [243, 126, 199], [85, 227, 153], [125, 22, 134], [57, 71, 223], [162, 32, 136], [59, 124, 180], [42, 251, 91], [178, 200, 76], [122, 178, 149], [239, 78, 10], [139, 156, 22], [0, 20, 128], [233, 108, 37], [142, 160, 72], [43, 136, 131], [186, 204, 6], [242, 96, 147], [107, 62, 153], [38, 186, 40], [129, 142, 83], [234, 156, 98], [202, 142, 154], [154, 63, 196], [33, 175, 205], [132, 208, 43], [14, 173, 36], [138, 18, 91], [195, 149, 95], [78, 157, 70], [223, 187, 6], [105, 17, 166], [241, 22, 6], [32, 8, 242], [35, 56, 100], [162, 25, 128], [105, 32, 25], [26, 78, 244], [29, 196, 162], [90, 57, 16], [155, 193, 245], [187, 219, 156], [36, 196, 64], [183, 147, 9], [27, 30, 126], [228, 149, 103], [252, 173, 151], [24, 55, 61], [160, 243, 66], [254, 21, 171], [55, 151, 125], [220, 31, 25], [105, 239, 235], [143, 68, 84], [163, 36, 126], [50, 56, 177], [40, 2, 29], [186, 33, 32], [238, 113, 49], [232, 144, 161], [29, 7, 165], [159, 181, 211], [174, 6, 19], [9, 208, 209], [156, 52, 180], [37, 11, 134], [8, 156, 132], [98, 36, 67], [100, 248, 55], [255, 74, 145], [93, 146, 186], [198, 255, 210], [208, 174, 33], [9, 50, 86], [171, 253, 209], [76, 88, 107], [25, 185, 185], [119, 98, 125], [68, 133, 175], [136, 64, 40], [182, 74, 34], [151, 164, 204], [168, 181, 9], [250, 115, 232], [32, 24, 118], [112, 51, 233], [244, 159, 28], [160, 228, 52], [121, 55, 74], [59, 227, 2], [10, 104, 63], [25, 154, 133], [163, 215, 229], [150, 212, 99], [193, 103, 197], [198, 23, 93], [39, 162, 107], [98, 96, 196], [221, 79, 224], [82, 235, 219], [145, 200, 48], [36, 251, 52], [23, 128, 114], [254, 52, 40], [245, 0, 108], [40, 74, 149], [92, 113, 185], [31, 180, 38], [193, 104, 210], [212, 208, 113], [179, 22, 98], [124, 4, 35], [236, 200, 32], [106, 52, 89], [52, 13, 173], [113, 87, 149], [169, 32, 177], [156, 2, 14], [246, 66, 249], [237, 81, 7], [149, 138, 187], [191, 223, 125], [52, 163, 111], [186, 117, 16], [37, 84, 29], [72, 170, 177], [182, 208, 210], [144, 25, 84], [141, 152, 116], [214, 115, 36], [54, 179, 226], [5, 0, 113], [147, 146, 123], [210, 210, 74], [0, 113, 206], [176, 85, 19], [238, 239, 217], [174, 184, 93], [228, 11, 127], [56, 120, 149], [148, 128, 57], [6, 193, 151], [89, 96, 145], [77, 51, 207], [59, 237, 238], [128, 122, 64], [157, 4, 31], [51, 175, 128], [47, 32, 125], [104, 16, 228], [87, 74, 57], [134, 112, 72], [78, 240, 17], [77, 99, 101], [57, 128, 109], [205, 46, 254], [18, 26, 180], [240, 65, 63], [186, 242, 21], [85, 114, 156], [28, 90, 59], [93, 223, 124], [41, 138, 15], [52, 95, 145], [169, 195, 173], [246, 90, 53], [212, 84, 248], [223, 239, 236], [119, 212, 102], [9, 249, 186], [242, 18, 249], [28, 118, 238], [104, 43, 175], [209, 115, 121], [123, 183, 136], [146, 245, 61], [75, 99, 109], [231, 42, 128], [49, 78, 117], [45, 115, 181], [160, 214, 241], [163, 21, 213], [125, 231, 107], [93, 225, 162], [198, 249, 197], [212, 131, 97], [52, 63, 219], [78, 247, 3], [45, 76, 125], [91, 170, 37], [82, 7, 183], [105, 80, 140], [21, 222, 40], [118, 183, 135], [32, 151, 113], [114, 249, 172], [91, 114, 188], [50, 8, 133], [195, 22, 35], [62, 199, 167], [56, 247, 100], [85, 15, 1], [244, 92, 170], [226, 200, 96], [172, 106, 66], [65, 93, 80], [168, 114, 97], [30, 170, 119], [98, 112, 241], [86, 222, 233], [138, 18, 102], [254, 52, 131], [250, 183, 248], [12, 200, 143], [7, 166, 243], [93, 210, 101], [243, 19, 17], [11, 24, 194], [175, 65, 71], [17, 255, 234], [2, 97, 83], [96, 225, 100], [174, 50, 10], [118, 123, 120], [54, 166, 74], [181, 236, 171], [143, 226, 179], [34, 126, 237], [40, 202, 65], [111, 148, 228], [158, 15, 200], [205, 47, 230], [10, 176, 167], [92, 130, 153], [31, 18, 228], [200, 145, 157], [212, 15, 195], [115, 172, 231], [234, 222, 192], [85, 213, 14], [235, 213, 201], [157, 47, 85], [194, 179, 80], [111, 106, 13], [174, 70, 143], [7, 220, 46], [217, 172, 201], [73, 19, 245], [191, 88, 119], [156, 67, 5], [53, 162, 128], [96, 13, 251], [163, 65, 94], [199, 176, 162], [88, 109, 220], [49, 62, 218], [37, 76, 175], [117, 15, 216], [215, 148, 116], [235, 9, 160], [157, 231, 54], [246, 178, 131], [157, 57, 237], [35, 253, 185], [1, 12, 114], [72, 167, 47], [89, 44, 121], [187, 109, 244], [3, 20, 159], [77, 189, 220], [24, 208, 129], [45, 98, 151], [74, 133, 171], [103, 58, 48], [112, 115, 188], [1, 67, 44], [42, 220, 59], [248, 158, 203], [81, 86, 201], [117, 71, 237], [176, 37, 135], [169, 203, 112], [48, 54, 98], [36, 126, 51], [206, 94, 155], [190, 44, 99], [237, 167, 91], [92, 57, 220], [57, 238, 133], [187, 16, 166], [50, 253, 35], [141, 187, 53], [6, 110, 54], [27, 243, 22], [141, 156, 12], [199, 155, 41], [239, 181, 132], [30, 214, 186], [175, 10, 246], [133, 229, 242], [163, 124, 85], [220, 236, 73], [28, 60, 95], [246, 140, 72], [150, 95, 112], [84, 154, 111], [46, 49, 196], [238, 221, 203], [169, 11, 213], [75, 216, 2], [222, 158, 238], [116, 181, 229], [115, 86, 234], [104, 171, 21], [189, 227, 1], [20, 116, 67], [236, 177, 14], [153, 130, 249], [228, 57, 197], [166, 104, 89], [156, 5, 86], [62, 89, 143], [109, 159, 115], [135, 233, 47], [8, 208, 191], [112, 129, 39], [188, 214, 185], [25, 76, 72], [164, 93, 146], [69, 214, 220], [73, 131, 181], [253, 139, 23], [240, 231, 14], [44, 88, 45], [148, 192, 201], [58, 253, 35], [231, 236, 175], [56, 217, 205], [62, 86, 130], [128, 244, 42], [178, 141, 11], [43, 110, 8], [57, 127, 221], [165, 162, 182], [146, 139, 18], [206, 43, 178], [86, 241, 48], [252, 112, 208], [96, 152, 78], [253, 231, 80], [65, 180, 30], [17, 140, 134], [207, 12, 138], [123, 255, 194], [189, 140, 240], [1, 168, 250], [199, 202, 251], [54, 99, 124], [83, 179, 184], [114, 134, 86], [225, 199, 168], [116, 132, 215], [138, 56, 253], [179, 152, 156], [56, 202, 113], [77, 16, 189], [29, 69, 14], [206, 68, 183], [234, 255, 19], [112, 175, 92], [236, 134, 125], [138, 3, 82], [28, 179, 165], [111, 112, 17], [194, 154, 68], [37, 115, 157], [34, 11, 72], [199, 84, 73], [214, 103, 144], [93, 12, 196], [125, 159, 5], [140, 200, 84], [50, 246, 207], [77, 46, 143], [203, 74, 157], [41, 245, 64], [130, 245, 19], [87, 116, 76], [86, 76, 101], [109, 236, 230], [76, 78, 92], [216, 131, 150], [17, 255, 182], [126, 174, 244], [35, 249, 247], [206, 95, 66], [28, 212, 17], [55, 125, 0], [11, 50, 16], [5, 204, 147], [227, 237, 240], [86, 109, 129], [116, 94, 242], [50, 34, 65], [157, 209, 134], [191, 201, 148], [3, 66, 74], [254, 120, 143], [49, 84, 41], [129, 99, 21], [226, 15, 170], [201, 250, 141], [219, 250, 1], [173, 248, 82], [162, 26, 38], [127, 222, 135], [237, 157, 12], [225, 176, 10], [136, 203, 235], [37, 111, 252], [14, 61, 191], [30, 87, 49], [174, 7, 67], [138, 251, 198], [138, 209, 168], [210, 133, 166], [17, 148, 220], [200, 191, 185], [89, 37, 114], [53, 77, 165], [236, 103, 123], [80, 208, 73], [43, 114, 59], [253, 100, 190], [145, 140, 113], [192, 47, 173], [245, 230, 224], [76, 142, 9], [57, 135, 173], [88, 120, 125], [84, 176, 39], [16, 123, 244], [248, 153, 184], [131, 215, 201], [246, 156, 126], [193, 51, 121], [75, 49, 212], [36, 140, 60], [140, 174, 113], [213, 198, 128], [99, 68, 140], [176, 64, 42], [54, 0, 90], [76, 180, 2], [187, 6, 195], [225, 243, 47], [189, 46, 213], [56, 78, 87], [249, 216, 231], [129, 12, 246], [21, 162, 205], [33, 82, 71], [165, 255, 130], [255, 172, 203], [96, 58, 67], [216, 35, 185], [35, 248, 98], [245, 40, 166], [20, 59, 149], [119, 125, 103], [118, 106, 183], [63, 5, 232], [46, 201, 204], [239, 209, 123], [235, 37, 94], [36, 238, 32], [227, 21, 58], [191, 177, 55], [46, 208, 140], [232, 255, 115], [118, 132, 62], [107, 176, 198], [113, 138, 131], [122, 153, 12], [222, 85, 29], [197, 40, 119], [150, 209, 130], [86, 148, 244], [140, 3, 238], [126, 142, 51], [158, 23, 188], [240, 51, 47], [132, 181, 96], [36, 32, 182], [158, 83, 136], [23, 30, 191], [12, 170, 18], [118, 96, 184], [3, 7, 32], [51, 171, 67], [42, 74, 216], [123, 208, 110], [35, 162, 216], [128, 114, 149], [71, 143, 64], [137, 13, 98], [131, 156, 54], [223, 130, 181], [208, 235, 172], [64, 48, 215], [132, 216, 209], [231, 170, 229], [15, 145, 139], [178, 107, 72], [226, 36, 215], [140, 93, 129], [34, 207, 6], [178, 92, 216], [45, 242, 154], [60, 52, 249], [8, 101, 26], [138, 230, 101], [123, 142, 216], [87, 82, 197], [13, 229, 120], [13, 129, 105], [229, 32, 83], [28, 140, 119], [116, 74, 226], [167, 114, 214], [78, 108, 174], [219, 213, 204], [185, 212, 167], [110, 119, 179], [216, 89, 75], [140, 167, 214], [122, 136, 67], [124, 234, 6], [90, 141, 1], [135, 188, 154], [123, 195, 47], [56, 12, 74], [171, 181, 17], [16, 251, 155], [190, 241, 1], [88, 126, 152], [244, 192, 104], [63, 57, 193], [8, 176, 108], [7, 170, 40], [39, 35, 51], [219, 135, 94], [32, 4, 108], [16, 125, 30], [194, 86, 5], [66, 178, 156], [215, 184, 215], [65, 114, 239], [21, 14, 207], [40, 165, 248], [36, 197, 6], [47, 84, 250], [222, 165, 79], [2, 89, 220], [53, 24, 137], [67, 63, 178], [50, 123, 245], [44, 108, 19], [226, 23, 61], [194, 149, 205], [46, 114, 119], [238, 118, 158], [233, 204, 165], [95, 245, 85], [214, 194, 133], [202, 225, 122], [225, 33, 193], [189, 237, 59], [32, 196, 170], [16, 158, 104], [229, 17, 247], [183, 234, 149], [188, 41, 58], [75, 197, 87], [146, 105, 32], [89, 219, 215], [210, 183, 199], [124, 3, 173], [234, 225, 133], [242, 167, 40], [28, 253, 45], [43, 46, 21], [83, 168, 173], [221, 9, 210], [195, 192, 7], [230, 253, 12], [84, 125, 14], [239, 32, 13], [175, 192, 238], [165, 142, 179], [66, 147, 44], [119, 35, 54], [74, 162, 102], [62, 145, 135], [35, 70, 214], [55, 121, 146], [74, 218, 60], [126, 173, 211], [171, 227, 83], [66, 207, 23], [68, 97, 228], [180, 144, 255], [214, 230, 147], [79, 147, 192], [35, 155, 13], [75, 116, 178], [45, 106, 243], [19, 1, 113], [191, 72, 215], [109, 109, 180], [129, 151, 178], [144, 148, 95], [104, 68, 97], [57, 183, 103], [1, 255, 74], [132, 132, 251], [95, 51, 108], [177, 104, 206], [135, 230, 118], [244, 155, 123], [231, 153, 103], [10, 183, 160], [60, 155, 19], [174, 152, 72], [167, 113, 178], [95, 143, 179], [2, 49, 182], [21, 252, 142], [41, 228, 154], [171, 137, 190], [197, 250, 66], [255, 226, 203], [128, 78, 193], [195, 130, 244], [168, 51, 39], [29, 53, 166], [151, 228, 60], [89, 229, 24], [63, 27, 229], [133, 44, 160], [100, 98, 217], [90, 201, 161], [37, 110, 100], [39, 20, 156], [164, 117, 60], [163, 163, 100], [61, 233, 239], [161, 215, 9], [101, 99, 81], [30, 6, 66], [182, 113, 151], [154, 144, 187], [177, 128, 45], [114, 12, 13], [218, 192, 104], [56, 14, 108], [100, 101, 9], [36, 137, 7], [122, 192, 183], [132, 194, 9], [139, 224, 67], [207, 158, 149], [101, 26, 161], [72, 30, 140], [226, 177, 191], [31, 240, 236], [67, 64, 231], [64, 140, 202], [19, 59, 26], [166, 140, 245], [124, 246, 56], [72, 252, 26], [254, 215, 69], [79, 198, 95], [4, 190, 80], [81, 254, 36], [88, 191, 243], [62, 174, 219], [197, 27, 181], [41, 17, 13], [213, 110, 193], [254, 185, 185], [176, 217, 203], [94, 198, 192], [244, 199, 32], [120, 47, 191], [11, 88, 60], [76, 214, 135], [210, 82, 181], [73, 166, 176], [202, 94, 7], [180, 195, 8], [241, 204, 10], [241, 57, 87], [95, 255, 69], [203, 137, 31], [254, 198, 111], [84, 62, 89], [105, 18, 44], [49, 14, 211], [187, 83, 208], [207, 133, 66], [0, 229, 236], [255, 140, 201], [231, 57, 34], [238, 36, 11], [209, 62, 234], [204, 126, 158], [55, 18, 149], [101, 201, 78], [227, 149, 73], [213, 192, 225], [208, 184, 28], [43, 88, 1], [181, 105, 155], [146, 192, 162], [114, 148, 58], [150, 190, 78], [50, 141, 122], [103, 130, 191], [133, 34, 25], [220, 204, 249], [34, 98, 18], [212, 97, 179], [98, 112, 115], [189, 100, 97], [180, 228, 57], [152, 171, 3], [203, 69, 89], [0, 138, 142], [167, 219, 41], [184, 213, 71], [55, 217, 52], [94, 202, 52], [28, 229, 172], [116, 211, 166], [136, 56, 97], [49, 230, 11], [176, 116, 216], [27, 148, 105], [217, 43, 233], [194, 139, 197], [131, 96, 200], [68, 174, 208], [31, 144, 12], [84, 99, 132], [144, 234, 29], [217, 40, 211], [60, 128, 89], [242, 124, 216], [1, 15, 181], [216, 203, 36], [16, 29, 50], [60, 195, 50], [131, 31, 185], [29, 1, 26], [160, 34, 39], [11, 237, 207], [166, 165, 177], [108, 246, 34], [169, 218, 67], [245, 253, 87], [78, 113, 53], [13, 242, 32], [185, 80, 46], [98, 85, 234], [75, 4, 225], [134, 77, 43], [23, 22, 248], [211, 68, 172], [34, 177, 161], [165, 172, 81], [23, 129, 115], [62, 209, 162], [142, 164, 144], [7, 224, 194], [144, 45, 247], [46, 208, 138], [174, 2, 88], [193, 141, 149], [199, 158, 193], [67, 193, 162], [249, 57, 194], [105, 175, 3], [54, 176, 197], [98, 108, 136], [185, 32, 174], [10, 59, 123], [179, 108, 237], [216, 85, 204], [248, 99, 83], [216, 83, 136], [44, 132, 221], [190, 49, 24], [13, 38, 15], [232, 177, 72], [170, 149, 70], [132, 122, 226], [150, 80, 56], [127, 222, 191], [188, 237, 202], [218, 219, 153], [195, 202, 167], [129, 235, 192], [31, 9, 241], [98, 92, 180], [39, 246, 117], [144, 47, 10], [180, 242, 28], [89, 250, 97], [137, 17, 172], [83, 152, 83], [79, 122, 79], [81, 231, 12], [202, 56, 15], [62, 61, 196], [14, 73, 185], [13, 32, 198], [160, 1, 19], [198, 101, 140], [114, 217, 214], [236, 175, 34], [124, 46, 109], [177, 254, 67], [107, 1, 77], [77, 178, 49], [3, 137, 77], [177, 117, 216], [179, 152, 67], [183, 147, 168], [159, 164, 245], [114, 179, 169], [115, 64, 207], [236, 188, 18], [40, 124, 128], [203, 77, 48], [202, 129, 20], [65, 14, 73], [70, 120, 17], [206, 148, 61], [183, 144, 178], [78, 194, 13], [97, 148, 227], [180, 102, 190], [101, 9, 32], [27, 83, 32], [23, 31, 211], [249, 67, 185], [106, 2, 37], [60, 168, 7], [6, 66, 0], [102, 199, 104], [165, 167, 242], [117, 118, 136], [16, 224, 210], [96, 80, 13], [148, 117, 77], [164, 52, 128], [66, 248, 173], [44, 119, 63], [153, 187, 113], [69, 101, 25], [76, 160, 127], [184, 25, 203], [164, 23, 100], [7, 211, 78], [218, 157, 173], [201, 165, 47], [37, 116, 188], [151, 220, 53], [57, 209, 23], [131, 81, 120], [53, 144, 102], [174, 50, 145], [76, 187, 201], [183, 90, 211], [60, 2, 241], [128, 124, 221], [196, 196, 127], [113, 217, 187], [20, 118, 210], [53, 6, 123], [124, 123, 151], [146, 63, 203], [14, 227, 168], [247, 119, 105], [209, 22, 142], [196, 238, 134], [212, 205, 255], [188, 140, 151], [26, 167, 114], [249, 222, 209], [38, 237, 135], [88, 71, 255], [146, 224, 43], [67, 114, 76], [199, 0, 209], [149, 249, 24], [34, 177, 122], [44, 193, 252], [121, 221, 245], [41, 182, 160], [100, 131, 81], [28, 115, 46], [128, 11, 227], [244, 184, 163], [138, 70, 93], [38, 168, 206], [227, 81, 168], [208, 218, 10], [206, 194, 6], [174, 167, 242], [126, 112, 185], [100, 78, 69], [201, 153, 2], [159, 201, 165], [237, 14, 66], [93, 24, 180], [69, 219, 227], [103, 105, 0], [107, 193, 134], [24, 227, 45], [132, 2, 142], [153, 139, 4], [75, 127, 57], [146, 146, 254], [59, 183, 245], [87, 195, 222], [140, 27, 213], [250, 47, 17], [22, 161, 230], [244, 84, 85], [44, 57, 143], [36, 216, 173], [206, 247, 81], [96, 206, 66], [250, 222, 195], [252, 219, 160], [253, 42, 182], [232, 86, 85], [97, 195, 21], [165, 231, 120], [213, 193, 140], [108, 213, 35], [252, 78, 84], [239, 121, 103], [146, 197, 166], [251, 70, 173], [149, 1, 202], [93, 120, 96], [0, 99, 123], [116, 93, 123], [124, 65, 97], [125, 43, 89], [128, 164, 240], [118, 199, 147], [85, 75, 93], [26, 227, 155], [120, 233, 187], [114, 3, 242], [36, 215, 116], [80, 37, 29], [244, 115, 168], [252, 20, 151], [210, 214, 21], [19, 240, 208], [36, 132, 151], [57, 139, 70], [203, 9, 70], [3, 169, 195], [182, 135, 42], [38, 17, 100], [91, 213, 204], [108, 142, 28], [243, 157, 115], [18, 162, 238], [45, 171, 135], [190, 142, 44], [50, 233, 221], [168, 102, 15], [118, 100, 206], [60, 148, 37], [145, 141, 98], [211, 176, 0], [253, 144, 136], [97, 15, 26], [15, 52, 192], [218, 177, 194], [144, 229, 216], [137, 71, 110], [76, 171, 3], [96, 168, 157], [89, 243, 66], [104, 135, 204], [104, 30, 180], [110, 78, 108], [69, 11, 12], [199, 182, 250], [46, 193, 15], [209, 178, 120], [116, 39, 253], [130, 164, 213], [225, 207, 162], [113, 187, 65], [31, 95, 165], [58, 83, 82], [112, 191, 70], [216, 200, 47], [7, 171, 160], [134, 72, 226], [3, 124, 100], [137, 235, 5], [8, 30, 15], [37, 182, 66], [216, 248, 126], [200, 106, 9], [30, 48, 207], [114, 213, 95], [91, 191, 212], [61, 69, 188], [98, 36, 221], [71, 99, 220], [233, 205, 178], [219, 19, 112], [202, 86, 54], [102, 187, 68], [51, 24, 87], [1, 18, 214], [126, 61, 245], [251, 83, 77], [140, 94, 206], [38, 76, 185], [99, 4, 178], [209, 91, 144], [117, 205, 68], [97, 115, 232], [212, 23, 1], [241, 17, 1], [94, 241, 113], [109, 143, 127], [142, 236, 41], [119, 17, 217], [141, 89, 203], [233, 91, 67], [240, 41, 165], [19, 47, 38], [71, 251, 2], [112, 113, 105], [154, 184, 228], [120, 66, 233], [81, 116, 217], [61, 65, 67], [91, 97, 65], [110, 227, 219], [219, 209, 31], [78, 135, 37], [152, 114, 69], [132, 122, 135], [70, 44, 233], [44, 65, 185], [191, 166, 58], [52, 130, 48], [146, 143, 178], [198, 30, 135], [44, 51, 246], [60, 38, 232], [28, 197, 200], [190, 212, 185], [86, 58, 93], [68, 213, 25], [149, 144, 6], [127, 142, 161], [227, 99, 4], [25, 60, 14], [136, 80, 222], [165, 128, 138], [148, 136, 209], [242, 212, 165], [168, 191, 33], [35, 114, 110], [162, 143, 204], [43, 146, 145], [17, 72, 243], [145, 135, 141], [172, 89, 103], [6, 106, 155], [190, 171, 57], [12, 166, 69], [149, 236, 123], [169, 63, 205], [245, 233, 21], [89, 65, 89], [6, 246, 227], [187, 251, 78], [106, 183, 39], [169, 216, 173], [116, 6, 107], [192, 1, 83], [14, 45, 97], [41, 125, 199], [49, 34, 100], [25, 10, 227], [2, 83, 126], [145, 33, 137], [231, 110, 215], [54, 14, 175], [135, 41, 74], [28, 207, 189], [168, 93, 190], [104, 164, 23], [35, 221, 3], [18, 148, 140], [204, 110, 169], [249, 99, 193], [62, 57, 249], [140, 57, 108], [28, 221, 241], [74, 60, 182], [134, 113, 149], [8, 154, 180], [204, 23, 134], [141, 221, 125], [163, 246, 162], [163, 32, 240], [21, 116, 41], [113, 49, 195], [4, 241, 168], [56, 86, 154], [65, 157, 161], [92, 220, 34], [191, 92, 88], [155, 160, 238], [155, 41, 73], [133, 72, 63], [193, 159, 249], [41, 129, 131], [106, 32, 69], [196, 239, 149], [232, 82, 66], [225, 244, 111], [232, 72, 201], [38, 76, 215], [120, 40, 46], [19, 245, 215], [73, 223, 128], [154, 217, 112], [171, 78, 75], [35, 126, 27], [3, 78, 12], [115, 82, 193], [113, 85, 70], [217, 59, 69], [104, 199, 209], [172, 54, 168], [209, 100, 153], [124, 136, 56], [103, 255, 53], [198, 120, 177], [201, 157, 194], [11, 145, 230], [222, 156, 138], [155, 186, 173], [254, 247, 244], [90, 109, 108], [82, 193, 194], [247, 81, 237], [187, 91, 114], [86, 80, 194], [109, 195, 229], [224, 147, 198], [228, 48, 128], [176, 73, 203], [238, 111, 45], [50, 65, 69], [195, 18, 102], [209, 196, 214], [1, 39, 193], [167, 240, 204], [46, 37, 196], [184, 219, 123], [122, 13, 116], [154, 56, 65], [86, 246, 105], [23, 186, 216], [183, 88, 148], [103, 42, 221], [107, 29, 5], [153, 73, 97], [144, 181, 66], [186, 163, 48], [23, 25, 110], [206, 139, 18], [176, 175, 229], [98, 221, 103], [186, 107, 140], [62, 64, 97], [196, 154, 180], [28, 7, 44], [61, 111, 112], [220, 46, 129], [213, 201, 9], [250, 180, 18], [82, 237, 174], [72, 134, 49], [45, 212, 49], [36, 202, 221], [22, 136, 15], [198, 110, 98], [86, 213, 181], [81, 101, 68], [55, 18, 20], [163, 101, 212], [80, 54, 230], [191, 59, 214], [148, 117, 45], [26, 167, 131], [232, 249, 145], [177, 205, 61], [222, 233, 72], [65, 234, 148], [135, 146, 134], [48, 62, 109], [81, 159, 186], [176, 56, 201], [32, 111, 108], [188, 191, 173], [64, 127, 80], [94, 42, 103], [222, 149, 26], [74, 187, 210], [145, 110, 231], [221, 233, 168], [243, 162, 251], [208, 58, 250], [222, 189, 224], [0, 216, 14], [63, 58, 196], [118, 34, 149], [126, 166, 196], [18, 46, 151], [19, 172, 192], [240, 160, 112], [110, 217, 250], [14, 93, 146], [90, 8, 203], [172, 237, 168], [65, 25, 244], [137, 243, 229], [63, 14, 212], [26, 150, 220], [121, 83, 27], [125, 74, 82], [84, 70, 128], [172, 122, 23], [215, 111, 43], [244, 3, 83], [215, 205, 60], [251, 119, 201], [45, 101, 220], [72, 121, 96], [68, 76, 50], [244, 233, 225], [109, 67, 77], [56, 114, 118], [222, 111, 104], [183, 143, 175], [44, 204, 95], [251, 84, 212], [175, 31, 73], [237, 38, 71], [219, 47, 12], [2, 93, 86], [27, 14, 11], [212, 150, 16], [72, 55, 175], [186, 131, 56], [121, 86, 199], [211, 134, 108], [94, 170, 69], [35, 139, 198], [150, 14, 32], [133, 50, 179], [113, 150, 103], [177, 23, 128], [130, 52, 238], [248, 236, 93], [252, 10, 161], [244, 20, 44], [77, 143, 228], [55, 0, 195], [146, 24, 166], [4, 133, 78], [147, 112, 12], [151, 65, 106], [177, 161, 213], [165, 8, 100], [231, 54, 92], [54, 250, 135], [209, 207, 208], [253, 24, 254], [99, 216, 144], [121, 201, 216], [226, 159, 9], [161, 125, 0], [186, 243, 68], [134, 232, 154], [165, 186, 104], [14, 212, 93], [78, 34, 47], [147, 199, 145], [253, 223, 249], [154, 62, 124], [54, 162, 147], [92, 204, 171], [56, 76, 239], [199, 191, 222], [78, 222, 82], [41, 223, 31], [11, 27, 20], [231, 141, 1], [21, 215, 243], [228, 246, 240], [241, 36, 40], [203, 198, 181], [43, 97, 189], [183, 179, 225], [181, 26, 199], [3, 107, 160], [112, 222, 142], [90, 59, 106], [29, 103, 77], [254, 60, 167], [160, 28, 130], [177, 210, 248], [93, 230, 163], [253, 192, 65], [121, 216, 38], [130, 244, 90], [238, 178, 201], [105, 102, 234], [190, 157, 0], [230, 200, 193], [127, 153, 175], [161, 74, 24], [188, 95, 119], [205, 72, 178], [138, 98, 165], [238, 151, 94], [145, 27, 26], [113, 61, 193], [88, 180, 239], [6, 84, 126], [40, 91, 142], [44, 156, 209], [245, 49, 27], [158, 142, 112], [138, 190, 221], [54, 39, 253], [36, 7, 38], [98, 136, 5], [179, 49, 244], [210, 70, 124], [10, 87, 117], [172, 118, 120], [43, 249, 86], [163, 16, 22], [176, 164, 121], [191, 109, 61], [220, 246, 64], [126, 215, 189], [206, 13, 14], [55, 68, 74], [44, 225, 27], [26, 28, 10], [102, 136, 3], [10, 133, 10], [237, 151, 63], [199, 210, 189], [44, 248, 77], [229, 142, 78], [51, 162, 78], [150, 94, 212], [112, 4, 36], [118, 214, 202], [38, 195, 215], [241, 235, 17], [233, 91, 245], [43, 131, 99], [152, 23, 167], [177, 239, 106], [63, 40, 155], [191, 18, 45], [141, 151, 102], [33, 221, 232], [1, 175, 45], [131, 159, 122], [185, 184, 131], [204, 61, 4], [238, 113, 96], [200, 47, 196], [22, 93, 35], [148, 86, 70], [23, 248, 81], [200, 254, 122], [37, 81, 213], [229, 249, 122], [99, 20, 236], [32, 9, 91], [12, 93, 234], [26, 213, 178], [187, 197, 2], [255, 134, 253], [155, 194, 180], [56, 29, 148], [165, 241, 195], [6, 203, 51], [3, 211, 45], [75, 252, 38], [134, 3, 188], [19, 225, 116], [41, 111, 194], [123, 132, 166], [151, 48, 95], [207, 239, 107], [90, 227, 165], [8, 128, 172], [192, 138, 21], [219, 151, 239], [254, 108, 240], [1, 105, 55], [33, 9, 224], [31, 6, 83], [40, 34, 23], [12, 137, 243], [182, 95, 106], [16, 116, 141], [55, 100, 193], [117, 16, 230], [69, 131, 221], [130, 186, 249], [127, 179, 91], [41, 173, 141], [65, 186, 66], [209, 134, 234], [219, 124, 118], [206, 109, 234], [156, 137, 36], [131, 207, 15], [112, 109, 240], [235, 36, 212], [240, 79, 1], [123, 154, 136], [92, 234, 96], [80, 244, 130], [75, 26, 124], [83, 249, 12], [202, 83, 94], [194, 133, 151], [212, 147, 225], [109, 177, 39], [221, 44, 236], [120, 251, 146], [90, 161, 151], [21, 51, 149], [8, 205, 156], [244, 211, 125], [34, 140, 76], [86, 23, 215], [216, 150, 131], [20, 195, 102], [44, 41, 81], [78, 231, 254], [223, 198, 169], [162, 220, 230], [138, 61, 223], [249, 92, 240], [44, 87, 188], [94, 213, 176], [231, 136, 59], [167, 227, 4], [237, 31, 92], [58, 6, 155], [47, 159, 90], [215, 113, 220], [11, 62, 151], [52, 48, 220], [87, 206, 14], [143, 215, 62], [99, 26, 133], [159, 230, 253], [20, 195, 170], [162, 135, 57], [226, 55, 190], [173, 115, 151], [128, 160, 210], [163, 138, 169], [97, 3, 2], [77, 18, 71], [254, 57, 155], [107, 253, 228], [130, 223, 13], [232, 248, 220], [66, 210, 93], [149, 209, 150], [135, 73, 80], [130, 82, 85], [76, 164, 251], [27, 34, 37], [20, 100, 201], [67, 65, 46], [174, 247, 251], [254, 20, 61], [243, 0, 21], [28, 24, 11], [46, 37, 202], [126, 229, 253], [166, 13, 164], [217, 146, 0], [28, 190, 205], [121, 252, 243], [213, 94, 201], [129, 212, 77], [157, 78, 127], [240, 114, 247], [216, 237, 49], [20, 24, 128], [106, 41, 229], [95, 38, 151], [5, 176, 79], [193, 135, 172], [155, 163, 116], [192, 150, 156], [24, 225, 4], [51, 136, 172], [186, 53, 211], [183, 202, 93], [228, 231, 172], [123, 168, 169], [65, 229, 120], [62, 227, 251], [228, 101, 14], [238, 42, 29], [110, 254, 8], [132, 70, 47], [113, 45, 34], [30, 208, 43], [217, 225, 158], [28, 47, 203], [51, 91, 25], [14, 30, 229], [57, 207, 168], [87, 147, 220], [53, 173, 125], [172, 131, 132], [144, 147, 40], [1, 171, 249], [130, 12, 193], [92, 120, 143], [0, 202, 172], [197, 29, 201], [158, 8, 157], [89, 100, 204], [72, 151, 84], [40, 131, 181], [39, 153, 9], [23, 11, 211], [134, 62, 49], [198, 25, 168], [37, 147, 7], [93, 114, 156], [183, 222, 127], [185, 210, 83], [182, 221, 163], [238, 10, 199], [159, 117, 184], [0, 207, 34], [43, 205, 225], [112, 187, 186], [120, 183, 222], [231, 64, 142], [80, 102, 236], [154, 116, 83], [44, 51, 127], [139, 243, 27], [108, 166, 172], [216, 223, 57], [7, 248, 137], [136, 151, 0], [221, 50, 133], [121, 93, 214], [14, 207, 22], [62, 61, 115], [119, 90, 236], [189, 202, 132], [210, 212, 19], [6, 218, 69], [252, 80, 26], [33, 98, 156], [17, 207, 2], [103, 60, 177], [227, 138, 116], [185, 66, 33], [50, 172, 216], [156, 243, 121], [151, 143, 146], [137, 131, 169], [72, 220, 1], [173, 51, 6], [102, 156, 151], [47, 48, 31], [38, 116, 99], [130, 203, 75], [85, 138, 178], [131, 202, 32], [50, 222, 97], [25, 197, 177], [87, 230, 147], [145, 133, 64], [247, 220, 218], [227, 55, 20], [134, 231, 4], [80, 157, 127], [113, 119, 50], [156, 252, 209], [129, 33, 219], [73, 152, 78], [175, 32, 250], [192, 82, 120], [118, 30, 252], [48, 113, 182], [108, 212, 48], [59, 227, 208], [77, 254, 83], [117, 27, 163], [63, 147, 54], [160, 247, 255], [253, 120, 31], [197, 131, 123], [103, 74, 1], [249, 7, 135], [132, 113, 134], [31, 203, 225], [168, 91, 57], [30, 56, 222], [48, 195, 36], [101, 103, 155], [11, 61, 182], [141, 177, 14], [176, 102, 210], [92, 14, 90], [195, 102, 87], [41, 195, 32], [43, 80, 51], [243, 255, 172], [217, 108, 136], [78, 201, 102], [83, 53, 200], [115, 244, 103], [159, 33, 184], [169, 146, 207], [49, 180, 133], [90, 254, 90], [114, 155, 111], [5, 40, 208], [3, 155, 94], [160, 192, 236], [157, 220, 151], [62, 166, 98], [109, 213, 28], [238, 179, 125], [121, 211, 242], [252, 73, 197], [217, 239, 248], [3, 253, 6], [129, 14, 221], [155, 116, 102], [239, 181, 133], [183, 19, 217], [196, 105, 142], [211, 151, 110], [160, 176, 35], [158, 170, 122], [165, 156, 181], [190, 196, 94], [4, 160, 182], [79, 226, 94], [101, 235, 148], [8, 135, 90], [38, 27, 7], [35, 49, 24], [126, 74, 18], [84, 211, 93], [6, 15, 21], [26, 5, 109], [237, 67, 250], [214, 195, 44], [10, 101, 186], [7, 211, 8], [167, 103, 3], [53, 104, 95], [234, 180, 17], [205, 187, 14], [149, 187, 54], [107, 33, 25], [41, 79, 26], [125, 234, 0], [146, 219, 99], [170, 166, 241], [154, 5, 163], [34, 88, 17], [196, 32, 153], [35, 197, 253], [88, 117, 177], [163, 4, 67], [77, 198, 219], [85, 232, 74], [174, 202, 179], [188, 80, 22], [226, 202, 199], [157, 134, 162], [173, 122, 15], [122, 221, 100], [102, 176, 79], [240, 57, 61], [141, 236, 23], [3, 143, 129], [224, 151, 100], [205, 217, 170], [140, 112, 26], [148, 172, 160], [232, 108, 107], [55, 225, 118], [234, 110, 48], [163, 60, 136], [184, 60, 253], [155, 11, 144], [207, 40, 47], [38, 14, 213], [222, 30, 168], [13, 190, 187], [60, 60, 5], [119, 103, 92], [176, 205, 212], [139, 201, 87], [64, 125, 131], [23, 45, 8], [170, 169, 1], [29, 109, 234], [42, 229, 26], [45, 3, 213], [255, 82, 166], [19, 11, 254], [54, 137, 142], [251, 29, 61], [242, 79, 175], [217, 23, 16], [229, 113, 79], [118, 118, 144], [148, 197, 62], [154, 120, 55], [64, 135, 228], [226, 62, 185], [80, 151, 67], [194, 75, 254], [48, 52, 201], [59, 241, 212], [95, 158, 34], [147, 133, 242], [206, 42, 122], [16, 59, 201], [77, 183, 137], [101, 33, 71], [237, 169, 58], [87, 200, 105], [177, 219, 82], [71, 93, 247], [50, 151, 217], [77, 109, 232], [140, 228, 20], [52, 137, 249], [72, 79, 31], [254, 238, 170], [206, 254, 174], [215, 156, 148], [223, 74, 179], [73, 71, 253], [21, 198, 39], [113, 146, 221], [98, 139, 253], [46, 138, 50], [225, 32, 181], [127, 85, 254], [216, 252, 100], [234, 162, 255], [25, 81, 244], [127, 29, 108], [127, 214, 141], [2, 211, 227], [3, 227, 178], [104, 149, 77], [60, 201, 36], [185, 184, 120], [43, 177, 59], [112, 244, 248], [193, 20, 24], [180, 170, 75], [139, 252, 211], [78, 40, 7], [204, 155, 181], [117, 116, 61], [60, 76, 218], [3, 112, 87], [231, 59, 245], [147, 183, 215], [60, 13, 134], [185, 64, 110], [25, 7, 23], [29, 173, 96], [71, 35, 11], [222, 99, 138], [146, 40, 7], [102, 68, 52], [151, 17, 50], [120, 67, 104], [61, 243, 131], [171, 159, 132], [159, 50, 151], [192, 216, 10], [2, 150, 234], [148, 199, 106], [92, 95, 235], [3, 182, 82], [101, 201, 35], [198, 41, 170], [36, 169, 210], [190, 253, 214], [170, 57, 131], [162, 4, 241], [191, 70, 72], [126, 214, 56], [122, 214, 12], [46, 7, 227], [175, 75, 146], [1, 154, 26], [63, 216, 28], [10, 20, 26], [204, 44, 241], [251, 11, 235], [12, 230, 85], [59, 22, 70], [94, 165, 38], [31, 237, 121], [254, 142, 70], [215, 37, 109], [44, 192, 113], [207, 130, 184], [248, 238, 206], [50, 3, 105], [163, 147, 248], [95, 215, 19], [234, 95, 82], [34, 115, 126], [32, 191, 221], [1, 130, 148], [121, 225, 34], [200, 57, 38], [82, 153, 189], [34, 251, 163], [241, 46, 90], [90, 204, 161], [220, 148, 102], [84, 89, 202], [89, 169, 228], [83, 189, 215], [63, 73, 93], [155, 184, 148], [150, 201, 4], [51, 20, 48], [99, 59, 193], [89, 56, 225], [76, 13, 187], [160, 102, 52], [25, 120, 90], [173, 144, 155], [241, 62, 181], [33, 180, 168], [29, 158, 64], [155, 254, 36], [44, 186, 51], [233, 225, 203], [230, 230, 127], [77, 19, 241], [118, 228, 16], [98, 250, 160], [215, 230, 171], [99, 230, 219], [57, 78, 32], [90, 27, 75], [68, 37, 175], [67, 150, 223], [87, 184, 186], [242, 32, 65], [62, 117, 215], [213, 7, 77], [229, 159, 109], [229, 129, 248], [116, 192, 63], [40, 66, 169], [40, 197, 165], [217, 211, 117], [83, 216, 122], [145, 192, 24], [42, 248, 152], [18, 53, 12], [92, 39, 78], [103, 233, 74], [90, 227, 130], [169, 171, 6], [0, 8, 188], [127, 126, 159], [142, 133, 112], [14, 31, 88], [124, 236, 43], [244, 205, 51], [124, 129, 96], [238, 179, 130], [162, 219, 58], [8, 75, 119], [59, 225, 168], [15, 106, 4], [8, 187, 103], [215, 156, 147], [249, 11, 188], [185, 89, 231], [152, 83, 90], [70, 227, 157], [64, 138, 191], [235, 94, 91], [101, 96, 149], [168, 77, 222], [48, 238, 6], [80, 143, 244], [17, 196, 216], [94, 181, 58], [245, 218, 179], [53, 29, 124], [249, 103, 255], [36, 228, 51], [78, 69, 193], [28, 127, 74], [143, 164, 160], [173, 175, 201], [224, 110, 172], [173, 163, 175], [139, 142, 28], [237, 71, 13], [245, 35, 220], [111, 166, 63], [54, 210, 114], [152, 204, 178], [157, 129, 214], [72, 93, 40], [101, 245, 165], [139, 77, 204], [106, 175, 43], [154, 12, 71], [75, 15, 47], [87, 22, 251], [73, 167, 20], [89, 204, 144], [79, 229, 45], [121, 203, 66], [22, 65, 65], [222, 135, 84], [134, 198, 225], [21, 84, 153], [137, 255, 251], [120, 139, 15], [95, 246, 52], [244, 186, 101], [70, 27, 48], [145, 254, 94], [231, 79, 61], [249, 149, 97], [91, 181, 246], [182, 92, 84], [39, 124, 147], [235, 121, 80], [171, 251, 11], [105, 74, 138], [95, 137, 231], [12, 101, 251], [63, 46, 2], [208, 80, 138], [54, 135, 78], [152, 215, 93], [200, 38, 179], [13, 83, 120], [2, 54, 23], [88, 85, 28], [250, 68, 188], [116, 103, 194], [182, 196, 9], [85, 94, 66], [133, 89, 160], [232, 118, 51], [73, 136, 250], [24, 188, 140], [231, 106, 84], [55, 191, 244], [38, 33, 182], [98, 143, 216], [47, 204, 144], [246, 42, 212], [243, 223, 77], [219, 200, 209], [155, 49, 227], [240, 155, 126], [99, 42, 21], [222, 193, 217], [248, 206, 6], [12, 221, 101], [59, 179, 67], [211, 66, 26], [74, 188, 12], [7, 120, 30], [14, 245, 30], [92, 251, 14], [33, 87, 114], [229, 102, 60], [174, 54, 222], [124, 35, 227], [15, 60, 161], [192, 177, 152], [217, 111, 239], [242, 150, 246], [198, 143, 35], [105, 41, 68], [142, 103, 235], [148, 153, 46], [32, 249, 101], [18, 131, 98], [248, 5, 10], [138, 110, 253], [57, 72, 162], [98, 22, 146], [252, 116, 190], [83, 249, 32], [254, 75, 76], [132, 52, 162], [237, 171, 224], [214, 138, 38], [174, 194, 164], [237, 76, 10], [192, 190, 24], [83, 28, 1], [253, 23, 60], [218, 243, 193], [79, 182, 171], [116, 122, 51], [207, 26, 219], [223, 32, 184], [134, 251, 129], [152, 119, 211], [56, 197, 33], [36, 226, 178], [22, 81, 110], [148, 155, 77], [230, 247, 211], [174, 196, 86], [142, 85, 15], [70, 13, 75], [78, 123, 192], [105, 187, 21], [229, 85, 143], [25, 127, 106], [105, 64, 115], [87, 10, 214], [12, 4, 156], [127, 90, 206], [193, 227, 160], [38, 169, 61], [232, 88, 250], [21, 3, 22], [86, 224, 249], [73, 33, 87], [198, 126, 93], [151, 159, 16], [87, 16, 199], [190, 212, 38], [51, 153, 204], [27, 126, 172], [89, 83, 228], [119, 145, 213], [100, 10, 172], [29, 133, 127], [99, 24, 94], [159, 251, 89], [3, 254, 129], [129, 34, 140], [250, 28, 125], [44, 189, 108], [130, 129, 118], [34, 242, 153], [194, 179, 252], [211, 170, 61], [59, 164, 74], [154, 60, 158], [129, 102, 203], [102, 56, 145], [120, 29, 128], [114, 248, 204], [204, 95, 185], [208, 93, 217], [9, 57, 42], [178, 177, 85], [245, 251, 41], [104, 115, 247], [241, 180, 247], [136, 99, 26], [33, 6, 1], [104, 64, 234], [63, 85, 145], [211, 66, 94], [100, 149, 239], [129, 240, 161], [107, 27, 113], [233, 48, 47], [95, 155, 230], [159, 9, 43], [146, 28, 149], [69, 253, 209], [105, 107, 177], [215, 218, 191], [215, 77, 11], [96, 10, 121], [112, 229, 241], [82, 106, 154], [165, 130, 61], [241, 42, 33], [117, 62, 133], [23, 227, 114], [246, 237, 141], [68, 76, 98], [41, 57, 254], [225, 88, 217], [9, 253, 220], [162, 163, 241], [165, 21, 79], [252, 113, 93], [160, 113, 154], [245, 182, 228], [199, 87, 41], [194, 93, 244], [227, 167, 248], [69, 124, 167], [171, 195, 39], [240, 229, 129], [95, 158, 215], [11, 16, 144], [251, 63, 159], [238, 88, 215], [248, 135, 72], [30, 136, 4], [224, 68, 163], [6, 54, 104], [70, 174, 74], [39, 69, 176], [226, 33, 109], [17, 98, 73], [57, 10, 250], [233, 18, 85], [135, 218, 129], [66, 202, 241], [49, 204, 37], [187, 131, 109], [137, 185, 53], [108, 58, 83], [242, 168, 97], [95, 121, 173], [34, 244, 87], [61, 52, 143], [108, 48, 25], [91, 5, 153], [149, 21, 130], [31, 101, 116], [206, 150, 110], [228, 11, 113], [42, 206, 213], [79, 109, 134], [26, 186, 214], [160, 56, 176], [51, 14, 75], [35, 78, 255], [29, 118, 8], [195, 77, 16], [174, 64, 239], [167, 235, 103], [19, 208, 197], [251, 190, 47], [222, 51, 148], [1, 155, 210], [36, 241, 42], [187, 144, 184], [55, 104, 83], [172, 104, 126], [116, 60, 171], [21, 30, 167], [8, 156, 174], [84, 202, 11], [156, 5, 62], [233, 33, 191], [92, 4, 213], [220, 239, 242], [66, 250, 140], [76, 148, 245], [45, 71, 60], [118, 13, 75], [153, 132, 59], [190, 184, 197], [187, 181, 236], [183, 32, 209], [70, 80, 86], [215, 226, 119], [189, 55, 64], [111, 240, 246], [180, 214, 251], [13, 212, 35], [93, 7, 213], [75, 188, 77], [103, 88, 22], [85, 234, 108], [181, 68, 82], [163, 225, 188], [32, 21, 47], [81, 217, 87], [80, 141, 26], [74, 214, 233], [35, 188, 237], [129, 63, 201], [253, 65, 80], [80, 195, 73], [157, 111, 96], [165, 235, 95], [221, 183, 250], [67, 111, 118], [117, 116, 17], [232, 183, 160], [196, 110, 156], [205, 53, 70], [61, 214, 9], [155, 97, 153], [214, 239, 19], [218, 54, 170], [73, 137, 162], [162, 13, 208], [64, 0, 81], [207, 32, 74], [243, 187, 27], [172, 212, 7], [182, 170, 97], [146, 19, 215], [111, 234, 73], [170, 13, 24], [99, 149, 223], [105, 71, 165], [45, 238, 226], [204, 202, 48], [182, 59, 171], [244, 253, 123], [156, 255, 252], [97, 117, 202], [47, 127, 10], [62, 213, 218], [189, 249, 4], [123, 251, 143], [75, 134, 62], [50, 157, 153], [200, 230, 112], [106, 190, 0], [200, 191, 132], [242, 234, 186], [138, 217, 103], [154, 17, 189], [231, 170, 4], [201, 168, 85], [195, 75, 195], [46, 54, 24], [47, 146, 213], [39, 139, 31], [119, 161, 4], [81, 76, 214], [193, 233, 5], [58, 249, 104], [23, 224, 65], [130, 253, 33], [70, 87, 12], [59, 161, 4], [234, 158, 136], [189, 96, 236], [99, 92, 195], [194, 15, 240], [94, 8, 25], [147, 81, 198], [225, 252, 95], [63, 216, 8], [99, 114, 55], [87, 149, 22], [94, 247, 207], [223, 174, 100], [192, 219, 240], [218, 137, 19], [199, 236, 52], [41, 98, 218], [70, 109, 199], [70, 47, 14], [26, 32, 55], [214, 109, 223], [73, 59, 98], [73, 221, 92], [103, 60, 137], [62, 144, 60], [66, 3, 119], [184, 206, 82], [177, 3, 205], [21, 53, 180], [121, 97, 30], [214, 196, 111], [151, 28, 233], [46, 124, 220], [116, 16, 127], [137, 18, 41], [101, 158, 35], [121, 215, 19], [200, 192, 10], [88, 69, 56], [237, 184, 74], [222, 152, 178], [40, 232, 236], [106, 64, 19], [31, 52, 72], [159, 144, 79], [133, 65, 200], [22, 3, 93], [233, 127, 87], [202, 139, 244], [90, 42, 191], [248, 73, 163], [156, 93, 110], [25, 135, 90], [47, 126, 140], [198, 182, 205], [218, 92, 178], [143, 227, 168], [148, 134, 165], [12, 117, 86], [31, 150, 221], [241, 220, 208], [214, 35, 244], [54, 148, 65], [48, 248, 108], [153, 18, 14], [201, 251, 89], [189, 152, 87], [149, 50, 19], [22, 96, 191], [233, 33, 187], [118, 74, 102], [55, 113, 49], [17, 143, 236], [181, 244, 179], [51, 187, 88], [208, 110, 130], [16, 73, 214], [18, 47, 20], [214, 134, 97], [96, 223, 193], [88, 196, 213], [27, 175, 227], [5, 59, 94], [213, 43, 169], [139, 46, 162], [57, 99, 129], [134, 243, 52], [20, 230, 71], [124, 104, 37], [60, 175, 2], [14, 112, 144], [253, 236, 169], [87, 204, 32], [130, 222, 241], [149, 2, 235], [82, 79, 47], [245, 206, 77], [117, 220, 38], [226, 197, 136], [161, 71, 90], [25, 186, 159], [8, 108, 2], [106, 162, 215], [242, 92, 242], [33, 166, 209], [183, 99, 248], [59, 194, 223], [187, 166, 228], [98, 153, 134], [45, 71, 182], [82, 251, 100], [106, 125, 137], [218, 162, 52], [106, 58, 244], [66, 40, 244], [246, 240, 143], [254, 85, 128], [165, 194, 157], [252, 204, 8], [253, 124, 224], [17, 158, 215], [96, 32, 24], [155, 186, 186], [252, 77, 250], [107, 42, 173], [58, 219, 65], [132, 162, 252], [6, 95, 78], [178, 106, 7], [61, 166, 63], [153, 24, 70], [133, 238, 232], [82, 238, 57], [219, 170, 185], [79, 91, 200], [42, 184, 55], [124, 217, 72], [237, 10, 33], [84, 190, 204], [108, 187, 113], [2, 198, 235], [46, 24, 215], [208, 75, 55], [42, 13, 115], [86, 178, 214], [182, 102, 253], [147, 169, 233], [168, 107, 255], [50, 109, 36], [193, 204, 134], [133, 156, 207], [126, 43, 13], [215, 255, 237], [222, 223, 46], [249, 140, 71], [32, 73, 24], [15, 180, 234], [140, 189, 81], [36, 138, 7], [19, 41, 200], [156, 161, 217], [200, 37, 23], [103, 53, 150], [61, 238, 215], [157, 118, 175], [10, 203, 9], [248, 217, 90], [243, 129, 52], [166, 15, 176], [47, 213, 92], [255, 95, 87], [96, 31, 185], [31, 141, 73], [189, 110, 166], [211, 18, 78], [43, 16, 40], [5, 124, 81], [215, 219, 12], [141, 98, 30], [173, 36, 152], [175, 170, 132], [107, 220, 125], [61, 33, 209], [229, 148, 238], [8, 255, 118], [232, 133, 241], [22, 85, 196], [29, 15, 118], [17, 85, 17], [236, 202, 17], [127, 221, 30], [232, 178, 182], [118, 133, 91], [207, 131, 253], [149, 226, 70], [221, 252, 236], [42, 0, 209], [196, 231, 64], [160, 47, 33], [249, 230, 103], [96, 170, 67], [107, 21, 190], [66, 113, 13], [125, 14, 33], [179, 107, 155], [108, 48, 73], [250, 244, 28], [178, 116, 11], [157, 78, 146], [5, 31, 240], [237, 227, 233], [160, 42, 169], [89, 41, 5], [174, 87, 61], [5, 130, 187], [216, 153, 28], [141, 127, 10], [21, 86, 15], [37, 236, 146], [30, 135, 96], [185, 44, 100], [59, 159, 111], [161, 22, 24], [52, 179, 94], [131, 62, 191], [195, 3, 212], [134, 131, 108], [11, 82, 86], [31, 201, 153], [242, 67, 150], [226, 228, 41], [53, 139, 68], [146, 102, 75], [165, 203, 8], [246, 252, 157], [94, 150, 85], [34, 38, 178], [18, 149, 164], [67, 60, 43], [245, 164, 127], [172, 251, 86], [95, 175, 77], [146, 159, 119], [164, 48, 65], [199, 105, 165], [118, 176, 181], [47, 59, 104], [206, 82, 20], [46, 214, 168], [210, 206, 123], [9, 52, 124], [110, 99, 198], [191, 188, 195], [171, 149, 40], [151, 62, 113], [27, 152, 146], [77, 234, 82], [94, 119, 25], [44, 191, 19], [44, 78, 191], [74, 240, 33], [2, 190, 75], [133, 42, 145], [76, 153, 204], [178, 126, 141], [57, 101, 93], [29, 164, 50], [118, 165, 171], [64, 35, 15], [172, 210, 245], [106, 157, 27], [78, 115, 161], [243, 118, 33], [129, 196, 185], [218, 212, 143], [127, 163, 207], [89, 204, 174], [151, 172, 81], [9, 98, 215], [212, 9, 236], [164, 86, 144], [250, 106, 44], [183, 8, 229], [200, 155, 72], [86, 30, 142], [226, 232, 11], [224, 63, 101], [192, 40, 63], [195, 237, 72], [125, 168, 243], [196, 0, 199], [230, 90, 40], [170, 22, 232], [79, 196, 176], [183, 40, 229], [60, 37, 140], [43, 245, 231], [178, 45, 203], [107, 197, 251], [204, 248, 7], [75, 70, 199], [70, 122, 92], [187, 4, 123], [107, 126, 187], [229, 128, 0], [206, 155, 91], [171, 253, 149], [137, 128, 158], [32, 213, 136], [11, 28, 120], [30, 77, 247], [85, 48, 142], [49, 161, 190], [185, 172, 59], [96, 171, 79], [254, 57, 244], [45, 91, 36], [105, 94, 138], [145, 244, 33], [135, 17, 162], [119, 244, 208], [93, 21, 253], [195, 186, 128], [38, 29, 11], [196, 107, 146], [206, 1, 177], [67, 74, 79], [227, 135, 5], [13, 162, 153], [50, 37, 147], [52, 131, 233], [1, 113, 37], [181, 141, 198], [211, 60, 246], [197, 201, 229], [119, 236, 83], [89, 219, 135], [92, 36, 74], [96, 34, 141], [153, 70, 178], [217, 38, 236], [150, 83, 38], [40, 14, 166], [179, 185, 248], [180, 36, 51], [24, 231, 89], [49, 194, 12], [112, 204, 150], [81, 142, 223], [29, 223, 178], [82, 219, 212], [209, 119, 32], [103, 120, 186], [96, 224, 192], [63, 225, 37], [206, 206, 129], [179, 119, 159], [232, 93, 128], [232, 166, 176], [143, 233, 64], [175, 91, 218], [46, 93, 136], [159, 146, 32], [246, 39, 165], [172, 199, 13], [154, 191, 94], [249, 93, 190], [83, 116, 124], [38, 49, 55], [209, 114, 152], [92, 130, 55], [227, 140, 70], [90, 245, 216], [73, 133, 226], [161, 29, 202], [182, 132, 229], [224, 93, 253], [200, 143, 131], [116, 237, 161], [157, 91, 45], [76, 238, 129], [87, 95, 92], [205, 94, 248], [104, 96, 75], [196, 80, 37], [216, 148, 198], [58, 104, 189], [252, 142, 134], [217, 226, 117], [175, 30, 173], [253, 219, 86], [128, 167, 74], [215, 187, 52], [139, 200, 216], [216, 51, 12], [172, 68, 8], [179, 55, 182], [112, 226, 193], [1, 123, 240], [132, 202, 54], [201, 36, 26], [163, 48, 121], [85, 104, 201], [198, 218, 45], [52, 202, 33], [162, 232, 9], [202, 211, 202], [17, 205, 15], [45, 69, 5], [181, 170, 242], [241, 83, 54], [0, 152, 193], [75, 146, 8], [171, 138, 114], [166, 192, 171], [139, 225, 163], [124, 18, 21], [217, 17, 237], [237, 199, 35], [38, 207, 255], [172, 188, 248], [121, 146, 141], [202, 91, 222], [230, 68, 70], [28, 145, 30], [10, 36, 210], [235, 230, 236], [93, 3, 37], [99, 216, 242], [107, 60, 220], [83, 239, 31], [1, 174, 233], [228, 217, 51], [46, 110, 121], [196, 216, 9], [45, 246, 121], [210, 244, 119], [134, 147, 184], [197, 159, 214], [86, 32, 133], [25, 215, 167], [226, 95, 81], [119, 111, 234], [113, 119, 45], [69, 102, 212], [39, 60, 30], [58, 123, 128], [32, 115, 230], [8, 39, 75], [66, 45, 239], [77, 16, 118], [234, 105, 9], [103, 24, 103], [251, 181, 221], [25, 252, 176], [121, 199, 187], [18, 15, 30], [37, 243, 54], [53, 195, 95], [234, 112, 88], [156, 202, 55], [219, 6, 251], [160, 127, 143], [61, 47, 225], [23, 92, 108], [46, 81, 217], [141, 188, 16], [143, 130, 118], [108, 185, 160], [112, 178, 234], [181, 165, 115], [41, 198, 49], [75, 197, 49], [29, 150, 97], [135, 224, 221], [33, 159, 64], [5, 72, 156], [223, 254, 93], [42, 86, 197], [75, 34, 193], [252, 205, 248], [214, 128, 113], [53, 195, 167], [7, 52, 169], [141, 87, 241], [63, 101, 88], [133, 248, 31], [213, 54, 210], [154, 86, 91], [178, 119, 50], [51, 172, 211], [23, 203, 249], [108, 111, 220], [33, 162, 154], [71, 160, 65], [128, 97, 53], [143, 84, 142], [70, 147, 220], [1, 86, 165], [57, 156, 90], [83, 33, 172], [173, 143, 128], [185, 251, 185], [248, 104, 84], [161, 163, 57], [204, 178, 233], [61, 78, 238], [37, 46, 6], [27, 232, 232], [95, 168, 96], [136, 53, 215], [46, 14, 244], [248, 166, 157], [145, 27, 241], [24, 82, 132], [173, 88, 100], [112, 53, 159], [56, 219, 59], [175, 15, 165], [148, 21, 154], [67, 244, 252], [197, 242, 47], [227, 75, 60], [132, 155, 146], [249, 225, 57], [32, 207, 25], [144, 74, 8], [107, 128, 124], [140, 181, 214], [108, 156, 245], [237, 86, 223], [156, 54, 42], [39, 121, 162], [177, 19, 139], [252, 122, 128], [29, 224, 74], [237, 217, 49], [189, 100, 240], [96, 247, 47], [237, 5, 177], [203, 139, 26], [92, 143, 12], [62, 242, 146], [144, 48, 46], [54, 92, 221], [83, 99, 182], [100, 236, 210], [200, 162, 38], [59, 128, 203], [63, 69, 202], [164, 60, 219], [189, 157, 221], [190, 124, 156], [186, 249, 45], [65, 191, 88], [250, 212, 182], [223, 158, 97], [0, 51, 117], [105, 88, 124], [47, 72, 199], [8, 174, 233], [128, 188, 181], [12, 224, 41], [185, 235, 4], [125, 7, 250], [226, 175, 214], [67, 57, 236], [144, 40, 75], [243, 32, 153], [55, 209, 122], [228, 200, 205], [3, 72, 93], [223, 145, 97], [170, 208, 41], [105, 111, 165], [130, 136, 109], [208, 222, 220], [147, 222, 85], [43, 181, 243], [118, 156, 205], [130, 218, 229], [39, 95, 241], [78, 127, 24], [183, 173, 135], [148, 6, 73], [40, 0, 80], [201, 220, 11], [102, 194, 254], [242, 48, 157], [92, 195, 68], [229, 197, 235], [140, 20, 157], [130, 221, 146], [225, 163, 129], [213, 144, 108], [230, 107, 210], [105, 145, 201], [28, 63, 200], [252, 183, 153], [57, 176, 152], [194, 218, 164], [134, 6, 175], [4, 174, 45], [215, 199, 47], [219, 182, 212], [230, 55, 9], [48, 37, 208], [94, 56, 216], [97, 177, 124], [205, 69, 211], [71, 172, 155], [93, 166, 23], [78, 81, 199], [28, 244, 72], [203, 80, 227], [76, 212, 29], [60, 104, 89], [226, 95, 149], [228, 124, 184], [165, 197, 225], [182, 145, 124], [109, 224, 89], [219, 170, 120], [114, 184, 30], [156, 177, 176], [237, 109, 21], [242, 13, 36], [162, 22, 254], [21, 215, 24], [159, 191, 104], [26, 176, 237], [208, 205, 208], [177, 124, 234], [170, 142, 124], [112, 48, 234], [161, 108, 203], [56, 166, 198], [57, 181, 163], [80, 98, 130], [225, 190, 115], [103, 142, 155], [212, 233, 163], [237, 63, 140], [20, 96, 135], [172, 56, 151], [75, 115, 104], [16, 24, 94], [50, 14, 47], [0, 35, 109], [230, 18, 181], [123, 42, 160], [252, 196, 164], [199, 155, 2], [182, 251, 168], [28, 122, 139], [140, 246, 242], [127, 69, 17], [12, 195, 247], [107, 48, 36], [229, 182, 193], [234, 73, 9], [253, 178, 200], [105, 23, 78], [108, 111, 223], [186, 245, 255], [219, 240, 85], [177, 42, 46], [2, 85, 149], [159, 229, 219], [234, 136, 2], [2, 162, 82], [143, 196, 126], [103, 34, 2], [136, 76, 164], [45, 17, 170], [128, 205, 251], [139, 217, 141], [128, 195, 238], [204, 117, 125], [109, 88, 127], [250, 75, 217], [252, 208, 142], [177, 242, 108], [169, 76, 115], [186, 173, 67], [206, 65, 15], [171, 233, 195], [204, 44, 172], [15, 208, 29], [98, 158, 215], [15, 249, 65], [65, 73, 8], [25, 240, 224], [49, 104, 13], [237, 24, 106], [84, 131, 100], [246, 101, 73], [167, 152, 20], [111, 169, 244], [73, 104, 209], [105, 233, 149], [189, 207, 117], [21, 130, 227], [74, 32, 76], [248, 17, 42], [28, 182, 47], [57, 181, 231], [147, 0, 109], [18, 76, 20], [44, 59, 44], [223, 24, 26], [194, 60, 4], [44, 195, 36], [248, 214, 145], [176, 60, 175], [134, 199, 250], [38, 205, 82], [72, 201, 194], [122, 43, 28], [68, 60, 101], [22, 55, 87], [162, 168, 3], [240, 49, 179], [101, 24, 186], [231, 64, 117], [153, 203, 159], [169, 191, 205], [189, 182, 0], [40, 119, 36], [234, 52, 211], [186, 187, 198], [111, 169, 163], [212, 131, 225], [176, 51, 61], [104, 36, 82], [54, 167, 35], [4, 95, 223], [193, 5, 131], [90, 54, 193], [91, 103, 168], [107, 163, 124], [245, 174, 240], [147, 16, 218], [19, 109, 19], [64, 151, 151], [233, 65, 207], [156, 48, 34], [83, 127, 131], [77, 195, 142], [7, 222, 128], [251, 54, 200], [113, 224, 79], [158, 213, 156], [226, 242, 182], [62, 136, 161], [162, 141, 41], [29, 66, 47], [158, 108, 20], [112, 25, 36], [29, 159, 198], [45, 250, 149], [148, 197, 0], [36, 158, 186], [94, 184, 5], [22, 88, 131], [212, 166, 33], [68, 125, 243], [2, 63, 156], [176, 133, 25], [82, 123, 240], [237, 88, 250], [178, 68, 98], [223, 143, 124], [100, 193, 47], [39, 197, 206], [143, 204, 229], [10, 193, 107], [2, 164, 245], [83, 149, 172], [136, 37, 221], [63, 203, 45], [0, 223, 195], [249, 5, 123], [106, 194, 224], [189, 48, 204], [170, 143, 212], [10, 101, 2], [11, 10, 111], [21, 165, 23], [189, 131, 61], [205, 104, 214], [224, 228, 92], [3, 181, 125], [9, 85, 50], [62, 28, 58], [78, 93, 171], [6, 184, 11], [87, 245, 105], [158, 220, 133], [123, 74, 187], [150, 25, 65], [209, 120, 100], [222, 241, 49], [239, 120, 101], [248, 198, 203], [117, 99, 226], [11, 248, 190], [123, 14, 48], [162, 93, 251], [105, 52, 211], [57, 237, 23], [180, 170, 17], [147, 190, 1], [207, 85, 131], [104, 69, 24], [88, 233, 189], [24, 162, 26], [19, 33, 191], [174, 109, 219], [38, 213, 197], [30, 120, 149], [112, 0, 203], [167, 193, 123], [117, 189, 164], [61, 25, 232], [237, 245, 133], [137, 245, 208], [69, 35, 160], [62, 253, 28], [108, 176, 24], [107, 185, 54], [140, 32, 204], [212, 189, 214], [18, 135, 170], [169, 199, 143], [156, 253, 178], [92, 95, 112], [125, 10, 16], [41, 238, 44], [135, 144, 90], [6, 144, 210], [4, 217, 35], [136, 33, 161], [52, 39, 206], [78, 213, 120], [17, 80, 61], [19, 120, 116], [0, 18, 100], [245, 171, 40], [206, 131, 17], [175, 110, 214], [47, 137, 228], [97, 165, 174], [151, 242, 106], [14, 184, 88], [99, 25, 222], [37, 12, 14], [7, 237, 228], [9, 203, 40], [185, 67, 228], [170, 71, 201], [119, 163, 132], [112, 235, 31], [165, 212, 234], [70, 63, 59], [182, 187, 45], [70, 68, 96], [222, 137, 15], [177, 70, 71], [153, 7, 164], [38, 108, 42], [213, 117, 16], [84, 67, 91], [213, 206, 235], [178, 38, 250], [26, 5, 65], [156, 228, 63], [10, 148, 52], [27, 139, 145], [154, 223, 98], [95, 46, 240], [44, 56, 78], [171, 83, 238], [106, 165, 224], [184, 242, 227], [237, 159, 180], [120, 35, 122], [14, 137, 57], [52, 92, 88], [214, 14, 243], [111, 30, 191], [135, 86, 250], [129, 97, 78], [13, 35, 171], [96, 97, 14], [26, 49, 31], [69, 124, 210], [88, 100, 85], [163, 142, 233], [52, 111, 182], [179, 158, 40], [46, 131, 112], [225, 89, 197], [72, 142, 146], [218, 107, 166], [111, 120, 34], [99, 51, 69], [53, 83, 6], [235, 204, 33], [168, 241, 20], [245, 4, 149], [141, 23, 202], [117, 103, 108], [13, 154, 24], [245, 231, 187], [75, 63, 173], [12, 199, 126], [145, 173, 228], [173, 141, 174], [212, 131, 121], [217, 195, 221], [67, 249, 194], [7, 44, 210], [76, 200, 95], [11, 234, 137], [147, 20, 163], [225, 41, 51], [186, 54, 130], [75, 203, 108], [17, 210, 93], [125, 98, 58], [133, 117, 217], [253, 229, 155], [69, 205, 22], [13, 221, 199], [81, 115, 100], [237, 97, 12], [155, 121, 30], [109, 53, 248], [175, 7, 34], [199, 38, 203], [214, 148, 214], [30, 63, 251], [83, 156, 116], [213, 23, 116], [203, 95, 161], [136, 55, 195], [172, 65, 121], [197, 192, 1], [89, 162, 119], [93, 131, 99], [161, 117, 86], [186, 115, 200], [97, 154, 115], [104, 40, 41], [240, 5, 141], [136, 203, 96], [31, 161, 181], [86, 147, 18], [110, 147, 123], [242, 108, 96], [39, 194, 111], [202, 29, 73], [56, 13, 32], [231, 202, 135], [148, 11, 165], [195, 165, 87], [198, 186, 115], [169, 99, 178], [30, 205, 15], [62, 58, 97], [62, 254, 227], [97, 0, 178], [155, 183, 57], [202, 83, 105], [212, 10, 168], [72, 100, 15], [54, 145, 145], [18, 187, 142], [241, 158, 156], [119, 84, 123], [145, 158, 52], [234, 110, 173], [161, 248, 85], [201, 70, 254], [108, 33, 96], [7, 233, 13], [77, 24, 88], [57, 212, 122], [74, 145, 133], [91, 235, 234], [222, 154, 211], [207, 250, 221], [156, 140, 83], [226, 196, 125], [104, 3, 155], [50, 224, 195], [250, 131, 221], [35, 183, 85], [127, 31, 126], [239, 162, 46], [123, 21, 164], [103, 16, 29], [159, 65, 65], [127, 12, 18], [55, 238, 20], [21, 29, 192], [17, 117, 66], [127, 60, 252], [104, 199, 25], [204, 9, 73], [219, 13, 247], [155, 54, 52], [1, 126, 95], [35, 90, 169], [211, 12, 212], [186, 50, 22], [224, 6, 196], [204, 45, 108], [129, 179, 80], [15, 126, 197], [28, 34, 113], [37, 252, 210], [161, 92, 211], [87, 235, 185], [235, 35, 132], [166, 199, 85], [88, 244, 130], [41, 246, 59], [60, 39, 131], [6, 137, 91], [46, 172, 206], [46, 109, 253], [52, 113, 154], [89, 38, 70], [131, 64, 102], [49, 40, 13], [184, 154, 189], [174, 135, 232], [2, 33, 178], [26, 90, 36], [19, 162, 38], [73, 126, 192], [237, 21, 73], [210, 27, 103], [212, 1, 136], [10, 199, 156], [116, 202, 213], [194, 17, 148], [44, 249, 219], [222, 189, 47], [12, 168, 245], [78, 18, 154], [98, 206, 177], [177, 113, 173], [48, 254, 80], [156, 25, 94], [57, 245, 196], [244, 71, 5], [115, 211, 178], [43, 239, 188], [4, 133, 83], [232, 237, 9], [72, 67, 90], [227, 223, 9], [64, 13, 195], [91, 15, 98], [237, 109, 47], [175, 255, 194], [127, 246, 65], [179, 45, 208], [79, 167, 7], [105, 102, 21], [253, 203, 129], [179, 135, 252], [246, 198, 0], [95, 33, 236], [185, 1, 52], [176, 75, 169], [67, 81, 45], [206, 159, 13], [30, 99, 226], [26, 89, 85], [220, 176, 25], [124, 237, 116], [56, 171, 41], [38, 42, 155], [217, 77, 236], [140, 246, 33], [222, 54, 206], [40, 254, 134], [153, 171, 34], [145, 84, 80], [248, 62, 112], [44, 203, 111], [13, 148, 82], [229, 152, 207], [144, 95, 75], [125, 232, 71], [33, 181, 255], [194, 33, 46], [131, 177, 47], [111, 94, 93], [80, 79, 97], [65, 140, 166], [153, 214, 125], [243, 244, 200], [245, 221, 204], [108, 130, 223], [103, 34, 40], [94, 189, 133], [98, 227, 130], [237, 159, 233], [56, 126, 221], [151, 191, 209], [245, 22, 89], [157, 114, 21], [25, 96, 3], [130, 163, 10], [177, 185, 224], [138, 11, 229], [213, 225, 233], [118, 56, 234], [189, 220, 88], [172, 47, 146], [152, 207, 28], [28, 162, 224], [9, 125, 117], [223, 221, 156], [163, 221, 214], [121, 7, 222], [148, 182, 25], [107, 20, 74], [0, 51, 98], [168, 172, 177], [77, 61, 52], [199, 202, 33], [144, 208, 17], [165, 115, 10], [49, 255, 143], [1, 157, 37], [12, 65, 108], [174, 123, 250], [195, 183, 4], [201, 119, 193], [179, 0, 246], [48, 193, 236], [137, 98, 166], [169, 219, 242], [107, 172, 241], [30, 213, 136], [30, 248, 74], [217, 230, 235], [94, 250, 90], [122, 159, 85], [176, 132, 94], [108, 90, 248], [240, 121, 254], [140, 91, 196], [5, 187, 2], [206, 217, 212], [101, 137, 227], [229, 80, 102], [217, 170, 30], [203, 251, 45], [118, 187, 3], [33, 229, 190], [248, 221, 88], [74, 93, 133], [156, 53, 114], [146, 192, 95], [162, 192, 94], [192, 87, 26], [175, 66, 239], [91, 134, 99], [249, 95, 57], [223, 129, 51], [210, 204, 37], [171, 97, 195], [126, 51, 212], [85, 130, 197], [203, 222, 239], [188, 186, 67], [252, 232, 59], [198, 211, 29], [35, 171, 65], [35, 239, 238], [196, 178, 164], [36, 131, 188], [53, 126, 232], [70, 197, 204], [192, 187, 232], [197, 112, 195], [205, 159, 148], [161, 227, 238], [198, 147, 68], [5, 248, 159], [123, 70, 58], [138, 234, 252], [253, 41, 58], [153, 104, 203], [65, 219, 203], [255, 193, 119], [140, 242, 230], [171, 222, 184], [5, 140, 48], [8, 161, 10], [208, 13, 131], [30, 20, 188], [134, 180, 200], [88, 43, 4], [225, 112, 126], [205, 48, 211], [35, 196, 51], [129, 208, 155], [201, 223, 73], [157, 221, 193], [216, 1, 162], [67, 222, 177], [238, 154, 70], [1, 4, 25], [22, 71, 148], [213, 4, 203], [10, 103, 248], [72, 210, 89], [181, 10, 41], [112, 68, 214], [0, 21, 24], [40, 33, 208], [241, 111, 221], [0, 229, 48], [246, 178, 162], [213, 34, 171], [244, 23, 14], [3, 200, 100], [154, 215, 143], [223, 13, 121], [179, 117, 238], [200, 85, 123], [189, 40, 240], [75, 67, 67], [227, 112, 247], [154, 240, 6], [76, 168, 212], [232, 116, 220], [79, 213, 117], [166, 42, 96], [48, 101, 17], [194, 239, 146], [208, 209, 254], [11, 90, 224], [195, 67, 22], [107, 202, 173], [78, 197, 199], [65, 99, 24], [168, 126, 111], [158, 219, 162], [150, 50, 252], [78, 144, 23], [203, 116, 60], [98, 66, 144], [183, 110, 167], [53, 11, 63], [98, 71, 217], [102, 26, 200], [117, 56, 108], [63, 166, 121], [248, 48, 216], [20, 227, 121], [229, 147, 247], [92, 103, 53], [125, 88, 64], [122, 155, 151], [122, 87, 138], [69, 238, 229], [27, 81, 94], [115, 154, 87], [173, 70, 223], [155, 126, 53], [211, 99, 96], [237, 240, 118], [73, 75, 157], [105, 20, 239], [67, 192, 35], [18, 149, 229], [96, 32, 65], [103, 105, 104], [81, 60, 89], [253, 142, 238], [216, 251, 213], [214, 30, 24], [183, 206, 47], [90, 89, 158], [61, 92, 72], [211, 84, 210], [204, 41, 98], [184, 10, 155], [20, 249, 143], [245, 95, 108], [76, 18, 62], [127, 240, 70], [183, 90, 104], [4, 56, 32], [2, 78, 127], [56, 81, 129], [197, 246, 94], [173, 23, 122], [207, 17, 88], [60, 5, 0], [160, 229, 149], [84, 26, 61], [34, 63, 10], [182, 59, 58], [103, 217, 108], [87, 72, 222], [118, 77, 192], [19, 65, 154], [129, 159, 15], [7, 159, 164], [69, 197, 176], [167, 127, 51], [122, 8, 4], [57, 94, 45], [111, 170, 197], [106, 43, 238], [153, 73, 200], [239, 239, 112], [97, 59, 239], [215, 195, 25], [34, 127, 234], [202, 44, 33], [38, 255, 65], [120, 82, 123], [62, 54, 8], [242, 113, 164], [194, 65, 84], [244, 210, 142], [146, 151, 195], [160, 118, 134], [55, 124, 233], [251, 181, 57], [217, 111, 84], [204, 23, 25], [50, 159, 182], [14, 231, 86], [104, 181, 127], [144, 218, 252], [3, 143, 191], [42, 191, 207], [250, 30, 7], [86, 53, 121], [4, 227, 191], [127, 66, 227], [166, 0, 64], [8, 195, 99], [54, 138, 43], [172, 196, 91], [251, 246, 209], [32, 224, 40], [70, 97, 192], [254, 249, 196], [62, 233, 242], [118, 240, 112], [181, 241, 135], [5, 172, 139], [40, 36, 208], [217, 62, 148], [35, 105, 233], [129, 231, 149], [186, 199, 171], [17, 242, 213], [50, 161, 20], [234, 157, 184], [83, 195, 62], [153, 36, 20], [220, 63, 247], [152, 46, 194], [16, 217, 238], [29, 143, 234], [180, 76, 114], [44, 26, 229], [85, 80, 91], [243, 63, 36], [54, 121, 240], [69, 119, 26], [118, 160, 76], [16, 172, 62], [211, 16, 213], [192, 226, 217], [121, 46, 248], [163, 231, 203], [195, 30, 153], [72, 22, 103], [203, 33, 48], [93, 47, 95], [95, 196, 14], [219, 200, 128], [251, 62, 237], [89, 113, 105], [184, 235, 1], [106, 28, 221], [11, 45, 237], [203, 59, 242], [52, 63, 176], [176, 243, 25], [215, 128, 152], [187, 5, 197], [237, 189, 129], [115, 181, 87], [32, 104, 172], [86, 166, 1], [201, 235, 242], [208, 76, 23], [167, 23, 75], [244, 248, 99], [195, 56, 240], [175, 227, 254], [17, 20, 81], [187, 242, 195], [11, 60, 61], [125, 141, 231], [238, 202, 176], [129, 86, 81], [221, 177, 132], [178, 222, 168], [136, 225, 223], [232, 189, 241], [249, 227, 65], [26, 26, 190], [202, 173, 160], [212, 206, 239], [191, 26, 63], [128, 26, 25], [199, 137, 61], [192, 205, 88], [90, 161, 163], [160, 163, 72], [210, 231, 154], [43, 192, 157], [250, 41, 254], [16, 156, 108], [92, 168, 18], [64, 197, 66], [5, 6, 77], [229, 200, 215], [18, 178, 205], [28, 157, 25], [9, 8, 107], [246, 31, 210], [229, 185, 120], [89, 93, 33], [144, 21, 98], [218, 176, 17], [74, 62, 210], [74, 103, 74], [117, 179, 145], [211, 113, 182], [195, 203, 133], [221, 240, 247], [253, 120, 170], [15, 106, 133], [130, 202, 238], [84, 87, 85], [64, 2, 8], [46, 218, 80], [230, 35, 217], [37, 98, 117], [61, 117, 202], [104, 156, 196], [20, 248, 18], [112, 222, 9], [0, 49, 216], [66, 82, 39], [251, 229, 39], [79, 145, 89], [98, 217, 193], [152, 44, 132], [187, 61, 2], [246, 107, 75], [229, 94, 47], [104, 228, 155], [168, 193, 94], [136, 222, 214], [0, 42, 253], [199, 67, 218], [252, 34, 101], [153, 182, 251], [116, 206, 172], [19, 51, 154], [47, 205, 134], [100, 56, 252], [173, 77, 27], [92, 198, 223], [73, 241, 197], [137, 190, 128], [1, 73, 140], [73, 55, 97], [188, 102, 180], [24, 99, 83], [138, 189, 92], [144, 242, 106], [248, 66, 98], [30, 21, 246], [109, 163, 1], [49, 107, 94], [231, 247, 228], [22, 59, 14], [21, 150, 164], [52, 216, 45], [72, 186, 86], [195, 144, 107], [231, 82, 35], [47, 45, 88], [95, 7, 182], [23, 243, 182], [214, 35, 51], [230, 247, 154], [243, 31, 242], [153, 210, 148], [9, 233, 56], [157, 211, 42], [119, 136, 208], [24, 236, 10], [61, 111, 137], [75, 124, 32], [65, 63, 124], [236, 14, 165], [236, 90, 57], [113, 235, 174], [204, 178, 126], [107, 103, 60], [243, 230, 161], [58, 198, 35], [81, 23, 140], [170, 159, 17], [255, 237, 176], [162, 102, 101], [19, 86, 111], [48, 155, 243], [199, 234, 203], [229, 199, 113], [119, 72, 145], [225, 223, 238], [240, 169, 86], [34, 243, 212], [108, 152, 177], [183, 130, 97], [183, 11, 212], [98, 68, 247], [254, 179, 160], [74, 18, 54], [72, 48, 218], [101, 124, 145], [108, 134, 92], [226, 31, 219], [83, 81, 130], [239, 77, 145], [203, 82, 196], [86, 91, 84], [79, 57, 219], [80, 112, 218], [37, 180, 87], [155, 67, 245], [128, 15, 175], [198, 1, 35], [241, 180, 0], [162, 9, 58], [85, 76, 255], [55, 32, 168], [42, 103, 162], [30, 168, 202], [251, 56, 165], [106, 88, 169], [50, 180, 152], [93, 54, 59], [20, 106, 216], [24, 117, 13], [141, 247, 122], [137, 235, 36], [3, 233, 108], [91, 87, 43], [137, 130, 105], [93, 228, 108], [247, 42, 136], [77, 225, 163], [8, 192, 38], [134, 217, 171], [99, 29, 80], [225, 175, 138], [165, 203, 207], [12, 60, 104], [147, 99, 221], [232, 239, 172], [191, 166, 125], [122, 36, 244], [138, 68, 188], [58, 101, 214], [131, 144, 33], [84, 90, 224], [135, 151, 170], [241, 156, 90], [72, 176, 237], [17, 94, 221], [82, 169, 114], [191, 171, 217], [150, 21, 128], [40, 2, 33], [223, 229, 90], [36, 6, 55], [35, 7, 136], [114, 99, 72], [211, 254, 205], [135, 249, 250], [15, 184, 37], [252, 227, 247], [178, 210, 226], [212, 196, 247], [224, 108, 166], [36, 154, 104], [101, 188, 184], [255, 124, 20], [110, 82, 91], [192, 221, 151], [54, 224, 142], [133, 240, 139], [242, 39, 90], [201, 208, 234], [25, 110, 59], [55, 3, 162], [112, 67, 115], [159, 157, 249], [91, 89, 199], [190, 8, 78], [94, 139, 12], [108, 160, 172], [237, 184, 234], [37, 116, 65], [230, 126, 189], [146, 114, 34], [158, 236, 92], [13, 220, 34], [48, 231, 172], [205, 239, 89], [100, 160, 55], [95, 49, 249], [144, 37, 43], [149, 101, 6], [194, 54, 9], [134, 111, 92], [113, 26, 152], [22, 98, 15], [246, 251, 250], [176, 195, 254], [193, 48, 227], [5, 212, 227], [203, 212, 212], [234, 235, 171], [182, 10, 242], [156, 83, 127], [92, 50, 42], [205, 27, 41], [170, 240, 167], [58, 251, 52], [62, 138, 93], [71, 129, 137], [249, 82, 62], [167, 151, 19], [73, 252, 157], [197, 35, 233], [67, 113, 7], [76, 121, 141], [160, 13, 44], [228, 104, 226], [155, 213, 232], [143, 48, 167], [145, 231, 70], [130, 4, 208], [43, 108, 53], [171, 143, 189], [248, 93, 178], [40, 48, 155], [59, 180, 37], [173, 91, 53], [22, 31, 174], [242, 46, 86], [194, 181, 126], [18, 247, 68], [133, 133, 57], [247, 131, 107], [230, 160, 42], [65, 58, 236], [47, 149, 194], [93, 131, 195], [153, 178, 84], [125, 63, 227], [253, 151, 29], [66, 140, 191], [201, 225, 185], [131, 226, 105], [168, 138, 183], [110, 186, 198], [99, 154, 189], [98, 107, 155], [199, 231, 220], [140, 189, 147], [141, 0, 101], [146, 253, 173], [181, 188, 186], [6, 120, 188], [130, 20, 44], [103, 165, 236], [109, 144, 132], [28, 31, 229], [77, 138, 74], [221, 140, 152], [105, 251, 188], [164, 102, 219], [72, 156, 246], [128, 250, 9], [220, 34, 94], [227, 182, 63], [210, 21, 155], [226, 106, 124], [203, 196, 166], [198, 119, 10], [102, 240, 35], [172, 52, 114], [196, 6, 75], [251, 38, 111], [248, 192, 211], [198, 44, 216], [117, 245, 109], [49, 163, 61], [129, 48, 80], [67, 42, 90], [134, 135, 183], [174, 34, 195], [18, 182, 104], [133, 56, 67], [63, 250, 63], [58, 213, 125], [149, 63, 196], [96, 196, 20], [229, 158, 14], [44, 64, 72], [207, 251, 32], [81, 105, 171], [223, 149, 198], [141, 72, 195], [215, 135, 189], [82, 84, 51], [217, 139, 166], [89, 222, 171], [186, 25, 105], [110, 142, 114], [110, 248, 136], [122, 35, 60], [208, 90, 251], [48, 7, 0], [189, 80, 194], [111, 85, 233], [162, 230, 9], [123, 194, 13], [46, 84, 151], [64, 5, 245], [185, 197, 206], [211, 193, 236], [239, 9, 222], [86, 75, 236], [66, 56, 69], [10, 254, 29], [102, 157, 138], [109, 152, 8], [124, 121, 95], [23, 160, 205], [8, 38, 183], [200, 216, 150], [231, 7, 58], [123, 37, 86], [253, 74, 33], [73, 9, 175], [197, 160, 217], [174, 30, 7], [154, 205, 230], [68, 148, 46], [93, 108, 144], [118, 188, 195], [15, 223, 8], [179, 133, 71], [73, 187, 117], [9, 178, 157], [224, 39, 100], [17, 50, 12], [251, 36, 38], [38, 175, 58], [10, 221, 179], [74, 64, 23], [103, 140, 101], [40, 121, 155], [160, 235, 90], [210, 56, 171], [167, 48, 70], [49, 220, 135], [0, 54, 97], [34, 142, 144], [46, 231, 42], [133, 92, 194], [140, 19, 246], [8, 97, 84], [32, 135, 253], [118, 106, 27], [120, 126, 239], [176, 76, 151], [68, 53, 173], [66, 62, 185], [189, 191, 54], [246, 205, 49], [186, 7, 242], [184, 187, 115], [99, 150, 27], [39, 235, 239], [51, 217, 192], [167, 61, 128], [233, 44, 208], [32, 80, 17], [199, 173, 75], [213, 19, 136], [79, 222, 59], [34, 198, 55], [213, 239, 160], [203, 117, 207], [139, 95, 211], [74, 134, 133], [87, 14, 195], [104, 87, 119], [57, 196, 155], [102, 3, 79], [30, 54, 70], [149, 58, 72], [241, 176, 231], [18, 55, 255], [135, 103, 218], [60, 239, 113], [12, 28, 132], [231, 170, 247], [205, 199, 12], [163, 220, 234], [83, 42, 25], [2, 148, 184], [130, 47, 111], [204, 245, 10], [25, 96, 50], [219, 129, 252], [80, 158, 222], [36, 11, 180], [56, 188, 54], [13, 81, 237], [202, 59, 118], [21, 78, 94], [164, 20, 85], [64, 148, 144], [74, 127, 232], [102, 164, 131], [175, 227, 173], [36, 143, 8], [111, 168, 137], [209, 239, 220], [203, 161, 171], [152, 119, 218], [252, 129, 214], [95, 108, 177], [39, 131, 82], [234, 106, 181], [27, 124, 254], [165, 216, 141], [173, 204, 140], [187, 210, 6], [178, 91, 16], [78, 172, 151], [21, 111, 73], [217, 247, 93], [245, 122, 21], [232, 116, 243], [239, 26, 132], [135, 65, 166], [239, 32, 45], [145, 83, 205], [39, 71, 79], [96, 186, 169], [254, 39, 114], [147, 247, 228], [193, 84, 1], [45, 219, 23], [67, 45, 160], [247, 19, 130], [85, 52, 156], [133, 132, 99], [3, 249, 32], [35, 37, 156], [231, 160, 28], [165, 92, 161], [164, 237, 19], [83, 252, 205], [132, 50, 150], [140, 71, 237], [99, 7, 240], [179, 202, 209], [53, 90, 138], [18, 245, 108], [147, 32, 22], [42, 210, 182], [246, 83, 208], [19, 46, 33], [87, 95, 135], [142, 89, 100], [67, 183, 30], [225, 29, 35], [31, 3, 28], [88, 181, 47], [118, 10, 1], [215, 111, 103], [194, 242, 151], [21, 175, 104], [140, 83, 196], [207, 236, 200], [126, 18, 58], [57, 71, 12], [172, 157, 76], [150, 25, 32], [253, 59, 218], [80, 68, 62], [184, 209, 51], [107, 244, 10], [150, 143, 161], [97, 0, 168], [216, 175, 7], [182, 78, 178], [49, 86, 179], [100, 138, 245], [110, 178, 129], [38, 92, 115], [243, 8, 175], [10, 164, 14], [176, 170, 226], [205, 28, 184], [51, 169, 187], [130, 53, 224], [196, 61, 212], [159, 146, 46], [31, 203, 99], [230, 166, 34], [188, 185, 174], [131, 28, 73], [164, 134, 9], [240, 111, 199], [60, 176, 200], [0, 163, 47], [151, 137, 81], [9, 220, 136], [68, 37, 160], [103, 152, 105], [114, 39, 236], [163, 149, 109], [77, 48, 177], [218, 218, 120], [175, 83, 245], [253, 141, 198], [205, 101, 138], [204, 39, 235], [252, 92, 10], [148, 29, 225], [93, 117, 197], [0, 217, 13], [192, 15, 253], [198, 154, 245], [20, 22, 10], [46, 57, 129], [1, 50, 153], [244, 120, 235], [34, 125, 92], [70, 99, 93], [18, 145, 216], [140, 153, 194], [83, 245, 93], [216, 217, 24], [19, 252, 12], [142, 180, 215], [61, 18, 129], [250, 16, 24], [51, 239, 76], [213, 28, 243], [88, 34, 207], [131, 3, 100], [1, 94, 173], [254, 114, 240], [88, 126, 120], [201, 3, 74], [226, 77, 200], [80, 210, 185], [120, 147, 162], [59, 169, 151], [134, 190, 177], [14, 212, 95], [117, 59, 37], [165, 119, 80], [17, 103, 155], [170, 87, 15], [151, 249, 34], [95, 100, 138], [6, 80, 225], [34, 159, 169], [59, 131, 162], [98, 201, 66], [166, 92, 35], [200, 160, 96], [167, 205, 201], [33, 116, 179], [197, 149, 143], [210, 2, 203], [243, 243, 140], [9, 217, 140], [20, 39, 137], [152, 81, 30], [76, 175, 105], [133, 169, 69], [175, 0, 152], [70, 187, 103], [32, 238, 20], [26, 131, 142], [98, 81, 26], [214, 168, 234], [230, 30, 95], [241, 128, 173], [228, 33, 219], [244, 123, 48], [130, 231, 119], [69, 174, 45], [137, 70, 9], [25, 113, 120], [148, 80, 212], [182, 110, 151], [48, 91, 177], [245, 222, 26], [14, 246, 14], [143, 198, 60], [100, 144, 146], [118, 214, 162], [238, 82, 97], [217, 240, 94], [132, 31, 18], [172, 184, 175], [39, 227, 95], [203, 148, 36], [3, 146, 148], [187, 12, 120], [107, 228, 172], [122, 119, 18], [170, 22, 141], [133, 71, 134], [85, 6, 185], [2, 129, 2], [40, 172, 195], [156, 99, 223], [227, 201, 242], [116, 207, 62], [172, 242, 42], [201, 159, 143], [98, 59, 44], [132, 138, 30], [2, 144, 190], [94, 253, 212], [218, 98, 177], [247, 120, 24], [205, 82, 7], [172, 147, 134], [193, 154, 94], [234, 148, 235], [55, 70, 94], [12, 141, 70], [145, 185, 103], [193, 91, 112], [30, 217, 30], [60, 20, 149], [105, 216, 3], [148, 17, 53], [79, 7, 42], [91, 237, 247], [165, 76, 20], [99, 185, 147], [161, 239, 252], [186, 32, 209], [35, 76, 115], [122, 166, 173], [172, 28, 146], [18, 219, 126], [218, 124, 26], [108, 104, 35], [248, 129, 74], [152, 195, 68], [190, 203, 155], [211, 77, 34], [62, 122, 219], [15, 71, 227], [84, 86, 224], [133, 52, 38], [61, 65, 223], [219, 83, 7], [211, 21, 116], [52, 59, 122], [157, 91, 58], [71, 197, 5], [146, 193, 102], [3, 93, 240], [239, 230, 10], [240, 158, 4], [67, 144, 166], [74, 57, 136], [210, 157, 186], [57, 21, 92], [36, 130, 107], [202, 228, 137], [144, 124, 114], [151, 132, 198], [202, 31, 185], [110, 254, 50], [104, 5, 106], [177, 249, 69], [181, 205, 181], [87, 1, 102], [146, 20, 112], [8, 79, 115], [64, 112, 31], [0, 216, 80], [25, 43, 210], [78, 147, 198], [207, 198, 210], [14, 39, 204], [226, 114, 7], [237, 87, 123], [125, 188, 40], [254, 30, 247], [13, 61, 7], [102, 249, 135], [193, 43, 198], [167, 119, 53], [213, 172, 78], [201, 116, 248], [253, 158, 248], [133, 208, 244], [221, 72, 171], [38, 165, 31], [50, 122, 242], [1, 179, 180], [58, 159, 126], [203, 207, 75], [150, 66, 204], [239, 239, 130], [129, 185, 26], [103, 254, 172], [165, 21, 61], [51, 98, 172], [176, 58, 126], [232, 239, 58], [170, 160, 151], [149, 157, 246], [225, 223, 154], [65, 185, 106], [93, 252, 41], [65, 45, 220], [67, 74, 188], [41, 166, 7], [23, 49, 198], [92, 37, 71], [176, 155, 17], [201, 63, 213], [255, 114, 161], [124, 98, 248], [192, 193, 211], [53, 115, 82], [31, 243, 247], [206, 217, 38], [193, 182, 9], [110, 137, 110], [141, 0, 208], [175, 38, 46], [98, 194, 144], [153, 240, 89], [44, 194, 58], [101, 19, 8], [227, 136, 139], [216, 128, 98], [37, 7, 151], [115, 242, 185], [168, 163, 14], [48, 98, 71], [123, 108, 95], [19, 164, 190], [168, 183, 225], [221, 76, 133], [40, 90, 230], [80, 1, 194], [150, 241, 107], [197, 237, 217], [12, 139, 135], [226, 5, 224], [236, 217, 197], [111, 107, 19], [43, 215, 2], [76, 2, 105], [247, 241, 235], [103, 12, 187], [202, 183, 182], [231, 164, 107], [173, 70, 40], [185, 163, 110], [252, 62, 129], [19, 205, 159], [122, 154, 90], [62, 240, 54], [189, 29, 7], [168, 67, 67], [79, 62, 84], [99, 215, 64], [110, 57, 190], [143, 22, 243], [82, 106, 207], [168, 115, 177], [221, 149, 71], [178, 113, 184], [97, 175, 41], [224, 200, 49], [226, 48, 126], [242, 237, 82], [100, 237, 84], [140, 109, 243], [223, 177, 53], [168, 173, 43], [246, 226, 117], [238, 204, 238], [38, 187, 100], [104, 20, 76], [35, 6, 87], [83, 185, 153], [8, 74, 16], [78, 52, 118], [202, 137, 186], [226, 205, 226], [8, 205, 147], [15, 225, 91], [139, 42, 50], [5, 176, 121], [69, 97, 112], [50, 170, 174], [19, 206, 25], [150, 8, 91], [252, 109, 212], [183, 188, 236], [223, 175, 182], [104, 42, 162], [122, 2, 229], [60, 108, 244], [89, 65, 196], [206, 90, 202], [11, 187, 177], [222, 221, 1], [49, 61, 235], [236, 98, 193], [247, 184, 73], [90, 204, 212], [48, 201, 220], [192, 198, 173], [168, 219, 149], [130, 214, 160], [56, 166, 120], [138, 74, 9], [55, 217, 17], [48, 221, 134], [177, 100, 104], [35, 250, 196], [72, 242, 145], [74, 52, 116], [89, 80, 2], [254, 181, 255], [94, 132, 212], [91, 190, 230], [251, 45, 16], [222, 225, 76], [106, 152, 146], [77, 250, 117], [124, 31, 195], [188, 78, 228], [81, 150, 13], [86, 25, 145], [236, 40, 34], [89, 147, 174], [116, 239, 123], [2, 211, 203], [74, 47, 163], [49, 0, 78], [15, 33, 201], [86, 63, 12], [114, 243, 62], [249, 29, 84], [49, 171, 82], [209, 213, 178], [127, 216, 97], [29, 208, 250], [33, 176, 144], [117, 41, 151], [121, 19, 111], [233, 184, 74], [152, 245, 214], [102, 44, 237], [93, 194, 83], [148, 174, 4], [99, 250, 66], [53, 222, 68], [239, 164, 92], [168, 52, 60], [192, 68, 141], [22, 149, 206], [40, 210, 83], [60, 161, 90], [234, 195, 52], [40, 197, 155], [90, 201, 62], [212, 133, 170], [225, 75, 58], [180, 103, 31], [51, 218, 216], [247, 207, 172], [118, 189, 106], [254, 56, 234], [192, 57, 228], [41, 66, 235], [221, 144, 190], [193, 175, 223], [192, 82, 68], [151, 50, 246], [59, 181, 132], [186, 73, 246], [38, 12, 33], [50, 157, 181], [168, 21, 207], [116, 138, 194], [175, 66, 162], [164, 51, 46], [94, 30, 67], [80, 217, 99], [141, 152, 15], [166, 178, 193], [121, 135, 255], [42, 126, 172], [133, 141, 158], [229, 78, 9], [104, 47, 108], [80, 216, 64], [11, 31, 42], [103, 9, 100], [55, 103, 95], [40, 197, 94], [51, 117, 191], [209, 40, 76], [67, 61, 171], [77, 189, 238], [212, 123, 240], [8, 220, 199], [25, 128, 238], [22, 67, 76], [81, 197, 12], [147, 234, 228], [180, 95, 74], [188, 25, 167], [74, 33, 45], [236, 56, 135], [23, 236, 219], [60, 130, 183], [172, 119, 55], [34, 220, 27], [89, 107, 106], [209, 137, 54], [156, 12, 179], [225, 58, 12], [165, 99, 81], [172, 23, 26], [177, 201, 74], [162, 79, 112], [150, 26, 134], [231, 31, 230], [23, 16, 133], [10, 164, 244], [18, 191, 212], [145, 220, 232], [40, 14, 76], [200, 121, 246], [18, 247, 29], [180, 131, 67], [101, 76, 1], [187, 167, 13], [122, 158, 192], [173, 198, 155], [42, 222, 103], [132, 41, 253], [232, 26, 154], [19, 171, 69], [248, 149, 93], [128, 170, 106], [63, 163, 226], [16, 159, 249], [123, 118, 162], [41, 248, 114], [174, 213, 202], [207, 129, 123], [47, 21, 141], [244, 196, 114], [97, 142, 145], [133, 26, 122], [170, 0, 208], [99, 219, 7], [238, 218, 154], [176, 176, 135], [69, 156, 5], [5, 218, 44], [228, 61, 22], [38, 126, 130], [193, 170, 111], [126, 143, 68], [217, 164, 14], [49, 185, 141], [63, 27, 197], [112, 34, 206], [187, 113, 189], [169, 62, 138], [13, 201, 9], [117, 114, 89], [255, 99, 150], [150, 245, 105], [52, 213, 214], [6, 180, 119], [94, 35, 115], [209, 98, 77], [134, 221, 248], [111, 219, 99], [126, 20, 114], [165, 82, 148], [102, 105, 104], [216, 164, 35], [131, 227, 184], [253, 111, 199], [31, 42, 228], [165, 41, 145], [4, 25, 234], [67, 130, 216], [113, 82, 160], [178, 132, 139], [138, 191, 114], [135, 191, 175], [47, 102, 145], [204, 84, 132], [142, 71, 163], [164, 227, 60], [22, 117, 143], [33, 50, 70], [74, 156, 73], [109, 233, 50], [153, 244, 240], [189, 177, 115], [182, 31, 223], [15, 67, 54], [247, 134, 127], [22, 201, 47], [111, 196, 99], [224, 255, 67], [159, 39, 164], [1, 166, 116], [189, 165, 107], [64, 148, 59], [180, 185, 122], [229, 157, 135], [3, 90, 51], [142, 245, 130], [234, 93, 221], [231, 233, 36], [75, 166, 172], [161, 18, 245], [251, 65, 151], [171, 104, 115], [129, 162, 184], [200, 199, 61], [142, 152, 53], [248, 33, 235], [179, 8, 231], [179, 124, 133], [67, 88, 208], [1, 73, 224], [106, 90, 170], [150, 210, 68], [110, 225, 86], [228, 245, 83], [141, 184, 244], [182, 22, 11], [115, 99, 240], [66, 196, 68], [200, 122, 110], [22, 97, 138], [56, 32, 175], [54, 238, 233], [6, 43, 154], [250, 65, 224], [152, 88, 67], [139, 244, 225], [238, 99, 80], [133, 215, 119], [51, 219, 134], [127, 253, 5], [109, 251, 140], [111, 21, 19], [166, 227, 41], [198, 96, 88], [79, 223, 98], [63, 3, 187], [237, 130, 93], [225, 132, 243], [30, 188, 216], [23, 49, 131], [140, 56, 74], [224, 75, 107], [111, 207, 143], [66, 199, 53], [21, 142, 229], [13, 179, 138], [85, 39, 238], [230, 15, 34], [248, 8, 135], [53, 42, 18], [109, 35, 145], [180, 189, 174], [82, 233, 84], [8, 252, 3], [137, 134, 231], [231, 181, 52], [192, 83, 133], [30, 44, 97], [203, 35, 76], [83, 250, 49], [93, 128, 195], [214, 115, 25], [169, 4, 113], [218, 221, 158], [137, 55, 231], [237, 13, 238], [218, 5, 172], [168, 93, 2], [127, 130, 212], [68, 101, 70], [149, 165, 87], [93, 22, 2], [187, 146, 12], [36, 250, 11], [30, 126, 239], [142, 68, 226], [75, 241, 136], [176, 250, 197], [181, 255, 231], [192, 108, 186], [11, 189, 252], [44, 10, 136], [44, 57, 228], [13, 43, 183], [159, 1, 82], [138, 217, 14], [20, 204, 163], [105, 161, 153], [152, 224, 174], [79, 142, 43], [3, 73, 105], [26, 16, 53], [117, 126, 253], [216, 226, 108], [238, 11, 241], [7, 255, 34], [26, 206, 7], [17, 211, 30], [37, 2, 105], [122, 55, 165], [141, 248, 213], [72, 121, 55], [246, 161, 91], [241, 159, 134], [122, 248, 27], [30, 128, 141], [161, 86, 144], [185, 231, 242], [188, 71, 208], [181, 65, 16], [24, 135, 168], [112, 80, 134], [245, 110, 68], [142, 22, 243], [111, 6, 45], [182, 18, 90], [254, 190, 51], [94, 214, 172], [57, 72, 16], [40, 76, 32], [239, 120, 86], [66, 201, 8], [205, 86, 143], [223, 193, 222], [8, 9, 137], [191, 151, 17], [37, 215, 2], [224, 143, 159], [244, 148, 67], [39, 97, 141], [161, 164, 26], [239, 195, 184], [147, 92, 180], [237, 199, 212], [136, 251, 14], [44, 200, 253], [190, 83, 240], [38, 208, 101], [120, 23, 247], [107, 88, 229], [131, 192, 70], [179, 91, 1], [111, 51, 66], [160, 59, 159], [28, 112, 143], [28, 29, 142], [60, 163, 114], [81, 216, 8], [25, 148, 220], [8, 48, 113], [213, 166, 204], [39, 170, 144], [103, 211, 21], [38, 118, 135], [49, 126, 124], [32, 43, 200], [157, 16, 185], [150, 124, 37], [142, 105, 151], [68, 120, 129], [149, 61, 162], [140, 109, 134], [88, 214, 211], [124, 159, 200], [85, 5, 98], [6, 80, 6], [77, 121, 103], [233, 63, 181], [214, 227, 140], [210, 242, 133], [239, 86, 114], [54, 63, 190], [102, 72, 189], [55, 120, 133], [41, 17, 91], [62, 165, 234], [123, 199, 46], [139, 156, 70], [204, 41, 67], [46, 175, 141], [45, 219, 242], [95, 164, 6], [48, 99, 164], [149, 197, 54], [162, 233, 25], [159, 61, 200], [8, 203, 27], [30, 153, 97], [209, 228, 128], [230, 220, 86], [70, 10, 128], [244, 67, 169], [75, 209, 218], [209, 137, 207], [223, 75, 17], [154, 157, 254], [111, 83, 188], [84, 135, 1], [243, 11, 84], [6, 100, 4], [64, 118, 55], [131, 137, 240], [172, 205, 205], [22, 72, 243], [161, 28, 35], [224, 124, 149], [184, 100, 62], [247, 94, 21], [93, 174, 201], [229, 132, 203], [134, 148, 176], [29, 90, 31], [24, 110, 255], [70, 106, 15], [10, 140, 197], [78, 89, 224], [80, 251, 115], [190, 63, 59], [207, 135, 95], [252, 68, 64], [194, 73, 123], [192, 38, 16], [128, 107, 164], [87, 6, 6], [247, 181, 147], [164, 181, 183], [205, 88, 29], [161, 40, 124], [207, 255, 24], [70, 24, 114], [7, 39, 162], [199, 192, 36], [199, 32, 151], [201, 250, 115], [89, 49, 59], [63, 4, 188], [189, 64, 47], [148, 133, 162], [209, 246, 17], [16, 42, 224], [169, 227, 33], [198, 38, 17], [135, 40, 103], [148, 100, 210], [106, 72, 56], [64, 20, 209], [61, 170, 147], [84, 112, 194], [218, 90, 92], [30, 105, 150], [29, 159, 23], [219, 233, 254], [139, 41, 12], [42, 191, 70], [195, 33, 200], [53, 144, 25], [138, 80, 44], [35, 18, 81], [104, 61, 236], [39, 99, 238], [20, 83, 34], [24, 112, 180], [192, 4, 125], [105, 212, 1], [164, 83, 29], [141, 48, 207], [201, 106, 40], [158, 70, 109], [17, 103, 109], [60, 218, 137], [24, 158, 212], [29, 95, 226], [32, 234, 187], [24, 116, 23], [201, 93, 108], [95, 168, 130], [4, 18, 75], [39, 206, 19], [24, 38, 137], [164, 142, 15], [206, 119, 0], [121, 16, 192], [222, 139, 120], [127, 188, 137], [43, 71, 229], [147, 173, 74], [253, 56, 183], [226, 65, 229], [138, 152, 171], [94, 41, 169], [9, 237, 121], [111, 19, 97], [82, 217, 78], [163, 149, 240], [139, 126, 127], [99, 242, 10], [114, 49, 144], [128, 117, 56], [14, 102, 91], [189, 64, 136], [93, 34, 24], [48, 207, 39], [14, 12, 138], [243, 178, 168], [16, 177, 108], [96, 4, 115], [150, 79, 201], [79, 169, 67], [61, 41, 194], [130, 97, 184], [176, 219, 47], [60, 228, 110], [116, 89, 158], [36, 135, 254], [87, 199, 193], [252, 15, 180], [216, 108, 120], [229, 50, 136], [30, 75, 100], [119, 9, 244], [85, 76, 117], [47, 132, 113], [194, 110, 149], [57, 81, 1], [35, 117, 63], [59, 173, 49], [166, 56, 144], [141, 187, 161], [247, 67, 111], [243, 53, 152], [4, 183, 69], [85, 210, 61], [126, 47, 65], [136, 231, 83], [136, 176, 220], [0, 89, 75], [26, 43, 124], [135, 5, 251], [189, 194, 81], [144, 254, 29], [220, 244, 199], [254, 68, 165], [4, 227, 61], [194, 237, 96], [82, 90, 247], [43, 252, 50], [29, 247, 107], [93, 165, 2], [183, 136, 247], [182, 20, 245], [232, 178, 83], [240, 44, 56], [11, 41, 16], [118, 147, 70], [93, 52, 14], [166, 38, 54], [173, 209, 22], [21, 131, 100], [79, 177, 70], [250, 114, 21], [207, 40, 120], [55, 85, 255], [6, 23, 109], [230, 77, 111], [246, 21, 69], [108, 253, 123], [89, 138, 247], [105, 0, 233], [138, 83, 38], [76, 183, 62], [150, 163, 235], [182, 102, 17], [133, 214, 31], [186, 209, 157], [107, 43, 192], [40, 53, 172], [239, 11, 107], [128, 235, 117], [159, 86, 148], [72, 245, 20], [186, 252, 58], [218, 68, 60], [6, 210, 163], [82, 104, 104], [59, 5, 224], [145, 18, 2], [225, 13, 121], [55, 141, 217], [238, 26, 195], [101, 47, 70], [200, 49, 63], [153, 89, 70], [147, 28, 50], [54, 159, 107], [124, 167, 111], [169, 160, 244], [1, 74, 38], [220, 236, 62], [180, 60, 193], [195, 246, 43], [209, 62, 63], [140, 103, 210], [132, 155, 178], [33, 88, 5], [45, 62, 230], [51, 161, 85], [64, 152, 204], [138, 45, 188], [80, 113, 160], [148, 151, 248], [207, 236, 176], [118, 161, 238], [212, 21, 209], [41, 101, 2], [145, 214, 54], [89, 237, 130], [168, 135, 66], [156, 216, 174], [109, 213, 122], [240, 92, 9], [210, 59, 241], [139, 72, 92], [15, 23, 8], [68, 116, 116], [38, 130, 177], [70, 34, 24], [154, 242, 2], [213, 116, 68], [164, 131, 97], [255, 90, 172], [164, 73, 169], [115, 177, 140], [54, 132, 47], [255, 203, 236], [74, 174, 205], [91, 250, 216], [227, 223, 90], [31, 60, 208], [92, 38, 85], [110, 36, 5], [163, 195, 27], [23, 117, 27], [71, 151, 109], [137, 62, 55], [230, 25, 220], [128, 237, 0], [110, 197, 132], [232, 182, 30], [2, 159, 206], [30, 44, 183], [30, 88, 57], [176, 32, 113], [133, 89, 160], [201, 15, 250], [119, 237, 161], [170, 235, 51], [44, 162, 134], [43, 113, 108], [180, 74, 118], [143, 202, 152], [209, 123, 49], [78, 109, 44], [198, 218, 174], [189, 3, 248], [6, 106, 49], [64, 64, 254], [13, 123, 149], [130, 222, 105], [46, 3, 161], [131, 79, 5], [136, 7, 172], [143, 236, 67], [91, 163, 208], [224, 138, 24], [77, 65, 208], [230, 174, 9], [230, 237, 12], [95, 192, 155], [45, 150, 130], [201, 38, 82], [173, 125, 151], [144, 241, 169], [165, 0, 166], [221, 126, 166], [94, 74, 17], [120, 95, 132], [153, 159, 73], [230, 95, 47], [5, 110, 185], [209, 112, 70], [137, 171, 51], [192, 91, 86], [99, 138, 75], [254, 217, 143], [87, 179, 40], [65, 51, 107], [187, 124, 203], [103, 141, 65], [24, 174, 99], [106, 52, 93], [14, 74, 169], [184, 225, 234], [47, 185, 27], [156, 3, 243], [191, 5, 31], [223, 37, 188], [6, 111, 129], [187, 74, 238], [13, 230, 2], [15, 212, 93], [18, 17, 70], [99, 132, 117], [241, 202, 85], [161, 157, 66], [84, 196, 18], [121, 21, 156], [23, 122, 55], [14, 15, 234], [195, 142, 233], [246, 218, 15], [101, 241, 29], [172, 114, 104], [22, 32, 45], [188, 62, 31], [146, 106, 79], [54, 10, 202], [91, 149, 161], [150, 74, 174], [124, 152, 220], [127, 22, 87], [170, 164, 181], [8, 108, 82], [156, 239, 119], [124, 181, 212], [138, 73, 70], [42, 20, 116], [174, 193, 79], [209, 227, 197], [16, 212, 94], [39, 192, 40], [157, 169, 7], [128, 49, 10], [53, 14, 66], [203, 110, 100], [214, 143, 105], [37, 26, 133], [67, 116, 76], [51, 0, 189], [222, 209, 40], [20, 69, 177], [60, 133, 66], [178, 150, 218], [87, 125, 154], [128, 81, 65], [201, 244, 178], [35, 78, 126], [69, 139, 193], [187, 185, 10], [124, 175, 247], [59, 42, 9], [199, 229, 233], [113, 55, 237], [44, 9, 186], [127, 0, 251], [81, 121, 91], [225, 213, 29], [244, 90, 194], [85, 177, 195], [29, 215, 79], [55, 154, 90], [135, 254, 156], [16, 204, 79], [192, 251, 189], [230, 205, 65], [132, 167, 18], [255, 13, 243], [160, 229, 138], [146, 203, 211], [3, 120, 114], [195, 243, 117], [44, 184, 246], [29, 45, 98], [77, 97, 90], [78, 135, 124], [42, 232, 226], [2, 223, 68], [213, 235, 6], [39, 45, 103], [241, 182, 120], [123, 254, 116], [27, 235, 168], [5, 49, 39], [214, 130, 211], [130, 158, 95], [189, 225, 151], [67, 99, 61], [122, 122, 105], [194, 243, 103], [16, 231, 169], [210, 254, 223], [3, 133, 143], [26, 117, 45], [132, 32, 243], [224, 125, 91], [209, 217, 58], [3, 37, 27], [110, 148, 25], [40, 37, 223], [101, 234, 120], [251, 137, 12], [195, 10, 255], [23, 85, 117], [100, 238, 140], [108, 139, 136], [18, 3, 161], [165, 138, 57], [61, 218, 122], [216, 85, 161], [232, 218, 97], [89, 162, 6], [82, 145, 232], [179, 121, 115], [91, 148, 160], [53, 4, 195], [91, 190, 246], [5, 164, 23], [157, 226, 245], [162, 224, 234], [130, 174, 88], [69, 33, 234], [117, 155, 4], [68, 211, 82], [228, 83, 63], [28, 240, 93], [84, 158, 151], [119, 227, 10], [34, 121, 207], [159, 39, 182], [74, 231, 239], [3, 9, 21], [226, 71, 222], [61, 131, 139], [89, 129, 161], [169, 110, 100], [194, 105, 9], [250, 81, 248], [41, 132, 219], [192, 238, 229], [76, 112, 107], [251, 149, 47], [28, 15, 173], [69, 77, 234], [163, 145, 47], [158, 124, 55], [112, 216, 11], [187, 48, 216], [102, 2, 143], [166, 88, 16], [34, 201, 84], [76, 243, 20], [46, 190, 173], [21, 177, 108], [15, 212, 31], [27, 53, 41], [17, 242, 122], [206, 182, 120], [216, 19, 79], [255, 15, 142], [238, 190, 34], [83, 109, 253], [28, 82, 177], [0, 238, 114], [59, 14, 70], [236, 79, 239], [63, 92, 73], [86, 38, 81], [80, 171, 113], [22, 29, 80], [31, 62, 149], [12, 9, 93], [129, 236, 75], [202, 190, 106], [110, 41, 250], [112, 2, 3], [57, 104, 183], [115, 150, 217], [43, 16, 83], [179, 161, 56], [242, 176, 24], [1, 228, 225], [200, 49, 48], [135, 66, 229], [18, 249, 233], [63, 27, 214], [15, 50, 161], [242, 95, 141], [67, 24, 80], [112, 195, 4], [61, 192, 36], [39, 31, 120], [45, 155, 120], [27, 15, 186], [131, 68, 38], [184, 196, 208], [65, 241, 116], [178, 247, 122], [213, 75, 32], [211, 152, 163], [116, 148, 34], [27, 117, 219], [179, 111, 120], [0, 218, 224], [196, 201, 126], [117, 238, 137], [35, 115, 181], [224, 153, 90], [190, 129, 73], [88, 65, 209], [162, 207, 51], [81, 4, 74], [80, 205, 41], [102, 243, 125], [240, 179, 127], [122, 178, 58], [120, 231, 161], [12, 245, 201], [22, 160, 246], [242, 216, 249], [46, 204, 35], [192, 160, 139], [122, 120, 141], [42, 35, 97], [212, 246, 232], [217, 209, 218], [84, 61, 158], [32, 245, 85], [207, 131, 109], [152, 229, 226], [221, 177, 201], [28, 7, 133], [14, 59, 159], [56, 233, 77], [92, 103, 227], [216, 232, 66], [113, 0, 122], [92, 200, 234], [84, 52, 223], [170, 62, 9], [105, 99, 99], [230, 252, 10], [123, 160, 30], [244, 57, 110], [177, 27, 76], [17, 195, 165], [171, 196, 236], [237, 194, 181], [88, 157, 71], [59, 15, 214], [204, 119, 21], [231, 133, 58], [35, 153, 95], [213, 151, 9], [159, 101, 26], [19, 8, 219], [72, 253, 252], [148, 197, 82], [195, 220, 217], [5, 69, 65], [6, 144, 168], [45, 55, 252], [86, 211, 100], [144, 172, 121], [58, 143, 137], [93, 158, 57], [222, 117, 98], [9, 252, 170], [55, 196, 234], [31, 30, 21], [115, 154, 152], [12, 76, 235], [123, 33, 241], [97, 167, 164], [207, 230, 128], [207, 48, 13], [198, 52, 239], [109, 192, 75], [232, 33, 44], [90, 203, 43], [89, 128, 208], [78, 114, 124], [243, 123, 10], [125, 124, 7], [8, 94, 220], [180, 140, 135], [232, 69, 174], [133, 19, 76], [11, 228, 64], [186, 142, 236], [250, 235, 109], [228, 223, 149], [227, 129, 93], [21, 83, 155], [121, 53, 104], [215, 104, 69], [250, 216, 195], [197, 163, 250], [79, 178, 30], [117, 176, 74], [243, 247, 143], [34, 132, 78], [75, 128, 83], [162, 170, 225], [126, 211, 105], [98, 127, 130], [142, 238, 144], [127, 230, 81], [130, 37, 48], [253, 46, 40], [232, 28, 200], [132, 121, 156], [250, 24, 79], [118, 239, 127], [183, 86, 220], [131, 249, 169], [115, 252, 224], [207, 76, 12], [223, 154, 52], [10, 144, 51], [217, 95, 95], [223, 164, 190], [82, 219, 204], [77, 172, 80], [86, 3, 92], [163, 140, 0], [138, 125, 214], [209, 225, 208], [189, 253, 224], [221, 29, 111], [25, 86, 226], [48, 147, 206], [226, 54, 213], [44, 214, 54], [244, 196, 129], [43, 254, 119], [132, 114, 166], [203, 233, 232], [207, 237, 109], [107, 162, 84], [2, 47, 68], [157, 105, 158], [115, 21, 250], [87, 1, 33], [6, 59, 77], [188, 183, 221], [204, 24, 95], [213, 126, 252], [254, 5, 31], [118, 23, 66], [23, 82, 108], [233, 65, 121], [104, 195, 136], [16, 164, 167], [184, 23, 17], [23, 245, 61], [142, 220, 80], [27, 209, 0], [223, 98, 224], [22, 230, 95], [215, 26, 14], [11, 254, 113], [123, 14, 250], [66, 38, 67], [144, 107, 233], [23, 203, 47], [154, 233, 35], [132, 249, 116], [55, 70, 214], [90, 199, 53], [207, 59, 16], [241, 235, 94], [116, 202, 107], [7, 214, 84], [90, 165, 162], [229, 19, 89], [109, 96, 110], [3, 207, 141], [107, 254, 103], [127, 208, 111], [127, 180, 81], [140, 195, 184], [0, 190, 103], [104, 191, 235], [164, 205, 220], [51, 234, 234], [51, 164, 4], [178, 110, 89], [128, 227, 138], [169, 156, 97], [176, 164, 176], [57, 6, 150], [51, 98, 239], [148, 213, 62], [198, 132, 158], [143, 103, 106], [99, 73, 144], [113, 209, 47], [242, 22, 193], [249, 65, 36], [211, 151, 135], [176, 74, 147], [13, 109, 136], [195, 0, 180], [119, 243, 253], [247, 93, 139], [67, 86, 189], [83, 16, 240], [90, 122, 16], [3, 209, 171], [138, 59, 52], [185, 198, 223], [45, 21, 6], [32, 124, 163], [225, 44, 105], [221, 47, 200], [159, 23, 201], [249, 8, 64], [146, 122, 14], [150, 132, 251], [124, 60, 91], [9, 59, 76], [22, 33, 175], [124, 170, 55], [201, 54, 158], [229, 247, 210], [39, 195, 101], [31, 50, 114], [41, 31, 111], [23, 147, 146], [248, 186, 82], [138, 89, 60], [104, 188, 188], [171, 71, 187], [59, 111, 206], [167, 81, 20], [240, 46, 65], [193, 254, 4], [134, 85, 60], [59, 57, 73], [185, 48, 101], [29, 246, 76], [15, 132, 250], [182, 21, 242], [75, 27, 83], [20, 187, 73], [110, 170, 152], [83, 68, 255], [7, 91, 8], [111, 254, 38], [241, 31, 230], [210, 154, 59], [15, 94, 254], [234, 188, 7], [139, 218, 64], [231, 90, 216], [37, 226, 161], [92, 252, 131], [131, 148, 143], [184, 18, 226], [38, 168, 161], [63, 194, 124], [140, 213, 166], [191, 140, 118], [88, 116, 8], [255, 127, 147], [241, 203, 125], [87, 211, 209], [214, 42, 5], [9, 129, 77], [66, 42, 40], [101, 225, 43], [105, 21, 255], [249, 26, 184], [182, 86, 36], [208, 67, 10], [253, 242, 140], [112, 174, 160], [16, 116, 185], [241, 124, 6], [206, 59, 55], [87, 254, 92], [174, 61, 98], [43, 188, 159], [134, 102, 11], [56, 147, 139], [207, 93, 200], [173, 120, 194], [238, 118, 65], [224, 60, 53], [210, 247, 211], [123, 220, 193], [218, 196, 162], [31, 173, 88], [187, 187, 39], [53, 208, 61], [221, 67, 70], [69, 127, 243], [95, 100, 42], [37, 33, 174], [217, 11, 205], [51, 40, 168], [22, 62, 248], [126, 128, 23], [162, 59, 198], [46, 44, 33], [185, 192, 222], [98, 59, 8], [218, 66, 191], [85, 30, 117], [150, 252, 175], [81, 214, 61], [164, 25, 60], [117, 126, 218], [131, 77, 217], [218, 123, 150], [130, 217, 145], [44, 103, 231], [147, 152, 100], [55, 20, 106], [174, 247, 142], [205, 115, 45], [99, 214, 61], [124, 139, 40], [170, 245, 126], [21, 254, 215], [88, 67, 200], [103, 208, 97], [127, 85, 179], [154, 195, 230], [133, 39, 134], [158, 87, 97], [36, 105, 36], [204, 72, 24], [67, 91, 185], [61, 222, 3], [5, 165, 161], [201, 173, 155], [188, 133, 246], [112, 8, 103], [240, 161, 62], [72, 24, 209], [119, 46, 98], [88, 245, 183], [85, 53, 223], [84, 174, 6], [212, 132, 133], [78, 28, 100], [199, 187, 199], [221, 157, 29], [208, 103, 50], [196, 85, 110], [108, 89, 114], [64, 136, 178], [71, 240, 14], [180, 108, 102], [198, 214, 139], [148, 124, 47], [135, 178, 202], [13, 113, 241], [179, 108, 140], [77, 33, 134], [101, 26, 53], [166, 84, 37], [206, 198, 179], [43, 35, 4], [160, 105, 122], [72, 253, 243], [2, 27, 175], [104, 239, 99], [130, 9, 220], [5, 204, 149], [203, 50, 39], [85, 101, 192], [162, 152, 234], [213, 233, 184], [242, 23, 255], [183, 178, 116], [167, 84, 212], [184, 81, 51], [251, 8, 202], [15, 211, 55], [175, 72, 49], [9, 64, 74], [120, 74, 83], [144, 108, 251], [24, 121, 167], [57, 53, 12], [64, 122, 100], [174, 15, 11], [75, 237, 147], [141, 234, 147], [196, 142, 84], [245, 186, 253], [108, 99, 55], [66, 176, 30], [143, 83, 242], [154, 215, 51], [146, 54, 10], [137, 243, 121], [15, 220, 248], [108, 38, 158], [206, 91, 192], [225, 220, 47], [175, 97, 214], [161, 228, 243], [204, 240, 55], [35, 16, 159], [222, 156, 212], [85, 122, 215], [176, 206, 196], [234, 240, 17], [242, 18, 248], [48, 243, 230], [41, 97, 134], [210, 90, 59], [45, 194, 129], [132, 192, 35], [154, 86, 40], [92, 152, 199], [14, 140, 151], [118, 122, 216], [15, 245, 58], [31, 78, 96], [56, 171, 166], [42, 191, 156], [189, 122, 48], [68, 86, 99], [158, 165, 202], [6, 1, 110], [155, 207, 179], [139, 248, 217], [239, 92, 129], [243, 46, 92], [3, 97, 144], [237, 108, 252], [37, 56, 170], [132, 24, 116], [201, 202, 90], [60, 87, 227], [51, 104, 154], [143, 215, 98], [57, 90, 149], [68, 12, 5], [241, 33, 74], [242, 113, 29], [220, 117, 134], [76, 104, 195], [56, 69, 7], [225, 219, 2], [8, 51, 54], [34, 29, 91], [106, 74, 116], [255, 50, 62], [53, 111, 99], [85, 119, 140], [141, 154, 65], [135, 164, 14], [35, 168, 115], [166, 127, 244], [228, 45, 65], [74, 179, 248], [68, 23, 253], [205, 170, 45], [253, 68, 67], [240, 29, 171], [134, 230, 28], [9, 168, 182], [36, 218, 35], [128, 28, 242], [140, 102, 64], [126, 29, 47], [98, 46, 62], [124, 141, 4], [74, 15, 159], [148, 255, 244], [154, 245, 96], [59, 242, 62], [162, 140, 130], [128, 30, 158], [7, 227, 180], [57, 234, 134], [100, 159, 46], [55, 212, 75], [42, 210, 61], [150, 108, 13], [201, 143, 72], [247, 123, 86], [55, 49, 251], [28, 7, 195], [60, 83, 151], [10, 104, 200], [37, 5, 12], [88, 5, 252], [240, 95, 184], [7, 233, 207], [138, 175, 38], [216, 37, 157], [31, 158, 191], [151, 65, 130], [196, 244, 220], [39, 34, 105], [154, 186, 145], [253, 65, 142], [78, 232, 222], [251, 138, 181], [2, 198, 203], [118, 45, 132], [52, 85, 163], [86, 185, 14], [50, 121, 4], [16, 116, 218], [249, 35, 241], [154, 226, 33], [56, 166, 173], [6, 163, 186], [250, 222, 4], [69, 230, 228], [50, 185, 41], [185, 119, 102], [171, 107, 118], [255, 50, 184], [185, 231, 101], [133, 83, 171], [14, 110, 60], [146, 13, 218], [220, 58, 204], [48, 209, 90], [178, 239, 188], [183, 87, 130], [14, 20, 164], [32, 128, 39], [133, 114, 65], [0, 93, 211], [239, 183, 177], [189, 110, 13], [15, 61, 168], [148, 131, 176], [255, 151, 212], [96, 57, 29], [206, 39, 196], [110, 77, 219], [92, 161, 183], [213, 247, 57], [112, 49, 95], [212, 160, 248], [118, 113, 10], [193, 182, 232], [70, 94, 20], [169, 170, 66], [190, 194, 103], [105, 147, 119], [147, 124, 202], [120, 91, 44], [70, 103, 13], [244, 49, 85], [165, 248, 45], [28, 134, 83], [192, 45, 60], [249, 241, 187], [80, 218, 36], [142, 127, 245], [164, 207, 149], [26, 23, 146], [220, 132, 144], [39, 9, 6], [209, 201, 115], [112, 127, 16], [116, 80, 176], [189, 96, 59], [50, 58, 117], [53, 86, 103], [81, 207, 47], [161, 182, 185], [144, 170, 123], [100, 86, 75], [29, 135, 171], [183, 74, 85], [45, 73, 180], [144, 145, 253], [20, 0, 183], [16, 145, 238], [61, 89, 154], [237, 251, 100], [130, 186, 187], [41, 208, 1], [48, 35, 129], [245, 162, 107], [251, 238, 38], [1, 249, 134], [128, 61, 160], [217, 158, 241], [82, 180, 214], [235, 205, 78], [215, 139, 236], [217, 221, 154], [22, 16, 154], [103, 165, 141], [171, 125, 29], [194, 254, 227], [40, 220, 24], [240, 186, 105], [198, 50, 18], [246, 45, 36], [230, 244, 95], [43, 91, 198], [94, 73, 187], [87, 217, 247], [33, 201, 3], [190, 142, 56], [33, 11, 17], [169, 159, 196], [146, 4, 181], [150, 88, 164], [115, 111, 160], [204, 74, 0], [164, 199, 213], [22, 151, 143], [170, 55, 7], [163, 183, 52], [10, 155, 203], [64, 174, 131], [137, 131, 25], [178, 49, 32], [24, 127, 234], [177, 29, 69], [163, 248, 32], [151, 215, 150], [238, 125, 75], [223, 28, 242], [89, 106, 53], [235, 30, 136], [253, 190, 186], [32, 138, 7], [28, 28, 122], [32, 232, 140], [38, 100, 115], [38, 250, 146], [5, 192, 27], [24, 240, 148], [176, 113, 146], [84, 53, 84], [191, 21, 169], [138, 220, 142], [196, 213, 186], [187, 250, 90], [42, 236, 122], [60, 38, 120], [213, 232, 3], [64, 229, 41], [210, 150, 200], [67, 164, 25], [36, 211, 100], [37, 50, 51], [85, 75, 113], [48, 230, 227], [110, 148, 15], [114, 44, 21], [254, 62, 42], [252, 233, 128], [141, 47, 184], [176, 129, 149], [178, 210, 142], [142, 247, 73], [165, 67, 2], [104, 150, 151], [116, 136, 150], [7, 86, 103], [217, 238, 165], [191, 228, 59], [55, 141, 95], [234, 254, 175], [212, 227, 0], [208, 37, 81], [150, 113, 96], [212, 140, 214], [4, 44, 145], [66, 153, 109], [48, 172, 116], [211, 9, 79], [86, 100, 148], [97, 71, 123], [67, 130, 94], [247, 50, 152], [56, 149, 181], [231, 231, 202], [221, 212, 89], [40, 245, 114], [169, 26, 2], [56, 102, 34], [29, 78, 121], [251, 246, 230], [10, 255, 34], [108, 190, 40], [59, 129, 130], [125, 200, 97], [32, 29, 97], [89, 244, 172], [44, 10, 224], [44, 247, 93], [48, 61, 242], [210, 28, 173], [92, 191, 79], [215, 179, 149], [132, 161, 218], [71, 80, 218], [158, 35, 53], [63, 41, 224], [229, 63, 74], [173, 135, 27], [241, 22, 32], [144, 51, 67], [226, 152, 28], [115, 192, 238], [121, 169, 253], [237, 43, 194], [18, 136, 186], [25, 2, 36], [14, 152, 77], [4, 71, 229], [222, 51, 135], [156, 167, 52], [171, 211, 132], [249, 188, 25], [141, 101, 191], [238, 219, 68], [118, 171, 113], [181, 183, 130], [206, 106, 180], [181, 146, 66], [192, 7, 104], [214, 55, 175], [198, 139, 232], [80, 247, 181], [52, 234, 106], [150, 60, 93], [45, 96, 31], [74, 227, 133], [227, 133, 44], [254, 135, 69], [110, 43, 100], [157, 129, 165], [0, 120, 49], [111, 19, 91], [160, 235, 133], [113, 95, 31], [239, 49, 40], [241, 254, 163], [154, 99, 89], [170, 77, 206], [36, 60, 242], [26, 105, 117], [190, 38, 137], [61, 138, 18], [67, 153, 153], [23, 198, 130], [192, 191, 2], [187, 7, 237], [228, 17, 61], [100, 73, 129], [34, 4, 107], [138, 144, 238], [86, 88, 161], [198, 34, 49], [53, 238, 48], [192, 70, 245], [206, 173, 244], [101, 208, 230], [126, 135, 0], [136, 56, 192], [7, 211, 164], [208, 18, 110], [95, 33, 223], [64, 211, 33], [252, 195, 207], [87, 91, 27], [134, 120, 133], [139, 232, 220], [194, 254, 242], [197, 214, 119], [205, 54, 173], [24, 157, 193], [35, 138, 223], [97, 40, 61], [251, 137, 159], [54, 4, 164], [13, 22, 153], [25, 201, 204], [58, 89, 194], [52, 130, 2], [14, 44, 124], [97, 44, 96], [253, 36, 61], [168, 232, 184], [96, 119, 143], [17, 184, 55], [190, 192, 242], [250, 254, 130], [159, 153, 91], [133, 45, 233], [226, 14, 123], [72, 81, 43], [65, 49, 242], [36, 56, 93], [91, 164, 194], [16, 88, 85], [197, 119, 47], [189, 118, 238], [91, 208, 196], [78, 4, 217], [232, 219, 112], [24, 206, 118], [154, 36, 183], [106, 137, 62], [177, 198, 67], [149, 181, 15], [186, 144, 123], [37, 143, 28], [11, 197, 31], [144, 115, 205], [145, 177, 97], [174, 74, 64], [80, 143, 63], [251, 167, 108], [65, 244, 53], [213, 95, 137], [229, 176, 191], [226, 204, 96], [214, 189, 103], [115, 238, 85], [20, 189, 117], [254, 199, 147], [180, 83, 181], [33, 221, 158], [154, 107, 68], [31, 73, 200], [91, 108, 141], [11, 75, 131], [193, 31, 39], [199, 132, 159], [46, 194, 205], [82, 120, 211], [63, 163, 16], [225, 101, 12], [208, 89, 191], [40, 7, 26], [215, 212, 8], [217, 196, 238], [240, 68, 12], [22, 248, 224], [220, 178, 177], [236, 161, 67], [160, 203, 89], [164, 89, 53], [181, 248, 73], [104, 14, 152], [180, 69, 67], [233, 185, 17], [55, 57, 159], [165, 248, 234], [196, 4, 255], [220, 75, 130], [230, 206, 18], [245, 157, 42], [9, 196, 136], [225, 26, 58], [209, 173, 204], [188, 94, 132], [65, 251, 26], [147, 191, 196], [110, 192, 235], [60, 251, 5], [228, 167, 57], [215, 214, 186], [172, 115, 6], [63, 222, 255], [129, 226, 234], [18, 40, 141], [33, 209, 4], [20, 212, 176], [242, 221, 85], [17, 172, 206], [60, 51, 229], [57, 33, 12], [109, 122, 171], [217, 208, 219], [33, 98, 164], [8, 58, 123], [33, 174, 6], [97, 163, 235], [108, 157, 85], [201, 149, 144], [147, 83, 189], [169, 172, 103], [59, 114, 145], [198, 36, 82], [72, 152, 30], [171, 42, 76], [10, 193, 36], [84, 241, 147], [23, 53, 143], [160, 149, 216], [170, 109, 14], [238, 25, 90], [129, 209, 85], [8, 198, 172], [212, 128, 141], [192, 30, 165], [193, 152, 53], [157, 74, 208], [1, 161, 31], [41, 109, 227], [211, 227, 195], [66, 1, 157], [69, 78, 30], [239, 188, 192], [92, 255, 31], [228, 167, 98], [53, 214, 149], [37, 172, 226], [210, 172, 40], [138, 51, 254], [178, 168, 235], [180, 241, 122], [142, 1, 160], [123, 208, 107], [40, 53, 92], [237, 243, 77], [228, 116, 10], [163, 227, 161], [32, 187, 131], [238, 25, 65], [186, 213, 21], [98, 209, 218], [91, 144, 232], [91, 73, 164], [1, 61, 61], [221, 66, 7], [50, 116, 242], [112, 100, 157], [108, 226, 21], [48, 86, 172], [210, 105, 226], [53, 90, 59], [40, 10, 133], [61, 50, 60], [155, 87, 235], [164, 122, 93], [95, 67, 208], [30, 229, 214], [231, 89, 248], [199, 222, 11], [176, 183, 198], [230, 126, 5], [90, 106, 136], [180, 188, 176], [7, 24, 215], [70, 200, 127], [28, 236, 185], [171, 40, 123], [229, 71, 255], [254, 76, 238], [76, 137, 221], [162, 38, 88], [249, 251, 196], [161, 200, 117], [80, 112, 29], [172, 11, 18], [2, 21, 244], [50, 122, 216], [234, 190, 81], [192, 38, 128], [23, 98, 175], [63, 46, 220], [138, 223, 184], [135, 21, 150], [227, 119, 11], [195, 229, 155], [43, 145, 214], [223, 188, 218], [126, 195, 4], [243, 252, 13], [131, 93, 149], [69, 164, 81], [84, 228, 73], [4, 223, 141], [59, 156, 202], [15, 182, 80], [115, 48, 37], [109, 193, 138], [240, 146, 235], [12, 4, 145], [96, 27, 11], [85, 161, 98], [161, 97, 217], [73, 104, 215], [10, 236, 234], [5, 204, 113], [127, 117, 81], [48, 102, 200], [253, 228, 64], [169, 23, 112], [86, 88, 75], [44, 122, 94], [70, 82, 133], [93, 224, 226], [79, 153, 133], [159, 47, 134], [154, 15, 88], [131, 43, 13], [92, 94, 252], [28, 15, 18], [132, 37, 72], [109, 134, 195], [169, 246, 227], [204, 86, 57], [159, 111, 218], [197, 223, 48], [112, 44, 247], [37, 232, 69], [237, 121, 188], [245, 82, 173], [135, 222, 153], [198, 148, 38], [98, 90, 172], [237, 127, 139], [102, 201, 117], [162, 197, 125], [99, 159, 73], [170, 198, 224], [122, 168, 5], [78, 218, 42], [9, 13, 188], [4, 5, 50], [6, 194, 39], [212, 106, 175], [247, 174, 168], [82, 184, 55], [204, 249, 34], [64, 194, 91], [20, 77, 236], [94, 21, 100], [135, 57, 118], [7, 25, 195], [87, 248, 210], [14, 30, 118], [93, 107, 49], [158, 7, 205], [104, 135, 200], [194, 27, 48], [26, 159, 169], [12, 61, 75], [40, 135, 211], [106, 173, 32], [89, 227, 240], [23, 171, 177], [208, 130, 83], [203, 251, 63], [230, 112, 163], [46, 253, 94], [186, 229, 7], [123, 111, 170], [130, 36, 66], [153, 214, 195], [189, 176, 188], [47, 41, 191], [37, 146, 151], [135, 135, 150], [5, 195, 82], [135, 17, 101], [216, 231, 210], [54, 50, 183], [175, 75, 167], [66, 157, 223], [29, 212, 169], [210, 191, 244], [119, 151, 137], [181, 39, 6], [8, 182, 185], [142, 80, 220], [89, 51, 254], [54, 249, 9], [188, 15, 98], [207, 194, 70], [239, 142, 205], [56, 167, 82], [229, 138, 208], [33, 84, 137], [156, 192, 183], [199, 50, 254], [214, 238, 71], [115, 105, 147], [207, 88, 13], [92, 144, 51], [177, 242, 43], [252, 147, 236], [184, 86, 230], [190, 30, 67], [207, 235, 244], [58, 128, 102], [50, 75, 39], [106, 134, 247], [249, 252, 106], [230, 182, 8], [121, 7, 46], [51, 230, 168], [54, 66, 224], [48, 227, 133], [148, 7, 100], [34, 137, 79], [189, 120, 245], [235, 103, 111], [7, 246, 24], [125, 175, 181], [131, 129, 124], [174, 27, 167], [220, 78, 68], [202, 73, 61], [237, 49, 215], [64, 65, 208], [203, 162, 138], [233, 4, 237], [224, 76, 140], [3, 224, 83], [183, 96, 250], [177, 63, 69], [197, 128, 84], [43, 227, 240], [250, 22, 95], [164, 175, 51], [121, 3, 60], [171, 84, 134], [80, 134, 83], [35, 53, 67], [200, 157, 156], [216, 212, 171], [5, 67, 69], [126, 111, 86], [204, 124, 202], [180, 127, 160], [79, 216, 176], [67, 39, 17], [51, 128, 167], [92, 202, 251], [214, 139, 67], [94, 185, 18], [216, 133, 244], [77, 114, 85], [106, 92, 8], [36, 123, 20], [220, 243, 165], [69, 64, 253], [235, 75, 228], [172, 78, 79], [106, 224, 37], [239, 167, 78], [101, 135, 27], [19, 15, 68], [57, 23, 0], [246, 159, 132], [239, 52, 10], [160, 179, 127], [59, 159, 162], [179, 72, 97], [84, 45, 96], [198, 179, 34], [131, 9, 242], [251, 27, 9], [144, 156, 46], [199, 248, 18], [62, 238, 56], [89, 21, 79], [214, 119, 157], [129, 62, 65], [133, 57, 90], [239, 95, 15], [38, 96, 111], [142, 137, 87], [79, 26, 133], [15, 38, 47], [131, 160, 19], [211, 15, 48], [70, 187, 138], [134, 219, 161], [87, 9, 194], [66, 204, 135], [55, 214, 10], [242, 43, 221], [76, 21, 124], [167, 119, 43], [221, 7, 200], [100, 114, 77], [121, 223, 237], [199, 160, 198], [239, 233, 59], [107, 214, 218], [104, 225, 10], [94, 220, 77], [52, 168, 148], [170, 91, 204], [162, 194, 218], [100, 49, 196], [10, 176, 202], [166, 176, 194], [221, 167, 164], [159, 131, 32], [86, 219, 248], [100, 224, 171], [138, 35, 60], [79, 126, 178], [161, 163, 77], [112, 102, 207], [139, 156, 254], [106, 176, 27], [217, 138, 73], [80, 149, 1], [32, 18, 85], [185, 186, 120], [78, 183, 192], [45, 57, 202], [163, 110, 7], [122, 23, 224], [23, 192, 246], [198, 79, 223], [40, 58, 232], [185, 40, 201], [178, 15, 139], [173, 156, 72], [168, 199, 106], [105, 147, 134], [138, 123, 213], [194, 216, 91], [211, 116, 227], [234, 171, 168], [4, 198, 217], [2, 213, 162], [172, 79, 23], [182, 56, 122], [164, 232, 168], [73, 241, 35], [85, 145, 37], [141, 111, 160], [32, 159, 132], [118, 242, 78], [69, 227, 76], [7, 209, 21], [175, 227, 190], [56, 83, 187], [202, 74, 224], [154, 107, 118], [227, 106, 161], [131, 234, 140], [227, 63, 79], [139, 248, 30], [196, 97, 55], [2, 140, 27], [77, 167, 48], [13, 251, 45], [64, 99, 65], [89, 88, 193], [74, 196, 127], [154, 244, 78], [133, 102, 54], [74, 104, 207], [122, 70, 224], [205, 102, 108], [8, 4, 92], [110, 93, 204], [170, 28, 31], [210, 21, 111], [227, 90, 58], [21, 85, 231], [160, 146, 210], [72, 231, 72], [25, 224, 39], [210, 214, 117], [40, 202, 51], [36, 55, 101], [148, 220, 53], [0, 67, 1], [250, 93, 252], [200, 226, 112], [79, 194, 2], [152, 54, 148], [106, 179, 142], [98, 187, 19], [65, 90, 109], [63, 223, 13], [15, 163, 190], [235, 92, 99], [1, 47, 35], [237, 227, 221], [29, 245, 64], [189, 184, 194], [136, 193, 125], [201, 193, 51], [233, 248, 205], [147, 26, 19], [195, 129, 66], [5, 64, 7], [11, 187, 102], [99, 208, 32], [203, 93, 93], [54, 232, 189], [88, 86, 125], [197, 108, 239], [28, 254, 78], [26, 7, 61], [72, 82, 131], [23, 167, 203], [117, 175, 22], [81, 194, 47], [34, 120, 167], [39, 57, 131], [194, 76, 181], [145, 43, 5], [119, 36, 153], [185, 112, 74], [85, 203, 133], [129, 219, 166], [182, 27, 222], [76, 57, 81], [12, 219, 84], [115, 140, 168], [211, 33, 249], [158, 42, 1], [83, 157, 24], [50, 93, 27], [96, 214, 24], [121, 125, 34], [213, 62, 118], [21, 103, 50], [67, 14, 227], [173, 168, 207], [57, 44, 77], [17, 210, 219], [30, 13, 120], [139, 55, 108], [58, 180, 232], [16, 21, 141], [141, 182, 245], [239, 103, 34], [219, 143, 80], [111, 115, 144], [192, 79, 16], [46, 213, 165], [39, 197, 42], [177, 11, 160], [104, 123, 42], [35, 157, 144], [170, 36, 30], [7, 178, 196], [172, 149, 133], [230, 94, 81], [95, 124, 134], [73, 68, 153], [11, 252, 242], [10, 20, 24], [100, 190, 22], [198, 172, 239], [82, 246, 99], [143, 69, 196], [123, 135, 203], [59, 41, 191], [254, 154, 54], [41, 190, 43], [194, 6, 68], [142, 255, 113], [87, 206, 209], [64, 204, 22], [129, 174, 52], [91, 148, 237], [220, 0, 194], [158, 33, 147], [41, 78, 228], [25, 50, 138], [21, 255, 125], [180, 92, 184], [73, 225, 29], [137, 151, 227], [100, 158, 171], [213, 102, 174], [183, 168, 197], [149, 22, 248], [155, 40, 156], [142, 236, 235], [142, 78, 66], [79, 229, 151], [53, 3, 138], [111, 102, 169], [209, 104, 185], [77, 255, 76], [19, 238, 241], [228, 37, 143], [158, 136, 41], [202, 172, 95], [34, 244, 131], [156, 85, 206], [201, 49, 163], [154, 188, 246], [90, 30, 27], [44, 61, 94], [140, 69, 188], [0, 116, 122], [100, 44, 192], [190, 219, 179], [72, 123, 1], [0, 174, 143], [31, 236, 53], [186, 66, 86], [55, 90, 5], [49, 239, 154], [28, 160, 135], [135, 111, 102], [83, 195, 111], [168, 243, 216], [49, 246, 12], [185, 224, 154], [73, 237, 7], [20, 164, 155], [151, 148, 159], [175, 241, 57], [156, 12, 206], [69, 146, 34], [251, 159, 55], [165, 53, 68], [57, 2, 44], [171, 143, 40], [3, 193, 150], [205, 228, 42], [132, 2, 254], [168, 112, 169], [118, 218, 16], [131, 198, 247], [122, 140, 3], [172, 23, 66], [164, 35, 43], [189, 152, 26], [55, 183, 91], [142, 215, 114], [116, 207, 86], [57, 89, 0], [232, 117, 21], [2, 181, 178], [25, 27, 117], [141, 93, 148], [231, 164, 194], [59, 146, 249], [204, 224, 45], [46, 65, 184], [11, 211, 104], [149, 126, 85], [236, 107, 143], [188, 131, 83], [154, 254, 82], [155, 233, 205], [101, 142, 175], [77, 95, 165], [170, 180, 220], [136, 163, 11], [149, 170, 103], [175, 17, 129], [159, 15, 79], [71, 255, 48], [22, 160, 177], [28, 84, 234], [162, 63, 198], [95, 173, 33], [192, 195, 89], [0, 3, 54], [68, 50, 83], [237, 188, 15], [34, 141, 75], [25, 73, 131], [145, 65, 133], [207, 141, 202], [89, 30, 145], [87, 141, 180], [252, 9, 34], [208, 178, 127], [211, 220, 76], [224, 51, 99], [193, 36, 48], [122, 237, 66], [25, 170, 200], [78, 7, 137], [50, 54, 248], [236, 104, 200], [233, 200, 11], [106, 1, 254], [88, 246, 197], [102, 212, 236], [61, 12, 135], [192, 123, 204], [26, 214, 184], [131, 102, 94], [109, 95, 37], [23, 75, 213], [227, 124, 24], [243, 98, 178], [140, 135, 213], [90, 215, 77], [250, 224, 20], [209, 37, 82], [142, 162, 37], [213, 72, 224], [43, 150, 244], [125, 187, 232], [104, 138, 142], [5, 115, 121], [23, 2, 194], [160, 189, 47], [152, 88, 106], [135, 101, 90], [57, 25, 183], [128, 227, 54], [82, 7, 239], [33, 223, 97], [149, 40, 64], [157, 44, 177], [17, 214, 53], [138, 52, 80], [219, 250, 170], [16, 78, 61], [22, 149, 58], [222, 136, 101], [55, 170, 231], [199, 245, 44], [66, 244, 229], [230, 77, 73], [137, 250, 170], [149, 244, 196], [44, 81, 133], [218, 52, 109], [96, 78, 125], [29, 199, 211], [85, 243, 109], [180, 138, 126], [4, 205, 100], [67, 60, 89], [13, 185, 17], [226, 226, 100], [195, 73, 111], [214, 157, 63], [216, 253, 42], [224, 128, 48], [237, 115, 227], [90, 114, 74], [132, 138, 149], [179, 135, 166], [44, 228, 69], [97, 243, 6], [74, 47, 164], [85, 239, 250], [167, 120, 111], [87, 221, 240], [189, 190, 104], [85, 52, 3], [189, 49, 135], [157, 134, 60], [204, 202, 149], [68, 237, 66], [208, 225, 219], [171, 177, 9], [148, 83, 4], [34, 2, 74], [46, 172, 252], [36, 175, 232], [55, 162, 53], [231, 131, 111], [209, 234, 177], [173, 173, 10], [250, 189, 240], [164, 4, 177], [191, 186, 240], [137, 84, 252], [238, 57, 10], [242, 109, 187], [175, 216, 160], [207, 65, 183], [92, 177, 96], [68, 188, 144], [153, 12, 219], [214, 238, 49], [238, 104, 133], [71, 148, 59], [120, 247, 168], [191, 238, 155], [90, 112, 8], [56, 133, 101], [75, 223, 222], [25, 238, 133], [198, 79, 76], [59, 226, 90], [125, 39, 140], [225, 53, 45], [159, 101, 116], [102, 110, 217], [16, 61, 216], [209, 85, 180], [27, 249, 209], [186, 159, 162], [141, 207, 89], [155, 152, 74], [108, 126, 194], [143, 113, 109], [162, 69, 59], [189, 210, 103], [60, 240, 205], [140, 201, 236], [88, 111, 215], [225, 250, 135], [143, 128, 14], [235, 83, 26], [143, 125, 22], [130, 121, 68], [188, 191, 228], [46, 160, 147], [167, 205, 71], [83, 34, 7], [145, 51, 200], [237, 172, 96], [21, 108, 10], [14, 177, 154], [226, 243, 32], [204, 200, 205], [169, 169, 71], [206, 14, 46], [106, 13, 106], [2, 56, 83], [75, 212, 97], [88, 246, 222], [56, 68, 83], [219, 26, 73], [251, 232, 177], [234, 26, 241], [82, 250, 206], [46, 5, 37], [33, 176, 21], [174, 132, 183], [169, 30, 221], [199, 154, 121], [96, 157, 166], [132, 67, 47], [58, 71, 216], [54, 93, 118], [215, 126, 59], [235, 245, 46], [30, 77, 59], [163, 169, 139], [78, 149, 34], [134, 115, 175], [171, 21, 189], [78, 89, 22], [123, 176, 71], [222, 198, 80], [115, 241, 181], [215, 91, 178], [207, 249, 47], [135, 148, 229], [193, 252, 37], [183, 107, 236], [146, 221, 203], [77, 7, 118], [19, 156, 157], [224, 93, 125], [177, 197, 146], [6, 156, 196], [138, 232, 25], [8, 149, 5], [111, 251, 155], [20, 74, 138], [131, 192, 18], [0, 207, 17], [95, 225, 32], [31, 91, 150], [23, 203, 144], [170, 124, 174], [224, 172, 109], [76, 120, 3], [181, 29, 86], [253, 1, 244], [127, 184, 213], [159, 38, 15], [130, 37, 191], [154, 47, 176], [18, 153, 172], [78, 189, 172], [101, 86, 87], [194, 59, 72], [168, 30, 146], [87, 132, 118], [150, 214, 101], [183, 142, 138], [42, 172, 67], [40, 245, 244], [88, 191, 39], [21, 177, 188], [249, 189, 203], [18, 135, 9], [37, 50, 104], [175, 61, 151], [74, 73, 140], [130, 215, 188], [73, 238, 141], [30, 170, 74], [74, 95, 14], [211, 149, 56], [20, 106, 193], [80, 69, 30], [127, 225, 198], [191, 69, 179], [11, 195, 150], [173, 11, 155], [155, 22, 123], [123, 85, 24], [60, 108, 206], [65, 152, 219], [85, 252, 189], [165, 232, 210], [118, 144, 132], [44, 183, 126], [116, 234, 234], [10, 246, 137], [234, 139, 40], [106, 160, 174], [147, 118, 46], [18, 195, 197], [192, 105, 51], [39, 231, 125], [243, 19, 155], [101, 196, 237], [47, 214, 195], [193, 217, 0], [173, 168, 116], [249, 120, 0], [41, 73, 227], [93, 210, 39], [114, 87, 238], [155, 26, 82], [64, 166, 205], [195, 130, 235], [40, 115, 58], [210, 84, 66], [169, 126, 109], [89, 176, 181], [189, 57, 236], [231, 216, 160], [87, 27, 66], [63, 190, 4], [210, 149, 206], [118, 236, 63], [187, 46, 140], [153, 61, 197], [157, 231, 87], [121, 225, 0], [220, 124, 157], [245, 78, 37], [65, 105, 233], [212, 140, 242], [239, 250, 28], [198, 207, 132], [103, 214, 171], [251, 78, 96], [90, 225, 119], [164, 32, 78], [186, 7, 154], [69, 208, 71], [23, 23, 1], [197, 201, 226], [71, 181, 253], [142, 224, 18], [55, 125, 210], [121, 159, 167], [92, 81, 193], [83, 253, 30], [132, 243, 213], [86, 108, 103], [144, 184, 151], [225, 33, 111], [197, 178, 216], [184, 98, 250], [87, 57, 9], [111, 99, 60], [160, 130, 60], [90, 227, 66], [32, 119, 235], [139, 129, 133], [208, 146, 68], [154, 144, 85], [211, 164, 119], [154, 99, 216], [251, 194, 55], [251, 170, 233], [123, 188, 8], [161, 185, 211], [50, 207, 214], [121, 200, 83], [183, 0, 216], [165, 35, 164], [201, 107, 181], [226, 66, 159], [179, 21, 12], [215, 111, 10], [155, 48, 188], [88, 30, 128], [151, 198, 198], [107, 187, 80], [194, 99, 102], [149, 185, 188], [46, 231, 81], [242, 222, 89], [182, 155, 95], [145, 220, 249], [136, 97, 50], [247, 20, 187], [245, 140, 214], [239, 179, 35], [252, 77, 53], [147, 174, 16], [242, 95, 133], [126, 247, 174], [179, 237, 72], [128, 78, 200], [180, 116, 178], [21, 97, 190], [48, 54, 104], [7, 83, 3], [201, 155, 165], [210, 57, 35], [27, 181, 121], [180, 197, 154], [24, 165, 221], [63, 90, 171], [139, 43, 191], [231, 115, 83], [7, 63, 54], [120, 159, 233], [75, 2, 47], [254, 186, 66], [89, 210, 187], [87, 197, 182], [91, 11, 12], [33, 15, 17], [49, 144, 76], [137, 74, 254], [152, 50, 149], [66, 231, 84], [127, 65, 135], [104, 174, 130], [32, 174, 171], [128, 184, 130], [242, 19, 151], [82, 252, 10], [211, 158, 81], [198, 81, 152], [210, 94, 182], [199, 148, 165], [103, 118, 22], [191, 125, 223], [146, 4, 96], [229, 205, 78], [95, 11, 165], [197, 74, 27], [42, 147, 148], [20, 214, 118], [160, 155, 20], [129, 18, 228], [64, 108, 243], [102, 130, 140], [243, 86, 81], [59, 82, 157], [56, 250, 64], [231, 134, 15], [49, 39, 109], [190, 139, 84], [189, 180, 68], [237, 104, 196], [196, 94, 185], [155, 20, 41], [82, 56, 7], [17, 208, 5], [100, 253, 85], [8, 39, 168], [1, 243, 42], [145, 131, 157], [26, 180, 182], [181, 230, 31], [178, 99, 254], [156, 160, 23], [81, 113, 25], [198, 247, 44], [203, 203, 44], [13, 56, 135], [40, 67, 210], [189, 18, 210], [245, 113, 217], [136, 147, 62], [105, 99, 209], [95, 16, 204], [57, 219, 237], [35, 159, 227], [81, 148, 48], [128, 227, 11], [137, 210, 118], [65, 147, 219], [235, 91, 181], [187, 82, 6], [235, 161, 43], [45, 153, 163], [231, 105, 10], [111, 75, 15], [158, 52, 164], [88, 220, 163], [182, 194, 159], [243, 6, 138], [111, 194, 45], [232, 214, 228], [105, 106, 12], [180, 132, 160], [49, 169, 88], [48, 64, 157], [117, 235, 136], [176, 80, 223], [35, 204, 131], [215, 116, 158], [15, 118, 10], [52, 227, 225], [250, 245, 134], [202, 209, 23], [170, 235, 34], [65, 151, 246], [215, 176, 132], [62, 161, 69], [244, 27, 218], [117, 254, 51], [15, 177, 254], [95, 202, 153], [123, 197, 103], [218, 172, 181], [199, 88, 225], [158, 64, 180], [89, 91, 65], [62, 75, 230], [251, 118, 108], [248, 158, 202], [70, 176, 69], [175, 168, 167], [78, 23, 97], [34, 88, 67], [197, 165, 188], [166, 88, 51], [156, 131, 166], [18, 108, 161], [51, 176, 162], [173, 237, 231], [141, 160, 120], [82, 86, 142], [74, 146, 200], [97, 195, 87], [74, 187, 245], [108, 47, 152], [185, 24, 172], [38, 12, 237], [107, 78, 107], [18, 228, 195], [56, 69, 60], [127, 129, 130], [221, 184, 40], [210, 70, 238], [77, 162, 49], [90, 213, 249], [192, 221, 178], [36, 219, 211], [214, 144, 129], [197, 135, 4], [159, 175, 173], [126, 104, 138], [118, 169, 87], [145, 194, 219], [135, 154, 123], [236, 12, 127], [26, 156, 147], [167, 78, 86], [106, 176, 169], [155, 196, 92], [77, 101, 29], [224, 253, 225], [181, 145, 93], [128, 173, 7], [102, 84, 231], [87, 249, 26], [200, 44, 18], [227, 5, 178], [106, 73, 207], [34, 200, 165], [46, 77, 172], [206, 158, 109], [20, 27, 76], [193, 93, 79], [38, 174, 53], [123, 99, 52], [108, 30, 114], [160, 200, 161], [213, 12, 87], [118, 174, 36], [255, 88, 187], [141, 181, 13], [230, 204, 201], [29, 155, 142], [89, 55, 112], [150, 16, 157], [74, 30, 71], [60, 86, 130], [250, 67, 243], [180, 66, 119], [219, 150, 19], [55, 149, 110], [160, 48, 74], [63, 201, 158], [2, 55, 189], [206, 48, 61], [72, 63, 154], [103, 180, 187], [149, 168, 247], [221, 30, 68], [212, 252, 218], [101, 204, 6], [140, 219, 19], [66, 39, 76], [53, 66, 117], [99, 225, 74], [136, 44, 229], [37, 242, 83], [180, 78, 103], [175, 19, 40], [186, 179, 15], [41, 210, 2], [165, 7, 160], [147, 50, 155], [21, 171, 177], [181, 137, 148], [4, 88, 250], [165, 76, 61], [140, 197, 12], [252, 60, 47], [80, 227, 247], [242, 209, 139], [249, 62, 137], [159, 190, 232], [36, 21, 120], [248, 116, 206], [70, 129, 47], [54, 68, 154], [162, 233, 44], [39, 241, 220], [65, 227, 73], [33, 159, 0], [129, 30, 1], [157, 156, 74], [87, 162, 101], [67, 178, 125], [205, 227, 92], [239, 253, 122], [168, 173, 222], [167, 146, 148], [121, 46, 241], [89, 134, 163], [96, 255, 97], [193, 183, 172], [91, 160, 34], [209, 68, 146], [62, 254, 161], [34, 56, 164], [11, 79, 130], [213, 98, 166], [254, 80, 55], [62, 143, 65], [128, 96, 126], [236, 136, 142], [23, 96, 255], [91, 74, 193], [75, 159, 85], [36, 242, 173], [89, 233, 205], [188, 79, 243], [61, 56, 255], [74, 166, 19], [147, 230, 132], [37, 165, 223], [215, 59, 117], [213, 189, 166], [134, 218, 206], [65, 216, 34], [62, 114, 196], [236, 7, 216], [177, 61, 9], [150, 158, 195], [233, 101, 119], [40, 29, 79], [120, 86, 134], [53, 224, 5], [117, 255, 168], [152, 184, 117], [228, 165, 78], [241, 48, 132], [69, 161, 159], [145, 124, 193], [42, 19, 163], [127, 36, 2], [25, 219, 4], [16, 106, 33], [47, 253, 91], [97, 161, 61], [199, 24, 238], [247, 172, 140], [141, 53, 244], [255, 15, 41], [182, 143, 77], [0, 74, 190], [138, 94, 91], [142, 31, 75], [48, 232, 85], [35, 32, 53], [222, 186, 103], [47, 230, 115], [130, 141, 16], [160, 177, 52], [202, 243, 126], [251, 252, 90], [185, 85, 100], [158, 148, 146], [123, 242, 228], [17, 214, 92], [160, 144, 205], [9, 168, 206], [21, 14, 114], [133, 138, 178], [113, 144, 231], [121, 138, 255], [127, 13, 103], [186, 64, 75], [193, 98, 226], [182, 180, 2], [106, 41, 218], [141, 176, 212], [203, 175, 40], [10, 130, 76], [170, 37, 171], [243, 37, 34], [183, 112, 165], [130, 252, 47], [13, 186, 63], [205, 2, 116], [54, 151, 20], [57, 172, 58], [45, 133, 70], [106, 248, 163], [179, 80, 173], [113, 218, 198], [24, 72, 165], [20, 135, 110], [31, 191, 121], [252, 63, 75], [68, 128, 179], [96, 104, 158], [44, 14, 246], [247, 174, 241], [61, 224, 161], [31, 236, 152], [25, 211, 175], [145, 221, 10], [3, 170, 175], [224, 246, 23], [19, 222, 97], [209, 161, 82], [249, 154, 205], [133, 49, 4], [166, 134, 50], [36, 6, 54], [45, 41, 238], [45, 242, 8], [125, 84, 120], [131, 231, 36], [237, 241, 186], [165, 196, 231], [210, 204, 191], [52, 9, 231], [42, 67, 52], [96, 170, 191], [248, 232, 31], [152, 142, 103], [95, 198, 211], [75, 37, 90], [211, 129, 142], [210, 15, 35], [241, 168, 38], [206, 50, 29], [153, 209, 63], [73, 44, 193], [71, 132, 182], [114, 215, 148], [84, 76, 183], [86, 123, 64], [164, 199, 89], [91, 43, 238], [152, 132, 19], [47, 233, 164], [27, 7, 31], [69, 162, 141], [13, 68, 172], [181, 241, 245], [35, 190, 206], [77, 50, 209], [79, 60, 9], [175, 200, 167], [153, 48, 181], [41, 215, 181], [178, 173, 58], [235, 38, 254], [15, 169, 64], [179, 149, 128], [154, 5, 210], [203, 52, 6], [201, 178, 219], [154, 84, 171], [103, 150, 60], [201, 30, 5], [232, 140, 174], [191, 16, 215], [122, 185, 111], [208, 235, 94], [126, 106, 77], [105, 6, 32], [103, 51, 40], [220, 136, 31], [143, 246, 202], [116, 151, 134], [119, 104, 116], [71, 242, 104], [187, 195, 134], [152, 170, 72], [171, 78, 185], [138, 86, 133], [223, 4, 217], [121, 23, 135], [243, 234, 247], [87, 221, 250], [99, 88, 179], [96, 18, 56], [146, 179, 56], [32, 87, 63], [210, 10, 104], [141, 81, 222], [177, 43, 13], [247, 118, 17], [176, 49, 239], [19, 73, 65], [136, 93, 162], [183, 79, 85], [155, 159, 0], [94, 111, 232], [221, 35, 144], [10, 239, 2], [71, 134, 146], [241, 168, 31], [23, 145, 142], [177, 154, 188], [216, 79, 193], [240, 98, 110], [213, 90, 37], [109, 145, 122], [212, 36, 209], [116, 238, 211], [40, 39, 50], [10, 111, 161], [106, 189, 224], [80, 36, 54], [211, 33, 86], [61, 94, 218], [89, 87, 137], [242, 194, 174], [200, 43, 93], [94, 174, 170], [170, 251, 203], [77, 127, 80], [246, 196, 99], [9, 4, 65], [79, 76, 1], [213, 72, 246], [12, 197, 114], [89, 57, 157], [27, 163, 232], [106, 162, 62], [77, 229, 88], [126, 165, 246], [151, 44, 84], [55, 123, 233], [162, 128, 129], [49, 24, 94], [133, 205, 150], [123, 37, 237], [2, 139, 18], [167, 121, 126], [111, 70, 30], [161, 175, 70], [130, 75, 210], [16, 112, 27], [253, 211, 226], [47, 77, 192], [151, 196, 155], [15, 109, 72], [16, 145, 210], [96, 51, 32], [112, 231, 196], [19, 185, 105], [125, 158, 42], [80, 229, 88], [123, 137, 49], [200, 78, 241], [121, 106, 39], [191, 194, 250], [255, 161, 165], [61, 114, 131], [232, 57, 161], [94, 134, 245], [8, 11, 39], [52, 164, 22], [91, 4, 130], [150, 52, 126], [19, 208, 147], [245, 148, 249], [198, 211, 13], [96, 175, 223], [224, 182, 86], [241, 50, 71], [246, 115, 29], [252, 254, 185], [210, 159, 179], [233, 63, 64], [53, 48, 254], [222, 162, 0], [61, 175, 131], [134, 105, 159], [228, 180, 132], [143, 93, 106], [13, 43, 37], [219, 22, 70], [187, 118, 101], [97, 226, 21], [197, 82, 63], [108, 208, 159], [227, 156, 132], [197, 23, 81], [72, 250, 108], [61, 205, 166], [227, 20, 137], [128, 111, 75], [220, 216, 90], [16, 108, 241], [20, 247, 168], [108, 197, 191], [62, 23, 242], [192, 2, 49], [183, 167, 50], [167, 180, 19], [117, 58, 126], [230, 40, 110], [134, 28, 143], [205, 219, 185], [126, 1, 185], [227, 221, 111], [3, 12, 166], [85, 174, 116], [152, 86, 137], [41, 219, 125], [10, 142, 201], [92, 172, 168], [244, 120, 46], [152, 75, 112], [190, 38, 123], [209, 113, 215], [19, 133, 36], [112, 237, 138], [71, 21, 186], [93, 39, 8], [127, 55, 206], [72, 84, 143], [24, 146, 227], [44, 102, 239], [225, 46, 52], [114, 87, 140], [104, 119, 139], [115, 158, 23], [47, 227, 86], [96, 214, 1], [156, 154, 18], [178, 111, 90], [134, 71, 75], [160, 35, 200], [46, 112, 227], [103, 117, 43], [36, 139, 141], [161, 25, 29], [74, 129, 170], [77, 138, 144], [184, 230, 192], [54, 73, 117], [94, 6, 70], [187, 113, 186], [144, 237, 129], [22, 218, 81], [149, 246, 28], [198, 102, 7], [230, 111, 162], [132, 249, 11], [95, 215, 66], [18, 9, 119], [251, 65, 129], [167, 112, 45], [103, 48, 52], [170, 225, 56], [71, 184, 234], [247, 219, 104], [115, 39, 172], [26, 248, 213], [196, 238, 102], [60, 131, 48], [3, 167, 151], [156, 205, 115], [231, 49, 190], [37, 158, 69], [194, 37, 19], [54, 252, 97], [201, 189, 96], [75, 165, 219], [177, 190, 253], [41, 134, 41], [132, 19, 115], [193, 96, 194], [217, 65, 200], [69, 45, 173], [212, 18, 190], [193, 105, 105], [93, 180, 128], [177, 6, 9], [46, 22, 10], [232, 249, 234], [61, 173, 110], [90, 122, 126], [204, 163, 16], [45, 9, 61], [121, 177, 3], [184, 37, 146], [132, 6, 206], [205, 113, 74], [183, 12, 111], [91, 72, 84], [68, 15, 188], [116, 134, 1], [137, 142, 183], [24, 31, 27], [61, 106, 231], [133, 109, 100], [74, 101, 47], [243, 92, 10], [167, 154, 92], [203, 162, 134], [5, 62, 236], [111, 233, 64], [6, 138, 46], [10, 239, 19], [134, 243, 160], [70, 251, 221], [197, 14, 228], [201, 36, 78], [88, 40, 58], [198, 88, 253], [254, 119, 164], [85, 44, 146], [207, 37, 201], [201, 6, 200], [160, 169, 198], [136, 253, 227], [84, 69, 185], [203, 137, 106], [76, 140, 227], [147, 221, 92], [178, 162, 141], [80, 82, 60], [243, 57, 186], [94, 193, 56], [177, 18, 133], [116, 54, 132], [235, 192, 96], [128, 100, 188], [17, 49, 50], [251, 178, 211], [154, 253, 62], [31, 153, 100], [39, 238, 39], [53, 29, 190], [42, 131, 208], [144, 96, 96], [168, 163, 243], [29, 56, 10], [245, 231, 62], [167, 137, 239], [231, 115, 87], [240, 235, 160], [204, 203, 192], [87, 196, 8], [136, 29, 138], [133, 10, 170], [38, 255, 44], [43, 35, 138], [157, 78, 227], [233, 172, 4], [7, 136, 248], [19, 108, 152], [93, 58, 234], [7, 193, 29], [29, 196, 135], [53, 139, 99], [88, 95, 210], [31, 240, 142], [43, 10, 68], [49, 227, 6], [4, 101, 191], [90, 195, 142], [101, 42, 94], [120, 247, 64], [190, 202, 165], [23, 65, 126], [46, 136, 171], [51, 178, 252], [144, 127, 143], [174, 254, 83], [74, 156, 119], [163, 133, 16], [26, 127, 198], [8, 5, 161], [64, 129, 136], [189, 172, 229], [81, 238, 144], [87, 145, 186], [137, 7, 118], [6, 180, 185], [77, 208, 144], [93, 199, 111], [248, 48, 152], [214, 173, 28], [7, 60, 78], [80, 46, 156], [204, 145, 69], [165, 175, 175], [159, 133, 22], [89, 249, 149], [115, 96, 114], [176, 107, 86], [189, 193, 43], [145, 36, 244], [121, 214, 39], [241, 34, 115], [172, 198, 213], [78, 214, 40], [227, 247, 209], [54, 191, 154], [230, 131, 59], [78, 204, 128], [79, 97, 31], [174, 252, 227], [180, 87, 84], [138, 190, 17], [157, 160, 83], [147, 214, 211], [166, 240, 195], [107, 82, 143], [27, 6, 177], [116, 82, 98], [88, 8, 211], [215, 183, 139], [109, 205, 34], [0, 125, 80], [61, 92, 194], [29, 102, 68], [76, 96, 42], [101, 243, 57], [105, 220, 136], [81, 162, 172], [7, 185, 80], [54, 218, 152], [74, 4, 247], [136, 217, 153], [209, 244, 157], [244, 13, 18], [193, 68, 137], [63, 7, 247], [231, 122, 156], [51, 49, 225], [65, 210, 124], [105, 218, 67], [241, 255, 237], [92, 58, 89], [207, 139, 81], [125, 176, 217], [52, 247, 235], [32, 97, 210], [177, 105, 127], [162, 127, 169], [73, 117, 224], [12, 103, 106], [154, 247, 134], [105, 172, 31], [27, 119, 70], [87, 66, 13], [107, 170, 196], [94, 175, 133], [118, 212, 74], [248, 176, 111], [178, 180, 39], [21, 250, 135], [196, 245, 182], [72, 172, 172], [18, 139, 18], [70, 16, 168], [247, 42, 82], [63, 89, 152], [87, 168, 235], [33, 56, 193], [56, 174, 56], [78, 147, 247], [225, 98, 219], [250, 222, 208], [190, 104, 185], [92, 107, 210], [49, 192, 134], [233, 142, 76], [5, 52, 19], [36, 255, 17], [198, 160, 95], [130, 44, 183], [240, 93, 171], [115, 72, 155], [9, 85, 188], [44, 220, 120], [84, 206, 139], [29, 192, 62], [163, 30, 193], [45, 111, 67], [146, 162, 27], [193, 184, 31], [216, 90, 128], [61, 47, 21], [115, 143, 19], [209, 132, 28], [219, 148, 16], [174, 168, 28], [79, 198, 33], [207, 230, 118], [138, 132, 64], [93, 113, 22], [220, 170, 179], [113, 243, 175], [56, 149, 35], [72, 212, 170], [161, 194, 160], [178, 26, 156], [3, 220, 190], [72, 193, 21], [195, 149, 87], [237, 165, 168], [215, 172, 34], [187, 62, 247], [200, 136, 32], [252, 176, 114], [233, 221, 226], [26, 61, 164], [155, 103, 77], [170, 117, 173], [42, 245, 186], [189, 54, 220], [150, 85, 131], [216, 150, 47], [56, 237, 8], [198, 18, 75], [224, 243, 43], [43, 3, 11], [33, 198, 96], [33, 14, 248], [157, 152, 65], [152, 57, 220], [245, 86, 206], [144, 35, 131], [208, 107, 20], [142, 93, 203], [168, 153, 0], [170, 254, 192], [34, 131, 170], [0, 56, 52], [217, 33, 123], [207, 103, 6], [153, 181, 41], [107, 156, 127], [19, 60, 132], [145, 147, 229], [158, 140, 149], [209, 56, 117], [58, 164, 48], [133, 79, 16], [12, 8, 211], [44, 206, 206], [63, 116, 184], [218, 245, 209], [123, 156, 183], [73, 75, 149], [210, 117, 187], [90, 154, 238], [56, 31, 218], [136, 48, 146], [55, 221, 170], [65, 1, 124], [6, 243, 149], [212, 46, 62], [210, 196, 233], [212, 232, 98], [95, 96, 220], [44, 119, 225], [67, 248, 171], [126, 184, 194], [213, 176, 216], [185, 32, 24], [174, 110, 115], [19, 197, 143], [48, 160, 243], [154, 168, 178], [176, 94, 8], [132, 164, 122], [213, 37, 192], [85, 111, 100], [35, 109, 13], [170, 223, 87], [106, 164, 45], [180, 64, 182], [238, 29, 1], [211, 59, 147], [135, 201, 163], [244, 164, 46], [168, 198, 110], [252, 237, 63], [178, 107, 249], [141, 146, 0], [209, 164, 56], [136, 104, 115], [219, 204, 131], [105, 1, 65], [215, 99, 52], [223, 218, 214], [7, 201, 239], [69, 111, 159], [122, 76, 123], [137, 131, 72], [90, 104, 155], [152, 161, 14], [234, 220, 216], [194, 0, 112], [236, 219, 131], [246, 117, 236], [106, 246, 236], [209, 137, 163], [51, 205, 39], [224, 222, 83], [119, 21, 203], [56, 184, 29], [67, 52, 35], [10, 171, 242], [67, 169, 205], [155, 37, 1], [250, 9, 212], [168, 244, 78], [109, 156, 158], [157, 121, 62], [38, 123, 228], [79, 89, 235], [206, 251, 232], [29, 95, 76], [96, 11, 248], [106, 75, 101], [185, 32, 122], [204, 50, 171], [160, 176, 144], [10, 74, 84], [203, 161, 191], [235, 3, 77], [56, 172, 25], [234, 66, 44], [72, 164, 198], [71, 175, 182], [106, 162, 25], [135, 181, 226], [144, 25, 166], [116, 133, 204], [40, 176, 114], [82, 177, 243], [49, 113, 254], [134, 22, 172], [167, 217, 97], [183, 29, 196], [78, 83, 225], [174, 43, 98], [15, 36, 231], [199, 80, 2], [197, 101, 242], [219, 16, 178], [205, 232, 57], [220, 99, 134], [77, 59, 185], [2, 246, 105], [18, 125, 8], [125, 37, 114], [47, 43, 219], [148, 229, 80], [80, 4, 219], [165, 233, 90], [123, 248, 245], [98, 209, 4], [131, 229, 165], [41, 62, 210], [78, 159, 94], [48, 13, 126], [133, 28, 233], [139, 197, 171], [81, 182, 166], [55, 128, 105], [215, 232, 221], [68, 154, 175], [21, 183, 214], [252, 34, 234], [142, 119, 63], [192, 118, 68], [234, 3, 23], [210, 240, 220], [171, 8, 82], [151, 232, 48], [157, 166, 226], [133, 123, 112], [3, 253, 159], [25, 231, 21], [84, 29, 59], [134, 150, 122], [71, 115, 157], [3, 20, 95], [1, 33, 0], [65, 175, 190], [50, 52, 31], [143, 107, 60], [252, 232, 9], [24, 204, 225], [146, 170, 72], [170, 205, 59], [100, 110, 148], [36, 36, 184], [28, 206, 65], [201, 56, 230], [56, 63, 115], [240, 14, 253], [37, 168, 135], [113, 170, 254], [156, 19, 166], [208, 46, 36], [84, 75, 63], [112, 202, 75], [81, 225, 122], [43, 213, 94], [185, 144, 119], [51, 216, 104], [252, 211, 106], [125, 169, 234], [111, 123, 69], [184, 183, 217], [53, 46, 210], [250, 243, 252], [108, 254, 195], [199, 56, 233], [23, 91, 42], [47, 50, 27], [166, 182, 29], [106, 205, 145], [164, 42, 135], [67, 64, 94], [61, 59, 224], [246, 128, 72], [21, 56, 50], [148, 194, 20], [122, 53, 98], [6, 44, 24], [201, 144, 48], [211, 200, 210], [87, 153, 232], [62, 232, 78], [196, 19, 139], [209, 33, 249], [62, 47, 67], [66, 12, 35], [137, 146, 14], [63, 238, 226], [206, 208, 6], [227, 243, 20], [145, 94, 69], [7, 83, 144], [140, 130, 145], [80, 236, 234], [104, 161, 172], [32, 44, 145], [201, 0, 49], [119, 106, 81], [96, 39, 202], [210, 192, 146], [90, 48, 79], [98, 34, 142], [15, 215, 1], [104, 12, 0], [15, 20, 172], [198, 128, 78], [44, 91, 238], [129, 11, 171], [58, 2, 208], [219, 234, 243], [73, 5, 104], [95, 246, 58], [144, 41, 133], [223, 24, 244], [59, 228, 254], [240, 21, 155], [191, 207, 16], [51, 128, 54], [157, 113, 79], [153, 97, 168], [33, 149, 14], [74, 246, 148], [243, 146, 211], [137, 53, 29], [119, 43, 147], [188, 125, 103], [70, 33, 18], [64, 205, 151], [14, 38, 221], [239, 133, 145], [81, 220, 203], [151, 204, 65], [209, 148, 87], [27, 34, 103], [84, 79, 217], [181, 31, 60], [83, 189, 106], [9, 249, 127], [4, 58, 244], [220, 161, 10], [137, 146, 13], [126, 115, 99], [240, 205, 175], [33, 146, 188], [251, 144, 90], [164, 243, 154], [1, 89, 85], [227, 1, 85], [67, 209, 221], [70, 78, 137], [46, 135, 253], [100, 30, 248], [220, 8, 221], [120, 9, 25], [183, 54, 50], [31, 153, 86], [205, 44, 29], [229, 225, 190], [133, 100, 223], [207, 215, 141], [35, 238, 187], [173, 113, 108], [168, 16, 65], [174, 191, 45], [28, 119, 168], [88, 242, 123], [196, 32, 202], [126, 131, 9], [67, 205, 126], [179, 8, 233], [23, 39, 214], [231, 195, 209], [22, 134, 198], [164, 26, 140], [147, 28, 173], [45, 219, 188], [150, 198, 232], [63, 7, 118], [101, 20, 183], [6, 89, 234], [129, 133, 213], [65, 103, 214], [157, 12, 219], [79, 50, 195], [46, 119, 152], [239, 117, 92], [222, 89, 146], [177, 0, 125], [65, 187, 140], [74, 90, 246], [170, 98, 211], [62, 95, 81], [143, 66, 243], [60, 80, 194], [181, 154, 104], [99, 90, 80], [148, 79, 131], [253, 14, 105], [94, 21, 192], [231, 240, 42], [188, 207, 85], [19, 93, 223], [101, 71, 150], [220, 211, 61], [249, 114, 233], [111, 224, 35], [83, 122, 247], [188, 210, 198], [170, 72, 92], [25, 217, 188], [217, 113, 23], [21, 48, 232], [216, 85, 177], [249, 254, 104], [231, 1, 182], [147, 132, 70], [138, 146, 141], [109, 107, 66], [205, 100, 15], [39, 117, 98], [197, 135, 92], [102, 232, 68], [176, 186, 99], [76, 204, 236], [0, 155, 195], [172, 120, 169], [145, 218, 158], [87, 251, 129], [11, 85, 133], [137, 51, 142], [182, 83, 87], [234, 124, 194], [129, 123, 143], [88, 178, 176], [70, 21, 82], [123, 101, 175], [231, 89, 146], [104, 204, 80], [138, 201, 236], [208, 164, 55], [150, 153, 90], [38, 209, 75], [188, 42, 2], [72, 131, 222], [215, 63, 202], [250, 19, 72], [108, 150, 99], [104, 163, 177], [230, 88, 83], [212, 98, 29], [8, 170, 189], [42, 180, 163], [20, 73, 170], [212, 4, 27], [133, 200, 79], [178, 0, 9], [175, 64, 249], [131, 246, 249], [206, 69, 181], [166, 200, 117], [197, 249, 186], [71, 193, 182], [210, 188, 85], [57, 220, 48], [64, 229, 74], [114, 37, 102], [215, 117, 112], [155, 165, 23], [73, 210, 251], [22, 203, 232], [127, 38, 69], [121, 208, 144], [146, 213, 53], [36, 12, 255], [11, 119, 16], [121, 129, 210], [183, 135, 88], [191, 210, 44], [159, 198, 5], [48, 89, 101], [190, 43, 50], [204, 192, 47], [165, 59, 224], [173, 8, 80], [190, 249, 93], [242, 96, 123], [16, 162, 145], [127, 177, 153], [77, 190, 238], [99, 151, 121], [244, 113, 154], [167, 193, 92], [28, 198, 94], [220, 159, 4], [132, 213, 153], [1, 209, 195], [223, 11, 214], [235, 252, 21], [53, 172, 122], [205, 42, 68], [48, 12, 186], [254, 128, 133], [182, 5, 198], [164, 97, 216], [196, 56, 127], [13, 145, 140], [4, 162, 31], [88, 154, 38], [129, 247, 95], [34, 70, 188], [167, 64, 25], [64, 219, 206], [68, 114, 85], [218, 240, 169], [66, 37, 254], [34, 246, 108], [250, 84, 24], [45, 82, 142], [188, 161, 24], [79, 242, 168], [182, 38, 97], [4, 11, 235], [34, 75, 88], [40, 161, 67], [253, 66, 158], [135, 252, 158], [92, 103, 206], [182, 226, 60], [38, 119, 1], [187, 154, 97], [241, 228, 125], [163, 60, 80], [144, 14, 74], [9, 12, 44], [99, 34, 154], [162, 125, 17], [120, 71, 168], [177, 80, 99], [192, 87, 126], [194, 138, 20], [198, 98, 97], [3, 42, 218], [62, 8, 99], [133, 174, 233], [135, 91, 247], [150, 76, 117], [149, 14, 49], [223, 197, 252], [151, 224, 33], [80, 60, 84], [30, 128, 0], [194, 43, 163], [13, 177, 203], [175, 227, 106], [210, 161, 60], [186, 162, 114], [234, 106, 177], [114, 147, 164], [17, 12, 53], [27, 127, 226], [21, 225, 44], [126, 92, 178], [241, 7, 45], [213, 67, 80], [219, 191, 196], [206, 231, 230], [171, 34, 117], [191, 56, 92], [127, 71, 82], [205, 22, 97], [23, 77, 153], [249, 226, 138], [68, 226, 191], [127, 181, 32], [125, 237, 82], [175, 20, 83], [75, 78, 189], [189, 111, 198], [115, 243, 236], [140, 207, 152], [91, 107, 35], [3, 198, 90], [134, 225, 158], [178, 7, 192], [215, 185, 103], [99, 8, 151], [189, 85, 228], [118, 10, 136], [249, 182, 148], [175, 18, 84], [141, 89, 90], [164, 17, 152], [92, 193, 187], [245, 120, 70], [203, 150, 1], [112, 223, 129], [192, 174, 202], [26, 29, 82], [70, 232, 251], [34, 175, 140], [190, 25, 179], [128, 39, 203], [15, 99, 189], [141, 8, 170], [246, 244, 38], [84, 25, 198], [210, 32, 229], [155, 11, 243], [74, 202, 194], [59, 190, 97], [22, 231, 74], [201, 19, 100], [151, 182, 30], [205, 149, 75], [215, 245, 190], [222, 24, 210], [9, 16, 228], [112, 8, 42], [25, 190, 228], [79, 106, 49], [51, 171, 40], [201, 82, 254], [149, 121, 80], [32, 171, 140], [218, 183, 57], [30, 121, 65], [170, 116, 91], [82, 174, 51], [191, 251, 238], [15, 154, 150], [212, 11, 96], [122, 89, 146], [44, 31, 28], [89, 78, 102], [180, 183, 200], [107, 57, 116], [201, 29, 252], [67, 94, 0], [151, 81, 179], [169, 221, 212], [203, 211, 95], [0, 37, 150], [150, 241, 111], [183, 170, 9], [162, 32, 32], [85, 130, 140], [210, 104, 43], [191, 137, 53], [26, 107, 244], [9, 37, 67], [177, 248, 27], [2, 182, 90], [29, 3, 9], [240, 247, 205], [130, 218, 89], [35, 104, 246], [40, 15, 128], [9, 45, 149], [9, 201, 194], [180, 62, 211], [75, 255, 105], [139, 206, 244], [9, 53, 144], [5, 111, 202], [231, 253, 115], [210, 44, 39], [103, 130, 144], [150, 94, 131], [111, 225, 151], [47, 188, 237], [36, 167, 34], [150, 157, 33], [3, 193, 8], [225, 184, 94], [69, 250, 75], [5, 108, 230], [190, 6, 253], [13, 251, 100], [3, 89, 108], [180, 73, 36], [240, 88, 180], [114, 7, 164], [121, 158, 196], [73, 108, 62], [159, 208, 245], [22, 40, 66], [172, 97, 3], [248, 168, 137], [24, 175, 193], [102, 15, 95], [138, 18, 180], [17, 196, 148], [84, 58, 159], [211, 115, 208], [243, 36, 95], [179, 55, 4], [27, 67, 16], [154, 147, 203], [55, 111, 63], [43, 20, 133], [10, 37, 41], [121, 11, 82], [232, 43, 7], [65, 244, 196], [211, 67, 96], [7, 135, 188], [64, 66, 40], [123, 62, 79], [155, 74, 4], [216, 15, 10], [63, 84, 124], [146, 201, 241], [197, 47, 193], [100, 28, 115], [43, 15, 229], [157, 140, 168], [36, 137, 216], [164, 201, 192], [124, 223, 143], [57, 99, 166], [21, 228, 10], [210, 33, 49], [193, 204, 42], [131, 219, 238], [121, 84, 222], [159, 151, 119], [20, 44, 67], [186, 182, 249], [225, 221, 214], [25, 231, 249], [56, 159, 100], [126, 205, 187], [112, 25, 229], [118, 78, 225], [36, 76, 17], [56, 171, 101], [234, 215, 233], [220, 61, 103], [252, 3, 158], [213, 125, 143], [17, 53, 192], [156, 130, 83], [130, 167, 255], [130, 217, 49], [190, 2, 250], [48, 28, 53], [89, 140, 169], [163, 148, 238], [214, 36, 101], [164, 21, 93], [41, 137, 160], [115, 134, 85], [78, 94, 224], [165, 128, 16], [142, 57, 218], [78, 19, 201], [115, 159, 0], [92, 118, 99], [147, 110, 152], [41, 82, 66], [106, 72, 171], [64, 69, 213], [70, 14, 235], [95, 9, 163], [105, 59, 224], [77, 244, 76], [27, 144, 165], [237, 171, 125], [51, 220, 186], [252, 100, 94], [207, 94, 26], [200, 213, 160], [29, 198, 166], [6, 155, 119], [31, 37, 215], [187, 138, 20], [170, 125, 170], [23, 208, 43], [47, 203, 48], [152, 34, 165], [146, 227, 42], [44, 144, 210], [106, 104, 65], [88, 30, 200], [195, 231, 107], [141, 49, 37], [245, 228, 166], [51, 140, 57], [207, 77, 171], [147, 221, 188], [217, 198, 233], [140, 156, 90], [112, 160, 19], [7, 134, 155], [139, 181, 253], [75, 183, 4], [125, 133, 174], [125, 78, 104], [48, 12, 191], [219, 174, 110], [88, 148, 142], [20, 163, 134], [120, 21, 15], [174, 32, 64], [34, 208, 74], [3, 236, 182], [71, 93, 80], [214, 232, 153], [170, 189, 225], [60, 195, 177], [233, 247, 35], [110, 189, 174], [117, 230, 193], [244, 165, 83], [167, 32, 245], [247, 237, 80], [44, 11, 234], [9, 35, 59], [218, 67, 72], [94, 9, 217], [123, 166, 121], [122, 107, 196], [248, 115, 63], [30, 237, 104], [251, 228, 149], [229, 161, 110], [136, 26, 153], [45, 131, 102], [99, 30, 116], [18, 239, 158], [193, 75, 123], [245, 134, 188], [230, 101, 60], [32, 169, 98], [71, 76, 118], [248, 60, 170], [242, 247, 236], [162, 16, 148], [95, 119, 255], [197, 28, 75], [73, 247, 208], [141, 225, 11], [110, 129, 65], [125, 74, 59], [206, 204, 29], [212, 140, 230], [104, 94, 232], [180, 187, 106], [213, 202, 216], [124, 17, 78], [133, 232, 32], [96, 89, 124], [161, 130, 2], [201, 66, 98], [174, 105, 216], [69, 118, 77], [149, 175, 9], [110, 86, 0], [246, 177, 170], [165, 193, 29], [190, 222, 241], [149, 26, 185], [211, 235, 236], [149, 173, 6], [79, 91, 199], [195, 81, 52], [89, 83, 132], [9, 111, 89], [196, 28, 165], [99, 0, 169], [17, 127, 39], [161, 151, 229], [63, 131, 107], [163, 231, 97], [194, 187, 14], [88, 106, 224], [70, 102, 171], [14, 242, 57], [208, 221, 245], [112, 22, 250], [213, 5, 219], [143, 7, 188], [160, 220, 151], [16, 234, 156], [242, 132, 102], [65, 67, 235], [107, 176, 253], [57, 171, 147], [49, 188, 194], [17, 250, 251], [252, 37, 17], [94, 60, 209], [216, 105, 10], [243, 14, 166], [217, 170, 2], [198, 21, 111], [90, 207, 132], [120, 37, 141], [27, 47, 159], [27, 95, 29], [59, 142, 211], [80, 112, 182], [36, 152, 127], [138, 211, 146], [207, 100, 251], [182, 68, 3], [64, 0, 124], [143, 242, 25], [113, 40, 5], [112, 153, 13], [194, 61, 97], [208, 128, 10], [12, 117, 66], [221, 26, 98], [140, 37, 41], [141, 148, 22], [26, 196, 141], [86, 183, 7], [166, 180, 251], [145, 225, 84], [21, 69, 192], [186, 177, 211], [24, 58, 0], [32, 95, 107], [11, 113, 4], [74, 183, 202], [105, 222, 19], [69, 178, 180], [33, 130, 65], [175, 56, 178], [197, 5, 179], [192, 99, 188], [184, 241, 170], [36, 121, 214], [232, 210, 219], [118, 42, 82], [89, 129, 212], [184, 49, 40], [199, 149, 35], [122, 189, 214], [92, 83, 92], [75, 52, 160], [31, 237, 96], [190, 209, 11], [172, 33, 120], [181, 236, 84], [245, 125, 122], [210, 141, 38], [175, 217, 72], [219, 7, 232], [168, 170, 116], [168, 21, 140], [32, 238, 81], [248, 96, 90], [189, 243, 131], [222, 183, 138], [238, 81, 57], [209, 235, 31], [218, 35, 22], [224, 88, 158], [118, 42, 119], [199, 162, 24], [87, 69, 85], [235, 139, 95], [145, 186, 126], [148, 51, 17], [184, 200, 15], [8, 21, 229], [24, 186, 24], [88, 129, 203], [65, 190, 52], [217, 5, 20], [63, 104, 70], [248, 138, 128], [103, 13, 38], [230, 6, 202], [221, 35, 150], [37, 242, 152], [162, 143, 143], [39, 61, 196], [149, 229, 17], [170, 253, 100], [58, 12, 140], [111, 88, 53], [184, 108, 37], [60, 132, 205], [172, 185, 60], [231, 4, 28], [100, 54, 199], [218, 38, 97], [180, 240, 153], [34, 135, 120], [46, 8, 155], [153, 48, 82], [197, 36, 198], [194, 250, 48], [16, 225, 26], [42, 131, 63], [151, 21, 190], [90, 175, 177], [214, 98, 61], [14, 145, 211], [124, 0, 222], [95, 151, 141], [27, 226, 145], [186, 70, 170], [170, 235, 148], [9, 170, 138], [151, 88, 180], [124, 155, 228], [220, 183, 102], [51, 193, 201], [61, 250, 37], [134, 242, 102], [192, 59, 214], [200, 171, 207], [95, 163, 93], [31, 14, 190], [240, 156, 19], [46, 130, 108], [137, 113, 78], [57, 124, 8], [195, 61, 99], [134, 252, 254], [180, 139, 140], [252, 247, 233], [1, 193, 216], [142, 212, 251], [111, 35, 93], [169, 192, 143], [31, 81, 112], [182, 39, 223], [215, 129, 253], [188, 190, 16], [5, 75, 48], [18, 27, 222], [41, 56, 65], [46, 229, 145], [102, 93, 207], [118, 62, 248], [251, 120, 139], [19, 97, 75], [200, 169, 188], [250, 211, 16], [56, 91, 67], [230, 120, 170], [65, 96, 221], [52, 54, 187], [64, 250, 8], [228, 17, 214], [14, 8, 19], [229, 144, 33], [35, 215, 78], [177, 54, 129], [34, 26, 58], [171, 22, 163], [211, 232, 21], [111, 85, 173], [18, 135, 34], [199, 115, 167], [240, 76, 185], [199, 77, 10], [131, 145, 169], [223, 175, 125], [93, 110, 240], [122, 107, 42], [154, 32, 111], [51, 224, 70], [120, 171, 58], [33, 35, 25], [42, 192, 87], [190, 110, 95], [32, 90, 112], [30, 228, 114], [161, 167, 15], [39, 70, 131], [200, 112, 99], [31, 246, 216], [41, 74, 211], [250, 210, 20], [0, 243, 219], [181, 95, 59], [194, 234, 71], [72, 51, 220], [231, 212, 194], [10, 195, 65], [9, 193, 209], [68, 59, 94], [172, 76, 54], [139, 9, 211], [115, 27, 171], [102, 7, 51], [149, 141, 60], [236, 8, 48], [213, 178, 111], [4, 48, 208], [139, 60, 151], [198, 233, 55], [175, 178, 228], [7, 201, 119], [179, 184, 239], [193, 255, 126], [76, 114, 252], [30, 244, 76], [133, 173, 176], [228, 87, 197], [76, 27, 236], [160, 138, 133], [243, 91, 29], [119, 248, 43], [193, 17, 217], [110, 253, 70], [82, 183, 149], [80, 41, 230], [166, 140, 66], [152, 235, 68], [124, 198, 168], [123, 167, 99], [120, 86, 228], [73, 76, 39], [30, 15, 0], [91, 225, 158], [48, 33, 99], [167, 4, 93], [192, 81, 105], [211, 198, 230], [175, 221, 198], [236, 179, 142], [76, 58, 34], [217, 123, 237], [145, 113, 23], [203, 93, 203], [109, 193, 24], [17, 187, 135], [246, 28, 115], [42, 50, 95], [104, 220, 4], [225, 38, 201], [176, 187, 154], [66, 111, 104], [14, 217, 105], [249, 42, 240], [3, 126, 148], [195, 78, 117], [109, 33, 83], [166, 32, 68], [108, 211, 113], [86, 0, 49], [45, 110, 20], [36, 120, 45], [149, 53, 189], [111, 8, 58], [230, 98, 13], [245, 189, 173], [26, 38, 231], [11, 147, 205], [73, 178, 79], [232, 228, 139], [139, 181, 120], [232, 141, 237], [78, 241, 121], [38, 99, 0], [108, 71, 178], [132, 234, 238], [187, 85, 53], [8, 72, 183], [211, 51, 78], [178, 179, 11], [216, 236, 60], [164, 95, 74], [146, 219, 212], [134, 73, 32], [51, 30, 212], [14, 112, 51], [14, 2, 204], [32, 226, 186], [142, 70, 180], [90, 41, 103], [98, 121, 40], [6, 53, 57], [1, 168, 245], [17, 180, 151], [14, 154, 3], [207, 188, 148], [72, 101, 201], [200, 156, 128], [182, 122, 241], [217, 253, 126], [123, 50, 229], [144, 202, 196], [173, 113, 183], [210, 107, 26], [155, 66, 57], [61, 174, 233], [36, 249, 3], [220, 205, 126], [40, 141, 128], [149, 129, 191], [99, 196, 9], [164, 132, 114], [165, 22, 57], [23, 59, 7], [12, 64, 95], [143, 33, 232], [194, 60, 221], [158, 57, 148], [92, 87, 216], [151, 102, 27], [133, 149, 56], [181, 96, 201], [164, 38, 245], [18, 81, 149], [229, 67, 170], [8, 95, 161], [22, 123, 130], [46, 154, 66], [144, 35, 138], [221, 80, 177], [148, 162, 116], [243, 108, 232], [91, 102, 113], [14, 5, 74], [124, 77, 78], [88, 41, 229], [131, 80, 139], [120, 245, 33], [172, 150, 64], [75, 158, 133], [236, 202, 154], [209, 169, 190], [103, 239, 0], [30, 71, 179], [93, 12, 80], [145, 83, 202], [158, 154, 131], [147, 7, 173], [125, 105, 163], [16, 98, 61], [82, 120, 223], [86, 114, 7], [225, 72, 192], [21, 255, 50], [109, 42, 29], [219, 23, 148], [14, 69, 97], [159, 35, 178], [218, 89, 20], [110, 151, 119], [9, 222, 132], [50, 253, 52], [246, 70, 195], [116, 65, 86], [105, 32, 180], [142, 49, 151], [229, 133, 154], [126, 192, 254], [226, 32, 250], [131, 204, 113], [151, 132, 24], [255, 141, 120], [145, 226, 30], [158, 64, 200], [171, 52, 175], [225, 7, 172], [228, 208, 100], [80, 33, 157], [173, 239, 63], [44, 152, 161], [149, 110, 216], [235, 10, 33], [113, 22, 50], [30, 20, 158], [90, 146, 145], [201, 108, 138], [82, 51, 204], [152, 251, 168], [234, 41, 64], [3, 1, 85], [168, 248, 65], [210, 1, 231], [95, 27, 246], [105, 93, 90], [10, 158, 198], [101, 172, 164], [201, 250, 233], [60, 240, 235], [166, 10, 48], [60, 52, 121], [40, 234, 73], [20, 29, 35], [129, 143, 80], [167, 144, 96], [103, 85, 129], [114, 77, 106], [198, 237, 17], [106, 231, 234], [155, 23, 139], [73, 250, 86], [76, 192, 131], [134, 79, 35], [182, 242, 211], [146, 161, 48], [248, 181, 3], [81, 252, 13], [36, 251, 102], [40, 68, 76], [226, 136, 154], [161, 75, 39], [133, 96, 165], [193, 40, 67], [151, 229, 198], [243, 226, 128], [203, 99, 139], [176, 6, 89], [33, 22, 164], [218, 239, 13], [128, 232, 211], [62, 243, 136], [110, 68, 215], [117, 148, 205], [35, 78, 126], [177, 218, 69], [133, 10, 91], [230, 33, 130], [184, 198, 249], [222, 155, 240], [96, 220, 22], [90, 153, 10], [133, 70, 227], [220, 171, 199], [207, 208, 30], [193, 8, 108], [150, 192, 90], [106, 79, 44], [195, 13, 183], [119, 154, 4], [119, 55, 115], [20, 107, 20], [125, 121, 1], [26, 123, 238], [50, 65, 24], [67, 33, 204], [121, 205, 251], [23, 233, 0], [34, 109, 21], [132, 113, 128], [1, 175, 217], [158, 44, 226], [149, 122, 139], [152, 90, 237], [90, 94, 112], [129, 25, 198], [235, 163, 222], [239, 38, 177], [130, 164, 241], [158, 218, 229], [129, 44, 200], [167, 98, 142], [21, 6, 134], [96, 35, 148], [209, 72, 71], [85, 119, 137], [130, 2, 238], [192, 77, 77], [226, 32, 29], [54, 148, 125], [39, 235, 102], [97, 205, 24], [213, 7, 1], [193, 124, 37], [100, 157, 225], [76, 164, 134], [37, 217, 207], [118, 236, 181], [208, 231, 41], [186, 209, 191], [136, 104, 90], [109, 31, 239], [146, 175, 91], [222, 163, 153], [183, 187, 164], [139, 191, 45], [61, 141, 97], [104, 67, 194], [30, 37, 30], [118, 13, 35], [182, 145, 60], [15, 176, 125], [70, 63, 43], [173, 91, 100], [71, 73, 53], [14, 189, 135], [144, 140, 240], [71, 37, 159], [235, 217, 101], [67, 16, 104], [121, 129, 186], [69, 102, 231], [60, 158, 237], [186, 122, 200], [229, 55, 25], [135, 141, 119], [250, 234, 132], [176, 80, 92], [167, 16, 143], [237, 24, 96], [214, 178, 127], [214, 137, 137], [113, 166, 129], [134, 251, 216], [72, 219, 197], [167, 54, 174], [98, 214, 232], [89, 110, 85], [235, 148, 106], [35, 27, 107], [178, 99, 133], [245, 241, 111], [196, 244, 195], [95, 203, 111], [70, 242, 160], [238, 69, 161], [242, 102, 39], [45, 98, 206], [91, 74, 237], [250, 22, 85], [220, 1, 0], [108, 33, 123], [221, 238, 141], [187, 39, 235], [240, 183, 164], [241, 122, 130], [110, 253, 82], [5, 227, 106], [36, 81, 21], [147, 90, 100], [209, 139, 124], [216, 181, 122], [47, 6, 27], [10, 225, 87], [51, 119, 58], [207, 178, 214], [223, 173, 240], [123, 129, 215], [139, 70, 137], [61, 104, 8], [76, 193, 97], [198, 190, 71], [170, 135, 142], [85, 233, 111], [221, 217, 181], [146, 79, 219], [0, 201, 169], [174, 1, 8], [222, 217, 28], [234, 151, 218], [217, 231, 31], [108, 157, 77], [7, 177, 75], [81, 248, 172], [146, 180, 64], [143, 120, 208], [235, 70, 181], [222, 65, 251], [251, 191, 148], [209, 200, 169], [218, 49, 132], [219, 228, 54], [125, 239, 138], [38, 139, 9], [146, 3, 140], [116, 149, 8], [128, 25, 62], [192, 117, 209], [167, 88, 177], [138, 250, 118], [163, 188, 171], [204, 246, 49], [34, 91, 126], [78, 95, 238], [203, 154, 177], [161, 75, 180], [12, 190, 222], [135, 129, 72], [208, 167, 62], [166, 51, 243], [2, 4, 187], [61, 176, 11], [217, 156, 6], [106, 134, 149], [162, 113, 107], [243, 253, 145], [124, 145, 220], [205, 199, 241], [37, 248, 12], [16, 47, 165], [168, 216, 105], [41, 130, 23], [168, 53, 229], [224, 185, 247], [91, 190, 106], [114, 47, 61], [141, 36, 228], [29, 247, 55], [234, 170, 176], [194, 37, 134], [10, 90, 16], [39, 43, 6], [245, 113, 28], [105, 0, 37], [223, 82, 124], [227, 147, 33], [153, 104, 34], [106, 154, 25], [138, 28, 157], [93, 172, 165], [52, 203, 194], [107, 178, 160], [147, 109, 88], [4, 236, 117], [191, 130, 207], [61, 47, 110], [95, 97, 117], [160, 97, 190], [56, 88, 32], [182, 233, 63], [123, 198, 104], [199, 91, 249], [116, 233, 38], [78, 164, 251], [32, 241, 133], [61, 234, 49], [12, 169, 206], [111, 42, 7], [195, 156, 239], [46, 115, 171], [216, 131, 156], [31, 191, 111], [2, 150, 134], [153, 105, 252], [69, 173, 146], [70, 84, 147], [92, 144, 220], [84, 103, 224], [141, 233, 53], [209, 64, 244], [197, 236, 105], [214, 218, 31], [171, 62, 169], [193, 226, 229], [218, 187, 25], [12, 51, 198], [242, 87, 16], [147, 108, 10], [242, 192, 104], [58, 13, 182], [67, 47, 217], [42, 15, 115], [117, 19, 205], [213, 230, 171], [153, 215, 207], [147, 42, 54], [175, 51, 240], [68, 126, 47], [58, 202, 230], [185, 63, 160], [181, 120, 46], [19, 138, 30], [80, 93, 252], [255, 87, 155], [93, 127, 209], [253, 61, 95], [82, 16, 204], [48, 212, 154], [164, 170, 0], [138, 183, 178], [69, 27, 232], [63, 103, 105], [27, 176, 116], [175, 207, 77], [127, 28, 215], [181, 111, 126], [79, 73, 94], [152, 99, 228], [9, 40, 247], [117, 232, 62], [176, 131, 73], [19, 208, 244], [216, 190, 233], [88, 99, 200], [97, 250, 40], [93, 58, 27], [128, 65, 150], [11, 8, 249], [79, 103, 125], [19, 193, 100], [126, 142, 181], [8, 219, 188], [229, 111, 97], [77, 220, 32], [228, 195, 166], [250, 32, 113], [109, 197, 86], [66, 31, 27], [151, 244, 254], [120, 170, 85], [61, 225, 109], [136, 248, 168], [21, 194, 76], [219, 207, 214], [23, 92, 230], [190, 11, 201], [235, 248, 155], [157, 7, 249], [225, 125, 226], [83, 43, 49], [166, 189, 98], [238, 70, 211], [179, 198, 230], [154, 172, 36], [135, 251, 232], [41, 229, 203], [158, 135, 216], [243, 10, 76], [191, 253, 198], [102, 123, 235], [216, 198, 132], [107, 159, 229], [8, 239, 145], [138, 221, 56], [11, 101, 17], [208, 186, 167], [194, 150, 39], [120, 221, 189], [159, 10, 246], [40, 86, 205], [55, 203, 20], [241, 91, 113], [140, 206, 159], [235, 59, 48], [42, 213, 111], [143, 108, 216], [119, 248, 223], [235, 64, 192], [215, 28, 219], [184, 32, 155], [190, 81, 252], [244, 211, 230], [139, 74, 54], [245, 29, 107], [86, 17, 247], [7, 24, 221], [109, 17, 243], [113, 3, 158], [78, 241, 36], [146, 119, 72], [112, 233, 195], [45, 140, 55], [73, 178, 18], [92, 17, 207], [170, 183, 143], [29, 136, 193], [89, 244, 149], [166, 242, 230], [25, 170, 123], [153, 8, 254], [128, 47, 5], [123, 94, 37], [9, 87, 51], [158, 174, 12], [139, 80, 194], [99, 220, 215], [30, 181, 244], [56, 180, 2], [206, 43, 206], [98, 156, 150], [203, 221, 208], [140, 31, 216], [191, 48, 4], [12, 17, 142], [26, 231, 110], [17, 107, 121], [246, 92, 35], [10, 143, 236], [165, 29, 254], [182, 169, 238], [22, 171, 212], [214, 92, 173], [181, 239, 216], [119, 251, 148], [26, 98, 101], [208, 53, 95], [135, 146, 227], [2, 162, 145], [37, 33, 118], [21, 202, 2], [244, 252, 224], [65, 9, 203], [139, 236, 15], [201, 14, 132], [21, 46, 211], [117, 99, 93], [255, 222, 11], [38, 163, 154], [203, 124, 117], [100, 151, 107], [227, 69, 245], [97, 73, 68], [116, 210, 20], [28, 197, 53], [146, 133, 109], [184, 193, 54], [123, 128, 31], [247, 224, 131], [185, 178, 162], [75, 143, 131], [54, 93, 165], [197, 19, 197], [126, 106, 74], [19, 145, 15], [1, 81, 128], [96, 188, 100], [204, 242, 178], [46, 143, 108], [189, 100, 109], [236, 81, 87], [249, 0, 221], [231, 113, 88], [87, 71, 86], [56, 69, 245], [175, 195, 98], [48, 221, 159], [179, 211, 179], [75, 72, 162], [21, 93, 213], [114, 137, 167], [81, 241, 248], [133, 96, 19], [119, 206, 223], [67, 198, 43], [62, 210, 139], [22, 240, 252], [216, 44, 72], [13, 228, 195], [128, 57, 174], [136, 196, 25], [249, 83, 76], [111, 62, 154], [135, 128, 84], [212, 115, 192], [158, 217, 13], [246, 201, 119], [95, 22, 204], [252, 98, 173], [233, 88, 30], [243, 147, 97], [255, 170, 95], [39, 249, 233], [25, 67, 129], [10, 108, 26], [127, 82, 250], [3, 238, 43], [16, 169, 63], [93, 228, 185], [225, 164, 174], [62, 91, 196], [129, 160, 80], [51, 137, 39], [158, 21, 9], [120, 118, 51], [92, 235, 140], [173, 18, 193], [96, 137, 173], [208, 201, 254], [36, 141, 119], [22, 89, 118], [73, 75, 83], [29, 133, 78], [125, 99, 104], [163, 52, 34], [80, 55, 100], [250, 221, 62], [40, 29, 28], [146, 137, 95], [89, 148, 211], [66, 102, 96], [130, 27, 28], [0, 71, 216], [58, 86, 83], [25, 202, 102], [159, 163, 121], [39, 180, 138], [48, 61, 17], [228, 167, 196], [124, 51, 33], [65, 168, 220], [164, 191, 164], [185, 16, 157], [218, 254, 211], [35, 82, 20], [190, 207, 142], [29, 21, 54], [215, 203, 72], [244, 192, 96], [96, 63, 173], [253, 192, 82], [250, 247, 127], [139, 141, 188], [76, 14, 48], [2, 135, 203], [142, 173, 99], [100, 116, 151], [241, 220, 136], [147, 229, 50], [245, 247, 4], [215, 131, 173], [79, 179, 158], [136, 168, 139], [163, 149, 124], [144, 34, 178], [41, 236, 142], [1, 129, 104], [20, 173, 45], [247, 129, 43], [223, 21, 131], [0, 46, 162], [150, 149, 70], [51, 192, 113], [56, 106, 232], [79, 165, 192], [127, 105, 246], [77, 23, 87], [87, 174, 105], [255, 178, 62], [41, 215, 38], [227, 91, 255], [72, 177, 49], [179, 19, 65], [148, 203, 170], [119, 3, 27], [230, 146, 253], [95, 113, 246], [19, 86, 72], [41, 25, 102], [114, 76, 43], [106, 11, 132], [214, 173, 247], [6, 239, 182], [114, 152, 165], [146, 111, 35], [43, 64, 210], [9, 197, 8], [143, 129, 156], [76, 203, 185], [199, 222, 168], [107, 198, 37], [131, 233, 1], [235, 126, 69], [48, 111, 132], [113, 98, 231], [137, 184, 13], [102, 183, 46], [187, 124, 151], [84, 99, 184], [60, 130, 155], [80, 243, 157], [37, 11, 165], [128, 163, 230], [86, 151, 142], [241, 138, 174], [236, 47, 35], [188, 173, 137], [47, 212, 82], [251, 78, 135], [143, 176, 81], [35, 164, 26], [158, 106, 151], [155, 239, 50], [13, 97, 162], [71, 218, 190], [106, 32, 189], [119, 81, 210], [51, 88, 249], [191, 182, 190], [75, 59, 81], [80, 147, 3], [190, 136, 216], [66, 211, 231], [119, 199, 184], [61, 241, 65], [104, 133, 126], [236, 76, 2], [181, 251, 15], [68, 146, 157], [41, 89, 7], [183, 234, 172], [209, 176, 64], [123, 70, 130], [54, 34, 159], [52, 202, 177], [159, 27, 203], [112, 123, 1], [206, 69, 12], [249, 184, 182], [209, 21, 99], [18, 99, 96], [41, 49, 220], [155, 69, 189], [216, 19, 215], [24, 108, 214], [218, 147, 65], [54, 38, 208], [112, 120, 16], [198, 8, 156], [57, 8, 102], [143, 159, 243], [230, 31, 227], [110, 249, 211], [207, 130, 98], [10, 189, 156], [144, 131, 173], [70, 116, 141], [237, 57, 219], [239, 108, 188], [205, 238, 186], [242, 143, 36], [202, 22, 137], [139, 153, 98], [98, 149, 74], [233, 127, 200], [161, 193, 3], [216, 215, 51], [67, 152, 197], [1, 71, 253], [131, 70, 238], [242, 150, 240], [96, 13, 25], [234, 15, 227], [248, 115, 242], [221, 239, 105], [117, 94, 254], [8, 139, 51], [91, 34, 39], [131, 37, 13], [241, 128, 185], [35, 234, 205], [97, 17, 226], [128, 32, 121], [224, 242, 221], [65, 215, 45], [20, 254, 48], [18, 176, 153], [189, 250, 207], [98, 190, 184], [44, 172, 157], [129, 167, 208], [189, 107, 81], [22, 207, 242], [108, 107, 41], [208, 4, 168], [16, 253, 80], [93, 235, 191], [15, 116, 81], [125, 24, 126], [144, 117, 5], [242, 203, 141], [106, 7, 227], [199, 105, 183], [72, 188, 143], [142, 110, 156], [161, 200, 195], [37, 140, 254], [170, 210, 4], [210, 144, 94], [144, 50, 17], [193, 186, 121], [213, 75, 177], [123, 173, 3], [140, 116, 245], [132, 141, 153], [201, 55, 229], [213, 24, 190], [139, 201, 37], [5, 50, 184], [177, 91, 135], [41, 199, 184], [228, 119, 240], [68, 76, 74], [143, 89, 93], [53, 168, 209], [87, 126, 251], [163, 232, 250], [33, 138, 156], [237, 127, 80], [243, 56, 115], [146, 168, 80], [30, 11, 161], [162, 102, 57], [94, 16, 16], [203, 236, 219], [110, 84, 95], [163, 42, 179], [109, 147, 27], [255, 89, 62], [150, 128, 108], [53, 47, 34], [186, 172, 244], [208, 241, 132], [149, 162, 54], [82, 186, 226], [69, 166, 19], [1, 169, 25], [122, 72, 143], [38, 55, 221], [198, 6, 222], [133, 30, 120], [161, 4, 222], [165, 151, 220], [167, 200, 133], [144, 145, 38], [45, 112, 96], [132, 226, 217], [217, 64, 32], [198, 39, 198], [39, 11, 230], [67, 50, 251], [237, 51, 199], [150, 170, 30], [52, 124, 165], [5, 218, 235], [88, 137, 162], [0, 84, 236], [222, 233, 201], [41, 183, 101], [73, 51, 144], [226, 138, 255], [161, 219, 129], [194, 204, 250], [233, 175, 106], [243, 159, 226], [0, 60, 82], [193, 178, 37], [112, 159, 244], [20, 250, 100], [130, 220, 171], [241, 226, 178], [86, 67, 185], [138, 4, 32], [130, 113, 11], [144, 175, 190], [138, 196, 187], [203, 140, 148], [229, 188, 64], [103, 125, 114], [215, 179, 219], [122, 85, 186], [6, 254, 96], [186, 84, 25], [0, 79, 210], [45, 96, 215], [150, 220, 191], [25, 59, 141], [226, 131, 27], [167, 237, 192], [167, 72, 77], [120, 113, 219], [253, 184, 82], [62, 23, 126], [215, 43, 123], [41, 93, 152], [45, 73, 99], [132, 133, 237], [106, 146, 42], [235, 121, 246], [134, 200, 89], [174, 59, 184], [224, 68, 191], [204, 38, 54], [138, 121, 63], [22, 182, 208], [127, 152, 30], [5, 206, 97], [239, 250, 179], [139, 121, 60], [106, 89, 44], [42, 107, 153], [145, 152, 63], [54, 107, 24], [194, 99, 101], [22, 238, 3], [166, 229, 77], [112, 188, 245], [8, 231, 74], [80, 133, 41], [27, 64, 190], [224, 239, 84], [183, 63, 240], [187, 62, 218], [224, 177, 220], [216, 225, 215], [250, 193, 42], [249, 152, 220], [14, 243, 97], [98, 158, 119], [224, 253, 237], [2, 233, 177], [155, 123, 206], [66, 135, 109], [215, 241, 150], [150, 251, 56], [148, 72, 203], [234, 199, 245], [192, 155, 38], [125, 152, 207], [118, 139, 182], [86, 138, 209], [75, 6, 204], [3, 146, 227], [120, 211, 170], [160, 168, 164], [108, 228, 144], [246, 191, 3], [135, 245, 116], [54, 252, 237], [188, 47, 219], [57, 3, 222], [46, 255, 172], [169, 172, 69], [50, 149, 196], [88, 4, 173], [21, 68, 162], [178, 183, 207], [72, 242, 69], [30, 24, 85], [42, 138, 211], [249, 210, 124], [35, 160, 52], [137, 154, 158], [59, 155, 19], [249, 246, 45], [218, 194, 155], [95, 33, 194], [177, 22, 172], [124, 95, 89], [213, 195, 231], [82, 17, 228], [83, 81, 28], [157, 215, 187], [161, 109, 232], [90, 25, 86], [115, 108, 37], [160, 189, 203], [109, 242, 85], [67, 80, 7], [163, 129, 45], [26, 127, 145], [208, 168, 56], [120, 54, 227], [85, 141, 51], [60, 143, 25], [183, 143, 75], [132, 89, 171], [250, 52, 107], [238, 167, 203], [53, 185, 156], [47, 223, 53], [180, 138, 172], [41, 85, 113], [149, 84, 104], [174, 68, 37], [189, 111, 11], [50, 123, 5], [157, 90, 170], [165, 187, 255], [170, 230, 0], [121, 147, 183], [119, 99, 129], [92, 72, 135], [220, 248, 48], [165, 162, 52], [248, 151, 155], [101, 193, 21], [74, 26, 234], [59, 243, 100], [190, 28, 41], [82, 74, 231], [170, 163, 14], [148, 57, 117], [68, 75, 92], [117, 3, 232], [54, 238, 28], [170, 209, 125], [167, 6, 226], [189, 36, 14], [100, 163, 190], [110, 255, 230], [150, 196, 180], [23, 112, 93], [104, 248, 174], [48, 143, 170], [204, 208, 84], [22, 148, 148], [248, 156, 248], [201, 78, 141], [110, 77, 45], [180, 161, 195], [0, 176, 233], [55, 181, 15], [214, 248, 119], [101, 230, 51], [244, 201, 39], [143, 110, 117], [89, 61, 201], [146, 9, 249], [227, 214, 47], [205, 251, 174], [15, 124, 137], [153, 243, 59], [186, 77, 147], [91, 95, 241], [167, 30, 159], [234, 26, 189], [235, 158, 112], [53, 201, 213], [150, 130, 149], [3, 51, 58], [202, 253, 65], [165, 79, 210], [171, 34, 228], [203, 26, 174], [189, 12, 2], [202, 56, 177], [121, 15, 116], [97, 44, 232], [144, 222, 92], [23, 186, 48], [247, 15, 79], [200, 78, 165], [159, 23, 153], [176, 27, 238], [193, 58, 192], [22, 94, 84], [37, 33, 25], [14, 71, 14], [12, 39, 40], [38, 44, 82], [39, 60, 187], [130, 229, 193], [241, 77, 164], [164, 21, 98], [138, 121, 255], [120, 80, 166], [242, 249, 133], [241, 82, 65], [239, 152, 250], [158, 29, 84], [175, 34, 197], [67, 62, 132], [128, 226, 196], [8, 132, 12], [145, 136, 61], [68, 240, 255], [203, 10, 98], [237, 80, 209], [162, 200, 52], [122, 161, 232], [146, 214, 160], [154, 111, 34], [204, 86, 240], [125, 54, 67], [220, 4, 149], [84, 60, 8], [96, 148, 211], [198, 90, 249], [9, 125, 248], [32, 78, 221], [235, 226, 65], [103, 26, 94], [131, 87, 218], [194, 48, 168], [115, 66, 247], [78, 201, 106], [49, 107, 69], [113, 69, 103], [79, 120, 237], [88, 173, 150], [146, 128, 243], [12, 236, 102], [165, 8, 108], [12, 19, 166], [158, 104, 188], [136, 25, 66], [57, 76, 149], [93, 196, 13], [81, 28, 174], [244, 14, 138], [226, 170, 22], [177, 44, 129], [98, 150, 31], [16, 231, 157], [49, 31, 199], [89, 91, 191], [1, 9, 199], [172, 57, 207], [137, 72, 43], [21, 144, 38], [68, 35, 221], [121, 54, 70], [32, 122, 122], [80, 20, 157], [103, 78, 43], [73, 70, 6], [251, 244, 130], [33, 111, 148], [157, 144, 45], [132, 46, 226], [91, 120, 182], [175, 46, 139], [67, 1, 93], [31, 232, 98], [15, 207, 210], [132, 66, 187], [79, 216, 190], [47, 152, 90], [126, 216, 5], [114, 155, 170], [164, 160, 132], [48, 220, 68], [197, 154, 253], [81, 7, 130], [192, 238, 97], [249, 197, 68], [86, 147, 22], [14, 37, 249], [138, 89, 147], [149, 101, 155], [21, 44, 160], [77, 184, 142], [250, 213, 160], [195, 96, 206], [230, 137, 179], [128, 212, 15], [107, 255, 16], [197, 42, 244], [47, 168, 65], [102, 166, 133], [133, 137, 232], [107, 86, 86], [40, 82, 121], [126, 241, 164], [108, 38, 2], [113, 227, 190], [35, 62, 153], [121, 250, 252], [255, 155, 195], [167, 236, 3], [21, 191, 157], [40, 172, 241], [206, 67, 142], [112, 12, 63], [162, 66, 243], [11, 54, 189], [228, 227, 45], [63, 254, 188], [198, 21, 167], [65, 28, 229], [35, 140, 45], [144, 222, 91], [187, 29, 1], [97, 146, 125], [170, 195, 161], [177, 245, 95], [23, 17, 204], [240, 93, 66], [136, 144, 131], [60, 183, 225], [247, 164, 195], [60, 58, 107], [181, 84, 175], [16, 182, 24], [46, 111, 196], [26, 46, 79], [238, 15, 216], [36, 136, 206], [212, 127, 134], [228, 52, 122], [46, 174, 232], [228, 102, 14], [75, 68, 95], [164, 192, 177], [12, 194, 201], [74, 78, 109], [190, 243, 7], [64, 113, 146], [59, 189, 126], [116, 118, 54], [132, 131, 126], [89, 194, 206], [109, 150, 146], [125, 250, 158], [18, 115, 205], [169, 35, 199], [76, 3, 218], [49, 142, 249], [233, 112, 82], [25, 206, 63], [211, 173, 81], [223, 40, 251], [232, 147, 203], [58, 65, 53], [123, 15, 174], [48, 3, 97], [181, 80, 25], [43, 166, 91], [177, 132, 139], [110, 21, 224], [178, 233, 29], [117, 161, 9], [92, 241, 44], [197, 173, 178], [129, 7, 94], [255, 61, 234], [254, 75, 69], [212, 193, 5], [202, 107, 54], [6, 36, 195], [71, 40, 150], [89, 124, 212], [198, 141, 142], [204, 240, 27], [242, 212, 4], [94, 58, 101], [22, 171, 23], [45, 73, 217], [135, 20, 146], [119, 112, 151], [208, 231, 169], [52, 253, 76], [122, 17, 31], [84, 101, 105], [101, 173, 36], [225, 118, 129], [90, 130, 150], [28, 204, 80], [254, 126, 63], [0, 144, 172], [82, 70, 93], [29, 4, 172], [144, 53, 168], [252, 253, 68], [250, 238, 14], [80, 243, 85], [192, 224, 41], [172, 184, 68], [14, 97, 127], [225, 201, 236], [16, 246, 197], [217, 13, 204], [143, 20, 177], [213, 24, 164], [171, 215, 77], [126, 229, 130], [186, 39, 182], [148, 121, 185], [213, 19, 210], [193, 218, 240], [35, 73, 192], [95, 163, 73], [207, 106, 143], [154, 229, 51], [136, 252, 54], [163, 54, 128], [212, 190, 185], [229, 227, 103], [249, 175, 36], [7, 205, 4], [156, 143, 37], [6, 186, 23], [157, 49, 152], [132, 43, 78], [208, 68, 139], [222, 146, 48], [7, 135, 111], [125, 90, 27], [94, 197, 199], [107, 29, 79], [142, 121, 22], [117, 36, 107], [179, 215, 117], [200, 63, 52], [94, 192, 112], [35, 114, 96], [7, 222, 209], [248, 253, 241], [171, 208, 211], [66, 159, 143], [108, 91, 3], [143, 209, 203], [99, 93, 220], [168, 222, 182], [251, 85, 151], [24, 21, 153], [109, 165, 106], [180, 200, 73], [224, 24, 158], [161, 68, 82], [103, 47, 100], [112, 177, 148], [71, 215, 169], [216, 162, 126], [139, 108, 34], [235, 197, 239], [6, 112, 10], [174, 213, 142], [63, 10, 16], [99, 127, 179], [110, 209, 2], [190, 91, 72], [36, 241, 44], [22, 92, 19], [10, 0, 23], [194, 186, 22], [155, 24, 21], [80, 146, 1], [239, 187, 145], [22, 172, 240], [24, 247, 185], [132, 185, 213], [158, 189, 29], [133, 212, 134], [40, 90, 223], [123, 98, 131], [215, 82, 209], [76, 190, 97], [43, 52, 46], [142, 166, 224], [2, 211, 34], [192, 50, 23], [176, 163, 137], [136, 109, 226], [92, 86, 109], [130, 30, 65], [130, 90, 31], [249, 117, 170], [27, 123, 148], [114, 12, 206], [168, 16, 166], [242, 116, 78], [29, 66, 74], [49, 230, 180], [48, 186, 48], [182, 42, 26], [28, 223, 48], [88, 50, 18], [180, 59, 3], [220, 121, 15], [15, 4, 182], [96, 142, 112], [235, 178, 101], [171, 66, 166], [225, 117, 59], [160, 71, 114], [66, 92, 238], [44, 50, 61], [223, 69, 202], [181, 67, 84], [180, 155, 208], [173, 194, 228], [249, 44, 106], [5, 10, 133], [108, 210, 77], [128, 48, 40], [206, 177, 159], [179, 39, 86], [186, 7, 216], [39, 142, 139], [158, 219, 124], [136, 147, 3], [124, 19, 17], [184, 220, 183], [229, 153, 125], [157, 76, 243], [118, 27, 178], [211, 207, 155], [48, 20, 44], [174, 140, 130], [124, 11, 79], [182, 232, 27], [24, 174, 244], [56, 8, 152], [134, 35, 159], [210, 194, 183], [135, 67, 91], [4, 136, 43], [17, 173, 218], [70, 39, 197], [113, 59, 246], [7, 20, 219], [42, 67, 158], [111, 114, 2], [100, 239, 99], [224, 135, 21], [2, 6, 0], [192, 77, 124], [242, 244, 225], [178, 183, 136], [189, 119, 145], [237, 244, 63], [0, 160, 13], [29, 29, 215], [43, 41, 1], [64, 63, 80], [64, 54, 77], [143, 163, 91], [1, 121, 96], [222, 29, 106], [62, 35, 48], [22, 166, 123], [133, 106, 224], [234, 200, 111], [76, 106, 182], [217, 130, 33], [95, 2, 246], [108, 36, 206], [83, 187, 62], [249, 18, 147], [9, 9, 70], [174, 147, 101], [254, 39, 13], [239, 210, 255], [40, 48, 226], [145, 48, 9], [159, 6, 19], [154, 250, 107], [227, 194, 181], [216, 171, 159], [181, 62, 43], [42, 54, 165], [156, 137, 245], [241, 154, 116], [237, 128, 109], [39, 216, 242], [58, 93, 170], [244, 84, 37], [61, 203, 65], [39, 165, 59], [160, 198, 180], [191, 27, 177], [204, 10, 132], [124, 226, 74], [132, 213, 123], [143, 24, 145], [17, 190, 105], [38, 136, 143], [100, 200, 138], [204, 60, 243], [114, 110, 69], [211, 86, 125], [228, 27, 81], [89, 12, 10], [208, 150, 32], [37, 151, 111], [145, 3, 165], [63, 59, 190], [123, 24, 39], [229, 78, 144], [11, 191, 166], [243, 140, 49], [175, 106, 189], [84, 42, 178], [253, 237, 68], [14, 30, 103], [201, 176, 216], [116, 128, 66], [112, 165, 201], [116, 41, 110], [249, 91, 186], [20, 16, 90], [217, 84, 242], [4, 171, 165], [85, 115, 253], [220, 10, 83], [250, 96, 27], [51, 52, 104], [152, 193, 96], [164, 156, 246], [92, 39, 89], [86, 215, 201], [219, 151, 245], [119, 123, 225], [1, 195, 80], [49, 245, 21], [67, 177, 137], [180, 178, 164], [36, 242, 178], [157, 162, 119], [25, 186, 106], [45, 238, 175], [230, 1, 150], [253, 208, 163], [136, 139, 212], [243, 35, 144], [143, 106, 210], [128, 204, 71], [206, 222, 242], [104, 126, 12], [80, 5, 62], [85, 28, 83], [224, 183, 153], [124, 184, 220], [202, 113, 55], [147, 92, 137], [124, 172, 222], [169, 122, 57], [46, 110, 36], [212, 0, 185], [62, 76, 73], [31, 110, 243], [252, 22, 157], [159, 22, 18], [227, 50, 106], [121, 93, 80], [123, 80, 254], [123, 39, 15], [194, 108, 190], [204, 99, 120], [67, 223, 142], [83, 139, 159], [197, 215, 91], [49, 76, 12], [210, 166, 4], [13, 204, 98], [87, 215, 118], [53, 223, 158], [171, 134, 46], [76, 53, 37], [173, 227, 113], [195, 99, 179], [124, 196, 207], [180, 86, 168], [28, 52, 42], [248, 253, 169], [177, 15, 147], [39, 154, 93], [119, 83, 224], [117, 90, 159], [56, 101, 127], [164, 144, 217], [135, 148, 81], [154, 18, 251], [111, 32, 102], [217, 227, 73], [104, 91, 60], [128, 120, 82], [172, 234, 29], [251, 13, 111], [175, 152, 16], [31, 48, 112], [6, 76, 108], [45, 92, 16], [81, 140, 153], [192, 6, 122], [84, 44, 99], [84, 169, 163], [158, 82, 44], [233, 214, 204], [52, 88, 137], [220, 169, 17], [251, 108, 6], [174, 27, 251], [164, 132, 82], [72, 88, 92], [3, 55, 212], [60, 135, 248], [118, 88, 157], [127, 146, 5], [173, 133, 192], [253, 129, 83], [235, 195, 47], [191, 165, 120], [1, 168, 115], [237, 36, 174], [187, 171, 249], [248, 16, 85], [53, 63, 174], [126, 86, 24], [25, 171, 31], [83, 209, 236], [248, 213, 96], [173, 153, 198], [239, 202, 118], [165, 83, 135], [180, 193, 76], [209, 108, 234], [14, 44, 7], [6, 242, 133], [178, 29, 189], [82, 20, 10], [91, 19, 86], [190, 191, 64], [247, 246, 174], [62, 77, 132], [177, 98, 129], [92, 200, 37], [138, 16, 228], [21, 154, 88], [233, 26, 63], [50, 221, 70], [126, 45, 186], [19, 15, 89], [181, 248, 179], [29, 57, 140], [81, 217, 132], [54, 64, 227], [78, 248, 6], [73, 1, 99], [172, 221, 205], [28, 107, 156], [77, 45, 67], [212, 185, 206], [166, 98, 224], [102, 93, 189], [152, 55, 26], [255, 94, 82], [144, 78, 65], [45, 0, 77], [3, 16, 119], [159, 14, 106], [39, 97, 99], [8, 153, 185], [107, 90, 31], [171, 207, 82], [194, 56, 41], [171, 59, 90], [206, 182, 244], [158, 41, 101], [154, 52, 151], [206, 218, 130], [136, 164, 24], [132, 187, 136], [219, 67, 61], [31, 92, 239], [52, 165, 137], [125, 254, 66], [67, 65, 167], [109, 209, 127], [65, 104, 25], [15, 235, 47], [98, 244, 64], [13, 94, 143], [95, 77, 4], [169, 174, 216], [171, 201, 92], [39, 242, 104], [126, 84, 185], [34, 14, 81], [244, 81, 193], [197, 254, 125], [41, 206, 152], [27, 226, 109], [157, 155, 87], [27, 175, 84], [160, 205, 37], [22, 252, 37], [51, 76, 105], [88, 206, 180], [201, 190, 233], [115, 72, 158], [20, 133, 139], [212, 31, 126], [230, 29, 128], [157, 75, 49], [22, 48, 213], [145, 224, 4], [168, 163, 165], [73, 150, 171], [75, 46, 9], [228, 12, 77], [121, 9, 18], [148, 108, 27], [17, 84, 237], [203, 182, 79], [201, 139, 242], [83, 173, 4], [35, 199, 77], [90, 161, 52], [158, 123, 147], [228, 117, 9], [191, 247, 42], [187, 153, 248], [143, 54, 192], [48, 103, 95], [80, 94, 153], [249, 116, 112], [206, 131, 238], [69, 230, 158], [9, 50, 160], [230, 72, 30], [205, 76, 219], [132, 32, 144], [220, 119, 62], [133, 60, 164], [116, 207, 60], [73, 19, 16], [113, 74, 92], [191, 30, 141], [1, 232, 111], [238, 113, 113], [90, 95, 140], [148, 197, 168], [161, 202, 148], [223, 165, 5], [251, 204, 93], [240, 134, 203], [11, 223, 185], [246, 35, 89], [3, 6, 67], [73, 166, 141], [204, 55, 15], [28, 153, 5], [22, 226, 31], [131, 106, 102], [38, 178, 44], [223, 121, 249], [18, 33, 214], [216, 107, 109], [165, 232, 61], [10, 103, 211], [236, 109, 133], [0, 106, 202], [246, 169, 83], [110, 196, 218], [20, 59, 83], [162, 98, 130], [105, 89, 128], [161, 74, 4], [11, 75, 116], [5, 179, 146], [42, 111, 130], [137, 125, 171], [114, 63, 19], [70, 77, 101], [214, 54, 109], [141, 62, 206], [102, 204, 205], [238, 207, 235], [244, 33, 158], [141, 26, 58], [69, 146, 242], [184, 204, 233], [53, 107, 169], [95, 254, 10], [187, 137, 90], [243, 249, 69], [118, 244, 94], [8, 199, 105], [89, 66, 120], [29, 44, 39], [80, 176, 130], [156, 50, 38], [142, 154, 13], [73, 83, 167], [17, 150, 16], [112, 119, 69], [94, 160, 53], [47, 112, 192], [76, 10, 163], [18, 141, 22], [183, 57, 175], [10, 128, 177], [62, 201, 162], [255, 24, 247], [28, 132, 105], [196, 189, 185], [170, 90, 53], [65, 194, 188], [192, 181, 183], [111, 152, 93], [120, 217, 169], [144, 100, 220], [63, 29, 3], [55, 57, 100], [74, 69, 232], [113, 217, 130], [192, 64, 172], [146, 37, 83], [5, 217, 131], [126, 146, 24], [164, 123, 155], [99, 208, 178], [218, 143, 87], [180, 195, 105], [221, 34, 194], [7, 192, 73], [117, 169, 254], [233, 243, 199], [161, 172, 22], [147, 100, 171], [154, 200, 253], [145, 112, 247], [53, 146, 180], [179, 237, 235], [9, 230, 175], [141, 170, 185], [65, 5, 76], [138, 195, 167], [146, 97, 134], [195, 36, 227], [3, 128, 103], [26, 218, 119], [235, 52, 226], [8, 223, 190], [165, 189, 165], [14, 179, 93], [228, 209, 232], [249, 76, 50], [97, 234, 81], [9, 128, 222], [221, 180, 70], [227, 154, 221], [137, 18, 195], [61, 205, 60], [234, 99, 6], [107, 117, 24], [239, 81, 229], [157, 61, 73], [13, 4, 78], [1, 131, 94], [97, 164, 7], [36, 141, 186], [195, 195, 184], [230, 143, 244], [213, 75, 54], [230, 204, 233], [198, 222, 59], [147, 41, 228], [155, 46, 150], [208, 8, 171], [254, 106, 198], [154, 129, 97], [118, 67, 22], [83, 27, 179], [39, 27, 96], [174, 150, 11], [253, 159, 145], [186, 138, 63], [157, 91, 92], [165, 29, 165], [44, 123, 136], [109, 225, 160], [152, 33, 73], [41, 240, 22], [192, 43, 232], [115, 239, 253], [223, 77, 117], [51, 255, 76], [251, 221, 65], [51, 97, 254], [230, 237, 79], [209, 203, 43], [154, 57, 1], [52, 218, 248], [180, 58, 167], [235, 129, 131], [81, 116, 1], [92, 112, 84], [223, 64, 26], [138, 177, 142], [125, 213, 156], [212, 10, 150], [80, 160, 60], [64, 116, 29], [54, 254, 148], [168, 220, 168], [9, 49, 221], [92, 220, 4], [171, 70, 41], [154, 210, 255], [180, 16, 153], [85, 122, 166], [62, 165, 215], [63, 41, 14], [32, 110, 146], [23, 103, 82], [95, 81, 33], [36, 23, 200], [63, 23, 221], [204, 98, 212], [240, 165, 115], [118, 220, 235], [115, 208, 180], [132, 203, 220], [6, 227, 90], [51, 132, 222], [210, 182, 156], [60, 53, 61], [206, 209, 189], [120, 105, 96], [87, 118, 87], [212, 97, 72], [133, 183, 175], [179, 40, 121], [76, 43, 180], [150, 101, 248], [21, 178, 111], [168, 146, 220], [113, 196, 158], [237, 94, 204], [234, 241, 165], [221, 62, 222], [53, 111, 86], [61, 85, 74], [195, 142, 200], [54, 35, 59], [33, 62, 37], [142, 34, 181], [199, 99, 124], [83, 124, 104], [244, 245, 49], [45, 229, 253], [199, 106, 206], [31, 152, 201], [22, 236, 44], [115, 214, 234], [23, 187, 108], [137, 205, 200], [70, 4, 2], [51, 51, 211], [248, 113, 202], [230, 123, 49], [33, 152, 193], [114, 95, 187], [243, 174, 209], [190, 180, 244], [234, 206, 135], [124, 117, 169], [84, 90, 53], [69, 235, 126], [251, 142, 74], [61, 3, 50], [193, 161, 35], [199, 189, 98], [226, 77, 249], [194, 215, 207], [69, 251, 66], [77, 127, 63], [243, 63, 1], [31, 178, 69], [111, 238, 29], [199, 72, 147], [143, 218, 213], [214, 250, 65], [174, 78, 27], [36, 216, 230], [95, 165, 102], [220, 125, 230], [253, 42, 178], [106, 135, 132], [3, 92, 254], [140, 27, 87], [71, 79, 85], [179, 219, 5], [241, 222, 127], [121, 190, 215], [204, 0, 33], [208, 210, 124], [224, 181, 31], [145, 122, 41], [136, 225, 31], [100, 93, 111], [128, 153, 9], [250, 28, 19], [212, 232, 39], [236, 175, 199], [97, 125, 236], [31, 178, 42], [9, 245, 71], [216, 252, 32], [106, 82, 79], [171, 133, 150], [94, 39, 106], [182, 184, 92], [184, 58, 2], [231, 72, 235], [108, 63, 28], [219, 225, 162], [180, 59, 157], [64, 170, 80], [141, 60, 152], [126, 166, 129], [217, 217, 122], [49, 192, 136], [56, 180, 233], [47, 135, 196], [48, 165, 117], [217, 71, 244], [206, 60, 156], [7, 104, 5], [73, 182, 206], [139, 248, 68], [13, 2, 169], [1, 115, 119], [218, 68, 33], [12, 73, 4], [111, 210, 207], [135, 234, 27], [255, 48, 164], [84, 192, 234], [15, 158, 104], [228, 89, 71], [197, 227, 80], [103, 192, 65], [49, 129, 39], [236, 159, 98], [211, 183, 142], [26, 186, 118], [120, 131, 37], [111, 193, 186], [30, 97, 87], [223, 109, 22], [188, 160, 18], [49, 93, 231], [44, 132, 30], [161, 189, 36], [183, 90, 205], [160, 178, 11], [215, 45, 246], [158, 183, 183], [71, 167, 126], [246, 2, 204], [106, 33, 9], [20, 204, 6], [183, 143, 85], [149, 16, 251], [65, 90, 21], [37, 46, 173], [176, 138, 112], [212, 139, 49], [9, 126, 236], [103, 74, 86], [54, 38, 150], [142, 15, 88], [95, 188, 133], [254, 176, 82], [38, 224, 98], [26, 42, 216], [195, 157, 119], [95, 149, 145], [30, 33, 6], [22, 152, 240], [61, 230, 171], [198, 138, 248], [70, 69, 93], [83, 217, 217], [46, 247, 140], [59, 207, 246], [70, 74, 184], [4, 27, 28], [122, 218, 104], [135, 95, 251], [17, 216, 145], [96, 94, 99], [99, 17, 56], [94, 136, 98], [67, 80, 130], [127, 232, 221], [243, 59, 87], [167, 126, 203], [174, 130, 98], [134, 162, 211], [196, 198, 149], [245, 108, 171], [219, 128, 3], [125, 120, 245], [138, 167, 174], [182, 184, 156], [37, 78, 5], [49, 141, 39], [219, 169, 172], [229, 59, 114], [194, 95, 252], [162, 229, 232], [186, 53, 216], [253, 162, 209], [176, 103, 163], [58, 46, 86], [70, 183, 10], [205, 127, 51], [163, 155, 89], [76, 194, 188], [223, 49, 218], [206, 121, 105], [118, 150, 38], [67, 242, 159], [100, 26, 228], [60, 228, 175], [209, 235, 2], [21, 52, 152], [54, 115, 252], [138, 50, 132], [113, 183, 126], [96, 10, 84], [36, 201, 227], [191, 90, 105], [52, 232, 21], [24, 177, 200], [155, 253, 237], [175, 84, 146], [67, 110, 78], [112, 102, 194], [180, 203, 6], [45, 198, 28], [245, 255, 245], [161, 154, 135], [248, 200, 172], [167, 179, 118], [142, 231, 239], [8, 135, 120], [230, 75, 255], [159, 227, 166], [226, 104, 251], [217, 109, 206], [174, 251, 233], [21, 152, 129], [79, 181, 233], [162, 80, 170], [232, 200, 33], [224, 98, 193], [160, 96, 185], [149, 34, 189], [21, 162, 246], [209, 68, 180], [247, 58, 183], [205, 3, 154], [49, 170, 66], [22, 64, 196], [222, 154, 112], [14, 234, 119], [78, 238, 57], [130, 3, 251], [116, 208, 25], [211, 120, 68], [110, 49, 35], [222, 153, 60], [3, 86, 194], [125, 69, 108], [239, 123, 32], [168, 4, 30], [0, 116, 71], [1, 120, 97], [252, 219, 54], [255, 53, 75], [110, 174, 87], [189, 201, 9], [202, 142, 28], [56, 151, 233], [132, 201, 225], [203, 181, 192], [227, 3, 152], [39, 176, 13], [224, 228, 205], [57, 209, 131], [180, 182, 156], [210, 95, 206], [189, 219, 37], [134, 90, 93], [94, 93, 237], [29, 94, 103], [32, 52, 10], [219, 146, 95], [209, 88, 97], [25, 90, 170], [159, 98, 107], [71, 59, 9], [169, 131, 64], [46, 140, 208], [90, 114, 39], [117, 13, 209], [34, 146, 155], [232, 231, 188], [20, 242, 246], [246, 194, 253], [5, 86, 177], [246, 53, 190], [1, 244, 191], [217, 249, 178], [231, 64, 249], [95, 136, 84], [162, 177, 101], [225, 115, 237], [101, 47, 44], [170, 237, 188], [4, 115, 249], [132, 110, 132], [213, 71, 67], [6, 114, 156], [237, 29, 188], [78, 206, 42], [41, 101, 39], [28, 111, 79], [22, 158, 9], [114, 134, 59], [146, 89, 102], [131, 110, 35], [110, 52, 182], [66, 60, 221], [4, 76, 81], [240, 172, 114], [159, 138, 210], [92, 102, 11], [224, 99, 233], [109, 192, 153], [131, 221, 90], [143, 207, 253], [253, 155, 177], [170, 99, 240], [12, 218, 63], [79, 46, 243], [166, 213, 227], [207, 101, 42], [198, 227, 134], [45, 189, 105], [118, 186, 82], [110, 9, 28], [41, 175, 132], [40, 185, 150], [77, 182, 70], [38, 22, 91], [153, 219, 249], [148, 102, 80], [230, 4, 181], [108, 110, 29], [239, 55, 86], [105, 229, 42], [107, 102, 1], [87, 187, 25], [181, 126, 40], [206, 206, 11], [206, 116, 65], [113, 146, 20], [95, 164, 20], [21, 40, 206], [233, 253, 46], [91, 188, 8], [148, 110, 201], [145, 164, 27], [137, 42, 77], [212, 96, 70], [89, 8, 111], [78, 111, 166], [87, 71, 97], [132, 104, 19], [67, 152, 215], [191, 62, 69], [124, 217, 130], [160, 224, 37], [90, 204, 47], [74, 160, 66], [99, 90, 255], [34, 81, 248], [171, 97, 2], [254, 215, 123], [253, 60, 51], [49, 222, 19], [12, 22, 238], [245, 129, 119], [92, 165, 38], [12, 1, 0], [112, 232, 156], [107, 231, 248], [85, 207, 66], [189, 218, 37], [93, 253, 210], [213, 106, 16], [13, 137, 47], [240, 162, 162], [135, 217, 236], [213, 188, 180], [210, 28, 247], [136, 11, 90], [98, 75, 209], [22, 26, 39], [92, 97, 119], [69, 152, 216], [120, 100, 86], [90, 199, 173], [187, 168, 87], [248, 15, 160], [21, 16, 215], [230, 142, 189], [173, 210, 242], [119, 168, 119], [84, 161, 220], [17, 246, 228], [71, 20, 206], [25, 82, 99], [160, 184, 130], [4, 103, 243], [35, 104, 73], [106, 110, 2], [150, 190, 193], [221, 65, 169], [116, 200, 173], [70, 106, 80], [45, 99, 206], [10, 239, 31], [142, 69, 146], [42, 85, 252], [131, 78, 143], [173, 61, 99], [143, 234, 150], [202, 72, 224], [67, 65, 44], [120, 162, 191], [170, 113, 72], [38, 134, 121], [81, 217, 32], [75, 73, 109], [78, 166, 27], [66, 27, 128], [201, 200, 164], [131, 76, 235], [115, 171, 102], [137, 7, 29], [152, 188, 176], [121, 201, 236], [245, 115, 7], [183, 104, 175], [79, 254, 13], [61, 43, 253], [177, 84, 90], [88, 141, 37], [210, 72, 232], [232, 145, 134], [188, 224, 210], [213, 85, 235], [232, 144, 222], [195, 253, 127], [215, 241, 250], [167, 129, 127], [82, 144, 217], [204, 154, 172], [204, 8, 213], [179, 90, 117], [95, 237, 235], [158, 192, 246], [196, 140, 26], [234, 161, 194], [4, 156, 25], [187, 135, 169], [224, 66, 173], [103, 170, 87], [149, 221, 138], [241, 80, 252], [112, 177, 148], [154, 53, 172], [146, 86, 13], [119, 218, 214], [67, 229, 68], [25, 17, 229], [63, 82, 14], [178, 77, 206], [187, 41, 99], [69, 185, 144], [250, 89, 238], [117, 204, 198], [65, 26, 184], [32, 139, 167], [175, 185, 140], [57, 176, 181], [196, 242, 37], [3, 68, 153], [188, 25, 180], [162, 146, 130], [55, 236, 171], [237, 179, 169], [198, 128, 134], [60, 112, 7], [233, 3, 53], [138, 84, 18], [162, 242, 215], [69, 208, 138], [97, 4, 194], [2, 253, 92], [170, 153, 86], [172, 130, 76], [246, 105, 9], [181, 200, 78], [133, 191, 231], [109, 127, 227], [140, 106, 20], [223, 1, 15], [27, 123, 153], [232, 236, 142], [196, 45, 150], [62, 93, 105], [226, 223, 110], [188, 172, 248], [184, 255, 4], [130, 98, 197], [151, 239, 67], [52, 46, 71], [176, 75, 60], [95, 9, 3], [140, 82, 42], [90, 209, 54], [84, 113, 240], [216, 196, 196], [150, 190, 204], [86, 40, 92], [77, 243, 255], [126, 13, 32], [91, 184, 42], [173, 228, 10], [227, 219, 75], [90, 126, 194], [194, 251, 70], [32, 175, 88], [105, 49, 109], [246, 208, 174], [142, 94, 251], [97, 125, 67], [24, 105, 58], [108, 43, 153], [136, 49, 78], [208, 125, 152], [121, 12, 65], [69, 82, 31], [245, 191, 58], [148, 94, 110], [230, 199, 1], [219, 130, 112], [14, 14, 156], [98, 150, 146], [240, 82, 215], [33, 74, 61], [89, 118, 163], [14, 171, 225], [4, 59, 206], [209, 183, 208], [72, 242, 171], [229, 231, 20], [161, 108, 37], [180, 71, 204], [58, 220, 248], [31, 195, 36], [1, 197, 60], [139, 10, 69], [20, 129, 99], [128, 201, 29], [0, 244, 43], [237, 143, 184], [96, 222, 1], [37, 84, 43], [101, 108, 97], [172, 99, 42], [135, 38, 169], [217, 82, 209], [159, 140, 193], [44, 201, 228], [19, 8, 230], [89, 8, 40], [0, 87, 20], [178, 225, 219], [49, 84, 239], [156, 236, 140], [12, 151, 50], [93, 56, 249], [239, 160, 178], [197, 231, 107], [16, 29, 33], [95, 1, 85], [50, 115, 72], [205, 98, 87], [210, 145, 226], [154, 37, 237], [205, 49, 6], [129, 68, 209], [206, 132, 70], [95, 225, 55], [55, 213, 148], [133, 94, 216], [222, 186, 184], [177, 169, 25], [58, 220, 117], [154, 110, 61], [228, 146, 206], [249, 255, 74], [60, 202, 41], [38, 199, 227], [48, 125, 103], [64, 212, 12], [157, 122, 213], [196, 73, 59], [75, 232, 187], [104, 38, 192], [9, 54, 73], [239, 81, 76], [219, 100, 235], [228, 88, 169], [179, 141, 150], [233, 153, 29], [151, 172, 249], [110, 223, 128], [216, 172, 46], [68, 13, 33], [81, 241, 121], [158, 138, 106], [0, 24, 236], [72, 73, 120], [57, 95, 253], [101, 225, 245], [92, 1, 233], [216, 65, 77], [209, 7, 58], [141, 61, 4], [82, 240, 250], [43, 96, 95], [93, 193, 99], [236, 36, 182], [79, 225, 18], [14, 176, 222], [216, 112, 103], [237, 89, 145], [168, 183, 39], [238, 54, 156], [229, 25, 124], [98, 96, 213], [26, 43, 115], [213, 171, 234], [200, 62, 128], [105, 44, 119], [11, 149, 181], [69, 128, 77], [225, 245, 144], [232, 253, 255], [151, 236, 219], [15, 130, 83], [194, 218, 24], [185, 47, 0], [115, 58, 54], [127, 19, 29], [251, 230, 180], [79, 50, 80], [70, 109, 150], [124, 99, 237], [196, 152, 91], [27, 84, 50], [204, 187, 249], [199, 201, 132], [43, 255, 12], [130, 209, 115], [136, 236, 214], [53, 184, 205], [214, 63, 248], [52, 132, 210], [192, 38, 24], [42, 58, 150], [243, 84, 147], [67, 140, 170], [116, 135, 97], [192, 34, 173], [50, 9, 91], [90, 206, 239], [30, 156, 83], [231, 202, 82], [14, 245, 231], [8, 137, 154], [223, 13, 28], [199, 21, 8], [96, 96, 226], [241, 101, 207], [72, 128, 89], [193, 153, 72], [113, 228, 124], [140, 149, 61], [244, 63, 150], [102, 183, 110], [159, 190, 205], [4, 98, 94], [127, 194, 206], [166, 211, 220], [193, 243, 98], [64, 115, 128], [104, 102, 146], [198, 180, 153], [92, 106, 125], [179, 182, 120], [191, 157, 177], [65, 198, 209], [175, 113, 243], [23, 64, 147], [252, 181, 103], [27, 1, 141], [76, 3, 109], [211, 205, 130], [17, 156, 31], [80, 50, 165], [150, 189, 99], [114, 135, 129], [126, 183, 157], [48, 35, 38], [131, 130, 186], [98, 20, 210], [91, 178, 82], [117, 226, 155], [209, 36, 252], [233, 218, 38], [25, 0, 90], [98, 146, 26], [90, 228, 88], [133, 49, 209], [28, 54, 141], [38, 250, 99], [48, 251, 214], [197, 237, 227], [128, 136, 170], [81, 0, 163], [70, 252, 11], [55, 244, 197], [125, 31, 192], [152, 182, 21], [109, 90, 140], [98, 74, 17], [71, 226, 95], [42, 205, 19], [243, 36, 47], [141, 161, 175], [138, 161, 44], [156, 133, 157], [90, 161, 102], [144, 171, 27], [72, 159, 187], [116, 212, 34], [102, 127, 192], [198, 186, 13], [98, 85, 131], [119, 188, 43], [169, 205, 196], [17, 38, 80], [86, 56, 34], [113, 150, 183], [170, 61, 68], [144, 183, 182], [229, 240, 76], [23, 207, 5], [163, 160, 240], [8, 244, 64], [102, 50, 129], [30, 73, 102], [235, 6, 155], [55, 93, 8], [27, 58, 242], [74, 72, 5], [33, 39, 26], [47, 21, 37], [214, 73, 20], [79, 172, 112], [25, 134, 107], [71, 41, 167], [176, 130, 18], [32, 129, 92], [93, 147, 45], [54, 87, 104], [97, 141, 65], [93, 242, 104], [63, 155, 194], [250, 141, 152], [38, 77, 198], [126, 86, 46], [159, 189, 248], [245, 247, 45], [38, 101, 87], [102, 95, 102], [5, 169, 97], [210, 8, 162], [129, 87, 65], [11, 87, 86], [73, 228, 128], [251, 211, 231], [77, 230, 40], [199, 231, 60], [215, 46, 239], [145, 119, 25], [3, 43, 146], [185, 137, 54], [91, 186, 221], [78, 149, 191], [100, 81, 50], [120, 136, 117], [205, 232, 208], [152, 9, 82], [50, 204, 232], [65, 141, 97], [180, 97, 227], [254, 201, 107], [113, 1, 92], [142, 26, 113], [95, 91, 221], [113, 48, 93], [230, 179, 48], [53, 147, 41], [81, 118, 65], [80, 130, 171], [20, 132, 178], [146, 49, 183], [128, 17, 204], [161, 156, 17], [157, 29, 6], [172, 75, 246], [253, 41, 66], [185, 206, 250], [128, 214, 129], [108, 178, 140], [134, 116, 162], [109, 224, 33], [36, 182, 248], [131, 9, 71], [250, 91, 181], [77, 226, 34], [117, 59, 96], [70, 114, 87], [226, 19, 71], [128, 73, 221], [111, 238, 65], [99, 39, 187], [131, 221, 187], [127, 228, 115], [197, 97, 131], [106, 171, 32], [63, 1, 14], [12, 18, 2], [159, 91, 193], [81, 111, 252], [189, 22, 4], [119, 135, 79], [142, 215, 230], [24, 163, 198], [147, 179, 191], [154, 196, 114], [136, 235, 118], [2, 162, 146], [185, 160, 65], [211, 84, 68], [121, 174, 208], [137, 177, 38], [208, 30, 161], [7, 254, 103], [91, 154, 40], [70, 83, 150], [93, 70, 12], [169, 253, 206], [54, 241, 112], [52, 80, 118], [223, 212, 39], [83, 122, 7], [194, 252, 103], [113, 153, 156], [81, 35, 19], [130, 23, 12], [72, 5, 30], [80, 7, 43], [72, 19, 238], [94, 235, 62], [77, 114, 3], [61, 186, 237], [33, 223, 96], [145, 32, 17], [150, 181, 64], [37, 61, 248], [100, 170, 250], [139, 25, 174], [48, 44, 33], [149, 188, 83], [197, 34, 69], [85, 92, 242], [123, 196, 217], [100, 125, 102], [36, 32, 68], [176, 95, 231], [31, 1, 140], [69, 177, 204], [108, 150, 38], [234, 114, 77], [205, 102, 176], [51, 212, 250], [144, 245, 34], [108, 60, 198], [120, 136, 20], [159, 161, 119], [241, 194, 41], [241, 163, 157], [49, 36, 127], [51, 106, 219], [72, 139, 162], [166, 249, 28], [62, 200, 154], [77, 239, 79], [115, 144, 162], [75, 82, 191], [167, 29, 155], [82, 24, 61], [224, 121, 155], [4, 251, 207], [99, 230, 130], [44, 164, 85], [139, 85, 2], [139, 254, 72], [37, 56, 237], [31, 143, 140], [40, 85, 161], [11, 230, 59], [162, 17, 0], [14, 94, 100], [32, 54, 227], [215, 72, 106], [25, 43, 26], [42, 76, 175], [111, 218, 33], [231, 179, 63], [15, 140, 109], [153, 109, 182], [4, 103, 24], [188, 39, 53], [199, 115, 4], [187, 153, 0], [217, 51, 48], [233, 235, 155], [61, 59, 134], [107, 123, 49], [217, 205, 130], [163, 184, 154], [34, 97, 49], [191, 247, 27], [126, 224, 156], [206, 31, 130], [10, 251, 177], [237, 66, 118], [43, 132, 221], [151, 45, 87], [162, 170, 49], [22, 202, 33], [198, 255, 229], [21, 242, 92], [162, 195, 69], [223, 115, 1], [93, 31, 65], [108, 172, 179], [75, 71, 204], [11, 126, 182], [38, 20, 159], [234, 92, 50], [235, 85, 217], [104, 14, 199], [82, 89, 87], [183, 103, 163], [167, 198, 0], [133, 229, 133], [123, 131, 135], [159, 249, 96], [168, 126, 128], [140, 99, 122], [201, 203, 19], [53, 81, 5], [23, 112, 229], [60, 48, 33], [227, 63, 212], [130, 14, 176], [151, 172, 147], [133, 59, 77], [167, 105, 177], [120, 9, 250], [80, 3, 36], [132, 34, 162], [180, 72, 112], [46, 254, 247], [150, 101, 55], [201, 239, 194], [172, 133, 119], [84, 89, 104], [36, 0, 119], [158, 78, 81], [68, 115, 100], [64, 47, 26], [204, 216, 246], [180, 57, 106], [34, 161, 233], [131, 246, 204], [74, 255, 45], [223, 234, 73], [34, 233, 198], [74, 125, 217], [180, 190, 211], [233, 120, 196], [217, 13, 64], [95, 69, 82], [243, 199, 43], [214, 207, 133], [62, 196, 2], [238, 231, 85], [27, 22, 121], [25, 127, 160], [159, 75, 40], [186, 40, 155], [61, 222, 127], [118, 246, 108], [110, 169, 34], [214, 64, 126], [176, 115, 103], [216, 28, 57], [166, 20, 52], [103, 190, 90], [202, 188, 109], [204, 137, 171], [207, 246, 16], [250, 108, 221], [60, 115, 140], [141, 78, 238], [146, 176, 100], [140, 220, 41], [164, 158, 154], [69, 158, 183], [149, 48, 22], [187, 155, 207], [161, 249, 9], [98, 166, 120], [47, 130, 26], [180, 104, 231], [221, 69, 158], [172, 187, 242], [53, 170, 102], [62, 231, 219], [212, 15, 148], [66, 148, 122], [137, 27, 86], [154, 124, 33], [97, 165, 153], [192, 129, 230], [23, 211, 123], [63, 100, 16], [216, 155, 186], [166, 202, 13], [176, 92, 149], [184, 161, 157], [57, 57, 173], [116, 60, 167], [234, 197, 17], [133, 218, 148], [105, 186, 202], [200, 196, 126], [237, 174, 41], [117, 49, 153], [218, 206, 43], [37, 87, 221], [192, 219, 101], [5, 18, 102], [155, 58, 244], [99, 73, 181], [210, 177, 136], [222, 204, 186], [52, 90, 110], [252, 128, 248], [11, 35, 121], [23, 33, 188], [157, 114, 73], [82, 194, 169], [47, 207, 69], [129, 112, 196], [177, 197, 172], [5, 101, 19], [28, 9, 114], [63, 251, 98], [111, 68, 232], [197, 99, 63], [37, 133, 149], [89, 122, 141], [139, 139, 46], [4, 46, 208], [139, 64, 171], [185, 189, 223], [249, 162, 224], [27, 51, 17], [109, 174, 91], [187, 240, 31], [181, 100, 22], [134, 196, 24], [11, 142, 66], [223, 21, 138], [204, 28, 13], [71, 245, 73], [232, 241, 178], [216, 174, 243], [42, 106, 64], [162, 108, 215], [227, 209, 193], [48, 195, 203], [171, 244, 50], [98, 227, 92], [42, 95, 60], [188, 208, 232], [190, 240, 240], [189, 52, 186], [53, 54, 104], [100, 182, 52], [243, 38, 60], [168, 127, 170], [194, 147, 226], [94, 53, 22], [173, 219, 21], [202, 64, 120], [123, 68, 231], [233, 116, 94], [132, 24, 238], [109, 199, 113], [240, 9, 23], [159, 100, 72], [162, 111, 44], [184, 168, 148], [125, 204, 120], [139, 112, 161], [103, 206, 229], [71, 202, 187], [112, 176, 60], [192, 220, 52], [250, 85, 237], [145, 74, 197], [43, 94, 40], [86, 105, 121], [156, 90, 249], [244, 207, 36], [73, 173, 58], [84, 154, 186], [113, 244, 89], [183, 113, 86], [63, 185, 62], [249, 57, 23], [197, 162, 43], [144, 183, 62], [186, 65, 158], [39, 211, 21], [83, 14, 161], [53, 38, 106], [205, 63, 36], [225, 221, 205], [121, 56, 201], [47, 129, 146], [57, 119, 228], [146, 200, 49], [32, 233, 102], [41, 50, 87], [20, 230, 146], [228, 136, 85], [244, 39, 176], [0, 49, 132], [249, 224, 232], [230, 118, 93], [212, 95, 31], [233, 199, 137], [35, 57, 125], [200, 99, 206], [54, 191, 251], [162, 143, 170], [254, 208, 68], [170, 71, 209], [224, 120, 96], [100, 65, 201], [184, 192, 102], [52, 187, 166], [88, 220, 10], [57, 129, 222], [212, 58, 4], [63, 234, 4], [60, 252, 94], [230, 76, 127], [146, 113, 175], [72, 235, 1], [154, 236, 92], [41, 251, 57], [147, 30, 144], [226, 212, 4], [121, 195, 14], [88, 164, 118], [251, 79, 217], [171, 187, 175], [53, 221, 185], [204, 56, 25], [212, 203, 121], [10, 89, 196], [62, 86, 100], [245, 40, 109], [36, 168, 212], [124, 131, 153], [7, 169, 174], [125, 92, 247], [149, 181, 171], [203, 153, 96], [25, 138, 69], [118, 184, 165], [189, 11, 170], [69, 11, 16], [50, 95, 98], [82, 124, 114], [42, 52, 216], [131, 195, 160], [223, 14, 62], [80, 87, 64], [79, 211, 127], [193, 229, 249], [50, 187, 232], [87, 209, 224], [69, 22, 81], [226, 182, 103], [15, 139, 43], [173, 12, 7], [93, 158, 22], [184, 18, 237], [180, 164, 138], [154, 21, 103], [210, 157, 182], [37, 58, 226], [242, 156, 184], [55, 216, 226], [76, 49, 170], [154, 227, 53], [124, 229, 15], [146, 122, 13], [254, 215, 180], [196, 132, 75], [4, 234, 138], [166, 122, 174], [108, 182, 228], [107, 120, 149], [32, 76, 151], [42, 54, 191], [248, 74, 38], [248, 109, 41], [56, 3, 61], [63, 146, 86], [212, 28, 18], [187, 184, 32], [207, 11, 102], [83, 206, 3], [36, 127, 133], [29, 205, 52], [181, 91, 129], [123, 2, 209], [211, 216, 66], [15, 20, 202], [62, 141, 179], [199, 119, 71], [86, 209, 192], [106, 179, 103], [193, 123, 225], [89, 28, 143], [168, 9, 117], [109, 207, 185], [94, 152, 148], [26, 192, 221], [138, 27, 100], [209, 115, 120], [154, 254, 121], [28, 150, 125], [49, 26, 131], [194, 108, 112], [151, 145, 12], [53, 223, 124], [146, 143, 217], [253, 212, 103], [133, 78, 29], [65, 164, 184], [250, 155, 204], [112, 70, 8], [68, 107, 99], [196, 238, 247], [227, 91, 243], [242, 195, 98], [230, 219, 211], [114, 19, 124], [50, 14, 118], [35, 174, 150], [174, 120, 90], [90, 72, 84], [247, 74, 111], [163, 118, 183], [251, 154, 149], [225, 112, 45], [17, 175, 113], [54, 221, 210], [116, 17, 90], [174, 204, 146], [13, 204, 138], [166, 172, 106], [116, 108, 54], [109, 94, 217], [76, 122, 56], [190, 246, 14], [52, 138, 250], [29, 58, 95], [36, 10, 128], [127, 215, 54], [80, 113, 84], [78, 83, 38], [24, 87, 195], [204, 252, 120], [227, 255, 146], [233, 103, 189], [92, 217, 90], [223, 4, 148], [33, 214, 205], [218, 18, 33], [106, 123, 92], [85, 43, 138], [129, 237, 179], [35, 184, 99], [42, 226, 107], [7, 55, 126], [51, 117, 37], [224, 28, 18], [250, 138, 89], [53, 9, 42], [31, 153, 85], [231, 171, 118], [36, 1, 143], [48, 112, 157], [208, 28, 8], [212, 118, 159], [145, 211, 43], [63, 156, 85], [162, 217, 242], [12, 161, 71], [71, 15, 157], [229, 107, 79], [32, 101, 97], [90, 98, 241], [30, 61, 134], [171, 249, 111], [136, 92, 193], [67, 184, 173], [219, 100, 86], [26, 66, 32], [113, 69, 136], [176, 212, 58], [225, 119, 216], [184, 37, 57], [141, 181, 116], [90, 48, 120], [202, 102, 212], [14, 110, 117], [16, 66, 232], [144, 216, 65], [139, 152, 18], [170, 96, 47], [173, 104, 127], [61, 230, 239], [164, 108, 170], [106, 145, 132], [22, 74, 167], [101, 17, 237], [112, 106, 158], [223, 30, 40], [71, 19, 72], [6, 155, 163], [249, 77, 155], [195, 87, 214], [61, 144, 254], [124, 161, 209], [254, 90, 153], [166, 69, 224], [87, 189, 98], [156, 14, 94], [29, 112, 52], [119, 188, 99], [95, 203, 190], [192, 28, 103], [232, 36, 234], [21, 147, 83], [168, 134, 226], [243, 192, 137], [240, 126, 83], [170, 208, 245], [208, 83, 118], [130, 239, 143], [229, 181, 135], [220, 42, 84], [198, 30, 188], [189, 34, 233], [84, 76, 195], [215, 170, 48], [238, 58, 130], [148, 186, 147], [45, 60, 61], [150, 10, 146], [122, 181, 90], [31, 184, 128], [38, 49, 17], [70, 96, 134], [228, 138, 215], [162, 148, 222], [68, 136, 146], [76, 44, 183], [76, 79, 113], [78, 186, 82], [159, 151, 83], [3, 46, 3], [190, 86, 52], [93, 54, 121], [167, 72, 209], [127, 123, 31], [132, 214, 254], [42, 91, 203], [71, 35, 94], [10, 29, 173], [135, 9, 2], [89, 254, 153], [152, 97, 24], [147, 166, 181], [129, 87, 89], [240, 159, 138], [213, 217, 178], [73, 237, 79], [153, 92, 172], [101, 72, 117], [96, 188, 177], [59, 226, 85], [148, 197, 170], [115, 114, 118], [110, 239, 171], [190, 54, 46], [25, 48, 135], [244, 66, 129], [193, 61, 181], [96, 167, 170], [246, 234, 90], [195, 127, 85], [255, 49, 163], [3, 236, 196], [210, 255, 59], [88, 8, 57], [202, 208, 82], [16, 24, 211], [35, 247, 27], [68, 21, 173], [42, 33, 196], [168, 90, 116], [162, 113, 162], [10, 129, 31], [163, 255, 254], [200, 65, 124], [147, 44, 40], [42, 43, 83], [192, 176, 242], [179, 109, 178], [136, 136, 34], [22, 100, 79], [102, 227, 138], [226, 9, 102], [186, 98, 132], [93, 111, 240], [238, 176, 83], [196, 149, 143], [81, 151, 149], [241, 58, 7], [243, 67, 0], [242, 58, 130], [233, 29, 8], [126, 243, 205], [250, 144, 115], [213, 148, 95], [208, 200, 243], [75, 94, 132], [176, 80, 249], [191, 146, 39], [182, 52, 127], [8, 71, 11], [253, 204, 225], [252, 7, 101], [82, 100, 200], [223, 101, 9], [4, 200, 163], [91, 59, 54], [251, 92, 125], [90, 119, 231], [77, 143, 255], [166, 28, 92], [182, 13, 204], [120, 139, 72], [204, 89, 202], [119, 140, 104], [62, 210, 70], [227, 72, 14], [114, 228, 180], [124, 126, 201], [70, 254, 20], [197, 203, 8], [179, 139, 174], [219, 134, 13], [74, 162, 226], [184, 9, 201], [147, 165, 145], [201, 211, 80], [119, 31, 103], [246, 204, 157], [192, 35, 10], [179, 32, 100], [73, 45, 243], [16, 75, 173], [159, 184, 160], [87, 29, 7], [98, 0, 144], [126, 145, 186], [148, 201, 103], [57, 44, 187], [169, 74, 224], [183, 180, 109], [78, 105, 224], [124, 202, 11], [42, 160, 7], [195, 73, 43], [75, 22, 97], [231, 190, 36], [216, 12, 178], [149, 133, 208], [35, 230, 123], [53, 29, 118], [132, 242, 42], [55, 187, 50], [214, 109, 5], [209, 231, 83], [209, 151, 196], [149, 151, 172], [22, 132, 152], [237, 173, 25], [48, 134, 208], [223, 224, 29], [203, 161, 167], [40, 165, 82], [84, 172, 105], [242, 133, 50], [224, 162, 132], [153, 74, 221], [237, 84, 136], [128, 236, 137], [56, 167, 137], [118, 253, 155], [103, 245, 21], [166, 255, 72], [164, 96, 7], [26, 204, 30], [59, 163, 39], [26, 153, 81], [92, 202, 152], [57, 223, 35], [104, 206, 15], [229, 56, 198], [251, 126, 98], [168, 106, 233], [48, 56, 130], [144, 196, 222], [27, 211, 180], [156, 5, 170], [220, 13, 45], [35, 78, 136], [232, 50, 170], [150, 43, 75], [192, 115, 149], [27, 38, 19], [6, 208, 237], [96, 161, 139], [198, 26, 32], [163, 207, 137], [108, 132, 187], [151, 25, 51], [195, 46, 194], [141, 252, 139], [11, 19, 175], [3, 135, 169], [16, 181, 222], [217, 79, 119], [203, 216, 25], [154, 209, 228], [171, 119, 18], [201, 160, 222], [247, 114, 68], [202, 252, 17], [248, 198, 67], [219, 48, 76], [212, 235, 168], [214, 23, 38], [252, 75, 106], [68, 26, 238], [209, 200, 1], [157, 161, 137], [91, 26, 216], [141, 23, 28], [168, 199, 156], [89, 230, 242], [123, 18, 236], [216, 40, 85], [132, 139, 238], [105, 103, 67], [185, 75, 178], [43, 83, 226], [194, 200, 175], [17, 20, 251], [8, 250, 213], [73, 169, 67], [101, 16, 186], [218, 10, 199], [3, 77, 178], [180, 104, 143], [0, 198, 64], [168, 211, 144], [180, 160, 164], [116, 140, 68], [244, 148, 222], [173, 62, 9], [56, 24, 225], [155, 201, 208], [194, 244, 151], [31, 23, 174], [28, 238, 90], [8, 127, 98], [80, 36, 191], [200, 172, 36], [198, 44, 13], [136, 251, 64], [205, 161, 172], [233, 190, 64], [253, 163, 174], [78, 57, 111], [227, 122, 231], [19, 47, 117], [139, 133, 250], [67, 34, 92], [29, 82, 177], [241, 249, 207], [169, 236, 211], [54, 144, 0], [84, 152, 247], [127, 52, 234], [124, 252, 213], [118, 119, 112], [127, 158, 133], [120, 40, 121], [216, 223, 248], [84, 43, 19], [13, 131, 200], [144, 20, 167], [2, 58, 222], [84, 155, 149], [214, 4, 81], [142, 236, 23], [23, 159, 96], [38, 118, 228], [79, 237, 217], [139, 51, 157], [174, 4, 225], [235, 5, 17], [28, 43, 250], [41, 136, 202], [87, 179, 58], [58, 154, 164], [61, 57, 2], [167, 219, 153], [10, 86, 107], [142, 6, 43], [48, 184, 125], [180, 88, 42], [195, 198, 88], [149, 252, 133], [2, 171, 107], [172, 166, 44], [15, 157, 5], [101, 123, 209], [112, 8, 206], [204, 189, 195], [128, 232, 60], [230, 183, 81], [248, 188, 134], [139, 160, 101], [98, 193, 121], [98, 125, 217], [235, 67, 24], [113, 36, 135], [219, 112, 98], [116, 124, 123], [81, 193, 214], [238, 65, 247], [251, 35, 158], [118, 151, 203], [248, 210, 193], [141, 164, 1], [55, 254, 92], [150, 46, 29], [222, 163, 207], [227, 224, 63], [227, 136, 235], [21, 202, 157], [233, 129, 75], [129, 110, 11], [154, 102, 47], [179, 77, 42], [57, 160, 151], [251, 127, 52], [1, 91, 114], [126, 189, 191], [130, 130, 184], [234, 152, 162], [207, 127, 127], [180, 111, 133], [90, 223, 250], [150, 46, 83], [248, 188, 78], [115, 224, 195], [102, 40, 75], [44, 10, 174], [13, 197, 93], [179, 33, 127], [51, 62, 62], [206, 172, 152], [132, 248, 6], [79, 34, 181], [180, 191, 153], [17, 114, 253], [61, 190, 236], [98, 134, 84], [199, 69, 66], [166, 104, 90], [105, 246, 140], [111, 48, 223], [69, 130, 85], [176, 154, 159], [238, 5, 180], [116, 253, 45], [136, 212, 131], [100, 176, 123], [170, 37, 42], [9, 35, 70], [167, 191, 157], [203, 190, 211], [9, 171, 241], [194, 36, 112], [191, 250, 151], [71, 219, 211], [58, 135, 4], [117, 220, 157], [13, 251, 39], [140, 154, 249], [182, 60, 31], [229, 17, 60], [206, 84, 145], [52, 81, 108], [135, 107, 46], [151, 106, 14], [91, 229, 42], [50, 143, 92], [4, 53, 86], [112, 109, 216], [193, 76, 144], [204, 36, 171], [154, 249, 40], [191, 44, 219], [36, 63, 221], [231, 81, 58], [41, 4, 143], [172, 204, 251], [14, 58, 163], [45, 118, 116], [86, 58, 32], [209, 70, 213], [177, 102, 80], [168, 205, 201], [55, 80, 244], [9, 127, 179], [68, 248, 255], [153, 127, 74], [191, 119, 101], [12, 229, 78], [5, 7, 83], [225, 28, 208], [42, 106, 185], [248, 0, 167], [0, 80, 132], [98, 35, 9], [53, 96, 162], [195, 53, 100], [237, 124, 88], [254, 200, 189], [221, 76, 129], [185, 57, 130], [14, 101, 167], [172, 72, 178], [129, 236, 50], [240, 113, 160], [189, 79, 194], [133, 71, 243], [66, 76, 249], [86, 80, 83], [23, 181, 99], [26, 100, 159], [216, 118, 115], [200, 10, 34], [114, 223, 52], [75, 45, 116], [100, 136, 185], [74, 249, 250], [53, 191, 211], [153, 85, 45], [114, 205, 165], [89, 45, 144], [219, 118, 40], [127, 238, 168], [166, 248, 196], [133, 27, 238], [149, 37, 41], [253, 115, 104], [62, 31, 51], [141, 135, 50], [13, 137, 47], [40, 12, 0], [231, 141, 57], [194, 93, 62], [12, 249, 250], [91, 179, 154], [135, 215, 125], [27, 103, 59], [239, 132, 121], [132, 120, 101], [113, 158, 75], [85, 97, 131], [142, 120, 204], [14, 211, 242], [76, 18, 152], [63, 155, 94], [99, 86, 205], [216, 59, 243], [30, 227, 21], [132, 116, 210], [243, 230, 46], [0, 249, 172], [54, 248, 29], [4, 58, 69], [196, 104, 75], [157, 126, 241], [203, 47, 167], [60, 35, 87], [26, 158, 94], [231, 82, 187], [182, 191, 101], [76, 222, 42], [225, 194, 130], [121, 54, 9], [221, 16, 115], [146, 162, 49], [52, 92, 56], [22, 121, 203], [248, 239, 60], [254, 84, 15], [140, 72, 248], [250, 191, 209], [194, 166, 202], [197, 232, 101], [37, 222, 208], [82, 21, 196], [251, 121, 248], [225, 140, 58], [236, 211, 64], [209, 178, 216], [194, 210, 41], [148, 86, 137], [172, 199, 1], [8, 107, 133], [98, 169, 179], [66, 173, 98], [6, 113, 113], [33, 32, 193], [58, 206, 224], [117, 140, 101], [120, 242, 147], [68, 105, 235], [3, 40, 4], [160, 201, 246], [109, 167, 244], [187, 67, 169], [88, 170, 187], [54, 63, 121], [25, 179, 175], [104, 118, 180], [20, 134, 252], [21, 3, 35], [17, 152, 119], [125, 46, 105], [233, 15, 80], [7, 251, 133], [51, 166, 206], [20, 29, 241], [112, 39, 87], [5, 147, 123], [161, 2, 217], [6, 105, 235], [239, 48, 152], [143, 20, 174], [142, 145, 15], [34, 155, 156], [207, 208, 26], [139, 158, 218], [140, 26, 155], [31, 173, 111], [94, 89, 153], [179, 233, 184], [16, 99, 47], [192, 98, 168], [126, 140, 168], [137, 203, 123], [72, 56, 33], [17, 8, 203], [202, 152, 151], [180, 187, 186], [40, 125, 179], [81, 153, 88], [195, 221, 205], [17, 183, 237], [125, 51, 177], [124, 71, 170], [47, 91, 134], [180, 205, 52], [123, 19, 246], [36, 74, 88], [106, 144, 78], [66, 113, 90], [169, 134, 65], [86, 39, 152], [14, 24, 178], [80, 146, 67], [242, 226, 31], [204, 26, 201], [232, 183, 185], [212, 82, 2], [241, 90, 96], [54, 58, 213], [48, 74, 176], [201, 100, 27], [72, 16, 245], [9, 72, 163], [105, 88, 80], [75, 112, 122], [31, 121, 201], [199, 47, 99], [224, 51, 197], [210, 149, 0], [211, 72, 237], [104, 79, 232], [60, 173, 76], [232, 169, 165], [244, 207, 122], [202, 88, 180], [221, 237, 152], [1, 233, 225], [121, 43, 217], [109, 180, 133], [25, 248, 146], [30, 251, 180], [212, 235, 100], [68, 179, 163], [145, 198, 37], [70, 5, 58], [88, 82, 51], [13, 40, 249], [10, 139, 159], [168, 2, 207], [114, 245, 216], [210, 131, 36], [88, 148, 65], [253, 7, 219], [120, 122, 4], [166, 208, 65], [36, 65, 111], [94, 122, 237], [52, 129, 245], [92, 2, 251], [35, 234, 201], [112, 130, 212], [124, 37, 84], [16, 237, 15], [165, 106, 88], [167, 17, 154], [84, 247, 166], [74, 83, 138], [15, 162, 17], [218, 250, 16], [126, 10, 199], [223, 112, 250], [134, 210, 190], [68, 6, 217], [180, 71, 190], [88, 135, 205], [195, 93, 248], [22, 23, 59], [207, 136, 254], [10, 251, 89], [232, 163, 120], [252, 92, 67], [72, 106, 174], [173, 80, 206], [33, 54, 201], [206, 210, 185], [35, 24, 119], [138, 172, 9], [189, 70, 97], [208, 5, 97], [181, 41, 110], [199, 118, 20], [155, 53, 38], [140, 208, 141], [19, 243, 7], [146, 108, 127], [67, 78, 172], [18, 65, 182], [107, 167, 193], [216, 8, 132], [106, 140, 195], [83, 240, 159], [33, 140, 16], [183, 67, 58], [33, 100, 68], [134, 239, 64], [138, 220, 72], [49, 3, 247], [28, 31, 69], [189, 75, 84], [117, 81, 67], [136, 211, 238], [129, 238, 80], [147, 6, 138], [90, 234, 153], [58, 189, 43], [42, 253, 244], [235, 106, 181], [41, 250, 192], [223, 6, 178], [71, 38, 24], [29, 254, 43], [68, 190, 27], [42, 250, 107], [245, 230, 216], [172, 133, 42], [235, 205, 195], [57, 187, 112], [48, 244, 84], [0, 163, 172], [118, 26, 101], [190, 191, 254], [198, 204, 38], [40, 160, 151], [40, 152, 229], [136, 96, 218], [237, 175, 152], [211, 25, 92], [28, 49, 19], [97, 215, 73], [199, 226, 147], [131, 33, 80], [29, 51, 119], [101, 191, 166], [13, 230, 43], [145, 179, 44], [168, 164, 92], [63, 208, 225], [5, 190, 208], [162, 201, 82], [209, 213, 33], [46, 15, 126], [200, 49, 23], [213, 222, 31], [121, 18, 128], [211, 157, 83], [243, 142, 100], [253, 13, 43], [184, 140, 134], [209, 85, 211], [34, 166, 80], [25, 28, 66], [18, 42, 136], [208, 179, 240], [68, 213, 233], [235, 148, 238], [206, 128, 137], [44, 96, 188], [19, 38, 127], [79, 153, 65], [123, 51, 170], [71, 159, 119], [212, 237, 203], [219, 95, 156], [51, 252, 83], [29, 243, 82], [73, 146, 243], [253, 130, 198], [13, 61, 14], [29, 160, 90], [194, 125, 14], [81, 10, 133], [26, 139, 14], [242, 238, 94], [126, 74, 255], [201, 65, 233], [109, 117, 31], [115, 75, 172], [137, 246, 105], [94, 210, 151], [11, 50, 170], [225, 222, 187], [90, 252, 113], [143, 140, 137], [210, 72, 20], [132, 250, 243], [207, 220, 113], [225, 223, 55], [227, 49, 88], [40, 134, 209], [22, 184, 106], [122, 243, 86], [148, 138, 122], [162, 121, 88], [23, 207, 71], [228, 104, 59], [152, 248, 105], [31, 246, 201], [19, 115, 153], [106, 53, 82], [155, 103, 71], [93, 160, 117], [100, 193, 17], [34, 44, 25], [147, 56, 242], [186, 102, 250], [142, 222, 31], [121, 221, 104], [64, 114, 63], [44, 13, 244], [88, 227, 9], [102, 105, 175], [163, 86, 180], [240, 182, 190], [120, 30, 111], [217, 100, 99], [46, 31, 83], [35, 152, 170], [109, 59, 18], [244, 91, 42], [185, 145, 88], [66, 18, 255], [153, 245, 214], [123, 177, 116], [93, 223, 195], [23, 159, 252], [188, 103, 141], [105, 145, 140], [169, 214, 219], [189, 192, 211], [41, 88, 152], [173, 170, 172], [193, 62, 18], [104, 95, 78], [34, 179, 187], [122, 165, 0], [19, 215, 189], [27, 88, 120], [214, 254, 57], [72, 223, 89], [1, 107, 112], [36, 61, 40], [91, 99, 158], [234, 250, 237], [29, 177, 28], [147, 50, 29], [152, 119, 84], [49, 78, 204], [166, 89, 54], [175, 161, 215], [77, 217, 108], [103, 113, 249], [165, 200, 75], [54, 19, 214], [191, 231, 25], [18, 22, 124], [148, 202, 26], [210, 179, 182], [178, 249, 102], [164, 167, 46], [79, 22, 129], [115, 98, 98], [203, 109, 30], [65, 243, 32], [124, 176, 5], [113, 191, 33], [157, 128, 181], [208, 93, 72], [185, 44, 162], [154, 27, 87], [251, 145, 63], [29, 54, 162], [95, 175, 179], [68, 243, 197], [195, 37, 193], [80, 41, 218], [31, 248, 206], [97, 40, 39], [202, 39, 158], [72, 53, 48], [21, 104, 243], [157, 235, 212], [20, 240, 62], [36, 35, 168], [82, 229, 59], [252, 19, 73], [184, 235, 73], [61, 195, 118], [189, 66, 3], [176, 48, 200], [39, 67, 171], [90, 84, 229], [197, 209, 114], [196, 65, 9], [47, 51, 93], [6, 77, 19], [134, 209, 85], [89, 201, 166], [202, 78, 150], [147, 15, 243], [234, 79, 151], [85, 212, 42], [209, 135, 194], [128, 47, 234], [20, 247, 235], [148, 83, 254], [30, 75, 176], [15, 89, 69], [199, 188, 235], [197, 206, 153], [104, 253, 48], [60, 225, 247], [182, 198, 102], [93, 49, 128], [225, 105, 136], [102, 148, 32], [68, 24, 208], [132, 19, 75], [184, 172, 186], [151, 95, 131], [121, 57, 6], [175, 170, 49], [244, 29, 85], [141, 16, 37], [204, 153, 174], [212, 214, 54], [162, 163, 136], [230, 20, 46], [131, 72, 154], [97, 24, 28], [99, 40, 64], [93, 248, 229], [16, 62, 175], [135, 173, 136], [3, 161, 220], [22, 183, 156], [166, 37, 212], [22, 237, 216], [173, 191, 102], [179, 1, 94], [19, 158, 194], [225, 35, 192], [65, 246, 124], [122, 196, 2], [201, 49, 140], [242, 60, 243], [162, 68, 48], [191, 185, 245], [147, 177, 101], [211, 27, 190], [253, 37, 21], [140, 202, 245], [162, 239, 203], [233, 128, 169], [165, 161, 207], [10, 89, 35], [90, 217, 68], [255, 149, 73], [1, 95, 143], [17, 245, 8], [92, 248, 109], [39, 226, 60], [23, 218, 10], [115, 129, 223], [240, 11, 242], [25, 36, 24], [224, 211, 226], [218, 207, 201], [219, 200, 167], [81, 89, 44], [227, 83, 23], [76, 96, 191], [24, 20, 230], [149, 65, 132], [122, 156, 83], [45, 199, 115], [196, 61, 52], [10, 61, 171], [21, 9, 44], [90, 226, 123], [64, 244, 254], [51, 29, 190], [37, 22, 135], [41, 187, 41], [19, 130, 5], [96, 244, 24], [11, 159, 221], [252, 39, 147], [254, 219, 212], [175, 91, 106], [60, 4, 170], [38, 103, 143], [51, 150, 241], [134, 129, 185], [54, 222, 133], [189, 75, 43], [196, 44, 18], [113, 39, 16], [69, 114, 168], [5, 202, 174], [159, 4, 127], [160, 235, 183], [146, 220, 190], [188, 200, 183], [70, 91, 68], [92, 2, 56], [195, 19, 145], [245, 250, 198], [38, 42, 4], [125, 214, 99], [113, 228, 169], [249, 105, 213], [240, 193, 226], [217, 162, 55], [204, 128, 26], [63, 86, 245], [144, 228, 76], [170, 92, 110], [117, 7, 131], [193, 234, 217], [77, 115, 31], [246, 48, 139], [111, 131, 185], [21, 105, 203], [96, 101, 61], [77, 202, 126], [31, 158, 100], [112, 146, 164], [105, 26, 125], [112, 75, 161], [227, 56, 106], [249, 220, 83], [131, 112, 21], [181, 102, 60], [86, 178, 133], [11, 190, 53], [153, 218, 46], [143, 172, 44], [126, 49, 180], [108, 245, 105], [111, 0, 228], [137, 10, 76], [213, 112, 166], [69, 202, 7], [182, 148, 179], [51, 52, 215], [245, 63, 108], [93, 183, 35], [135, 72, 210], [92, 96, 162], [201, 78, 30], [196, 82, 144], [3, 45, 180], [132, 4, 115], [10, 209, 147], [139, 135, 65], [200, 150, 224], [117, 12, 250], [203, 137, 135], [121, 129, 0], [20, 20, 204], [91, 29, 48], [96, 120, 249], [76, 247, 149], [103, 77, 221], [182, 233, 120], [168, 165, 224], [67, 160, 252], [89, 59, 78], [194, 169, 124], [73, 204, 108], [132, 235, 38], [244, 175, 164], [62, 87, 195], [179, 51, 33], [134, 96, 58], [12, 176, 201], [103, 44, 5], [172, 0, 51], [190, 233, 66], [4, 230, 0], [21, 17, 70], [97, 126, 218], [21, 147, 237], [144, 9, 171], [103, 24, 2], [206, 76, 242], [35, 7, 166], [230, 62, 28], [204, 75, 144], [58, 199, 250], [115, 32, 243], [149, 51, 154], [207, 230, 56], [0, 47, 201], [214, 201, 175], [244, 183, 238], [245, 2, 94], [250, 195, 173], [134, 53, 215], [239, 239, 131], [81, 41, 249], [36, 14, 163], [57, 238, 179], [87, 56, 46], [111, 227, 253], [123, 178, 111], [249, 38, 200], [106, 218, 229], [175, 49, 206], [87, 46, 236], [62, 108, 62], [227, 21, 184], [169, 134, 52], [67, 149, 164], [40, 181, 138], [9, 133, 52], [234, 19, 56], [132, 87, 22], [9, 59, 100], [221, 164, 249], [247, 1, 9], [211, 221, 30], [1, 135, 24], [71, 36, 200], [129, 231, 193], [192, 152, 90], [37, 134, 229], [211, 130, 185], [17, 46, 79], [129, 45, 53], [35, 16, 146], [109, 210, 31], [155, 172, 152], [18, 161, 127], [20, 15, 181], [233, 173, 38], [146, 12, 85], [187, 147, 14], [103, 198, 220], [216, 201, 82], [34, 5, 79], [5, 227, 47], [174, 176, 124], [243, 72, 27], [228, 74, 207], [107, 61, 82], [100, 40, 193], [237, 219, 84], [124, 142, 87], [220, 232, 31], [17, 111, 35], [204, 204, 178], [13, 243, 99], [59, 52, 216], [169, 84, 156], [157, 234, 151], [50, 52, 248], [128, 221, 61], [91, 8, 174], [167, 166, 177], [119, 12, 245], [216, 241, 15], [63, 170, 14], [118, 79, 177], [247, 205, 33], [108, 149, 26], [17, 20, 172], [65, 217, 106], [166, 232, 154], [177, 7, 190], [114, 64, 36], [113, 118, 41], [180, 189, 193], [72, 28, 37], [25, 187, 4], [131, 29, 71], [33, 29, 75], [94, 31, 205], [198, 72, 102], [235, 248, 164], [207, 240, 63], [249, 137, 215], [98, 228, 26], [25, 118, 168], [10, 170, 66], [233, 143, 37], [195, 181, 85], [125, 40, 158], [27, 251, 106], [41, 71, 54], [11, 124, 170], [82, 216, 26], [202, 20, 86], [92, 243, 59], [234, 161, 165], [21, 8, 118], [223, 167, 244], [202, 30, 35], [85, 121, 99], [168, 252, 60], [90, 183, 199], [140, 33, 3], [35, 136, 124], [244, 88, 45], [67, 237, 154], [149, 136, 36], [40, 90, 89], [40, 236, 85], [153, 105, 139], [41, 241, 21], [184, 66, 62], [92, 63, 107], [19, 18, 211], [166, 75, 184], [105, 97, 173], [130, 6, 228], [122, 96, 65], [239, 0, 231], [180, 203, 207], [222, 41, 50], [188, 195, 172], [48, 2, 112], [254, 148, 212], [85, 218, 134], [45, 133, 59], [11, 134, 12], [55, 8, 116], [91, 75, 206], [84, 222, 104], [126, 27, 212], [148, 228, 225], [173, 108, 96], [129, 215, 214], [10, 139, 4], [188, 154, 119], [110, 148, 229], [150, 165, 163], [121, 142, 109], [189, 241, 80], [72, 205, 102], [182, 95, 62], [69, 79, 244], [155, 182, 251], [128, 109, 63], [80, 150, 200], [145, 227, 165], [232, 104, 169], [8, 129, 31], [102, 225, 47], [35, 217, 168], [243, 62, 136], [221, 146, 79], [146, 50, 147], [239, 111, 53], [113, 177, 80], [157, 12, 54], [253, 15, 183], [47, 34, 120], [170, 201, 217], [71, 161, 173], [175, 68, 202], [186, 43, 24], [212, 147, 243], [87, 254, 154], [172, 39, 102], [104, 143, 13], [239, 42, 195], [127, 20, 130], [62, 35, 29], [133, 127, 199], [150, 110, 243], [179, 34, 113], [87, 239, 76], [136, 221, 161], [106, 4, 69], [241, 236, 255], [187, 7, 34], [61, 102, 134], [234, 30, 11], [153, 45, 207], [178, 198, 96], [78, 151, 228], [143, 113, 245], [5, 66, 19], [31, 230, 66], [207, 140, 196], [249, 14, 45], [202, 24, 65], [158, 117, 65], [123, 0, 154], [166, 48, 182], [119, 246, 72], [175, 136, 150], [20, 143, 63], [242, 196, 117], [0, 188, 210], [20, 203, 69], [108, 183, 94], [22, 208, 101], [185, 101, 33], [161, 24, 203], [205, 104, 212], [242, 40, 100], [65, 19, 113], [29, 32, 48], [224, 249, 168], [18, 224, 207], [165, 83, 57], [244, 210, 105], [77, 111, 195], [27, 174, 105], [34, 164, 253], [47, 95, 179], [85, 1, 241], [204, 115, 15], [5, 87, 162], [245, 157, 68], [109, 11, 21], [183, 241, 3], [131, 84, 176], [185, 17, 27], [142, 27, 119], [60, 111, 33], [164, 158, 57], [18, 247, 52], [115, 217, 240], [94, 40, 142], [220, 132, 117], [72, 145, 110], [248, 40, 11], [134, 37, 80], [174, 138, 147], [241, 216, 135], [131, 89, 147], [179, 57, 198], [13, 17, 40], [30, 78, 56], [130, 248, 52], [25, 60, 246], [170, 101, 242], [102, 177, 114], [102, 11, 34], [99, 126, 37], [40, 43, 159], [254, 124, 106], [243, 108, 225], [205, 97, 54], [102, 15, 0], [138, 80, 83], [119, 28, 222], [93, 58, 229], [5, 104, 214], [130, 10, 185], [105, 171, 43], [21, 242, 100], [68, 51, 143], [237, 131, 222], [13, 12, 221], [94, 15, 239], [11, 206, 202], [164, 66, 68], [9, 56, 152], [81, 72, 130], [21, 146, 169], [47, 198, 133], [152, 120, 87], [135, 213, 164], [221, 243, 238], [60, 215, 79], [94, 243, 147], [64, 251, 255], [252, 32, 27], [97, 205, 233], [115, 221, 251], [15, 67, 108], [137, 96, 20], [142, 164, 74], [17, 249, 74], [193, 244, 203], [254, 84, 78], [34, 117, 39], [223, 13, 214], [191, 113, 1], [186, 118, 29], [31, 156, 67], [140, 37, 226], [103, 186, 48], [189, 228, 97], [94, 122, 168], [152, 230, 13], [211, 187, 4], [240, 106, 200], [58, 16, 135], [162, 42, 65], [43, 138, 117], [119, 11, 172], [77, 62, 43], [210, 124, 18], [186, 226, 199], [21, 255, 202], [250, 9, 234], [75, 78, 94], [65, 203, 180], [51, 203, 217], [25, 8, 27], [128, 167, 181], [5, 198, 122], [172, 50, 239], [129, 210, 70], [89, 23, 167], [226, 82, 47], [79, 229, 81], [44, 87, 192], [83, 184, 38], [145, 230, 249], [76, 55, 12], [251, 127, 134], [178, 206, 141], [60, 69, 144], [87, 242, 99], [75, 14, 221], [217, 177, 30], [176, 201, 86], [31, 180, 242], [111, 212, 1], [69, 210, 225], [198, 83, 188], [198, 136, 147], [98, 37, 208], [29, 171, 166], [193, 190, 95], [208, 86, 92], [44, 126, 96], [38, 187, 167], [79, 121, 196], [234, 49, 201], [231, 95, 97], [24, 16, 110], [121, 196, 71], [174, 2, 48], [191, 65, 104], [192, 170, 148], [92, 19, 215], [206, 215, 216], [171, 75, 183], [18, 231, 175], [172, 98, 155], [200, 203, 252], [224, 48, 255], [34, 35, 243], [71, 168, 250], [209, 188, 171], [125, 79, 17], [221, 241, 3], [46, 53, 93], [70, 156, 135], [36, 246, 205], [57, 186, 157], [182, 60, 151], [36, 105, 33], [84, 60, 130], [145, 152, 85], [208, 45, 187], [149, 78, 76], [148, 146, 97], [224, 125, 73], [53, 207, 245], [84, 99, 193], [1, 31, 30], [18, 45, 1], [158, 216, 103], [168, 189, 158], [16, 181, 229], [93, 58, 58], [210, 82, 97], [3, 252, 191], [32, 71, 99], [22, 184, 226], [92, 213, 145], [164, 57, 59], [41, 120, 106], [230, 125, 123], [228, 18, 3], [219, 66, 5], [249, 77, 35], [237, 151, 178], [215, 151, 127], [71, 132, 44], [185, 207, 17], [254, 158, 229], [137, 139, 141], [79, 102, 34], [88, 52, 187], [95, 251, 115], [41, 55, 182], [183, 15, 123], [188, 40, 233], [225, 205, 158], [99, 217, 125], [56, 225, 132], [51, 162, 120], [125, 246, 189], [173, 247, 29], [143, 87, 244], [48, 7, 166], [31, 128, 211], [226, 55, 254], [78, 200, 211], [56, 206, 207], [67, 251, 25], [156, 232, 141], [197, 120, 210], [113, 197, 142], [34, 19, 135], [125, 196, 229], [179, 172, 29], [112, 187, 176], [27, 218, 49], [231, 12, 218], [34, 74, 209], [182, 253, 170], [25, 211, 39], [231, 169, 69], [40, 12, 151], [69, 1, 112], [224, 214, 228], [54, 244, 189], [223, 111, 33], [216, 240, 103], [141, 162, 9], [57, 131, 159], [110, 240, 171], [42, 205, 20], [124, 160, 6], [135, 67, 95], [74, 57, 191], [1, 224, 36], [11, 250, 201], [132, 82, 196], [31, 89, 230], [193, 1, 38], [211, 242, 57], [50, 7, 152], [7, 184, 224], [3, 79, 5], [137, 214, 148], [168, 171, 196], [173, 74, 9], [90, 100, 75], [68, 60, 147], [239, 111, 230], [224, 152, 17], [8, 210, 216], [71, 210, 191], [200, 28, 211], [191, 240, 225], [186, 119, 45], [198, 97, 12], [216, 10, 16], [237, 65, 132], [252, 144, 14], [192, 231, 14], [56, 59, 100], [19, 145, 63], [9, 161, 4], [15, 236, 120], [139, 205, 215], [132, 55, 43], [199, 160, 76], [160, 73, 2], [35, 17, 209], [5, 42, 81], [80, 102, 194], [234, 124, 114], [219, 64, 142], [224, 211, 68], [137, 177, 157], [1, 201, 89], [80, 225, 25], [46, 124, 217], [204, 63, 32], [78, 81, 61], [229, 227, 185], [158, 87, 67], [235, 17, 17], [2, 36, 229], [182, 144, 171], [8, 43, 81], [198, 181, 163], [225, 45, 28], [115, 78, 116], [75, 135, 38], [29, 94, 180], [216, 171, 94], [169, 149, 196], [16, 218, 237], [125, 51, 211], [106, 3, 254], [175, 158, 72], [19, 153, 5], [147, 242, 148], [132, 168, 231], [17, 234, 25], [201, 233, 138], [178, 186, 160], [24, 174, 27], [65, 206, 95], [165, 64, 32], [188, 6, 129], [200, 57, 52], [195, 8, 32], [15, 203, 35], [88, 182, 74], [11, 190, 192], [54, 27, 54], [212, 110, 85], [49, 240, 19], [183, 73, 236], [150, 219, 27], [68, 225, 69], [51, 184, 202], [153, 160, 4], [23, 36, 102], [72, 8, 99], [162, 29, 148], [183, 236, 144], [38, 182, 88], [217, 132, 240], [156, 79, 194], [208, 51, 93], [19, 12, 95], [20, 155, 81], [74, 166, 24], [23, 244, 57], [68, 12, 121], [242, 173, 245], [81, 21, 102], [105, 204, 78], [80, 66, 94], [139, 176, 151], [83, 74, 253], [113, 155, 73], [4, 208, 11], [180, 46, 236], [181, 132, 215], [181, 77, 115], [128, 207, 207], [66, 60, 222], [35, 43, 41], [229, 78, 44], [254, 195, 60], [186, 140, 159], [3, 27, 171], [123, 222, 115], [33, 22, 120], [25, 95, 34], [39, 208, 143], [97, 103, 244], [237, 143, 21], [141, 20, 89], [219, 101, 161], [149, 115, 57], [86, 226, 135], [235, 17, 60], [119, 247, 104], [147, 215, 30], [224, 187, 46], [25, 159, 187], [15, 132, 177], [244, 138, 179], [224, 137, 109], [162, 146, 92], [37, 133, 254], [131, 235, 71], [199, 186, 229], [187, 202, 31], [60, 130, 155], [232, 168, 249], [242, 116, 141], [106, 83, 52], [134, 247, 78], [8, 15, 74], [16, 214, 163], [178, 98, 198], [157, 44, 203], [3, 62, 51], [15, 116, 36], [194, 201, 201], [76, 6, 27], [136, 109, 188], [207, 196, 43], [243, 107, 82], [77, 86, 149], [226, 21, 32], [143, 43, 34], [109, 119, 185], [134, 130, 42], [102, 87, 251], [5, 199, 202], [205, 5, 130], [106, 198, 59], [123, 142, 13], [215, 128, 215], [73, 234, 103], [110, 179, 163], [110, 147, 145], [9, 59, 139], [232, 53, 103], [212, 42, 115], [61, 254, 41], [95, 240, 219], [43, 54, 10], [209, 31, 255], [255, 219, 126], [80, 56, 208], [147, 36, 168], [113, 70, 166], [204, 221, 159], [9, 243, 243], [103, 209, 0], [20, 192, 26], [57, 104, 184], [221, 243, 47], [239, 222, 28], [34, 34, 200], [89, 58, 74], [14, 196, 141], [188, 13, 14], [96, 165, 142], [237, 77, 181], [147, 149, 53], [3, 172, 156], [24, 11, 21], [152, 7, 190], [12, 203, 195], [128, 58, 13], [29, 127, 51], [81, 243, 0], [101, 60, 225], [124, 34, 124], [102, 130, 15], [171, 6, 187], [126, 170, 64], [162, 11, 161], [188, 49, 95], [252, 209, 233], [223, 81, 100], [128, 152, 127], [64, 231, 108], [130, 232, 37], [215, 242, 190], [48, 48, 148], [241, 74, 75], [238, 85, 1], [152, 234, 212], [216, 163, 65], [200, 133, 87], [201, 170, 34], [135, 203, 23], [134, 11, 111], [225, 53, 1], [175, 94, 39], [161, 114, 69], [156, 248, 72], [129, 104, 226], [47, 191, 137], [154, 177, 193], [192, 173, 147], [183, 188, 72], [222, 101, 194], [214, 247, 21], [142, 145, 234], [115, 89, 87], [186, 211, 212], [102, 172, 234], [2, 212, 72], [151, 32, 18], [119, 125, 178], [212, 79, 232], [251, 189, 15], [166, 119, 64], [160, 47, 163], [233, 100, 78], [35, 17, 154], [176, 106, 40], [254, 145, 99], [122, 22, 181], [98, 110, 2], [233, 154, 167], [228, 36, 9], [140, 184, 157], [36, 245, 50], [200, 0, 237], [63, 254, 247], [18, 67, 54], [235, 91, 15], [132, 146, 180], [196, 135, 190], [179, 155, 115], [104, 9, 39], [76, 21, 210], [18, 225, 1], [143, 19, 190], [85, 18, 232], [38, 57, 63], [199, 49, 178], [154, 225, 3], [36, 246, 118], [72, 21, 199], [132, 96, 88], [73, 88, 1], [236, 248, 50], [236, 139, 252], [75, 239, 203], [195, 146, 208], [254, 16, 160], [31, 164, 140], [119, 28, 239], [129, 173, 69], [21, 161, 75], [127, 118, 111], [202, 218, 50], [238, 40, 220], [223, 157, 187], [68, 126, 84], [203, 242, 140], [25, 155, 174], [53, 107, 222], [168, 94, 202], [0, 203, 25], [65, 66, 200], [109, 198, 173], [214, 178, 156], [181, 226, 109], [190, 81, 194], [76, 76, 169], [185, 112, 235], [230, 237, 39], [75, 9, 117], [180, 230, 107], [178, 199, 219], [129, 97, 10], [85, 103, 81], [18, 104, 244], [141, 115, 15], [64, 208, 26], [94, 11, 147], [227, 36, 109], [74, 155, 132], [14, 204, 186], [67, 74, 126], [45, 235, 12], [244, 95, 17], [97, 154, 160], [182, 182, 81], [183, 4, 200], [168, 38, 83], [144, 255, 121], [192, 50, 110], [88, 127, 237], [108, 85, 133], [117, 104, 61], [208, 250, 138], [250, 56, 138], [68, 227, 189], [247, 249, 84], [227, 143, 231], [45, 129, 181], [199, 80, 50], [73, 217, 36], [31, 7, 111], [211, 91, 18], [212, 104, 227], [34, 75, 25], [40, 130, 136], [207, 122, 0], [99, 76, 172], [84, 129, 120], [116, 9, 32], [236, 61, 187], [17, 40, 189], [51, 228, 111], [125, 114, 132], [175, 233, 55], [234, 208, 205], [36, 14, 31], [17, 60, 71], [7, 120, 37], [133, 50, 57], [65, 214, 95], [107, 155, 216], [223, 76, 157], [168, 218, 192], [221, 143, 68], [72, 222, 49], [99, 183, 45], [78, 214, 61], [219, 18, 41], [57, 191, 44], [247, 94, 50], [166, 197, 170], [151, 113, 125], [206, 217, 52], [46, 82, 44], [64, 91, 242], [223, 29, 244], [188, 219, 55], [81, 42, 35], [108, 107, 42], [138, 157, 134], [27, 243, 6], [18, 149, 210], [145, 21, 20], [25, 199, 56], [0, 104, 65], [212, 109, 86], [48, 193, 110], [0, 212, 34], [49, 71, 245], [93, 245, 99], [171, 47, 48], [10, 9, 115], [72, 115, 15], [241, 99, 84], [63, 195, 151], [178, 190, 46], [79, 244, 50], [160, 227, 149], [97, 153, 25], [14, 55, 27], [245, 179, 134], [145, 211, 132], [107, 163, 142], [137, 93, 191], [79, 147, 90], [149, 46, 222], [104, 30, 187], [144, 10, 114], [56, 207, 66], [219, 48, 91], [42, 201, 11], [6, 144, 91], [106, 102, 89], [79, 115, 107], [150, 249, 15], [154, 191, 192], [182, 157, 194], [229, 22, 42], [199, 188, 3], [82, 78, 155], [139, 58, 83], [15, 117, 175], [28, 235, 0], [191, 2, 57], [149, 116, 73], [242, 36, 38], [34, 19, 247], [142, 206, 34], [156, 59, 34], [116, 71, 26], [151, 226, 144], [191, 21, 74], [108, 43, 206], [61, 59, 0], [142, 182, 114], [236, 136, 73], [205, 73, 112], [29, 198, 181], [95, 131, 189], [75, 124, 230], [50, 218, 189], [63, 21, 25], [27, 241, 160], [136, 154, 219], [50, 47, 197], [15, 254, 77], [111, 250, 187], [137, 112, 6], [113, 169, 247], [243, 125, 97], [17, 35, 60], [141, 27, 24], [179, 205, 253], [117, 1, 27], [6, 127, 141], [131, 167, 162], [160, 240, 0], [219, 88, 82], [252, 51, 156], [138, 51, 126], [0, 138, 5], [255, 105, 193], [237, 65, 141], [252, 126, 219], [179, 111, 61], [144, 43, 142], [221, 187, 126], [189, 122, 38], [77, 39, 90], [255, 191, 127], [108, 88, 81], [133, 113, 188], [123, 222, 218], [18, 102, 11], [19, 190, 34], [239, 198, 244], [162, 212, 23], [11, 102, 252], [110, 89, 184], [97, 110, 169], [131, 239, 239], [44, 248, 131], [228, 82, 54], [17, 197, 93], [89, 111, 40], [116, 92, 73], [160, 96, 160], [172, 11, 245], [128, 156, 184], [221, 224, 241], [135, 248, 123], [181, 242, 88], [216, 234, 250], [164, 255, 238], [3, 109, 199], [46, 244, 19], [13, 174, 94], [44, 25, 68], [101, 80, 143], [162, 251, 1], [243, 4, 188], [82, 247, 59], [112, 229, 27], [142, 157, 143], [183, 17, 19], [189, 51, 66], [107, 64, 114], [248, 139, 64], [166, 135, 187], [10, 93, 120], [118, 96, 234], [113, 197, 132], [135, 2, 54], [64, 104, 172], [132, 232, 73], [120, 72, 99], [63, 84, 163], [21, 228, 51], [115, 158, 228], [127, 3, 52], [242, 98, 41], [153, 27, 60], [46, 76, 41], [202, 121, 19], [221, 148, 104], [196, 174, 61], [164, 162, 56], [0, 12, 29], [150, 0, 137], [70, 40, 162], [119, 0, 124], [253, 103, 131], [3, 152, 51], [21, 135, 64], [172, 82, 85], [120, 65, 169], [202, 164, 83], [201, 189, 219], [189, 36, 83], [212, 251, 74], [250, 168, 224], [204, 169, 142], [229, 192, 23], [94, 134, 145], [209, 140, 124], [63, 21, 200], [228, 68, 50], [244, 35, 83], [65, 102, 227], [64, 213, 28], [181, 157, 80], [19, 225, 168], [101, 215, 230], [71, 191, 164], [85, 19, 38], [5, 240, 96], [82, 255, 71], [80, 12, 27], [170, 146, 216], [218, 62, 133], [154, 195, 36], [195, 161, 175], [162, 128, 58], [249, 96, 46], [180, 188, 193], [77, 63, 75], [34, 103, 133], [162, 32, 198], [184, 234, 104], [151, 135, 247], [59, 203, 164], [118, 173, 183], [216, 224, 107], [100, 111, 117], [87, 227, 138], [31, 4, 246], [238, 116, 89], [78, 189, 125], [14, 155, 216], [172, 226, 238], [151, 107, 65], [71, 151, 161], [85, 6, 39], [160, 126, 172], [117, 5, 55], [133, 99, 13], [50, 45, 198], [30, 239, 105], [239, 212, 205], [66, 10, 90], [192, 114, 116], [79, 35, 171], [82, 165, 174], [201, 6, 36], [125, 30, 106], [54, 80, 117], [143, 207, 254], [72, 157, 204], [169, 129, 107], [195, 48, 77], [215, 114, 211], [226, 110, 55], [128, 25, 227], [92, 20, 60], [62, 91, 188], [159, 21, 118], [3, 119, 217], [122, 73, 72], [9, 73, 161], [141, 96, 12], [182, 128, 149], [195, 196, 192], [234, 90, 72], [238, 75, 187], [120, 164, 99], [204, 137, 103], [182, 230, 130], [135, 0, 237], [199, 240, 78], [37, 183, 195], [151, 217, 157], [170, 120, 158], [208, 63, 114], [252, 69, 47], [200, 127, 193], [153, 193, 169], [172, 241, 254], [194, 135, 133], [243, 123, 61], [61, 166, 64], [81, 64, 76], [80, 56, 162], [21, 17, 178], [214, 160, 162], [14, 86, 113], [186, 177, 240], [208, 96, 58], [181, 97, 115], [98, 131, 131], [201, 250, 243], [225, 151, 167], [244, 148, 9], [85, 48, 233], [129, 194, 234], [62, 110, 75], [40, 108, 217], [18, 46, 54], [129, 90, 206], [203, 158, 252], [54, 6, 236], [248, 247, 254], [61, 232, 241], [76, 156, 201], [226, 80, 192], [128, 169, 143], [238, 18, 50], [64, 68, 251], [246, 72, 210], [212, 11, 135], [72, 113, 17], [53, 220, 181], [116, 76, 103], [214, 219, 77], [62, 55, 123], [253, 19, 15], [14, 188, 14], [98, 89, 134], [101, 231, 71], [53, 198, 66], [160, 178, 23], [249, 53, 197], [238, 250, 107], [152, 145, 1], [43, 252, 204], [90, 48, 203], [106, 8, 85], [29, 56, 152], [224, 53, 232], [130, 116, 199], [193, 158, 30], [8, 43, 13], [92, 57, 92], [34, 235, 171], [179, 105, 17], [111, 246, 128], [178, 87, 157], [9, 7, 181], [126, 218, 197], [35, 85, 38], [71, 199, 81], [30, 87, 101], [111, 49, 68], [108, 43, 42], [113, 143, 80], [147, 66, 118], [56, 124, 131], [96, 194, 20], [26, 70, 188], [210, 84, 52], [55, 50, 43], [10, 180, 127], [255, 142, 129], [159, 89, 40], [0, 67, 129], [201, 243, 184], [79, 132, 72], [251, 184, 29], [140, 73, 131], [161, 86, 249], [189, 93, 141], [23, 209, 241], [154, 21, 126], [172, 227, 195], [107, 130, 135], [57, 34, 122], [77, 37, 47], [108, 255, 56], [187, 247, 57], [186, 116, 157], [157, 255, 85], [249, 47, 83], [249, 49, 13], [251, 70, 172], [31, 213, 25], [216, 5, 251], [119, 35, 8], [36, 69, 183], [232, 230, 7], [66, 220, 120], [229, 251, 13], [187, 153, 250], [35, 205, 172], [60, 251, 116], [203, 141, 113], [226, 236, 101], [233, 192, 123], [49, 46, 6], [48, 25, 241], [148, 146, 9], [53, 130, 39], [101, 243, 181], [183, 70, 36], [27, 189, 95], [178, 134, 135], [32, 222, 241], [208, 219, 232], [141, 153, 28], [250, 221, 146], [59, 20, 25], [113, 1, 249], [46, 105, 121], [165, 203, 15], [124, 50, 9], [133, 201, 15], [209, 192, 175], [84, 181, 250], [87, 252, 46], [139, 219, 103], [191, 93, 128], [216, 179, 220], [108, 51, 174], [96, 140, 181], [177, 23, 125], [46, 126, 68], [107, 33, 138], [210, 87, 118], [104, 8, 131], [57, 192, 60], [13, 106, 250], [94, 86, 5], [29, 57, 252], [60, 59, 29], [104, 43, 49], [26, 54, 108], [113, 136, 24], [200, 78, 139], [72, 207, 106], [170, 15, 249], [93, 71, 87], [184, 16, 214], [67, 147, 122], [74, 62, 199], [45, 106, 155], [142, 80, 51], [99, 104, 193], [62, 211, 9], [100, 218, 171], [184, 172, 39], [211, 83, 206], [226, 41, 160], [111, 17, 33], [166, 154, 31], [146, 246, 106], [214, 128, 104], [43, 150, 38], [181, 84, 127], [168, 48, 150], [158, 44, 198], [63, 45, 240], [126, 67, 145], [161, 226, 215], [182, 129, 101], [86, 133, 148], [200, 64, 177], [173, 247, 162], [28, 220, 141], [156, 87, 118], [39, 109, 252], [169, 253, 231], [48, 21, 192], [9, 87, 108], [144, 40, 186], [54, 41, 214], [16, 27, 72], [236, 249, 31], [124, 246, 50], [99, 8, 24], [152, 150, 46], [0, 47, 35], [186, 214, 102], [168, 137, 104], [196, 62, 9], [173, 71, 98], [123, 211, 203], [133, 130, 30], [189, 225, 205], [188, 24, 56], [148, 178, 28], [116, 32, 17], [154, 246, 86], [216, 140, 95], [233, 1, 148], [180, 238, 100], [89, 209, 116], [82, 245, 217], [101, 185, 200], [32, 108, 153], [78, 79, 82], [173, 167, 13], [252, 15, 248], [60, 19, 180], [90, 102, 232], [133, 241, 147], [47, 138, 71], [109, 94, 48], [59, 246, 195], [50, 204, 135], [106, 75, 40], [51, 150, 13], [79, 102, 19], [229, 64, 182], [102, 137, 96], [131, 131, 203], [119, 202, 237], [78, 111, 92], [251, 227, 243], [53, 243, 162], [149, 48, 215], [119, 243, 129], [227, 120, 3], [200, 13, 177], [159, 153, 76], [156, 21, 225], [190, 74, 104], [67, 106, 184], [78, 27, 1], [242, 78, 3], [49, 116, 163], [249, 241, 20], [234, 113, 108], [183, 80, 198], [82, 21, 122], [105, 144, 76], [126, 10, 231], [168, 204, 168], [115, 52, 189], [101, 236, 155], [67, 191, 154], [219, 50, 247], [42, 111, 27], [236, 153, 101], [66, 207, 117], [115, 73, 51], [66, 198, 0], [103, 180, 8], [181, 126, 249], [246, 37, 198], [78, 59, 137], [212, 107, 232], [149, 28, 93], [247, 128, 55], [196, 13, 203], [164, 117, 124], [106, 73, 37], [235, 37, 127], [94, 93, 217], [213, 68, 141], [98, 15, 68], [241, 109, 224], [173, 177, 62], [236, 122, 61], [10, 198, 58], [207, 176, 29], [122, 131, 16], [50, 246, 40], [51, 32, 135], [249, 117, 75], [108, 32, 107], [162, 59, 2], [176, 55, 133], [193, 41, 16], [72, 165, 169], [34, 88, 160], [198, 32, 192], [183, 39, 64], [125, 122, 6], [146, 194, 165], [56, 79, 204], [116, 8, 17], [201, 148, 112], [177, 181, 179], [162, 235, 49], [72, 121, 171], [185, 248, 184], [43, 90, 48], [213, 250, 42], [204, 48, 35], [213, 214, 182], [178, 147, 168], [50, 213, 128], [159, 69, 15], [126, 131, 204], [47, 3, 164], [114, 22, 23], [42, 120, 202], [183, 4, 124], [91, 64, 19], [37, 42, 33], [26, 0, 86], [133, 22, 129], [70, 50, 174], [234, 243, 82], [244, 16, 80], [237, 207, 105], [56, 116, 163], [227, 61, 85], [216, 210, 95], [36, 180, 50], [38, 123, 190], [133, 92, 46], [118, 223, 201], [235, 7, 5], [155, 112, 190], [49, 26, 197], [240, 182, 155], [72, 86, 4], [92, 213, 174], [199, 219, 37], [76, 178, 163], [251, 109, 7], [188, 204, 35], [192, 147, 156], [123, 76, 145], [62, 81, 252], [9, 132, 243], [31, 31, 32], [240, 128, 120], [90, 75, 163], [79, 147, 205], [128, 112, 13], [17, 160, 190], [51, 38, 222], [129, 134, 10], [106, 19, 202], [250, 232, 10], [168, 229, 79], [209, 165, 63], [0, 93, 178], [113, 1, 194], [71, 253, 199], [127, 130, 185], [156, 243, 78], [180, 170, 166], [104, 157, 95], [210, 161, 12], [241, 112, 236], [35, 128, 228], [62, 111, 121], [62, 97, 127], [225, 239, 204], [233, 52, 51], [69, 203, 96], [92, 232, 182], [157, 54, 156], [241, 137, 177], [200, 117, 95], [160, 229, 73], [223, 128, 194], [192, 130, 1], [14, 105, 152], [65, 92, 215], [22, 251, 61], [19, 169, 84], [169, 105, 243], [28, 49, 60], [226, 122, 238], [142, 111, 152], [181, 27, 28], [45, 65, 41], [58, 59, 190], [176, 243, 83], [45, 90, 9], [153, 219, 59], [241, 212, 89], [173, 181, 189], [0, 180, 186], [171, 40, 49], [157, 135, 76], [22, 189, 3], [121, 78, 181], [179, 172, 119], [184, 167, 134], [31, 229, 43], [102, 247, 95], [120, 76, 188], [95, 54, 199], [147, 230, 250], [13, 235, 30], [161, 105, 227], [41, 227, 243], [218, 116, 157], [218, 8, 194], [141, 64, 125], [159, 19, 145], [99, 229, 222], [177, 8, 210], [119, 176, 128], [22, 158, 111], [214, 236, 98], [123, 214, 117], [146, 89, 105], [192, 52, 31], [87, 88, 216], [115, 17, 241], [99, 29, 187], [102, 203, 66], [110, 75, 7], [225, 102, 174], [138, 253, 132], [21, 42, 166], [171, 183, 207], [11, 8, 10], [145, 236, 159], [51, 60, 16], [242, 208, 197], [81, 97, 179], [183, 217, 58], [199, 199, 196], [86, 88, 3], [76, 190, 82], [146, 193, 183], [253, 171, 109], [70, 35, 194], [214, 140, 247], [169, 108, 143], [125, 116, 235], [202, 94, 53], [24, 4, 43], [81, 140, 160], [103, 109, 48], [6, 225, 28], [216, 57, 135], [15, 214, 182], [146, 13, 213], [126, 203, 32], [39, 27, 64], [26, 213, 225], [191, 89, 101], [247, 210, 230], [181, 19, 37], [182, 79, 100], [13, 27, 241], [138, 224, 5], [204, 63, 166], [138, 111, 46], [154, 70, 79], [195, 155, 212], [59, 80, 81], [8, 245, 116], [184, 67, 40], [18, 206, 210], [213, 98, 37], [201, 217, 160], [37, 159, 6], [131, 7, 12], [212, 193, 92], [19, 218, 245], [57, 31, 34], [219, 122, 9], [78, 80, 156], [132, 136, 179], [219, 139, 202], [176, 19, 13], [88, 220, 129], [151, 204, 9], [191, 22, 219], [60, 192, 158], [33, 220, 8], [224, 120, 96], [151, 103, 154], [101, 180, 155], [133, 150, 193], [32, 158, 196], [168, 112, 43], [49, 153, 80], [207, 26, 133], [233, 219, 216], [128, 212, 212], [72, 96, 147], [236, 233, 70], [123, 181, 17], [204, 111, 159], [177, 102, 81], [10, 112, 172], [254, 100, 230], [255, 33, 84], [73, 31, 170], [9, 13, 35], [41, 127, 14], [133, 172, 252], [236, 192, 1], [128, 245, 8], [227, 175, 184], [92, 206, 203], [14, 30, 22], [123, 170, 86], [192, 148, 205], [78, 190, 57], [47, 52, 195], [80, 149, 254], [43, 209, 100], [141, 76, 83], [242, 70, 144], [197, 97, 67], [252, 55, 101], [43, 190, 207], [32, 199, 171], [57, 7, 25], [167, 151, 101], [71, 201, 132], [90, 113, 23], [207, 53, 134], [235, 99, 246], [52, 246, 157], [107, 84, 98], [81, 175, 184], [172, 43, 140], [82, 211, 24], [254, 117, 77], [152, 18, 104], [226, 89, 55], [216, 243, 33], [10, 220, 203], [187, 170, 152], [194, 237, 242], [241, 197, 187], [149, 213, 196], [31, 4, 240], [242, 178, 178], [236, 36, 149], [209, 249, 153], [251, 0, 230], [120, 136, 168], [186, 105, 114], [242, 246, 67], [3, 194, 96], [122, 159, 144], [168, 191, 39], [187, 16, 98], [29, 254, 226], [166, 252, 18], [126, 66, 37], [179, 65, 161], [234, 133, 99], [47, 59, 247], [119, 102, 133], [83, 53, 230], [251, 196, 6], [167, 117, 18], [191, 136, 188], [197, 85, 74], [203, 1, 8], [34, 129, 127], [230, 139, 107], [4, 212, 79], [229, 64, 100], [58, 75, 220], [219, 127, 95], [41, 118, 31], [157, 53, 212], [111, 39, 176], [234, 67, 94], [158, 2, 237], [182, 252, 87], [0, 125, 245], [172, 179, 74], [236, 147, 216], [96, 41, 9], [219, 93, 5], [142, 219, 164], [235, 4, 242], [144, 86, 195], [85, 217, 239], [49, 104, 168], [171, 69, 0], [83, 133, 6], [32, 42, 249], [23, 181, 244], [176, 20, 233], [188, 152, 98], [180, 28, 51], [175, 150, 100], [72, 158, 108], [119, 16, 13], [211, 161, 117], [101, 171, 91], [124, 237, 227], [41, 110, 116], [116, 50, 99], [102, 207, 56], [114, 34, 166], [43, 61, 245], [84, 81, 10], [66, 195, 34], [31, 201, 2], [78, 140, 121], [216, 63, 71], [239, 26, 129], [190, 143, 137], [34, 108, 174], [105, 175, 33], [180, 107, 165], [161, 49, 67], [227, 99, 181], [210, 45, 144], [26, 14, 98], [223, 173, 156], [165, 83, 251], [236, 108, 108], [73, 75, 109], [138, 70, 38], [31, 58, 174], [40, 67, 242], [98, 188, 164], [1, 186, 26], [224, 185, 230], [74, 16, 143], [38, 164, 155], [29, 171, 94], [55, 54, 129], [33, 171, 24], [102, 163, 46], [71, 87, 43], [168, 252, 234], [127, 59, 216], [125, 33, 244], [65, 58, 25], [71, 245, 10], [244, 251, 94], [188, 124, 152], [226, 105, 192], [229, 130, 103], [182, 109, 187], [46, 36, 71], [148, 229, 33], [70, 35, 71], [88, 61, 36], [251, 10, 131], [128, 217, 91], [25, 110, 107], [81, 52, 186], [31, 213, 218], [200, 97, 124], [240, 255, 12], [221, 121, 238], [81, 38, 246], [100, 74, 183], [162, 146, 187], [8, 211, 65], [93, 50, 9], [131, 148, 177], [88, 159, 250], [33, 77, 22], [75, 239, 247], [164, 155, 179], [220, 92, 109], [103, 39, 229], [28, 101, 148], [109, 41, 65], [161, 34, 85], [112, 27, 137], [104, 59, 121], [1, 59, 23], [252, 248, 51], [196, 20, 12], [72, 138, 43], [253, 217, 177], [109, 145, 49], [78, 101, 143], [178, 158, 236], [199, 230, 20], [28, 0, 15], [154, 23, 81], [237, 117, 94], [143, 74, 216], [223, 123, 80], [63, 102, 224], [9, 223, 138], [198, 243, 82], [199, 215, 209], [15, 149, 137], [63, 96, 8], [191, 172, 25], [63, 241, 48], [114, 150, 196], [171, 3, 87], [20, 249, 220], [68, 160, 180], [255, 52, 240], [227, 121, 133], [86, 89, 17], [25, 184, 206], [112, 19, 137], [15, 230, 161], [44, 195, 145], [255, 58, 93], [39, 144, 227], [39, 156, 35], [216, 59, 53], [235, 165, 129], [71, 11, 81], [6, 34, 98], [22, 7, 15], [147, 87, 156], [235, 80, 206], [7, 205, 150], [18, 103, 76], [11, 200, 199], [238, 187, 113], [240, 40, 210], [24, 147, 194], [159, 222, 97], [60, 197, 184], [79, 235, 93], [254, 154, 195], [130, 48, 153], [157, 218, 120], [63, 227, 211], [91, 171, 229], [22, 232, 133], [93, 103, 58], [63, 193, 45], [178, 239, 126], [140, 252, 102], [253, 243, 72], [227, 184, 134], [186, 165, 64], [250, 94, 87], [144, 238, 53], [26, 182, 193], [246, 132, 84], [124, 163, 137], [89, 218, 211], [248, 16, 195], [199, 86, 190], [251, 211, 33], [24, 218, 51], [164, 148, 39], [94, 82, 100], [3, 44, 141], [77, 110, 106], [145, 66, 182], [114, 49, 181], [190, 185, 72], [55, 216, 203], [20, 154, 76], [236, 98, 139], [239, 235, 132], [146, 169, 68], [32, 169, 16], [20, 253, 178], [244, 80, 142], [78, 67, 24], [219, 202, 200], [198, 44, 120], [206, 126, 100], [135, 139, 146], [24, 216, 213], [252, 155, 126], [39, 219, 249], [99, 161, 64], [79, 233, 65], [42, 109, 72], [26, 251, 159], [254, 254, 115], [169, 235, 175], [105, 15, 173], [67, 224, 165], [27, 242, 54], [126, 152, 80], [30, 2, 171], [107, 71, 70], [203, 215, 84], [44, 170, 44], [244, 115, 220], [16, 24, 236], [4, 42, 173], [189, 94, 214], [94, 14, 21], [160, 95, 117], [132, 251, 141], [88, 31, 202], [105, 176, 68], [126, 175, 240], [194, 152, 197], [76, 117, 250], [99, 123, 117], [22, 155, 148], [214, 219, 159], [86, 210, 209], [209, 181, 236], [6, 117, 27], [246, 30, 64], [31, 239, 139], [97, 248, 184], [120, 169, 159], [112, 152, 9], [247, 68, 122], [120, 140, 209], [228, 90, 145], [248, 165, 188], [60, 109, 38], [121, 169, 216], [245, 9, 86], [79, 234, 236], [152, 252, 187], [176, 137, 195], [232, 4, 239], [83, 30, 19], [33, 106, 239], [65, 57, 141], [122, 64, 243], [112, 121, 80], [200, 13, 149], [65, 169, 189], [148, 250, 84], [211, 186, 220], [131, 101, 54], [69, 59, 95], [24, 49, 42], [78, 112, 27], [244, 231, 156], [248, 63, 32], [238, 189, 8], [203, 18, 29], [153, 44, 93], [168, 229, 187], [183, 119, 234], [74, 9, 11], [94, 235, 187], [189, 10, 228], [175, 245, 143], [134, 237, 50], [160, 36, 175], [128, 186, 152], [95, 52, 38], [143, 100, 23], [75, 49, 69], [2, 92, 135], [65, 147, 96], [174, 244, 71], [210, 75, 16], [35, 176, 221], [132, 213, 117], [82, 89, 40], [162, 111, 176], [199, 157, 121], [9, 136, 208], [138, 80, 218], [135, 50, 201], [115, 139, 253], [149, 29, 103], [108, 98, 30], [158, 215, 205], [169, 95, 155], [67, 25, 218], [82, 75, 209], [180, 74, 103], [55, 194, 63], [83, 37, 188], [113, 88, 83], [82, 199, 174], [12, 71, 37], [215, 109, 125], [185, 72, 57], [194, 224, 58], [123, 176, 66], [53, 82, 47], [54, 42, 27], [40, 77, 172], [82, 133, 93], [175, 17, 174], [198, 59, 115], [196, 12, 28], [207, 226, 29], [37, 161, 236], [134, 31, 8], [139, 17, 130], [80, 100, 237], [48, 185, 77], [133, 170, 78], [181, 226, 145], [210, 216, 218], [98, 138, 168], [168, 14, 147], [227, 209, 142], [215, 44, 252], [122, 131, 71], [156, 39, 71], [184, 59, 252], [225, 167, 149], [119, 172, 136], [120, 227, 109], [180, 36, 45], [105, 146, 186], [154, 124, 80], [201, 71, 216], [248, 29, 188], [118, 78, 124], [159, 250, 156], [246, 195, 221], [101, 202, 109], [227, 32, 31], [177, 29, 60], [5, 68, 88], [213, 235, 196], [136, 44, 200], [1, 47, 116], [239, 175, 35], [106, 191, 216], [127, 143, 79], [197, 125, 66], [222, 145, 145], [240, 80, 49], [204, 253, 170], [239, 65, 122], [182, 181, 60], [155, 42, 47], [154, 239, 175], [195, 108, 223], [2, 27, 252], [162, 162, 232], [45, 244, 123], [11, 70, 151], [123, 57, 15], [64, 152, 106], [228, 174, 94], [58, 252, 0], [26, 96, 86], [48, 113, 83], [76, 74, 238], [25, 216, 138], [27, 231, 53], [130, 228, 198], [233, 133, 85], [238, 79, 185], [62, 0, 210], [106, 207, 61], [89, 29, 96], [37, 251, 233], [194, 253, 139], [180, 67, 221], [168, 85, 178], [133, 96, 107], [129, 61, 252], [45, 145, 49], [206, 177, 187], [151, 225, 140], [161, 175, 250], [5, 243, 164], [0, 59, 160], [155, 237, 127], [49, 82, 96], [22, 58, 186], [32, 207, 125], [17, 184, 225], [168, 150, 90], [220, 211, 44], [153, 68, 161], [13, 64, 36], [50, 180, 139], [141, 183, 247], [111, 88, 215], [210, 144, 212], [236, 157, 47], [45, 248, 58], [1, 35, 172], [22, 171, 254], [214, 105, 23], [81, 16, 127], [204, 47, 183], [64, 49, 211], [244, 246, 171], [238, 235, 37], [60, 252, 206], [222, 11, 232], [70, 37, 206], [199, 2, 210], [25, 145, 211], [198, 153, 76], [44, 211, 237], [205, 96, 204], [38, 55, 3], [29, 190, 98], [92, 241, 121], [90, 30, 236], [213, 186, 64], [25, 69, 79], [37, 4, 51], [243, 42, 98], [55, 64, 172], [26, 216, 219], [24, 249, 47], [248, 9, 212], [201, 124, 133], [65, 89, 30], [148, 191, 55], [111, 94, 37], [45, 48, 121], [156, 133, 233], [1, 53, 145], [152, 96, 36], [163, 83, 81], [198, 238, 35], [231, 86, 156], [223, 50, 10], [35, 190, 119], [173, 251, 129], [248, 178, 208], [221, 151, 150], [211, 192, 43], [5, 234, 55], [244, 10, 130], [252, 147, 36], [221, 73, 68], [16, 175, 128], [13, 176, 7], [142, 149, 165], [27, 215, 93], [88, 20, 130], [209, 251, 76], [138, 18, 81], [0, 129, 139], [54, 126, 64], [84, 60, 15], [4, 58, 15], [87, 248, 76], [236, 148, 198], [78, 79, 11], [193, 40, 254], [31, 140, 92], [190, 138, 126], [128, 228, 157], [10, 116, 13], [29, 132, 131], [110, 188, 4], [144, 164, 112], [171, 77, 38], [136, 222, 58], [149, 149, 31], [193, 210, 21], [20, 7, 105], [250, 120, 113], [157, 254, 148], [183, 242, 103], [64, 101, 5], [210, 51, 137], [151, 134, 93], [182, 174, 141], [36, 184, 117], [61, 10, 205], [171, 3, 241], [210, 180, 187], [28, 97, 164], [229, 24, 140], [117, 157, 22], [195, 228, 219], [216, 209, 193], [115, 213, 128], [80, 179, 142], [118, 79, 160], [98, 253, 227], [20, 73, 42], [215, 174, 7], [238, 5, 125], [233, 247, 208], [171, 211, 212], [37, 22, 85], [159, 64, 77], [123, 131, 98], [201, 135, 185], [155, 34, 109], [162, 154, 173], [223, 96, 54], [231, 103, 2], [179, 110, 59], [210, 14, 36], [95, 13, 177], [35, 78, 9], [2, 169, 142], [246, 222, 243], [147, 157, 92], [236, 22, 201], [194, 211, 13], [33, 109, 213], [10, 90, 190], [211, 240, 40], [246, 197, 143], [164, 191, 112], [177, 23, 113], [209, 96, 48], [195, 160, 74], [138, 129, 54], [123, 112, 111], [86, 34, 71], [107, 175, 138], [85, 116, 38], [237, 172, 3], [219, 230, 231], [81, 179, 111], [238, 151, 85], [149, 2, 29], [252, 20, 54], [249, 246, 160], [253, 103, 126], [73, 220, 132], [64, 203, 60], [63, 155, 193], [147, 118, 3], [118, 164, 18], [43, 168, 120], [112, 173, 140], [9, 183, 45], [140, 59, 145], [224, 235, 193], [40, 195, 190], [102, 144, 251], [188, 197, 203], [81, 216, 242], [26, 84, 79], [75, 113, 5], [186, 126, 242], [16, 217, 121], [245, 234, 242], [220, 118, 99], [46, 107, 202], [46, 149, 31], [188, 19, 232], [241, 126, 237], [118, 63, 38], [190, 7, 41], [97, 148, 225], [118, 53, 27], [203, 155, 172], [237, 98, 15], [16, 119, 36], [156, 98, 10], [15, 14, 56], [68, 19, 56], [253, 69, 111], [24, 159, 106], [219, 66, 254], [246, 223, 232], [175, 241, 12], [228, 90, 41], [95, 77, 106], [163, 176, 20], [169, 113, 43], [132, 150, 118], [186, 117, 211], [177, 87, 102], [129, 215, 109], [254, 15, 64], [151, 155, 12], [93, 183, 189], [76, 15, 173], [72, 233, 14], [33, 162, 249], [61, 62, 39], [191, 118, 16], [224, 135, 0], [195, 210, 29], [111, 199, 160], [131, 239, 220], [153, 184, 140], [132, 108, 43], [235, 51, 162], [46, 15, 228], [44, 108, 161], [188, 178, 102], [234, 29, 199], [35, 35, 250], [8, 182, 229], [117, 173, 247], [36, 208, 77], [6, 219, 101], [64, 199, 197], [207, 241, 44], [244, 159, 110], [14, 38, 73], [131, 5, 81], [174, 177, 192], [57, 90, 47], [223, 86, 190], [232, 42, 101], [118, 117, 5], [178, 144, 65], [45, 249, 183], [44, 211, 121], [239, 216, 198], [163, 130, 195], [123, 19, 69], [168, 218, 77], [113, 135, 209], [30, 196, 188], [27, 215, 209], [6, 88, 95], [126, 189, 4], [94, 158, 1], [30, 60, 109], [207, 82, 175], [116, 47, 133], [246, 23, 43], [242, 81, 64], [40, 20, 207], [119, 240, 47], [242, 48, 49], [26, 203, 16], [226, 46, 88], [161, 226, 60], [181, 133, 42], [160, 128, 237], [84, 115, 245], [193, 33, 160], [47, 156, 71], [105, 172, 127], [122, 252, 232], [67, 105, 136], [22, 244, 177], [137, 94, 136], [223, 12, 188], [249, 127, 6], [135, 53, 47], [199, 154, 13], [55, 164, 27], [179, 33, 103], [157, 234, 124], [208, 115, 121], [213, 24, 67], [231, 194, 244], [170, 10, 194], [146, 220, 200], [220, 7, 87], [103, 198, 75], [189, 172, 130], [225, 168, 41], [144, 98, 77], [93, 86, 41], [240, 21, 57], [159, 212, 213], [227, 71, 14], [23, 192, 153], [201, 109, 52], [222, 120, 26], [108, 171, 128], [246, 166, 124], [19, 238, 128], [253, 251, 55], [167, 174, 248], [226, 64, 6], [125, 189, 198], [120, 221, 182], [107, 248, 22], [60, 43, 3], [70, 207, 87], [26, 83, 30], [6, 65, 140], [93, 171, 238], [125, 108, 31], [254, 19, 166], [1, 208, 244], [7, 121, 229], [140, 41, 16], [224, 166, 217], [16, 115, 108], [37, 125, 53], [38, 70, 208], [16, 138, 98], [85, 255, 39], [39, 142, 231], [165, 124, 69], [71, 135, 22], [107, 79, 16], [162, 205, 118], [200, 167, 42], [243, 146, 14], [133, 245, 38], [253, 177, 157], [150, 55, 136], [132, 115, 31], [134, 163, 60], [48, 226, 31], [233, 53, 61], [218, 174, 158], [186, 185, 18], [233, 152, 210], [108, 148, 119], [237, 17, 177], [97, 233, 150], [138, 185, 250], [179, 50, 47], [200, 97, 237], [21, 195, 25], [66, 53, 247], [48, 128, 34], [201, 243, 39], [203, 32, 196], [30, 32, 215], [116, 15, 70], [31, 211, 133], [121, 160, 9], [54, 100, 152], [22, 248, 8], [55, 144, 234], [12, 46, 67], [236, 210, 129], [183, 168, 167], [197, 231, 170], [82, 170, 243], [96, 101, 213], [222, 135, 241], [167, 10, 245], [7, 72, 72], [137, 69, 159], [163, 188, 240], [154, 78, 208], [119, 130, 74], [153, 84, 248], [61, 179, 208], [97, 157, 223], [80, 245, 174], [7, 45, 116], [13, 71, 90], [241, 25, 141], [66, 180, 182], [173, 114, 66], [209, 176, 57], [90, 253, 251], [123, 240, 28], [157, 77, 10], [142, 87, 37], [70, 175, 33], [40, 194, 79], [27, 210, 18], [32, 33, 92], [47, 79, 56], [98, 191, 167], [214, 73, 212], [33, 222, 20], [135, 169, 171], [131, 1, 227], [199, 182, 86], [167, 78, 245], [201, 182, 3], [104, 61, 204], [78, 197, 202], [5, 62, 237], [64, 247, 23], [11, 25, 71], [133, 25, 223], [172, 18, 32], [225, 160, 2], [254, 119, 167], [40, 221, 94], [190, 138, 176], [119, 183, 184], [130, 79, 131], [40, 112, 151], [121, 149, 122], [118, 234, 53], [32, 214, 190], [234, 47, 229], [30, 18, 103], [67, 111, 240], [251, 96, 94], [63, 153, 149], [128, 118, 89], [154, 102, 228], [25, 152, 19], [60, 6, 70], [148, 94, 95], [27, 50, 106], [107, 51, 34], [4, 218, 101], [242, 156, 213], [222, 61, 81], [242, 130, 202], [119, 68, 207], [218, 15, 232], [27, 251, 139], [46, 234, 111], [112, 94, 151], [211, 123, 28], [40, 134, 45], [144, 71, 15], [203, 21, 208], [119, 146, 74], [173, 97, 206], [212, 4, 238], [98, 157, 241], [37, 130, 25], [148, 94, 108], [75, 98, 244], [130, 39, 243], [216, 21, 147], [77, 140, 240], [182, 61, 210], [113, 212, 46], [40, 224, 183], [81, 253, 9], [74, 251, 213], [125, 234, 123], [236, 223, 205], [53, 185, 52], [78, 7, 11], [207, 26, 96], [31, 58, 44], [94, 37, 189], [132, 246, 180], [254, 66, 61], [65, 147, 166], [230, 110, 10], [71, 143, 117], [16, 228, 208], [96, 220, 125], [134, 218, 41], [1, 70, 190], [37, 157, 148], [172, 197, 157], [76, 101, 74], [61, 216, 175], [127, 70, 140], [41, 23, 87], [248, 74, 163], [0, 93, 113], [203, 47, 60], [109, 93, 222], [203, 98, 110], [165, 107, 136], [88, 98, 187], [139, 225, 224], [39, 99, 144], [15, 234, 44], [222, 87, 208], [148, 126, 72], [72, 252, 81], [183, 235, 144], [246, 233, 163], [34, 246, 52], [36, 34, 236], [142, 118, 64], [69, 242, 88], [231, 85, 139], [65, 235, 251], [106, 89, 151], [19, 195, 80], [3, 114, 128], [251, 239, 62], [106, 191, 79], [213, 132, 117], [96, 232, 154], [124, 81, 110], [46, 247, 106], [58, 149, 69], [139, 150, 3], [98, 18, 10], [23, 133, 116], [171, 86, 175], [165, 139, 20], [239, 175, 227], [126, 195, 111], [32, 114, 164], [48, 87, 117], [129, 201, 52], [188, 234, 207], [168, 198, 89], [238, 17, 84], [73, 122, 135], [103, 79, 27], [248, 31, 65], [189, 140, 186], [225, 75, 48], [194, 4, 19], [188, 220, 235], [72, 195, 210], [241, 235, 83], [253, 231, 10], [236, 61, 189], [140, 73, 153], [124, 9, 66], [64, 235, 241], [92, 176, 155], [159, 73, 170], [11, 223, 47], [60, 247, 218], [205, 115, 16], [244, 207, 162], [162, 10, 207], [123, 173, 43], [104, 98, 5], [212, 42, 201], [67, 186, 147], [32, 218, 104], [215, 234, 93], [95, 234, 56], [64, 137, 181], [139, 225, 199], [3, 100, 106], [124, 174, 51], [208, 214, 127], [0, 61, 124], [205, 235, 175], [151, 190, 119], [209, 235, 250], [170, 178, 85], [46, 51, 49], [20, 121, 209], [86, 183, 80], [43, 12, 178], [159, 237, 130], [122, 211, 234], [101, 111, 71], [24, 54, 221], [108, 2, 57], [108, 30, 183], [163, 230, 29], [104, 12, 113], [193, 127, 217], [20, 240, 141], [55, 204, 72], [227, 212, 216], [187, 139, 2], [246, 61, 123], [28, 24, 154], [149, 235, 116], [127, 245, 125], [28, 94, 192], [103, 75, 227], [225, 252, 113], [250, 44, 75], [165, 176, 3], [52, 213, 232], [63, 57, 225], [120, 106, 5], [31, 218, 197], [92, 246, 15], [29, 227, 206], [94, 161, 76], [254, 146, 195], [114, 210, 89], [60, 84, 124], [234, 10, 49], [193, 66, 69], [138, 181, 178], [247, 49, 55], [239, 70, 144], [249, 166, 137], [24, 114, 107], [253, 23, 165], [37, 160, 253], [189, 148, 249], [51, 116, 46], [76, 92, 151], [202, 45, 250], [8, 137, 170], [133, 243, 147], [4, 175, 24], [166, 162, 6], [13, 35, 204], [17, 45, 98], [170, 112, 58], [175, 17, 224], [32, 241, 196], [107, 238, 90], [21, 220, 61], [50, 226, 149], [124, 7, 236], [28, 27, 93], [110, 134, 78], [201, 117, 58], [155, 34, 95], [131, 176, 38], [184, 24, 104], [172, 0, 140], [3, 145, 72], [95, 217, 165], [64, 46, 31], [42, 70, 207], [141, 17, 167], [215, 216, 51], [38, 217, 221], [183, 46, 60], [28, 159, 8], [82, 154, 213], [190, 62, 168], [60, 212, 36], [149, 137, 127], [212, 150, 116], [107, 40, 98], [81, 122, 2], [231, 240, 102], [228, 175, 218], [187, 8, 250], [156, 177, 115], [223, 91, 19], [16, 187, 186], [143, 146, 245], [86, 96, 73], [21, 11, 138], [7, 75, 13], [204, 10, 160], [246, 19, 101], [213, 154, 84], [229, 165, 228], [20, 144, 145], [179, 23, 90], [191, 11, 103], [92, 132, 159], [73, 116, 246], [90, 140, 148], [91, 173, 178], [168, 64, 218], [21, 72, 102], [105, 137, 78], [245, 13, 141], [240, 76, 89], [190, 85, 118], [66, 44, 184], [3, 100, 107], [210, 207, 103], [158, 253, 174], [113, 220, 237], [134, 224, 63], [231, 61, 2], [247, 97, 183], [189, 141, 148], [58, 114, 59], [59, 21, 111], [120, 119, 114], [228, 143, 201], [109, 123, 4], [183, 85, 131], [219, 87, 85], [252, 142, 243], [142, 22, 220], [96, 146, 94], [223, 171, 171], [141, 215, 40], [208, 220, 222], [151, 51, 161], [83, 215, 173], [136, 180, 195], [197, 91, 117], [64, 45, 125], [228, 192, 81], [251, 253, 247], [90, 203, 23], [105, 237, 34], [125, 104, 156], [73, 13, 24], [190, 225, 170], [55, 209, 234], [206, 61, 191], [211, 153, 67], [43, 253, 47], [188, 152, 204], [243, 53, 245], [232, 107, 242], [60, 186, 83], [93, 187, 124], [23, 31, 147], [17, 253, 187], [35, 60, 173], [97, 23, 251], [225, 0, 14], [251, 13, 129], [15, 78, 144], [120, 190, 178], [163, 87, 72], [190, 217, 224], [83, 92, 67], [213, 186, 208], [143, 78, 95], [168, 27, 109], [34, 54, 174], [247, 252, 177], [27, 95, 35], [227, 178, 100], [70, 128, 138], [37, 153, 198], [70, 155, 231], [238, 90, 233], [231, 45, 144], [85, 188, 223], [7, 121, 250], [66, 77, 49], [132, 28, 3], [175, 221, 47], [61, 159, 67], [218, 178, 1], [11, 123, 189], [73, 107, 125], [209, 165, 4], [124, 238, 185], [158, 91, 122], [61, 86, 14], [185, 179, 170], [92, 180, 58], [146, 119, 167], [223, 121, 72], [112, 192, 15], [129, 43, 53], [249, 19, 49], [45, 56, 18], [145, 63, 161], [61, 130, 60], [169, 243, 168], [4, 65, 13], [115, 23, 92], [145, 209, 172], [192, 91, 26], [218, 253, 222], [182, 135, 52], [117, 78, 228], [155, 7, 178], [139, 28, 130], [91, 248, 66], [169, 163, 159], [235, 105, 80], [90, 175, 22], [98, 178, 235], [184, 88, 229], [60, 114, 141], [77, 109, 15], [235, 87, 53], [223, 235, 126], [213, 164, 162], [11, 8, 168], [198, 208, 202], [4, 95, 211], [43, 140, 68], [86, 158, 55], [6, 102, 52], [211, 88, 2], [115, 57, 173], [67, 136, 113], [180, 88, 206], [146, 216, 187], [121, 238, 193], [106, 253, 196], [166, 153, 103], [49, 210, 29], [102, 30, 98], [18, 141, 104], [177, 142, 191], [104, 222, 239], [161, 77, 121], [101, 53, 68], [96, 233, 205], [138, 22, 26], [142, 91, 93], [69, 92, 173], [164, 74, 99], [56, 182, 54], [110, 136, 124], [81, 168, 141], [176, 233, 71], [184, 144, 198], [18, 38, 153], [9, 226, 249], [181, 130, 186], [198, 237, 118], [6, 28, 94], [193, 191, 120], [139, 222, 152], [14, 7, 180], [8, 96, 161], [71, 60, 149], [11, 82, 84], [85, 219, 99], [6, 87, 19], [149, 158, 47], [151, 217, 87], [180, 113, 174], [166, 153, 238], [18, 193, 135], [49, 146, 209], [181, 175, 232], [53, 253, 59], [143, 88, 168], [144, 141, 99], [49, 222, 81], [136, 146, 215], [230, 40, 45], [184, 32, 92], [126, 141, 113], [211, 154, 125], [208, 146, 48], [180, 23, 33], [82, 227, 86], [133, 7, 224], [231, 254, 47], [5, 192, 122], [237, 217, 189], [143, 180, 124], [134, 57, 159], [103, 75, 17], [34, 78, 76], [236, 246, 158], [222, 137, 123], [55, 248, 117], [1, 251, 195], [218, 248, 221], [53, 52, 253], [25, 27, 153], [175, 255, 81], [153, 252, 240], [97, 96, 221], [172, 209, 239], [87, 110, 25], [33, 111, 247], [98, 254, 208], [53, 22, 86], [119, 199, 201], [101, 18, 184], [123, 92, 221], [64, 44, 155], [165, 5, 95], [89, 160, 151], [35, 40, 230], [238, 253, 3], [38, 164, 65], [146, 238, 25], [80, 144, 75], [165, 11, 210], [123, 46, 131], [121, 118, 69], [71, 43, 131], [120, 92, 169], [88, 79, 41], [211, 59, 208], [78, 129, 89], [91, 49, 123], [221, 82, 46], [233, 133, 144], [79, 142, 121], [110, 75, 157], [133, 131, 255], [229, 18, 45], [247, 210, 37], [103, 210, 117], [209, 68, 148], [168, 112, 222], [49, 6, 170], [251, 66, 121], [88, 106, 97], [89, 233, 16], [113, 187, 228], [38, 109, 167], [103, 53, 205], [146, 49, 98], [80, 63, 171], [114, 198, 240], [152, 133, 57], [133, 83, 253], [71, 166, 144], [182, 164, 223], [154, 115, 240], [206, 189, 235], [242, 96, 89], [147, 54, 188], [202, 158, 143], [28, 168, 72], [152, 152, 190], [175, 60, 206], [172, 77, 227], [2, 34, 170], [108, 183, 117], [219, 172, 113], [250, 206, 186], [202, 50, 102], [91, 165, 224], [162, 5, 41], [185, 216, 145], [101, 9, 209], [181, 3, 250], [79, 46, 20], [219, 38, 48], [173, 56, 150], [248, 250, 109], [180, 54, 42], [76, 105, 80], [45, 81, 77], [207, 27, 149], [202, 90, 68], [117, 40, 34], [101, 85, 125], [181, 62, 2], [126, 27, 26], [175, 10, 117], [34, 148, 89], [251, 112, 155], [88, 233, 188], [2, 236, 200], [112, 177, 72], [192, 143, 121], [197, 221, 120], [167, 199, 64], [183, 237, 72], [1, 146, 122], [180, 241, 149], [14, 180, 128], [18, 245, 95], [156, 162, 77], [209, 129, 226], [227, 175, 56], [228, 56, 151], [233, 172, 134], [108, 130, 207], [7, 148, 106], [102, 96, 227], [127, 138, 26], [156, 11, 156], [8, 160, 236], [82, 151, 184], [193, 212, 204], [23, 45, 221], [253, 141, 154], [172, 239, 26], [16, 119, 96], [25, 10, 3], [52, 52, 69], [250, 212, 194], [22, 66, 119], [233, 216, 224], [194, 211, 38], [169, 116, 123], [40, 112, 139], [182, 113, 68], [191, 147, 111], [170, 157, 217], [62, 24, 254], [129, 35, 0], [150, 217, 121], [236, 59, 124], [36, 14, 95], [186, 64, 240], [100, 77, 205], [212, 36, 217], [52, 45, 191], [119, 18, 71], [65, 112, 137], [156, 109, 169], [16, 144, 81], [199, 5, 39], [160, 240, 247], [130, 136, 210], [177, 51, 159], [147, 21, 83], [200, 46, 53], [151, 234, 135], [37, 145, 56], [111, 135, 177], [118, 219, 220], [217, 57, 120], [104, 213, 216], [136, 63, 187], [146, 89, 242], [203, 232, 114], [85, 163, 60], [104, 74, 100], [17, 255, 254], [173, 231, 8], [2, 229, 124], [218, 119, 150], [47, 134, 161], [127, 205, 0], [186, 63, 82], [69, 94, 210], [210, 196, 100], [193, 1, 242], [178, 21, 62], [153, 19, 247], [20, 140, 62], [219, 184, 207], [36, 7, 197], [188, 211, 88], [54, 74, 57], [168, 131, 232], [160, 121, 247], [66, 252, 88], [129, 161, 135], [210, 52, 167], [74, 82, 76], [207, 239, 131], [53, 217, 138], [140, 149, 255], [235, 4, 147], [225, 224, 206], [95, 30, 183], [253, 237, 214], [243, 120, 139], [99, 192, 27], [152, 47, 80], [159, 230, 222], [151, 183, 61], [194, 202, 82], [217, 108, 189], [5, 120, 195], [15, 49, 244], [125, 95, 208], [142, 95, 205], [74, 211, 28], [179, 105, 132], [119, 17, 74], [145, 79, 143], [227, 189, 44], [83, 3, 183], [252, 41, 68], [101, 70, 244], [35, 179, 30], [28, 44, 73], [3, 42, 157], [56, 27, 26], [245, 168, 226], [52, 47, 58], [90, 42, 36], [137, 34, 185], [246, 232, 2], [146, 235, 195], [166, 248, 218], [20, 78, 0], [73, 206, 126], [225, 136, 1], [134, 147, 98], [15, 91, 239], [220, 72, 154], [112, 135, 169], [104, 138, 170], [60, 163, 71], [49, 164, 84], [124, 241, 148], [250, 163, 20], [102, 39, 35], [183, 152, 254], [40, 161, 100], [140, 93, 119], [252, 35, 252], [10, 38, 87], [112, 50, 116], [129, 178, 253], [206, 247, 254], [122, 109, 106], [195, 39, 138], [138, 159, 23], [166, 113, 11], [253, 72, 247], [128, 75, 79], [243, 132, 172], [17, 79, 54], [36, 1, 221], [102, 210, 253], [192, 135, 51], [75, 94, 70], [253, 85, 126], [124, 84, 187], [207, 63, 98], [86, 206, 198], [112, 135, 85], [94, 19, 67], [166, 184, 215], [104, 112, 105], [110, 212, 104], [203, 255, 64], [142, 153, 98], [105, 209, 195], [172, 231, 189], [253, 30, 51], [39, 218, 89], [123, 81, 188], [17, 47, 44], [254, 225, 136], [33, 20, 207], [220, 196, 9], [103, 86, 89], [59, 89, 40], [110, 131, 76], [160, 13, 198], [235, 213, 56], [243, 33, 80], [11, 250, 243], [239, 49, 178], [1, 158, 167], [153, 120, 241], [86, 151, 60], [246, 31, 148], [232, 54, 14], [213, 225, 238], [65, 198, 219], [54, 249, 235], [22, 212, 84], [69, 246, 173], [43, 25, 67], [194, 84, 50], [20, 89, 80], [4, 249, 141], [251, 21, 63], [84, 124, 125], [94, 50, 52], [103, 13, 110], [183, 235, 245], [243, 229, 247], [56, 167, 10], [115, 53, 32], [206, 186, 236], [63, 35, 153], [183, 193, 58], [101, 23, 148], [234, 214, 218], [136, 171, 159], [196, 153, 133], [136, 190, 252], [9, 31, 100], [142, 157, 71], [40, 163, 18], [8, 13, 151], [225, 74, 98], [95, 1, 142], [231, 111, 121], [81, 61, 90], [151, 113, 228], [90, 131, 238], [69, 72, 5], [31, 3, 77], [51, 169, 231], [71, 227, 80], [110, 33, 189], [126, 25, 200], [191, 53, 180], [186, 192, 109], [160, 99, 89], [18, 101, 48], [91, 76, 121], [71, 19, 121], [38, 91, 168], [222, 39, 27], [19, 192, 144], [99, 150, 4], [174, 232, 186], [180, 223, 32], [128, 37, 50], [171, 29, 210], [107, 134, 30], [223, 181, 14], [77, 122, 99], [224, 51, 207], [208, 99, 170], [62, 131, 202], [201, 101, 179], [132, 62, 213], [91, 136, 43], [5, 245, 130], [130, 100, 17], [231, 132, 74], [210, 41, 93], [174, 79, 4], [235, 36, 128], [227, 211, 116], [33, 202, 64], [235, 203, 243], [121, 46, 177], [169, 105, 145], [249, 233, 173], [105, 44, 174], [118, 208, 26], [152, 18, 127], [20, 71, 40], [232, 92, 155], [171, 65, 124], [150, 198, 121], [36, 157, 10], [69, 249, 25], [153, 100, 201], [135, 226, 153], [140, 45, 111], [122, 108, 100], [92, 47, 176], [39, 45, 213], [252, 174, 33], [182, 211, 101], [146, 17, 253], [32, 27, 50], [105, 65, 67], [169, 21, 112], [246, 53, 217], [240, 76, 103], [26, 117, 137], [205, 208, 156], [167, 45, 84], [111, 194, 167], [181, 220, 8], [140, 4, 143], [172, 151, 61], [240, 30, 111], [238, 77, 26], [9, 254, 185], [235, 167, 37], [241, 164, 194], [71, 133, 170], [143, 154, 94], [79, 18, 111], [1, 207, 212], [150, 170, 216], [234, 183, 151], [171, 119, 221], [55, 12, 134], [111, 85, 149], [2, 121, 102], [248, 149, 156], [146, 7, 97], [91, 20, 126], [55, 1, 96], [196, 113, 31], [81, 125, 201], [204, 75, 135], [163, 246, 88], [112, 231, 4], [250, 244, 141], [152, 14, 124], [52, 48, 143], [34, 244, 161], [110, 165, 240], [190, 131, 17], [211, 146, 13], [128, 80, 195], [101, 76, 234], [21, 68, 67], [177, 120, 7], [189, 108, 63], [22, 91, 3], [139, 84, 10], [243, 22, 30], [204, 242, 180], [121, 63, 153], [249, 250, 13], [137, 15, 123], [250, 169, 28], [124, 165, 43], [253, 125, 112], [151, 246, 60], [245, 116, 180], [171, 5, 162], [93, 194, 224], [106, 88, 72], [113, 58, 154], [112, 219, 72], [133, 194, 96], [255, 158, 179], [30, 172, 242], [201, 61, 59], [44, 100, 86], [58, 11, 67], [220, 64, 36], [120, 91, 250], [40, 37, 226], [130, 22, 45], [70, 101, 51], [23, 71, 127], [62, 50, 152], [100, 130, 21], [251, 69, 253], [148, 223, 116], [148, 107, 167], [189, 38, 38], [206, 226, 12], [148, 254, 205], [250, 128, 48], [95, 122, 205], [212, 174, 166], [89, 196, 146], [188, 93, 98], [83, 197, 148], [104, 34, 201], [37, 95, 186], [172, 217, 129], [182, 110, 249], [226, 106, 111], [199, 58, 127], [21, 97, 115], [158, 87, 63], [84, 74, 140], [23, 239, 148], [33, 184, 6], [245, 23, 20], [243, 239, 104], [28, 213, 169], [92, 122, 139], [6, 173, 175], [31, 101, 19], [22, 169, 120], [169, 207, 225], [80, 13, 9], [175, 89, 12], [138, 173, 10], [116, 125, 207], [208, 235, 63], [149, 95, 159], [245, 108, 121], [214, 27, 95], [58, 218, 19], [139, 202, 232], [137, 250, 224], [32, 125, 214], [53, 249, 225], [106, 151, 144], [54, 26, 49], [197, 248, 77], [19, 179, 187], [93, 59, 231], [184, 95, 93], [234, 201, 100], [182, 163, 67], [8, 135, 125], [189, 41, 70], [25, 175, 72], [141, 163, 88], [250, 1, 68], [250, 250, 122], [218, 167, 33], [9, 185, 100], [240, 117, 83], [230, 176, 129], [94, 191, 182], [112, 164, 193], [106, 226, 3], [195, 154, 67], [99, 142, 158], [93, 248, 125], [252, 98, 28], [24, 165, 99], [118, 151, 183], [104, 219, 111], [117, 120, 150], [74, 115, 215], [211, 63, 53], [144, 86, 170], [118, 118, 81], [178, 252, 97], [21, 191, 228], [177, 172, 51], [54, 226, 42], [145, 49, 34], [65, 191, 26], [238, 148, 9], [216, 220, 122], [220, 97, 139], [24, 27, 55], [29, 246, 21], [236, 54, 137], [33, 92, 255], [118, 70, 142], [93, 26, 114], [146, 230, 140], [221, 44, 51], [235, 173, 20], [77, 43, 126], [163, 16, 165], [185, 76, 30], [76, 115, 236], [208, 213, 160], [165, 90, 123], [143, 73, 27], [107, 245, 89], [113, 123, 235], [55, 205, 154], [155, 251, 106], [184, 24, 132], [222, 54, 178], [34, 42, 183], [127, 111, 228], [142, 250, 91], [230, 34, 237], [253, 152, 59], [112, 150, 199], [64, 239, 1], [197, 255, 168], [216, 154, 111], [124, 46, 40], [52, 169, 29], [246, 32, 162], [60, 68, 100], [80, 37, 217], [172, 55, 152], [176, 64, 55], [146, 17, 85], [164, 145, 63], [116, 26, 138], [25, 57, 13], [35, 161, 205], [38, 14, 96], [205, 17, 41], [180, 1, 56], [208, 18, 128], [7, 247, 84], [39, 254, 245], [179, 216, 195], [109, 128, 180], [113, 89, 253], [31, 32, 8], [145, 150, 80], [227, 0, 77], [127, 42, 36], [23, 168, 86], [189, 115, 47], [65, 204, 39], [229, 130, 37], [203, 113, 31], [200, 71, 253], [252, 29, 187], [179, 177, 59], [188, 191, 153], [153, 91, 21], [28, 47, 57], [241, 227, 158], [230, 220, 233], [130, 250, 242], [250, 41, 127], [250, 211, 46], [26, 47, 153], [206, 113, 36], [215, 111, 84], [164, 167, 160], [164, 180, 120], [244, 179, 191], [124, 15, 230], [93, 171, 226], [192, 223, 65], [155, 7, 84], [8, 193, 174], [57, 142, 87], [25, 82, 192], [88, 109, 35], [229, 19, 255], [109, 208, 135], [218, 123, 163], [118, 18, 251], [192, 213, 8], [43, 236, 205], [59, 31, 145], [80, 113, 175], [79, 22, 37], [227, 79, 169], [70, 86, 119], [71, 234, 233], [41, 28, 152], [251, 227, 242], [238, 64, 201], [91, 83, 171], [113, 211, 102], [21, 188, 52], [240, 218, 244], [171, 149, 229], [172, 2, 242], [181, 15, 159], [106, 220, 149], [8, 238, 246], [50, 252, 15], [15, 235, 87], [184, 228, 134], [130, 102, 9], [2, 96, 251], [79, 75, 240], [130, 16, 34], [120, 62, 191], [169, 28, 9], [104, 14, 214], [195, 145, 77], [10, 112, 136], [86, 202, 26], [133, 74, 46], [27, 232, 138], [145, 246, 252], [234, 107, 137], [33, 130, 207], [163, 121, 30], [46, 150, 56], [216, 124, 71], [204, 216, 126], [136, 166, 14], [101, 107, 209], [146, 184, 55], [29, 133, 139], [196, 176, 141], [174, 197, 239], [162, 89, 246], [84, 61, 99], [54, 75, 213], [106, 226, 36], [33, 89, 129], [196, 244, 168], [240, 35, 197], [142, 61, 149], [109, 137, 44], [115, 137, 50], [137, 19, 140], [220, 197, 178], [8, 57, 22], [125, 46, 123], [236, 126, 252], [6, 43, 205], [65, 216, 97], [121, 191, 83], [114, 223, 105], [230, 223, 121], [172, 6, 179], [207, 27, 234], [165, 138, 235], [168, 209, 197], [246, 88, 103], [95, 154, 77], [53, 46, 236], [62, 241, 80], [244, 161, 160], [230, 130, 100], [163, 233, 172], [49, 95, 163], [210, 136, 184], [235, 153, 76], [165, 109, 98], [111, 82, 81], [86, 176, 38], [213, 250, 212], [236, 239, 167], [59, 152, 180], [200, 248, 82], [159, 34, 84], [179, 120, 10], [162, 30, 79], [86, 149, 162], [205, 230, 109], [216, 236, 4], [222, 4, 183], [252, 95, 108], [15, 191, 10], [18, 70, 37], [224, 9, 233], [223, 18, 138], [204, 134, 111], [151, 33, 1], [191, 107, 241], [231, 22, 29], [163, 77, 91], [144, 248, 239], [159, 93, 176], [0, 59, 88], [8, 15, 45], [171, 73, 181], [108, 45, 208], [9, 2, 120], [36, 203, 159], [128, 6, 27], [118, 109, 39], [22, 166, 248], [189, 5, 99], [130, 180, 44], [95, 30, 84], [167, 207, 71], [91, 253, 212], [222, 152, 218], [37, 116, 151], [66, 136, 253], [135, 82, 230], [187, 237, 35], [178, 1, 102], [217, 24, 237], [15, 117, 167], [152, 30, 172], [124, 107, 248], [131, 163, 251], [167, 249, 89], [128, 104, 31], [182, 122, 88], [17, 60, 104], [163, 157, 180], [83, 169, 66], [231, 230, 225], [124, 79, 228], [108, 106, 248], [189, 72, 189], [140, 100, 126], [19, 93, 1], [243, 223, 39], [130, 106, 190], [66, 0, 133], [193, 46, 33], [228, 123, 99], [73, 14, 94], [24, 52, 23], [37, 96, 15], [94, 149, 15], [236, 113, 198], [107, 23, 175], [162, 232, 181], [71, 234, 176], [106, 0, 175], [97, 197, 111], [162, 143, 61], [218, 18, 22], [64, 81, 53], [64, 213, 30], [19, 196, 127], [116, 176, 31], [169, 97, 154], [222, 49, 46], [114, 66, 165], [110, 215, 153], [21, 66, 86], [88, 139, 230], [28, 15, 136], [230, 38, 144], [29, 47, 27], [39, 103, 63], [55, 3, 45], [205, 49, 72], [237, 25, 50], [203, 73, 46], [19, 8, 218], [193, 119, 206], [237, 252, 46], [50, 145, 229], [208, 197, 111], [103, 124, 143], [211, 157, 129], [46, 205, 127], [56, 52, 79], [109, 82, 213], [109, 9, 89], [161, 195, 54], [228, 30, 103], [27, 106, 222], [147, 164, 0], [206, 52, 26], [242, 86, 1], [28, 183, 58], [2, 19, 157], [38, 212, 159], [215, 25, 223], [24, 165, 176], [238, 111, 209], [181, 189, 252], [184, 181, 199], [149, 177, 88], [216, 121, 122], [189, 56, 97], [153, 114, 29], [117, 62, 85], [183, 145, 98], [71, 18, 189], [97, 175, 230], [104, 86, 224], [13, 25, 159], [20, 76, 237], [108, 210, 56], [225, 98, 7], [148, 138, 233], [222, 199, 184], [194, 244, 244], [206, 51, 125], [171, 223, 16], [101, 58, 238], [182, 142, 154], [195, 59, 75], [2, 76, 11], [29, 132, 186], [243, 28, 154], [19, 146, 212], [52, 236, 162], [236, 91, 15], [213, 201, 14], [114, 110, 111], [248, 22, 20], [86, 21, 200], [92, 243, 153], [34, 159, 50], [142, 77, 181], [192, 250, 149], [71, 48, 38], [127, 129, 159], [94, 36, 202], [117, 129, 175], [229, 239, 251], [75, 6, 49], [32, 37, 164], [47, 182, 159], [41, 50, 147], [161, 192, 164], [53, 151, 48], [216, 231, 205], [87, 246, 64], [102, 3, 48], [108, 205, 80], [151, 246, 224], [158, 233, 113], [163, 127, 202], [69, 17, 7], [124, 89, 105], [162, 80, 61], [180, 70, 240], [149, 156, 226], [113, 137, 203], [91, 178, 192], [149, 185, 49], [178, 118, 183], [26, 130, 183], [183, 36, 36], [162, 215, 231], [164, 222, 236], [137, 3, 123], [105, 148, 176], [70, 84, 76], [139, 94, 137], [138, 86, 158], [144, 89, 225], [223, 139, 7], [185, 174, 35], [163, 56, 5], [183, 101, 227], [138, 193, 29], [157, 140, 50], [63, 246, 191], [73, 36, 213], [205, 200, 114], [9, 224, 72], [227, 214, 144], [172, 122, 84], [43, 204, 170], [109, 140, 215], [96, 64, 212], [111, 176, 89], [186, 88, 110], [214, 176, 115], [152, 127, 149], [247, 25, 6], [36, 126, 46], [251, 55, 234], [110, 233, 152], [106, 87, 149], [103, 94, 103], [106, 253, 149], [69, 42, 159], [184, 217, 135], [71, 64, 13], [86, 76, 104], [220, 195, 157], [204, 61, 113], [105, 83, 136], [15, 28, 247], [113, 203, 133], [201, 95, 167], [201, 119, 34], [13, 163, 38], [163, 143, 62], [241, 50, 34], [77, 77, 195], [248, 177, 130], [66, 42, 127], [78, 11, 238], [36, 252, 251], [159, 156, 170], [212, 175, 138], [187, 113, 74], [241, 186, 59], [191, 54, 215], [18, 52, 198], [220, 101, 221], [227, 39, 33], [5, 42, 44], [219, 154, 64], [148, 183, 8], [40, 71, 161], [28, 26, 143], [191, 122, 7], [21, 162, 167], [239, 170, 97], [29, 23, 38], [227, 94, 154], [142, 254, 118], [34, 100, 36], [170, 211, 7], [10, 214, 81], [73, 46, 62], [59, 89, 201], [31, 237, 13], [35, 105, 177], [223, 174, 243], [59, 93, 238], [162, 228, 36], [224, 225, 144], [152, 58, 195], [224, 81, 73], [72, 102, 17], [151, 254, 202], [220, 181, 172], [88, 183, 19], [114, 189, 109], [111, 123, 77], [129, 144, 206], [238, 185, 139], [43, 78, 158], [163, 100, 108], [81, 39, 28], [2, 101, 176], [203, 157, 169], [37, 122, 196], [115, 254, 165], [123, 186, 51], [9, 187, 9], [183, 132, 182], [198, 241, 155], [105, 58, 56], [99, 105, 96], [25, 21, 108], [184, 50, 48], [222, 210, 134], [103, 31, 201], [129, 235, 148], [253, 70, 243], [188, 219, 72], [129, 9, 24], [4, 153, 223], [136, 241, 179], [36, 188, 184], [196, 191, 180], [8, 227, 196], [67, 242, 115], [181, 152, 222], [32, 219, 99], [53, 178, 43], [49, 113, 55], [43, 196, 206], [104, 24, 72], [214, 100, 106], [78, 208, 129], [21, 225, 152], [59, 177, 29], [186, 103, 108], [165, 253, 124], [25, 215, 212], [239, 76, 233], [2, 124, 71], [179, 7, 148], [98, 78, 132], [66, 65, 15], [161, 199, 141], [50, 204, 23], [101, 117, 229], [35, 199, 152], [208, 23, 97], [147, 2, 65], [59, 245, 186], [239, 209, 181], [102, 129, 167], [62, 116, 179], [171, 117, 43], [174, 141, 119], [188, 234, 90], [184, 5, 133], [245, 60, 78], [98, 58, 200], [26, 132, 100], [212, 41, 55], [250, 182, 156], [105, 205, 16], [247, 122, 184], [129, 4, 228], [113, 146, 151], [122, 23, 64], [211, 228, 242], [4, 175, 96], [104, 211, 218], [80, 102, 20], [20, 131, 136], [251, 9, 113], [26, 112, 101], [185, 22, 24], [91, 183, 233], [214, 255, 63], [81, 142, 220], [53, 115, 226], [49, 40, 44], [136, 18, 10], [120, 150, 132], [247, 107, 22], [0, 248, 212], [120, 151, 2], [8, 43, 42], [54, 238, 9], [127, 211, 153], [152, 135, 135], [147, 85, 3], [191, 42, 137], [231, 40, 74], [200, 179, 17], [241, 195, 245], [56, 152, 244], [198, 31, 123], [74, 109, 210], [97, 17, 190], [80, 201, 90], [169, 215, 31], [57, 69, 63], [144, 247, 236], [77, 43, 197], [213, 6, 126], [200, 111, 110], [142, 255, 25], [196, 71, 85], [115, 213, 228], [226, 232, 26], [82, 128, 37], [77, 46, 165], [140, 243, 149], [100, 115, 110], [68, 49, 220], [212, 184, 58], [161, 187, 174], [75, 59, 11], [234, 19, 100], [129, 130, 245], [139, 25, 234], [79, 168, 182], [247, 147, 57], [102, 164, 49], [128, 253, 165], [115, 104, 42], [48, 62, 72], [125, 106, 235], [30, 223, 195], [220, 29, 14], [174, 44, 254], [29, 137, 236], [37, 154, 147], [94, 79, 158], [108, 244, 122], [82, 198, 102], [232, 105, 108], [53, 206, 231], [70, 65, 52], [77, 72, 67], [253, 174, 39], [151, 182, 131], [41, 47, 198], [215, 120, 121], [224, 172, 69], [197, 221, 234], [162, 101, 183], [25, 35, 34], [180, 64, 52], [24, 33, 143], [62, 75, 194], [44, 25, 16], [240, 170, 48], [83, 64, 254], [39, 241, 86], [124, 215, 3], [179, 205, 241], [46, 19, 104], [168, 19, 144], [230, 139, 168], [196, 222, 30], [204, 231, 186], [51, 125, 136], [171, 32, 113], [172, 60, 168], [100, 169, 220], [102, 116, 97], [202, 252, 60], [11, 93, 108], [216, 231, 140], [235, 60, 224], [16, 124, 177], [111, 204, 246], [65, 235, 46], [181, 161, 31], [74, 5, 114], [171, 193, 60], [131, 167, 199], [144, 168, 101], [134, 23, 5], [44, 10, 103], [93, 88, 82], [89, 217, 137], [216, 207, 125], [179, 31, 191], [48, 161, 246], [175, 111, 83], [169, 173, 96], [100, 48, 110], [133, 121, 28], [57, 14, 113], [43, 91, 111], [175, 98, 127], [2, 117, 153], [126, 1, 158], [200, 120, 251], [174, 144, 92], [182, 198, 206], [215, 189, 248], [141, 77, 205], [228, 21, 103], [49, 230, 7], [130, 57, 38], [31, 19, 228], [198, 101, 110], [37, 194, 32], [170, 225, 100], [67, 78, 141], [235, 225, 177], [189, 94, 30], [48, 76, 62], [189, 152, 190], [53, 208, 186], [161, 21, 232], [176, 2, 181], [68, 139, 126], [127, 90, 59], [26, 34, 214], [174, 56, 103], [211, 133, 101], [43, 196, 186], [231, 203, 238], [128, 40, 5], [4, 23, 229], [47, 4, 68], [125, 163, 234], [91, 180, 8], [212, 144, 217], [239, 129, 149], [72, 120, 233], [93, 208, 10], [188, 204, 61], [123, 132, 226], [111, 143, 25], [184, 244, 200], [186, 63, 65], [251, 141, 14], [61, 81, 1], [19, 153, 49], [37, 101, 240], [176, 236, 183], [51, 29, 118], [148, 107, 154], [160, 129, 76], [20, 85, 57], [211, 134, 206], [71, 4, 162], [166, 249, 191], [213, 155, 32], [111, 179, 3], [239, 253, 73], [28, 192, 126], [26, 235, 14], [252, 66, 245], [44, 247, 133], [108, 144, 247], [203, 216, 52], [160, 45, 235], [9, 230, 38], [199, 126, 99], [250, 155, 168], [122, 2, 141], [114, 121, 57], [62, 138, 159], [247, 212, 185], [218, 236, 140], [128, 40, 66], [227, 134, 196], [96, 100, 229], [20, 13, 102], [215, 234, 144], [109, 31, 146], [22, 184, 247], [50, 175, 125], [73, 6, 167], [89, 12, 69], [253, 46, 250], [252, 112, 238], [25, 69, 34], [174, 76, 201], [69, 171, 177], [185, 245, 139], [149, 154, 223], [197, 103, 141], [19, 86, 165], [185, 140, 180], [63, 101, 250], [2, 24, 64], [54, 7, 244], [61, 229, 196], [203, 118, 133], [70, 43, 244], [18, 113, 237], [164, 101, 89], [35, 223, 77], [30, 76, 254], [94, 81, 56], [211, 118, 131], [157, 196, 117], [125, 172, 222], [126, 252, 192], [43, 61, 113], [145, 97, 100], [186, 177, 147], [16, 207, 247], [130, 96, 194], [53, 226, 26], [124, 2, 130], [122, 87, 101], [3, 124, 28], [159, 20, 209], [59, 165, 102], [98, 124, 169], [26, 193, 249], [169, 224, 150], [47, 103, 50], [167, 176, 156], [47, 222, 79], [177, 2, 247], [179, 241, 139], [45, 202, 16], [50, 134, 119], [89, 251, 32], [16, 66, 43], [181, 231, 136], [160, 10, 48], [11, 171, 39], [163, 231, 220], [133, 219, 157], [132, 67, 241], [180, 209, 195], [223, 48, 57], [161, 95, 166], [22, 45, 194], [4, 23, 169], [141, 177, 166], [111, 67, 1], [123, 41, 216], [105, 250, 230], [202, 198, 253], [170, 70, 163], [166, 149, 168], [6, 29, 100], [206, 162, 227], [160, 43, 251], [5, 31, 73], [119, 202, 199], [223, 149, 114], [35, 169, 133], [223, 167, 253], [96, 114, 99], [95, 172, 161], [250, 170, 143], [65, 7, 144], [208, 132, 246], [109, 225, 91], [92, 141, 178], [149, 123, 186], [183, 157, 7], [195, 6, 102], [39, 235, 108], [123, 16, 80], [27, 77, 124], [194, 30, 191], [146, 46, 51], [228, 158, 171], [197, 80, 6], [128, 32, 247], [183, 233, 185], [188, 22, 228], [224, 63, 0], [249, 231, 226], [166, 21, 131], [233, 37, 140], [145, 120, 95], [98, 229, 28], [38, 240, 151], [13, 206, 155], [135, 139, 142], [140, 126, 42], [213, 21, 113], [53, 199, 191], [253, 97, 131], [189, 71, 138], [57, 64, 65], [147, 252, 87], [62, 25, 85], [33, 45, 225], [225, 174, 233], [156, 239, 185], [8, 194, 38], [18, 241, 57], [78, 92, 206], [220, 55, 82], [231, 119, 55], [70, 39, 92], [165, 161, 115], [19, 244, 1], [212, 161, 195], [17, 217, 166], [168, 131, 172], [207, 230, 252], [51, 47, 114], [226, 108, 24], [69, 247, 117], [116, 174, 227], [99, 109, 120], [67, 69, 199], [210, 116, 60], [172, 116, 255], [129, 47, 105], [180, 168, 2], [154, 191, 122], [194, 87, 243], [37, 4, 214], [77, 201, 35], [165, 210, 161], [11, 69, 106], [23, 14, 120], [96, 154, 179], [245, 76, 60], [144, 229, 109], [239, 30, 151], [221, 131, 31], [110, 180, 169], [76, 93, 177], [250, 12, 51], [31, 78, 162], [120, 153, 250], [177, 36, 26], [15, 112, 113], [39, 56, 85], [255, 35, 55], [255, 84, 220], [245, 32, 158], [70, 171, 60], [137, 215, 204], [173, 24, 93], [104, 215, 165], [165, 232, 107], [15, 39, 131], [13, 145, 224], [116, 251, 42], [110, 111, 47], [243, 94, 161], [138, 101, 140], [173, 118, 161], [129, 209, 129], [154, 1, 164], [125, 23, 243], [242, 81, 18], [14, 156, 243], [85, 57, 239], [40, 248, 242], [250, 46, 155], [160, 208, 49], [168, 6, 119], [154, 63, 80], [91, 87, 158], [18, 174, 91], [12, 15, 159], [121, 128, 96], [27, 50, 225], [44, 190, 123], [131, 120, 42], [121, 122, 134], [182, 135, 106], [36, 69, 119], [217, 132, 49], [24, 207, 186], [238, 104, 133], [221, 5, 65], [171, 34, 20], [36, 84, 145], [206, 63, 25], [242, 214, 138], [191, 248, 108], [145, 78, 235], [206, 198, 10], [116, 155, 95], [117, 248, 22], [183, 141, 61], [88, 22, 126], [206, 161, 26], [3, 75, 77], [74, 24, 176], [119, 56, 6], [90, 204, 109], [234, 97, 69], [107, 217, 98], [87, 90, 4], [242, 121, 39], [110, 22, 250], [116, 192, 131], [136, 54, 60], [45, 105, 198], [244, 228, 164], [101, 223, 173], [161, 71, 222], [5, 70, 50], [16, 149, 173], [188, 185, 100], [102, 148, 155], [32, 213, 100], [66, 251, 89], [119, 75, 26], [195, 231, 233], [78, 190, 164], [212, 9, 252], [124, 147, 116], [146, 65, 28], [242, 231, 184], [225, 227, 135], [146, 197, 146], [58, 146, 115], [106, 166, 223], [235, 51, 185], [172, 85, 169], [194, 112, 252], [94, 158, 154], [25, 49, 195], [21, 145, 12], [245, 26, 139], [114, 207, 75], [192, 143, 115], [235, 50, 249], [149, 51, 0], [85, 103, 94], [108, 133, 197], [242, 13, 184], [139, 236, 16], [17, 115, 254], [228, 98, 195], [245, 143, 3], [252, 251, 192], [169, 93, 122], [141, 80, 211], [101, 246, 46], [126, 197, 6], [72, 188, 64], [54, 68, 201], [47, 78, 114], [184, 107, 163], [116, 243, 155], [60, 170, 74], [122, 220, 251], [254, 194, 96], [231, 237, 1], [112, 105, 126], [31, 107, 89], [143, 152, 242], [177, 114, 31], [162, 109, 114], [190, 99, 208], [243, 246, 188], [219, 99, 66], [6, 6, 249], [13, 108, 111], [250, 92, 8], [166, 113, 197], [50, 27, 35], [200, 45, 28], [87, 249, 15], [92, 163, 245], [206, 31, 145], [193, 147, 197], [29, 147, 252], [192, 189, 9], [221, 120, 150], [17, 76, 92], [117, 187, 0], [131, 173, 200], [125, 163, 55], [34, 209, 185], [95, 194, 7], [244, 47, 10], [227, 41, 193], [173, 78, 66], [78, 144, 3], [113, 221, 13], [241, 223, 45], [199, 26, 255], [28, 145, 216], [206, 102, 20], [181, 162, 5], [32, 74, 39], [51, 255, 77], [156, 54, 144], [122, 23, 43], [73, 214, 204], [201, 194, 127], [158, 157, 115], [82, 24, 157], [167, 191, 77], [91, 51, 80], [107, 157, 56], [190, 145, 37], [9, 178, 253], [156, 133, 187], [68, 153, 252], [122, 191, 41], [40, 43, 212], [243, 252, 208], [96, 145, 138], [227, 150, 213], [90, 155, 186], [182, 118, 23], [233, 111, 39], [193, 178, 24], [143, 52, 251], [110, 253, 201], [160, 243, 112], [139, 54, 229], [18, 94, 125], [235, 61, 100], [121, 117, 59], [168, 235, 142], [17, 232, 29], [152, 173, 14], [165, 180, 207], [139, 63, 125], [253, 111, 8], [48, 171, 137], [186, 119, 75], [77, 130, 35], [189, 181, 172], [138, 185, 5], [177, 71, 78], [255, 124, 96], [59, 224, 57], [157, 68, 33], [224, 230, 145], [203, 76, 217], [19, 214, 170], [212, 23, 230], [174, 219, 102], [46, 178, 105], [188, 50, 120], [126, 237, 114], [175, 98, 197], [150, 194, 25], [199, 236, 2], [179, 59, 171], [69, 165, 120], [201, 70, 28], [75, 181, 18], [34, 254, 116], [191, 77, 173], [30, 138, 34], [14, 11, 18], [210, 198, 75], [130, 198, 53], [227, 159, 172], [70, 24, 91], [51, 60, 93], [190, 80, 219], [181, 200, 176], [179, 172, 240], [221, 184, 23], [79, 4, 93], [53, 242, 252], [233, 169, 45], [65, 133, 97], [240, 150, 87], [1, 94, 20], [223, 16, 244], [196, 238, 208], [128, 12, 41], [35, 90, 250], [205, 215, 145], [48, 178, 237], [87, 48, 61], [104, 136, 5], [234, 17, 177], [69, 158, 80], [170, 182, 169], [48, 151, 124], [44, 52, 131], [106, 123, 181], [84, 1, 98], [47, 50, 80], [177, 195, 177], [213, 75, 197], [12, 3, 81], [14, 20, 38], [31, 109, 78], [242, 87, 120], [2, 244, 6], [180, 25, 171], [2, 120, 237], [164, 103, 233], [31, 30, 217], [139, 200, 136], [43, 156, 210], [77, 79, 153], [199, 133, 241], [16, 70, 137], [238, 101, 139], [2, 250, 107], [122, 199, 184], [255, 92, 75], [130, 5, 190], [114, 218, 105], [185, 120, 89], [119, 125, 87], [169, 181, 5], [73, 39, 79], [152, 147, 249], [80, 234, 62], [125, 166, 127], [46, 174, 126], [67, 165, 223], [109, 199, 193], [245, 172, 232], [24, 20, 248], [50, 164, 190], [82, 219, 215], [220, 250, 84], [190, 12, 200], [57, 137, 148], [79, 1, 171], [200, 135, 255], [207, 110, 4], [194, 82, 18], [172, 173, 230], [210, 176, 239], [59, 47, 128], [81, 115, 198], [84, 3, 221], [236, 99, 168], [42, 143, 203], [184, 154, 107], [145, 218, 113], [234, 177, 142], [177, 157, 11], [195, 77, 113], [155, 146, 207], [183, 83, 71], [245, 193, 179], [60, 108, 229], [55, 169, 33], [241, 26, 148], [64, 109, 0], [4, 109, 226], [239, 141, 122], [89, 56, 89], [205, 153, 130], [109, 79, 144], [212, 235, 40], [78, 186, 22], [219, 217, 225], [162, 210, 189], [167, 99, 159], [28, 183, 136], [232, 71, 136], [125, 64, 151], [63, 14, 170], [151, 240, 98], [135, 232, 141], [30, 99, 47], [205, 236, 226], [161, 31, 207], [41, 153, 151], [128, 240, 68], [106, 94, 98], [182, 43, 65], [186, 135, 108], [247, 16, 181], [138, 120, 223], [59, 6, 177], [10, 236, 187], [240, 245, 87], [169, 241, 80], [171, 176, 151], [174, 8, 219], [36, 76, 182], [105, 21, 42], [66, 37, 188], [98, 190, 112], [175, 114, 168], [85, 156, 83], [119, 146, 65], [50, 63, 36], [184, 231, 1], [240, 206, 129], [190, 195, 225], [228, 21, 200], [95, 221, 68], [14, 140, 101], [13, 80, 174], [151, 10, 42], [60, 247, 230], [131, 32, 201], [205, 18, 235], [136, 42, 43], [56, 48, 171], [181, 246, 193], [62, 107, 21], [32, 161, 37], [89, 155, 154], [98, 138, 50], [40, 83, 89], [134, 31, 88], [43, 92, 99], [50, 221, 107], [252, 73, 177], [139, 77, 75], [203, 170, 200], [254, 131, 54], [90, 251, 7], [241, 124, 107], [140, 87, 3], [64, 146, 238], [1, 54, 146], [128, 138, 147], [203, 246, 54], [49, 191, 15], [171, 254, 125], [108, 83, 77], [198, 109, 51], [184, 240, 193], [191, 128, 61], [35, 233, 115], [47, 161, 241], [178, 67, 50], [225, 5, 75], [75, 71, 153], [42, 209, 196], [253, 149, 160], [161, 95, 151], [101, 77, 101], [44, 122, 206], [122, 144, 58], [168, 66, 234], [15, 26, 99], [174, 130, 54], [20, 185, 89], [2, 48, 65], [169, 121, 246], [9, 101, 218], [60, 169, 17], [172, 34, 110], [223, 176, 56], [76, 65, 166], [61, 55, 252], [155, 136, 30], [112, 114, 185], [92, 225, 155], [225, 254, 71], [45, 138, 76], [222, 35, 17], [250, 225, 106], [99, 30, 203], [133, 103, 10], [151, 92, 250], [253, 213, 59], [58, 119, 27], [80, 133, 202], [167, 207, 153], [145, 7, 90], [92, 108, 125], [72, 134, 177], [251, 80, 233], [202, 128, 231], [246, 14, 98], [65, 168, 148], [231, 194, 214], [73, 28, 251], [58, 10, 42], [54, 149, 53], [76, 56, 15], [197, 209, 146], [195, 208, 52], [183, 31, 21], [220, 30, 121], [140, 219, 254], [163, 98, 31], [212, 104, 94], [39, 85, 168], [101, 123, 17], [168, 183, 7], [79, 239, 78], [16, 23, 157], [212, 196, 198], [141, 84, 208], [21, 86, 57], [210, 127, 84], [138, 163, 114], [39, 229, 18], [192, 223, 25], [245, 158, 224], [187, 210, 147], [130, 224, 151], [229, 236, 38], [147, 55, 238], [177, 158, 190], [225, 77, 56], [52, 159, 92], [129, 106, 249], [1, 171, 202], [100, 212, 238], [29, 18, 215], [31, 96, 163], [44, 63, 170], [216, 55, 98], [198, 66, 128], [2, 234, 60], [165, 115, 237], [142, 75, 65], [63, 3, 72], [144, 131, 217], [30, 107, 56], [198, 162, 185], [174, 167, 239], [123, 66, 71], [152, 8, 161], [208, 216, 215], [237, 168, 93], [174, 19, 35], [254, 151, 14], [58, 144, 251], [24, 92, 193], [199, 16, 197], [159, 94, 250], [121, 165, 241], [13, 17, 228], [92, 64, 105], [101, 51, 163], [158, 196, 25], [69, 223, 236], [252, 118, 11], [31, 61, 249], [153, 139, 121], [27, 34, 202], [8, 38, 17], [155, 192, 66], [16, 87, 165], [147, 9, 112], [117, 10, 159], [60, 60, 52], [233, 153, 120], [139, 199, 149], [239, 200, 127], [122, 230, 177], [105, 255, 70], [34, 74, 156], [143, 173, 82], [117, 78, 88], [18, 67, 212], [40, 172, 247], [40, 7, 240], [210, 100, 94], [110, 105, 87], [68, 41, 58], [42, 179, 58], [77, 54, 193], [22, 171, 22], [193, 178, 91], [182, 76, 182], [17, 165, 7], [230, 49, 206], [186, 3, 9], [115, 236, 199], [141, 236, 87], [57, 181, 215], [37, 210, 72], [240, 81, 93], [9, 0, 5], [38, 175, 13], [202, 201, 127], [113, 140, 157], [39, 109, 206], [121, 13, 194], [27, 175, 115], [42, 253, 76], [168, 171, 153], [234, 236, 93], [62, 109, 151], [56, 0, 70], [205, 119, 6], [17, 87, 182], [198, 10, 15], [169, 23, 217], [186, 220, 184], [167, 43, 248], [71, 175, 65], [91, 224, 183], [85, 163, 1], [39, 162, 11], [99, 50, 181], [20, 213, 56], [252, 15, 112], [45, 172, 73], [1, 90, 243], [135, 173, 134], [51, 34, 249], [188, 12, 88], [33, 142, 70], [102, 118, 167], [155, 247, 58], [137, 31, 44], [64, 211, 19], [85, 189, 33], [216, 39, 228], [254, 85, 167], [61, 122, 107], [35, 225, 34], [133, 105, 99], [100, 58, 70], [194, 63, 99], [132, 155, 165], [99, 21, 247], [187, 119, 125], [36, 210, 233], [199, 61, 81], [243, 3, 158], [239, 94, 244], [220, 135, 22], [3, 12, 76], [133, 80, 166], [69, 30, 18], [76, 155, 77], [75, 112, 6], [143, 252, 112], [183, 174, 56], [125, 73, 38], [134, 117, 51], [64, 253, 65], [18, 27, 124], [255, 71, 43], [148, 247, 30], [187, 151, 212], [7, 117, 104], [163, 77, 131], [162, 212, 58], [57, 199, 82], [10, 182, 177], [224, 245, 221], [132, 13, 90], [209, 117, 232], [230, 124, 144], [41, 53, 64], [62, 162, 30], [133, 133, 126], [196, 104, 60], [191, 136, 203], [2, 154, 171], [238, 151, 238], [160, 13, 110], [221, 78, 67], [237, 137, 7], [173, 187, 210], [213, 207, 103], [206, 33, 132], [178, 225, 184], [83, 194, 75], [4, 11, 167], [220, 102, 86], [20, 102, 226], [181, 181, 109], [157, 60, 59], [224, 128, 84], [58, 63, 28], [85, 6, 208], [225, 154, 142], [61, 145, 239], [219, 17, 130], [122, 127, 69], [84, 172, 109], [134, 33, 153], [119, 213, 90], [248, 220, 37], [152, 76, 162], [241, 254, 139], [243, 110, 145], [47, 228, 53], [0, 241, 159], [141, 50, 131], [76, 168, 14], [16, 90, 220], [68, 0, 248], [227, 212, 204], [173, 70, 239], [89, 91, 0], [156, 211, 122], [91, 8, 126], [233, 232, 135], [1, 143, 196], [241, 7, 106], [35, 206, 153], [216, 71, 156], [221, 185, 11], [39, 236, 177], [160, 157, 88], [199, 167, 230], [101, 97, 162], [89, 164, 69], [51, 99, 234], [22, 176, 183], [54, 135, 63], [238, 37, 73], [165, 139, 49], [43, 174, 10], [251, 206, 173], [170, 169, 199], [174, 70, 105], [21, 192, 208], [226, 56, 162], [21, 252, 159], [15, 113, 74], [187, 29, 25], [251, 147, 193], [196, 16, 227], [78, 51, 161], [33, 37, 168], [93, 255, 66], [133, 150, 212], [216, 187, 86], [181, 226, 177], [78, 43, 146], [112, 198, 95], [40, 102, 63], [149, 221, 137], [229, 71, 234], [49, 215, 65], [164, 27, 222], [63, 168, 103], [67, 173, 46], [6, 212, 243], [233, 58, 68], [88, 9, 253], [96, 105, 92], [53, 179, 60], [34, 20, 82], [74, 43, 36], [47, 216, 172], [62, 245, 196], [102, 51, 72], [77, 114, 226], [246, 37, 253], [50, 240, 46], [30, 175, 80], [216, 163, 149], [226, 25, 39], [72, 216, 75], [155, 128, 58], [58, 124, 139], [217, 186, 193], [18, 39, 248], [71, 250, 38], [147, 54, 93], [221, 230, 8], [132, 126, 52], [155, 202, 132], [191, 128, 205], [149, 52, 118], [40, 124, 161], [144, 152, 122], [65, 103, 121], [114, 235, 175], [237, 99, 168], [12, 75, 92], [62, 69, 149], [137, 54, 143], [39, 61, 184], [101, 115, 39], [59, 142, 197], [230, 139, 58], [110, 86, 26], [218, 151, 203], [49, 2, 58], [84, 211, 135], [50, 107, 108], [196, 69, 135], [204, 30, 212], [171, 251, 166], [243, 166, 185], [121, 61, 16], [62, 239, 223], [243, 213, 254], [105, 157, 87], [213, 49, 28], [22, 116, 17], [198, 31, 197], [223, 147, 161], [27, 236, 207], [38, 81, 122], [147, 8, 3], [17, 26, 169], [208, 241, 225], [194, 36, 108], [189, 143, 102], [121, 107, 142], [27, 253, 206], [119, 55, 18], [154, 243, 214], [146, 57, 111], [22, 88, 104], [51, 214, 76], [235, 173, 33], [18, 226, 107], [48, 96, 134], [4, 6, 182], [170, 66, 224], [200, 11, 134], [84, 127, 108], [30, 18, 99], [243, 151, 59], [112, 37, 245], [162, 251, 152], [135, 214, 3], [142, 202, 163], [49, 76, 177], [183, 170, 118], [246, 69, 126], [140, 238, 126], [33, 6, 154], [76, 63, 15], [225, 224, 133], [37, 6, 65], [17, 162, 151], [115, 13, 109], [66, 169, 129], [147, 247, 159], [38, 83, 5], [92, 70, 164], [11, 182, 105], [172, 188, 219], [177, 103, 188], [182, 158, 151], [130, 197, 208], [87, 241, 145], [34, 8, 131], [11, 32, 3], [113, 86, 43], [212, 197, 181], [119, 16, 68], [204, 11, 167], [188, 5, 224], [119, 150, 136], [92, 174, 9], [197, 200, 89], [51, 98, 207], [166, 72, 90], [19, 142, 20], [201, 152, 91], [68, 108, 49], [163, 98, 226], [37, 127, 196], [49, 122, 61], [97, 70, 110], [169, 110, 137], [19, 218, 172], [231, 210, 38], [177, 99, 114], [174, 240, 181], [14, 237, 126], [195, 208, 90], [149, 119, 139], [179, 121, 251], [212, 79, 223], [224, 62, 211], [85, 6, 0], [227, 97, 119], [195, 209, 102], [75, 142, 91], [52, 179, 253], [7, 193, 43], [143, 128, 81], [37, 176, 184], [102, 175, 37], [131, 139, 21], [110, 108, 18], [78, 4, 38], [246, 190, 27], [200, 57, 245], [47, 85, 178], [134, 85, 130], [158, 150, 27], [49, 126, 82], [99, 190, 133], [66, 202, 44], [1, 118, 4], [28, 221, 233], [46, 159, 132], [104, 98, 118], [166, 156, 93], [52, 14, 134], [55, 69, 149], [227, 171, 217], [197, 88, 115], [159, 240, 78], [243, 178, 70], [80, 66, 215], [9, 5, 9], [216, 201, 68], [45, 169, 79], [144, 132, 215], [157, 204, 66], [193, 110, 246], [55, 77, 170], [67, 10, 19], [139, 168, 237], [25, 70, 68], [32, 158, 95], [218, 6, 235], [84, 244, 149], [104, 139, 42], [205, 234, 123], [30, 16, 131], [211, 99, 214], [175, 58, 236], [127, 228, 149], [72, 65, 230], [177, 185, 145], [199, 128, 87], [200, 85, 87], [10, 225, 123], [125, 140, 95], [190, 235, 69], [78, 191, 146], [172, 80, 164], [171, 112, 241], [201, 212, 189], [146, 57, 163], [60, 182, 121], [21, 55, 24], [238, 154, 145], [83, 187, 75], [23, 24, 10], [38, 241, 255], [29, 177, 244], [171, 83, 53], [190, 248, 240], [162, 19, 6], [103, 150, 98], [79, 220, 239], [99, 96, 109], [76, 254, 208], [140, 75, 245], [216, 217, 39], [228, 48, 216], [252, 152, 233], [49, 140, 69], [173, 236, 70], [83, 73, 217], [20, 243, 23], [10, 114, 76], [241, 254, 142], [199, 99, 89], [98, 118, 233], [33, 42, 126], [116, 90, 13], [239, 215, 21], [131, 22, 9], [130, 126, 221], [35, 198, 120], [132, 248, 0], [187, 19, 146], [145, 40, 217], [111, 240, 162], [137, 6, 44], [89, 78, 205], [117, 11, 193], [60, 253, 224], [232, 55, 236], [241, 237, 173], [59, 149, 136], [111, 141, 33], [159, 28, 50], [108, 175, 220], [24, 106, 165], [230, 75, 205], [12, 249, 175], [133, 186, 175], [77, 171, 112], [69, 111, 113], [209, 245, 134], [21, 177, 136], [253, 10, 154], [77, 27, 255], [239, 59, 145], [157, 101, 189], [103, 148, 45], [97, 222, 187], [189, 108, 52], [114, 117, 83], [20, 181, 229], [49, 178, 192], [141, 129, 58], [239, 204, 231], [215, 202, 181], [151, 46, 207], [158, 7, 162], [151, 75, 103], [129, 249, 211], [19, 193, 55], [60, 213, 103], [29, 113, 207], [162, 242, 246], [190, 44, 79], [119, 176, 26], [246, 100, 51], [178, 37, 95], [183, 130, 156], [213, 207, 174], [185, 27, 125], [30, 113, 238], [130, 89, 112], [70, 63, 53], [253, 109, 137], [66, 4, 108], [154, 195, 62], [43, 31, 253], [43, 232, 41], [193, 116, 238], [32, 156, 5], [59, 179, 53], [241, 140, 207], [32, 88, 205], [47, 122, 178], [146, 169, 18], [86, 199, 18], [136, 230, 66], [40, 194, 230], [179, 72, 233], [146, 135, 230], [143, 22, 67], [78, 41, 191], [129, 154, 80], [237, 187, 215], [144, 49, 72], [218, 13, 84], [72, 60, 153], [218, 250, 228], [193, 95, 253], [186, 186, 59], [97, 169, 91], [84, 208, 218], [226, 67, 193], [57, 141, 92], [12, 61, 208], [150, 76, 231], [224, 12, 10], [22, 183, 146], [114, 40, 84], [253, 251, 29], [156, 63, 11], [39, 105, 119], [150, 72, 200], [107, 103, 18], [112, 41, 56], [115, 54, 250], [226, 47, 177], [45, 4, 157], [55, 54, 31], [108, 195, 227], [26, 73, 176], [125, 106, 178], [41, 0, 56], [124, 238, 250], [11, 0, 57], [108, 167, 232], [191, 228, 149], [129, 63, 98], [64, 152, 155], [41, 49, 243], [29, 212, 227], [111, 236, 253], [50, 130, 64], [232, 33, 158], [186, 92, 205], [210, 250, 120], [31, 89, 32], [25, 255, 126], [9, 224, 137], [202, 247, 252], [174, 67, 189], [242, 189, 23], [16, 82, 103], [160, 160, 245], [69, 243, 9], [169, 162, 185], [62, 41, 217], [98, 146, 110], [251, 37, 121], [143, 193, 167], [107, 25, 117], [206, 224, 142], [77, 171, 242], [30, 29, 220], [5, 240, 31], [77, 98, 163], [202, 7, 158], [171, 180, 15], [32, 206, 16], [25, 45, 154], [120, 24, 102], [241, 188, 67], [232, 139, 120], [160, 120, 199], [94, 183, 239], [146, 95, 37], [131, 84, 214], [156, 240, 166], [96, 135, 7], [237, 214, 230], [248, 182, 195], [75, 87, 111], [227, 181, 122], [214, 198, 146], [217, 247, 88], [193, 70, 199], [77, 140, 17], [191, 57, 26], [177, 122, 177], [183, 206, 20], [236, 158, 78], [247, 244, 236], [1, 39, 238], [19, 102, 150], [242, 174, 201], [100, 17, 238], [107, 7, 225], [242, 97, 214], [78, 116, 87], [31, 194, 245], [176, 180, 141], [97, 100, 98], [191, 45, 112], [241, 138, 96], [224, 11, 207], [237, 220, 35], [237, 39, 225], [111, 226, 28], [99, 146, 198], [39, 174, 193], [32, 248, 32], [35, 203, 166], [198, 164, 24], [251, 55, 175], [188, 128, 102], [36, 55, 156], [133, 42, 146], [30, 150, 95], [233, 2, 9], [228, 175, 237], [247, 229, 150], [198, 232, 27], [175, 59, 141], [105, 207, 64], [210, 124, 215], [115, 161, 37], [90, 209, 141], [232, 12, 190], [134, 99, 20], [117, 40, 55], [221, 187, 199], [226, 69, 221], [81, 226, 198], [56, 92, 212], [230, 230, 60], [11, 4, 238], [34, 185, 170], [147, 186, 228], [145, 130, 36], [40, 134, 15], [56, 170, 118], [137, 72, 207], [125, 40, 193], [193, 70, 180], [224, 162, 96], [95, 58, 224], [32, 234, 158], [25, 133, 230], [88, 107, 103], [119, 77, 227], [113, 175, 135], [125, 31, 230], [34, 237, 75], [63, 177, 23], [219, 189, 239], [194, 57, 183], [69, 58, 71], [246, 223, 55], [11, 155, 242], [61, 11, 75], [83, 148, 45], [187, 116, 151], [187, 93, 84], [128, 173, 48], [105, 37, 156], [230, 178, 164], [108, 254, 209], [133, 201, 249], [199, 240, 171], [129, 11, 85], [213, 145, 40], [247, 77, 115], [171, 116, 247], [9, 150, 171], [201, 211, 205], [20, 152, 131], [21, 9, 165], [22, 35, 102], [180, 112, 119], [43, 147, 244], [38, 68, 210], [233, 221, 156], [173, 159, 36], [146, 35, 169], [238, 51, 244], [149, 140, 129], [224, 162, 58], [27, 2, 43], [169, 222, 104], [93, 187, 74], [228, 253, 240], [180, 49, 12], [64, 120, 79], [208, 151, 235], [56, 243, 18], [214, 205, 134], [178, 132, 240], [103, 112, 62], [114, 172, 45], [225, 194, 146], [36, 216, 205], [112, 30, 196], [166, 244, 249], [132, 19, 255], [20, 218, 235], [220, 227, 105], [51, 129, 241], [229, 175, 253], [96, 89, 88], [48, 21, 223], [89, 21, 22], [169, 178, 201], [252, 91, 129], [13, 172, 250], [112, 76, 204], [94, 4, 213], [194, 198, 134], [225, 118, 94], [27, 78, 163], [235, 147, 241], [217, 40, 103], [251, 169, 47], [161, 169, 16], [7, 215, 43], [170, 126, 172], [61, 128, 228], [121, 51, 55], [244, 88, 96], [132, 181, 183], [103, 62, 196], [147, 47, 216], [52, 193, 8], [101, 36, 82], [238, 4, 92], [101, 3, 67], [30, 46, 1], [241, 233, 211], [193, 80, 223], [97, 117, 50], [161, 255, 137], [113, 220, 143], [130, 133, 236], [197, 68, 205], [125, 127, 108], [220, 189, 88], [242, 179, 10], [74, 141, 100], [188, 73, 169], [78, 32, 252], [195, 215, 65], [35, 208, 208], [126, 226, 205], [91, 41, 21], [137, 131, 33], [120, 134, 188], [163, 72, 147], [89, 70, 254], [148, 237, 154], [109, 171, 46], [186, 11, 205], [88, 106, 139], [102, 209, 252], [62, 205, 42], [206, 2, 138], [177, 81, 204], [232, 57, 123], [87, 1, 251], [99, 152, 240], [68, 234, 102], [22, 98, 229], [167, 138, 70], [165, 138, 156], [249, 78, 21], [67, 7, 105], [84, 160, 220], [216, 115, 149], [149, 189, 92], [226, 208, 169], [111, 207, 20], [176, 22, 50], [65, 186, 161], [186, 61, 237], [44, 178, 69], [207, 18, 126], [102, 48, 125], [153, 33, 46], [75, 120, 84], [104, 133, 219], [113, 216, 8], [1, 156, 233], [50, 77, 214], [23, 210, 89], [79, 228, 11], [41, 157, 115], [85, 121, 179], [70, 151, 162], [155, 38, 16], [180, 117, 20], [147, 164, 69], [211, 227, 109], [187, 22, 105], [97, 81, 189], [165, 216, 236], [185, 181, 157], [212, 161, 71], [112, 231, 132], [208, 14, 225], [22, 225, 60], [188, 16, 77], [68, 15, 221], [129, 200, 88], [176, 231, 242], [168, 14, 109], [165, 32, 249], [146, 251, 144], [243, 46, 223], [119, 192, 242], [161, 252, 211], [227, 154, 156], [102, 96, 177], [61, 145, 56], [1, 89, 140], [28, 199, 27], [54, 182, 161], [30, 147, 164], [219, 165, 176], [32, 136, 33], [86, 210, 135], [169, 29, 35], [194, 149, 240], [82, 123, 142], [144, 8, 24], [53, 221, 211], [200, 232, 127], [242, 45, 26], [134, 145, 109], [140, 8, 176], [247, 17, 31], [215, 208, 107], [50, 72, 253], [12, 27, 123], [35, 233, 32], [230, 45, 78], [52, 53, 182], [14, 234, 93], [128, 161, 106], [4, 246, 118], [126, 137, 201], [125, 166, 40], [134, 215, 83], [84, 237, 4], [127, 176, 213], [217, 63, 67], [152, 121, 239], [147, 106, 37], [136, 137, 13], [148, 249, 121], [21, 166, 37], [166, 170, 123], [142, 211, 120], [172, 26, 202], [20, 118, 219], [66, 221, 3], [39, 221, 161], [45, 253, 152], [190, 42, 52], [82, 88, 122], [85, 99, 211], [206, 80, 112], [35, 141, 133], [224, 108, 144], [26, 42, 105], [227, 23, 58], [71, 247, 237], [137, 123, 252], [180, 248, 123], [142, 128, 43], [207, 207, 87], [185, 31, 28], [198, 108, 223], [172, 165, 237], [3, 69, 216], [35, 3, 122], [205, 31, 92], [32, 141, 105], [3, 86, 216], [111, 206, 212], [201, 216, 200], [2, 17, 243], [140, 98, 105], [152, 64, 240], [182, 24, 104], [173, 28, 108], [133, 161, 66], [24, 11, 20], [80, 71, 91], [131, 54, 203], [31, 159, 52], [27, 75, 13], [194, 160, 32], [99, 229, 182], [198, 110, 170], [66, 31, 165], [40, 204, 112], [95, 204, 160], [78, 29, 195], [28, 227, 51], [146, 206, 250], [129, 109, 38], [150, 158, 144], [71, 18, 131], [233, 189, 35], [34, 7, 214], [165, 70, 66], [166, 110, 68], [85, 251, 250], [187, 56, 104], [94, 114, 232], [93, 100, 219], [10, 34, 37], [215, 5, 54], [40, 221, 221], [187, 121, 3], [110, 125, 81], [126, 244, 53], [78, 147, 113], [132, 26, 107], [141, 244, 128], [73, 35, 93], [103, 238, 176], [17, 43, 134], [157, 99, 178], [207, 234, 89], [99, 160, 240], [10, 239, 135], [166, 123, 171], [198, 192, 217], [223, 159, 152], [10, 238, 127], [110, 95, 255], [176, 46, 77], [247, 29, 199], [158, 133, 200], [138, 6, 223], [0, 0, 87], [244, 139, 194], [138, 39, 223], [204, 184, 111], [215, 228, 165], [91, 244, 217], [85, 238, 250], [43, 7, 64], [176, 248, 245], [162, 223, 35], [97, 42, 114], [44, 112, 160], [234, 56, 8], [174, 2, 125], [210, 41, 215], [136, 193, 118], [34, 171, 223], [58, 190, 78], [216, 61, 171], [193, 235, 150], [15, 173, 176], [83, 64, 251], [25, 69, 180], [199, 231, 179], [159, 111, 211], [29, 236, 30], [82, 150, 159], [13, 241, 14], [76, 1, 32], [232, 247, 230], [242, 194, 199], [32, 197, 104], [137, 170, 177], [232, 106, 61], [180, 18, 103], [242, 70, 57], [104, 173, 145], [238, 206, 141], [104, 201, 144], [62, 65, 225], [173, 176, 65], [215, 97, 244], [122, 235, 231], [191, 124, 51], [48, 171, 212], [130, 217, 73], [170, 163, 83], [164, 241, 135], [100, 141, 91], [162, 116, 51], [71, 124, 38], [128, 149, 180], [254, 216, 231], [230, 72, 70], [182, 250, 228], [172, 155, 4], [118, 244, 135], [77, 126, 170], [20, 177, 45], [168, 108, 155], [131, 158, 75], [75, 63, 9], [65, 114, 155], [113, 63, 121], [223, 16, 31], [196, 223, 0], [211, 155, 54], [38, 176, 119], [52, 49, 224], [184, 194, 51], [91, 227, 182], [197, 197, 233], [151, 165, 132], [219, 63, 136], [16, 153, 219], [164, 216, 207], [189, 114, 60], [223, 110, 131], [67, 28, 197], [223, 166, 230], [230, 77, 55], [71, 64, 178], [252, 22, 5], [166, 80, 211], [61, 62, 123], [154, 96, 165], [92, 41, 128], [134, 99, 10], [118, 92, 155], [128, 135, 103], [179, 55, 59], [48, 228, 64], [68, 233, 59], [244, 112, 58], [241, 148, 75], [249, 193, 136], [50, 34, 103], [135, 131, 167], [127, 82, 147], [49, 123, 227], [129, 92, 94], [138, 166, 237], [23, 111, 33], [205, 39, 189], [211, 227, 232], [21, 130, 99], [76, 86, 98], [53, 31, 172], [199, 98, 99], [193, 242, 126], [140, 65, 241], [54, 65, 242], [199, 220, 96], [174, 172, 113], [50, 164, 253], [248, 30, 200], [156, 119, 140], [61, 127, 233], [230, 126, 113], [155, 158, 7], [199, 125, 237], [229, 198, 11], [203, 36, 204], [26, 80, 101], [5, 84, 158], [150, 141, 239], [76, 115, 11], [161, 3, 185], [196, 223, 123], [244, 53, 176], [132, 162, 206], [79, 193, 65], [206, 73, 6], [106, 40, 213], [177, 66, 204], [79, 225, 63], [203, 93, 2], [120, 43, 56], [111, 176, 199], [16, 141, 30], [226, 158, 178], [150, 35, 4], [9, 233, 145], [144, 222, 77], [67, 75, 230], [23, 29, 156], [16, 232, 195], [85, 255, 121], [20, 47, 76], [141, 185, 226], [234, 130, 15], [103, 98, 215], [222, 226, 161], [208, 82, 162], [57, 8, 102], [54, 100, 58], [126, 95, 50], [137, 240, 154], [26, 71, 54], [211, 86, 19], [240, 30, 112], [167, 187, 164], [201, 53, 255], [126, 53, 162], [125, 137, 7], [154, 92, 143], [234, 75, 36], [141, 79, 243], [83, 209, 103], [230, 225, 132], [46, 245, 225], [141, 164, 233], [183, 71, 213], [3, 183, 170], [253, 239, 173], [63, 47, 65], [168, 137, 22], [2, 203, 218], [90, 78, 19], [115, 15, 65], [95, 77, 196], [84, 41, 161], [15, 6, 188], [227, 87, 227], [13, 48, 155], [79, 174, 118], [188, 224, 228], [119, 226, 7], [141, 230, 157], [95, 139, 147], [92, 129, 100], [128, 42, 248], [248, 237, 92], [141, 5, 156], [63, 150, 64], [232, 206, 77], [100, 100, 171], [69, 196, 21], [196, 94, 175], [70, 75, 71], [244, 252, 0], [84, 233, 49], [95, 143, 255], [226, 245, 27], [35, 108, 159], [193, 150, 190], [92, 149, 190], [151, 45, 8], [119, 40, 194], [225, 218, 239], [33, 195, 13], [78, 231, 138], [96, 236, 59], [165, 128, 36], [227, 175, 124], [0, 160, 117], [236, 18, 67], [15, 8, 240], [118, 133, 170], [188, 0, 217], [104, 16, 47], [58, 86, 147], [167, 61, 31], [14, 92, 3], [97, 128, 139], [159, 133, 45], [16, 117, 40], [61, 234, 64], [157, 88, 197], [150, 116, 160], [166, 83, 150], [87, 213, 19], [104, 143, 128], [124, 91, 234], [214, 222, 165], [209, 243, 54], [191, 235, 164], [135, 207, 244], [146, 178, 150], [241, 130, 172], [242, 10, 253], [135, 75, 71], [14, 224, 48], [191, 220, 48], [113, 124, 30], [85, 86, 245], [232, 128, 127], [94, 95, 105], [40, 209, 201], [79, 109, 131], [91, 35, 213], [144, 78, 4], [85, 44, 39], [139, 47, 118], [142, 116, 115], [39, 3, 252], [198, 103, 131], [190, 142, 161], [95, 177, 248], [188, 156, 210], [11, 195, 89], [159, 162, 33], [122, 36, 11], [7, 92, 202], [159, 231, 159], [34, 75, 206], [142, 11, 180], [98, 98, 47], [103, 1, 252], [131, 206, 183], [211, 77, 194], [162, 246, 141], [209, 217, 197], [218, 124, 197], [32, 155, 3], [174, 151, 219], [38, 137, 68], [99, 174, 239], [220, 237, 153], [37, 112, 133], [165, 58, 40], [14, 193, 102], [39, 134, 75], [170, 203, 77], [168, 238, 156], [104, 70, 233], [208, 25, 50], [123, 180, 59], [164, 234, 211], [52, 205, 28], [206, 188, 238], [100, 77, 110], [50, 244, 97], [119, 174, 137], [5, 6, 112], [67, 248, 202], [139, 143, 61], [81, 223, 8], [178, 237, 151], [13, 237, 187], [48, 5, 145], [204, 178, 14], [187, 119, 93], [41, 137, 131], [180, 238, 48], [196, 222, 110], [25, 245, 107], [65, 25, 115], [45, 37, 78], [40, 5, 2], [144, 82, 72], [88, 98, 112], [118, 63, 111], [2, 232, 168], [218, 242, 16], [214, 224, 89], [193, 128, 244], [218, 112, 133], [51, 135, 246], [56, 151, 164], [70, 69, 116], [65, 231, 157], [241, 202, 17], [227, 26, 100], [155, 250, 143], [158, 232, 218], [143, 51, 117], [69, 101, 126], [168, 110, 64], [208, 202, 210], [57, 168, 216], [250, 223, 48], [205, 79, 0], [73, 28, 175], [8, 192, 28], [84, 141, 131], [194, 81, 71], [239, 86, 157], [72, 232, 14], [185, 73, 181], [176, 12, 135], [47, 231, 251], [79, 100, 123], [136, 3, 159], [122, 242, 170], [216, 173, 41], [166, 206, 120], [169, 93, 106], [108, 75, 46], [43, 26, 155], [32, 126, 60], [32, 82, 67], [64, 150, 223], [73, 21, 72], [186, 46, 28], [56, 20, 212], [230, 208, 13], [230, 168, 144], [169, 49, 76], [166, 8, 48], [21, 42, 111], [4, 165, 146], [159, 65, 112], [181, 155, 6], [186, 216, 48], [224, 181, 62], [64, 135, 75], [238, 247, 163], [137, 184, 123], [185, 78, 120], [145, 52, 178], [172, 39, 224], [36, 113, 253], [114, 220, 234], [124, 18, 233], [4, 112, 65], [144, 34, 98], [249, 192, 251], [112, 206, 173], [169, 202, 44], [77, 220, 78], [13, 217, 156], [75, 210, 180], [168, 177, 67], [104, 220, 87], [48, 53, 194], [154, 223, 156], [228, 57, 185], [218, 166, 180], [192, 27, 81], [217, 171, 83], [217, 163, 122], [124, 201, 52], [73, 193, 28], [44, 47, 199], [95, 191, 134], [113, 107, 60], [198, 107, 127], [37, 27, 158], [114, 13, 125], [187, 74, 24], [0, 158, 16], [187, 184, 202], [79, 59, 121], [158, 55, 110], [62, 15, 246], [69, 6, 122], [249, 19, 167], [191, 91, 191], [173, 42, 194], [10, 59, 210], [188, 165, 135], [213, 164, 237], [253, 96, 84], [36, 207, 113], [13, 177, 126], [1, 209, 186], [59, 138, 146], [159, 16, 100], [254, 91, 73], [157, 135, 69], [215, 225, 188], [150, 51, 187], [15, 230, 126], [97, 135, 194], [253, 206, 83], [245, 31, 194], [247, 180, 117], [85, 4, 158], [104, 71, 232], [31, 152, 150], [144, 40, 214], [230, 223, 161], [159, 114, 20], [251, 231, 27], [175, 213, 40], [114, 162, 161], [239, 177, 130], [84, 41, 190], [182, 177, 5], [116, 194, 59], [172, 209, 124], [157, 108, 11], [27, 199, 230], [18, 84, 136], [246, 118, 77], [16, 177, 180], [70, 111, 106], [67, 149, 217], [165, 16, 10], [231, 182, 222], [147, 171, 111], [29, 81, 161], [112, 120, 203], [248, 108, 48], [140, 129, 73], [66, 225, 136], [30, 250, 116], [71, 141, 48], [112, 177, 96], [127, 131, 8], [178, 38, 42], [46, 73, 71], [183, 61, 80], [36, 191, 87], [125, 228, 173], [88, 54, 146], [121, 143, 164], [76, 51, 85], [133, 51, 43], [123, 160, 214], [144, 29, 90], [25, 131, 76], [243, 15, 47], [125, 246, 70], [37, 221, 154], [17, 45, 119], [240, 138, 196], [68, 173, 226], [73, 238, 217], [5, 72, 40], [39, 168, 75], [192, 227, 44], [240, 188, 107], [210, 109, 115], [165, 253, 54], [252, 13, 228], [251, 254, 253], [195, 214, 136], [146, 71, 9], [56, 28, 59], [40, 89, 46], [235, 77, 67], [241, 112, 169], [77, 15, 72], [210, 124, 87], [164, 188, 141], [220, 196, 107], [62, 10, 7], [55, 121, 213], [228, 94, 119], [78, 52, 203], [235, 115, 119], [246, 144, 210], [195, 112, 187], [17, 77, 124], [28, 147, 254], [17, 59, 183], [138, 171, 71], [202, 72, 235], [156, 245, 127], [32, 200, 16], [246, 1, 122], [166, 254, 252], [180, 223, 68], [178, 94, 67], [107, 248, 135], [47, 200, 11], [125, 58, 96], [47, 2, 5], [215, 229, 23], [71, 50, 200], [179, 165, 56], [35, 236, 157], [104, 184, 69], [168, 36, 244], [250, 224, 52], [144, 2, 216], [120, 210, 197], [232, 44, 197], [69, 33, 255], [220, 86, 69], [77, 138, 163], [21, 198, 33], [184, 164, 231], [242, 81, 99], [74, 195, 203], [195, 196, 117], [46, 47, 139], [88, 163, 34], [86, 158, 69], [140, 43, 64], [51, 251, 117], [114, 199, 225], [230, 11, 36], [197, 101, 118], [81, 113, 1], [117, 160, 171], [95, 127, 214], [247, 234, 113], [17, 182, 98], [125, 228, 138], [77, 91, 6], [151, 233, 81], [38, 115, 94], [125, 188, 91], [191, 116, 164], [211, 98, 168], [228, 168, 63], [180, 254, 206], [154, 120, 37], [166, 155, 38], [16, 225, 11], [183, 139, 3], [235, 57, 29], [240, 194, 82], [29, 45, 131], [205, 185, 87], [212, 120, 201], [96, 98, 97], [62, 142, 245], [181, 28, 89], [31, 80, 249], [247, 210, 92], [12, 45, 67], [120, 244, 22], [138, 192, 250], [189, 182, 77], [175, 191, 227], [86, 74, 13], [173, 108, 104], [241, 136, 225], [153, 31, 98], [250, 52, 235], [252, 223, 140], [12, 90, 106], [214, 179, 20], [137, 54, 127], [37, 60, 231], [188, 171, 246], [36, 31, 156], [48, 107, 124], [39, 178, 32], [246, 238, 116], [175, 168, 89], [149, 54, 253], [158, 96, 234], [239, 108, 203], [212, 140, 240], [92, 232, 154], [148, 136, 68], [136, 121, 239], [24, 82, 250], [125, 136, 194], [174, 111, 211], [237, 19, 171], [127, 249, 245], [155, 93, 48], [125, 192, 250], [174, 138, 232], [96, 140, 48], [67, 105, 147], [80, 205, 4], [239, 193, 193], [130, 131, 246], [49, 27, 129], [168, 113, 59], [115, 30, 29], [76, 165, 174], [40, 188, 63], [52, 23, 23], [241, 210, 147], [251, 111, 116], [7, 217, 50], [55, 214, 152], [47, 244, 187], [144, 26, 189], [226, 199, 102], [89, 233, 195], [20, 144, 143], [143, 147, 77], [200, 93, 201], [53, 112, 217], [80, 85, 181], [153, 226, 123], [239, 120, 185], [25, 15, 167], [211, 13, 110], [55, 246, 238], [110, 110, 150], [221, 103, 162], [103, 224, 158], [45, 230, 188], [235, 148, 112], [25, 255, 251], [168, 157, 183], [217, 204, 226], [251, 109, 53], [77, 227, 58], [23, 138, 169], [55, 210, 1], [10, 225, 224], [55, 143, 95], [17, 174, 184], [70, 148, 188], [138, 92, 159], [201, 199, 14], [28, 131, 71], [53, 197, 245], [232, 86, 93], [95, 16, 217], [244, 34, 171], [49, 252, 201], [254, 200, 158], [83, 62, 183], [165, 1, 131], [9, 134, 171], [57, 46, 206], [18, 169, 67], [87, 100, 198], [12, 14, 74], [163, 251, 104], [214, 29, 214], [7, 197, 167], [17, 199, 5], [239, 98, 168], [133, 204, 182], [46, 132, 13], [195, 115, 164], [187, 4, 91], [121, 149, 177], [107, 87, 211], [159, 10, 170], [106, 103, 160], [46, 85, 7], [148, 239, 14], [160, 169, 86], [222, 157, 26], [137, 210, 244], [104, 144, 18], [240, 124, 176], [177, 253, 217], [7, 255, 181], [165, 99, 125], [220, 199, 156], [209, 188, 215], [208, 72, 28], [97, 7, 111], [203, 239, 16], [104, 219, 113], [188, 9, 255], [101, 205, 45], [136, 203, 101], [85, 98, 141], [45, 26, 93], [171, 66, 39], [124, 187, 253], [116, 11, 116], [182, 180, 23], [102, 161, 36], [147, 186, 243], [173, 138, 131], [12, 212, 69], [192, 59, 49], [40, 245, 149], [54, 57, 29], [89, 67, 71], [32, 126, 150], [206, 6, 69], [88, 181, 188], [32, 157, 186], [240, 216, 48], [112, 231, 220], [231, 8, 242], [219, 247, 222], [132, 103, 87], [32, 165, 65], [42, 106, 48], [135, 59, 161], [97, 112, 68], [149, 195, 236], [235, 88, 9], [130, 96, 180], [239, 139, 14], [42, 205, 78], [168, 4, 120], [60, 247, 154], [170, 157, 83], [119, 46, 226], [113, 142, 229], [33, 238, 181], [27, 35, 133], [61, 87, 38], [112, 65, 173], [5, 0, 16], [4, 171, 164], [240, 0, 242], [59, 121, 0], [203, 6, 114], [116, 201, 32], [105, 215, 40], [6, 80, 219], [105, 206, 42], [245, 9, 39], [123, 183, 187], [9, 44, 105], [150, 65, 5], [97, 198, 82], [226, 144, 232], [28, 199, 118], [146, 246, 164], [115, 231, 93], [189, 168, 97], [211, 4, 13], [14, 124, 61], [215, 54, 118], [123, 224, 16], [146, 19, 199], [128, 214, 81], [220, 55, 191], [175, 188, 38], [145, 145, 196], [229, 31, 229], [231, 191, 123], [46, 144, 138], [47, 37, 149], [25, 120, 247], [138, 69, 51], [145, 50, 187], [1, 17, 211], [240, 2, 46], [221, 173, 96], [147, 194, 69], [92, 160, 119], [179, 31, 110], [11, 242, 115], [197, 120, 130], [205, 183, 164], [131, 42, 240], [61, 76, 228], [72, 206, 197], [55, 78, 46], [64, 44, 253], [55, 31, 48], [46, 164, 240], [193, 141, 83], [242, 244, 16], [128, 207, 214], [111, 121, 105], [127, 233, 139], [207, 76, 43], [21, 168, 108], [104, 6, 143], [1, 235, 104], [153, 179, 196], [36, 112, 209], [220, 248, 168], [239, 22, 123], [89, 239, 142], [107, 175, 8], [13, 155, 110], [106, 117, 42], [52, 55, 131], [174, 155, 13], [133, 214, 140], [34, 90, 254], [57, 149, 204], [127, 129, 143], [76, 129, 187], [207, 203, 159], [251, 42, 228], [4, 182, 249], [104, 231, 135], [29, 71, 231], [205, 171, 235], [141, 106, 220], [174, 176, 161], [165, 47, 87], [149, 93, 11], [195, 240, 124], [99, 229, 214], [128, 22, 15], [142, 87, 102], [38, 94, 205], [168, 32, 238], [84, 173, 64], [67, 74, 126], [36, 136, 113], [12, 195, 99], [149, 246, 74], [141, 40, 224], [155, 248, 140], [31, 13, 71], [104, 52, 249], [114, 130, 11], [208, 252, 118], [220, 155, 180], [151, 141, 119], [230, 42, 126], [242, 102, 55], [251, 215, 255], [201, 208, 171], [15, 40, 91], [243, 99, 51], [60, 13, 112], [69, 237, 47], [36, 236, 206], [44, 157, 48], [54, 59, 111], [129, 134, 22], [147, 30, 76], [57, 227, 157], [201, 146, 118], [91, 87, 17], [203, 234, 210], [95, 173, 235], [54, 232, 168], [105, 63, 90], [154, 108, 124], [139, 229, 0], [178, 61, 84], [231, 38, 164], [233, 15, 103], [10, 42, 106], [8, 254, 96], [236, 165, 124], [232, 51, 216], [252, 84, 135], [226, 129, 86], [75, 94, 93], [182, 156, 168], [48, 182, 104], [36, 231, 87], [115, 32, 223], [0, 217, 21], [63, 79, 206], [209, 99, 15], [237, 73, 155], [50, 193, 91], [254, 86, 125], [56, 118, 130], [210, 123, 217], [252, 130, 231], [73, 27, 127], [219, 104, 124], [185, 240, 210], [105, 101, 210], [246, 140, 97], [36, 168, 192], [34, 49, 234], [69, 232, 23], [209, 61, 108], [81, 24, 207], [103, 176, 75], [253, 90, 2], [33, 251, 184], [189, 28, 220], [94, 190, 188], [140, 239, 101], [209, 164, 200], [127, 252, 51], [103, 248, 213], [206, 88, 199], [117, 90, 45], [55, 223, 68], [169, 98, 65], [195, 216, 91], [161, 37, 151], [13, 132, 45], [54, 194, 219], [71, 245, 215], [141, 106, 124], [46, 75, 139], [250, 174, 99], [22, 196, 142], [91, 43, 7], [174, 138, 56], [238, 71, 81], [251, 123, 254], [101, 198, 88], [35, 151, 250], [15, 83, 38], [199, 142, 201], [184, 231, 213], [4, 4, 146], [98, 212, 48], [45, 122, 116], [1, 194, 65], [127, 215, 104], [22, 109, 128], [104, 189, 18], [226, 80, 19], [13, 228, 193], [116, 102, 130], [233, 2, 137], [2, 7, 103], [115, 237, 117], [8, 75, 8], [68, 85, 163], [198, 222, 30], [211, 29, 195], [112, 93, 24], [169, 99, 128], [163, 127, 36], [218, 108, 198], [242, 162, 82], [223, 127, 244], [82, 106, 78], [132, 91, 98], [81, 29, 217], [59, 38, 247], [67, 167, 192], [188, 206, 77], [224, 199, 238], [228, 200, 119], [194, 249, 226], [240, 102, 188], [202, 167, 189], [141, 121, 244], [207, 163, 58], [74, 135, 140], [145, 73, 227], [200, 210, 37], [24, 212, 208], [89, 140, 210], [248, 17, 221], [124, 37, 57], [71, 99, 26], [44, 160, 94], [153, 163, 223], [186, 144, 255], [105, 228, 69], [180, 50, 48], [5, 64, 159], [227, 27, 55], [163, 62, 135], [6, 177, 6], [139, 103, 225], [42, 249, 47], [75, 245, 190], [250, 135, 83], [243, 189, 220], [68, 203, 160], [219, 28, 193], [164, 94, 99], [203, 253, 51], [42, 43, 149], [71, 93, 190], [7, 164, 26], [219, 143, 211], [123, 145, 7], [87, 6, 76], [140, 254, 16], [157, 25, 140], [150, 106, 87], [121, 29, 253], [94, 209, 97], [176, 182, 69], [174, 23, 184], [130, 146, 178], [24, 1, 41], [16, 203, 174], [105, 83, 88], [57, 110, 91], [241, 81, 118], [67, 15, 103], [131, 102, 243], [190, 2, 202], [135, 59, 221], [12, 55, 198], [207, 218, 207], [241, 193, 139], [25, 21, 153], [153, 62, 57], [174, 31, 23], [103, 251, 121], [109, 182, 203], [95, 224, 46], [105, 189, 5], [97, 98, 242], [127, 234, 83], [164, 132, 13], [22, 36, 76], [109, 130, 242], [192, 51, 153], [126, 65, 231], [31, 216, 60], [206, 137, 183], [44, 99, 6], [255, 101, 166], [158, 126, 29], [110, 190, 251], [68, 220, 223], [245, 245, 53], [210, 44, 99], [181, 100, 53], [206, 231, 244], [164, 138, 209], [74, 179, 123], [190, 243, 226], [166, 163, 111], [217, 179, 8], [130, 184, 35], [30, 93, 127], [55, 243, 68], [175, 36, 154], [244, 243, 158], [94, 181, 42], [237, 91, 92], [19, 98, 252], [92, 83, 165], [3, 197, 208], [39, 253, 216], [215, 113, 251], [63, 234, 63], [175, 88, 67], [46, 40, 139], [254, 222, 13], [56, 207, 143], [65, 1, 23], [247, 137, 208], [214, 65, 88], [107, 126, 223], [100, 116, 66], [105, 23, 125], [38, 94, 78], [48, 168, 188], [165, 2, 65], [135, 249, 126], [132, 61, 179], [68, 222, 11], [4, 128, 241], [3, 172, 245], [117, 50, 158], [70, 29, 72], [68, 23, 205], [235, 161, 197], [93, 194, 209], [155, 9, 27], [87, 7, 19], [190, 3, 0], [144, 241, 194], [62, 51, 34], [239, 213, 41], [242, 198, 81], [247, 110, 118], [97, 73, 60], [196, 197, 67], [145, 60, 227], [212, 173, 78], [46, 183, 83], [151, 62, 246], [149, 105, 197], [193, 254, 214], [227, 238, 226], [15, 96, 162], [43, 15, 13], [244, 147, 15], [140, 146, 167], [190, 114, 154], [222, 57, 36], [64, 147, 192], [94, 37, 38], [158, 11, 105], [217, 164, 119], [46, 211, 228], [162, 11, 119], [218, 77, 80], [121, 27, 10], [124, 45, 110], [8, 63, 63], [119, 242, 215], [85, 237, 127], [106, 71, 114], [170, 218, 206], [85, 223, 3], [218, 122, 6], [75, 134, 84], [8, 166, 175], [190, 32, 8], [87, 17, 184], [66, 148, 35], [26, 104, 40], [1, 130, 228], [76, 71, 124], [91, 209, 29], [129, 91, 45], [127, 95, 225], [146, 55, 23], [36, 120, 124], [198, 203, 231], [9, 114, 174], [148, 253, 91], [56, 113, 0], [55, 158, 37], [83, 43, 74], [21, 221, 161], [229, 224, 54], [103, 60, 180], [42, 62, 249], [23, 170, 204], [12, 123, 19], [150, 216, 144], [15, 26, 19], [245, 136, 108], [134, 100, 176], [70, 179, 58], [194, 198, 169], [232, 168, 193], [67, 181, 38], [175, 233, 145], [245, 34, 48], [128, 32, 178], [40, 210, 12], [49, 78, 100], [65, 137, 251], [107, 213, 97], [30, 198, 28], [230, 3, 78], [47, 93, 154], [202, 96, 66], [76, 199, 69], [96, 156, 156], [124, 131, 135], [222, 63, 98], [191, 57, 107], [183, 118, 176], [107, 182, 32], [37, 36, 114], [116, 148, 68], [117, 218, 116], [252, 113, 127], [41, 138, 64], [158, 240, 131], [244, 107, 201], [241, 193, 13], [36, 118, 7], [86, 251, 111], [115, 156, 63], [2, 81, 42], [225, 192, 202], [124, 97, 186], [201, 73, 254], [199, 98, 104], [200, 250, 215], [247, 118, 15], [94, 246, 229], [242, 133, 245], [244, 216, 134], [64, 15, 90], [107, 41, 68], [203, 209, 219], [107, 27, 177], [96, 243, 153], [30, 177, 200], [133, 236, 217], [128, 69, 202], [219, 5, 51], [204, 55, 5], [210, 243, 154], [243, 215, 191], [66, 74, 14], [84, 57, 154], [208, 202, 91], [115, 0, 56], [129, 67, 79], [150, 234, 122], [185, 60, 13], [136, 160, 55], [215, 223, 99], [241, 65, 70], [248, 17, 157], [129, 199, 182], [251, 199, 43], [85, 137, 178], [23, 35, 192], [244, 125, 200], [21, 198, 152], [41, 234, 167], [20, 95, 13], [90, 120, 35], [145, 178, 92], [132, 140, 170], [40, 229, 104], [149, 114, 58], [193, 200, 231], [239, 160, 229], [8, 236, 143], [214, 57, 244], [188, 200, 142], [17, 175, 120], [36, 245, 4], [214, 60, 109], [82, 27, 113], [123, 8, 54], [42, 79, 68], [74, 42, 186], [182, 63, 243], [40, 102, 83], [122, 157, 86], [205, 196, 182], [198, 198, 158], [124, 239, 29], [173, 138, 80], [244, 110, 190], [146, 240, 34], [79, 171, 44], [116, 120, 60], [32, 0, 165], [147, 238, 236], [254, 48, 97], [142, 241, 89], [40, 93, 13], [52, 103, 229], [53, 56, 89], [223, 210, 201], [52, 245, 175], [219, 237, 23], [105, 203, 83], [159, 184, 55], [103, 238, 163], [44, 229, 102], [223, 240, 28], [96, 147, 241], [218, 144, 158], [157, 112, 115], [35, 165, 128], [162, 230, 36], [128, 248, 173], [136, 252, 234], [167, 144, 71], [191, 135, 49], [111, 128, 148], [120, 51, 114], [20, 190, 59], [12, 139, 59], [62, 97, 192], [157, 83, 98], [71, 138, 100], [19, 154, 24], [250, 90, 224], [52, 251, 123], [195, 172, 79], [22, 200, 98], [96, 244, 95], [203, 65, 210], [137, 169, 155], [123, 137, 239], [108, 199, 74], [2, 68, 33], [4, 4, 209], [200, 92, 179], [129, 217, 56], [126, 180, 44], [31, 189, 49], [32, 93, 194], [21, 84, 242], [168, 149, 18], [25, 101, 212], [189, 65, 149], [211, 153, 59], [173, 27, 134], [42, 133, 87], [64, 253, 253], [49, 82, 25], [142, 103, 6], [21, 155, 0], [54, 34, 17], [97, 111, 124], [246, 6, 159], [50, 252, 122], [140, 81, 241], [147, 121, 252], [81, 43, 81], [118, 120, 6], [99, 115, 149], [95, 176, 170], [23, 141, 206], [56, 168, 135], [79, 126, 9], [156, 143, 166], [18, 158, 146], [217, 116, 124], [48, 2, 136], [85, 6, 45], [249, 241, 119], [246, 243, 230], [214, 92, 254], [126, 190, 206], [56, 216, 34], [252, 209, 213], [207, 96, 16], [139, 108, 26], [150, 247, 33], [236, 3, 95], [68, 51, 198], [250, 40, 91], [145, 47, 170], [164, 175, 162], [72, 163, 93], [123, 26, 42], [31, 26, 183], [151, 121, 113], [110, 166, 48], [117, 180, 93], [131, 253, 70], [79, 159, 57], [231, 160, 166], [249, 136, 196], [243, 99, 103], [80, 156, 191], [150, 111, 206], [133, 185, 214], [81, 32, 186], [119, 180, 119], [68, 162, 52], [13, 201, 94], [226, 247, 44], [225, 212, 54], [230, 109, 229], [231, 113, 164], [43, 60, 202], [34, 23, 101], [61, 211, 228], [167, 235, 48], [51, 133, 202], [186, 0, 128], [245, 42, 143], [135, 5, 103], [245, 39, 7], [106, 151, 179], [132, 152, 107], [73, 113, 231], [20, 227, 186], [58, 51, 5], [167, 214, 164], [249, 232, 57], [1, 135, 253], [231, 178, 121], [2, 114, 172], [137, 53, 0], [64, 224, 111], [130, 116, 56], [192, 250, 103], [48, 19, 33], [126, 163, 92], [99, 193, 115], [50, 209, 39], [6, 155, 217], [245, 19, 47], [241, 52, 37], [218, 130, 58], [64, 137, 221], [227, 129, 235], [26, 127, 76], [44, 94, 62], [54, 210, 214], [149, 184, 167], [185, 10, 155], [184, 38, 110], [179, 53, 85], [172, 225, 11], [148, 162, 223], [236, 32, 56], [242, 21, 171], [239, 55, 202], [92, 50, 124], [201, 241, 138], [47, 131, 42], [205, 45, 69], [50, 146, 2], [110, 29, 174], [121, 120, 45], [112, 72, 180], [186, 230, 221], [218, 98, 222], [51, 110, 207], [161, 125, 45], [164, 89, 24], [124, 184, 208], [72, 55, 32], [92, 151, 208], [19, 210, 38], [169, 224, 90], [173, 35, 37], [5, 176, 43], [35, 244, 255], [248, 181, 239], [39, 48, 159], [205, 27, 218], [195, 5, 222], [156, 127, 241], [4, 135, 248], [212, 38, 150], [83, 16, 177], [100, 214, 183], [29, 187, 102], [43, 130, 154], [16, 40, 35], [42, 142, 92], [16, 242, 164], [211, 163, 22], [119, 69, 180], [95, 128, 133], [201, 251, 189], [140, 90, 241], [120, 248, 113], [2, 162, 62], [14, 79, 129], [154, 143, 254], [14, 35, 131], [158, 28, 25], [143, 151, 255], [5, 195, 176], [84, 198, 103], [243, 227, 0], [29, 71, 134], [201, 123, 232], [106, 216, 250], [148, 166, 208], [72, 253, 163], [103, 162, 28], [10, 118, 119], [130, 194, 254], [88, 133, 230], [3, 12, 58], [238, 104, 220], [99, 1, 115], [218, 175, 185], [70, 85, 25], [94, 35, 62], [94, 72, 19], [17, 179, 99], [124, 140, 109], [68, 5, 6], [97, 122, 158], [74, 179, 18], [168, 75, 243], [44, 125, 108], [219, 63, 49], [199, 183, 206], [179, 55, 185], [244, 223, 161], [66, 217, 204], [213, 35, 96], [74, 158, 134], [113, 186, 23], [171, 48, 128], [172, 109, 212], [254, 212, 88], [74, 173, 38], [137, 124, 151], [155, 50, 67], [2, 175, 49], [42, 47, 192], [152, 150, 121], [87, 116, 32], [124, 136, 136], [173, 30, 64], [144, 143, 37], [117, 102, 85], [133, 32, 232], [117, 200, 181], [253, 123, 95], [248, 164, 209], [226, 206, 33], [32, 222, 162], [237, 102, 96], [66, 11, 193], [39, 67, 6], [224, 51, 100], [167, 0, 107], [230, 147, 231], [177, 131, 36], [194, 191, 194], [197, 224, 160], [52, 92, 20], [149, 124, 114], [236, 138, 48], [25, 1, 97], [186, 38, 177], [248, 0, 147], [169, 171, 14], [106, 22, 252], [21, 66, 176], [243, 228, 55], [174, 17, 149], [194, 193, 70], [26, 196, 104], [106, 207, 44], [181, 195, 106], [22, 84, 181], [113, 166, 27], [8, 249, 44], [142, 22, 220], [7, 170, 195], [95, 71, 94], [25, 250, 49], [201, 179, 93], [93, 54, 224], [79, 141, 72], [112, 246, 21], [78, 59, 24], [42, 216, 147], [202, 172, 250], [212, 114, 11], [223, 240, 160], [207, 76, 95], [93, 143, 165], [73, 83, 134], [24, 173, 24], [148, 223, 171], [59, 65, 5], [116, 42, 74], [240, 238, 99], [73, 186, 81], [108, 177, 203], [245, 168, 197], [240, 134, 136], [152, 117, 216], [55, 51, 45], [96, 114, 7], [33, 18, 205], [161, 113, 80], [71, 196, 61], [224, 197, 227], [166, 96, 7], [28, 192, 230], [197, 162, 226], [49, 2, 255], [145, 18, 113], [194, 160, 192], [74, 4, 173], [105, 84, 100], [2, 134, 124], [177, 83, 100], [28, 215, 217], [105, 194, 80], [120, 127, 94], [32, 72, 0], [43, 213, 99], [47, 138, 42], [114, 108, 1], [130, 238, 145], [230, 91, 115], [127, 126, 115], [212, 12, 210], [253, 115, 245], [185, 195, 197], [81, 215, 34], [250, 23, 41], [41, 76, 121], [78, 132, 174], [220, 74, 174], [136, 19, 11], [217, 8, 187], [221, 12, 236], [103, 184, 19], [157, 103, 171], [109, 40, 186], [0, 93, 7], [38, 43, 109], [110, 173, 66], [85, 184, 12], [54, 66, 55], [192, 157, 46], [219, 157, 69], [41, 232, 255], [164, 197, 246], [71, 108, 14], [237, 13, 97], [112, 98, 44], [50, 254, 110], [30, 27, 5], [100, 157, 182], [251, 65, 251], [211, 189, 82], [209, 74, 161], [68, 190, 123], [120, 128, 239], [73, 118, 230], [40, 44, 198], [169, 158, 28], [209, 87, 54], [40, 182, 13], [52, 129, 180], [116, 40, 91], [111, 165, 53], [43, 106, 114], [7, 225, 175], [72, 31, 170], [250, 25, 225], [192, 25, 15], [184, 211, 255], [72, 154, 186], [77, 233, 115], [242, 109, 189], [71, 25, 184], [103, 38, 4], [241, 27, 109], [88, 229, 71], [178, 120, 89], [36, 218, 246], [73, 158, 80], [235, 49, 44], [250, 159, 224], [242, 33, 158], [235, 78, 170], [230, 134, 34], [185, 173, 17], [85, 217, 130], [244, 207, 151], [40, 110, 78], [105, 164, 133], [121, 65, 46], [76, 243, 164], [131, 219, 59], [237, 111, 228], [140, 211, 207], [97, 82, 164], [127, 81, 135], [245, 49, 30], [62, 185, 237], [57, 9, 31], [33, 184, 197], [127, 137, 108], [78, 228, 180], [9, 180, 193], [77, 40, 89], [1, 151, 43], [22, 121, 53], [63, 29, 126], [230, 69, 174], [212, 27, 91], [7, 107, 245], [229, 190, 25], [8, 241, 36], [128, 242, 13], [75, 208, 249], [5, 142, 133], [168, 188, 224], [210, 158, 156], [212, 96, 160], [90, 110, 87], [123, 166, 165], [71, 82, 37], [241, 174, 68], [79, 104, 170], [63, 20, 217], [159, 116, 171], [171, 124, 99], [13, 206, 178], [154, 18, 47], [8, 179, 35], [132, 22, 141], [255, 190, 237], [56, 253, 56], [222, 181, 239], [115, 107, 120], [49, 199, 182], [242, 71, 15], [49, 236, 236], [90, 137, 197], [158, 111, 207], [255, 195, 182], [63, 137, 244], [102, 254, 143], [31, 66, 152], [60, 75, 39], [92, 182, 64], [10, 151, 68], [126, 44, 134], [58, 77, 236], [165, 165, 0], [182, 101, 240], [149, 13, 51], [16, 105, 72], [107, 140, 122], [162, 247, 151], [110, 220, 71], [14, 76, 131], [142, 206, 179], [77, 136, 214], [103, 31, 251], [87, 32, 44], [120, 181, 229], [70, 128, 5], [117, 84, 26], [206, 21, 251], [134, 86, 246], [115, 14, 132], [224, 254, 58], [177, 117, 252], [225, 184, 210], [143, 94, 95], [244, 3, 58], [182, 208, 180], [104, 108, 234], [186, 158, 130], [118, 232, 184], [245, 17, 125], [98, 242, 79], [35, 143, 241], [59, 154, 39], [206, 235, 199], [174, 146, 10], [62, 16, 139], [190, 167, 57], [73, 239, 236], [245, 137, 154], [37, 219, 233], [143, 69, 56], [82, 214, 203], [105, 12, 1], [113, 235, 223], [171, 105, 17], [128, 110, 5], [51, 116, 255], [56, 62, 45], [189, 188, 142], [242, 93, 186], [72, 57, 201], [75, 62, 63], [127, 195, 235], [68, 14, 109], [149, 229, 133], [160, 19, 3], [93, 124, 168], [58, 89, 191], [198, 230, 61], [195, 229, 132], [224, 61, 155], [21, 38, 254], [248, 199, 20], [103, 99, 107], [138, 118, 100], [69, 85, 34], [133, 250, 6], [206, 223, 124], [196, 128, 39], [136, 53, 2], [202, 221, 157], [133, 201, 249], [223, 28, 13], [180, 72, 171], [249, 169, 237], [255, 241, 125], [8, 174, 96], [225, 60, 7], [199, 232, 235], [47, 30, 220], [223, 121, 140], [141, 128, 111], [148, 104, 223], [236, 244, 28], [151, 145, 46], [101, 163, 47], [27, 215, 68], [59, 204, 30], [48, 6, 141], [151, 111, 44], [139, 61, 139], [155, 170, 186], [100, 102, 123], [207, 30, 213], [213, 175, 84], [162, 198, 101], [17, 26, 252], [204, 49, 61], [233, 156, 202], [89, 87, 60], [183, 153, 231], [34, 59, 107], [111, 43, 71], [96, 248, 6], [3, 127, 236], [231, 215, 201], [209, 142, 208], [42, 105, 223], [15, 34, 186], [42, 146, 43], [21, 8, 246], [114, 33, 218], [15, 185, 83], [54, 197, 137], [76, 225, 76], [93, 161, 194], [78, 197, 109], [25, 66, 176], [144, 222, 232], [183, 73, 99], [124, 208, 71], [104, 84, 60], [220, 60, 239], [201, 92, 135], [65, 12, 59], [7, 140, 26], [43, 219, 95], [83, 199, 35], [140, 44, 216], [125, 135, 234], [175, 102, 159], [3, 57, 248], [79, 166, 221], [132, 187, 160], [43, 157, 147], [151, 194, 143], [93, 2, 104], [132, 52, 199], [228, 154, 200], [78, 75, 44], [40, 152, 117], [254, 145, 42], [70, 109, 96], [178, 163, 136], [168, 83, 32], [177, 219, 192], [173, 60, 51], [178, 58, 16], [231, 189, 166], [229, 175, 192], [190, 112, 253], [169, 207, 89], [239, 12, 253], [4, 211, 77], [115, 120, 38], [178, 246, 114], [36, 69, 1], [209, 248, 152], [65, 145, 144], [239, 177, 64], [88, 188, 36], [96, 231, 225], [153, 201, 49], [251, 249, 37], [249, 167, 199], [227, 75, 49], [229, 71, 174], [126, 173, 234], [219, 18, 251], [4, 108, 97], [24, 41, 13], [167, 88, 206], [185, 225, 26], [218, 207, 34], [70, 254, 177], [237, 142, 145], [57, 50, 192], [123, 41, 107], [204, 89, 32], [98, 67, 234], [181, 209, 137], [144, 115, 105], [200, 211, 254], [105, 7, 133], [100, 146, 71], [75, 28, 73], [255, 86, 176], [136, 118, 181], [211, 251, 16], [219, 27, 246], [248, 150, 17], [21, 221, 61], [95, 172, 196], [202, 161, 170], [56, 147, 8], [193, 112, 45], [213, 140, 90], [244, 103, 121], [88, 246, 33], [178, 161, 78], [77, 67, 214], [235, 97, 242], [136, 27, 173], [65, 52, 166], [89, 33, 227], [4, 128, 191], [70, 232, 233], [107, 6, 105], [175, 171, 172], [43, 129, 69], [190, 216, 166], [249, 18, 66], [160, 35, 170], [42, 41, 137], [42, 162, 17], [63, 39, 55], [55, 144, 2], [119, 79, 120], [11, 109, 39], [79, 22, 20], [34, 56, 10], [163, 19, 159], [55, 253, 79], [212, 48, 5], [41, 142, 155], [209, 2, 126], [187, 10, 153], [229, 70, 88], [51, 246, 169], [237, 202, 199], [107, 204, 52], [183, 3, 91], [234, 59, 14], [73, 252, 16], [99, 188, 105], [106, 145, 102], [38, 51, 165], [36, 208, 248], [58, 72, 247], [122, 1, 28], [242, 0, 3], [252, 184, 103], [144, 38, 106], [225, 137, 197], [63, 63, 151], [86, 19, 228], [173, 116, 44], [229, 87, 171], [31, 98, 141], [81, 148, 202], [188, 243, 220], [153, 120, 50], [21, 148, 218], [82, 69, 7], [200, 184, 30], [61, 84, 233], [71, 130, 40], [167, 250, 180], [100, 186, 2], [38, 8, 65], [139, 152, 81], [169, 89, 94], [226, 212, 2], [189, 113, 92], [4, 203, 139], [192, 255, 190], [179, 40, 2], [149, 8, 5], [99, 39, 212], [119, 74, 88], [1, 253, 56], [194, 95, 181], [36, 217, 242], [72, 52, 7], [189, 8, 16], [249, 217, 113], [162, 210, 42], [244, 213, 141], [47, 5, 244], [31, 44, 83], [11, 111, 182], [8, 249, 69], [115, 180, 196], [127, 170, 50], [199, 136, 6], [233, 169, 132], [171, 51, 171], [191, 150, 147], [9, 31, 161], [144, 204, 171], [174, 97, 34], [160, 213, 1], [168, 188, 202], [104, 60, 245], [157, 49, 248], [253, 86, 46], [205, 159, 203], [114, 94, 12], [134, 85, 54], [143, 69, 186], [96, 47, 41], [238, 216, 87], [122, 166, 156], [150, 74, 110], [23, 53, 90], [61, 31, 58], [209, 93, 158], [166, 211, 42], [246, 253, 164], [125, 66, 242], [113, 226, 37], [207, 107, 145], [208, 179, 163], [254, 225, 101], [133, 218, 8], [224, 173, 182], [75, 4, 236], [254, 97, 82], [46, 39, 102], [97, 152, 71], [81, 89, 199], [97, 211, 172], [218, 159, 233], [190, 42, 43], [135, 5, 22], [128, 235, 123], [174, 10, 121], [143, 118, 157], [26, 48, 17], [156, 129, 7], [33, 53, 166], [189, 66, 237], [10, 66, 212], [229, 58, 56], [227, 221, 97], [228, 115, 106], [105, 49, 235], [75, 19, 5], [142, 149, 213], [221, 128, 244], [141, 213, 89], [156, 99, 13], [183, 73, 39], [90, 139, 127], [130, 129, 159], [110, 22, 245], [75, 93, 133], [250, 173, 38], [228, 46, 6], [31, 236, 229], [27, 183, 18], [88, 155, 37], [62, 133, 200], [7, 9, 118], [184, 84, 204], [226, 82, 50], [236, 121, 222], [234, 124, 137], [87, 178, 121], [58, 12, 119], [13, 129, 107], [133, 61, 14], [24, 44, 60], [158, 217, 6], [177, 248, 107], [110, 57, 57], [207, 199, 8], [158, 21, 15], [100, 172, 1], [210, 58, 139], [79, 180, 141], [30, 150, 80], [132, 123, 35], [18, 91, 85], [47, 212, 146], [78, 161, 2], [72, 194, 209], [40, 215, 75], [17, 154, 19], [73, 38, 18], [173, 1, 32], [246, 131, 70], [3, 251, 197], [53, 142, 90], [196, 77, 20], [198, 32, 15], [233, 199, 70], [223, 238, 91], [127, 216, 60], [151, 183, 188], [113, 186, 136], [87, 191, 193], [221, 114, 29], [97, 35, 207], [126, 83, 47], [223, 225, 241], [196, 127, 18], [226, 97, 67], [1, 36, 33], [140, 123, 205], [253, 2, 90], [33, 152, 128], [7, 47, 240], [53, 13, 81], [217, 174, 220], [128, 41, 179], [24, 97, 207], [239, 57, 170], [225, 110, 218], [233, 37, 222], [107, 106, 116], [110, 72, 77], [164, 170, 160], [74, 93, 183], [166, 128, 230], [163, 141, 220], [62, 155, 30], [161, 207, 198], [73, 183, 27], [231, 92, 206], [196, 43, 211], [246, 245, 162], [217, 83, 46], [217, 183, 155], [235, 54, 237], [229, 114, 226], [158, 178, 147], [15, 185, 54], [190, 162, 166], [60, 123, 199], [245, 191, 118], [110, 249, 140], [31, 175, 32], [39, 50, 30], [116, 34, 69], [166, 128, 195], [134, 142, 58], [251, 80, 197], [99, 9, 95], [108, 231, 59], [229, 26, 100], [33, 219, 148], [218, 234, 154], [181, 58, 133], [85, 139, 159], [167, 22, 144], [183, 45, 208], [3, 192, 83], [15, 252, 153], [122, 69, 208], [126, 158, 89], [50, 136, 173], [167, 90, 205], [235, 10, 166], [55, 170, 125], [30, 146, 157], [238, 227, 151], [76, 93, 42], [96, 151, 202], [177, 80, 175], [134, 198, 77], [94, 226, 47], [180, 137, 63], [134, 86, 51], [99, 111, 47], [205, 84, 27], [245, 6, 52], [101, 188, 121], [46, 18, 135], [254, 10, 10], [9, 138, 227], [97, 180, 6], [14, 0, 165], [123, 10, 113], [32, 153, 199], [54, 48, 137], [168, 58, 218], [52, 9, 133], [131, 196, 184], [240, 186, 65], [30, 176, 200], [181, 8, 5], [15, 96, 52], [15, 176, 213], [32, 234, 5], [5, 47, 160], [184, 220, 123], [200, 142, 204], [61, 5, 128], [118, 121, 79], [17, 197, 205], [23, 133, 208], [224, 178, 230], [23, 246, 47], [191, 138, 22], [155, 12, 107], [172, 15, 80], [171, 210, 185], [234, 20, 166], [238, 233, 106], [213, 222, 60], [198, 95, 192], [154, 159, 172], [239, 5, 123], [178, 253, 81], [93, 144, 231], [254, 252, 201], [83, 89, 21], [248, 17, 250], [180, 19, 109], [25, 125, 170], [201, 194, 97], [71, 134, 90], [82, 3, 205], [238, 215, 163], [34, 44, 34], [82, 89, 179], [222, 184, 128], [103, 92, 146], [104, 70, 39], [240, 90, 91], [164, 178, 43], [57, 165, 89], [29, 77, 8], [98, 18, 84], [179, 9, 245], [153, 147, 207], [248, 184, 189], [178, 142, 1], [3, 178, 218], [60, 236, 252], [40, 67, 235], [13, 40, 234], [26, 251, 43], [39, 232, 92], [17, 219, 235], [36, 253, 104], [188, 243, 140], [7, 67, 109], [97, 247, 20], [203, 19, 236], [134, 198, 32], [10, 127, 112], [88, 117, 126], [220, 6, 185], [5, 41, 116], [41, 116, 55], [111, 155, 41], [174, 82, 21], [12, 192, 95], [228, 196, 54], [178, 115, 174], [181, 244, 18], [84, 229, 238], [122, 14, 130], [188, 174, 247], [195, 83, 183], [59, 200, 20], [223, 15, 18], [46, 195, 77], [144, 10, 40], [179, 165, 28], [175, 68, 76], [112, 245, 116], [105, 33, 147], [131, 85, 46], [91, 161, 240], [162, 72, 159], [92, 139, 12], [125, 190, 17], [128, 144, 228], [167, 230, 77], [17, 218, 94], [15, 73, 34], [253, 187, 146], [212, 66, 47], [99, 201, 29], [220, 232, 106], [140, 243, 48], [48, 244, 56], [126, 34, 96], [203, 8, 198], [222, 22, 175], [170, 168, 252], [69, 123, 30], [89, 192, 173], [242, 85, 235], [33, 154, 144], [138, 60, 218], [26, 55, 101], [63, 86, 162], [75, 76, 23], [192, 193, 234], [65, 159, 10], [88, 147, 209], [150, 177, 244], [143, 220, 128], [100, 195, 141], [79, 142, 38], [232, 241, 55], [5, 28, 194], [22, 166, 215], [169, 78, 66], [125, 5, 181], [168, 56, 2], [26, 14, 163], [245, 9, 86], [147, 189, 212], [13, 148, 34], [76, 99, 153], [246, 153, 40], [184, 12, 203], [172, 35, 31], [248, 3, 177], [243, 28, 245], [46, 174, 67], [112, 157, 106], [238, 226, 99], [132, 30, 95], [171, 213, 139], [152, 103, 169], [194, 54, 73], [1, 225, 129], [210, 116, 181], [244, 171, 52], [25, 39, 22], [103, 255, 27], [223, 51, 72], [133, 156, 254], [153, 188, 209], [2, 182, 41], [197, 97, 109], [176, 170, 8], [5, 164, 129], [211, 208, 209], [142, 103, 101], [45, 27, 15], [164, 139, 1], [6, 151, 242], [236, 110, 213], [245, 190, 99], [233, 175, 57], [14, 154, 104], [14, 255, 192], [77, 213, 62], [7, 188, 102], [251, 97, 154], [131, 6, 86], [254, 0, 216], [135, 183, 250], [82, 4, 117], [207, 187, 41], [86, 145, 88], [116, 159, 101], [180, 208, 78], [198, 133, 254], [137, 219, 59], [100, 149, 89], [144, 158, 28], [91, 66, 36], [204, 31, 50], [200, 6, 203], [210, 224, 48], [146, 189, 223], [213, 2, 247], [201, 204, 119], [170, 14, 120], [107, 68, 2], [80, 27, 22], [230, 62, 51], [84, 177, 135], [110, 220, 140], [216, 192, 168], [96, 191, 219], [4, 17, 35], [205, 35, 151], [70, 30, 226], [153, 55, 249], [111, 209, 218], [3, 90, 19], [87, 67, 99], [211, 250, 16], [72, 193, 18], [128, 245, 220], [131, 117, 154], [219, 231, 103], [118, 202, 157], [73, 113, 207], [149, 214, 167], [17, 123, 63], [99, 127, 108], [94, 31, 244], [82, 97, 119], [136, 145, 207], [144, 43, 107], [19, 241, 50], [114, 111, 164], [61, 224, 111], [35, 59, 196], [63, 123, 55], [40, 17, 106], [11, 92, 187], [125, 100, 148], [247, 162, 117], [161, 65, 160], [171, 196, 244], [246, 66, 165], [161, 30, 89], [101, 189, 105], [46, 168, 57], [52, 185, 162], [16, 214, 44], [182, 107, 255], [112, 110, 151], [115, 35, 72], [67, 25, 79], [122, 139, 144], [31, 63, 163], [154, 199, 210], [200, 100, 7], [111, 173, 133], [96, 14, 221], [52, 26, 133], [238, 255, 211], [206, 111, 65], [247, 198, 60], [83, 192, 42], [218, 185, 81], [105, 84, 42], [195, 66, 230], [155, 86, 115], [10, 222, 49], [115, 245, 168], [236, 137, 215], [111, 86, 149], [189, 0, 85], [61, 147, 250], [41, 84, 148], [120, 209, 158], [56, 251, 189], [206, 8, 110], [233, 243, 15], [224, 189, 68], [167, 175, 134], [227, 195, 35], [212, 166, 30], [52, 225, 96], [179, 181, 118], [100, 227, 148], [165, 201, 129], [230, 222, 70], [218, 201, 98], [129, 85, 59], [209, 86, 29], [206, 240, 145], [225, 185, 15], [178, 206, 195], [7, 175, 79], [201, 141, 234], [213, 220, 119], [250, 110, 144], [40, 212, 143], [82, 229, 126], [217, 243, 6], [201, 123, 244], [11, 250, 229], [116, 246, 59], [230, 218, 245], [221, 53, 139], [155, 65, 250], [56, 43, 240], [203, 34, 169], [164, 106, 72], [195, 92, 79], [76, 216, 28], [225, 14, 126], [116, 89, 215], [110, 18, 55], [139, 184, 230], [75, 38, 49], [138, 243, 127], [71, 84, 58], [190, 227, 237], [12, 95, 172], [49, 252, 94], [237, 11, 34], [40, 242, 247], [111, 12, 188], [248, 57, 194], [129, 44, 51], [95, 106, 189], [22, 113, 210], [79, 241, 148], [2, 136, 252], [67, 133, 29], [41, 201, 88], [128, 7, 131], [253, 153, 29], [118, 84, 27], [196, 18, 249], [153, 202, 39], [10, 96, 241], [158, 238, 214], [119, 87, 189], [150, 62, 209], [116, 233, 48], [216, 20, 231], [240, 71, 178], [154, 44, 150], [72, 67, 158], [102, 164, 157], [169, 220, 86], [115, 78, 44], [230, 198, 247], [206, 104, 194], [66, 154, 105], [44, 132, 14], [252, 43, 42], [70, 156, 161], [24, 13, 100], [187, 209, 132], [176, 182, 82], [216, 191, 56], [81, 62, 202], [146, 46, 166], [63, 209, 217], [8, 19, 137], [56, 86, 181], [30, 156, 178], [59, 74, 106], [43, 115, 23], [254, 127, 73], [2, 87, 154], [15, 78, 229], [21, 21, 209], [205, 36, 86], [72, 27, 42], [1, 218, 226], [145, 218, 211], [66, 132, 144], [191, 10, 230], [72, 131, 67], [59, 248, 46], [113, 148, 215], [35, 127, 187], [189, 255, 215], [193, 249, 183], [254, 116, 142], [27, 142, 61], [131, 194, 228], [118, 140, 241], [26, 110, 184], [43, 67, 173], [86, 103, 218], [187, 126, 205], [215, 149, 87], [173, 74, 120], [39, 1, 23], [71, 48, 91], [160, 63, 30], [201, 117, 185], [132, 171, 239], [143, 172, 132], [12, 1, 163], [27, 84, 3], [37, 168, 193], [33, 45, 94], [190, 249, 159], [222, 18, 255], [67, 126, 44], [162, 90, 0], [168, 212, 210], [237, 187, 254], [19, 108, 123], [14, 179, 23], [33, 159, 59], [150, 217, 220], [42, 44, 204], [96, 98, 248], [35, 125, 88], [140, 136, 178], [192, 11, 213], [139, 233, 217], [61, 156, 215], [101, 208, 79], [189, 64, 18], [182, 78, 217], [135, 48, 248], [100, 5, 89], [6, 98, 229], [237, 176, 55], [161, 194, 235], [159, 245, 209], [22, 173, 23], [249, 224, 121], [15, 178, 24], [232, 141, 134], [131, 220, 30], [36, 149, 28], [84, 182, 34], [173, 253, 243], [60, 163, 252], [95, 234, 77], [127, 25, 199], [110, 222, 1], [64, 174, 214], [132, 206, 253], [124, 26, 201], [211, 53, 152], [182, 169, 35], [142, 219, 88], [33, 85, 57], [0, 144, 95], [203, 255, 138], [86, 202, 64], [17, 251, 56], [16, 204, 214], [117, 235, 136], [68, 167, 145], [201, 23, 59], [195, 126, 121], [241, 54, 74], [97, 235, 50], [27, 45, 79], [8, 65, 74], [82, 154, 219], [147, 26, 191], [31, 149, 220], [58, 54, 16], [245, 204, 149], [82, 97, 82], [7, 26, 50], [18, 35, 26], [150, 95, 187], [49, 175, 100], [237, 25, 12], [75, 112, 129], [30, 160, 22], [37, 158, 163], [140, 187, 123], [20, 30, 249], [62, 3, 19], [7, 180, 249], [61, 93, 27], [209, 22, 209], [144, 122, 54], [86, 206, 189], [99, 240, 238], [203, 88, 212], [166, 137, 18], [7, 205, 215], [254, 205, 205], [143, 121, 134], [148, 223, 89], [83, 170, 149], [74, 102, 175], [136, 141, 3], [231, 43, 130], [213, 26, 235], [167, 195, 2], [208, 60, 213], [13, 152, 190], [212, 63, 53], [189, 228, 69], [13, 184, 122], [246, 184, 35], [67, 186, 253], [196, 129, 239], [38, 192, 127], [36, 46, 175], [140, 3, 100], [226, 124, 19], [112, 176, 196], [155, 40, 153], [166, 150, 204], [8, 254, 252], [139, 107, 97], [105, 92, 213], [124, 132, 35], [121, 19, 74], [9, 11, 130], [139, 114, 18], [46, 114, 193], [0, 155, 187], [104, 50, 201], [181, 170, 186], [36, 36, 127], [239, 227, 221], [199, 150, 117], [162, 234, 130], [150, 237, 103], [207, 210, 108], [237, 4, 107], [255, 147, 80], [193, 35, 92], [148, 28, 243], [184, 117, 230], [132, 53, 200], [197, 155, 190], [62, 104, 104], [111, 77, 31], [66, 2, 228], [87, 143, 132], [174, 90, 110], [254, 219, 29], [229, 220, 1], [240, 82, 201], [154, 223, 181], [174, 241, 21], [77, 240, 148], [118, 153, 141], [59, 125, 147], [122, 140, 25], [212, 242, 24], [30, 190, 127], [13, 163, 4], [167, 252, 153], [156, 58, 239], [61, 186, 81], [173, 173, 145], [203, 82, 67], [31, 172, 197], [123, 197, 77], [103, 165, 21], [5, 81, 102], [102, 140, 188], [211, 172, 105], [61, 177, 100], [168, 246, 226], [45, 14, 101], [23, 121, 154], [62, 22, 55], [5, 125, 213], [113, 98, 53], [60, 107, 168], [125, 178, 62], [236, 244, 216], [133, 162, 84], [164, 160, 34], [34, 61, 97], [115, 233, 216], [72, 39, 73], [6, 213, 60], [199, 124, 22], [137, 88, 78], [38, 99, 196], [121, 18, 0], [139, 238, 183], [124, 91, 241], [253, 224, 181], [54, 212, 198], [144, 91, 23], [203, 165, 123], [214, 59, 187], [172, 37, 225], [102, 50, 0], [189, 39, 64], [128, 91, 6], [6, 51, 181], [241, 142, 201], [14, 39, 43], [235, 70, 198], [90, 47, 25], [51, 1, 64], [95, 142, 132], [109, 39, 25], [212, 12, 210], [121, 94, 200], [162, 101, 201], [91, 189, 55], [183, 119, 194], [73, 118, 210], [14, 54, 109], [210, 53, 120], [54, 90, 24], [42, 230, 49], [239, 94, 84], [162, 82, 132], [23, 160, 198], [104, 75, 72], [90, 251, 44], [164, 52, 74], [146, 212, 80], [18, 140, 12], [219, 115, 180], [125, 88, 127], [223, 42, 95], [130, 255, 231], [168, 16, 166], [94, 211, 220], [218, 15, 19], [186, 179, 189], [184, 86, 4], [1, 88, 201], [139, 242, 228], [93, 73, 65], [191, 36, 212], [16, 224, 31], [106, 87, 27], [233, 83, 116], [71, 5, 220], [1, 1, 68], [221, 14, 243], [119, 63, 145], [171, 202, 71], [158, 7, 201], [133, 70, 19], [79, 89, 182], [186, 165, 108], [250, 110, 249], [122, 70, 108], [204, 162, 96], [35, 139, 23], [53, 31, 28], [95, 91, 164], [25, 10, 77], [252, 162, 54], [168, 140, 47], [134, 23, 224], [217, 47, 14], [175, 141, 51], [117, 14, 45], [111, 56, 249], [24, 225, 181], [231, 142, 147], [170, 251, 40], [33, 88, 51], [28, 100, 63], [24, 44, 152], [89, 40, 200], [239, 154, 246], [140, 58, 162], [238, 144, 244], [115, 218, 139], [232, 204, 71], [174, 22, 60], [190, 12, 127], [169, 8, 89], [250, 219, 171], [113, 82, 228], [31, 90, 46], [140, 100, 48], [127, 36, 126], [46, 93, 61], [80, 18, 33], [20, 173, 43], [151, 96, 31], [179, 169, 80], [95, 36, 162], [100, 241, 116], [11, 145, 167], [247, 146, 29], [255, 158, 38], [255, 172, 214], [48, 69, 41], [179, 250, 165], [8, 81, 180], [76, 157, 168], [17, 116, 131], [140, 148, 176], [140, 207, 192], [200, 214, 7], [212, 76, 235], [20, 144, 221], [4, 73, 45], [94, 57, 41], [30, 162, 125], [170, 235, 9], [222, 148, 109], [84, 4, 9], [31, 132, 71], [197, 105, 115], [163, 114, 86], [181, 72, 6], [76, 102, 53], [225, 15, 72], [36, 98, 31], [160, 198, 8], [193, 158, 22], [251, 39, 252], [22, 183, 202], [184, 56, 141], [106, 169, 47], [254, 51, 6], [149, 35, 74], [224, 179, 18], [85, 17, 214], [220, 161, 132], [199, 156, 65], [156, 0, 196], [155, 117, 170], [2, 46, 110], [120, 42, 53], [42, 45, 90], [165, 208, 87], [113, 54, 96], [34, 132, 202], [205, 117, 204], [129, 7, 145], [18, 70, 231], [64, 135, 237], [23, 201, 57], [254, 112, 234], [227, 217, 30], [0, 225, 220], [152, 242, 61], [106, 193, 154], [255, 203, 66], [40, 163, 165], [166, 214, 28], [92, 49, 74], [97, 186, 111], [46, 202, 27], [214, 223, 252], [215, 61, 75], [36, 204, 232], [71, 99, 230], [224, 120, 252], [251, 87, 50], [146, 0, 39], [107, 40, 35], [107, 21, 67], [206, 102, 209], [44, 7, 99], [99, 222, 2], [116, 112, 175], [113, 107, 186], [80, 247, 119], [115, 197, 107], [157, 255, 231], [233, 157, 46], [109, 68, 110], [70, 242, 132], [10, 172, 112], [73, 147, 125], [23, 173, 75], [233, 155, 204], [2, 222, 21], [155, 214, 124], [160, 211, 184], [91, 25, 107], [61, 143, 39], [65, 130, 211], [18, 219, 63], [59, 39, 154], [233, 237, 130], [241, 249, 178], [0, 6, 196], [106, 22, 70], [89, 236, 138], [59, 175, 210], [197, 248, 90], [17, 143, 219], [133, 186, 29], [237, 133, 13], [219, 126, 118], [47, 194, 251], [57, 65, 98], [230, 199, 102], [140, 242, 226], [57, 170, 160], [67, 82, 96], [248, 44, 176], [44, 149, 66], [37, 62, 139], [210, 229, 182], [187, 224, 48], [234, 253, 222], [207, 69, 111], [127, 88, 88], [17, 54, 231], [163, 161, 145], [53, 3, 209], [144, 188, 96], [171, 211, 93], [105, 180, 157], [160, 24, 202], [37, 56, 64], [64, 232, 188], [27, 122, 8], [238, 169, 200], [187, 75, 22], [46, 11, 187], [139, 197, 226], [85, 250, 18], [184, 115, 179], [190, 31, 114], [120, 126, 208], [208, 35, 129], [128, 54, 180], [169, 32, 0], [229, 150, 45], [168, 190, 199], [97, 212, 33], [12, 233, 20], [27, 178, 23], [62, 39, 64], [110, 131, 100], [182, 233, 225], [233, 160, 102], [35, 12, 170], [31, 79, 87], [147, 51, 36], [183, 189, 31], [155, 246, 252], [63, 171, 238], [186, 165, 126], [197, 87, 106], [82, 171, 153], [105, 193, 239], [51, 120, 136], [247, 115, 8], [47, 123, 13], [50, 73, 255], [36, 234, 190], [170, 23, 243], [140, 160, 3], [47, 27, 160], [172, 119, 213], [226, 136, 35], [208, 168, 143], [37, 170, 159], [212, 200, 127], [225, 21, 159], [64, 47, 37], [126, 243, 134], [222, 246, 162], [128, 143, 62], [32, 180, 243], [70, 139, 98], [108, 57, 99], [106, 223, 226], [99, 51, 38], [12, 28, 181], [204, 188, 74], [236, 18, 41], [227, 220, 230], [11, 57, 152], [188, 176, 132], [163, 200, 88], [166, 207, 239], [196, 239, 159], [0, 55, 26], [46, 64, 253], [229, 255, 254], [242, 151, 199], [31, 46, 11], [130, 209, 60], [4, 90, 217], [7, 220, 127], [253, 29, 129], [46, 55, 175], [215, 104, 18], [218, 217, 235], [207, 49, 213], [6, 240, 227], [170, 213, 156], [45, 237, 196], [201, 38, 14], [60, 174, 192], [22, 13, 247], [126, 177, 150], [47, 106, 122], [122, 160, 129], [200, 180, 89], [135, 180, 116], [204, 79, 171], [251, 195, 93], [73, 173, 52], [212, 16, 71], [244, 38, 16], [206, 85, 219], [109, 204, 48], [219, 73, 191], [134, 89, 11], [35, 19, 40], [238, 45, 44], [11, 88, 141], [5, 77, 173], [54, 91, 172], [132, 249, 151], [46, 146, 152], [78, 144, 103], [32, 171, 72], [238, 204, 207], [85, 68, 209], [247, 202, 155], [91, 227, 161], [100, 71, 203], [161, 200, 106], [115, 21, 205], [249, 16, 208], [62, 93, 149], [211, 222, 226], [204, 132, 175], [248, 84, 86], [110, 158, 67], [116, 80, 8], [64, 251, 153], [190, 192, 221], [41, 185, 148], [173, 111, 44], [178, 93, 18], [138, 81, 57], [159, 82, 25], [49, 131, 114], [174, 87, 183], [10, 114, 65], [25, 92, 255], [90, 168, 187], [167, 41, 190], [12, 228, 161], [107, 153, 22], [185, 199, 199], [110, 100, 187], [96, 188, 32], [146, 7, 65], [181, 106, 155], [106, 95, 165], [77, 180, 72], [120, 249, 71], [59, 125, 218], [36, 43, 185], [228, 86, 151], [207, 101, 219], [92, 167, 206], [215, 156, 126], [153, 208, 169], [3, 192, 247], [159, 135, 45], [8, 86, 137], [146, 95, 116], [165, 96, 168], [56, 45, 27], [236, 207, 127], [181, 29, 109], [140, 127, 36], [161, 141, 224], [119, 178, 81], [75, 135, 166], [185, 185, 131], [98, 165, 110], [0, 96, 70], [217, 199, 57], [135, 244, 158], [50, 165, 114], [148, 78, 12], [40, 228, 16], [219, 143, 39], [212, 244, 81], [54, 144, 68], [47, 111, 41], [226, 123, 213], [213, 120, 27], [232, 192, 175], [132, 182, 154], [203, 197, 151], [245, 230, 172], [183, 255, 25], [78, 9, 30], [159, 139, 94], [16, 134, 135], [96, 118, 190], [63, 16, 234], [35, 124, 145], [79, 204, 157], [10, 194, 249], [32, 1, 48], [37, 116, 110], [220, 225, 235], [180, 189, 196], [199, 9, 164], [118, 222, 47], [87, 229, 219], [126, 234, 59], [237, 175, 39], [176, 59, 33], [224, 41, 64], [58, 83, 119], [134, 53, 75], [43, 100, 12], [128, 109, 136], [221, 50, 91], [224, 34, 47], [201, 110, 190], [156, 41, 196], [80, 188, 65], [148, 176, 233], [4, 137, 126], [193, 241, 230], [128, 120, 110], [68, 78, 52], [212, 49, 228], [142, 250, 220], [150, 232, 67], [149, 247, 172], [193, 125, 185], [85, 5, 19], [23, 59, 161], [54, 4, 121], [77, 6, 139], [143, 213, 17], [69, 43, 17], [94, 162, 11], [195, 48, 214], [149, 135, 69], [230, 251, 118], [221, 149, 53], [132, 23, 213], [120, 88, 83], [208, 192, 18], [57, 15, 12], [68, 182, 238], [64, 68, 138], [146, 57, 147], [25, 126, 138], [27, 36, 7], [93, 129, 101], [49, 166, 38], [141, 118, 122], [187, 202, 2], [228, 215, 191], [237, 56, 202], [189, 226, 114], [76, 37, 114], [6, 51, 50], [219, 113, 243], [98, 58, 47], [30, 156, 219], [202, 212, 236], [219, 212, 16], [59, 227, 215], [167, 22, 60], [7, 116, 46], [174, 165, 33], [175, 231, 139], [228, 243, 92], [237, 139, 94], [215, 150, 106], [166, 55, 130], [4, 164, 104], [234, 187, 178], [226, 6, 185], [144, 62, 74], [53, 106, 51], [67, 186, 206], [192, 238, 94], [43, 39, 57], [195, 91, 104], [97, 167, 211], [124, 250, 112], [81, 54, 141], [116, 212, 226], [204, 235, 137], [70, 128, 38], [156, 209, 77], [229, 116, 121], [132, 219, 125], [231, 78, 54], [244, 57, 197], [53, 14, 228], [213, 160, 121], [194, 84, 66], [192, 184, 10], [238, 75, 227], [149, 155, 39], [140, 122, 163], [48, 120, 14], [150, 194, 194], [129, 37, 132], [206, 92, 114], [147, 103, 7], [244, 111, 69], [60, 204, 30], [246, 148, 157], [24, 140, 242], [225, 162, 246], [218, 115, 43], [179, 13, 70], [37, 215, 144], [223, 4, 201], [156, 75, 25], [32, 242, 163], [83, 26, 182], [11, 81, 22], [123, 56, 141], [213, 146, 29], [78, 175, 32], [153, 169, 171], [194, 235, 36], [53, 20, 182], [154, 212, 101], [73, 198, 35], [113, 111, 181], [151, 99, 20], [82, 122, 31], [177, 94, 52], [179, 254, 210], [103, 45, 88], [10, 218, 49], [217, 140, 206], [201, 81, 63], [111, 192, 243], [79, 102, 199], [198, 170, 3], [160, 14, 38], [17, 197, 182], [12, 77, 148], [11, 148, 104], [168, 77, 144], [76, 156, 129], [189, 77, 157], [169, 221, 34], [37, 102, 91], [7, 252, 9], [51, 28, 33], [55, 83, 9], [219, 245, 235], [206, 84, 190], [117, 16, 196], [170, 216, 149], [17, 155, 24], [38, 230, 250], [82, 242, 193], [85, 83, 94], [121, 164, 248], [2, 98, 148], [227, 240, 255], [80, 8, 89], [92, 97, 221], [196, 217, 221], [4, 27, 30], [172, 250, 123], [108, 57, 64], [63, 105, 163], [116, 233, 207], [74, 167, 37], [168, 190, 168], [115, 77, 224], [250, 117, 200], [237, 210, 58], [121, 28, 52], [141, 214, 110], [48, 61, 229], [172, 86, 92], [83, 136, 9], [38, 176, 241], [240, 231, 6], [17, 7, 145], [1, 193, 254], [31, 95, 20], [86, 30, 69], [164, 211, 139], [228, 18, 74], [234, 243, 12], [10, 120, 2], [196, 81, 124], [214, 196, 232], [159, 194, 23], [21, 99, 8], [20, 144, 229], [228, 182, 241], [163, 194, 34], [207, 140, 196], [137, 235, 203], [104, 247, 157], [212, 200, 150], [73, 228, 228], [67, 40, 230], [219, 153, 95], [43, 31, 100], [39, 229, 1], [247, 237, 249], [3, 125, 223], [64, 183, 155], [18, 94, 187], [48, 45, 9], [185, 78, 244], [225, 136, 51], [79, 144, 31], [79, 51, 143], [27, 33, 237], [2, 78, 97], [244, 198, 188], [0, 63, 128], [198, 43, 198], [128, 92, 179], [71, 141, 94], [20, 155, 15], [149, 140, 82], [226, 186, 22], [45, 132, 175], [21, 19, 104], [87, 112, 73], [51, 31, 139], [181, 194, 238], [104, 99, 96], [142, 199, 153], [220, 83, 86], [180, 102, 235], [138, 174, 164], [45, 200, 184], [84, 127, 254], [164, 155, 4], [213, 45, 195], [131, 145, 184], [149, 233, 80], [240, 229, 190], [48, 159, 176], [113, 88, 194], [146, 89, 172], [212, 115, 102], [18, 104, 10], [11, 198, 4], [114, 13, 56], [107, 197, 185], [25, 179, 231], [74, 96, 55], [83, 118, 116], [68, 52, 186], [233, 59, 125], [88, 90, 253], [46, 69, 118], [53, 28, 236], [244, 28, 254], [101, 158, 194], [192, 186, 24], [20, 149, 0], [30, 48, 217], [244, 191, 103], [89, 8, 154], [15, 215, 204], [114, 196, 8], [117, 67, 245], [3, 151, 145], [189, 198, 143], [218, 61, 9], [111, 246, 14], [224, 153, 149], [31, 155, 97], [78, 252, 23], [232, 255, 192], [158, 35, 210], [143, 226, 233], [71, 205, 63], [214, 109, 122], [223, 6, 19], [23, 245, 41], [37, 87, 120], [17, 49, 123], [190, 115, 49], [64, 14, 72], [49, 31, 167], [170, 24, 186], [71, 252, 59], [26, 26, 187], [38, 9, 80], [29, 196, 58], [120, 163, 101], [112, 193, 172], [157, 19, 0], [86, 78, 241], [239, 247, 53], [56, 34, 15], [85, 113, 176], [9, 16, 196], [84, 212, 137], [216, 242, 32], [6, 196, 217], [154, 96, 62], [90, 13, 180], [113, 191, 198], [75, 202, 209], [96, 67, 249], [191, 77, 221], [25, 67, 150], [188, 86, 20], [3, 183, 132], [10, 172, 235], [194, 223, 125], [195, 254, 255], [157, 188, 69], [124, 61, 141], [66, 174, 194], [137, 226, 105], [213, 55, 174], [31, 135, 162], [79, 50, 222], [209, 118, 214], [90, 210, 174], [25, 137, 251], [63, 193, 59], [246, 27, 123], [69, 187, 146], [12, 111, 71], [200, 74, 141], [104, 78, 195], [46, 28, 92], [201, 65, 98], [204, 252, 123], [247, 15, 246], [68, 135, 23], [242, 10, 177], [88, 136, 213], [110, 246, 186], [170, 130, 253], [230, 87, 0], [156, 14, 91], [115, 198, 122], [147, 149, 41], [193, 75, 167], [214, 98, 182], [236, 97, 20], [235, 138, 253], [219, 72, 80], [145, 135, 60], [98, 54, 133], [52, 226, 222], [214, 200, 32], [76, 79, 187], [214, 7, 49], [45, 171, 132], [47, 55, 87], [96, 88, 59], [94, 68, 253], [100, 38, 248], [82, 239, 130], [135, 146, 191], [74, 208, 119], [248, 202, 208], [67, 122, 19], [236, 147, 81], [245, 19, 246], [143, 47, 91], [161, 31, 191], [187, 151, 143], [61, 176, 21], [254, 102, 53], [19, 188, 150], [14, 19, 10], [155, 143, 213], [211, 136, 217], [111, 2, 114], [218, 222, 161], [202, 129, 60], [27, 125, 17], [54, 34, 179], [13, 194, 157], [21, 207, 92], [128, 1, 6], [229, 162, 241], [10, 25, 95], [7, 181, 53], [26, 23, 18], [107, 100, 133], [2, 163, 44], [26, 225, 47], [131, 160, 23], [248, 244, 171], [32, 155, 119], [163, 83, 47], [141, 236, 108], [49, 9, 214], [25, 172, 209], [38, 173, 44], [41, 92, 36], [232, 130, 214], [195, 255, 138], [53, 112, 139], [24, 121, 151], [224, 123, 195], [180, 73, 221], [0, 98, 228], [172, 195, 6], [55, 7, 149], [60, 155, 36], [155, 42, 194], [247, 215, 64], [180, 131, 25], [54, 114, 125], [71, 138, 125], [204, 118, 25], [218, 17, 139], [175, 48, 204], [120, 134, 240], [58, 203, 143], [97, 222, 169], [0, 224, 71], [239, 131, 195], [177, 147, 52], [210, 62, 22], [202, 22, 63], [174, 190, 107], [113, 125, 51], [206, 64, 53], [127, 173, 117], [19, 3, 186], [11, 219, 4], [123, 174, 19], [46, 49, 246], [55, 17, 182], [146, 4, 164], [215, 118, 93], [217, 188, 243], [214, 18, 35], [145, 26, 80], [201, 60, 79], [86, 44, 31], [64, 151, 106], [15, 14, 46], [55, 98, 43], [63, 233, 152], [152, 205, 250], [154, 107, 132], [225, 222, 28], [127, 74, 79], [248, 227, 134], [121, 12, 162], [75, 139, 133], [75, 39, 34], [59, 137, 150], [136, 216, 64], [80, 230, 32], [205, 32, 104], [144, 190, 118], [107, 82, 145], [143, 173, 170], [70, 111, 148], [13, 89, 49], [191, 161, 139], [163, 173, 22], [49, 249, 132], [170, 115, 152], [33, 189, 148], [221, 124, 200], [43, 250, 229], [64, 203, 56], [140, 219, 195], [190, 5, 125], [61, 32, 211], [138, 174, 33], [17, 16, 13], [235, 120, 46], [131, 40, 3], [48, 138, 228], [221, 102, 244], [124, 98, 48], [110, 63, 139], [2, 115, 101], [19, 71, 219], [163, 86, 64], [240, 126, 230], [81, 202, 91], [29, 215, 190], [6, 21, 212], [31, 219, 215], [200, 179, 75], [45, 178, 180], [200, 10, 5], [66, 21, 204], [78, 228, 127], [4, 204, 65], [155, 214, 247], [194, 25, 145], [188, 84, 50], [187, 239, 137], [129, 22, 255], [143, 200, 246], [217, 254, 66], [201, 50, 96], [47, 252, 41], [40, 184, 55], [0, 217, 236], [18, 25, 134], [51, 200, 124], [199, 76, 69], [233, 219, 31], [115, 229, 75], [101, 174, 34], [25, 50, 28], [34, 78, 187], [190, 0, 197], [97, 97, 179], [37, 33, 13], [169, 191, 227], [196, 170, 123], [252, 94, 114], [117, 129, 102], [141, 212, 156], [1, 191, 183], [137, 231, 35], [111, 213, 180], [123, 75, 45], [209, 226, 244], [244, 60, 163], [219, 39, 103], [202, 235, 222], [132, 220, 191], [212, 185, 101], [197, 182, 222], [154, 9, 30], [128, 248, 201], [218, 223, 243], [128, 205, 35], [81, 251, 116], [178, 47, 26], [64, 93, 108], [60, 119, 180], [119, 163, 218], [19, 123, 149], [5, 220, 41], [11, 65, 10], [136, 240, 212], [154, 4, 53], [112, 66, 49], [70, 245, 102], [191, 245, 192], [208, 14, 179], [220, 227, 84], [86, 181, 242], [1, 19, 245], [199, 40, 36], [15, 240, 107], [48, 47, 208], [30, 245, 117], [98, 130, 194], [238, 242, 131], [187, 180, 213], [218, 171, 106], [68, 208, 176], [6, 252, 78], [107, 215, 185], [128, 33, 122], [73, 149, 102], [2, 215, 191], [177, 199, 102], [160, 34, 192], [236, 9, 221], [165, 42, 166], [35, 26, 74], [165, 197, 199], [121, 5, 217], [96, 8, 175], [223, 47, 91], [208, 241, 156], [185, 65, 187], [222, 233, 238], [178, 122, 167], [115, 254, 104], [12, 6, 118], [116, 33, 91], [127, 107, 84], [118, 26, 46], [35, 112, 236], [142, 68, 12], [122, 121, 130], [103, 240, 96], [137, 249, 246], [150, 81, 60], [25, 72, 20], [162, 124, 213], [21, 60, 246], [60, 155, 154], [166, 106, 242], [172, 135, 148], [177, 14, 222], [85, 34, 249], [218, 215, 93], [60, 115, 225], [45, 194, 10], [206, 109, 250], [55, 2, 217], [239, 51, 205], [54, 82, 167], [69, 84, 140], [147, 83, 249], [66, 246, 112], [20, 102, 123], [149, 235, 181], [92, 115, 182], [196, 176, 142], [23, 19, 152], [127, 85, 134], [69, 18, 67], [157, 59, 250], [234, 152, 201], [67, 27, 105], [145, 35, 228], [184, 46, 175], [160, 33, 104], [158, 79, 250], [64, 90, 103], [101, 159, 218], [170, 125, 80], [140, 163, 169], [15, 65, 191], [248, 73, 122], [91, 48, 12], [32, 202, 173], [94, 132, 249], [89, 154, 11], [164, 227, 87], [27, 11, 248], [148, 126, 58], [99, 141, 213], [142, 222, 145], [44, 67, 205], [155, 250, 183], [2, 121, 253], [135, 76, 16], [125, 172, 167], [37, 192, 77], [93, 113, 134], [123, 151, 98], [24, 58, 221], [56, 155, 205], [214, 150, 10], [22, 160, 106], [82, 107, 166], [155, 121, 220], [160, 0, 161], [181, 128, 181], [253, 10, 89], [169, 118, 69], [102, 194, 22], [82, 199, 212], [89, 182, 78], [77, 85, 132], [89, 143, 50], [115, 101, 186], [231, 82, 3], [48, 12, 65], [147, 136, 192], [184, 251, 205], [144, 150, 124], [189, 65, 199], [170, 11, 247], [128, 125, 160], [105, 15, 74], [78, 24, 15], [162, 58, 90], [101, 127, 209], [198, 116, 155], [184, 212, 105], [157, 70, 26], [193, 70, 129], [99, 234, 43], [195, 42, 53], [125, 221, 72], [136, 158, 133], [246, 81, 13], [168, 125, 182], [89, 175, 165], [182, 194, 11], [91, 100, 112], [17, 252, 142], [226, 3, 159], [146, 129, 181], [171, 221, 198], [14, 174, 149], [120, 102, 215], [232, 163, 167], [27, 31, 223], [116, 167, 127], [190, 8, 194], [32, 10, 67], [77, 85, 9], [19, 182, 127], [107, 232, 18], [213, 61, 155], [25, 235, 67], [168, 125, 171], [213, 69, 215], [192, 219, 64], [196, 12, 138], [141, 232, 63], [56, 226, 204], [140, 122, 54], [176, 14, 147], [42, 27, 82], [33, 9, 21], [123, 254, 226], [0, 125, 150], [127, 125, 233], [57, 61, 9], [218, 163, 104], [186, 243, 220], [134, 153, 168], [213, 189, 28], [42, 166, 66], [246, 30, 220], [98, 52, 11], [91, 238, 179], [240, 75, 97], [107, 180, 143], [230, 97, 125], [82, 46, 131], [211, 127, 96], [215, 45, 126], [122, 68, 145], [171, 103, 105], [154, 167, 142], [132, 23, 25], [79, 3, 78], [52, 148, 206], [43, 199, 252], [218, 141, 140], [202, 136, 23], [215, 233, 127], [51, 103, 128], [113, 4, 77], [229, 232, 120], [157, 102, 250], [72, 31, 42], [172, 182, 39], [20, 223, 167], [80, 124, 101], [126, 231, 27], [155, 205, 41], [106, 119, 213], [176, 200, 78], [206, 135, 241], [150, 215, 9], [184, 9, 31], [136, 81, 104], [1, 123, 136], [76, 211, 65], [139, 242, 10], [193, 30, 161], [100, 170, 112], [105, 213, 166], [197, 208, 177], [117, 188, 136], [206, 219, 113], [148, 118, 21], [88, 186, 178], [108, 201, 198], [196, 20, 97], [185, 23, 170], [150, 188, 229], [20, 173, 73], [181, 235, 144], [31, 5, 112], [124, 255, 210], [144, 77, 150], [10, 30, 13], [156, 79, 84], [227, 183, 208], [177, 74, 213], [176, 93, 227], [104, 21, 228], [73, 94, 144], [75, 5, 186], [138, 44, 229], [52, 144, 7], [69, 73, 133], [98, 74, 236], [59, 38, 31], [167, 72, 130], [211, 82, 144], [16, 244, 249], [65, 144, 220], [251, 52, 152], [168, 175, 44], [32, 51, 138], [138, 186, 54], [110, 28, 120], [207, 92, 98], [210, 205, 42], [198, 215, 226], [76, 181, 175], [219, 247, 21], [81, 85, 4], [124, 250, 209], [68, 251, 21], [139, 132, 152], [68, 66, 93], [254, 192, 59], [103, 65, 22], [91, 245, 4], [162, 61, 78], [128, 246, 119], [202, 101, 164], [86, 15, 67], [27, 220, 232], [30, 61, 80], [62, 20, 82], [129, 89, 16], [225, 83, 237], [136, 227, 180], [219, 2, 208], [250, 88, 92], [24, 194, 56], [26, 163, 218], [144, 75, 77], [187, 84, 161], [209, 144, 38], [8, 58, 48], [90, 122, 61], [88, 191, 21], [97, 21, 105], [32, 60, 246], [204, 146, 225], [144, 154, 193], [151, 220, 90], [125, 251, 188], [4, 218, 170], [123, 110, 145], [142, 123, 39], [105, 140, 109], [91, 66, 140], [92, 37, 41], [30, 105, 93], [216, 135, 16], [235, 6, 33], [76, 126, 4], [208, 42, 57], [117, 90, 120], [48, 59, 14], [47, 60, 141], [165, 37, 127], [208, 86, 4], [138, 181, 136], [100, 198, 193], [190, 199, 10], [88, 81, 13], [14, 199, 176], [70, 224, 41], [237, 1, 82], [144, 53, 252], [54, 40, 118], [179, 254, 87], [73, 141, 250], [136, 244, 76], [78, 164, 205], [138, 88, 171], [239, 233, 41], [72, 14, 118], [108, 16, 254], [5, 75, 100], [184, 102, 92], [251, 142, 12], [68, 178, 200], [135, 165, 21], [138, 208, 85], [89, 103, 204], [161, 168, 208], [168, 218, 131], [88, 67, 189], [55, 103, 15], [83, 254, 198], [200, 139, 199], [205, 75, 164], [95, 84, 129], [87, 12, 0], [219, 166, 39], [123, 217, 212], [172, 113, 84], [146, 172, 102], [141, 238, 239], [150, 197, 0], [97, 25, 247], [93, 34, 117], [131, 64, 72], [168, 239, 241], [23, 227, 79], [166, 246, 208], [222, 149, 122], [227, 11, 225], [46, 113, 144], [149, 132, 225], [245, 151, 158], [49, 17, 173], [246, 103, 216], [117, 84, 24], [122, 78, 127], [141, 161, 46], [49, 25, 158], [16, 40, 17], [137, 84, 111], [196, 80, 145], [238, 41, 216], [208, 230, 65], [113, 181, 103], [104, 79, 116], [143, 45, 15], [15, 205, 148], [247, 81, 53], [141, 189, 14], [124, 91, 222], [198, 56, 126], [42, 175, 95], [125, 28, 145], [207, 136, 242], [188, 207, 218], [179, 241, 34], [76, 44, 41], [51, 18, 62], [225, 205, 223], [235, 132, 241], [201, 129, 179], [245, 21, 47], [145, 113, 116], [106, 26, 119], [241, 7, 17], [83, 73, 35], [224, 6, 181], [67, 57, 120], [246, 98, 241], [147, 89, 5], [246, 251, 66], [222, 71, 151], [243, 57, 125], [75, 22, 100], [194, 230, 185], [116, 0, 73], [149, 208, 105], [149, 202, 209], [24, 27, 88], [221, 237, 200], [52, 156, 46], [179, 53, 58], [226, 121, 228], [207, 13, 138], [0, 207, 186], [179, 17, 185], [8, 80, 94], [183, 233, 104], [201, 70, 80], [210, 186, 166], [229, 117, 28], [233, 25, 203], [6, 163, 66], [142, 120, 96], [17, 173, 113], [117, 145, 156], [235, 231, 232], [73, 131, 219], [64, 250, 121], [14, 114, 5], [89, 179, 116], [63, 168, 184], [20, 198, 61], [207, 83, 26], [101, 216, 48], [41, 45, 244], [251, 58, 239], [157, 60, 128], [38, 168, 178], [167, 99, 5], [140, 176, 125], [226, 175, 223], [40, 168, 64], [184, 218, 166], [165, 65, 76], [59, 165, 237], [244, 68, 104], [93, 142, 115], [72, 15, 7], [145, 181, 160], [70, 121, 131], [235, 143, 209], [86, 140, 38], [69, 233, 144], [190, 73, 154], [93, 24, 31], [70, 115, 198], [167, 35, 95], [220, 72, 200], [237, 134, 159], [53, 186, 253], [224, 91, 245], [137, 226, 29], [73, 133, 87], [229, 211, 135], [113, 11, 112], [252, 101, 236], [215, 214, 164], [47, 209, 3], [200, 96, 61], [208, 27, 8], [115, 217, 222], [72, 91, 54], [93, 220, 246], [199, 174, 24], [185, 200, 75], [248, 30, 251], [187, 235, 20], [14, 92, 208], [196, 39, 116], [239, 180, 250], [21, 251, 135], [97, 229, 48], [178, 172, 144], [101, 108, 252], [147, 228, 48], [61, 54, 171], [184, 207, 103], [240, 52, 73], [93, 86, 189], [29, 76, 28], [68, 180, 175], [128, 8, 149], [110, 109, 26], [102, 40, 212], [71, 7, 85], [62, 181, 215], [153, 54, 171], [227, 45, 253], [172, 69, 56], [127, 70, 216], [209, 105, 229], [87, 70, 154], [245, 74, 116], [111, 170, 171], [182, 201, 233], [32, 73, 115], [141, 116, 105], [151, 147, 13], [73, 76, 22], [57, 109, 163], [58, 18, 210], [177, 45, 202], [243, 60, 162], [169, 218, 216], [16, 233, 246], [217, 137, 56], [68, 125, 225], [197, 15, 148], [52, 3, 225], [177, 245, 254], [76, 140, 65], [27, 39, 239], [251, 133, 128], [38, 14, 237], [178, 255, 99], [152, 109, 118], [22, 145, 17], [147, 42, 38], [251, 149, 110], [187, 122, 190], [249, 244, 215], [169, 120, 145], [180, 213, 53], [38, 157, 14], [138, 250, 238], [45, 232, 161], [20, 37, 47], [217, 134, 72], [178, 221, 194], [22, 31, 124], [82, 5, 65], [8, 191, 136], [7, 91, 68], [126, 86, 164], [4, 32, 191], [166, 36, 55], [174, 85, 204], [77, 240, 166], [14, 250, 208], [231, 253, 204], [167, 161, 181], [66, 24, 160], [92, 182, 12], [58, 61, 126], [37, 250, 172], [51, 171, 244], [27, 146, 184], [77, 165, 205], [248, 70, 53], [199, 148, 6], [238, 29, 66], [232, 89, 43], [153, 15, 235], [93, 139, 60], [100, 135, 11], [86, 206, 112], [57, 187, 186], [173, 164, 117], [221, 206, 16], [45, 190, 214], [4, 13, 81], [75, 173, 162], [75, 13, 113], [51, 157, 54], [56, 26, 224], [181, 9, 8], [177, 73, 246], [184, 168, 143], [36, 77, 151], [5, 32, 42], [198, 178, 204], [239, 101, 217], [7, 89, 169], [80, 30, 53], [70, 32, 4], [40, 141, 206], [142, 216, 139], [158, 12, 134], [241, 139, 191], [104, 193, 113], [105, 132, 69], [197, 189, 98], [75, 134, 163], [11, 188, 56], [201, 134, 83], [9, 142, 110], [255, 225, 141], [214, 133, 58], [247, 55, 250], [91, 140, 231], [56, 207, 107], [149, 197, 220], [105, 216, 126], [22, 239, 186], [190, 157, 231], [241, 96, 128], [152, 216, 224], [117, 59, 57], [185, 230, 155], [107, 239, 37], [72, 149, 185], [169, 19, 185], [73, 200, 208], [36, 25, 140], [200, 31, 48], [97, 63, 186], [97, 242, 194], [235, 162, 70], [219, 39, 241], [131, 189, 26], [132, 111, 142], [145, 77, 102], [49, 34, 65], [3, 204, 124], [81, 125, 89], [47, 102, 23], [98, 140, 36], [134, 54, 247], [10, 77, 16], [166, 168, 242], [104, 115, 112], [201, 29, 145], [91, 83, 31], [139, 93, 213], [167, 122, 171], [225, 119, 55], [218, 140, 197], [187, 128, 59], [53, 198, 167], [255, 99, 64], [103, 179, 162], [35, 120, 132], [77, 13, 142], [157, 41, 110], [158, 115, 35], [238, 55, 110], [72, 73, 15], [255, 90, 200], [206, 142, 87], [72, 156, 166], [123, 27, 219], [158, 82, 245], [73, 201, 229], [240, 156, 172], [138, 226, 243], [235, 166, 130], [106, 93, 147], [195, 45, 140], [221, 169, 235], [22, 58, 119], [242, 225, 197], [169, 110, 27], [145, 139, 184], [160, 168, 191], [33, 138, 129], [66, 253, 119], [189, 36, 79], [243, 156, 171], [150, 80, 237], [34, 213, 163], [34, 113, 133], [103, 120, 244], [241, 228, 224], [20, 32, 47], [101, 68, 37], [8, 213, 45], [108, 211, 185], [35, 250, 222], [62, 104, 56], [55, 23, 2], [62, 146, 238], [96, 220, 158], [187, 254, 8], [168, 130, 93], [121, 12, 186], [205, 150, 129], [156, 18, 127], [97, 206, 136], [175, 246, 169], [126, 230, 254], [74, 214, 4], [228, 248, 15], [178, 246, 191], [191, 255, 13], [104, 138, 23], [29, 24, 212], [136, 10, 225], [210, 109, 128], [39, 199, 222], [204, 167, 123], [248, 68, 175], [104, 99, 222], [3, 77, 158], [145, 40, 183], [180, 127, 99], [29, 51, 116], [62, 202, 135], [169, 72, 8], [102, 58, 21], [198, 9, 42], [164, 73, 14], [55, 100, 231], [222, 37, 106], [129, 115, 249], [80, 23, 135], [73, 133, 154], [186, 159, 66], [119, 27, 126], [237, 125, 230], [83, 96, 36], [81, 6, 148], [20, 232, 198], [162, 237, 100], [57, 27, 190], [47, 150, 5], [65, 138, 17], [20, 93, 151], [27, 233, 108], [115, 111, 111], [34, 46, 224], [162, 106, 144], [13, 163, 16], [1, 184, 13], [40, 147, 5], [167, 102, 177], [143, 131, 244], [165, 131, 219], [99, 61, 217], [133, 1, 59], [26, 225, 195], [228, 90, 177], [248, 181, 244], [186, 5, 173], [165, 1, 59], [127, 195, 66], [108, 124, 244], [45, 17, 148], [68, 42, 50], [12, 148, 237], [93, 73, 162], [154, 18, 171], [130, 45, 168], [206, 187, 195], [140, 182, 191], [136, 38, 120], [249, 80, 56], [162, 47, 10], [219, 7, 224], [246, 111, 20], [189, 34, 1], [204, 112, 117], [28, 31, 17], [207, 58, 174], [122, 177, 127], [36, 89, 17], [168, 78, 230], [82, 145, 75], [121, 92, 234], [143, 39, 223], [255, 64, 73], [13, 168, 143], [60, 28, 112], [159, 195, 26], [0, 113, 35], [135, 182, 209], [54, 18, 30], [170, 87, 18], [187, 25, 219], [186, 69, 135], [72, 17, 124], [248, 85, 214], [52, 96, 215], [151, 107, 238], [210, 99, 99], [105, 91, 146], [173, 240, 63], [72, 30, 217], [179, 118, 244], [39, 165, 11], [233, 93, 127], [56, 226, 28], [207, 51, 111], [218, 225, 92], [199, 217, 40], [148, 28, 133], [149, 217, 98], [176, 97, 69], [11, 255, 197], [176, 75, 148], [22, 167, 43], [249, 90, 83], [143, 14, 90], [222, 22, 225], [155, 160, 119], [105, 47, 8], [185, 208, 15], [243, 0, 44], [220, 204, 166], [138, 126, 161], [124, 246, 148], [173, 17, 7], [183, 4, 124], [130, 245, 165], [20, 124, 200], [206, 54, 189], [40, 156, 243], [243, 130, 234], [197, 70, 208], [130, 123, 237], [115, 181, 77], [102, 125, 55], [19, 202, 130], [113, 79, 145], [85, 131, 157], [230, 238, 176], [76, 67, 65], [41, 32, 221], [189, 142, 178], [103, 48, 129], [29, 198, 86], [27, 96, 18], [205, 220, 22], [78, 41, 198], [110, 104, 206], [41, 72, 49], [206, 97, 70], [193, 125, 114], [160, 108, 19], [181, 185, 192], [56, 226, 7], [149, 226, 208], [165, 157, 24], [129, 5, 221], [225, 36, 81], [184, 138, 131], [155, 108, 210], [70, 145, 68], [19, 37, 149], [239, 55, 91], [48, 29, 188], [193, 144, 117], [197, 43, 199], [113, 224, 18], [51, 75, 178], [67, 113, 229], [55, 113, 161], [185, 206, 42], [185, 125, 18], [173, 139, 228], [224, 216, 1], [189, 248, 195], [38, 16, 212], [213, 125, 219], [64, 241, 107], [194, 85, 180], [232, 112, 204], [216, 142, 10], [215, 255, 205], [109, 49, 246], [145, 29, 86], [137, 142, 62], [77, 232, 160], [31, 193, 20], [234, 152, 216], [61, 185, 141], [132, 210, 121], [232, 167, 249], [113, 62, 126], [21, 112, 37], [37, 222, 16], [186, 222, 100], [103, 252, 200], [150, 15, 9], [233, 110, 135], [221, 229, 87], [173, 159, 224], [87, 140, 51], [168, 179, 145], [5, 5, 52], [12, 202, 74], [165, 82, 83], [235, 177, 98], [161, 249, 119], [231, 193, 131], [79, 217, 48], [145, 234, 85], [82, 3, 111], [229, 176, 107], [227, 184, 40], [48, 0, 9], [179, 25, 66], [201, 141, 51], [37, 253, 21], [84, 162, 139], [66, 175, 71], [212, 48, 52], [82, 176, 93], [223, 111, 3], [223, 218, 86], [75, 54, 98], [205, 106, 145], [222, 143, 131], [82, 22, 90], [74, 204, 191], [131, 73, 85], [128, 210, 45], [55, 155, 245], [192, 31, 220], [124, 171, 100], [130, 2, 81], [46, 173, 106], [116, 53, 254], [23, 122, 1], [49, 69, 78], [42, 138, 19], [123, 244, 95], [56, 227, 255], [19, 185, 27], [6, 216, 98], [232, 183, 46], [61, 43, 6], [18, 0, 16], [98, 185, 108], [225, 148, 3], [48, 192, 173], [72, 85, 135], [241, 235, 120], [51, 186, 249], [61, 38, 247], [103, 18, 248], [36, 138, 154], [93, 184, 227], [255, 212, 84], [34, 6, 43], [196, 96, 100], [43, 202, 212], [184, 96, 252], [203, 118, 43], [140, 193, 251], [81, 20, 89], [52, 248, 4], [131, 189, 249], [20, 96, 206], [213, 219, 116], [201, 13, 240], [17, 178, 7], [53, 31, 33], [181, 51, 148], [43, 112, 217], [26, 81, 192], [169, 19, 186], [19, 246, 217], [254, 232, 246], [134, 239, 11], [200, 188, 234], [244, 206, 147], [97, 167, 20], [152, 183, 148], [242, 134, 60], [140, 187, 176], [154, 28, 240], [40, 51, 160], [221, 35, 54], [231, 124, 143], [67, 83, 2], [115, 194, 247], [0, 155, 154], [114, 156, 172], [101, 145, 159], [91, 16, 19], [15, 245, 199], [212, 213, 83], [117, 255, 206], [177, 242, 180], [194, 13, 113], [227, 43, 167], [219, 84, 237], [208, 10, 243], [42, 189, 241], [183, 202, 30], [173, 30, 3], [200, 49, 54], [168, 225, 182], [148, 139, 21], [255, 127, 116], [62, 196, 50], [180, 86, 218], [230, 140, 198], [95, 193, 114], [242, 150, 105], [158, 24, 27], [250, 84, 54], [54, 210, 103], [173, 103, 239], [55, 177, 171], [209, 229, 153], [132, 85, 154], [198, 189, 236], [248, 69, 156], [115, 171, 151], [167, 240, 18], [232, 242, 66], [23, 110, 112], [55, 175, 0], [128, 130, 240], [145, 253, 201], [30, 17, 153], [57, 130, 116], [90, 220, 166], [244, 23, 17], [60, 179, 139], [156, 212, 92], [73, 93, 192], [177, 141, 17], [64, 81, 254], [79, 224, 158], [29, 60, 172], [208, 166, 17], [128, 199, 179], [223, 217, 118], [168, 241, 251], [44, 221, 145], [216, 130, 101], [85, 31, 156], [15, 121, 161], [32, 142, 227], [109, 91, 161], [0, 163, 240], [226, 193, 5], [200, 244, 14], [18, 233, 148], [229, 50, 249], [234, 104, 119], [40, 244, 16], [4, 163, 119], [177, 231, 71], [45, 219, 251], [56, 209, 70], [60, 119, 173], [30, 102, 17], [238, 174, 40], [164, 120, 88], [37, 238, 181], [236, 116, 60], [248, 172, 13], [55, 135, 205], [233, 159, 148], [11, 138, 52], [225, 228, 158], [40, 137, 189], [111, 29, 111], [116, 164, 210], [176, 152, 9], [158, 147, 182], [21, 55, 159], [20, 163, 48], [120, 31, 225], [87, 229, 254], [165, 113, 88], [231, 141, 156], [44, 135, 27], [52, 165, 187], [11, 126, 157], [220, 54, 28], [20, 186, 184], [106, 91, 15], [107, 209, 146], [167, 56, 245], [224, 115, 215], [116, 245, 235], [135, 239, 165], [134, 21, 70], [90, 130, 196], [233, 2, 30], [22, 134, 63], [215, 60, 164], [93, 119, 29], [154, 250, 150], [149, 148, 108], [149, 208, 165], [197, 76, 76], [228, 248, 121], [177, 57, 238], [235, 181, 41], [38, 168, 183], [201, 129, 189], [2, 35, 125], [107, 163, 46], [56, 180, 80], [162, 182, 29], [12, 145, 154], [106, 77, 143], [66, 51, 61], [245, 74, 7], [123, 118, 120], [142, 82, 218], [135, 120, 129], [94, 90, 212], [96, 179, 103], [2, 68, 149], [161, 178, 161], [26, 2, 245], [79, 96, 152], [65, 183, 143], [193, 132, 227], [236, 51, 44], [6, 187, 86], [1, 6, 78], [202, 220, 242], [85, 199, 222], [181, 184, 179], [48, 20, 212], [236, 134, 15], [31, 3, 87], [249, 233, 68], [65, 219, 32], [120, 164, 141], [111, 152, 164], [198, 25, 133], [139, 32, 143], [220, 42, 131], [253, 36, 80], [123, 62, 222], [165, 104, 58], [117, 98, 120], [217, 157, 194], [125, 193, 160], [206, 6, 211], [118, 184, 14], [188, 204, 210], [86, 247, 11], [75, 239, 69], [189, 163, 75], [46, 205, 156], [91, 175, 182], [170, 33, 205], [42, 55, 51], [12, 50, 98], [109, 154, 14], [218, 89, 170], [18, 97, 34], [199, 244, 30], [160, 251, 62], [161, 162, 149], [5, 199, 244], [127, 14, 102], [208, 231, 57], [6, 217, 137], [122, 183, 174], [80, 150, 22], [188, 183, 244], [83, 53, 107], [248, 220, 150], [158, 128, 161], [214, 163, 214], [108, 22, 91], [27, 205, 116], [27, 179, 214], [212, 148, 70], [212, 90, 85], [93, 150, 165], [188, 74, 78], [98, 41, 186], [49, 81, 1], [189, 104, 55], [172, 111, 23], [242, 84, 157], [232, 0, 60], [155, 55, 54], [67, 54, 109], [75, 160, 178], [90, 50, 177], [188, 223, 241], [227, 163, 15], [253, 98, 158], [87, 31, 16], [150, 94, 114], [135, 245, 116], [124, 228, 47], [63, 187, 181], [179, 221, 53], [226, 80, 174], [172, 235, 226], [105, 235, 114], [200, 49, 187], [208, 130, 106], [229, 206, 176], [13, 221, 91], [180, 88, 166], [164, 238, 39], [79, 196, 122], [10, 74, 242], [26, 163, 90], [157, 79, 67], [7, 167, 55], [80, 157, 40], [104, 197, 119], [23, 64, 33], [172, 243, 164], [11, 24, 223], [221, 13, 26], [19, 133, 51], [209, 21, 175], [77, 0, 239], [86, 57, 213], [33, 164, 43], [228, 238, 175], [206, 75, 123], [117, 152, 208], [233, 223, 103], [17, 47, 67], [150, 10, 203], [91, 150, 197], [123, 246, 146], [54, 245, 157], [119, 26, 154], [16, 238, 197], [120, 112, 196], [56, 42, 122], [112, 176, 157], [117, 68, 37], [118, 228, 35], [36, 150, 242], [187, 196, 239], [99, 254, 19], [254, 165, 38], [96, 197, 112], [128, 103, 221], [15, 145, 191], [242, 155, 40], [185, 252, 176], [97, 143, 241], [32, 32, 131], [53, 69, 236], [169, 169, 178], [10, 142, 38], [45, 117, 40], [85, 255, 179], [167, 144, 50], [60, 134, 1], [185, 7, 135], [185, 24, 234], [132, 3, 22], [6, 92, 105], [216, 28, 227], [205, 99, 223], [253, 154, 160], [116, 146, 55], [26, 215, 26], [113, 114, 203], [45, 210, 74], [94, 122, 32], [129, 46, 1], [200, 146, 38], [247, 173, 125], [65, 148, 115], [25, 124, 113], [127, 215, 72], [127, 216, 50], [137, 119, 242], [85, 46, 26], [56, 200, 157], [115, 0, 180], [220, 216, 113], [50, 11, 238], [24, 124, 141], [239, 182, 154], [172, 71, 73], [127, 20, 147], [42, 156, 251], [192, 38, 107], [115, 140, 120], [140, 208, 112], [213, 255, 18], [243, 96, 58], [240, 99, 251], [140, 63, 213], [198, 99, 111], [215, 85, 136], [62, 157, 192], [173, 51, 216], [189, 70, 87], [47, 12, 232], [83, 202, 179], [172, 3, 131], [136, 122, 130], [180, 103, 60], [71, 245, 172], [132, 119, 108], [71, 26, 55], [171, 173, 60], [35, 23, 11], [134, 193, 249], [211, 84, 117], [208, 111, 11], [90, 97, 18], [216, 133, 112], [170, 22, 242], [179, 62, 93], [190, 228, 154], [37, 46, 55], [182, 87, 192], [139, 233, 109], [89, 116, 157], [74, 193, 231], [61, 228, 176], [168, 186, 89], [171, 48, 130], [216, 190, 94], [149, 37, 167], [66, 194, 228], [49, 157, 30], [66, 237, 192], [249, 173, 231], [167, 229, 40], [52, 195, 149], [10, 19, 175], [248, 150, 243], [252, 234, 168], [27, 190, 146], [160, 176, 19], [147, 230, 38], [88, 17, 117], [188, 110, 206], [198, 159, 251], [62, 200, 106], [237, 169, 196], [247, 4, 120], [84, 188, 145], [231, 245, 187], [194, 235, 126], [222, 12, 229], [55, 66, 130], [57, 170, 106], [255, 174, 98], [136, 5, 1], [15, 114, 62], [160, 50, 221], [31, 40, 158], [185, 172, 175], [113, 125, 245], [222, 223, 219], [237, 73, 251], [219, 137, 94], [42, 231, 235], [204, 5, 211], [44, 249, 50], [91, 75, 44], [31, 25, 34], [212, 110, 167], [245, 63, 15], [67, 141, 26], [55, 22, 255], [86, 229, 146], [180, 220, 69], [200, 100, 14], [17, 9, 186], [202, 11, 235], [24, 33, 95], [15, 170, 97], [161, 88, 225], [128, 156, 43], [226, 160, 176], [211, 194, 150], [55, 166, 146], [172, 128, 70], [48, 234, 36], [70, 107, 100], [93, 90, 217], [144, 86, 44], [44, 65, 209], [25, 18, 55], [212, 112, 237], [136, 95, 43], [155, 11, 105], [152, 52, 6], [217, 69, 25], [35, 46, 246], [120, 248, 251], [194, 196, 50], [207, 206, 173], [191, 206, 83], [88, 125, 156], [252, 79, 180], [199, 141, 193], [22, 237, 184], [190, 69, 104], [108, 114, 108], [254, 100, 75], [241, 229, 44], [199, 238, 29], [65, 250, 45], [199, 65, 163], [218, 254, 20], [144, 187, 110], [149, 178, 104], [196, 129, 233], [82, 21, 105], [239, 108, 38], [237, 107, 160], [234, 150, 173], [194, 130, 47], [29, 123, 146], [107, 157, 29], [67, 201, 133], [239, 198, 117], [191, 112, 186], [70, 7, 11], [244, 167, 118], [6, 76, 27], [120, 219, 68], [138, 121, 205], [16, 216, 103], [252, 62, 62], [164, 7, 26], [52, 239, 3], [248, 253, 125], [108, 72, 120], [4, 14, 133], [75, 147, 74], [164, 120, 244], [234, 155, 63], [253, 136, 4], [167, 13, 102], [116, 11, 83], [14, 137, 171], [14, 248, 24], [48, 115, 25], [145, 204, 191], [228, 107, 48], [0, 91, 92], [26, 221, 160], [147, 42, 2], [146, 251, 128], [129, 123, 93], [101, 21, 11], [211, 219, 219], [102, 196, 176], [240, 239, 128], [77, 245, 54], [175, 41, 246], [226, 164, 169], [81, 65, 94], [16, 188, 238], [59, 103, 163], [17, 68, 176], [131, 58, 58], [110, 191, 219], [76, 98, 223], [179, 140, 132], [1, 195, 57], [29, 42, 167], [212, 57, 182], [1, 1, 233], [223, 226, 210], [154, 50, 139], [246, 227, 193], [209, 112, 254], [216, 109, 148], [245, 247, 182], [251, 231, 8], [161, 165, 38], [31, 45, 182], [217, 15, 253], [2, 39, 37], [162, 148, 248], [231, 109, 218], [207, 134, 25], [63, 255, 43], [242, 154, 249], [55, 51, 163], [60, 26, 28], [165, 46, 222], [247, 54, 103], [218, 57, 28], [205, 206, 99], [80, 165, 112], [199, 70, 27], [226, 122, 46], [27, 77, 203], [56, 81, 143], [184, 82, 165], [63, 202, 203], [101, 224, 239], [3, 210, 135], [227, 39, 231], [123, 23, 41], [39, 89, 88], [160, 240, 38], [220, 196, 236], [188, 206, 126], [50, 65, 118], [126, 61, 11], [187, 187, 64], [76, 148, 98], [72, 54, 26], [0, 75, 164], [38, 45, 90], [230, 105, 151], [136, 199, 229], [70, 213, 132], [128, 173, 176], [146, 88, 68], [205, 105, 251], [144, 75, 62], [139, 224, 22], [122, 144, 30], [105, 229, 63], [205, 109, 157], [80, 210, 38], [173, 247, 183], [183, 148, 207], [181, 204, 77], [240, 144, 169], [92, 104, 163], [49, 51, 100], [170, 16, 108], [114, 190, 93], [86, 156, 156], [189, 252, 118], [165, 83, 113], [11, 238, 80], [229, 7, 135], [90, 60, 207], [176, 82, 183], [23, 36, 232], [62, 101, 137], [0, 34, 170], [250, 193, 113], [62, 244, 137], [151, 56, 255], [85, 178, 183], [98, 96, 186], [218, 105, 186], [147, 179, 6], [210, 175, 133], [128, 235, 133], [148, 248, 203], [148, 246, 38], [26, 216, 178], [249, 44, 10], [76, 72, 99], [224, 223, 245], [233, 183, 164], [197, 45, 190], [55, 179, 39], [81, 185, 141], [108, 213, 223], [159, 234, 87], [55, 96, 143], [191, 193, 218], [22, 229, 209], [71, 44, 226], [48, 7, 152], [7, 235, 197], [22, 33, 253], [187, 131, 225], [166, 191, 96], [17, 110, 165], [223, 150, 8], [4, 214, 69], [35, 179, 196], [88, 120, 49], [19, 33, 86], [96, 72, 96], [20, 192, 224], [193, 51, 122], [33, 123, 185], [171, 59, 134], [45, 80, 185], [114, 199, 195], [20, 110, 20], [197, 193, 245], [140, 35, 10], [2, 53, 163], [44, 122, 159], [68, 247, 70], [255, 201, 206], [89, 69, 87], [198, 187, 172], [217, 246, 249], [78, 132, 196], [123, 65, 67], [111, 177, 30], [231, 68, 155], [113, 109, 155], [213, 26, 61], [101, 61, 70], [12, 199, 119], [46, 6, 241], [111, 193, 113], [39, 105, 225], [35, 13, 27], [225, 18, 46], [216, 15, 34], [116, 21, 56], [237, 24, 79], [234, 73, 113], [75, 224, 132], [174, 5, 209], [75, 31, 99], [151, 223, 56], [204, 168, 0], [195, 83, 62], [157, 82, 88], [92, 179, 216], [190, 37, 17], [142, 3, 78], [208, 81, 191], [161, 124, 2], [139, 162, 159], [245, 158, 131], [187, 208, 51], [146, 202, 144], [114, 55, 195], [97, 164, 205], [198, 197, 47], [217, 224, 35], [9, 170, 119], [171, 192, 117], [235, 46, 82], [23, 224, 33], [129, 10, 17], [81, 78, 79], [9, 65, 198], [132, 240, 144], [50, 214, 88], [133, 224, 141], [212, 193, 23], [127, 172, 90], [109, 212, 201], [189, 116, 139], [148, 128, 61], [231, 139, 60], [102, 214, 45], [82, 134, 38], [200, 205, 189], [186, 8, 186], [182, 45, 91], [26, 72, 224], [131, 186, 186], [162, 11, 179], [176, 229, 59], [195, 139, 64], [191, 189, 192], [249, 208, 252], [155, 22, 68], [209, 174, 31], [219, 156, 117], [144, 49, 147], [12, 81, 49], [195, 4, 159], [125, 131, 65], [230, 45, 74], [180, 162, 78], [128, 73, 38], [180, 91, 91], [189, 75, 197], [0, 33, 118], [129, 207, 83], [155, 247, 10], [32, 148, 32], [103, 117, 199], [7, 90, 181], [150, 88, 251], [215, 190, 69], [31, 171, 147], [110, 153, 184], [158, 198, 26], [33, 147, 60], [60, 194, 37], [44, 121, 66], [59, 124, 131], [5, 133, 242], [76, 211, 182], [86, 233, 71], [0, 183, 124], [186, 138, 67], [41, 109, 179], [22, 49, 252], [85, 64, 30], [145, 133, 217], [233, 88, 167], [39, 98, 79], [222, 38, 111], [176, 112, 230], [59, 202, 104], [110, 72, 9], [135, 242, 241], [99, 144, 104], [230, 197, 9], [197, 128, 224], [187, 82, 156], [28, 41, 232], [34, 48, 151], [22, 145, 145], [192, 25, 105], [169, 0, 92], [219, 151, 37], [32, 88, 196], [139, 114, 115], [188, 214, 135], [34, 116, 61], [127, 116, 93], [98, 215, 2], [211, 10, 101], [218, 45, 141], [144, 58, 19], [34, 129, 119], [177, 198, 19], [148, 230, 205], [227, 207, 57], [22, 142, 127], [186, 247, 68], [154, 18, 233], [94, 83, 255], [94, 10, 179], [48, 31, 117], [98, 2, 82], [246, 200, 72], [12, 67, 195], [15, 133, 54], [91, 171, 247], [92, 54, 110], [150, 157, 18], [180, 48, 199], [95, 149, 91], [8, 159, 124], [21, 90, 88], [247, 148, 222], [236, 140, 4], [218, 10, 176], [211, 113, 231], [3, 72, 176], [116, 68, 119], [249, 82, 192], [140, 115, 42], [81, 119, 116], [169, 180, 8], [2, 154, 56], [199, 97, 49], [41, 172, 92], [251, 189, 228], [193, 216, 109], [174, 11, 119], [10, 0, 165], [36, 122, 52], [188, 128, 212], [50, 237, 164], [115, 132, 197], [173, 234, 1], [144, 143, 230], [187, 229, 254], [88, 114, 198], [6, 21, 193], [114, 75, 215], [94, 50, 150], [65, 47, 96], [34, 164, 99], [227, 51, 109], [200, 162, 95], [203, 240, 32], [2, 197, 16], [155, 231, 41], [32, 239, 22], [42, 242, 35], [176, 184, 182], [89, 80, 83], [227, 61, 136], [80, 56, 86], [84, 110, 175], [53, 57, 173], [74, 241, 221], [44, 98, 55], [32, 244, 127], [30, 181, 226], [101, 18, 221], [217, 7, 162], [9, 149, 52], [228, 59, 158], [60, 37, 193], [52, 22, 100], [128, 193, 144], [10, 23, 164], [138, 235, 210], [3, 178, 249], [186, 80, 55], [245, 180, 147], [142, 228, 190], [87, 255, 236], [185, 10, 101], [203, 168, 172], [81, 137, 86], [116, 95, 31], [69, 62, 118], [80, 188, 217], [214, 16, 195], [197, 36, 217], [136, 92, 27], [217, 128, 141], [42, 75, 95], [10, 184, 191], [198, 103, 24], [223, 56, 226], [230, 246, 134], [48, 24, 142], [158, 222, 169], [187, 159, 63], [183, 177, 54], [167, 103, 248], [104, 250, 209], [208, 176, 245], [152, 179, 228], [12, 245, 22], [75, 146, 54], [29, 13, 169], [222, 21, 26], [112, 134, 180], [107, 143, 64], [43, 29, 9], [12, 186, 184], [107, 149, 135], [165, 128, 242], [48, 59, 64], [31, 62, 46], [246, 195, 220], [47, 87, 159], [36, 90, 168], [5, 251, 15], [213, 112, 161], [22, 210, 203], [240, 101, 183], [12, 0, 146], [240, 204, 236], [243, 126, 83], [71, 13, 218], [85, 201, 47], [15, 238, 250], [207, 100, 77], [18, 92, 26], [250, 34, 126], [175, 135, 194], [232, 237, 50], [45, 7, 112], [250, 46, 36], [22, 221, 174], [116, 177, 53], [66, 37, 144], [46, 65, 229], [29, 100, 99], [233, 175, 125], [102, 195, 223], [109, 222, 177], [44, 209, 219], [213, 223, 54], [7, 74, 12], [17, 68, 249], [24, 10, 101], [192, 27, 192], [212, 200, 249], [245, 155, 34], [101, 5, 136], [124, 149, 11], [59, 225, 67], [48, 113, 52], [52, 115, 73], [205, 126, 80], [112, 4, 150], [87, 145, 112], [228, 205, 194], [21, 198, 154], [53, 64, 87], [69, 19, 130], [162, 118, 40], [38, 32, 21], [224, 151, 118], [242, 231, 52], [169, 16, 30], [179, 109, 217], [160, 152, 199], [141, 6, 46], [17, 83, 103], [233, 203, 143], [90, 68, 147], [124, 89, 30], [143, 63, 105], [108, 214, 38], [68, 57, 42], [8, 245, 192], [226, 172, 151], [63, 138, 39], [235, 186, 222], [59, 67, 102], [206, 243, 218], [251, 110, 106], [140, 211, 105], [155, 212, 21], [41, 252, 205], [180, 192, 232], [78, 149, 227], [123, 14, 115], [170, 183, 152], [234, 143, 140], [172, 206, 168], [110, 207, 148], [245, 30, 38], [90, 62, 109], [180, 155, 67], [26, 5, 253], [99, 89, 163], [250, 152, 142], [26, 163, 95], [252, 254, 22], [50, 55, 150], [181, 193, 66], [227, 192, 171], [108, 238, 237], [57, 236, 174], [102, 234, 75], [252, 31, 67], [18, 133, 22], [164, 140, 79], [156, 55, 22], [255, 191, 186], [241, 2, 70], [80, 169, 253], [86, 181, 12], [24, 68, 234], [75, 35, 63], [158, 252, 145], [213, 230, 241], [17, 27, 70], [55, 216, 23], [28, 20, 168], [182, 54, 58], [197, 85, 70], [164, 97, 166], [234, 193, 71], [88, 108, 50], [82, 58, 248], [194, 247, 140], [105, 157, 180], [123, 178, 126], [115, 37, 225], [129, 62, 124], [21, 14, 14], [203, 52, 60], [229, 9, 201], [3, 38, 140], [215, 218, 206], [166, 2, 146], [71, 0, 68], [180, 121, 249], [197, 47, 59], [93, 94, 37], [37, 66, 238], [141, 187, 152], [70, 35, 66], [178, 177, 61], [7, 207, 197], [219, 5, 68], [181, 122, 184], [241, 151, 134], [173, 15, 30], [164, 193, 103], [176, 225, 156], [100, 164, 70], [182, 209, 84], [254, 73, 16], [237, 1, 213], [0, 27, 33], [134, 191, 155], [105, 109, 203], [198, 69, 42], [50, 233, 129], [55, 186, 178], [0, 89, 72], [183, 238, 57], [178, 72, 206], [207, 90, 134], [122, 252, 171], [21, 49, 12], [34, 201, 70], [237, 38, 83], [111, 162, 29], [129, 219, 44], [10, 187, 103], [38, 146, 98], [179, 105, 105], [38, 183, 15], [87, 44, 179], [25, 213, 243], [96, 171, 147], [109, 101, 225], [61, 233, 116], [168, 51, 213], [254, 188, 9], [98, 28, 134], [34, 78, 199], [139, 1, 128], [127, 26, 183], [135, 109, 198], [137, 195, 161], [196, 71, 223], [202, 69, 188], [222, 54, 72], [112, 136, 164], [63, 39, 246], [141, 54, 249], [158, 195, 127], [97, 138, 128], [234, 15, 124], [231, 132, 61], [232, 114, 0], [136, 188, 6], [53, 225, 56], [127, 141, 57], [119, 146, 181], [128, 152, 49], [104, 253, 161], [51, 212, 53], [203, 149, 238], [199, 15, 227], [79, 253, 160], [31, 245, 32], [192, 29, 223], [60, 2, 136], [77, 139, 113], [162, 138, 100], [59, 128, 189], [35, 31, 47], [118, 36, 114], [73, 8, 17], [201, 217, 226], [204, 166, 28], [76, 238, 140], [39, 181, 59], [165, 127, 192], [80, 149, 123], [243, 137, 4], [12, 47, 215], [164, 202, 54], [137, 208, 177], [92, 156, 135], [69, 206, 245], [222, 191, 35], [24, 3, 34], [185, 50, 93], [241, 43, 166], [219, 165, 109], [90, 251, 239], [95, 103, 158], [252, 146, 4], [134, 88, 35], [107, 152, 195], [62, 112, 128], [139, 175, 147], [234, 70, 212], [247, 139, 63], [13, 90, 163], [194, 86, 224], [179, 179, 31], [181, 132, 176], [207, 48, 110], [237, 131, 86], [158, 28, 17], [165, 95, 243], [28, 173, 183], [194, 212, 242], [58, 204, 189], [243, 58, 200], [2, 139, 238], [31, 21, 56], [7, 30, 91], [40, 240, 158], [97, 10, 49], [142, 164, 159], [96, 93, 138], [137, 52, 222], [138, 146, 249], [221, 4, 162], [129, 133, 129], [16, 74, 193], [40, 46, 172], [165, 123, 174], [160, 80, 161], [27, 74, 219], [139, 246, 53], [65, 146, 87], [86, 105, 175], [144, 239, 157], [62, 175, 2], [218, 170, 81], [143, 60, 148], [86, 180, 99], [17, 245, 139], [36, 63, 103], [141, 67, 58], [209, 171, 127], [144, 142, 121], [81, 68, 76], [199, 186, 163], [72, 175, 70], [39, 144, 160], [7, 206, 36], [169, 222, 70], [181, 105, 227], [117, 216, 79], [104, 53, 183], [89, 79, 247], [123, 84, 108], [188, 200, 128], [69, 238, 251], [235, 89, 40], [115, 253, 75], [249, 188, 72], [75, 24, 27], [7, 146, 128], [228, 144, 199], [83, 74, 65], [35, 135, 253], [107, 65, 87], [167, 228, 160], [35, 60, 236], [177, 59, 225], [28, 180, 144], [183, 97, 192], [255, 174, 229], [1, 68, 25], [195, 68, 146], [147, 207, 43], [208, 107, 197], [49, 169, 38], [117, 107, 94], [8, 70, 195], [83, 159, 193], [95, 94, 164], [109, 218, 252], [232, 113, 235], [187, 255, 247], [164, 176, 62], [76, 201, 201], [181, 11, 124], [255, 4, 93], [136, 29, 66], [194, 238, 154], [4, 87, 84], [71, 146, 121], [28, 183, 128], [118, 115, 177], [23, 231, 236], [180, 240, 176], [218, 115, 75], [46, 187, 20], [251, 54, 144], [64, 79, 255], [133, 6, 181], [17, 134, 8], [42, 174, 67], [182, 245, 58], [170, 196, 69], [236, 39, 191], [178, 197, 190], [98, 153, 109], [46, 75, 224], [17, 197, 162], [194, 135, 75], [184, 239, 168], [61, 157, 21], [49, 139, 196], [255, 144, 34], [226, 250, 8], [9, 197, 207], [106, 82, 88], [231, 160, 203], [114, 2, 59], [154, 218, 103], [48, 152, 44], [50, 187, 244], [124, 36, 227], [6, 116, 200], [30, 235, 30], [39, 139, 174], [7, 18, 116], [210, 254, 164], [182, 127, 240], [149, 20, 217], [91, 167, 111], [169, 85, 163], [107, 175, 216], [129, 217, 232], [30, 167, 223], [118, 255, 127], [66, 254, 226], [20, 27, 234], [122, 37, 222], [75, 119, 27], [16, 55, 187], [35, 248, 110], [219, 41, 178], [103, 4, 138], [184, 43, 237], [207, 236, 117], [70, 194, 77], [222, 191, 31], [74, 223, 121], [244, 231, 183], [116, 189, 182], [157, 227, 89], [83, 25, 112], [185, 38, 93], [137, 155, 252], [245, 204, 213], [203, 144, 171], [13, 153, 67], [42, 28, 200], [2, 94, 193], [30, 171, 111], [45, 108, 159], [130, 169, 202], [133, 140, 159], [62, 148, 123], [82, 247, 99], [141, 34, 49], [125, 211, 143], [249, 0, 139], [164, 175, 220], [17, 166, 142], [37, 128, 45], [13, 144, 146], [68, 229, 178], [60, 136, 190], [126, 154, 156], [209, 165, 107], [115, 10, 22], [52, 85, 93], [56, 229, 82], [11, 35, 185], [168, 95, 62], [123, 211, 88], [112, 236, 62], [254, 230, 29], [245, 72, 239], [113, 189, 76], [147, 178, 49], [44, 70, 38], [14, 249, 222], [200, 53, 58], [131, 198, 106], [204, 12, 14], [168, 204, 154], [91, 162, 228], [66, 160, 82], [144, 179, 79], [206, 220, 196], [53, 126, 82], [188, 187, 230], [189, 132, 38], [204, 89, 204], [59, 210, 243], [143, 232, 133], [62, 21, 153], [83, 24, 96], [43, 102, 55], [135, 165, 100], [91, 135, 93], [197, 186, 77], [207, 39, 238], [88, 104, 32], [79, 141, 74], [216, 66, 229], [192, 119, 204], [136, 176, 31], [174, 198, 209], [108, 248, 32], [237, 218, 195], [44, 118, 183], [34, 249, 13], [94, 181, 237], [73, 136, 29], [251, 158, 105], [108, 35, 135], [18, 107, 2], [23, 161, 62], [65, 155, 60], [118, 68, 198], [168, 193, 170], [11, 165, 182], [58, 53, 19], [43, 207, 32], [201, 200, 246], [132, 122, 116], [37, 93, 203], [184, 136, 74], [144, 95, 201], [231, 101, 106], [125, 38, 26], [200, 52, 38], [86, 0, 148], [159, 237, 23], [74, 130, 217], [164, 20, 222], [201, 193, 153], [73, 26, 80], [235, 240, 148], [33, 197, 186], [125, 129, 13], [77, 194, 199], [7, 68, 146], [76, 199, 128], [103, 70, 152], [48, 80, 138], [4, 232, 89], [77, 146, 230], [135, 8, 226], [203, 204, 183], [214, 18, 239], [165, 164, 97], [62, 203, 13], [201, 169, 185], [198, 35, 48], [26, 121, 114], [184, 10, 166], [202, 188, 250], [27, 84, 186], [153, 107, 63], [51, 223, 200], [133, 224, 187], [244, 90, 135], [136, 225, 250], [102, 142, 233], [30, 216, 190], [51, 149, 90], [79, 173, 6], [166, 35, 71], [160, 53, 183], [42, 251, 86], [130, 9, 104], [142, 31, 95], [56, 183, 134], [166, 14, 55], [61, 114, 138], [174, 160, 118], [179, 247, 195], [5, 103, 217], [169, 235, 202], [103, 103, 105], [161, 201, 237], [132, 211, 120], [188, 132, 134], [91, 174, 247], [12, 4, 144], [79, 250, 54], [39, 119, 112], [35, 121, 11], [24, 144, 172], [73, 151, 135], [222, 42, 252], [108, 119, 8], [102, 227, 138], [119, 143, 245], [25, 206, 16], [99, 148, 42], [90, 217, 225], [250, 22, 66], [112, 196, 216], [173, 179, 230], [56, 154, 38], [243, 160, 134], [95, 223, 91], [225, 246, 21], [248, 210, 30], [161, 117, 118], [206, 111, 99], [33, 110, 252], [148, 158, 96], [144, 2, 210], [103, 157, 34], [41, 188, 118], [45, 234, 211], [78, 115, 199], [167, 102, 175], [245, 235, 206], [124, 16, 170], [226, 205, 245], [132, 57, 249], [51, 51, 229], [99, 60, 96], [68, 47, 15], [79, 167, 9], [220, 78, 205], [122, 31, 23], [224, 239, 63], [124, 56, 206], [66, 252, 44], [40, 208, 86], [159, 68, 113], [217, 236, 254], [190, 163, 60], [250, 229, 65], [25, 33, 214], [136, 11, 57], [59, 200, 177], [122, 48, 89], [6, 191, 189], [31, 8, 95], [86, 39, 78], [208, 87, 58], [146, 136, 174], [157, 23, 115], [37, 112, 247], [244, 181, 56], [77, 15, 147], [146, 139, 216], [145, 94, 31], [242, 45, 67], [147, 203, 234], [243, 131, 103], [239, 127, 200], [176, 125, 61], [141, 48, 70], [177, 1, 185], [236, 206, 169], [234, 166, 47], [65, 41, 243], [108, 41, 18], [208, 27, 117], [99, 100, 162], [185, 144, 202], [76, 5, 144], [64, 35, 10], [241, 130, 225], [17, 36, 227], [60, 44, 209], [219, 144, 2], [116, 77, 173], [28, 75, 58], [67, 160, 195], [214, 74, 83], [54, 193, 186], [15, 89, 221], [129, 49, 59], [164, 10, 248], [84, 163, 140], [20, 236, 101], [185, 53, 23], [8, 64, 192], [11, 253, 160], [137, 41, 55], [139, 119, 184], [248, 201, 26], [135, 34, 45], [20, 5, 75], [25, 54, 97], [231, 164, 118], [49, 58, 248], [179, 108, 160], [32, 114, 211], [234, 184, 93], [36, 80, 228], [52, 80, 109], [22, 227, 41], [115, 64, 145], [132, 87, 0], [65, 21, 138], [17, 73, 95], [48, 150, 124], [74, 183, 70], [130, 157, 111], [56, 150, 124], [111, 39, 148], [161, 17, 242], [187, 44, 34], [83, 180, 194], [53, 82, 85], [186, 71, 37], [90, 11, 188], [134, 9, 246], [240, 96, 57], [246, 155, 19], [57, 26, 146], [68, 241, 41], [0, 5, 58], [172, 242, 39], [134, 62, 139], [202, 186, 210], [65, 121, 54], [32, 246, 44], [54, 110, 19], [206, 49, 86], [62, 64, 167], [211, 230, 107], [162, 230, 237], [110, 81, 250], [11, 242, 204], [59, 144, 125], [31, 248, 111], [179, 99, 62], [187, 31, 44], [113, 234, 54], [210, 201, 225], [76, 147, 111], [186, 218, 107], [108, 17, 226], [0, 185, 25], [133, 237, 127], [64, 218, 144], [98, 20, 14], [47, 48, 4], [118, 46, 149], [46, 25, 37], [32, 229, 27], [227, 251, 116], [158, 181, 126], [15, 203, 69], [244, 65, 215], [0, 108, 89], [56, 190, 178], [70, 150, 194], [32, 94, 15], [133, 94, 101], [57, 245, 239], [29, 176, 64], [193, 7, 219], [51, 118, 7], [57, 213, 172], [10, 238, 230], [255, 5, 220], [182, 33, 136], [136, 208, 21], [103, 221, 243], [118, 82, 56], [203, 204, 183], [172, 173, 74], [30, 125, 30], [19, 157, 67], [19, 108, 191], [165, 171, 165], [94, 151, 196], [42, 175, 69], [135, 166, 88], [197, 181, 119], [152, 214, 175], [207, 130, 205], [172, 131, 118], [24, 40, 32], [213, 110, 254], [106, 237, 28], [40, 87, 32], [12, 152, 68], [124, 122, 139], [202, 106, 159], [186, 63, 120], [57, 51, 56], [147, 253, 25], [115, 41, 252], [202, 207, 83], [52, 176, 250], [42, 130, 170], [32, 158, 181], [12, 225, 252], [5, 236, 90], [175, 11, 200], [252, 167, 169], [109, 74, 93], [213, 135, 211], [15, 251, 179], [170, 250, 125], [76, 201, 235], [146, 94, 222], [64, 104, 210], [218, 81, 138], [138, 204, 103], [136, 37, 90], [131, 86, 237], [244, 37, 4], [22, 139, 147], [188, 6, 78], [56, 29, 238], [181, 127, 11], [92, 67, 233], [101, 229, 233], [154, 79, 251], [28, 95, 241], [54, 210, 102], [98, 59, 128], [240, 137, 109], [5, 167, 129], [171, 235, 177], [146, 15, 73], [239, 31, 241], [163, 151, 230], [193, 142, 19], [10, 13, 252], [238, 114, 7], [255, 227, 223], [7, 178, 80], [91, 213, 187], [107, 208, 235], [156, 129, 239], [92, 75, 65], [61, 252, 75], [221, 88, 116], [103, 107, 207], [203, 178, 223], [89, 155, 208], [158, 69, 157], [31, 236, 139], [251, 233, 209], [214, 199, 116], [177, 211, 177], [184, 207, 242], [175, 94, 229], [8, 65, 3], [147, 241, 165], [167, 58, 139], [61, 229, 197], [14, 34, 53], [30, 117, 147], [201, 197, 120], [53, 155, 133], [85, 43, 172], [95, 223, 255], [99, 75, 124], [173, 226, 14], [222, 220, 155], [194, 115, 77], [227, 183, 85], [128, 225, 86], [118, 248, 20], [216, 126, 185], [180, 181, 109], [113, 230, 226], [38, 31, 168], [165, 72, 36], [155, 95, 2], [240, 55, 2], [224, 132, 7], [178, 239, 128], [176, 38, 249], [246, 106, 176], [106, 72, 63], [234, 188, 67], [199, 65, 124], [193, 122, 16], [121, 4, 216], [163, 58, 58], [19, 254, 203], [82, 80, 170], [137, 48, 116], [169, 137, 224], [197, 33, 23], [111, 88, 243], [183, 31, 129], [135, 175, 97], [167, 121, 193], [241, 237, 76], [243, 189, 83], [230, 141, 116], [156, 149, 43], [98, 197, 174], [75, 238, 109], [234, 145, 181], [228, 100, 12], [142, 69, 171], [247, 38, 84], [90, 188, 42], [0, 27, 211], [97, 214, 248], [23, 47, 121], [161, 157, 212], [232, 241, 139], [25, 162, 9], [234, 157, 0], [126, 188, 242], [149, 102, 10], [221, 107, 152], [206, 116, 248], [68, 53, 201], [202, 82, 171], [22, 231, 137], [139, 52, 110], [56, 34, 19], [58, 18, 154], [127, 102, 24], [57, 47, 131], [239, 131, 134], [77, 180, 208], [226, 122, 122], [54, 37, 121], [110, 64, 34], [139, 111, 222], [238, 113, 216], [39, 66, 200], [149, 12, 3], [127, 83, 137], [139, 110, 232], [224, 201, 8], [209, 210, 247], [117, 167, 1], [253, 154, 92], [66, 98, 158], [212, 0, 199], [34, 193, 97], [131, 174, 224], [244, 188, 109], [250, 5, 214], [210, 94, 65], [253, 238, 67], [106, 76, 201], [145, 157, 13], [126, 14, 99], [191, 108, 253], [214, 94, 105], [120, 118, 102], [217, 198, 38], [121, 14, 230], [95, 210, 249], [133, 221, 1], [83, 245, 230], [19, 129, 111], [37, 227, 146], [218, 147, 192], [173, 105, 180], [141, 44, 190], [176, 95, 112], [60, 243, 250], [195, 178, 70], [228, 43, 127], [234, 221, 223], [199, 61, 234], [79, 201, 108], [107, 236, 201], [113, 212, 126], [221, 106, 156], [12, 221, 120], [100, 81, 69], [124, 89, 72], [138, 182, 232], [23, 156, 45], [2, 235, 175], [168, 115, 39], [136, 187, 165], [253, 31, 255], [135, 192, 161], [122, 75, 129], [104, 3, 165], [52, 126, 162], [23, 144, 10], [6, 160, 237], [205, 245, 85], [61, 229, 210], [77, 2, 216], [114, 105, 253], [170, 146, 251], [36, 204, 7], [165, 94, 30], [185, 246, 4], [209, 180, 90], [225, 72, 223], [40, 99, 215], [149, 76, 74], [61, 75, 215], [152, 127, 88], [52, 141, 170], [28, 125, 79], [102, 231, 91], [84, 122, 123], [67, 39, 118], [102, 138, 227], [169, 99, 199], [253, 201, 97], [224, 62, 81], [9, 172, 175], [140, 189, 253], [162, 138, 161], [124, 72, 230], [109, 72, 133], [222, 84, 93], [169, 200, 235], [218, 236, 194], [163, 6, 214], [87, 160, 230], [199, 209, 4], [151, 123, 38], [78, 17, 230], [186, 105, 162], [198, 195, 125], [8, 52, 5], [72, 35, 87], [155, 169, 17], [101, 154, 194], [219, 47, 124], [196, 31, 26], [131, 63, 112], [63, 37, 72], [71, 229, 159], [139, 104, 41], [222, 149, 82], [26, 239, 46], [91, 118, 97], [34, 118, 35], [251, 160, 50], [94, 225, 245], [35, 192, 22], [150, 71, 240], [206, 20, 182], [168, 162, 228], [255, 160, 165], [245, 3, 170], [113, 52, 90], [87, 87, 95], [187, 208, 4], [90, 6, 111], [21, 4, 138], [107, 44, 42], [193, 151, 225], [98, 128, 190], [90, 149, 82], [79, 158, 135], [250, 90, 239], [147, 142, 231], [182, 134, 32], [42, 230, 106], [106, 140, 242], [136, 138, 92], [59, 159, 58], [77, 43, 32], [28, 215, 70], [245, 10, 29], [241, 22, 49], [251, 209, 194], [80, 151, 100], [101, 103, 41], [3, 132, 247], [163, 99, 17], [120, 38, 251], [139, 166, 44], [19, 226, 238], [236, 207, 15], [199, 250, 215], [183, 64, 114], [138, 174, 123], [76, 126, 197], [158, 63, 113], [255, 38, 31], [125, 118, 35], [11, 56, 77], [115, 239, 248], [20, 71, 58], [36, 30, 64], [115, 160, 141], [67, 252, 115], [152, 82, 34], [241, 193, 82], [121, 178, 37], [161, 152, 197], [223, 67, 189], [109, 203, 154], [116, 195, 167], [28, 97, 27], [212, 25, 203], [7, 156, 34], [217, 47, 115], [249, 201, 217], [115, 121, 220], [115, 181, 253], [196, 141, 6], [48, 221, 107], [88, 194, 225], [169, 203, 198], [146, 96, 3], [88, 134, 220], [244, 18, 81], [130, 144, 155], [30, 4, 20], [63, 149, 110], [38, 254, 36], [1, 74, 140], [14, 1, 184], [84, 203, 2], [74, 171, 235], [91, 162, 112], [5, 23, 139], [98, 15, 218], [182, 182, 83], [93, 202, 71], [170, 103, 63], [194, 17, 219], [185, 142, 97], [39, 254, 199], [195, 236, 32], [26, 30, 190], [228, 41, 32], [108, 51, 153], [227, 142, 73], [68, 170, 141], [88, 243, 251], [212, 216, 228], [184, 164, 172], [35, 196, 1], [56, 159, 232], [2, 121, 233], [41, 249, 209], [223, 244, 168], [233, 119, 95], [104, 44, 105], [176, 199, 164], [245, 232, 79], [190, 245, 137], [33, 240, 11], [63, 153, 0], [31, 238, 163], [58, 59, 233], [45, 159, 33], [223, 249, 160], [111, 114, 46], [152, 8, 132], [186, 81, 65], [227, 204, 45], [97, 94, 244], [229, 28, 8], [11, 214, 68], [99, 252, 58], [31, 233, 75], [212, 215, 140], [117, 175, 10], [1, 233, 185], [233, 211, 13], [202, 228, 138], [190, 235, 109], [248, 111, 227], [27, 96, 181], [181, 89, 231], [190, 99, 166], [71, 163, 227], [168, 129, 139], [108, 184, 179], [4, 40, 180], [107, 2, 38], [56, 28, 117], [11, 117, 230], [114, 223, 199], [34, 67, 114], [45, 113, 151], [13, 19, 53], [15, 123, 68], [69, 37, 18], [204, 154, 74], [253, 77, 141], [197, 47, 223], [241, 31, 70], [169, 104, 194], [46, 29, 29], [183, 179, 109], [81, 214, 69], [110, 131, 163], [141, 125, 193], [204, 83, 188], [242, 141, 83], [122, 247, 33], [127, 71, 29], [214, 24, 219], [124, 78, 79], [107, 153, 217], [61, 212, 154], [206, 213, 158], [140, 27, 151], [137, 128, 143], [44, 41, 225], [244, 113, 101], [69, 102, 134], [135, 213, 231], [36, 129, 196], [17, 133, 16], [116, 71, 122], [160, 27, 148], [87, 29, 221], [249, 63, 179], [38, 116, 74], [43, 137, 86], [169, 74, 214], [35, 89, 16], [56, 232, 99], [142, 123, 97], [132, 186, 79], [21, 54, 166], [14, 120, 86], [240, 174, 16], [12, 230, 89], [44, 73, 209], [130, 249, 9], [52, 132, 250], [38, 79, 163], [208, 44, 67], [153, 8, 29], [237, 205, 245], [130, 151, 79], [221, 76, 12], [245, 221, 68], [128, 27, 196], [41, 59, 129], [211, 91, 190], [124, 105, 44], [15, 217, 20], [192, 135, 111], [98, 138, 218], [242, 248, 248], [138, 6, 74], [96, 26, 215], [108, 51, 234], [217, 135, 141], [66, 142, 196], [133, 169, 198], [29, 240, 156], [110, 244, 185], [137, 9, 188], [44, 129, 226], [153, 20, 220], [118, 203, 176], [194, 222, 96], [80, 149, 111], [92, 25, 241], [64, 27, 39], [235, 225, 177], [140, 188, 22], [13, 157, 3], [117, 245, 39], [25, 96, 110], [222, 180, 95], [124, 225, 211], [100, 97, 7], [141, 91, 131], [72, 188, 97], [55, 73, 101], [91, 112, 149], [125, 201, 178], [121, 189, 75], [84, 97, 100], [255, 220, 125], [71, 133, 227], [16, 143, 244], [146, 232, 119], [202, 120, 246], [133, 187, 145], [254, 100, 248], [125, 225, 36], [134, 86, 245], [106, 145, 50], [245, 29, 117], [204, 60, 56], [19, 89, 140], [3, 17, 65], [191, 2, 58], [209, 160, 17], [147, 40, 191], [141, 62, 112], [72, 126, 249], [131, 222, 31], [222, 98, 217], [231, 36, 110], [50, 209, 49], [246, 208, 139], [90, 205, 25], [120, 16, 10], [165, 81, 206], [81, 3, 145], [127, 139, 71], [37, 29, 100], [7, 209, 12], [33, 238, 26], [167, 39, 146], [93, 78, 114], [11, 74, 11], [229, 60, 8], [237, 40, 86], [145, 210, 214], [20, 56, 230], [24, 84, 81], [108, 10, 15], [210, 49, 199], [71, 220, 33], [149, 118, 135], [179, 160, 112], [93, 171, 83], [242, 105, 188], [175, 16, 55], [12, 162, 143], [62, 192, 29], [112, 127, 128], [217, 217, 169], [41, 125, 167], [37, 227, 216], [246, 42, 72], [204, 171, 4], [50, 222, 93], [10, 110, 136], [214, 122, 148], [127, 134, 232], [56, 231, 87], [227, 231, 212], [131, 148, 176], [56, 113, 229], [110, 244, 229], [214, 16, 226], [223, 225, 214], [112, 17, 63], [82, 221, 159], [22, 242, 23], [141, 48, 155], [7, 115, 233], [119, 1, 24], [13, 4, 233], [178, 29, 49], [48, 104, 152], [164, 71, 127], [184, 230, 67], [53, 143, 155], [45, 82, 84], [125, 118, 74], [12, 63, 161], [212, 79, 88], [46, 183, 18], [182, 64, 137], [226, 77, 66], [147, 72, 87], [3, 176, 80], [253, 133, 89], [211, 161, 182], [195, 151, 252], [191, 47, 41], [33, 228, 99], [201, 96, 219], [231, 127, 151], [97, 23, 182], [72, 6, 254], [118, 109, 25], [177, 109, 226], [163, 254, 201], [141, 114, 49], [93, 53, 112], [182, 251, 157], [95, 63, 190], [175, 252, 73], [151, 180, 227], [79, 5, 113], [107, 242, 252], [201, 232, 50], [246, 253, 175], [70, 27, 222], [228, 212, 22], [143, 246, 15], [43, 80, 71], [164, 95, 143], [185, 102, 217], [58, 198, 140], [139, 87, 99], [187, 106, 188], [15, 10, 111], [171, 33, 75], [193, 81, 53], [47, 217, 202], [116, 146, 243], [106, 235, 171], [67, 234, 114], [187, 51, 80], [133, 120, 215], [212, 79, 27], [103, 57, 247], [109, 83, 195], [242, 26, 237], [208, 108, 200], [68, 81, 41], [70, 134, 47], [208, 167, 54], [16, 243, 144], [178, 29, 101], [146, 3, 95], [123, 5, 131], [55, 118, 23], [91, 232, 198], [135, 126, 74], [253, 246, 222], [6, 106, 53], [202, 114, 132], [241, 112, 78], [199, 247, 117], [154, 209, 189], [156, 254, 67], [243, 133, 97], [19, 119, 243], [142, 63, 240], [179, 181, 203], [39, 86, 135], [76, 205, 251], [66, 6, 205], [5, 52, 167], [65, 241, 180], [223, 25, 106], [112, 210, 238], [95, 162, 187], [0, 215, 41], [174, 188, 183], [96, 141, 199], [10, 138, 251], [198, 157, 109], [93, 63, 72], [49, 24, 171], [12, 110, 33], [246, 171, 129], [171, 46, 138], [63, 238, 148], [253, 178, 74], [93, 162, 223], [227, 42, 224], [229, 4, 98], [135, 165, 8], [209, 246, 164], [132, 45, 241], [189, 38, 33], [176, 17, 7], [181, 35, 114], [47, 109, 97], [154, 208, 30], [94, 189, 49], [30, 193, 117], [133, 95, 118], [29, 46, 166], [190, 122, 35], [198, 41, 148], [117, 176, 242], [57, 7, 168], [96, 48, 203], [182, 238, 233], [89, 81, 105], [65, 163, 238], [29, 97, 244], [149, 164, 247], [67, 248, 28], [140, 12, 205], [202, 47, 149], [128, 236, 101], [242, 255, 110], [114, 62, 96], [165, 72, 157], [215, 31, 94], [78, 15, 225], [136, 71, 34], [105, 95, 100], [234, 212, 108], [97, 6, 126], [62, 21, 28], [51, 200, 225], [205, 236, 65], [92, 139, 33], [162, 128, 187], [36, 64, 79], [233, 214, 220], [141, 92, 130], [141, 24, 115], [71, 253, 105], [202, 22, 5], [45, 66, 183], [103, 241, 180], [20, 17, 129], [27, 106, 64], [63, 48, 201], [68, 32, 116], [101, 92, 70], [245, 109, 200], [198, 12, 92], [222, 67, 77], [166, 107, 248], [133, 184, 245], [162, 16, 69], [88, 149, 30], [210, 175, 110], [19, 112, 20], [252, 78, 208], [201, 13, 222], [158, 79, 248], [186, 164, 199], [44, 138, 240], [92, 106, 179], [205, 23, 123], [252, 216, 122], [212, 150, 103], [31, 44, 38], [161, 26, 37], [150, 90, 229], [199, 84, 91], [82, 93, 29], [241, 6, 30], [89, 192, 105], [240, 70, 8], [12, 49, 207], [115, 94, 216], [61, 7, 83], [246, 226, 120], [46, 180, 195], [168, 9, 83], [57, 165, 67], [226, 110, 88], [55, 135, 27], [51, 157, 206], [228, 185, 64], [151, 134, 214], [225, 89, 136], [250, 225, 250], [111, 74, 115], [254, 161, 58], [25, 233, 68], [75, 65, 155], [11, 188, 75], [119, 248, 98], [135, 245, 135], [250, 0, 188], [178, 69, 15], [255, 217, 198], [199, 126, 122], [145, 76, 219], [4, 224, 190], [66, 169, 227], [181, 39, 199], [119, 146, 137], [168, 172, 73], [249, 115, 86], [208, 138, 138], [42, 67, 144], [171, 10, 174], [68, 202, 223], [146, 142, 90], [210, 104, 249], [50, 153, 236], [53, 172, 83], [138, 81, 223], [55, 42, 36], [219, 73, 185], [87, 109, 78], [39, 148, 9], [4, 213, 168], [178, 207, 211], [196, 235, 171], [234, 175, 232], [72, 164, 60], [120, 188, 42], [202, 44, 245], [133, 42, 114], [127, 0, 253], [198, 178, 113], [134, 163, 48], [215, 135, 244], [119, 135, 154], [25, 206, 237], [239, 161, 45], [56, 232, 232], [206, 164, 196], [109, 196, 253], [201, 203, 215], [222, 128, 131], [1, 138, 140], [217, 57, 150], [70, 2, 99], [52, 30, 105], [82, 179, 182], [5, 163, 179], [9, 129, 208], [147, 88, 213], [206, 151, 91], [184, 77, 59], [52, 18, 120], [180, 222, 10], [181, 92, 156], [137, 189, 115], [71, 141, 186], [242, 254, 182], [129, 232, 66], [246, 6, 211], [15, 78, 100], [134, 141, 130], [239, 72, 122], [221, 43, 54], [95, 138, 2], [4, 244, 103], [98, 11, 241], [28, 65, 131], [25, 110, 48], [7, 126, 54], [156, 192, 110], [21, 175, 21], [255, 59, 239], [156, 225, 211], [24, 49, 111], [213, 145, 178], [198, 53, 83], [80, 159, 115], [182, 233, 205], [75, 2, 252], [143, 144, 164], [115, 18, 132], [196, 151, 235], [104, 20, 103], [5, 149, 249], [196, 226, 27], [13, 235, 104], [254, 63, 149], [140, 119, 84], [239, 67, 52], [172, 201, 137], [71, 19, 38], [150, 183, 198], [45, 251, 234], [169, 225, 192], [128, 50, 124], [39, 135, 40], [30, 37, 158], [117, 75, 208], [172, 185, 60], [42, 8, 95], [69, 34, 227], [65, 110, 104], [101, 177, 164], [208, 156, 189], [176, 74, 21], [212, 174, 52], [218, 37, 249], [95, 61, 10], [30, 179, 42], [57, 128, 105], [18, 235, 48], [45, 94, 76], [51, 140, 233], [208, 137, 159], [194, 221, 93], [67, 126, 54], [99, 75, 46], [154, 164, 182], [242, 11, 252], [246, 26, 96], [136, 134, 149], [48, 210, 191], [35, 77, 128], [1, 200, 218], [205, 188, 46], [81, 93, 232], [209, 252, 63], [6, 112, 219], [114, 164, 26], [217, 104, 77], [120, 182, 18], [230, 171, 160], [154, 244, 110], [193, 190, 150], [158, 244, 118], [24, 24, 200], [65, 65, 29], [199, 37, 14], [149, 1, 18], [201, 32, 120], [34, 106, 32], [96, 82, 25], [44, 99, 65], [6, 211, 135], [39, 248, 224], [27, 220, 2], [181, 93, 252], [186, 53, 159], [249, 146, 105], [33, 71, 77], [121, 82, 214], [109, 104, 160], [171, 186, 84], [136, 23, 152], [141, 12, 155], [175, 247, 223], [24, 133, 136], [144, 99, 142], [15, 35, 76], [242, 122, 27], [6, 208, 41], [114, 145, 162], [190, 174, 61], [121, 175, 206], [24, 163, 234], [173, 73, 0], [117, 180, 183], [191, 23, 241], [196, 70, 9], [168, 20, 247], [119, 75, 163], [35, 124, 189], [41, 100, 124], [229, 210, 229], [44, 116, 161], [212, 214, 140], [107, 176, 253], [99, 177, 144], [223, 216, 214], [52, 53, 254], [153, 167, 38], [126, 211, 139], [255, 173, 195], [200, 156, 186], [214, 31, 223], [129, 174, 61], [175, 179, 26], [218, 98, 113], [21, 196, 25], [195, 193, 154], [168, 230, 245], [100, 222, 236], [229, 204, 229], [131, 251, 68], [52, 160, 98], [246, 182, 170], [96, 101, 60], [219, 210, 173], [225, 13, 252], [243, 249, 168], [160, 139, 185], [178, 5, 113], [112, 154, 142], [119, 190, 190], [111, 66, 184], [209, 7, 39], [45, 103, 98], [151, 247, 205], [71, 160, 63], [164, 52, 119], [74, 37, 119], [202, 35, 158], [210, 251, 37], [245, 16, 173], [102, 39, 252], [174, 235, 56], [116, 243, 150], [22, 206, 185], [33, 12, 245], [168, 134, 109], [6, 37, 148], [138, 171, 14], [124, 89, 102], [139, 15, 91], [93, 74, 225], [126, 22, 26], [129, 47, 149], [60, 16, 127], [132, 247, 124], [234, 246, 7], [242, 246, 140], [46, 42, 111], [107, 234, 110], [125, 235, 162], [190, 107, 31], [146, 117, 69], [1, 164, 37], [247, 240, 29], [196, 123, 211], [225, 132, 246], [113, 160, 214], [87, 92, 129], [33, 4, 163], [164, 80, 252], [87, 98, 43], [78, 251, 205], [230, 172, 246], [238, 65, 251], [113, 247, 103], [132, 174, 65], [178, 198, 165], [172, 79, 140], [252, 52, 149], [127, 40, 22], [114, 39, 82], [171, 155, 248], [134, 181, 171], [215, 15, 252], [195, 37, 100], [101, 224, 243], [154, 244, 0], [197, 24, 12], [118, 169, 92], [140, 17, 30], [41, 26, 68], [117, 211, 100], [15, 113, 213], [46, 243, 90], [118, 163, 145], [29, 114, 48], [248, 163, 51], [13, 155, 237], [229, 210, 183], [201, 3, 156], [254, 135, 111], [131, 66, 221], [12, 25, 70], [145, 211, 15], [119, 171, 20], [78, 113, 54], [5, 122, 17], [47, 183, 158], [133, 224, 223], [195, 134, 184], [225, 171, 109], [138, 202, 210], [165, 43, 89], [52, 162, 40], [68, 76, 25], [245, 135, 129], [21, 108, 139], [6, 194, 111], [7, 69, 203], [240, 65, 128], [244, 196, 46], [100, 130, 94], [32, 39, 246], [44, 231, 233], [144, 125, 183], [116, 181, 226], [221, 2, 18], [203, 167, 145], [41, 166, 177], [200, 105, 108], [104, 28, 197], [144, 47, 100], [239, 143, 204], [196, 243, 41], [78, 131, 167], [153, 238, 87], [152, 97, 80], [26, 15, 131], [99, 215, 106], [132, 45, 203], [167, 165, 64], [96, 2, 157], [212, 159, 117], [192, 189, 20], [37, 240, 21], [11, 95, 140], [209, 174, 193], [116, 248, 177], [25, 97, 113], [16, 12, 171], [89, 231, 224], [94, 0, 242], [151, 4, 38], [191, 153, 222], [89, 5, 111], [10, 58, 33], [47, 204, 35], [226, 33, 117], [90, 39, 94], [222, 158, 235], [135, 98, 38], [159, 111, 156], [134, 86, 35], [121, 226, 78], [20, 254, 79], [132, 237, 141], [197, 53, 76], [28, 200, 114], [42, 55, 184], [135, 142, 220], [128, 189, 24], [116, 110, 6], [216, 150, 86], [5, 60, 233], [194, 251, 23], [212, 205, 76], [252, 133, 136], [31, 29, 217], [112, 231, 34], [161, 191, 215], [12, 119, 37], [2, 67, 129], [17, 153, 89], [69, 218, 54], [204, 53, 132], [52, 238, 144], [168, 151, 211], [117, 101, 148], [194, 218, 34], [147, 99, 238], [41, 86, 198], [211, 133, 249], [152, 183, 86], [120, 191, 229], [206, 224, 152], [142, 94, 167], [136, 103, 16], [49, 166, 255], [189, 101, 41], [234, 31, 94], [181, 148, 15], [176, 53, 135], [118, 174, 77], [183, 220, 126], [19, 241, 236], [123, 150, 126], [255, 75, 225], [56, 97, 34], [154, 114, 38], [216, 70, 202], [10, 66, 202], [115, 16, 57], [144, 0, 32], [30, 51, 115], [155, 215, 30], [147, 154, 54], [206, 158, 218], [128, 184, 74], [173, 250, 251], [53, 37, 123], [34, 6, 68], [128, 10, 83], [102, 103, 155], [105, 187, 158], [100, 224, 194], [245, 145, 19], [60, 23, 186], [100, 213, 89], [255, 119, 86], [125, 205, 182], [83, 141, 106], [1, 38, 232], [42, 97, 135], [32, 95, 20], [192, 100, 73], [11, 77, 159], [99, 234, 230], [193, 203, 105], [234, 57, 216], [28, 2, 126], [235, 168, 13], [198, 39, 244], [15, 223, 82], [223, 66, 127], [192, 34, 79], [137, 100, 17], [152, 6, 131], [33, 68, 236], [163, 236, 149], [186, 223, 255], [193, 146, 141], [102, 255, 235], [35, 158, 12], [15, 252, 71], [193, 245, 37], [221, 121, 167], [125, 22, 141], [73, 149, 235], [217, 20, 5], [211, 36, 185], [47, 143, 67], [129, 241, 48], [219, 28, 143], [18, 145, 40], [208, 11, 0], [147, 179, 98], [127, 245, 201], [91, 146, 79], [131, 100, 237], [72, 63, 27], [80, 160, 164], [89, 230, 35], [118, 236, 200], [42, 0, 197], [37, 109, 218], [124, 1, 5], [240, 91, 167], [65, 67, 199], [12, 114, 142], [225, 210, 237], [145, 108, 21], [178, 155, 98], [160, 96, 180], [150, 221, 194], [195, 247, 137], [64, 39, 94], [185, 136, 28], [176, 235, 44], [231, 85, 81], [81, 73, 97], [203, 204, 187], [185, 234, 148], [2, 230, 209], [14, 98, 105], [71, 205, 238], [210, 100, 41], [104, 53, 86], [36, 192, 107], [205, 124, 152], [1, 89, 130], [54, 121, 100], [7, 155, 95], [249, 45, 106], [152, 200, 125], [4, 215, 190], [51, 162, 130], [9, 59, 106], [226, 158, 117], [164, 250, 241], [135, 180, 26], [143, 247, 136], [88, 128, 224], [71, 38, 164], [16, 98, 232], [143, 85, 214], [185, 127, 27], [94, 78, 124], [82, 175, 234], [1, 21, 95], [58, 33, 159], [88, 213, 170], [143, 25, 228], [13, 49, 9], [73, 84, 150], [49, 252, 71], [254, 90, 191], [86, 153, 17], [252, 124, 44], [237, 72, 156], [161, 207, 164], [203, 201, 29], [154, 147, 113], [151, 93, 255], [197, 149, 152], [99, 33, 188], [186, 155, 77], [242, 141, 129], [28, 210, 118], [121, 219, 135], [167, 50, 45], [243, 94, 99], [119, 142, 103], [173, 230, 206], [89, 150, 171], [121, 240, 165], [65, 91, 121], [157, 37, 216], [127, 100, 178], [184, 245, 208], [137, 188, 241], [199, 177, 74], [148, 233, 198], [179, 75, 26], [201, 88, 188], [40, 125, 59], [148, 65, 148], [17, 129, 117], [144, 205, 52], [254, 26, 229], [210, 133, 167], [35, 240, 0], [75, 233, 132], [193, 217, 224], [233, 198, 233], [66, 45, 175], [242, 253, 9], [202, 29, 252], [81, 229, 18], [27, 130, 74], [162, 129, 210], [39, 212, 93], [225, 134, 149], [38, 176, 159], [186, 100, 65], [24, 93, 233], [168, 71, 16], [193, 245, 84], [244, 183, 41], [114, 70, 112], [120, 131, 222], [51, 71, 106], [246, 152, 149], [64, 239, 170], [167, 175, 192], [167, 240, 85], [183, 33, 1], [118, 100, 185], [208, 81, 244], [9, 47, 4], [56, 150, 137], [210, 125, 254], [224, 100, 145], [37, 214, 202], [79, 108, 203], [92, 235, 27], [130, 177, 133], [15, 63, 109], [41, 34, 85], [3, 41, 82], [106, 126, 144], [115, 203, 204], [216, 129, 153], [181, 39, 109], [85, 193, 214], [197, 171, 50], [45, 80, 151], [243, 229, 44], [137, 70, 200], [178, 0, 62], [250, 177, 149], [205, 207, 15], [243, 49, 89], [130, 75, 131], [156, 9, 129], [142, 112, 43], [248, 228, 165], [109, 0, 207], [126, 87, 157], [75, 167, 23], [230, 253, 158], [164, 1, 10], [246, 209, 160], [197, 68, 224], [196, 23, 182], [217, 140, 189], [218, 91, 208], [38, 181, 6], [172, 186, 109], [121, 14, 45], [180, 47, 233], [164, 52, 44], [129, 136, 220], [56, 6, 113], [234, 90, 73], [210, 211, 144], [24, 6, 122], [80, 208, 78], [135, 219, 178], [219, 14, 91], [193, 229, 203], [26, 209, 163], [217, 98, 190], [72, 94, 77], [35, 133, 213], [255, 123, 216], [110, 11, 65], [79, 20, 42], [208, 4, 224], [59, 205, 94], [214, 241, 204], [111, 137, 8], [195, 187, 205], [49, 61, 47], [225, 96, 13], [149, 217, 103], [203, 51, 98], [146, 168, 125], [169, 245, 199], [44, 50, 192], [241, 204, 90], [197, 34, 190], [250, 110, 7], [80, 195, 108], [192, 212, 185], [56, 117, 69], [156, 202, 222], [165, 59, 64], [178, 193, 104], [221, 75, 100], [7, 152, 130], [218, 237, 170], [218, 60, 52], [207, 62, 229], [247, 117, 207], [25, 102, 171], [157, 28, 97], [163, 200, 121], [3, 1, 253], [39, 109, 93], [119, 19, 18], [112, 210, 34], [230, 214, 204], [2, 183, 214], [60, 181, 142], [87, 48, 212], [249, 38, 178], [185, 40, 5], [0, 228, 230], [75, 229, 63], [95, 152, 35], [130, 213, 241], [59, 203, 74], [139, 18, 239], [183, 251, 185], [62, 152, 116], [127, 122, 136], [250, 146, 195], [214, 10, 95], [33, 47, 208], [53, 132, 57], [191, 73, 113], [121, 26, 245], [26, 231, 7], [244, 240, 106], [195, 161, 223], [129, 168, 133], [127, 128, 210], [113, 247, 76], [242, 100, 188], [234, 195, 27], [204, 202, 3], [130, 176, 223], [94, 150, 191], [68, 214, 92], [30, 115, 229], [208, 206, 222], [19, 29, 207], [159, 23, 99], [105, 85, 50], [66, 105, 102], [105, 227, 122], [217, 144, 118], [182, 115, 97], [242, 52, 83], [228, 128, 234], [239, 27, 115], [234, 231, 213], [204, 13, 77], [104, 77, 219], [23, 43, 109], [182, 172, 253], [84, 157, 211], [160, 203, 178], [85, 203, 204], [59, 105, 13], [209, 243, 134], [113, 109, 211], [17, 148, 237], [228, 134, 58], [215, 20, 137], [78, 51, 234], [31, 121, 182], [89, 34, 28], [92, 55, 139], [55, 171, 37], [70, 16, 223], [128, 231, 199], [199, 191, 192], [242, 25, 246], [251, 124, 207], [74, 80, 139], [66, 112, 161], [234, 166, 78], [51, 119, 212], [87, 191, 23], [128, 2, 255], [220, 100, 146], [194, 17, 14], [231, 75, 67], [4, 19, 16], [55, 121, 210], [142, 134, 176], [120, 210, 5], [53, 84, 221], [22, 63, 215], [63, 22, 143], [72, 33, 247], [42, 43, 76], [185, 63, 142], [148, 76, 167], [189, 0, 248], [88, 42, 237], [249, 237, 230], [23, 57, 163], [105, 17, 83], [149, 223, 244], [255, 159, 33], [129, 175, 193], [20, 212, 195], [157, 83, 200], [202, 202, 95], [149, 90, 71], [109, 39, 23], [234, 48, 2], [102, 26, 193], [247, 150, 239], [141, 175, 88], [57, 8, 202], [177, 9, 194], [41, 114, 158], [147, 9, 187], [38, 240, 219], [71, 229, 64], [191, 254, 154], [123, 92, 133], [104, 121, 255], [163, 172, 170], [176, 180, 113], [43, 190, 31], [34, 124, 126], [225, 225, 20], [25, 23, 112], [136, 65, 164], [95, 173, 9], [180, 223, 97], [100, 184, 87], [207, 69, 81], [234, 167, 177], [168, 153, 100], [38, 78, 32], [184, 171, 180], [246, 161, 142], [189, 30, 193], [232, 250, 11], [55, 192, 85], [79, 221, 193], [185, 139, 92], [28, 25, 138], [88, 202, 125], [7, 25, 161], [19, 50, 227], [243, 86, 162], [93, 107, 35], [53, 218, 149], [200, 184, 91], [242, 134, 190], [130, 154, 73], [30, 130, 213], [152, 155, 250], [153, 25, 114], [195, 232, 4], [134, 30, 16], [205, 166, 57], [3, 153, 161], [10, 250, 84], [28, 94, 8], [87, 10, 253], [115, 160, 203], [163, 54, 108], [189, 206, 227], [186, 218, 149], [88, 52, 6], [231, 181, 8], [37, 189, 181], [252, 82, 114], [104, 174, 195], [152, 112, 104], [95, 7, 194], [104, 70, 81], [179, 232, 52], [200, 164, 191], [13, 237, 248], [43, 207, 59], [122, 223, 229], [105, 142, 191], [99, 105, 28], [16, 225, 65], [15, 139, 2], [178, 74, 143], [167, 111, 19], [33, 248, 225], [176, 79, 174], [104, 176, 202], [248, 226, 188], [72, 206, 39], [252, 122, 146], [181, 113, 75], [171, 81, 167], [248, 138, 218], [52, 99, 35], [189, 79, 136], [198, 146, 173], [78, 101, 207], [215, 113, 18], [175, 91, 154], [174, 82, 10], [219, 245, 248], [96, 59, 189], [204, 183, 31], [241, 221, 108], [53, 158, 11], [55, 192, 178], [11, 77, 90], [145, 147, 156], [127, 131, 214], [166, 65, 194], [37, 182, 6], [142, 91, 121], [148, 21, 237], [10, 58, 26], [137, 235, 96], [8, 92, 149], [69, 120, 161], [188, 151, 108], [132, 240, 145], [233, 203, 195], [5, 180, 221], [73, 218, 161], [51, 185, 15], [119, 156, 53], [95, 0, 61], [185, 29, 130], [64, 239, 199], [41, 230, 200], [7, 79, 146], [214, 78, 42], [127, 239, 52], [84, 153, 169], [196, 3, 105], [219, 191, 66], [53, 229, 251], [176, 214, 212], [134, 96, 236], [87, 3, 246], [191, 2, 144], [223, 78, 60], [192, 124, 143], [39, 250, 121], [121, 237, 87], [184, 146, 139], [248, 200, 255], [7, 15, 238], [127, 228, 143], [11, 231, 70], [122, 15, 120], [158, 89, 191], [72, 187, 250], [237, 212, 72], [56, 118, 159], [202, 22, 165], [64, 111, 167], [252, 124, 71], [152, 5, 167], [112, 194, 136], [162, 189, 136], [254, 144, 134], [143, 160, 205], [14, 22, 24], [70, 135, 120], [155, 110, 5], [123, 232, 46], [146, 157, 184], [216, 83, 17], [58, 199, 26], [86, 212, 61], [75, 119, 124], [50, 42, 248], [131, 184, 213], [46, 86, 214], [195, 220, 225], [8, 31, 154], [144, 78, 34], [85, 179, 221], [19, 119, 74], [254, 64, 211], [51, 217, 177], [60, 203, 47], [207, 134, 29], [4, 61, 228], [255, 98, 49], [66, 244, 4], [75, 204, 6], [149, 148, 98], [11, 95, 58], [232, 93, 75], [75, 207, 251], [234, 16, 4], [189, 12, 4], [19, 241, 46], [31, 166, 213], [127, 145, 1], [228, 24, 44], [159, 53, 197], [69, 100, 84], [136, 98, 115], [227, 92, 208], [182, 100, 195], [60, 48, 164], [20, 144, 132], [153, 54, 193], [214, 144, 168], [187, 68, 236], [217, 57, 150], [132, 165, 56], [227, 77, 13], [237, 118, 201], [74, 211, 43], [170, 255, 126], [6, 178, 131], [210, 222, 213], [15, 219, 199], [58, 58, 216], [210, 58, 214], [26, 171, 54], [217, 53, 205], [244, 122, 51], [247, 166, 231], [123, 145, 180], [192, 184, 68], [191, 57, 213], [99, 114, 209], [149, 194, 188], [236, 66, 115], [248, 49, 198], [221, 175, 10], [8, 203, 91], [160, 78, 79], [112, 53, 178], [0, 174, 60], [220, 205, 77], [48, 16, 28], [199, 126, 168], [2, 28, 97], [202, 230, 187], [168, 136, 31], [253, 97, 123], [12, 40, 49], [51, 18, 198], [75, 248, 43], [225, 138, 147], [83, 5, 248], [206, 83, 128], [212, 217, 66], [77, 149, 132], [148, 93, 47], [184, 161, 151], [102, 84, 252], [237, 7, 8], [194, 169, 184], [63, 107, 63], [185, 35, 98], [47, 168, 138], [151, 205, 130], [239, 74, 195], [106, 226, 193], [118, 217, 189], [200, 169, 4], [21, 100, 17], [43, 210, 175], [101, 151, 143], [124, 37, 69], [105, 17, 103], [178, 248, 233], [127, 225, 170], [27, 135, 186], [154, 199, 163], [106, 106, 137], [247, 54, 205], [88, 5, 108], [148, 40, 46], [163, 4, 173], [158, 22, 200], [228, 145, 170], [166, 162, 135], [91, 44, 77], [225, 97, 55], [173, 86, 214], [35, 12, 238], [64, 211, 182], [74, 183, 255], [255, 223, 31], [159, 111, 199], [193, 54, 16], [159, 2, 183], [162, 102, 243], [55, 135, 109], [216, 49, 242], [211, 52, 32], [1, 231, 234], [195, 6, 127], [2, 132, 67], [73, 151, 4], [183, 132, 176], [161, 86, 129], [138, 163, 243], [226, 96, 118], [59, 23, 224], [216, 216, 73], [65, 190, 228], [145, 77, 152], [36, 92, 26], [140, 8, 145], [85, 53, 241], [47, 175, 146], [237, 17, 165], [127, 16, 232], [12, 166, 106], [40, 213, 137], [236, 12, 251], [56, 85, 208], [224, 42, 249], [103, 250, 72], [175, 241, 64], [79, 250, 35], [29, 116, 8], [17, 178, 117], [220, 111, 135], [193, 83, 128], [95, 66, 197], [41, 181, 19], [70, 136, 14], [112, 137, 147], [175, 121, 223], [41, 218, 234], [42, 223, 18], [43, 93, 144], [112, 140, 131], [143, 133, 112], [190, 98, 26], [228, 61, 3], [32, 127, 64], [46, 92, 252], [79, 22, 120], [95, 73, 92], [62, 206, 71], [115, 58, 64], [220, 20, 66], [11, 30, 242], [239, 17, 233], [74, 214, 2], [18, 147, 127], [88, 172, 151], [109, 132, 125], [201, 13, 220], [146, 156, 122], [62, 119, 3], [253, 222, 5], [48, 189, 18], [101, 145, 64], [118, 246, 100], [182, 192, 202], [196, 155, 2], [173, 171, 189], [241, 104, 108], [225, 166, 238], [230, 3, 124], [220, 191, 123], [98, 3, 189], [241, 192, 83], [47, 179, 151], [191, 71, 109], [94, 109, 201], [69, 46, 33], [226, 185, 223], [237, 60, 224], [141, 253, 128], [241, 253, 244], [196, 27, 50], [109, 58, 52], [165, 49, 213], [65, 116, 75], [169, 0, 67], [100, 246, 223], [232, 181, 139], [183, 162, 132], [198, 136, 237], [148, 4, 36], [27, 224, 67], [95, 142, 43], [246, 253, 74], [92, 171, 232], [113, 65, 120], [144, 27, 38], [135, 92, 113], [107, 73, 98], [165, 169, 2], [184, 189, 32], [87, 98, 12], [233, 131, 39], [201, 147, 40], [50, 222, 206], [203, 204, 189], [14, 208, 117], [156, 6, 38], [115, 246, 189], [159, 249, 242], [130, 172, 238], [242, 188, 156], [124, 6, 122], [101, 235, 158], [219, 116, 239], [81, 34, 12], [177, 246, 126], [116, 65, 83], [36, 50, 69], [81, 14, 49], [97, 172, 202], [81, 142, 28], [117, 45, 121], [138, 7, 48], [110, 240, 4], [58, 100, 243], [64, 165, 188], [48, 148, 128], [73, 227, 81], [252, 160, 209], [64, 9, 16], [204, 113, 206], [110, 39, 228], [157, 97, 107], [37, 203, 212], [43, 150, 166], [96, 67, 58], [52, 126, 57], [163, 69, 52], [27, 208, 31], [50, 35, 21], [102, 42, 243], [164, 132, 177], [77, 41, 13], [52, 14, 109], [37, 255, 142], [26, 174, 184], [16, 79, 252], [157, 124, 120], [11, 103, 164], [241, 142, 47], [249, 235, 237], [91, 0, 59], [12, 209, 21], [219, 233, 90], [136, 39, 188], [72, 239, 109], [64, 49, 11], [203, 165, 239], [175, 2, 4], [49, 230, 155], [236, 245, 37], [148, 65, 97], [89, 79, 166], [248, 191, 34], [220, 213, 48], [78, 48, 50], [1, 186, 72], [208, 53, 214], [162, 42, 23], [202, 144, 25], [117, 93, 59], [28, 70, 2], [151, 195, 205], [158, 39, 224], [94, 153, 181], [162, 185, 100], [100, 184, 187], [55, 90, 235], [68, 136, 105], [168, 24, 131], [189, 121, 64], [55, 88, 247], [183, 193, 241], [204, 202, 71], [182, 141, 246], [86, 200, 70], [13, 211, 107], [81, 229, 120], [235, 245, 130], [234, 213, 103], [17, 192, 35], [224, 2, 178], [14, 80, 35], [53, 159, 164], [163, 51, 215], [70, 76, 152], [94, 239, 135], [174, 226, 72], [239, 1, 163], [173, 240, 219], [56, 45, 24], [100, 24, 180], [132, 35, 111], [52, 129, 149], [99, 12, 88], [110, 254, 67], [129, 28, 206], [222, 245, 41], [139, 151, 231], [54, 251, 48], [183, 220, 91], [136, 18, 170], [162, 83, 60], [231, 184, 250], [94, 43, 116], [235, 249, 168], [81, 190, 34], [83, 19, 219], [250, 211, 155], [101, 144, 14], [28, 50, 180], [140, 117, 196], [83, 151, 230], [132, 207, 60], [8, 75, 67], [80, 202, 41], [102, 83, 43], [252, 17, 100], [207, 173, 36], [53, 69, 164], [252, 155, 161], [51, 18, 4], [129, 223, 150], [247, 90, 26], [110, 35, 28], [100, 8, 116], [103, 166, 83], [194, 12, 62], [214, 79, 167], [89, 143, 10], [123, 234, 235], [47, 15, 71], [196, 221, 60], [234, 86, 224], [101, 98, 231], [27, 36, 92], [36, 108, 147], [55, 175, 157], [213, 99, 199], [52, 182, 66], [225, 145, 126], [168, 204, 166], [63, 148, 10], [122, 172, 133], [214, 88, 124], [106, 67, 210], [200, 117, 95], [67, 55, 166], [152, 153, 21], [255, 186, 253], [58, 237, 39], [42, 179, 53], [59, 203, 38], [85, 108, 3], [219, 199, 5], [222, 55, 209], [107, 77, 31], [254, 176, 10], [219, 64, 252], [173, 60, 119], [113, 8, 14], [41, 53, 210], [15, 100, 55], [238, 110, 196], [44, 154, 65], [49, 229, 253], [123, 187, 5], [235, 91, 52], [113, 170, 192], [219, 112, 176], [209, 228, 247], [204, 57, 198], [219, 26, 241], [181, 255, 248], [241, 176, 235], [192, 23, 84], [32, 79, 41], [115, 171, 255], [12, 184, 52], [36, 74, 181], [81, 141, 214], [16, 112, 14], [189, 190, 23], [243, 198, 162], [174, 9, 139], [236, 194, 114], [11, 249, 34], [56, 162, 220], [182, 109, 45], [177, 33, 207], [54, 46, 106], [6, 109, 151], [148, 24, 212], [73, 231, 64], [63, 254, 100], [150, 138, 105], [229, 121, 43], [182, 202, 175], [45, 133, 23], [79, 130, 79], [66, 238, 89], [140, 175, 136], [167, 125, 149], [247, 231, 66], [245, 209, 72], [21, 157, 152], [161, 224, 226], [241, 227, 148], [148, 253, 21], [10, 117, 136], [255, 221, 3], [53, 113, 68], [228, 243, 75], [102, 12, 14], [6, 23, 17], [61, 122, 243], [93, 231, 67], [238, 250, 50], [213, 92, 102], [57, 151, 96], [198, 10, 9], [193, 203, 203], [34, 229, 79], [40, 42, 66], [33, 129, 184], [67, 131, 9], [213, 35, 193], [59, 126, 204], [49, 152, 172], [182, 143, 39], [73, 79, 138], [201, 145, 87], [220, 19, 225], [227, 230, 55], [71, 50, 149], [39, 127, 153], [168, 54, 213], [182, 228, 229], [31, 120, 231], [90, 167, 139], [39, 44, 51], [182, 87, 48], [238, 188, 52], [109, 184, 218], [18, 156, 67], [221, 127, 217], [16, 160, 27], [89, 54, 178], [46, 182, 179], [209, 76, 145], [246, 132, 192], [187, 218, 196], [21, 150, 7], [193, 186, 243], [215, 162, 184], [64, 203, 169], [236, 134, 137], [7, 13, 226], [245, 106, 1], [50, 252, 46], [108, 215, 246], [58, 138, 241], [65, 58, 147], [43, 112, 231], [54, 121, 95], [14, 17, 46], [88, 93, 31], [65, 57, 222], [133, 226, 102], [230, 212, 144], [248, 248, 164], [128, 53, 180], [49, 36, 134], [233, 13, 30], [8, 118, 28], [11, 234, 224], [164, 77, 19], [250, 17, 73], [127, 231, 19], [124, 30, 111], [17, 176, 215], [181, 33, 251], [99, 169, 26], [112, 198, 20], [37, 110, 232], [230, 122, 98], [123, 96, 204], [39, 31, 237], [174, 71, 134], [30, 135, 67], [156, 17, 109], [34, 115, 114], [0, 204, 195], [66, 178, 239], [208, 120, 176], [130, 187, 215], [185, 238, 70], [175, 26, 156], [217, 81, 81], [57, 255, 206], [125, 228, 157], [179, 86, 204], [14, 208, 86], [169, 166, 11], [40, 213, 226], [126, 214, 105], [106, 80, 101], [4, 4, 15], [89, 121, 103], [96, 227, 88], [227, 217, 122], [104, 4, 127], [175, 116, 205], [198, 109, 79], [146, 150, 82], [157, 122, 194], [89, 149, 187], [59, 143, 62], [56, 168, 81], [2, 110, 154], [159, 170, 254], [28, 149, 35], [149, 118, 220], [18, 186, 32], [117, 246, 161], [208, 191, 15], [186, 201, 3], [139, 65, 21], [14, 245, 159], [252, 78, 203], [232, 194, 207], [215, 62, 192], [88, 98, 205], [41, 180, 121], [177, 137, 172], [145, 108, 233], [23, 47, 255], [189, 96, 223], [102, 111, 185], [42, 58, 119], [240, 171, 7], [99, 25, 197], [172, 150, 165], [44, 59, 175], [110, 244, 43], [119, 223, 171], [9, 197, 36], [72, 6, 41], [250, 42, 208], [157, 161, 199], [54, 177, 230], [21, 191, 97], [124, 222, 128], [224, 158, 70], [41, 89, 108], [253, 22, 138], [105, 76, 4], [69, 89, 91], [93, 185, 250], [154, 9, 77], [46, 171, 56], [166, 54, 32], [170, 86, 246], [152, 5, 245], [231, 189, 79], [230, 120, 194], [121, 176, 174], [190, 158, 19], [29, 235, 29], [84, 83, 36], [56, 182, 21], [87, 209, 218], [96, 123, 165], [134, 77, 22], [176, 131, 184], [227, 229, 144], [194, 158, 66], [253, 110, 235], [237, 7, 182], [85, 244, 16], [253, 12, 172], [35, 97, 160], [139, 177, 25], [77, 226, 38], [112, 155, 114], [82, 239, 115], [116, 72, 21], [148, 14, 239], [119, 54, 224], [70, 58, 28], [77, 126, 130], [57, 166, 248], [205, 163, 34], [40, 27, 115], [182, 136, 166], [40, 247, 150], [90, 182, 69], [16, 238, 108], [226, 189, 57], [124, 242, 230], [240, 41, 131], [18, 144, 19], [65, 254, 69], [85, 22, 194], [32, 29, 59], [63, 187, 107], [76, 180, 24], [16, 107, 203], [120, 93, 24], [242, 193, 121], [149, 3, 152], [46, 86, 93], [251, 172, 210], [147, 217, 99], [205, 232, 165], [194, 142, 99], [94, 236, 100], [251, 7, 7], [255, 203, 3], [196, 95, 77], [103, 36, 109], [134, 216, 70], [182, 119, 209], [168, 76, 214], [9, 190, 29], [90, 113, 149], [161, 230, 170], [233, 187, 15], [191, 205, 177], [11, 0, 208], [252, 182, 215], [167, 195, 112], [253, 245, 36], [225, 98, 220], [204, 238, 215], [36, 194, 106], [185, 40, 50], [145, 101, 1], [74, 205, 135], [214, 104, 236], [246, 123, 7], [67, 133, 2], [60, 169, 115], [85, 62, 89], [90, 65, 164], [85, 74, 139], [149, 73, 168], [197, 154, 248], [20, 230, 233], [218, 0, 36], [131, 70, 70], [214, 77, 139], [125, 180, 154], [205, 40, 115], [255, 169, 41], [200, 191, 98], [208, 163, 104], [134, 144, 154], [190, 162, 122], [183, 132, 207], [95, 137, 205], [72, 54, 201], [219, 103, 7], [171, 105, 41], [95, 227, 71], [74, 255, 146], [86, 181, 231], [156, 77, 75], [13, 136, 68], [53, 52, 53], [176, 53, 115], [163, 9, 70], [220, 160, 14], [251, 180, 224], [118, 175, 254], [206, 219, 64], [206, 151, 88], [212, 58, 110], [139, 164, 53], [215, 201, 177], [159, 218, 154], [37, 244, 218], [139, 210, 209], [28, 235, 252], [5, 144, 179], [78, 74, 200], [63, 206, 176], [236, 208, 216], [137, 140, 26], [231, 10, 116], [0, 197, 201], [244, 94, 102], [51, 4, 215], [144, 35, 191], [101, 96, 125], [11, 163, 168], [52, 146, 154], [72, 66, 238], [149, 153, 145], [91, 66, 222], [61, 100, 37], [217, 80, 3], [64, 115, 50], [207, 151, 220], [79, 10, 123], [189, 230, 216], [18, 85, 218], [143, 138, 12], [67, 200, 74], [17, 247, 154], [1, 209, 124], [225, 227, 178], [108, 87, 12], [20, 145, 18], [96, 189, 182], [22, 139, 123], [242, 81, 129], [176, 18, 144], [15, 166, 5], [193, 209, 194], [189, 115, 85], [215, 150, 196], [147, 252, 33], [214, 95, 15], [239, 135, 218], [209, 74, 22], [168, 232, 114], [95, 198, 93], [211, 40, 79], [33, 71, 73], [220, 188, 173], [92, 211, 215], [233, 17, 231], [60, 58, 13], [95, 219, 141], [219, 161, 228], [28, 241, 218], [127, 71, 74], [194, 241, 195], [42, 92, 239], [206, 79, 87], [12, 199, 51], [204, 236, 80], [50, 29, 252], [181, 23, 203], [136, 104, 10], [202, 200, 255], [193, 169, 194], [105, 229, 159], [242, 115, 102], [15, 0, 90], [26, 116, 67], [106, 151, 234], [37, 53, 95], [84, 173, 204], [76, 34, 179], [82, 126, 100], [191, 182, 40], [182, 49, 111], [105, 117, 194], [40, 10, 109], [144, 211, 153], [48, 120, 245], [60, 4, 130], [237, 108, 252], [246, 250, 90], [62, 128, 220], [210, 39, 237], [197, 3, 243], [24, 94, 238], [249, 146, 115], [169, 248, 177], [62, 6, 68], [246, 89, 115], [208, 194, 152], [136, 32, 105], [253, 133, 120], [213, 80, 30], [198, 48, 47], [5, 186, 251], [32, 157, 166], [214, 153, 27], [34, 241, 113], [132, 44, 168], [82, 170, 118], [255, 199, 131], [184, 89, 98], [189, 166, 24], [152, 1, 225], [163, 221, 83], [95, 184, 155], [25, 126, 133], [155, 116, 158], [211, 93, 198], [156, 201, 19], [34, 79, 176], [86, 238, 99], [147, 50, 169], [19, 94, 37], [43, 212, 72], [47, 36, 185], [236, 22, 139], [107, 16, 132], [187, 130, 103], [241, 170, 44], [61, 106, 238], [134, 137, 160], [85, 102, 102], [171, 81, 209], [30, 15, 35], [29, 196, 209], [79, 168, 181], [122, 187, 135], [228, 217, 86], [213, 114, 245], [48, 71, 217], [128, 65, 19], [154, 55, 144], [189, 70, 181], [135, 57, 188], [0, 69, 177], [150, 35, 170], [169, 206, 217], [233, 4, 250], [99, 197, 194], [39, 7, 137], [182, 158, 75], [140, 187, 104], [203, 40, 209], [253, 106, 9], [229, 207, 165], [228, 121, 112], [56, 105, 108], [181, 5, 74], [222, 18, 45], [126, 144, 253], [230, 225, 14], [63, 134, 87], [68, 249, 113], [22, 66, 102], [88, 50, 50], [219, 117, 132], [72, 104, 77], [118, 165, 229], [59, 28, 60], [18, 123, 167], [43, 66, 153], [77, 167, 97], [93, 41, 146], [204, 146, 117], [145, 208, 72], [53, 57, 205], [8, 214, 209], [72, 10, 26], [67, 93, 160], [154, 243, 153], [15, 97, 152], [136, 62, 227], [73, 89, 30], [218, 31, 67], [192, 0, 76], [206, 230, 20], [82, 142, 21], [53, 231, 234], [219, 233, 114], [161, 138, 162], [201, 131, 62], [141, 15, 92], [135, 106, 76], [95, 40, 32], [193, 250, 153], [134, 214, 147], [220, 191, 181], [253, 16, 235], [219, 70, 235], [16, 215, 34], [202, 79, 163], [114, 144, 80], [121, 51, 182], [138, 223, 205], [15, 20, 240], [121, 39, 172], [185, 238, 4], [78, 156, 153], [24, 165, 154], [2, 122, 206], [123, 14, 99], [154, 39, 204], [239, 19, 152], [175, 199, 148], [245, 193, 87], [107, 211, 110], [166, 32, 151], [203, 30, 124], [142, 247, 134], [55, 142, 114], [71, 57, 194], [44, 78, 104], [133, 189, 79], [194, 14, 206], [101, 204, 71], [161, 29, 42], [84, 202, 108], [107, 33, 110], [3, 67, 104], [28, 134, 74], [121, 253, 235], [51, 202, 96], [254, 31, 152], [5, 9, 214], [77, 135, 205], [203, 168, 249], [48, 200, 224], [160, 17, 226], [141, 172, 180], [104, 246, 130], [142, 113, 163], [29, 29, 184], [69, 236, 50], [43, 219, 49], [122, 65, 104], [29, 219, 10], [210, 103, 245], [61, 43, 68], [198, 248, 212], [224, 4, 67], [22, 179, 25], [186, 203, 120], [35, 116, 192], [236, 40, 141], [112, 141, 235], [39, 10, 62], [217, 60, 190], [169, 18, 147], [153, 80, 111], [100, 151, 41], [39, 42, 15], [79, 57, 48], [42, 190, 143], [179, 156, 192], [139, 245, 28], [178, 60, 159], [4, 60, 110], [90, 187, 186], [39, 239, 178], [66, 164, 79], [36, 11, 50], [241, 123, 6], [178, 47, 151], [240, 184, 23], [63, 36, 212], [17, 0, 146], [6, 16, 136], [95, 35, 15], [62, 200, 151], [130, 85, 79], [243, 199, 171], [87, 142, 242], [11, 201, 8], [76, 11, 147], [23, 131, 158], [214, 126, 255], [24, 253, 102], [112, 217, 218], [119, 6, 49], [197, 105, 70], [209, 217, 14], [129, 46, 248], [79, 36, 184], [16, 15, 77], [131, 170, 102], [0, 120, 225], [47, 102, 22], [132, 117, 159], [53, 94, 82], [40, 177, 238], [252, 151, 116], [9, 140, 93], [235, 38, 122], [95, 47, 143], [57, 11, 78], [151, 40, 247], [72, 48, 125], [225, 7, 237], [237, 17, 51], [105, 3, 201], [195, 13, 134], [78, 42, 85], [28, 63, 109], [2, 242, 150], [103, 21, 147], [100, 221, 145], [124, 24, 186], [38, 239, 36], [117, 126, 129], [253, 205, 47], [84, 205, 87], [117, 44, 0], [25, 68, 48], [199, 93, 6], [212, 27, 18], [228, 23, 2], [40, 102, 117], [154, 208, 23], [82, 115, 17], [63, 144, 180], [227, 8, 4], [159, 102, 228], [160, 95, 186], [230, 126, 86], [215, 91, 237], [251, 63, 209], [230, 143, 150], [66, 229, 200], [97, 50, 144], [188, 245, 62], [204, 127, 124], [40, 228, 80], [207, 129, 12], [185, 44, 141], [198, 23, 180], [230, 72, 59], [172, 0, 44], [248, 92, 129], [29, 79, 88], [158, 211, 136], [139, 79, 12], [27, 10, 163], [134, 136, 213], [105, 197, 243], [192, 99, 237], [133, 223, 244], [83, 202, 206], [34, 36, 142], [184, 32, 44], [141, 132, 45], [236, 19, 68], [102, 173, 179], [177, 164, 71], [237, 174, 110], [180, 74, 212], [3, 229, 226], [152, 175, 16], [135, 198, 203], [204, 16, 26], [208, 242, 89], [211, 219, 59], [234, 232, 225], [44, 165, 248], [115, 7, 250], [104, 5, 113], [28, 202, 149], [105, 35, 228], [77, 169, 159], [206, 2, 96], [244, 18, 200], [138, 51, 140], [118, 100, 55], [195, 108, 238], [67, 148, 105], [96, 217, 129], [60, 169, 99], [24, 75, 164], [53, 157, 103], [15, 132, 158], [198, 118, 109], [110, 220, 85], [252, 167, 116], [171, 247, 17], [189, 13, 54], [55, 10, 82], [69, 98, 30], [237, 62, 254], [252, 187, 46], [157, 88, 60], [148, 183, 96], [124, 204, 36], [38, 123, 22], [5, 254, 244], [24, 58, 106], [161, 134, 249], [194, 149, 230], [19, 32, 200], [223, 58, 139], [223, 34, 181], [28, 73, 71], [74, 46, 61], [94, 212, 184], [104, 15, 84], [178, 131, 62], [206, 49, 7], [159, 68, 213], [179, 7, 215], [53, 238, 8], [196, 124, 122], [60, 218, 46], [33, 67, 0], [57, 207, 253], [25, 244, 51], [120, 75, 228], [140, 251, 108], [60, 247, 168], [222, 73, 48], [37, 105, 75], [42, 140, 43], [159, 0, 122], [20, 208, 159], [196, 233, 237], [145, 175, 140], [191, 76, 101], [35, 195, 197], [18, 60, 98], [115, 163, 220], [90, 250, 60], [144, 2, 24], [131, 110, 63], [101, 35, 12], [154, 115, 248], [127, 182, 236], [231, 104, 196], [22, 47, 4], [254, 235, 122], [178, 218, 92], [160, 65, 51], [161, 30, 16], [9, 244, 223], [240, 134, 0], [247, 121, 87], [202, 254, 76], [20, 37, 172], [83, 63, 251], [245, 143, 22], [28, 181, 156], [36, 63, 228], [255, 171, 109], [215, 195, 138], [220, 61, 214], [120, 154, 190], [28, 129, 179], [134, 226, 130], [41, 127, 156], [93, 76, 192], [177, 86, 189], [84, 26, 153], [104, 35, 46], [49, 10, 59], [201, 160, 180], [209, 81, 86], [101, 202, 100], [8, 99, 152], [61, 143, 246], [66, 110, 183], [201, 105, 146], [222, 49, 201], [220, 9, 100], [115, 193, 30], [233, 136, 129], [110, 131, 208], [25, 58, 53], [200, 88, 95], [40, 79, 231], [239, 177, 68], [8, 247, 137], [38, 194, 14], [134, 42, 61], [112, 106, 75], [128, 113, 38], [135, 226, 31], [242, 194, 188], [211, 94, 106], [204, 230, 240], [34, 92, 79], [45, 203, 192], [84, 217, 197], [219, 74, 146], [85, 104, 12], [151, 3, 136], [241, 94, 29], [110, 205, 116], [219, 45, 135], [116, 247, 54], [131, 157, 157], [222, 230, 51], [175, 119, 61], [101, 164, 7], [209, 42, 99], [239, 53, 26], [17, 163, 184], [201, 14, 247], [47, 159, 130], [194, 123, 130], [16, 80, 235], [170, 12, 165], [77, 94, 88], [44, 178, 127], [86, 1, 139], [167, 252, 225], [119, 177, 224], [130, 42, 236], [26, 43, 188], [251, 155, 117], [192, 207, 37], [213, 100, 41], [101, 188, 152], [166, 40, 205], [223, 63, 18], [113, 43, 51], [115, 128, 166], [183, 92, 0], [160, 166, 93], [157, 124, 150], [198, 192, 74], [64, 167, 62], [49, 76, 241], [240, 62, 178], [65, 247, 103], [29, 84, 158], [66, 200, 238], [250, 15, 32], [20, 194, 210], [160, 235, 167], [158, 150, 102], [36, 98, 216], [130, 84, 196], [26, 165, 246], [58, 240, 212], [237, 117, 141], [71, 209, 184], [8, 251, 95], [64, 35, 213], [42, 33, 47], [145, 79, 117], [137, 65, 249], [8, 188, 58], [137, 209, 35], [82, 148, 113], [236, 247, 88], [139, 122, 158], [103, 188, 60], [31, 123, 221], [121, 110, 230], [243, 47, 156], [13, 32, 135], [47, 166, 225], [204, 207, 184], [120, 5, 249], [122, 111, 209], [223, 41, 60], [113, 166, 94], [54, 127, 90], [20, 254, 66], [225, 227, 190], [177, 83, 69], [212, 150, 3], [48, 212, 168], [114, 176, 61], [39, 74, 4], [34, 175, 120], [65, 117, 183], [42, 158, 95], [109, 39, 255], [177, 41, 175], [49, 6, 32], [111, 90, 231], [30, 201, 219], [203, 129, 233], [111, 117, 28], [31, 169, 156], [136, 246, 76], [14, 186, 161], [25, 247, 69], [157, 39, 25], [249, 103, 178], [12, 0, 216], [226, 93, 209], [6, 26, 135], [47, 180, 106], [27, 4, 206], [14, 44, 202], [160, 132, 135], [40, 199, 210], [94, 156, 219], [191, 201, 215], [186, 33, 214], [92, 141, 216], [142, 24, 207], [4, 117, 50], [89, 255, 83], [199, 139, 178], [77, 138, 77], [9, 14, 162], [65, 188, 251], [231, 62, 19], [15, 231, 110], [254, 162, 72], [235, 71, 119], [54, 1, 172], [108, 25, 60], [175, 88, 236], [146, 193, 205], [131, 231, 16], [236, 225, 240], [56, 118, 50], [48, 186, 107], [172, 26, 67], [210, 11, 30], [214, 71, 250], [49, 227, 193], [19, 218, 42], [166, 82, 172], [4, 128, 175], [119, 235, 26], [114, 89, 240], [243, 250, 218], [67, 58, 40], [65, 215, 20], [52, 60, 65], [142, 1, 11], [91, 28, 15], [221, 46, 110], [25, 49, 229], [87, 127, 209], [23, 248, 163], [184, 113, 19], [168, 73, 30], [104, 218, 111], [32, 126, 1], [243, 36, 161], [112, 199, 147], [89, 137, 239], [25, 150, 42], [76, 75, 173], [188, 149, 162], [24, 25, 120], [198, 233, 122], [88, 239, 241], [109, 13, 226], [183, 168, 83], [230, 153, 203], [60, 77, 72], [252, 105, 84], [187, 240, 107], [129, 203, 203], [9, 146, 96], [33, 23, 7], [57, 206, 151], [106, 252, 104], [34, 103, 118], [153, 244, 71], [203, 242, 120], [109, 181, 161], [113, 42, 104], [64, 104, 87], [93, 34, 149], [229, 127, 42], [88, 81, 238], [8, 246, 22], [110, 247, 156], [153, 176, 125], [20, 163, 224], [90, 81, 198], [128, 168, 44], [238, 169, 103], [112, 91, 63], [236, 210, 234], [15, 110, 249], [201, 38, 19], [231, 25, 22], [174, 74, 88], [162, 222, 23], [9, 50, 108], [147, 192, 156], [132, 64, 153], [85, 174, 188], [91, 89, 196], [122, 103, 43], [170, 73, 75], [87, 187, 153], [31, 26, 39], [211, 143, 160], [145, 189, 155], [242, 43, 215], [202, 69, 235], [30, 50, 242], [95, 174, 78], [18, 239, 209], [198, 199, 38], [57, 173, 12], [46, 124, 188], [201, 231, 226], [168, 187, 113], [129, 142, 115], [177, 100, 25], [184, 153, 86], [2, 150, 111], [131, 248, 165], [105, 159, 228], [103, 106, 128], [248, 103, 15], [186, 174, 136], [228, 4, 195], [232, 183, 132], [35, 219, 149], [185, 179, 84], [190, 72, 170], [232, 104, 181], [132, 122, 28], [135, 184, 194], [6, 187, 12], [150, 128, 187], [0, 153, 58], [106, 216, 85], [82, 180, 28], [46, 69, 215], [208, 185, 75], [13, 220, 241], [248, 101, 213], [161, 172, 16], [191, 79, 54], [64, 38, 99], [98, 31, 92], [58, 64, 208], [25, 181, 56], [197, 38, 155], [193, 137, 22], [200, 197, 198], [119, 195, 239], [121, 213, 129], [14, 160, 133], [149, 170, 114], [148, 10, 197], [97, 188, 248], [32, 186, 162], [12, 92, 69], [33, 93, 192], [154, 217, 92], [145, 50, 157], [210, 240, 84], [24, 161, 124], [216, 182, 102], [84, 161, 147], [248, 181, 226], [9, 171, 65], [229, 234, 176], [184, 15, 91], [70, 61, 32], [2, 139, 218], [173, 60, 165], [129, 60, 9], [146, 232, 91], [228, 121, 237], [190, 219, 102], [114, 163, 206], [151, 136, 37], [137, 89, 43], [28, 70, 105], [51, 254, 191], [44, 225, 228], [74, 240, 1], [61, 87, 27], [28, 80, 90], [23, 14, 197], [6, 125, 232], [94, 65, 231], [51, 117, 199], [92, 138, 137], [224, 164, 237], [136, 224, 127], [245, 151, 66], [97, 182, 183], [164, 196, 60], [47, 21, 70], [34, 138, 46], [251, 22, 219], [79, 150, 95], [27, 86, 77], [98, 193, 183], [181, 225, 41], [254, 2, 222], [21, 75, 123], [80, 79, 169], [107, 187, 198], [72, 208, 140], [222, 149, 125], [179, 233, 223], [152, 255, 170], [228, 219, 128], [163, 163, 222], [83, 152, 67], [166, 102, 56], [26, 129, 116], [117, 216, 70], [65, 15, 28], [1, 88, 71], [90, 100, 190], [248, 166, 157], [151, 156, 60], [142, 232, 178], [140, 216, 136], [89, 190, 152], [104, 80, 55], [39, 85, 181], [243, 180, 114], [26, 2, 198], [109, 69, 197], [38, 200, 98], [171, 185, 133], [123, 37, 201], [98, 43, 24], [102, 131, 223], [128, 234, 170], [142, 117, 114], [192, 141, 92], [198, 115, 22], [130, 49, 86], [41, 33, 91], [46, 170, 251], [131, 158, 38], [153, 229, 136], [23, 146, 44], [193, 208, 176], [210, 3, 38], [53, 198, 38], [183, 91, 101], [105, 135, 76], [205, 215, 248], [103, 230, 45], [166, 210, 187], [116, 245, 82], [121, 249, 38], [2, 224, 81], [238, 253, 175], [212, 26, 59], [195, 223, 201], [47, 233, 142], [58, 72, 192], [185, 160, 96], [112, 186, 181], [210, 183, 114], [109, 213, 193], [176, 118, 70], [56, 58, 134], [202, 38, 139], [218, 156, 134], [117, 201, 0], [226, 9, 21], [66, 253, 22], [17, 8, 228], [28, 69, 67], [100, 104, 86], [179, 130, 188], [29, 253, 225], [114, 27, 162], [52, 233, 10], [222, 102, 219], [111, 155, 161], [248, 51, 201], [79, 130, 57], [230, 159, 188], [47, 208, 162], [33, 180, 51], [197, 81, 92], [208, 46, 37], [186, 49, 211], [99, 190, 229], [223, 253, 62], [197, 240, 31], [211, 151, 49], [180, 59, 129], [80, 80, 255], [100, 8, 116], [136, 123, 185], [222, 163, 62], [138, 67, 117], [56, 104, 236], [216, 118, 128], [7, 75, 52], [133, 148, 2], [160, 97, 251], [248, 161, 173], [89, 231, 203], [73, 111, 118], [213, 198, 56], [113, 22, 188], [211, 74, 168], [157, 106, 170], [209, 150, 226], [14, 224, 82], [73, 210, 168], [114, 250, 29], [89, 19, 216], [91, 122, 132], [188, 93, 130], [45, 158, 197], [124, 13, 151], [175, 159, 136], [158, 101, 203], [101, 227, 67], [253, 140, 234], [115, 77, 109], [63, 182, 101], [239, 92, 20], [23, 1, 92], [252, 52, 74], [76, 78, 215], [30, 139, 84], [36, 225, 71], [193, 222, 32], [46, 238, 129], [130, 0, 93], [158, 104, 85], [247, 106, 175], [41, 105, 209], [85, 199, 132], [41, 121, 166], [8, 229, 115], [41, 40, 234], [42, 227, 122], [222, 104, 65], [148, 73, 63], [113, 21, 126], [3, 66, 63], [58, 20, 67], [110, 234, 104], [134, 166, 198], [47, 11, 199], [199, 96, 134], [101, 75, 189], [161, 239, 40], [168, 230, 101], [15, 163, 124], [166, 8, 152], [166, 52, 173], [123, 205, 189], [243, 46, 91], [189, 83, 147], [67, 41, 18], [66, 181, 67], [0, 168, 215], [22, 207, 85], [133, 140, 15], [87, 33, 112], [195, 18, 88], [252, 171, 91], [104, 113, 215], [233, 4, 88], [240, 193, 187], [61, 22, 67], [127, 222, 242], [116, 184, 220], [255, 73, 112], [27, 23, 189], [16, 17, 88], [254, 63, 132], [153, 102, 25], [150, 20, 215], [98, 81, 88], [226, 214, 126], [247, 208, 150], [238, 53, 143], [184, 182, 13], [131, 206, 71], [95, 87, 120], [250, 236, 193], [147, 151, 240], [219, 120, 164], [238, 97, 56], [143, 133, 36], [75, 192, 211], [143, 22, 0], [91, 47, 251], [226, 145, 122], [5, 122, 116], [18, 101, 186], [11, 104, 174], [240, 44, 129], [21, 75, 247], [199, 56, 46], [51, 229, 140], [124, 141, 184], [86, 243, 181], [238, 22, 216], [0, 74, 152], [231, 58, 239], [179, 46, 128], [84, 203, 119], [120, 92, 243], [22, 16, 66], [34, 244, 195], [194, 20, 55], [139, 131, 71], [200, 52, 104], [22, 79, 85], [151, 235, 134], [14, 68, 16], [84, 222, 213], [52, 106, 75], [169, 165, 52], [117, 18, 63], [23, 241, 176], [113, 128, 192], [128, 212, 109], [147, 106, 111], [19, 123, 206], [86, 36, 245], [221, 138, 187], [74, 155, 205], [188, 158, 124], [108, 245, 121], [247, 223, 2], [20, 59, 69], [247, 117, 120], [120, 89, 184], [21, 196, 225], [66, 17, 187], [249, 169, 156], [193, 244, 145], [252, 170, 3], [94, 102, 126], [249, 64, 113], [234, 82, 84], [124, 54, 81], [212, 84, 6], [108, 141, 253], [134, 180, 138], [227, 122, 245], [153, 251, 113], [128, 143, 130], [148, 157, 81], [124, 254, 156], [87, 248, 208], [224, 36, 215], [171, 70, 114], [153, 105, 207], [218, 179, 175], [51, 69, 15], [136, 207, 122], [193, 47, 45], [4, 242, 209], [52, 45, 144], [11, 17, 58], [100, 133, 99], [245, 76, 212], [99, 172, 172], [135, 96, 58], [196, 127, 91], [247, 94, 157], [244, 224, 89], [71, 118, 247], [140, 74, 40], [119, 184, 211], [169, 207, 20], [174, 34, 237], [155, 213, 38], [173, 251, 114], [98, 120, 222], [190, 241, 121], [158, 114, 11], [115, 253, 54], [255, 191, 234], [182, 117, 111], [10, 107, 88], [114, 145, 22], [162, 46, 159], [23, 127, 154], [144, 6, 37], [32, 42, 59], [136, 80, 50], [114, 138, 231], [39, 183, 183], [100, 153, 190], [20, 66, 43], [37, 105, 230], [237, 97, 201], [51, 41, 214], [45, 57, 136], [108, 158, 178], [111, 11, 135], [121, 116, 114], [10, 209, 100], [185, 59, 64], [151, 161, 42], [9, 232, 170], [114, 210, 231], [181, 169, 204], [224, 235, 187], [180, 32, 202], [67, 250, 119], [37, 48, 19], [17, 20, 153], [183, 99, 18], [95, 117, 68], [108, 104, 173], [175, 219, 77], [199, 37, 123], [97, 105, 25], [165, 33, 9], [105, 97, 160], [113, 240, 184], [125, 138, 251], [189, 224, 138], [166, 124, 114], [236, 74, 162], [12, 126, 88], [51, 175, 186], [168, 81, 65], [4, 24, 100], [224, 114, 205], [166, 203, 184], [188, 71, 153], [254, 157, 93], [144, 142, 225], [167, 211, 150], [144, 120, 70], [131, 104, 24], [41, 231, 159], [193, 122, 179], [33, 11, 65], [102, 35, 10], [170, 234, 187], [124, 20, 233], [227, 4, 159], [2, 39, 77], [233, 184, 144], [115, 85, 73], [0, 61, 165], [7, 114, 225], [39, 197, 58], [27, 94, 36], [75, 54, 181], [79, 76, 52], [247, 0, 23], [58, 68, 198], [55, 53, 79], [126, 120, 66], [174, 144, 153], [251, 128, 105], [7, 104, 123], [121, 42, 20], [179, 26, 130], [186, 166, 13], [113, 123, 38], [29, 6, 255], [31, 97, 63], [171, 195, 72], [220, 20, 206], [172, 239, 147], [210, 197, 214], [44, 67, 241], [219, 145, 0], [134, 179, 104], [221, 28, 126], [137, 225, 157], [250, 103, 250], [46, 227, 43], [148, 128, 133], [207, 235, 154], [2, 217, 56], [222, 187, 136], [232, 193, 115], [54, 122, 216], [146, 156, 232], [234, 8, 145], [138, 13, 0], [209, 211, 195], [195, 131, 64], [243, 127, 74], [45, 6, 7], [224, 150, 44], [41, 74, 26], [34, 136, 43], [186, 169, 99], [158, 150, 135], [211, 18, 177], [92, 65, 232], [120, 111, 59], [4, 248, 204], [105, 60, 133], [73, 246, 82], [35, 54, 86], [88, 37, 94], [73, 182, 104], [234, 41, 9], [23, 216, 37], [45, 106, 163], [19, 152, 104], [236, 183, 226], [226, 17, 99], [222, 8, 234], [206, 66, 135], [95, 30, 189], [18, 215, 55], [12, 12, 158], [45, 247, 155], [228, 159, 159], [193, 213, 176], [148, 248, 79], [164, 38, 11], [218, 243, 213], [47, 148, 193], [187, 91, 119], [215, 166, 118], [238, 59, 161], [214, 155, 213], [111, 28, 33], [5, 60, 208], [144, 0, 15], [45, 174, 20], [211, 101, 157], [244, 187, 204], [183, 184, 196], [143, 147, 50], [33, 221, 252], [5, 21, 172], [165, 67, 16], [211, 225, 253], [87, 41, 235], [27, 62, 197], [165, 133, 205], [88, 168, 9], [223, 8, 158], [82, 93, 222], [133, 174, 34], [163, 10, 216], [175, 210, 78], [47, 118, 30], [241, 151, 132], [64, 240, 229], [141, 201, 194], [192, 223, 146], [248, 159, 14], [184, 144, 219], [91, 143, 17], [106, 72, 244], [91, 98, 226], [225, 75, 122], [110, 149, 101], [190, 110, 21], [61, 232, 103], [162, 200, 162], [148, 233, 22], [25, 178, 14], [52, 170, 134], [238, 239, 100], [232, 94, 34], [160, 118, 202], [191, 233, 13], [115, 97, 220], [100, 140, 255], [237, 207, 197], [81, 153, 216], [206, 125, 141], [219, 159, 149], [173, 169, 179], [110, 192, 144], [129, 220, 161], [95, 253, 14], [23, 172, 7], [197, 64, 223], [130, 177, 77], [84, 180, 0], [40, 253, 64], [229, 175, 121], [176, 186, 188], [18, 164, 108], [5, 243, 213], [160, 254, 84], [76, 227, 140], [14, 124, 158], [122, 107, 5], [213, 87, 86], [204, 10, 204], [229, 51, 48], [94, 83, 178], [66, 167, 22], [209, 254, 106], [12, 190, 219], [109, 168, 4], [152, 216, 47], [30, 124, 65], [13, 101, 102], [139, 197, 241], [112, 104, 244], [167, 110, 51], [154, 225, 179], [186, 96, 69], [95, 96, 122], [114, 218, 118], [82, 50, 184], [55, 176, 208], [156, 4, 33], [232, 83, 15], [184, 210, 166], [247, 193, 50], [228, 226, 70], [153, 228, 202], [191, 202, 97], [178, 2, 171], [41, 136, 143], [233, 139, 171], [157, 46, 70], [148, 86, 125], [37, 156, 110], [9, 74, 52], [161, 57, 161], [197, 9, 76], [172, 103, 218], [100, 133, 135], [148, 82, 6], [147, 12, 240], [12, 219, 120], [209, 101, 204], [162, 187, 130], [236, 198, 243], [154, 144, 89], [88, 222, 236], [253, 105, 138], [137, 94, 225], [40, 162, 9], [37, 22, 251], [115, 150, 89], [26, 104, 193], [68, 192, 112], [124, 112, 74], [236, 132, 79], [130, 236, 72], [241, 80, 46], [225, 168, 23], [58, 167, 84], [220, 149, 156], [7, 58, 124], [238, 97, 130], [225, 120, 196], [84, 69, 114], [137, 160, 180], [94, 168, 89], [162, 168, 119], [114, 103, 64], [201, 243, 147], [61, 147, 0], [10, 24, 87], [224, 198, 240], [153, 61, 208], [218, 113, 156], [13, 86, 19], [176, 222, 208], [197, 249, 160], [76, 89, 236], [69, 78, 195], [227, 73, 27], [10, 250, 147], [159, 2, 72], [72, 204, 169], [55, 218, 252], [48, 60, 52], [175, 160, 79], [72, 123, 53], [175, 175, 203], [103, 191, 130], [143, 204, 8], [107, 46, 117], [222, 255, 194], [244, 135, 192], [25, 148, 95], [122, 192, 240], [57, 247, 143], [214, 57, 111], [222, 127, 233], [219, 251, 248], [146, 160, 8], [163, 198, 45], [131, 176, 13], [152, 154, 120], [132, 36, 226], [59, 255, 98], [223, 184, 166], [155, 46, 85], [155, 37, 16], [179, 219, 212], [169, 95, 59], [88, 252, 134], [248, 63, 32], [85, 227, 2], [156, 221, 9], [245, 130, 67], [252, 61, 61], [17, 166, 112], [60, 53, 83], [216, 196, 73], [154, 163, 177], [117, 48, 208], [183, 212, 181], [101, 130, 16], [1, 150, 126], [158, 109, 83], [161, 248, 71], [96, 34, 247], [14, 18, 23], [187, 240, 202], [224, 109, 129], [198, 183, 103], [171, 143, 114], [198, 51, 187], [63, 218, 225], [226, 211, 58], [250, 64, 171], [129, 221, 124], [26, 201, 87], [235, 72, 158], [113, 200, 67], [174, 71, 212], [246, 227, 193], [46, 18, 51], [63, 141, 237], [235, 20, 144], [30, 226, 183], [164, 249, 112], [102, 179, 225], [194, 89, 56], [46, 91, 149], [134, 128, 106], [36, 156, 54], [85, 143, 152], [123, 137, 239], [167, 13, 168], [237, 85, 205], [91, 111, 89], [144, 182, 3], [168, 255, 23], [135, 222, 15], [221, 109, 2], [238, 105, 117], [8, 7, 52], [184, 65, 17], [218, 4, 164], [164, 5, 75], [10, 131, 17], [67, 47, 172], [230, 174, 196], [168, 15, 185], [103, 157, 235], [204, 39, 134], [193, 62, 163], [85, 155, 246], [69, 30, 12], [86, 248, 149], [46, 186, 19], [184, 250, 47], [64, 113, 30], [206, 232, 210], [166, 126, 147], [127, 171, 223], [188, 53, 218], [95, 25, 155], [253, 231, 79], [23, 6, 44], [177, 111, 147], [217, 65, 230], [175, 253, 177], [207, 97, 109], [99, 245, 8], [178, 40, 189], [243, 191, 138], [61, 228, 92], [145, 224, 89], [252, 78, 254], [199, 44, 182], [198, 210, 120], [233, 76, 157], [249, 125, 45], [62, 14, 138], [242, 177, 83], [185, 243, 90], [234, 212, 44], [100, 224, 81], [18, 159, 4], [72, 1, 78], [254, 66, 103], [168, 62, 94], [129, 220, 48], [157, 153, 194], [42, 35, 173], [38, 96, 10], [108, 37, 106], [60, 218, 3], [180, 111, 204], [13, 162, 114], [162, 83, 5], [192, 228, 119], [235, 128, 15], [224, 164, 114], [124, 203, 141], [219, 155, 187], [241, 139, 13], [60, 160, 198], [131, 184, 116], [195, 56, 103], [247, 200, 102], [17, 146, 48], [211, 216, 120], [237, 14, 30], [3, 195, 85], [45, 225, 89], [221, 199, 185], [54, 254, 82], [34, 103, 46], [101, 148, 116], [86, 9, 16], [233, 92, 68], [103, 201, 181], [210, 65, 4], [85, 233, 178], [239, 187, 123], [57, 1, 223], [75, 58, 54], [186, 211, 210], [114, 96, 245], [174, 208, 139], [19, 176, 201], [36, 126, 64], [156, 109, 98], [67, 130, 44], [49, 238, 106], [192, 215, 157], [129, 113, 232], [65, 196, 9], [228, 118, 122], [50, 84, 4], [122, 211, 13], [89, 56, 111], [113, 105, 239], [79, 71, 207], [156, 125, 152], [90, 60, 127], [60, 49, 152], [221, 42, 81], [125, 211, 6], [71, 44, 70], [235, 9, 157], [51, 59, 152], [128, 108, 169], [130, 240, 6], [207, 123, 12], [29, 98, 136], [166, 247, 225], [2, 103, 108], [121, 234, 149], [65, 41, 137], [72, 107, 30], [14, 149, 123], [186, 213, 81], [221, 160, 62], [24, 91, 176], [93, 109, 133], [54, 33, 222], [4, 51, 73], [18, 213, 34], [9, 228, 64], [46, 221, 25], [97, 100, 62], [56, 226, 14], [80, 156, 196], [246, 106, 100], [253, 248, 101], [77, 4, 103], [67, 98, 169], [238, 180, 213], [18, 5, 63], [196, 238, 78], [224, 221, 70], [157, 115, 163], [191, 166, 134], [138, 184, 72], [38, 156, 75], [4, 213, 136], [187, 211, 66], [166, 182, 249], [55, 161, 23], [1, 21, 166], [125, 114, 215], [134, 156, 9], [81, 40, 210], [159, 229, 151], [185, 255, 249], [43, 165, 165], [124, 3, 119], [40, 128, 205], [133, 119, 246], [175, 255, 17], [253, 191, 10], [93, 16, 198], [12, 139, 2], [218, 16, 193], [247, 178, 23], [88, 252, 207], [148, 235, 179], [145, 227, 151], [187, 165, 19], [6, 215, 212], [99, 127, 84], [125, 245, 157], [55, 60, 158], [221, 251, 223], [71, 142, 156], [99, 161, 186], [75, 169, 145], [64, 7, 184], [8, 86, 2], [230, 215, 28], [46, 157, 144], [170, 221, 214], [69, 158, 211], [61, 114, 148], [250, 46, 211], [211, 234, 187], [192, 1, 35], [16, 134, 6], [18, 245, 156], [22, 77, 92], [138, 25, 210], [242, 68, 190], [99, 80, 116], [242, 22, 66], [185, 120, 230], [122, 84, 49], [238, 51, 100], [35, 20, 61], [145, 102, 106], [134, 4, 125], [200, 131, 193], [28, 235, 208], [2, 169, 112], [38, 152, 19], [134, 93, 147], [151, 48, 112], [34, 146, 43], [36, 212, 228], [95, 47, 144], [201, 181, 48], [161, 242, 100], [142, 255, 49], [5, 18, 60], [229, 24, 251], [251, 45, 92], [164, 75, 29], [0, 122, 214], [114, 73, 36], [81, 107, 165], [110, 194, 85], [102, 248, 227], [137, 78, 174], [197, 134, 142], [120, 241, 97], [246, 20, 104], [173, 196, 83], [156, 18, 94], [193, 43, 93], [119, 185, 205], [9, 63, 196], [198, 213, 126], [74, 171, 41], [175, 77, 126], [75, 231, 231], [28, 103, 190], [165, 6, 4], [197, 242, 10], [202, 27, 163], [201, 75, 214], [233, 85, 230], [153, 24, 47], [171, 80, 152], [109, 236, 240], [52, 158, 181], [3, 237, 212], [155, 11, 12], [7, 237, 186], [235, 78, 192], [58, 172, 26], [235, 237, 170], [152, 19, 175], [115, 36, 118], [194, 179, 157], [228, 8, 69], [115, 170, 111], [243, 140, 99], [133, 75, 127], [221, 237, 50], [21, 220, 222], [67, 116, 126], [253, 249, 19], [46, 159, 122], [83, 158, 181], [215, 61, 197], [200, 253, 3], [99, 50, 57], [17, 8, 217], [56, 205, 132], [19, 147, 4], [148, 171, 75], [248, 208, 195], [46, 124, 74], [122, 80, 186], [103, 47, 71], [81, 103, 102], [187, 76, 25], [172, 76, 184], [191, 35, 184], [79, 44, 194], [228, 41, 73], [208, 147, 23], [47, 205, 101], [112, 67, 15], [211, 113, 99], [74, 232, 204], [229, 92, 65], [52, 231, 179], [144, 228, 202], [168, 104, 54], [100, 207, 229], [42, 88, 60], [9, 134, 35], [197, 75, 74], [173, 79, 98], [210, 182, 81], [224, 250, 119], [60, 197, 73], [30, 245, 29], [127, 153, 248], [203, 196, 181], [74, 81, 163], [215, 3, 57], [167, 197, 152], [36, 78, 50], [179, 186, 99], [189, 161, 53], [51, 171, 30], [188, 71, 230], [200, 205, 208], [111, 56, 17], [73, 174, 32], [126, 181, 32], [14, 172, 20], [192, 236, 229], [91, 3, 231], [124, 2, 24], [213, 226, 228], [74, 193, 83], [234, 23, 45], [146, 102, 138], [75, 143, 78], [43, 96, 114], [115, 20, 185], [227, 159, 159], [91, 173, 237], [170, 117, 100], [61, 171, 91], [66, 125, 181], [43, 243, 35], [161, 84, 177], [115, 50, 250], [69, 119, 197], [208, 25, 129], [30, 234, 32], [102, 188, 166], [236, 56, 75], [209, 140, 22], [132, 165, 239], [34, 240, 126], [70, 163, 117], [208, 241, 16], [18, 22, 61], [165, 43, 22], [33, 191, 222], [59, 180, 222], [9, 187, 112], [38, 221, 9], [227, 230, 249], [59, 81, 94], [78, 169, 225], [104, 229, 101], [155, 216, 17], [192, 244, 106], [215, 106, 25], [99, 141, 129], [244, 188, 22], [156, 97, 243], [228, 106, 46], [114, 48, 54], [132, 44, 33], [206, 2, 173], [201, 163, 187], [73, 215, 214], [168, 145, 212], [247, 28, 239], [237, 190, 224], [45, 18, 197], [199, 154, 21], [222, 240, 105], [39, 88, 172], [253, 175, 86], [25, 94, 24], [208, 18, 126], [31, 41, 41], [200, 5, 227], [92, 91, 199], [223, 145, 53], [26, 220, 109], [250, 53, 169], [90, 126, 134], [237, 159, 100], [45, 40, 228], [45, 182, 145], [155, 113, 252], [151, 130, 119], [11, 19, 219], [35, 112, 61], [88, 47, 25], [115, 191, 91], [246, 183, 203], [224, 211, 62], [99, 91, 126], [125, 118, 55], [191, 160, 68], [128, 26, 113], [11, 15, 18], [147, 22, 221], [210, 152, 74], [57, 104, 129], [194, 88, 185], [65, 184, 209], [8, 130, 53], [183, 154, 120], [75, 223, 8], [69, 176, 234], [15, 48, 223], [132, 161, 207], [218, 65, 18], [194, 60, 12], [11, 226, 142], [96, 127, 71], [9, 62, 39], [240, 49, 247], [123, 134, 56], [6, 172, 88], [76, 230, 37], [117, 72, 249], [26, 237, 230], [95, 186, 72], [225, 236, 124], [207, 72, 155], [239, 113, 81], [190, 34, 230], [115, 14, 220], [136, 132, 21], [35, 216, 213], [54, 16, 98], [42, 239, 107], [49, 255, 177], [27, 101, 62], [53, 41, 16], [40, 136, 45], [235, 165, 240], [95, 143, 81], [232, 221, 16], [37, 233, 204], [113, 242, 18], [104, 27, 87], [201, 125, 147], [251, 162, 207], [77, 105, 107], [119, 197, 18], [194, 120, 56], [215, 201, 196], [140, 0, 132], [238, 87, 70], [36, 119, 127], [254, 126, 194], [24, 99, 157], [188, 1, 12], [63, 82, 101], [18, 253, 63], [53, 84, 144], [195, 107, 252], [166, 13, 106], [115, 250, 2], [158, 141, 198], [45, 5, 27], [198, 135, 149], [183, 172, 90], [185, 27, 57], [144, 81, 151], [55, 79, 19], [7, 247, 6], [54, 231, 168], [141, 175, 230], [188, 30, 127], [127, 124, 192], [138, 87, 0], [1, 98, 241], [227, 189, 69], [56, 237, 178], [42, 252, 52], [98, 104, 75], [68, 216, 195], [160, 108, 33], [174, 48, 133], [80, 69, 201], [225, 224, 139], [201, 215, 147], [141, 149, 33], [4, 101, 110], [224, 35, 86], [75, 163, 130], [17, 110, 198], [74, 87, 138], [183, 179, 160], [142, 4, 167], [25, 142, 213], [58, 84, 187], [51, 90, 217], [50, 77, 30], [45, 17, 245], [144, 44, 192], [124, 96, 68], [239, 179, 186], [113, 51, 125], [133, 130, 159], [94, 113, 154], [55, 246, 108], [146, 159, 9], [46, 74, 181], [164, 24, 89], [86, 114, 130], [217, 222, 210], [216, 227, 66], [133, 228, 129], [212, 10, 213], [88, 70, 253], [247, 208, 29], [217, 88, 32], [144, 21, 151], [62, 235, 55], [121, 193, 111], [16, 114, 51], [176, 70, 49], [52, 44, 92], [87, 154, 245], [119, 62, 66], [84, 203, 68], [199, 77, 95], [198, 46, 187], [136, 59, 128], [140, 120, 58], [223, 89, 189], [167, 61, 10], [20, 86, 40], [144, 239, 88], [15, 93, 5], [65, 118, 187], [20, 214, 7], [181, 30, 120], [107, 114, 122], [104, 201, 166], [11, 251, 97], [129, 170, 83], [79, 125, 80], [61, 66, 125], [198, 23, 185], [179, 236, 215], [69, 193, 77], [215, 209, 71], [128, 65, 254], [221, 18, 55], [186, 109, 227], [71, 48, 99], [112, 40, 192], [247, 150, 241], [108, 118, 164], [213, 103, 243], [106, 240, 251], [97, 149, 84], [106, 130, 84], [190, 177, 87], [239, 110, 213], [89, 237, 152], [241, 215, 124], [149, 65, 230], [159, 48, 50], [227, 195, 183], [124, 47, 62], [53, 59, 194], [226, 40, 235], [244, 198, 25], [235, 191, 71], [98, 127, 132], [51, 170, 140], [88, 12, 9], [88, 15, 65], [91, 232, 18], [153, 46, 126], [94, 107, 179], [44, 19, 65], [95, 222, 68], [164, 190, 247], [142, 104, 227], [44, 240, 132], [177, 85, 120], [128, 129, 166], [212, 23, 186], [130, 102, 131], [184, 228, 218], [20, 94, 145], [47, 42, 158], [255, 165, 25], [144, 59, 212], [6, 141, 27], [79, 242, 183], [229, 107, 70], [230, 191, 115], [243, 249, 186], [44, 111, 84], [54, 230, 219], [62, 30, 170], [62, 109, 86], [98, 249, 166], [18, 85, 17], [6, 97, 4], [147, 33, 84], [165, 163, 167], [36, 156, 79], [27, 245, 226], [175, 86, 75], [132, 227, 212], [168, 104, 143], [144, 226, 136], [205, 19, 11], [198, 166, 177], [132, 169, 158], [212, 179, 82], [130, 250, 77], [122, 185, 248], [173, 206, 222], [159, 53, 24], [17, 237, 227], [160, 144, 179], [77, 155, 102], [248, 12, 232], [131, 84, 46], [22, 203, 229], [137, 72, 148], [228, 135, 53], [99, 187, 241], [192, 173, 8], [106, 161, 199], [141, 67, 109], [254, 221, 192], [173, 201, 81], [58, 211, 239], [14, 5, 188], [1, 146, 150], [16, 140, 179], [121, 94, 115], [84, 108, 249], [49, 235, 164], [101, 78, 114], [202, 184, 215], [66, 223, 189], [122, 159, 227], [201, 86, 216], [4, 170, 216], [232, 68, 204], [189, 248, 139], [79, 131, 50], [33, 120, 154], [186, 80, 175], [92, 249, 170], [117, 176, 4], [41, 79, 92], [108, 125, 241], [65, 208, 20], [213, 21, 32], [73, 57, 66], [213, 39, 87], [191, 110, 36], [143, 126, 217], [74, 34, 202], [65, 191, 114], [201, 200, 152], [226, 236, 152], [253, 128, 215], [241, 76, 139], [142, 53, 2], [237, 58, 242], [145, 117, 52], [35, 143, 183], [98, 230, 28], [0, 219, 226], [206, 164, 63], [8, 172, 199], [47, 185, 119], [68, 141, 184], [170, 209, 109], [77, 106, 140], [151, 238, 76], [247, 70, 90], [27, 3, 190], [209, 96, 216], [250, 131, 9], [125, 204, 230], [145, 233, 29], [50, 44, 114], [175, 0, 186], [15, 45, 21], [186, 156, 202], [60, 97, 225], [10, 191, 173], [151, 127, 152], [227, 85, 184], [19, 240, 143], [168, 24, 166], [34, 146, 130], [227, 45, 151], [119, 224, 251], [216, 72, 138], [159, 173, 178], [88, 255, 247], [92, 183, 209], [143, 7, 29], [167, 149, 185], [67, 180, 247], [65, 2, 255], [28, 56, 28], [18, 233, 156], [65, 70, 93], [73, 249, 24], [231, 89, 85], [51, 151, 82], [246, 75, 28], [125, 22, 100], [114, 48, 152], [23, 172, 119], [89, 212, 242], [181, 242, 245], [103, 3, 246], [224, 193, 67], [222, 129, 3], [232, 243, 167], [212, 181, 53], [85, 155, 28], [110, 248, 176], [103, 236, 255], [87, 106, 255], [59, 245, 45], [140, 157, 18], [92, 227, 149], [108, 19, 176], [32, 122, 253], [142, 7, 210], [33, 198, 204], [117, 230, 105], [177, 125, 214], [166, 112, 132], [207, 70, 153], [57, 92, 33], [96, 191, 156], [157, 208, 198], [114, 225, 139], [175, 161, 25], [111, 235, 14], [219, 252, 46], [47, 249, 73], [86, 229, 91], [174, 27, 218], [178, 175, 203], [70, 49, 62], [250, 170, 150], [224, 254, 232], [157, 186, 245], [102, 155, 197], [213, 21, 127], [63, 255, 76], [247, 241, 134], [5, 156, 81], [138, 152, 161], [39, 168, 253], [9, 218, 61], [60, 130, 138], [133, 223, 34], [99, 95, 207], [16, 179, 121], [58, 163, 76], [37, 215, 64], [76, 96, 64], [61, 163, 242], [184, 219, 39], [4, 103, 71], [215, 137, 86], [222, 172, 213], [71, 34, 100], [94, 86, 166], [86, 155, 240], [238, 46, 57], [132, 252, 113], [119, 141, 120], [18, 187, 42], [170, 127, 201], [117, 166, 253], [235, 93, 157], [194, 107, 8], [113, 251, 193], [23, 9, 39], [24, 118, 27], [3, 48, 65], [23, 165, 63], [75, 122, 3], [125, 34, 251], [30, 100, 212], [253, 52, 98], [171, 121, 252], [204, 233, 250], [74, 212, 122], [30, 254, 207], [158, 239, 174], [148, 204, 2], [7, 42, 119], [63, 99, 185], [227, 78, 18], [140, 66, 140], [40, 51, 132], [4, 221, 231], [252, 128, 149], [152, 41, 41], [63, 32, 178], [203, 129, 128], [142, 57, 83], [81, 251, 170], [126, 76, 80], [208, 163, 36], [213, 223, 255], [62, 207, 132], [29, 253, 228], [205, 85, 194], [62, 45, 190], [0, 11, 30], [247, 139, 189], [224, 162, 161], [232, 163, 28], [228, 191, 84], [5, 42, 163], [230, 231, 49], [4, 116, 213], [210, 96, 180], [23, 73, 159], [216, 130, 243], [237, 79, 129], [71, 229, 157], [248, 136, 18], [105, 32, 156], [159, 127, 27], [228, 73, 154], [68, 241, 187], [211, 144, 138], [80, 44, 74], [159, 113, 92], [201, 196, 221], [134, 253, 139], [167, 165, 34], [3, 33, 179], [207, 113, 150], [209, 14, 99], [7, 130, 95], [140, 247, 16], [53, 138, 73], [192, 232, 82], [77, 212, 107], [191, 120, 29], [161, 54, 100], [200, 138, 88], [91, 67, 251], [81, 194, 201], [136, 191, 47], [67, 96, 44], [197, 114, 186], [100, 68, 174], [188, 104, 178], [117, 246, 241], [106, 151, 218], [61, 27, 154], [107, 234, 137], [26, 71, 230], [4, 164, 238], [254, 23, 202], [91, 45, 200], [179, 253, 231], [119, 232, 165], [145, 218, 201], [86, 253, 66], [183, 238, 169], [227, 234, 189], [114, 71, 53], [222, 79, 98], [168, 123, 152], [212, 150, 245], [1, 23, 67], [213, 228, 244], [59, 115, 78], [125, 166, 231], [117, 82, 143], [230, 7, 212], [157, 169, 195], [145, 22, 117], [105, 176, 120], [75, 154, 16], [56, 62, 66], [13, 146, 99], [11, 33, 73], [36, 206, 235], [253, 11, 4], [189, 49, 207], [4, 73, 98], [5, 42, 210], [247, 6, 245], [110, 201, 186], [98, 74, 21], [56, 90, 172], [79, 154, 220], [37, 218, 241], [181, 139, 125], [69, 123, 33], [153, 16, 39], [135, 30, 50], [62, 191, 31], [35, 41, 120], [135, 40, 176], [147, 9, 38], [177, 101, 247], [63, 91, 108], [59, 177, 235], [186, 131, 43], [84, 31, 118], [237, 225, 167], [151, 78, 94], [38, 235, 180], [250, 0, 36], [245, 84, 72], [69, 224, 213], [178, 206, 174], [138, 54, 3], [212, 248, 172], [155, 49, 167], [25, 123, 12], [226, 38, 157], [152, 245, 121], [76, 26, 47], [22, 191, 200], [79, 0, 242], [151, 205, 190], [176, 32, 100], [164, 175, 158], [130, 160, 147], [45, 177, 116], [109, 22, 252], [82, 205, 121], [225, 39, 190], [84, 146, 14], [34, 128, 85], [125, 216, 176], [185, 70, 144], [50, 45, 85], [128, 133, 15], [95, 37, 53], [52, 152, 80], [149, 109, 218], [169, 187, 16], [139, 220, 191], [231, 212, 132], [190, 195, 252], [209, 242, 163], [198, 73, 88], [143, 140, 45], [135, 149, 132], [94, 231, 101], [66, 236, 63], [206, 71, 171], [24, 41, 144], [252, 22, 29], [16, 50, 29], [232, 51, 114], [63, 214, 236], [233, 159, 242], [69, 78, 151], [209, 156, 224], [38, 211, 102], [86, 234, 197], [227, 228, 21], [120, 236, 165], [60, 30, 46], [108, 119, 195], [51, 98, 127], [55, 91, 8], [124, 59, 224], [106, 118, 64], [207, 118, 18], [18, 193, 138], [191, 6, 207], [167, 102, 23], [52, 35, 200], [12, 117, 89], [77, 114, 233], [160, 89, 173], [49, 196, 163], [74, 126, 70], [248, 95, 242], [185, 54, 214], [175, 160, 217], [115, 16, 106], [156, 197, 21], [89, 140, 234], [104, 9, 131], [223, 10, 58], [221, 101, 232], [72, 85, 159], [111, 58, 9], [129, 21, 89], [121, 181, 188], [194, 226, 172], [69, 227, 250], [168, 22, 48], [179, 194, 106], [83, 53, 217], [123, 116, 198], [68, 18, 160], [191, 152, 226], [64, 183, 26], [116, 146, 189], [172, 153, 255], [193, 255, 122], [46, 60, 33], [92, 126, 6], [147, 178, 102], [241, 38, 15], [105, 202, 82], [58, 22, 25], [54, 229, 116], [228, 177, 66], [68, 47, 56], [232, 160, 226], [111, 181, 78], [18, 4, 82], [5, 156, 52], [218, 174, 207], [187, 89, 111], [58, 106, 194], [241, 39, 115], [43, 91, 75], [145, 139, 181], [99, 159, 101], [91, 185, 122], [24, 73, 169], [190, 6, 206], [185, 81, 181], [67, 12, 160], [217, 63, 144], [137, 6, 233], [169, 140, 138], [63, 146, 137], [117, 73, 30], [14, 75, 127], [88, 52, 156], [234, 113, 142], [101, 199, 155], [161, 226, 166], [39, 208, 130], [130, 149, 198], [92, 141, 21], [76, 11, 79], [195, 224, 208], [106, 95, 219], [69, 75, 33], [112, 65, 239], [98, 174, 235], [82, 78, 80], [112, 156, 79], [51, 229, 154], [152, 0, 222], [133, 238, 212], [212, 120, 237], [99, 76, 117], [234, 93, 193], [232, 29, 15], [102, 26, 224], [20, 175, 187], [56, 222, 179], [237, 132, 99], [64, 200, 22], [30, 179, 184], [219, 121, 80], [3, 39, 38], [199, 101, 143], [132, 190, 107], [85, 187, 74], [186, 94, 34], [162, 150, 209], [72, 91, 163], [170, 159, 254], [29, 214, 1], [4, 17, 169], [143, 33, 121], [83, 45, 5], [70, 119, 169], [154, 69, 161], [143, 134, 153], [199, 104, 99], [75, 154, 42], [43, 8, 140], [248, 196, 50], [187, 143, 181], [239, 106, 178], [92, 198, 181], [161, 140, 76], [178, 107, 75], [69, 159, 185], [19, 95, 45], [8, 94, 97], [47, 69, 186], [197, 189, 63], [133, 46, 212], [103, 177, 205], [66, 188, 123], [31, 246, 98], [115, 246, 216], [198, 67, 186], [110, 96, 129], [34, 203, 240], [244, 55, 221], [28, 73, 211], [148, 244, 40], [67, 225, 18], [214, 182, 46], [173, 12, 233], [251, 201, 116], [204, 226, 185], [143, 16, 15], [209, 150, 128], [200, 20, 246], [155, 82, 216], [153, 189, 181], [93, 194, 144], [151, 84, 61], [249, 166, 247], [194, 97, 114], [138, 1, 224], [9, 248, 155], [244, 118, 248], [42, 44, 15], [135, 5, 11], [184, 3, 101], [29, 132, 117], [108, 171, 125], [122, 84, 74], [148, 210, 196], [84, 245, 174], [27, 171, 204], [58, 86, 253], [1, 74, 38], [151, 204, 132], [201, 69, 244], [66, 201, 241], [165, 214, 159], [117, 37, 249], [95, 226, 35], [101, 237, 80], [37, 189, 61], [135, 175, 254], [135, 237, 219], [31, 220, 193], [15, 6, 167], [15, 173, 134], [177, 88, 228], [9, 205, 0], [71, 129, 94], [37, 200, 101], [48, 72, 71], [21, 32, 86], [82, 32, 187], [236, 24, 55], [243, 150, 67], [189, 211, 230], [203, 176, 96], [77, 253, 83], [253, 183, 94], [251, 194, 201], [32, 246, 189], [243, 33, 93], [134, 180, 173], [190, 52, 44], [59, 217, 203], [144, 191, 194], [118, 24, 197], [42, 235, 105], [28, 246, 46], [61, 49, 18], [153, 223, 232], [219, 196, 171], [228, 11, 24], [179, 56, 60], [74, 133, 74], [247, 31, 156], [120, 88, 244], [21, 134, 202], [43, 209, 84], [223, 142, 105], [169, 112, 121], [252, 119, 152], [54, 146, 9], [9, 61, 205], [161, 132, 207], [98, 227, 90], [156, 91, 249], [157, 154, 156], [216, 72, 68], [222, 158, 163], [231, 76, 23], [184, 31, 53], [255, 119, 205], [41, 183, 117], [70, 58, 201], [136, 202, 105], [182, 25, 209], [58, 134, 60], [69, 21, 185], [153, 5, 120], [96, 69, 211], [185, 130, 18], [160, 171, 194], [130, 15, 180], [204, 180, 204], [27, 165, 34], [170, 97, 118], [8, 217, 102], [151, 68, 172], [18, 42, 239], [48, 214, 250], [8, 157, 165], [4, 108, 119], [164, 219, 193], [219, 195, 7], [111, 80, 72], [97, 24, 99], [122, 136, 151], [229, 173, 83], [192, 37, 211], [53, 251, 4], [197, 54, 62], [128, 129, 83], [152, 203, 225], [64, 13, 80], [238, 95, 165], [136, 96, 143], [170, 24, 84], [150, 10, 29], [220, 159, 79], [168, 103, 214], [127, 43, 169], [139, 189, 121], [249, 31, 217], [186, 78, 79], [45, 103, 135], [154, 158, 217], [176, 114, 27], [255, 32, 215], [122, 113, 226], [177, 168, 208], [249, 109, 212], [16, 175, 14], [230, 231, 34], [37, 6, 67]]

In [ ]:
from PIL import Image, ImageDraw
import random
import matplotlib.pyplot as plt
from utils import clrs2

def plot_instances(X):
    image = Image.fromarray(X, mode='L')
    sh = X.shape
    image2 = Image.new(mode='RGB', size=sh)
    draw = image.load()
    X = X.astype(np.int32)
    for w in range(sh[1]):
        for h in range(sh[0]):
            image2.putpixel((h, w), tuple(clrs2[X[w,h]]))
    plt.xticks([])
    plt.yticks([])
    plt.imshow(np.asarray(image2))

In [ ]:
import os
import h5py
import tensorflow as tf
import numpy as np
print(tf.__version__)
import tensorflow.keras as keras
print(tf.test.gpu_device_name())
%load_ext tensorboard
from utils import *

import tensorflow as tf
import numpy as np

def cluster_nearest_centroid(pred, centroids):
    # make channels last for broadcasted operations
    pred = np.moveaxis(pred,0,-1)
    # get num centroids
    num_cens = centroids.shape[0]
    distances = np.zeros(list(pred.shape[:-1])+[num_cens])
    for i in range(0,num_cens):
        distances[:,:,i] = np.linalg.norm(pred-centroids[i,],ord=2,axis=-1)

    segmentation = np.argmin(distances, axis=2) + 1
    return segmentation.astype(np.float32)

class MeanShift(tf.Module):
    def __init__(self, X, chan, n_seeds, bandwidth ,max_steps,  eps=1e-4):
        X_flat = tf.reshape(X, [chan, -1]) # chan x pixels
        X_flat = tf.transpose(X_flat,(1,0)) # pixels x chan
        perm = tf.random.shuffle(tf.range(start=0,limit=X_flat.shape[0], delta=1))
        idx = perm[0:n_seeds]
        self.C = tf.gather(X_flat, idx) # n_seeds x chan
        self.X1 = tf.expand_dims(tf.transpose(X_flat,(1,0)),0) # 1 x chan x pixels
        self.X2 = tf.expand_dims(X_flat,0) # 1  x pixels x chan
        self.diff = 10
        self.eps = eps
        self.max_steps = max_steps
        self.bandwidth = bandwidth
    #
    def _mean_shift_step(self, C):
        # data: emb_dim x pixels
        C = tf.expand_dims(C,2) # num_centroids x emb_dim x 1
        dist = tf.reduce_sum(tf.square(C-self.X1) / self.bandwidth, axis=1) # n_seeds x num_pixels
        dist = tf.exp(-dist)
        num = tf.reduce_sum(tf.expand_dims(dist,2) * self.X2, axis=1) # n_seeds x chan
        denom = tf.reduce_sum(dist, axis=1, keepdims=True) # n_seeds x 1
        C = num/denom
        return C
    #
    def fit(self):
        C = self.C
        steps = 0
        while self.diff > self.eps:
            C_old = C
            C = self._mean_shift_step(C)
            self.diff = tf.reduce_max(tf.reduce_sum(tf.abs(C_old - C), axis=1))
            steps += 1
            if steps > self.max_steps:
                break       
        return C

from tensorflow.python.ops import math_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
# modified to work for 2D/3D data and takes two arguments less than before

def discriminative_loss_single(prediction, correct_label, delta_v, delta_d, param_var, param_dist, param_reg):
    ''' Discriminative loss for a single prediction/label pair.
    :param prediction: inference of network, channel first: cdhw  [3, 36, 36, 36]
    :param correct_label: instance label, dhw [36,36,36]
    :feature_dim: feature dimension of prediction
    :param label_shape: shape of label
    :param delta_v: cutoff variance distance
    :param delta_d: curoff cluster distance
    :param param_var: weight for intra cluster variance
    :param param_dist: weight for inter cluster distances
    :param param_reg: weight regularization
    '''
    feature_dim = prediction.get_shape()[0]
    ### Reshape so pixels are aligned along a vector
    correct_label = tf.reshape(correct_label, [-1])
    perm = tf.roll(tf.range(tf.rank(prediction)),-1,0)
    reshaped_pred = tf.reshape(tf.transpose(prediction,perm), [-1, feature_dim]) 

    ### Count instances
    unique_labels, unique_id, counts = tf.unique_with_counts(correct_label)
    unique_labels, unique_id, counts = tf.unique_with_counts(correct_label)
    counts = tf.cast(counts, tf.float32)

    num_instances = tf.size(unique_labels)

    segmented_sum = tf.math.unsorted_segment_sum(reshaped_pred, unique_id, num_instances)

    mu = tf.math.divide(segmented_sum, tf.reshape(counts, (-1, 1)))
    mu_expand = tf.gather(mu, unique_id)

    ### Calculate l_var
    distance = tf.norm(tf.subtract(mu_expand, reshaped_pred), axis=1)
    distance = tf.subtract(distance, delta_v)
    distance = tf.clip_by_value(distance, 0., distance)
    distance = tf.square(distance)

    l_var = tf.math.unsorted_segment_sum(distance, unique_id, num_instances)
    l_var = tf.math.divide(l_var, counts)
    l_var = tf.reduce_sum(l_var)
    l_var = tf.math.divide(l_var, tf.cast(num_instances, tf.float32))


    ### Calculate l_dist
    # Get L1-,distance for each pair of clusters like this:
    #   mu_1 - mu_1
    #   mu_2 - mu_1
    #   mu_3 - mu_1

    mu_interleaved_rep = tf.tile(mu, [num_instances, 1])
    mu_band_rep = tf.tile(mu, [1, num_instances])
    mu_band_rep = tf.reshape(mu_band_rep, (num_instances*num_instances, feature_dim))

    mu_diff = tf.subtract(mu_band_rep, mu_interleaved_rep)

    # Filter out zeros from same cluster subtraction
    intermediate_tensor = tf.reduce_sum(tf.abs(mu_diff),axis=1)
    zero_vector = tf.zeros(1, dtype=tf.float32)
    bool_mask = tf.not_equal(intermediate_tensor, zero_vector)
    mu_diff_bool = tf.boolean_mask(mu_diff, bool_mask)

    mu_norm = tf.norm(mu_diff_bool, axis=1)
    mu_norm = tf.subtract(2.*delta_d, mu_norm)
    mu_norm = tf.clip_by_value(mu_norm, 0., mu_norm)
    mu_norm = tf.square(mu_norm)
    l_dist = tf.reduce_mean(mu_norm)

    ### Calculate l_reg
    l_reg = tf.reduce_mean(tf.norm(mu, axis=1))

    ### set NaNs to 0
    l_var = tf.cond(tf.math.is_nan(l_var), lambda: tf.identity(0.0), lambda: l_var)
    l_dist = tf.cond(tf.math.is_nan(l_dist), lambda: tf.identity(0.0), lambda: l_dist)
    l_reg = tf.cond(tf.math.is_nan(l_reg), lambda: tf.identity(0.0), lambda: l_reg)

    param_scale = 1.
    l_var = tf.math.multiply(param_var, l_var)
    l_dist = tf.math.multiply(param_dist, l_dist)
    l_reg = tf.math.multiply(param_reg, l_reg)
    loss = tf.math.multiply(param_scale,tf.math.add_n([l_var, l_dist, l_reg]))
    return loss, l_var, l_dist, l_reg

# Unet taken from https://github.com/funkelab/funlib.learn.tensorflow
import math

def conv_pass(
        fmaps_in,
        kernel_size,
        num_fmaps,
        num_repetitions,
        padding = 'valid',
        activation='relu',
        name='conv_pass'):
    '''Create a convolution pass::
        f_in --> f_1 --> ... --> f_n
    where each ``-->`` is a convolution followed by a (non-linear) activation
    function and ``n`` ``num_repetitions``. Each convolution will decrease the
    size of the feature maps by ``kernel_size-1``.
    Args:
        f_in:
            The input tensor of shape ``(batch_size, channels, depth, height,
            width)`` or ``(batch_size, channels, height, width)``.
        kernel_size:
            Size of the kernel. Forwarded to the tensorflow convolution layer.
        num_fmaps:
            The number of feature maps to produce with each convolution.
        num_repetitions:
            How many convolutions to apply.
        activation:
            Which activation to use after a convolution. Accepts the name of any
            tensorflow activation function (e.g., ``relu`` for ``tf.nn.relu``).
    '''

    fmaps = fmaps_in
    if activation is not None:
        activation = getattr(tf.nn, activation)

    conv_layer = getattr(
        tf.keras.layers,
        {2: 'Conv2D', 3: 'Conv3D'}[fmaps_in.get_shape().ndims - 2])

    for i in range(num_repetitions):
        fmaps = conv_layer(
            filters=num_fmaps,
            kernel_size=kernel_size,
            padding=padding,
            kernel_regularizer=tf.keras.regularizers.l2(l=1.0),
            data_format='channels_first',
            activation=activation,
            name=name + '_%i'%i)(fmaps)

    return fmaps

def downsample(fmaps_in, factors, padding='valid', name='down'):

    in_shape = fmaps_in.get_shape().as_list()
    if not np.all(np.array(in_shape[2:]) % np.array(factors) == 0):
        raise RuntimeWarning(
            "Input shape %s is not evenly divisible by downsample factor %s." %
            (in_shape[2:], factors))
        
    pooling_layer = getattr(
        tf.keras.layers,
        {2: 'MaxPool2D', 3: 'MaxPool3D'}[fmaps_in.get_shape().ndims - 2])

    fmaps = pooling_layer(
        pool_size=factors,
        strides=factors,
        padding=padding,
        data_format='channels_first',
        name=name)(fmaps_in)

    return fmaps

def upsample(fmaps_in, factors, num_fmaps, padding='valid', activation='relu', mode='learnt', name='up'):

    if activation is not None:
        activation = getattr(tf.nn, activation)

    if mode == 'learnt':
        conv_trans_layer = getattr(
            tf.keras.layers,
            {2: 'Conv2DTranspose', 3: 'Conv3DTranspose'}[fmaps_in.get_shape().ndims - 2])
        fmaps = conv_trans_layer(
          filters=num_fmaps,
          kernel_size=factors,
          strides=factors,
          padding=padding,
          data_format='channels_first',
          activation=activation,
          name=name)(fmaps_in)
    if mode == 'fixed':
        fixed_upsampling_layer = getattr(
            tf.keras.layers,
            {2: 'UpSampling2D', 3: 'UpSampling3D'}[fmaps_in.get_shape().ndims - 2])
        tf.keras.layers.UpSampling2D
        fmaps = fixed_upsampling_layer(
            size=factors,
            data_format='channels_first',
            interpolation='nearest',
            name=name)(fmaps_in)

    return fmaps

def crop_spatial(fmaps_in, shape):
    '''Crop only the spatial dimensions to match shape.
    Args:
        fmaps_in:
            The input tensor.
        shape:
            A list (not a tensor) with the requested shape [_, _, z, y, x] or
            [_, _, y, x].
    '''

    in_shape = fmaps_in.get_shape().as_list()
    # in_shape[0] = 1
    if in_shape[0] is None:
        in_shape[0] = -1
    offset = [0, 0] + [(in_shape[i] - shape[i]) // 2 for i in range(2, len(shape))]
    size = in_shape[0:2] + shape[2:]
    fmaps = tf.slice(fmaps_in, offset, size)

    return fmaps


def crop_to_factor(fmaps_in, factor, kernel_sizes, padding="valid"):
    '''Crop feature maps to ensure translation equivariance with stride of
    upsampling factor. This should be done right after upsampling, before
    application of the convolutions with the given kernel sizes.
    The crop could be done after the convolutions, but it is more efficient to
    do that before (feature maps will be smaller).
    '''

    shape = fmaps_in.get_shape().as_list()
    spatial_dims = len(shape) - 2
    spatial_shape = shape[-spatial_dims:]

    # the crop that will already be done due to the convolutions
    if padding == 'valid':
        convolution_crop = list(
            sum(
                (ks if isinstance(ks, int) else ks[d]) - 1
                for ks in kernel_sizes
            )
            for d in range(spatial_dims)
        )
    else:
        convolution_crop = [0 for d in range(spatial_dims)]
    print("crop_to_factor: factor =", factor)
    print("crop_to_factor: kernel_sizes =", kernel_sizes)
    print("crop_to_factor: convolution_crop =", convolution_crop)

    # we need (spatial_shape - convolution_crop) to be a multiple of factor,
    # i.e.:
    #
    # (s - c) = n*k
    #
    # we want to find the largest n for which s' = n*k + c <= s
    #
    # n = floor((s - c)/k)
    #
    # this gives us the target shape s'
    #
    # s' = n*k + c

    ns = (
        int(math.floor(float(s - c)/f))
        for s, c, f in zip(spatial_shape, convolution_crop, factor)
    )
    target_spatial_shape = tuple(
        n*f + c
        for n, c, f in zip(ns, convolution_crop, factor)
    )

    if target_spatial_shape != spatial_shape:

        assert all((
                (t > c) for t, c in zip(
                    target_spatial_shape,
                    convolution_crop))
            ), \
            "Feature map with shape %s is too small to ensure translation " \
            "equivariance with factor %s and following convolutions %s" % (
                shape,
                factor,
                kernel_sizes)

        target_shape = list(shape)
        target_shape[-spatial_dims:] = target_spatial_shape

        print("crop_to_factor: shape =", shape)
        print("crop_to_factor: spatial_shape =", spatial_shape)
        print("crop_to_factor: target_spatial_shape =", target_spatial_shape)
        print("crop_to_factor: target_shape =", target_shape)
        fmaps = crop(
            fmaps_in,
            target_shape)
    else:
        fmaps = fmaps_in

    return fmaps


def crop(fmaps_in, shape):
    '''Crop spatial and time dimensions to match shape.
    Args:
        fmaps_in:
            The input tensor of shape ``(b, c, z, y, x)`` (for 3D) or ``(b, c,
            t, z, y, x)`` (for 4D).
        shape:
            A list (not a tensor) with the requested shape ``[_, _, z, y, x]``
            (for 3D) or ``[_, _, t, z, y, x]`` (for 4D).
    '''

    in_shape = fmaps_in.get_shape().as_list()
    if in_shape[0] is None:
        in_shape[0] = -1

    # Explicitly handle number of dimensions
    in_is_4d = len(in_shape) == 6
    in_is_2d = len(in_shape) == 4
    out_is_4d = len(shape) == 6

    if in_is_4d and not out_is_4d:
        # set output shape for time to 1
        shape = shape[0:2] + [1] + shape[2:]

    if in_is_4d:
        offset = [
            0,  # batch
            0,  # channel
            (in_shape[2] - shape[2])//2,  # t
            (in_shape[3] - shape[3])//2,  # z
            (in_shape[4] - shape[4])//2,  # y
            (in_shape[5] - shape[5])//2,  # x
        ]
        size = [
            in_shape[0],
            in_shape[1],
            shape[2],
            shape[3],
            shape[4],
            shape[5],
        ]
    elif in_is_2d:
        offset = [
            0,  # batch
            0,  # channel
            (in_shape[2] - shape[2])//2,  # y
            (in_shape[3] - shape[3])//2,  # x
        ]
        size = [
            in_shape[0],
            in_shape[1],
            shape[2],
            shape[3],
        ]

    else:
        offset = [
            0,  # batch
            0,  # channel
            (in_shape[2] - shape[2])//2,  # z
            (in_shape[3] - shape[3])//2,  # y
            (in_shape[4] - shape[4])//2,  # x
        ]
        size = [
            in_shape[0],
            in_shape[1],
            shape[2],
            shape[3],
            shape[4],
        ]

    fmaps = tf.slice(fmaps_in, offset, size)

    if in_is_4d and not out_is_4d:
        # remove time dimension
        shape = shape[0:2] + shape[3:]
        fmaps = tf.reshape(fmaps, shape)

    return fmaps

def unet(
        fmaps_in,
        num_fmaps,
        fmap_inc_factor,
        downsample_factors,
        activation='relu',
        kernel_size_down=None,
        kernel_size_up=None,
        padding='valid',
        upsampling='learnt',
        layer=0):
    '''Create a 2D or 3D U-Net::
        f_in --> f_left --------------------------->> f_right--> f_out
                    |                                   ^
                    v                                   |
                 g_in --> g_left ------->> g_right --> g_out
                             |               ^
                             v               |
                                   ...
    where each ``-->`` is a convolution pass (see ``conv_pass``), each `-->>` a
    crop, and down and up arrows are max-pooling and transposed convolutions,
    respectively.
    The U-Net expects tensors to have shape ``(batch=1, channels, depth, height,
    width)`` for 3D or ``(batch=1, channels, height, width)`` for 2D.
    This U-Net performs only "valid" convolutions, i.e., sizes of the feature
    maps decrease after each convolution.
    Args:
        fmaps_in:
            The input tensor.
        num_fmaps:
            The number of feature maps in the first layer. This is also the
            number of output feature maps.
        fmap_inc_factor:
            By how much to multiply the number of feature maps between layers.
            If layer 0 has ``k`` feature maps, layer ``l`` will have
            ``k*fmap_inc_factor**l``.
        downsample_factors:
            List of lists ``[z, y, x]`` or ``[y, x]`` to use to down- and
            up-sample the feature maps between layers.
        activation:
            Which activation to use after a convolution. Accepts the name of any
            tensorflow activation function (e.g., ``relu`` for ``tf.nn.relu``).
        padding:
            One of 'valid' or 'same'
        upsampling:
            One of 'learnt' or 'fixed'
        layer:
            Used internally to build the U-Net recursively.
    '''

    prefix = "    "*layer
    print(prefix + "Creating U-Net layer %i"%layer)
    print(prefix + "f_in: " + str(fmaps_in.shape))
    if kernel_size_down is None:
        kernel_size_down = [[3, 3]]*(len(downsample_factors) + 1)
    if kernel_size_up is None:
        kernel_size_up = [[3, 3]]*len(downsample_factors)

    # convolve
    f_left = conv_pass(
        fmaps_in,
        kernel_size=3,
        num_fmaps=num_fmaps,
        num_repetitions=2,
        activation=activation,
        padding=padding,
        name='unet_layer_%i_left'%layer)

    # last layer does not recurse
    bottom_layer = (layer == len(downsample_factors))
    if bottom_layer:
        print(prefix + "bottom layer")
        print(prefix + "f_out: " + str(f_left.shape))
        return f_left

    # downsample
    g_in = downsample(
        f_left,
        downsample_factors[layer],
        padding=padding,
        name='unet_down_%i_to_%i'%(layer, layer + 1))

    # recursive U-net
    g_out = unet(
        g_in,
        num_fmaps=num_fmaps*fmap_inc_factor,
        fmap_inc_factor=fmap_inc_factor,
        downsample_factors=downsample_factors,
        kernel_size_down=kernel_size_down,
        kernel_size_up=kernel_size_up,
        activation=activation,
        padding=padding,
        upsampling=upsampling,
        layer=layer+1)

    print(prefix + "g_out: " + str(g_out.shape))

    # upsample
    g_out_upsampled = upsample(
        g_out,
        downsample_factors[layer],
        num_fmaps,
        activation=activation,
        padding=padding,
        mode=upsampling,
        name='unet_up_%i_to_%i'%(layer + 1, layer))

    print(prefix + "g_out_upsampled: " + str(g_out_upsampled.shape))

    # ensure translation equivariance with stride of product of
    # previous downsample factors
    if layer==0:
        factor_product = None
        for factor in downsample_factors[layer:]:
            if factor_product is None:
                factor_product = list(factor)
            else:
                factor_product = list(
                    f*ff
                    for f, ff in zip(factor, factor_product))
        g_out_upsampled = crop_to_factor(
            g_out_upsampled,
            factor=factor_product,
            kernel_sizes=kernel_size_up[layer],
            padding=padding)

    print(prefix + "g_out_upsampled_cropped: " +
          str(g_out_upsampled.shape))

    # copy-crop
    f_left_cropped = crop_spatial(f_left, g_out_upsampled.get_shape().as_list())

    print(prefix + "f_left_cropped: " + str(f_left_cropped.shape)) # f_left_cropped

    # concatenate along channel dimension
    f_right = tf.concat([f_left_cropped, g_out_upsampled], 1) #f_left_cropped

    print(prefix + "f_right: " + str(f_right.shape))

    # convolve
    f_out = conv_pass(
        f_right,
        kernel_size=3,
        num_fmaps=num_fmaps,
        num_repetitions=2,
        padding=padding,
        name='unet_layer_%i_right'%layer)

    print(prefix + "f_out: " + str(f_out.shape))

    return f_out

## Fig.2 a) $n\cdot f^l$ spaced instances and learnt upsampling

## Make data

In [ ]:
import numpy as np
from skimage import measure

def add_disc(X ,x_pos, y_pos, radius):
    n = X.shape[0]
    y,x = np.ogrid[-y_pos:n-y_pos, -x_pos:n-x_pos]
    mask = x*x + y*y < radius*radius
    X[mask] = 1
    return X

def add_square(X ,x_pos, y_pos, length_x, length_y, value=1):
    x_pos, y_pos = int(x_pos), int(y_pos)
    x_slice = slice(x_pos,x_pos+length_x)
    y_slice = slice(y_pos,y_pos+length_y)
    X[x_slice,y_slice] = value
    return X

pixels =  220
X = np.zeros((pixels, pixels))
object_distance_x = 16
object_distance_y = 16
radius = 5

for i in range(radius,pixels+object_distance_x, object_distance_x):
    for j in range(radius,pixels+object_distance_y, object_distance_y):
        X = add_disc(X, i+4, j+4, radius) #, object_size_y)
        #X = add_square(X, np.max(i+np.random.randint(-6,6),0), np.max(j+np.random.randint(-6,6),0), object_size)

img = X
labels = measure.label(img, background=0)

fig=plt.figure(figsize=(20, 20))
fig.add_subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
fig.add_subplot(1, 2, 2)
plt.imshow(labels)

In [ ]:
experiment = 'Fig2a'
checkpoint_path = os.path.join(experiment, 'model_checkpoints')
snapshot_path = os.path.join(experiment, 'snaps')
log_path = os.path.join(experiment,'logs')
#!rm -rf log_path # delete old logs at that location
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(snapshot_path, exist_ok=True)

## Train

In [ ]:
input_shape = tuple([1,220,220])

# tensorboard writer
writer = tf.summary.create_file_writer(log_path)
writer.set_as_default()
tf.summary.trace_on()

# make dataset
img_expand = np.expand_dims(img, 0).astype(np.float32)
labels_expand = np.expand_dims(labels, 0).astype(np.float32) 
data = tf.data.Dataset.from_tensor_slices({'raw':img_expand, 'gt':labels_expand})
data = data.batch(1).repeat().prefetch(1)
step=0

# make network
features = tf.keras.Input(shape=input_shape, name='input_features')
output = unet(features,
        16, 
        2,
        [[2,2],[2,2],[2,2]],
        padding='valid',
        upsampling='learnt')

prediction = conv_pass(
    output,
    kernel_size=1,
    num_fmaps=3,
    num_repetitions=1,
    activation=None,
    padding='valid',
    name='conv_embedding')


print("prediction: " + str(prediction.shape))

model = tf.keras.Model(
    inputs = features,
    outputs = prediction,
    name = 'U-Net')

optimizer = tf.keras.optimizers.Adam(learning_rate=0.5e-4)

def train_step(model, raw, gt):
    with tf.GradientTape() as t:
        pred = model(raw)
        b, g_c, g_h, g_w = gt.shape
        _, _, p_h, p_w = pred.shape
        # center crop gt to output dimensions
        gt = tf.slice(gt, begin=(0,0,(g_h-p_h)//2, (g_w-p_w)//2), size=(b,g_c,p_h,p_w))
        disc_loss , l_var, l_dist, l_reg = discriminative_loss_single(prediction=tf.squeeze(pred, axis=0),
                                                        correct_label=tf.squeeze(gt, axis=0),
                                                        delta_v= 0.0,
                                                        delta_d=3.0,
                                                        param_var=1.0,
                                                        param_dist=1.0,
                                                        param_reg=0.0001
                                                        )
        grads = t.gradient(disc_loss, model.trainable_weights)
    # optimize
    optimizer.apply_gradients(zip(grads,model.trainable_weights))
    return disc_loss, l_var, l_dist, l_reg, pred, gt 

step = 0
for sample in data:
    raw, gt = sample['raw'], sample['gt']
    # random cropping
    raw_gt = tf.concat([raw, gt], axis=0)
    raw_gt_cropped = tf.image.random_crop(raw_gt, size=[2, input_shape[1], input_shape[2]])
    raw, gt = tf.split(raw_gt_cropped, num_or_size_splits=2, axis=0)
    # add channel dimension
    raw = tf.expand_dims(raw,0)
    gt = tf.expand_dims(gt,0)
    #
    disc_loss, l_var, l_dist, l_reg, pred, gt = train_step(model, raw, gt)
    tf.summary.scalar('disc_loss', disc_loss, step)
    tf.summary.scalar('l_var', l_var, step)
    tf.summary.scalar('l_dist', l_dist, step)
    tf.summary.scalar('l_reg', l_reg, step)
    step += 1
    if step % 2000 == 0:
        print('step: ',step)
        print('loss:', disc_loss)
        ## Postprocessing
        centroids = MeanShift(X=pred, chan=1, n_seeds=1000, bandwidth=0.5, max_steps=1000).fit()
        centroids = np.unique(np.round(centroids,decimals=1), axis=0)
        pred_instances = cluster(pred, centroids, bandwidth=0.5)
        with h5py.File(os.path.join(snapshot_path,'snap_step_'+str(step)),'w') as f:
            f.create_dataset('raw',data=tf.squeeze(raw, axis=0))
            f.create_dataset('gt',data=tf.squeeze(gt, axis=0))
            f.create_dataset('pred_emb',data=tf.squeeze(pred, axis=0))
            f.create_dataset('pred_inst',data=pred_instances)
        print('Sample saved')
    if step % 20000 == 0:
            model.save(os.path.join(checkpoint_path,'u_net_{}.tf'.format(step)),overwrite=True)
            print('Model saved at step ', str(step))
    if step > 200000:
        break

In [ ]:
checkpoint_path = 'Fig2a/model_checkpoints/u_net_200000.tf'
model = tf.keras.models.load_model(checkpoint_path)

In [ ]:
import matplotlib.patches as patches
name = os.path.join(experiment,'plot')

img_expand = np.expand_dims(img, 0).astype(np.float32)
labels_expand = np.expand_dims(labels, 0).astype(np.float32) 
data = tf.data.Dataset.from_tensor_slices({'raw':img_expand, 'gt':labels_expand})
data = data.batch(1).repeat().prefetch(1)

sample = next(iter(data))
raw, gt = sample['raw'], sample['gt']
# random cropping
raw_gt = tf.concat([raw, gt], axis=0)
raw_gt_cropped = tf.slice(raw_gt, begin=(0, 0, 0), size=(2,220,220))
#raw_gt_cropped = tf.image.random_crop(raw_gt, size=[2, input_shape[1], input_shape[2]])
raw, gt = tf.split(raw_gt_cropped, num_or_size_splits=2, axis=0)
# add channel dimension
raw = tf.expand_dims(raw,0)
gt = tf.expand_dims(gt,0)
#
pred = model(raw)
b, g_c, g_h, g_w = gt.shape
_, _, p_h, p_w = pred.shape
# center crop gt to output dimensions
gt = tf.slice(gt, begin=(0,0,(g_h-p_h)//2, (g_w-p_w)//2), size=(b,g_c,p_h,p_w))

centroids = MeanShift(X=pred, chan=3, n_seeds=1000, bandwidth=2.5, max_steps=1000).fit()
centroids = np.unique(np.round(centroids,decimals=2), axis=0)
pred_instances = np.abs(cluster_nearest_centroid(np.squeeze(pred), centroids)-1)

fig=plt.figure(figsize=(50, 50))
plt.tight_layout()
plt.rcParams.update({'font.size': 28})
ax1 = fig.add_subplot(1, 4, 1)
ax1.title.set_text('Input')
plt.xticks([])
plt.yticks([])
plt.imshow(np.squeeze(raw), cmap='gray')
rect = patches.Rectangle(((g_h-p_h)//2,(g_w-p_w)//2),p_h,p_w,linewidth=4,edgecolor='r',facecolor='none')
ax1.add_patch(rect)
ax2 = fig.add_subplot(1, 4, 2)
ax2.title.set_text('Groundtruth')
plot_instances(np.squeeze(gt))
ax3 = fig.add_subplot(1, 4, 3)
ax3.title.set_text('Predicted Embeddings')
pred_show = np.squeeze(pred)
pred_show = np.transpose(pred_show,[1,2,0])
pred_show = pred_show - np.min(pred_show, axis=(0,1))
pred_show = pred_show/np.max(pred_show, axis=(0,1))
plt.xticks([])
plt.yticks([])
plt.imshow(pred_show)
ax4 = fig.add_subplot(1, 4, 4)
ax4.title.set_text('Predicted Instances')
pred_instances_show = np.squeeze(pred_instances)
plot_instances(pred_instances_show)
plt.savefig(name+'.png', bbox_inches='tight', dpi=72)
plt.show()

import h5py
with h5py.File(name+'.hdf','w') as f:
    f.create_dataset('Input', data=raw)
    f.create_dataset('Groundtruth', data=gt)
    f.create_dataset('Pred_Embedding', data=pred)
    f.create_dataset('Pred_Instances', data=pred_instances)